# Read me

## Run this code first. The hypotheses and the meta analysis require the results of this analysis

METADATA

trx_date: the date of the transaction;

trx_id: the ID of each transaction;

product_code: the ID of the product;

sub_family: subfamily is a product hierarchy;

family: family is a product hierarchy;

quantity: number of product units;

unit_price: the unit price of the product;

flag_pastry: boolean flag to identify products that belong to the pastry category;

shopID: the ID of the location/shop where the transaction occured;

customerID: the ID of the customer in the purchase transaction;

In [ ]:
!pip install openpyxl
!pip install mlxtend
!pip install networkx matplotlib
!pip install joypy
!pip install factor_analyzer


In [ ]:
def remove_outliers_zscore(data, threshold=3):
    z_scores = (data - data.mean()) / data.std()
    filtered_data = data[abs(z_scores) <= threshold]
    return filtered_data


## defines a function that plots multiple box plots
def plot_multiple_boxplots(data, feats, title="Box Plots"):

    # Prepare figure. Create individual axes where each histogram will be placed
    fig, axes = plt.subplots(2, ceil(len(feats) / 2), figsize=(20, 11))

    # Plot data
    # Iterate across axes objects and associate each histogram (hint: use the ax.hist() instead of plt.hist()):
    for ax, feat in zip(axes.flatten(), feats): # Notice the zip() function and flatten() method
      sns.boxplot(x=data[feat], ax=ax)
    
    # Layout
    # Add a centered title to the figure:
    plt.suptitle(title)

    plt.show()

    return


def is_redundant(rule, rules_df):
    """
    Check if a given rule is redundant within a set of rules.
    A rule is redundant if there's another rule with the same consequent,
    equal or higher confidence, and a subset of the antecedents.
    """
    for _, other_rule in rules_df.iterrows():
        if (rule['consequents'] == other_rule['consequents']) and \
           (rule['antecedents'].issubset(other_rule['antecedents'])) and \
           (rule['confidence'] <= other_rule['confidence']) and \
           (rule['antecedents'] != other_rule['antecedents']):  # Avoid comparing a rule to itself
            return True
    return False




In [ ]:
import numpy as np
import pandas as pd
import pandas as pd
import os
import warnings
import matplotlib.pyplot as plt
from scipy.cluster import hierarchy
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import ticker
import joypy
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import MinMaxScaler
from factor_analyzer.factor_analyzer import calculate_kmo
from scipy.cluster.hierarchy import fcluster
from sklearn.metrics import r2_score
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import KMeans
from math import ceil
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import networkx as nx
import matplotlib.colors as mcolors
from itertools import product
import pandas as pd
from matplotlib.colors import ListedColormap
from scipy.cluster.hierarchy import linkage, dendrogram, leaves_list
from sklearn.metrics import pairwise_distances
from sklearn.manifold import MDS
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
import re
import statsmodels.api as sm
from scipy.cluster.hierarchy import linkage as sch_linkage
import pickle
import warnings

In [ ]:
# Suppress all warnings
warnings.filterwarnings("ignore")

In [ ]:
sales_data = pd.read_csv('anon_sales.csv')
sales_data.describe().T

# Part I: Data Pre-Cleaning

In [ ]:
sales_data.info()

In [ ]:
sales_data.memory_usage()

In [ ]:
sales_data.head()

In [ ]:
sales_data['unit_price'].min()

In [ ]:
sales_data['unit_price'].max()

In [ ]:
# Replace 'sub_family_' with '' (empty string) to leave only the number part
sales_data['family'] = sales_data['family'].str.replace('family_', '')

# Optionally, if you want to convert the column to numeric type
sales_data['family'] = pd.to_numeric(sales_data['family'])

In [ ]:
# Replace 'sub_family_' with '' (empty string) to leave only the number part
sales_data['sub_family'] = sales_data['sub_family'].str.replace('sub_family_', '')

# Optionally, if you want to convert the column to numeric type
sales_data['sub_family'] = pd.to_numeric(sales_data['sub_family'])

In [ ]:
# Replace 'sub_family_' with '' (empty string) to leave only the number part
sales_data['shopID'] = sales_data['shopID'].str.replace('ShopID_', '')

# Optionally, if you want to convert the column to numeric type
sales_data['shopID'] = pd.to_numeric(sales_data['shopID'])

In [ ]:
sales_data['product_code_num'] = sales_data['product_code']

In [ ]:
# Replace 'sub_family_' with '' (empty string) to leave only the number part
sales_data['product_code_num'] = sales_data['product_code_num'].str.replace('Product_ID_', '')

# Optionally, if you want to convert the column to numeric type
sales_data['product_code_num'] = pd.to_numeric(sales_data['product_code_num'])

In [ ]:
# Replace 'sub_family_' with '' (empty string) to leave only the number part
sales_data['product_code'] = sales_data['product_code'].str.replace('Product_', '')

In [ ]:
# Replace 'trx_id' with '' (empty string) to leave only the number part
sales_data['trx_id'] = sales_data['trx_id'].str.replace('trxID_', '')

# Optionally, if you want to convert the column to numeric type
sales_data['trx_id'] = pd.to_numeric(sales_data['trx_id'])

In [ ]:
# Convert the 'trx_date' column to datetime format
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

In [ ]:
sales_data['unit_price'] = pd.to_numeric(sales_data['unit_price'].str.replace(',', '.'), errors='coerce')

In [ ]:
# Create the new variable total_value as the product of quantity and unit_price
sales_data['total_value'] = sales_data['quantity'] * sales_data['unit_price']

## Creating new Time Variables

In [ ]:
sales_data['trx_date']

In [ ]:
# Mapping from original years to new years
year_mapping = {
    1992: 2020,
    1993: 2021,
    1994: 2022,
    1995: 2023,
    1996: 2024
}

# Apply the mapping to change the year, keeping month and day the same
sales_data['trx_date'] = sales_data['trx_date'].apply(
    lambda x: x.replace(year=year_mapping[x.year])
)

sales_data['trx_date']

In [ ]:
# Create a 'year' column based on 'trx_date'
sales_data['year'] = sales_data['trx_date'].dt.to_period('Y')

# Create a 'quarter' column based on 'trx_date'
sales_data['quarter'] = sales_data['trx_date'].dt.to_period('Q')

In [ ]:
# Create 'year_integer' column by extracting year and converting to integer
sales_data['year_integer'] = sales_data['trx_date'].dt.year


In [ ]:
# Determine semester
sales_data['semester'] = (sales_data['trx_date'].dt.month - 1) // 6 + 1

In [ ]:
# Determine season
def get_season(month):
    seasons = {'winter': (12, 1, 2), 'spring': (3, 4, 5), 'summer': (6, 7, 8), 'autumn': (9, 10, 11)}
    for season, months in seasons.items():
        if month in months:
            return season
    return None

sales_data['season'] = sales_data['trx_date'].dt.month.apply(get_season)

In [ ]:
# Day of the week (Monday=1, Tuesday=2, ...)
sales_data['day_of_week_number'] = sales_data['trx_date'].dt.dayofweek + 1

In [ ]:
# Day of the week name
sales_data['day_of_week_name'] = sales_data['trx_date'].dt.day_name()

In [ ]:
sales_data['year_month'] = sales_data['trx_date'].dt.to_period('M')

sales_data.head()

## Memory Usage Reduction

In [ ]:
sales_data.memory_usage()

sales_data.drop(columns=["Unnamed: 0"], inplace=True)

sales_data['shopID'] = sales_data['shopID'].astype('int8')

sales_data['month'] = sales_data['month'].astype('int8')

sales_data['semester'] = sales_data['semester'].astype('int8')

sales_data['quantity'] = sales_data['quantity'].astype('float32')

sales_data['unit_price'] = sales_data['unit_price'].astype('float16')

sales_data['day_of_week_number'] = sales_data['day_of_week_number'].astype('int8')

sales_data['flag_pastry'] = sales_data['flag_pastry'].astype('bool')

sales_data['sub_family'] = sales_data['sub_family'].astype('category')

sales_data['family'] = sales_data['family'].astype('category')

sales_data['day_of_week_name'] = sales_data['day_of_week_name'].astype('category')

sales_data['month_name'] = sales_data['month_name'].astype('category')

sales_data['season'] = sales_data['season'].astype('category')

sales_data['customerID'] = sales_data['customerID'].astype('category')

sales_data['product_code'] = sales_data['product_code'].astype('category')

sales_data['product_code_num'] = sales_data['product_code_num'].astype('float16')

sales_data['total_value'] = sales_data['total_value'].astype('float32')

# CREATING NEW DATA SETS

In [ ]:
sales_data['shopID'].nunique()

In [ ]:
sales_data_shop1 = sales_data[sales_data['shopID'] == 1]
sales_data_shop2 = sales_data[sales_data['shopID'] == 2]
sales_data_shop3 = sales_data[sales_data['shopID'] == 3]
sales_data_shop4 = sales_data[sales_data['shopID'] == 4]
sales_data_shop5 = sales_data[sales_data['shopID'] == 5]
sales_data_shop6 = sales_data[sales_data['shopID'] == 6]
sales_data_shop7 = sales_data[sales_data['shopID'] == 7]
sales_data_shop8 = sales_data[sales_data['shopID'] == 8]
sales_data_shop9 = sales_data[sales_data['shopID'] == 9]
sales_data_shop10 = sales_data[sales_data['shopID'] == 10]
sales_data_shop11 = sales_data[sales_data['shopID'] == 11]
sales_data_shop12 = sales_data[sales_data['shopID'] == 12]
sales_data_shop13 = sales_data[sales_data['shopID'] == 13]
sales_data_shop14 = sales_data[sales_data['shopID'] == 14]
sales_data_shop15 = sales_data[sales_data['shopID'] == 15]
sales_data_shop16 = sales_data[sales_data['shopID'] == 16]

# EXPLORATION AND QUARTERLY VIEW


In [ ]:
numeric_variables = ['quantity', 'unit_price']

In [ ]:
sales_data.head()

In [ ]:
# Group by 'year'
yearly_groups = sales_data.groupby('year')

# Group by 'quarter'
quarterly_groups = sales_data.groupby('quarter')

In [ ]:
# Filter for a specific year, e.g., 2021
data_2021 = sales_data[sales_data['year'] == '2021']

# Filter for a specific quarter, e.g., Q2 of 2021
data_2021_Q2 = sales_data[sales_data['quarter'] == '2021Q2']

In [ ]:
sales_data.head()

In [ ]:
# Ensure trx_date is a datetime type
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Extract the quarter from trx_date
sales_data['quarter'] = sales_data['trx_date'].dt.to_period('Q')

# Assuming 'quarter' is a column in sales_data and 'total_value' needs to be summed up
quarterly_sums = sales_data.groupby('quarter')['total_value'].sum().reset_index()

# Convert 'quarter' from Period to string for plotting
quarterly_sums['quarter_str'] = quarterly_sums['quarter'].astype(str)

# Plot
plt.figure(figsize=(10, 6))
plt.bar(quarterly_sums['quarter_str'], quarterly_sums['total_value'], color='skyblue')
plt.title('Quarterly Total Value in Sales Data')
plt.xlabel('Quarter')
plt.ylabel('Total Value')
plt.xticks(rotation=45)
plt.tight_layout()  # Adjust layout to make room for the rotated x-axis labels
plt.show()


In [ ]:
# Optional: Filter for the year 1992 only
sales_data_2020 = sales_data[sales_data['trx_date'].dt.year == 2020]

quarterly_sums_2020 = sales_data_2020.groupby('quarter')['total_value'].sum().reset_index()

# Plot
plt.figure(figsize=(10, 6))
plt.bar(quarterly_sums_2020['quarter'].astype(str), quarterly_sums_2020['total_value'], color='skyblue')
plt.title('Quarterly total_value for 2021 in sales_data')
plt.xlabel('Quarter')
plt.ylabel('total_value')
plt.xticks(rotation=45)
plt.tight_layout()  # Adjust layout to make room for the rotated x-axis labels
plt.show()


In [ ]:
# Optional: Filter for the year 1992 only
sales_data_2021 = sales_data[sales_data['trx_date'].dt.year == 2021]

# Aggregate quantities by quarter
quarterly_sums_2021 = sales_data_2021.groupby('quarter')['total_value'].sum().reset_index()

# Plot
plt.figure(figsize=(10, 6))
plt.bar(quarterly_sums_2021['quarter'].astype(str), quarterly_sums_2021['total_value'], color='skyblue')
plt.title('Quarterly total_value for 2021 in sales_data')
plt.xlabel('Quarter')
plt.ylabel('total_value')
plt.xticks(rotation=45)
plt.tight_layout()  # Adjust layout to make room for the rotated x-axis labels
plt.show()

In [ ]:
# Optional: Filter for the year 2022 only
sales_data_2022 = sales_data[sales_data['trx_date'].dt.year == 2022]

# Aggregate quantities by quarter
quarterly_sums_2022 = sales_data_2022.groupby('quarter')['total_value'].sum().reset_index()

# Plot
plt.figure(figsize=(10, 6))
plt.bar(quarterly_sums_2022['quarter'].astype(str), quarterly_sums_2022['total_value'], color='skyblue')
plt.title('Quarterly total_value for 1995 in sales_data')
plt.xlabel('Quarter')
plt.ylabel('total_value')
plt.xticks(rotation=45)
plt.tight_layout()  # Adjust layout to make room for the rotated x-axis labels
plt.show()

In [ ]:
# Optional: Filter for the year 2023 only
sales_data_2023 = sales_data[sales_data['trx_date'].dt.year == 2023]

# Aggregate quantities by quarter
quarterly_sums_2023 = sales_data_2023.groupby('quarter')['total_value'].sum().reset_index()

# Plot
plt.figure(figsize=(10, 6))
plt.bar(quarterly_sums_2023['quarter'].astype(str), quarterly_sums_2023['total_value'], color='skyblue')
plt.title('Quarterly total_value for 1996 in sales_data')
plt.xlabel('Quarter')
plt.ylabel('total_value')
plt.xticks(rotation=45)
plt.tight_layout()  # Adjust layout to make room for the rotated x-axis labels
plt.show()


In [ ]:
# Filter for the years 2020 to 2023 and create an explicit copy
years_of_interest = [2020, 2021, 2022, 2023]
sales_data_filtered = sales_data[sales_data['trx_date'].dt.year.isin(years_of_interest)].copy()

# Adding year and quarter columns
sales_data_filtered['year'] = sales_data_filtered['trx_date'].dt.year
sales_data_filtered['quarter'] = sales_data_filtered['trx_date'].dt.to_period('Q')

# Aggregate quantities by year, quarter, and shopID
quarterly_shop_sums = sales_data_filtered.groupby(['shopID', 'year', 'quarter'])['total_value'].sum().reset_index()

# Get unique shop IDs
shop_ids = quarterly_shop_sums['shopID'].unique()

plt.figure(figsize=(14, 10))

# Plot each shop's quarterly data
for shop_id in shop_ids:
    shop_data = quarterly_shop_sums[quarterly_shop_sums['shopID'] == shop_id]
    plt.plot(shop_data['quarter'].astype(str), shop_data['total_value'], marker='o', label=f'Shop {shop_id}')

plt.title('Quarterly total_value Development for Each Shop (2020-2023)')
plt.xlabel('Quarter')
plt.ylabel('Total Value in €')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()  # Adjust layout to make room for the rotated x-axis labels

# Save the plot as an SVG file
plt.savefig("quarterly_total_value_development.svg", format='svg')

# Display the plot
plt.show()


We are going to focus on Shop 1, because it used to be the best shop, but then lost a lot
shop 2, because it has always been very low
shop 4, because it has a huge increase, while it started like everyone else
and shop 7, because it started even better than shop 4, but then went into the same curve like most of the shops, while shop 4 exploded.
Also we will look at shop 8,9 to see if they have a similar behaviour as shop 7

And we will look at shop 11,12,13 to see if they have a similar behaviour. So if the new shops also exhibit similar concepts 

In [ ]:
# Existing code for filtering data
years_of_interest = [2020, 2021, 2022, 2023]
sales_data_filtered = sales_data[sales_data['trx_date'].dt.year.isin(years_of_interest)].copy()
sales_data_filtered['year'] = sales_data_filtered['trx_date'].dt.year
sales_data_filtered['quarter'] = sales_data_filtered['trx_date'].dt.to_period('Q').astype(str)
quarterly_shop_sums = sales_data_filtered.groupby(['shopID', 'year', 'quarter'])['total_value'].sum().reset_index()

# Calculate average total value per quarter across all shops
average_quarterly_sums = quarterly_shop_sums.groupby('quarter')['total_value'].mean().reset_index()

# Prepare data for regression
x = np.array(range(len(average_quarterly_sums['quarter'])))  # Numerical x-axis for regression
x = sm.add_constant(x)  # Adds a constant term to the predictor
y = average_quarterly_sums['total_value']

# Fit regression model
model = sm.OLS(y, x)
results = model.fit()

# Plot actual data
plt.figure(figsize=(10, 6))
plt.plot(average_quarterly_sums['quarter'], y, marker='o', label='Average Total Value')

# Plot regression line
plt.plot(average_quarterly_sums['quarter'], results.predict(x), linestyle='--', label='Regression Line')

plt.title('Average Quarterly Total Value Across All Shops with Regression (2020-2023)')
plt.xlabel('Quarter')
plt.ylabel('Average Total Value')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()

# Save the plot as an SVG file
plt.savefig("average_quarterly_total_value.svg", format='svg')

# Display the plot
plt.show()

print(f"Plot saved as average_quarterly_total_value.svg")


In [ ]:
# Assuming 'trx_date' is already converted to datetime
sales_data['Quarter'] = sales_data['trx_date'].dt.to_period('Q').astype(str)

# Mark sales presence for each shop by quarter
sales_presence = sales_data.groupby(['shopID', 'Quarter']).size().unstack(fill_value=0)
sales_presence[sales_presence > 0] = 1

In [ ]:
sales_presence_long = sales_presence.reset_index().melt(id_vars='shopID', var_name='Quarter', value_name='Sales_Presence')

In [ ]:
# Sum 'Sales_Presence' for each 'Quarter' to get the count of shops with sales
shops_active_per_quarter = sales_presence_long.groupby('Quarter')['Sales_Presence'].sum().reset_index()


In [ ]:
plt.figure(figsize=(14, 7))
sns.lineplot(data=shops_active_per_quarter, x='Quarter', y='Sales_Presence', marker='o')

plt.title('Number of Active Shops per Quarter')
plt.xlabel('Quarter')
plt.ylabel('Number of Active Shops')
plt.xticks(rotation=45)
plt.grid(False)
plt.tight_layout()
plt.show()


In [ ]:
# Aggregate quantities by year and quarter
quarterly_sums_quantity = sales_data.groupby(['year', 'quarter'])['quantity'].sum().reset_index()

# Plotting
plt.figure(figsize=(12, 8))

# Get unique years
years_of_interest = quarterly_sums_quantity['year'].unique()

# Plot each year's quarterly data
for year in years_of_interest:
    yearly_data = quarterly_sums_quantity[quarterly_sums_quantity['year'] == year]
    plt.plot(yearly_data['quarter'].astype(str), yearly_data['quantity'], marker='o', label=f'Year {year}')

plt.title('Quaterly Summed Quantities for 2020, 2021, 2022, and 2023')
plt.xlabel('Quarter')
plt.ylabel('Quantities')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()  # Adjust layout to make room for the rotated x-axis labels
plt.show()


In [ ]:
# Aggregate quantities by year and quarter
quarterly_sums_quantity = sales_data.groupby(['year', 'quarter'])['total_value'].sum().reset_index()

# Plotting
plt.figure(figsize=(12, 8))

# Get unique years
years_of_interest = quarterly_sums_quantity['year'].unique()

# Plot each year's quarterly data
for year in years_of_interest:
    yearly_data = quarterly_sums_quantity[quarterly_sums_quantity['year'] == year]
    plt.plot(yearly_data['quarter'].astype(str), yearly_data['total_value'], marker='o', label=f'Year {year}')

plt.title('Quaterly Summed total_value for 2020, 2021, 2022, and 2023')
plt.xlabel('Quarter')
plt.ylabel('total_value')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()  # Adjust layout to make room for the rotated x-axis labels
plt.show()


# Which Month and Product Combination has empty Values?

In [ ]:
# Find all unique year_month values and products
all_year_months_all = sorted(sales_data['year_month'].unique())
unique_products_all = sorted(sales_data['product_code'].unique())

# Generate all possible combinations of products and months
all_combinations_all = pd.DataFrame(product(unique_products_all, all_year_months_all), columns=['product_code', 'year_month'])

# Group sales data by product and year_month, and count occurrences
sales_counts_all = sales_data.groupby(['product_code', 'year_month']).size().reset_index(name='sales_count')

# Merge the all possible combinations with the actual sales counts
merged_data_all = pd.merge(all_combinations_all, sales_counts_all, on=['product_code', 'year_month'], how='left')

# Identify missing sales by finding NaN values in the sales_count column
missing_sales_all = merged_data_all[merged_data_all['sales_count'] == 0]

# missing_sales_all now contains the combinations of product and month where no sales occurred



In [ ]:
merged_data_all.head()

In [ ]:
# Group by product_code and year_month, then count the number of transactions (if 'sales_count' column is absent)
monthly_sales_all = sales_data.groupby(['product_code', 'year_month']).size().reset_index(name='transaction_count')

# Pivot for easier visualization
pivot_sales_all = monthly_sales_all.pivot(index='product_code', columns='year_month', values='transaction_count')

# Fill NaNs with 0 to indicate months with 0 sales
pivot_sales_all.fillna(0, inplace=True)

# Mark months with sales as 1 and months with 0 sales as 0
pivot_sales_all = pivot_sales_all.applymap(lambda x: 1 if x > 0 else 0)

## Heatmap of Product availability in all shops by product Code

In [ ]:
# Assuming pivot_sales is already defined and prepared
# Perform hierarchical clustering
linkage_matrix_all = linkage(pivot_sales_all, method='ward')

# Get the order of rows according to the clustering
sorted_row_idx_all = leaves_list(linkage_matrix_all)

# Reorder pivot_sales according to the clustering result
sorted_pivot_sales_all = pivot_sales_all.iloc[sorted_row_idx_all]

# Define a custom colormap: red for 0 (no sale), green for 1 (sale)
custom_cmap = mcolors.ListedColormap(['red','green'])

# Plotting the sorted heatmap
plt.figure(figsize=(20, len(sorted_pivot_sales_all.index) * 0.5))
sns.heatmap(sorted_pivot_sales_all, cmap=custom_cmap, cbar_kws={'label': 'Sales Presence (1: Sale, 0: No Sale)'}, linewidths=.1)

plt.title('Monthly Sales Presence by Product Code in all shops (Sorted by Similarity)')
plt.xlabel('Month')
plt.ylabel('Product Code')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
pivot_sales_all.head()

# Exploring the best Seasonal Products

In [ ]:
sales_data.head()

In [ ]:
sales_data.info()

In [ ]:
def top_5_products_per_season_with_percentage(sales_data):
    # Check if sales_data is a DataFrame
    if not isinstance(sales_data, pd.DataFrame):
        raise ValueError("sales_data must be a pandas DataFrame")

    # Ensure the DataFrame has the required columns
    required_columns = {'product_code', 'season', 'total_value', 'year_integer'}
    if not required_columns.issubset(sales_data.columns):
        raise ValueError(f"sales_data must contain the columns: {required_columns}")

    # Calculate the total total_value of all products
    total_value_all_products = sales_data['total_value'].sum()

    # Filter out products that are available in multiple seasons
    product_season_counts = sales_data.groupby('product_code')['season'].nunique()
    single_season_products = product_season_counts[product_season_counts == 1].index
    filtered_sales_data = sales_data[sales_data['product_code'].isin(single_season_products)]

    # Aggregate the total_value for each product within each season
    aggregated_sales_data = filtered_sales_data.groupby(['year_integer','product_code', 'season'], as_index=False)['total_value'].sum()

    # Create sales_data_ready DataFrame
    sales_data_ready = aggregated_sales_data.copy()

    # Dictionary to store top 5 products for each season
    top_products = {}

    # List of seasons
    seasons = ['winter', 'summer', 'spring', 'autumn']

    # Iterate over each season and get the top 5 products
    for season in seasons:
        # Filter products for the current season
        season_products = aggregated_sales_data[aggregated_sales_data['season'] == season]

        # Check if there are products for the current season
        if not season_products.empty:
            # Sort products by total_value in descending order and get the top 5
            top_5 = season_products.nlargest(5, 'total_value')

            # Calculate the percentage of the total_value for each product
            top_5['percentage_of_total'] = (top_5['total_value'] / total_value_all_products) * 100

            # Store the result in the dictionary
            top_products[season] = top_5[['product_code', 'season', 'total_value', 'percentage_of_total']]

    return top_products, sales_data_ready



top_products, sales_data_ready = top_5_products_per_season_with_percentage(sales_data)

for season, products in top_products.items():
    print(f"Top 5 products for {season}:")
    print(products)



In [ ]:
sales_data_ready.head(100)

In [ ]:
sales_data_ready.info()

In [ ]:
# Filter out only records that have the year 2023
sales_data_2023_season = sales_data_ready[sales_data_ready['year_integer'] == 2023]

In [ ]:
sales_data_2023 = sales_data[sales_data['year_integer'] == 2023]

In [ ]:
sales_data_2023_season.info()

In [ ]:
# Function to get top 5 products for each season from filtered data
def get_top_5_products_per_season(filtered_data):
    # Dictionary to store top 5 products for each season
    top_products = {}

    # Calculate the total sales for the entire year
    total_annual_sales = sales_data_2023['total_value'].sum()

    # List of seasons
    seasons = ['winter', 'summer', 'spring', 'autumn']

    # Iterate over each season and get the top 5 products
    for season in seasons:
        # Filter products for the current season
        season_products = sales_data_2023_season[sales_data_2023_season['season'] == season]

        # Check if there are products for the current season
        if not season_products.empty:
            # Sort products by total_value in descending order and get the top 5
            top_5 = season_products.nlargest(5, 'total_value')

            # Calculate the percentage of the total annual sales for each product
            top_5['percentage_of_annual_sales'] = (top_5['total_value'] / total_annual_sales) * 100

            # Store the result in the dictionary
            top_products[season] = top_5[['product_code', 'season', 'total_value', 'percentage_of_annual_sales']]

    return top_products

# Get top 5 products for each season from the filtered data
top_products_2023 = get_top_5_products_per_season(sales_data_2023)

for season, products in top_products_2023.items():
    print(f"Top 5 products for {season} in 2023:")
    print(products)


In [ ]:
sales_data_2023.head()

## Total Monthly Sales across All Years, All Shops

In [ ]:
# Group by 'month' and sum 'transaction_count'
monthly_totals_all = monthly_sales_all.groupby('year_month')['transaction_count'].sum()


In [ ]:
monthly_totals_all.info()

In [ ]:
plt.figure(figsize=(30, 6))

# Convert index to string for compatibility with Matplotlib
index_as_string_all = monthly_totals_all.index.astype(str)

plt.plot(index_as_string_all, monthly_totals_all.values, marker='o', linestyle='-', color='blue')

plt.title('Total Monthly Transactions Across All Years')
plt.xlabel('Month')
plt.ylabel('Total Transactions')


# Rotate dates for better readability
plt.xticks(rotation=45)

plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
max_transaction_row_all = monthly_sales_all.loc[monthly_sales_all['transaction_count'].idxmax()]

In [ ]:
print(f"The product with the highest transaction count is: {max_transaction_row_all} with")

# Exploring How many Different Products were sold per month

In [ ]:
# Making a copy of the DataFrame to work on
sorted_pivot_sales_all_copy = sorted_pivot_sales_all.copy()

# Sum all rows per column and assign the result to a new row 'Total'
sorted_pivot_sales_all_copy.loc['Total'] = sorted_pivot_sales_all_copy.sum()

In [ ]:
# Ensure the 'Total' row is extracted correctly from the DataFrame
total_row_all = sorted_pivot_sales_all_copy.loc['Total']

In [ ]:
# Convert PeriodIndex to string directly for plotting
total_row_all.index = total_row_all.index.astype(str)

# Plotting remains the same
plt.figure(figsize=(30, 7))
plt.plot(total_row_all.index, total_row_all.values, marker='o', linestyle='-', color='blue')

plt.title('Total Number of Different Products Sold Each Month')
plt.xlabel('Month')
plt.ylabel('Total Products Sold')
plt.grid(True)

# Rotate dates for better readability
plt.xticks(rotation=45)

plt.tight_layout()

# Save the plot as an SVG file
plt.savefig("total_products_sold_each_month.svg", format='svg')

# Display the plot
plt.show()


In [ ]:
# Assuming 'trx_date' is already converted to datetime
# Create a 'Month' column formatted as 'YYYY-MM'
sales_data['Month'] = sales_data['trx_date'].dt.to_period('M').astype(str)

# Mark sales presence for each shop by month
sales_presence = sales_data.groupby(['shopID', 'Month']).size().unstack(fill_value=0)
sales_presence[sales_presence > 0] = 1

# Convert the matrix back to long format for easier plotting
sales_presence_long = sales_presence.reset_index().melt(id_vars='shopID', var_name='Month', value_name='Sales_Presence')

# Sum 'Sales_Presence' for each 'Month' to get the count of shops with sales
shops_active_per_month = sales_presence_long.groupby('Month')['Sales_Presence'].sum().reset_index()

# Plotting
plt.figure(figsize=(14, 7))
sns.lineplot(data=shops_active_per_month, x='Month', y='Sales_Presence', marker='o')

plt.title('Number of Active Shops per Month')
plt.xlabel('Month')
plt.ylabel('Number of Active Shops')
plt.xticks(rotation=45)
plt.grid(True)  # Enabling grid for better readability
plt.tight_layout()
plt.show()


# Families and Subfamilies

In [ ]:
# Filter for the years 1992 to 1995 and create an explicit copy
years_of_interest = [2020, 2021, 2022, 2023]
sales_data_filtered = sales_data[sales_data['trx_date'].dt.year.isin(years_of_interest)].copy()

# Adding year and quarter columns
sales_data_filtered['year'] = sales_data_filtered['trx_date'].dt.year
sales_data_filtered['quarter'] = sales_data_filtered['trx_date'].dt.to_period('Q')

# Aggregate quantities by year, quarter, and family
quarterly_shop_sums = sales_data_filtered.groupby(['family', 'year', 'quarter'])['total_value'].sum().reset_index()

# Get unique families
family = quarterly_shop_sums['family'].unique()

# Convert 'quarter' to string outside of the loop to avoid SettingWithCopyWarning
quarterly_shop_sums['quarter_str'] = quarterly_shop_sums['quarter'].astype(str)

plt.figure(figsize=(14, 10))

for fam in family:
    # Filter data for each family without additional modifications
    family_data = quarterly_shop_sums[quarterly_shop_sums['family'] == fam]
    plt.plot(family_data['quarter_str'], family_data['total_value'], marker='o', label=f'Family {fam}')

plt.title('Quarterly Total Value Development for Each Family (2020-2023)')
plt.xlabel('Quarter')
plt.ylabel('Total Value')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Assuming 'sub_family' is a column that categorizes products within families
# First, filter for Family 5
family_5_data = sales_data_filtered[sales_data_filtered['family'] == 5].copy()

# Adding year and quarter columns - this is likely already done but included for clarity
family_5_data['year'] = family_5_data['trx_date'].dt.year
family_5_data['quarter'] = family_5_data['trx_date'].dt.to_period('Q')

# Aggregate quantities by year, quarter, sub_family for Family 5
quarterly_family_5_sums = family_5_data.groupby(['sub_family', 'year', 'quarter'])['total_value'].sum().reset_index()

# Convert 'quarter' to string to avoid SettingWithCopyWarning and for ease of plotting
quarterly_family_5_sums['quarter_str'] = quarterly_family_5_sums['quarter'].astype(str)

# Get unique subfamilies within Family 5
sub_families = quarterly_family_5_sums['sub_family'].unique()

plt.figure(figsize=(14, 10))

# Plot each subfamily's quarterly data within Family 5
for sub_fam in sub_families:
    sub_fam_data = quarterly_family_5_sums[quarterly_family_5_sums['sub_family'] == sub_fam]
    plt.plot(sub_fam_data['quarter_str'], sub_fam_data['total_value'], marker='o', label=f'Subfamily {sub_fam}')

plt.title('Quarterly Total Value Development for Each Subfamily within Family 5 (2020-2023)')
plt.xlabel('Quarter')
plt.ylabel('Total Value')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Assuming 'sub_family' is a column that categorizes products within families
# First, filter for Family 1
family_1_data = sales_data_filtered[sales_data_filtered['family'] == 1].copy()

# Adding year and quarter columns - this is likely already done but included for clarity
family_1_data['year'] = family_1_data['trx_date'].dt.year
family_1_data['quarter'] = family_1_data['trx_date'].dt.to_period('Q')

# Aggregate quantities by year, quarter, sub_family for Family 1
quarterly_family_1_sums = family_1_data.groupby(['sub_family', 'year', 'quarter'])['total_value'].sum().reset_index()

# Convert 'quarter' to string to avoid SettingWithCopyWarning and for ease of plotting
quarterly_family_1_sums['quarter_str'] = quarterly_family_1_sums['quarter'].astype(str)

# Get unique subfamilies within Family 1
sub_families = quarterly_family_1_sums['sub_family'].unique()

plt.figure(figsize=(14, 10))

# Plot each subfamily's quarterly data within Family 1
for sub_fam in sub_families:
    sub_fam_data = quarterly_family_1_sums[quarterly_family_1_sums['sub_family'] == sub_fam]
    plt.plot(sub_fam_data['quarter_str'], sub_fam_data['total_value'], marker='o', label=f'Subfamily {sub_fam}')

plt.title('Quarterly Total Value Development for Each Subfamily within Family 1 (2020-2023)')
plt.xlabel('Quarter')
plt.ylabel('Total Value')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

# Pastries and Seasons

In [ ]:
# Assuming 'flag_pastry', 'shop_id', 'trx_date', and 'total_value' are columns in your sales_data DataFrame
# and that the get_season function has already been defined

# Filter for non-pastry sales for Shop 1 and Shop 5
shop_1_sales = sales_data[(sales_data['flag_pastry'] == 0) & (sales_data['shopID'] == 1)].copy()
shop_5_sales = sales_data[(sales_data['flag_pastry'] == 0) & (sales_data['shopID'] == 5)].copy()

# Adding 'season' column
shop_1_sales['season'] = shop_1_sales['trx_date'].dt.month.apply(get_season)
shop_5_sales['season'] = shop_5_sales['trx_date'].dt.month.apply(get_season)

# Group by season and sum up the total_value
seasonal_shop_1_sales = shop_1_sales.groupby('season')['total_value'].sum()
seasonal_shop_5_sales = shop_5_sales.groupby('season')['total_value'].sum()

# Plotting for Shop 1
plt.figure(figsize=(8, 6))
plt.pie(seasonal_shop_1_sales, labels=seasonal_shop_1_sales.index, autopct='%1.1f%%', startangle=140)
plt.title('Seasonal Distribution of Non-Pastry Sales for Shop 1')
plt.show()

# Plotting for Shop 5
plt.figure(figsize=(8, 6))
plt.pie(seasonal_shop_5_sales, labels=seasonal_shop_5_sales.index, autopct='%1.1f%%', startangle=140)
plt.title('Seasonal Distribution of Non-Pastry Sales for Shop 5')
plt.show()


In [ ]:
# Assuming 'flag_pastry', 'shop_id', 'trx_date', and 'total_value' are columns in your sales_data DataFrame
# and that the get_season function has already been defined

# Filter for non-pastry sales for Shop 1 and Shop 5
shop_1_sales = sales_data[(sales_data['flag_pastry'] == 1) & (sales_data['shopID'] == 1)].copy()
shop_5_sales = sales_data[(sales_data['flag_pastry'] == 1) & (sales_data['shopID'] == 5)].copy()

# Adding 'season' column
shop_1_sales['season'] = shop_1_sales['trx_date'].dt.month.apply(get_season)
shop_5_sales['season'] = shop_5_sales['trx_date'].dt.month.apply(get_season)

# Group by season and sum up the total_value
seasonal_shop_1_sales = shop_1_sales.groupby('season')['total_value'].sum()
seasonal_shop_5_sales = shop_5_sales.groupby('season')['total_value'].sum()

# Plotting for Shop 1
plt.figure(figsize=(8, 6))
plt.pie(seasonal_shop_1_sales, labels=seasonal_shop_1_sales.index, autopct='%1.1f%%', startangle=140)
plt.title('Seasonal Distribution of Pastry Sales for Shop 1')
plt.show()

# Plotting for Shop 5
plt.figure(figsize=(8, 6))
plt.pie(seasonal_shop_5_sales, labels=seasonal_shop_5_sales.index, autopct='%1.1f%%', startangle=140)
plt.title('Seasonal Distribution of Pastry Sales for Shop 5')
plt.show()


In [ ]:
# Assuming 'flag_pastries' and 'total_value' are columns in your sales_data DataFrame,
# and 'trx_date' is used to determine the season.

# Filter for flag_pastries = 1
pastry_sales = sales_data[sales_data['flag_pastry'] == 1].copy()

# Add a 'season' column based on the month of the transaction date, using the previously defined get_season function
pastry_sales['season'] = pastry_sales['trx_date'].dt.month.apply(get_season)

# Group by season and sum up the total_value
seasonal_pastry_sales = pastry_sales.groupby('season')['total_value'].sum()

# Plotting
plt.figure(figsize=(8, 6))
plt.pie(seasonal_pastry_sales, labels=seasonal_pastry_sales.index, autopct='%1.1f%%', startangle=140)
plt.title('Seasonal Distribution of Pastry Sales')
plt.show()


In [ ]:
# Filter for flag_pastries = 1
non_pastry_sales = sales_data[sales_data['flag_pastry'] == 0].copy()

# Add a 'season' column based on the month of the transaction date, using the previously defined get_season function
non_pastry_sales['season'] = non_pastry_sales['trx_date'].dt.month.apply(get_season)

# Group by season and sum up the total_value
seasonal_non_pastry_sales = non_pastry_sales.groupby('season')['total_value'].sum()

# Plotting
plt.figure(figsize=(8, 6))
plt.pie(seasonal_non_pastry_sales, labels=seasonal_non_pastry_sales.index, autopct='%1.1f%%', startangle=140)
plt.title('Seasonal Distribution of non-Pastry Sales')
plt.show()

In [ ]:
# Counting total products sold that were flagged as pastries
total_pastry_sold = sales_data[sales_data['flag_pastry'] == 1]['quantity'].sum()

# Counting total products sold that were not flagged as pastries
total_non_pastry_sold = sales_data[sales_data['flag_pastry'] == 0]['quantity'].sum()

# Pie chart
labels = 'Pastry Products', 'Non-Pastry Products'
sizes = [total_pastry_sold, total_non_pastry_sold]
colors = ['lightcoral', 'lightskyblue']
explode = (0.1, 0)  # explode the 1st slice (i.e., 'Pastry Products')

plt.figure(figsize=(8, 6))
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=140)
plt.title('Total Products Sold: Pastries vs Non-Pastries')
plt.show()

In [ ]:
# Counting total products sold that were flagged as pastries
total_pastry_sold = sales_data[sales_data['flag_pastry'] == 1]['total_value'].sum()

# Counting total products sold that were not flagged as pastries
total_non_pastry_sold = sales_data[sales_data['flag_pastry'] == 0]['total_value'].sum()

# Pie chart
labels = 'Pastry Products', 'Non-Pastry Products'
sizes = [total_pastry_sold, total_non_pastry_sold]
colors = ['lightcoral', 'lightskyblue']
explode = (0.1, 0)  # explode the 1st slice (i.e., 'Pastry Products')

plt.figure(figsize=(8, 6))
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=140)
plt.title('Total Value: Pastries vs Non-Pastries')
plt.show()

In [ ]:
sales_data_2023 = sales_data[sales_data['year_integer'] == 2023]

In [ ]:
# Counting total products sold that were flagged as pastries
total_pastry_sold_2023 = sales_data_2023[sales_data_2023['flag_pastry'] == 1]['total_value'].sum()

# Counting total products sold that were not flagged as pastries
total_non_pastry_sold_2023 = sales_data_2023[sales_data_2023['flag_pastry'] == 0]['total_value'].sum()

# Pie chart
labels = 'Pastry Products', 'Non-Pastry Products'
sizes = [total_pastry_sold_2023, total_non_pastry_sold_2023]
colors = ['lightcoral', 'lightskyblue']
explode = (0.1, 0)  # explode the 1st slice (i.e., 'Pastry Products')

plt.figure(figsize=(8, 6))
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=140)
plt.title('Total Value: Pastries vs Non-Pastries 2023')
plt.show()

# Day Of The Week 

In [ ]:
sales_data.head()

In [ ]:
# Group by 'day_of_week' and calculate the statistics
weekly_analysis = sales_data.groupby('day_of_week_name').agg(
    transaction_count=('day_of_week_name', 'size')
).reindex(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])  # To ensure days are in order

# Plotting the transaction count by day of the week
plt.figure(figsize=(12, 6))
plt.bar(weekly_analysis.index, weekly_analysis['transaction_count'], color='gray')
plt.title('Transaction Count by Day of the Week')
plt.ylabel('Transaction Count')
plt.xticks(rotation=45)

# Save the plot as an SVG file
plt.savefig("transaction_count_by_day_of_week.svg", format='svg')

# Display the plot
plt.show()

# Output the DataFrame for a tabular view
print(weekly_analysis)

# Analysis, Worst Items, which ones to drop?

In [ ]:
sales_data.head()

In [ ]:
# Assuming 'sales_data' is your DataFrame containing the sales data

# Group by 'product_code' and sum 'total_value' for each product
total_sales_per_product = sales_data.groupby('product_code')['total_value'].sum()

# Sort the aggregated sales in ascending order to get the products with the lowest sales
lowest_selling_products = total_sales_per_product.sort_values().head(10)

# Print the 10 products with the lowest total sales values
print(lowest_selling_products)


In [ ]:
# Group by 'shopID' and 'product_code' and calculate the sum of 'total_value' and 'quantity', and get the average 'price'
aggregated_data = sales_data.groupby(['shopID', 'product_code']).agg({
    'total_value': 'sum',
    'quantity': 'sum',
    'unit_price': 'mean'  # Assuming you want the average price; adjust if needed
}).reset_index()

# Sort within each shop by 'total_value' in ascending order to find the worst performing products
aggregated_data.sort_values(by=['shopID', 'total_value'], inplace=True)

# Get the top 10 worst-performing products for each shop
worst_products_per_shop = aggregated_data.groupby('shopID').head(10)

# Print the results
print(worst_products_per_shop)


In [ ]:
# Filter the aggregated data to include only products with at least 1 sale in December 2023
aggregated_data_december = aggregated_data[aggregated_data['quantity'] > 0]

# Initialize an empty dictionary to store the results for each shop
shop_worst_products = {}

# Get the top 20 worst-performing products for each shop
for shop_id in aggregated_data_december['shopID'].unique():
    shop_worst_products[shop_id] = aggregated_data_december[aggregated_data_december['shopID'] == shop_id].head(20)

# Output the results for each shop
for shop_id, data in shop_worst_products.items():
    print(f"Top 20 worst-performing products for Shop {shop_id}:")
    print(data)
    print("\n")  # Adds a newline for better readability between shops


In [ ]:
# Create a DataFrame to store the presence of each product in each shop
product_presence = aggregated_data_december.pivot_table(index='product_code', columns='shopID', aggfunc='size', fill_value=0)

# Calculate the hierarchical clustering
linkage = hierarchy.linkage(product_presence, method='average', metric='euclidean')

# Reorder the DataFrame based on the clustering
order = hierarchy.dendrogram(linkage, no_plot=True)['leaves']
product_presence_reordered = product_presence.iloc[order]

# Create a heatmap with larger cells
plt.figure(figsize=(16, 10))
sns.heatmap(product_presence_reordered, cmap="YlGnBu", cbar_kws={'label': 'Number of Shops'}, square=False)
plt.title('Presence of Products in "worst list" in each Shop (Clustered)')
plt.xlabel('Shop ID')
plt.ylabel('Product Code')

# Save the plot as an SVG file
plt.savefig("presence_of_products_clustered.svg", format='svg')

# Display the plot
plt.show()

In [ ]:
# Overwriting sales_data by dropping specified columns
sales_data = sales_data.drop(['season', 'semester', 'day_of_week_number', 'day_of_week_name'], axis=1, errors='ignore')

# sales_data now does not contain the columns 'season', 'semester', 'day_of_week_number', 'day_of_week_name'


In [ ]:
# Drop the year_integer column
sales_data_ready = sales_data_ready.drop(columns=['year_integer'])

In [ ]:
sales_data_shop1 = sales_data[sales_data['shopID'] == 1]
sales_data_shop2 = sales_data[sales_data['shopID'] == 2]
sales_data_shop3 = sales_data[sales_data['shopID'] == 3]
sales_data_shop4 = sales_data[sales_data['shopID'] == 4]
sales_data_shop5 = sales_data[sales_data['shopID'] == 5]
sales_data_shop6 = sales_data[sales_data['shopID'] == 6]
sales_data_shop7 = sales_data[sales_data['shopID'] == 7]
sales_data_shop8 = sales_data[sales_data['shopID'] == 8]
sales_data_shop9 = sales_data[sales_data['shopID'] == 9]
sales_data_shop10 = sales_data[sales_data['shopID'] == 10]
sales_data_shop11 = sales_data[sales_data['shopID'] == 11]
sales_data_shop12 = sales_data[sales_data['shopID'] == 12]
sales_data_shop13 = sales_data[sales_data['shopID'] == 13]
sales_data_shop14 = sales_data[sales_data['shopID'] == 14]
sales_data_shop15 = sales_data[sales_data['shopID'] == 15]
sales_data_shop16 = sales_data[sales_data['shopID'] == 16]

# Association Rule, 2020

## Everything

In [ ]:
# Grouping product codes by trx_id
transactions_2020 = sales_data_2020.groupby('trx_id')['product_code'].apply(list).tolist()

In [ ]:
# Encode transaction data
encoder_2020 = TransactionEncoder()
trans_array_2020 = encoder_2020.fit(transactions_2020).transform(transactions_2020)
trans_df_2020 = pd.DataFrame(trans_array_2020, columns=encoder_2020.columns_)

# Find frequent itemsets
frequent_itemsets_2020 = apriori(trans_df_2020, min_support=0.001, use_colnames=True)

# Generate association rules
rules_2020 = association_rules(frequent_itemsets_2020, metric="confidence", min_threshold=0.1)

In [ ]:
# Sort by confidence and lift for review
rules_sorted_2020 = rules_2020.sort_values(by=['confidence', 'lift'], ascending=False)

print(rules_sorted_2020[['antecedents', 'consequents', 'support', 'confidence', 'lift']])


# Association Rules, Modeling

# Shop1 2020 SEARCH 01

In [ ]:
# Filter for records from shop ID 1 and the year 1993
sales_data_2020_shop1 = sales_data_shop1[(sales_data_shop1['trx_date'].dt.year == 2020)]

In [ ]:
sales_data_2020_shop1.head()

In [ ]:
# Grouping product codes by trx_id
transactions_2020_shop1 = sales_data_2020_shop1.groupby('trx_id')['product_code'].apply(list).tolist()

In [ ]:
# Encode transaction data
encoder_2020_shop1 = TransactionEncoder()
trans_array_2020_shop1 = encoder_2020_shop1.fit(transactions_2020_shop1).transform(transactions_2020_shop1)
trans_df_2020_shop1 = pd.DataFrame(trans_array_2020_shop1, columns=encoder_2020_shop1.columns_)

In [ ]:
# Find frequent itemsets
frequent_itemsets_2020_shop1 = apriori(trans_df_2020_shop1, min_support=0.001, use_colnames=True)

In [ ]:
print(frequent_itemsets_2020_shop1)

In [ ]:
## Generate the association rules - by confidence
rulesConfidence_2020_shop1 = association_rules(frequent_itemsets_2020_shop1, metric="confidence", min_threshold=0.10)
rulesConfidence_2020_shop1.sort_values(by='confidence', ascending=False, inplace=True)


In [ ]:
# Note: This is a hypothetical example, as I can't directly manipulate or access dataframes not provided within this code cell.

# Replace frozen sets with strings
rulesConfidence_2020_shop1['antecedents_'] = rulesConfidence_2020_shop1['antecedents'].apply(lambda a: ','.join(list(a)))
rulesConfidence_2020_shop1['consequents_'] = rulesConfidence_2020_shop1['consequents'].apply(lambda a: ','.join(list(a)))

# Draw
fig, ax = plt.subplots(figsize=(14,30))
ax = plt.scatter(data=rulesConfidence_2020_shop1, x='consequents_', y='antecedents_', s=rulesConfidence_2020_shop1['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rulesConfidence_2020_shop1.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2020 Shop 1", fontsize=14)
plt.xlabel("Consequents (RHS)", fontsize=12)
plt.ylabel("Antecedents (LHS)", fontsize=12)
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate(rotation=45)
plt.tight_layout()

plt.show()



## Eliminate Redundant Rules

In [ ]:
rulesConfidence_2020_shop1.head()

In [ ]:
rulesConfidence_2020_shop1_sorted = rulesConfidence_2020_shop1.sort_values(by='lift', ascending=False)
rulesConfidence_2020_shop1_sorted.head(20)


In [ ]:
rulesConfidence_2020_shop1.info()

In [ ]:
# If 'antecedents' and 'consequents' are not frozensets, convert them
rules_2020_shop1_redundant = rulesConfidence_2020_shop1.copy()
rules_2020_shop1_redundant['antecedents'] = rules_2020_shop1_redundant['antecedents'].apply(lambda x: frozenset(x))
rules_2020_shop1_redundant['consequents'] = rules_2020_shop1_redundant['consequents'].apply(lambda x: frozenset(x))


In [ ]:
# Create a copy of the DataFrame to avoid modifying the original while iterating
rules_2020_shop1_checkredundance = rules_2020_shop1_redundant.copy()

# List to hold non-redundant rules
non_redundant_rules_2020_shop1 = []

for _, rule in rules_2020_shop1_checkredundance.iterrows():
    if not is_redundant(rule, rules_2020_shop1_checkredundance):
        non_redundant_rules_2020_shop1.append(rule)

# Convert the list back to a DataFrame
rules_2020_shop1_nonredundant = pd.DataFrame(non_redundant_rules_2020_shop1).reset_index(drop=True)


In [ ]:
print(rules_2020_shop1_nonredundant.head())
print(f"Original number of rules: {len(rules_2020_shop1_redundant)}")
print(f"Number of non-redundant rules: {len(rules_2020_shop1_nonredundant)}")


In [ ]:
rules_2020_shop1_nonredundant.head()

In [ ]:
rules_2020_shop1_redundant.info()

In [ ]:
rules_2020_shop1_nonredundant.head()

In [ ]:
rules_2020_shop1_nonredundant_backup = rules_2020_shop1_nonredundant.copy()

In [ ]:
rules_2020_shop1_nonredundant_unfiltered = rules_2020_shop1_nonredundant.copy()

In [ ]:
#reset
#rules_2020_shop1_nonredundant = rules_2020_shop1_nonredundant_backup.copy()

# Redundant and Unfiltered

In [ ]:
rules_2020_shop1_redundant.head()

In [ ]:
# Assuming 'rules_sorted_2022_shop4' is correctly defined and includes 'antecedents', 'consequents', and 'confidence'
G_2020_shop1 = nx.DiGraph()

# Iterate through rules for 2020 Shop 1 and add edges to the graph
for _, row in rules_2020_shop1_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as weight
    G_2020_shop1.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2020_shop1 = nx.spring_layout(G_2020_shop1)

# Recalculate node sizes based on indegree or another metric
in_degrees_2020_shop1 = dict(G_2020_shop1.in_degree())
node_sizes_2020_shop1 = [in_degrees_2020_shop1[node] * 100 + 1000 for node in G_2020_shop1.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2020_shop1 = (np.array(node_sizes_2020_shop1) - np.min(node_sizes_2020_shop1)) / (np.max(node_sizes_2020_shop1) - np.min(node_sizes_2020_shop1))

# Define a colormap from orange to red for visual consistency
cmap_2020_shop1 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2020_shop1 = [cmap_2020_shop1(size) for size in normalized_sizes_2020_shop1]

# Drawing the network
plt.figure(figsize=(60, 43))
nx.draw_networkx_nodes(G_2020_shop1, pos_2020_shop1, node_size=node_sizes_2020_shop1, node_color=node_colors_2020_shop1, alpha=0.8)
nx.draw_networkx_edges(G_2020_shop1, pos_2020_shop1, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2020_shop1, pos_2020_shop1, font_size=10)

plt.title('Top 5 Association Rules Network for 2020 Shop 1 with Color Gradient')
plt.axis('off')

# Save the plot as an SVG file
plt.savefig("association_rules_network_2020_shop1.svg", format='svg')

# Display the plot
plt.show()

### Based on LIFT

In [ ]:
# Assuming 'rules_2020_shop1_redundant' is correctly defined and includes 'antecedents', 'consequents', 'confidence', and 'lift'
G_2020_shop1_lift = nx.DiGraph()

lift_summary_2020_shop1 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2020_shop1_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2020_shop1_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2020_shop1:
        lift_summary_2020_shop1[consequents]['total_lift'] += row['lift']
        lift_summary_2020_shop1[consequents]['count'] += 1
    else:
        lift_summary_2020_shop1[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2020_shop1:
    lift_summary_2020_shop1[item]['avg_lift'] = lift_summary_2020_shop1[item]['total_lift'] / lift_summary_2020_shop1[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2020_shop1_lift = [lift_summary_2020_shop1[node]['avg_lift'] * 500 if node in lift_summary_2020_shop1 else 1000 for node in G_2020_shop1_lift.nodes()]

pos_2020_shop1_lift = nx.spring_layout(G_2020_shop1_lift)

# Colormap from orange to red
cmap_2020_shop1_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2020_shop1_lift = np.array(node_sizes_2020_shop1_lift)
normalized_sizes_2020_shop1_lift = (normalized_sizes_2020_shop1_lift - min(normalized_sizes_2020_shop1_lift)) / (max(normalized_sizes_2020_shop1_lift) - min(normalized_sizes_2020_shop1_lift))
node_colors_2020_shop1_lift = [cmap_2020_shop1_lift(size) for size in normalized_sizes_2020_shop1_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2020_shop1_lift, pos_2020_shop1_lift, node_size=node_sizes_2020_shop1_lift, node_color=node_colors_2020_shop1_lift, alpha=0.8)
nx.draw_networkx_edges(G_2020_shop1_lift, pos_2020_shop1_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2020_shop1_lift, pos_2020_shop1_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2020 Shop 1 with Color Gradient Based on Lift')
plt.axis('off')

# Save the plot as an SVG file
plt.savefig("association_rules_network_2020_shop1_lift.svg", format='svg')

# Display the plot
plt.show()

# nonredundant_unfiltered

## Based on Indegree

In [ ]:
G_2020_shop1_nonredundant_unfiltered = nx.DiGraph()

# Iterating through each row to add edges with confidence as an attribute
for _, row in rules_2020_shop1_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2020_shop1_nonredundant_unfiltered.add_edge(antecedents, consequents, weight=row['confidence'])

# Defining the layout for network visualization
pos_2020_shop1_nonredundant_unfiltered = nx.spring_layout(G_2020_shop1_nonredundant_unfiltered)

# Calculating node sizes based on the indegree for color mapping
in_degrees_2020_shop1_nonredundant_unfiltered = dict(G_2020_shop1_nonredundant_unfiltered.in_degree())
node_sizes_2020_shop1_nonredundant_unfiltered = [in_degrees_2020_shop1_nonredundant_unfiltered[node] * 100 + 1000 for node in G_2020_shop1_nonredundant_unfiltered.nodes()]

# Normalizing node sizes for color mapping
normalized_sizes_2020_shop1_nonredundant_unfiltered = (np.array(node_sizes_2020_shop1_nonredundant_unfiltered) - min(node_sizes_2020_shop1_nonredundant_unfiltered)) / (max(node_sizes_2020_shop1_nonredundant_unfiltered) - min(node_sizes_2020_shop1_nonredundant_unfiltered))

# Defining a color map from orange to red
cmap_2020_shop1_nonredundant_unfiltered = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])

# Mapping normalized node sizes to colors
node_colors_2020_shop1_nonredundant_unfiltered = [cmap_2020_shop1_nonredundant_unfiltered(size) for size in normalized_sizes_2020_shop1_nonredundant_unfiltered]

# Drawing the network with node sizes and colors
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2020_shop1_nonredundant_unfiltered, pos_2020_shop1_nonredundant_unfiltered, node_size=node_sizes_2020_shop1_nonredundant_unfiltered, node_color=node_colors_2020_shop1_nonredundant_unfiltered, alpha=0.8)
nx.draw_networkx_edges(G_2020_shop1_nonredundant_unfiltered, pos_2020_shop1_nonredundant_unfiltered, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2020_shop1_nonredundant_unfiltered, pos_2020_shop1_nonredundant_unfiltered, font_size=10)

plt.title('Node Size Based on Indegree for 2020 Shop 1')
plt.axis('off')
plt.show()

## Based on Lift

In [ ]:
G_2020_shop1_lift = nx.DiGraph()

lift_summary_2020_shop1 = {}

for _, row in rules_2020_shop1_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2020_shop1_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Update lift summary for consequents
    if consequents in lift_summary_2020_shop1:
        lift_summary_2020_shop1[consequents]['total_lift'] += row['lift']
        lift_summary_2020_shop1[consequents]['count'] += 1
    else:
        lift_summary_2020_shop1[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculate average lift for each consequent
for item in lift_summary_2020_shop1:
    lift_summary_2020_shop1[item]['avg_lift'] = lift_summary_2020_shop1[item]['total_lift'] / lift_summary_2020_shop1[item]['count']

# Node sizes based on average lift, scaled appropriately
node_sizes_2020_shop1_lift = [lift_summary_2020_shop1[node]['avg_lift'] * 500 if node in lift_summary_2020_shop1 else 1000 for node in G_2020_shop1_lift.nodes()]

pos_2020_shop1_lift = nx.spring_layout(G_2020_shop1_lift)

# Use a colormap that transitions from orange to red
cmap_2020_shop1_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2020_shop1_lift = np.array(node_sizes_2020_shop1_lift)
normalized_sizes_2020_shop1_lift = (normalized_sizes_2020_shop1_lift - min(normalized_sizes_2020_shop1_lift)) / (max(normalized_sizes_2020_shop1_lift) - min(normalized_sizes_2020_shop1_lift))
node_colors_2020_shop1_lift = [cmap_2020_shop1_lift(size) for size in normalized_sizes_2020_shop1_lift]

plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2020_shop1_lift, pos_2020_shop1_lift, node_size=node_sizes_2020_shop1_lift, node_color=node_colors_2020_shop1_lift, alpha=0.8)
nx.draw_networkx_edges(G_2020_shop1_lift, pos_2020_shop1_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2020_shop1_lift, pos_2020_shop1_lift, font_size=10)

plt.title('Node Size Based on Average Lift for 2020 Shop 1')
plt.axis('off')
plt.show()

# Based on conviction

In [ ]:
rules_2020_shop1_nonredundant.head()

In [ ]:
# Create a directed graph
G_2020_shop1_conviction = nx.DiGraph()

conviction_summary_2020_shop1 = {}
count_summary_2020_shop1 = {}

# Assuming 'rules_2020_shop1_nonredundant' contains a 'conviction' column
for _, row in rules_2020_shop1_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2020_shop1_conviction.add_edge(antecedents, consequents, weight=row['confidence'], conviction=row['conviction'])
    
    # Aggregate conviction values and count occurrences
    if consequents in conviction_summary_2020_shop1:
        conviction_summary_2020_shop1[consequents] += row['conviction']
        count_summary_2020_shop1[consequents] += 1
    else:
        conviction_summary_2020_shop1[consequents] = row['conviction']
        count_summary_2020_shop1[consequents] = 1

# Calculate average conviction values
average_conviction_summary_2020_shop1 = {k: conviction_summary_2020_shop1[k] / count_summary_2020_shop1[k] for k in conviction_summary_2020_shop1}

# Normalize conviction values for sizing and coloring
max_conviction = max(average_conviction_summary_2020_shop1.values())
min_conviction = min(average_conviction_summary_2020_shop1.values())
conviction_range = max_conviction - min_conviction
node_colors = []
node_sizes = []

for node in G_2020_shop1_conviction.nodes():
    # Get the average conviction value for the node, defaulting to min if not present
    conviction = average_conviction_summary_2020_shop1.get(node, min_conviction)
    # Normalize between 0 and 1
    normalized_conviction = (conviction - min_conviction) / conviction_range if conviction_range else 0
    node_colors.append(normalized_conviction)
    node_sizes.append(normalized_conviction * 1000 + 100)  # Scale and offset size

# Use a colormap for coloring nodes based on conviction
cmap = plt.cm.coolwarm

pos = nx.spring_layout(G_2020_shop1_conviction)

plt.figure(figsize=(40, 22))
nodes = nx.draw_networkx_nodes(G_2020_shop1_conviction, pos, node_color=node_colors, node_size=node_sizes, cmap=cmap, alpha=0.8)
edges = nx.draw_networkx_edges(G_2020_shop1_conviction, pos, arrowstyle='->', arrowsize=10, edge_color='grey', alpha=0.5)
labels = nx.draw_networkx_labels(G_2020_shop1_conviction, pos, font_size=8)

plt.colorbar(nodes, label='Normalized Conviction')
plt.title('Network Graph of 2020 Shop 1 Based on Conviction')
plt.axis('off')

# Save the plot as an SVG file
plt.savefig("network_graph_2020_shop1_conviction.svg", format='svg')

# Display the plot
plt.show()

In [ ]:
print(max_conviction)

# Filtered top 30 and non redundant

In [ ]:
rules_2020_shop1_nonredundant.head()

In [ ]:
# Assuming rules_1992_shop2_nonredundant is your sorted rules DataFrame for shop2
rules_2020_shop1_nonredundant_top30 = rules_2020_shop1_nonredundant.copy(deep=True)

In [ ]:
rules_2020_shop1_nonredundant_top30_lift = rules_2020_shop1_nonredundant.copy(deep=True)

In [ ]:
# Select the top 30 rules based on lift or another criterion
rules_2020_shop1_nonredundant_top30_lift = rules_2020_shop1_nonredundant_top30_lift.sort_values(by='lift', ascending=False).head(30)

In [ ]:
rules_2020_shop1_nonredundant_top30_lift.head()

In [ ]:
# Select the top 30 rules based on confidence or another criterion
rules_2020_shop1_nonredundant_top30 = rules_2020_shop1_nonredundant_top30.sort_values(by='confidence', ascending=False).head(30)

In [ ]:
rules_2020_shop1_nonredundant_top30.info()

In [ ]:
rules_2020_shop1_nonredundant_top30['LHS'] = [','.join(list(x)) for x in rules_2020_shop1_nonredundant_top30['antecedents']]
rules_2020_shop1_nonredundant_top30['RHS'] = [','.join(list(x)) for x in rules_2020_shop1_nonredundant_top30['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2020_shop1_nonredundant_top30.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2020_shop1_nonredundant_top30, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2020 Shop 2")
plt.axis('off')
plt.show()


## Based on Indegree

In [ ]:
G_2020_shop1 = nx.DiGraph()

# Assuming 'rules_sorted_2020_shop1' contains your association rules for shop 1 in 2020
for _, row in rules_2020_shop1_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as the weight
    G_2020_shop1.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2020_shop1 = nx.spring_layout(G_2020_shop1)

# Calculate node sizes based on indegree
in_degrees_2020_shop1 = dict(G_2020_shop1.in_degree())
node_sizes_2020_shop1 = [in_degrees_2020_shop1[node] * 100 + 1000 for node in G_2020_shop1.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2020_shop1 = (np.array(node_sizes_2020_shop1) - np.min(node_sizes_2020_shop1)) / (np.max(node_sizes_2020_shop1) - np.min(node_sizes_2020_shop1))

# Define a colormap from orange to red
cmap_2020_shop1 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2020_shop1 = [cmap_2020_shop1(size) for size in normalized_sizes_2020_shop1]

# Drawing the network
plt.figure(figsize=(15, 24))
nx.draw_networkx_nodes(G_2020_shop1, pos_2020_shop1, node_size=node_sizes_2020_shop1, node_color=node_colors_2020_shop1, alpha=0.8)
nx.draw_networkx_edges(G_2020_shop1, pos_2020_shop1, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2020_shop1, pos_2020_shop1, font_size=8)

plt.title('Top 5 Association Rules Network for 2020 Shop 1 with Color Gradient')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2020_shop1_lift = nx.DiGraph()

lift_summary_2020_shop1 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2020_shop1_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2020_shop1_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2020_shop1:
        lift_summary_2020_shop1[consequents]['total_lift'] += row['lift']
        lift_summary_2020_shop1[consequents]['count'] += 1
    else:
        lift_summary_2020_shop1[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2020_shop1:
    lift_summary_2020_shop1[item]['avg_lift'] = lift_summary_2020_shop1[item]['total_lift'] / lift_summary_2020_shop1[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2020_shop1_lift = [lift_summary_2020_shop1[node]['avg_lift'] * 500 if node in lift_summary_2020_shop1 else 1000 for node in G_2020_shop1_lift.nodes()]

pos_2020_shop1_lift = nx.spring_layout(G_2020_shop1_lift)

# Colormap from orange to red
cmap_2020_shop1_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2020_shop1_lift = np.array(node_sizes_2020_shop1_lift)
normalized_sizes_2020_shop1_lift = (normalized_sizes_2020_shop1_lift - min(normalized_sizes_2020_shop1_lift)) / (max(normalized_sizes_2020_shop1_lift) - min(normalized_sizes_2020_shop1_lift))
node_colors_2020_shop1_lift = [cmap_2020_shop1_lift(size) for size in normalized_sizes_2020_shop1_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(30, 22))
nx.draw_networkx_nodes(G_2020_shop1_lift, pos_2020_shop1_lift, node_size=node_sizes_2020_shop1_lift, node_color=node_colors_2020_shop1_lift, alpha=0.8)
nx.draw_networkx_edges(G_2020_shop1_lift, pos_2020_shop1_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2020_shop1_lift, pos_2020_shop1_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2020 Shop 1 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2020_shop1_lift = nx.DiGraph()

lift_summary_2020_shop1 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2020_shop1_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2020_shop1_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2020_shop1:
        lift_summary_2020_shop1[consequents]['total_lift'] += row['lift']
        lift_summary_2020_shop1[consequents]['count'] += 1
    else:
        lift_summary_2020_shop1[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2020_shop1:
    lift_summary_2020_shop1[item]['avg_lift'] = lift_summary_2020_shop1[item]['total_lift'] / lift_summary_2020_shop1[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2020_shop1_lift = [lift_summary_2020_shop1[node]['avg_lift'] * 500 if node in lift_summary_2020_shop1 else 1000 for node in G_2020_shop1_lift.nodes()]

pos_2020_shop1_lift = nx.spring_layout(G_2020_shop1_lift)

# Colormap from orange to red
cmap_2020_shop1_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2020_shop1_lift = np.array(node_sizes_2020_shop1_lift)
normalized_sizes_2020_shop1_lift = (normalized_sizes_2020_shop1_lift - min(normalized_sizes_2020_shop1_lift)) / (max(normalized_sizes_2020_shop1_lift) - min(normalized_sizes_2020_shop1_lift))
node_colors_2020_shop1_lift = [cmap_2020_shop1_lift(size) for size in normalized_sizes_2020_shop1_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2020_shop1_lift, pos_2020_shop1_lift, node_size=node_sizes_2020_shop1_lift, node_color=node_colors_2020_shop1_lift, alpha=0.8)
nx.draw_networkx_edges(G_2020_shop1_lift, pos_2020_shop1_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2020_shop1_lift, pos_2020_shop1_lift, font_size=10)

plt.title('Top 5 Association Rules Network for 2020 Shop 1 Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2020_shop1 = nx.DiGraph()

# Assuming 'rules_2020_shop1_nonredundant' contains association rules for 2020 Shop 1
for _, row in rules_2020_shop1_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2020_shop1.add_edge(antecedents, consequents, confidence=row['confidence'])

# Calculate node sizes based on indegree
in_degrees_2020_shop1 = dict(G_2020_shop1.in_degree())
node_sizes_2020_shop1 = [in_degrees_2020_shop1[node] * 100 + 1000 for node in G_2020_shop1.nodes()]

# Edge width based on confidence
edge_widths = [G_2020_shop1[u][v]['confidence'] * 15 for u, v in G_2020_shop1.edges()]

# Assuming normalized sizes and color mapping is desired
normalized_sizes_2020_shop1 = np.array(list(in_degrees_2020_shop1.values()))
normalized_sizes_2020_shop1 = (normalized_sizes_2020_shop1 - min(normalized_sizes_2020_shop1)) / (max(normalized_sizes_2020_shop1) - min(normalized_sizes_2020_shop1))
cmap_2020_shop1 = mcolors.LinearSegmentedColormap.from_list("MyCMapName", ["blue", "red"])

# Node colors based on a property (here, using indegree for example)
node_colors_2020_shop1 = [cmap_2020_shop1(size) for size in normalized_sizes_2020_shop1]

# Layout
pos = nx.spring_layout(G_2020_shop1, seed=42)

# Drawing
plt.figure(figsize=(40, 30))
nx.draw_networkx_edges(G_2020_shop1, pos, width=edge_widths, edge_color='grey', alpha=0.5)
nx.draw_networkx_nodes(G_2020_shop1, pos, node_size=node_sizes_2020_shop1, node_color=node_colors_2020_shop1, alpha=0.8)
nx.draw_networkx_labels(G_2020_shop1, pos, font_size=8, font_color='black')
plt.title('Network Graph of Association Rules Based on Confidence for 2020 Shop 1')
plt.axis('off')
plt.show()


In [ ]:
# Create a directed graph
G_2020_shop1 = nx.DiGraph()

convictions = []  # To store conviction values for normalization

# Assuming 'rules_2020_shop1_nonredundant_top30' contains 'antecedents', 'consequents', 'confidence', and 'conviction' columns
for _, row in rules_2020_shop1_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    conviction = row['conviction']
    convictions.append(conviction)
    G_2020_shop1.add_edge(antecedents, consequents, weight=row['confidence'], conviction=conviction)

# Normalize conviction values for edge width
max_conviction = max(convictions)
min_conviction = min(convictions)
conviction_range = max_conviction - min_conviction
edge_widths = [(G_2020_shop1[u][v]['conviction'] - min_conviction) / conviction_range * 30 + 0.5 for u, v in G_2020_shop1.edges()]  # Scale and offset widths

pos_2020_shop1 = nx.spring_layout(G_2020_shop1)

# Node sizes and colors as before
in_degrees_2020_shop1 = dict(G_2020_shop1.in_degree())
node_sizes_2020_shop1 = [in_degrees_2020_shop1[node] * 100 + 1000 for node in G_2020_shop1.nodes()]
normalized_sizes_2020_shop1 = (np.array(node_sizes_2020_shop1) - min(node_sizes_2020_shop1)) / (max(node_sizes_2020_shop1) - min(node_sizes_2020_shop1))
cmap_2020_shop1 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2020_shop1 = [cmap_2020_shop1(size) for size in normalized_sizes_2020_shop1]

# Drawing the network with conviction-based edge widths
plt.figure(figsize=(25, 25))
nx.draw_networkx_nodes(G_2020_shop1, pos_2020_shop1, node_size=node_sizes_2020_shop1, node_color=node_colors_2020_shop1, alpha=0.8)
nx.draw_networkx_edges(G_2020_shop1, pos_2020_shop1, width=edge_widths, edge_color='grey', alpha=0.5)
nx.draw_networkx_labels(G_2020_shop1, pos_2020_shop1, font_size=10)

plt.title('Top 5 Association Rules Network for 2020 Shop 1 with Conviction-based Edge Thickness')
plt.axis('off')

# Save the plot as an SVG file
plt.savefig("association_rules_network_2020_shop1_conviction.svg", format='svg')

# Display the plot
plt.show()

Thicker arrows: Indicate stronger associations between items. A thicker arrow suggests that the consequent item is highly dependent on the antecedent item(s) based on the conviction metric.

In [ ]:
#the node sizes are based on the indegree where each node acts as a consequent,

In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2020_shop1_nonredundant_top30.loc[:, 'antecedents_'] = rules_2020_shop1_nonredundant_top30['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2020_shop1_nonredundant_top30.loc[:, 'consequents_'] = rules_2020_shop1_nonredundant_top30['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2020_shop1_nonredundant_top30, x='consequents_', y='antecedents_', s=rules_2020_shop1_nonredundant_top30['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2020_shop1_nonredundant_top30.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2020 Shop 1", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
print(rules_2020_shop1_nonredundant_top30[['antecedents', 'consequents']].head(10))

In [ ]:
rules_2020_shop1_nonredundant_top30.info()

## Monetary Analysis of the Top Rules

In [ ]:
# Step 1: Extract unique product codes from the rules
unique_product_codes = set()
for idx, row in rules_2020_shop1_nonredundant_top30.iterrows():
    # Unite antecedents and consequents into a single set
    all_products = row['antecedents'].union(row['consequents'])
    unique_product_codes.update(all_products)

# Step 2: Create a DataFrame with unique product codes and their unit prices
# Filter sales_data for these product codes and drop duplicates to ensure each product code appears once
product_prices = sales_data[sales_data['product_code'].isin(unique_product_codes)][['product_code', 'unit_price']].drop_duplicates()

# Create a dictionary for quick lookup of prices
price_dict = product_prices.set_index('product_code')['unit_price'].to_dict()

# Step 3: Calculate the total price for each rule and collect data for the new DataFrame
data_for_new_df = []
for idx, row in rules_2020_shop1_nonredundant_top30.iterrows():
    # Combine sets of antecedents and consequents
    all_products = row['antecedents'].union(row['consequents'])
    total_price = sum(price_dict.get(prod, 0) for prod in all_products)  # Use get to avoid KeyError if any product code is missing
    data_for_new_df.append({
        'antecedents': row['antecedents'],
        'consequents': row['consequents'],
        'total_unit_price': total_price
    })

# Create a new DataFrame with only antecedents, consequents, and total unit price
rules_and_price_shop1_2020 = pd.DataFrame(data_for_new_df)


In [ ]:
# Sort the DataFrame by total_unit_price in descending order
rules_and_price_shop1_2020_sorted = rules_and_price_shop1_2020.sort_values(by='total_unit_price', ascending=False)

# Print the sorted DataFrame to see the specified columns along with their total unit prices
print(rules_and_price_shop1_2020_sorted)

In [ ]:
# Convert 'trx_date' to datetime format if not already done so
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Add a 'year' column if it doesn't exist
sales_data['year'] = sales_data['trx_date'].dt.year

# Step 2: Filter sales_data for shop1 and the year 2020
filtered_sales_data = sales_data[(sales_data['shopID'] == 1) & (sales_data['year'] == 2020)]

# Step 3: Initialize a list to hold the rule data including total sales value
rule_sales_summaries = []

# Initialize a set to track unique itemsets
unique_itemsets = set()

# Step 4: Iterate through each rule in the DataFrame
for index, rule in rules_2020_shop1_nonredundant_top30.iterrows():
    all_products = frozenset(rule['antecedents'].union(rule['consequents']))

    # Check if the itemset is unique
    if all_products not in unique_itemsets:
        unique_itemsets.add(all_products)

        # Filter the sales data for transactions that include all products in the rule
        mask = filtered_sales_data['product_code'].isin(all_products)
        transaction_ids = filtered_sales_data[mask].groupby('trx_id').filter(lambda x: all_products.issubset(set(x['product_code']))).trx_id.unique()
        matched_transactions = filtered_sales_data[filtered_sales_data['trx_id'].isin(transaction_ids)]
        
        # Step 5: Sum the total_value of these transactions
        total_sales_value = matched_transactions['total_value'].sum()

        # Append the result to the list
        rule_sales_summaries.append({
            'antecedents': rule['antecedents'],
            'consequents': rule['consequents'],
            'total_sales_value': total_sales_value
        })

# Step 6: Convert the list to a DataFrame
rules_sales_values_df = pd.DataFrame(rule_sales_summaries)

# Step 7: Sort by total_sales_value in descending order
sorted_rules_sales_values_df = rules_sales_values_df.sort_values(by='total_sales_value', ascending=False)

# Step 8: Print the sorted DataFrame
print(sorted_rules_sales_values_df)


In [ ]:
# Filter sales_data for product codes 'ID_44' and 'ID_60'
products_of_interest = sales_data[sales_data['product_code'].isin(['ID_44', 'ID_60'])]

# Get unique prices for these products
unique_prices = products_of_interest[['product_code', 'unit_price']].drop_duplicates()

# Print the unique prices
print(unique_prices)


## Narrowing down based on Lift

In [ ]:
rules_2020_shop1_nonredundant_top30_lift['LHS'] = [','.join(list(x)) for x in rules_2020_shop1_nonredundant_top30_lift['antecedents']]
rules_2020_shop1_nonredundant_top30_lift['RHS'] = [','.join(list(x)) for x in rules_2020_shop1_nonredundant_top30_lift['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2020_shop1_nonredundant_top30_lift.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2020_shop1_nonredundant_top30_lift, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2020 Shop 2")
plt.axis('off')
plt.show()


In [ ]:
G_2020_shop1_lift = nx.DiGraph()

lift_summary_2020_shop1 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2020_shop1_nonredundant_top30_lift.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2020_shop1_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2020_shop1:
        lift_summary_2020_shop1[consequents]['total_lift'] += row['lift']
        lift_summary_2020_shop1[consequents]['count'] += 1
    else:
        lift_summary_2020_shop1[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2020_shop1:
    lift_summary_2020_shop1[item]['avg_lift'] = lift_summary_2020_shop1[item]['total_lift'] / lift_summary_2020_shop1[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2020_shop1_lift = [lift_summary_2020_shop1[node]['avg_lift'] * 500 if node in lift_summary_2020_shop1 else 1000 for node in G_2020_shop1_lift.nodes()]

pos_2020_shop1_lift = nx.spring_layout(G_2020_shop1_lift)

# Colormap from orange to red
cmap_2020_shop1_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2020_shop1_lift = np.array(node_sizes_2020_shop1_lift)
normalized_sizes_2020_shop1_lift = (normalized_sizes_2020_shop1_lift - min(normalized_sizes_2020_shop1_lift)) / (max(normalized_sizes_2020_shop1_lift) - min(normalized_sizes_2020_shop1_lift))
node_colors_2020_shop1_lift = [cmap_2020_shop1_lift(size) for size in normalized_sizes_2020_shop1_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(30, 22))
nx.draw_networkx_nodes(G_2020_shop1_lift, pos_2020_shop1_lift, node_size=node_sizes_2020_shop1_lift, node_color=node_colors_2020_shop1_lift, alpha=0.8)
nx.draw_networkx_edges(G_2020_shop1_lift, pos_2020_shop1_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2020_shop1_lift, pos_2020_shop1_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2020 Shop 1 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2020_shop1_nonredundant_top30_lift.loc[:, 'antecedents_'] = rules_2020_shop1_nonredundant_top30_lift['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2020_shop1_nonredundant_top30_lift.loc[:, 'consequents_'] = rules_2020_shop1_nonredundant_top30_lift['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2020_shop1_nonredundant_top30_lift, x='consequents_', y='antecedents_', s=rules_2020_shop1_nonredundant_top30_lift['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2020_shop1_nonredundant_top30_lift.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2020 Shop 1", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
# Print only the 'antecedents' and 'consequents' columns of the top 10 rules
print(rules_2020_shop1_nonredundant_top30_lift[['antecedents', 'consequents']].head(10))


# Shop1 2021 SEARCH 2

In [ ]:
# Filter for records from shop ID 1 and the year 1993
sales_data_2021_shop1 = sales_data_shop1[(sales_data_shop1['trx_date'].dt.year == 2021)]

In [ ]:
sales_data_2021_shop1.head()

In [ ]:
# Grouping product codes by trx_id
transactions_2021_shop1 = sales_data_2021_shop1.groupby('trx_id')['product_code'].apply(list).tolist()

In [ ]:
# Encode transaction data
encoder_2021_shop1 = TransactionEncoder()
trans_array_2021_shop1 = encoder_2021_shop1.fit(transactions_2021_shop1).transform(transactions_2021_shop1)
trans_df_2021_shop1 = pd.DataFrame(trans_array_2021_shop1, columns=encoder_2021_shop1.columns_)

In [ ]:
# Find frequent itemsets
frequent_itemsets_2021_shop1 = apriori(trans_df_2021_shop1, min_support=0.001, use_colnames=True)

In [ ]:
print(frequent_itemsets_2021_shop1)

In [ ]:
## Generate the association rules - by confidence
rulesConfidence_2021_shop1 = association_rules(frequent_itemsets_2021_shop1, metric="confidence", min_threshold=0.10)
rulesConfidence_2021_shop1.sort_values(by='confidence', ascending=False, inplace=True)


In [ ]:
# Note: This is a hypothetical example, as I can't directly manipulate or access dataframes not provided within this code cell.

# Replace frozen sets with strings
rulesConfidence_2021_shop1['antecedents_'] = rulesConfidence_2021_shop1['antecedents'].apply(lambda a: ','.join(list(a)))
rulesConfidence_2021_shop1['consequents_'] = rulesConfidence_2021_shop1['consequents'].apply(lambda a: ','.join(list(a)))

# Draw
fig, ax = plt.subplots(figsize=(14,30))
ax = plt.scatter(data=rulesConfidence_2021_shop1, x='consequents_', y='antecedents_', s=rulesConfidence_2021_shop1['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rulesConfidence_2021_shop1.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2020 Shop 1", fontsize=14)
plt.xlabel("Consequents (RHS)", fontsize=12)
plt.ylabel("Antecedents (LHS)", fontsize=12)
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate(rotation=45)
plt.tight_layout()

plt.show()



## Eliminate Redundant Rules

In [ ]:
rulesConfidence_2021_shop1.head()

In [ ]:
rulesConfidence_2021_shop1.info()

In [ ]:
# If 'antecedents' and 'consequents' are not frozensets, convert them
rules_2021_shop1_redundant = rulesConfidence_2021_shop1.copy()
rules_2021_shop1_redundant['antecedents'] = rules_2021_shop1_redundant['antecedents'].apply(lambda x: frozenset(x))
rules_2021_shop1_redundant['consequents'] = rules_2021_shop1_redundant['consequents'].apply(lambda x: frozenset(x))


In [ ]:
# Create a copy of the DataFrame to avoid modifying the original while iterating
rules_2021_shop1_checkredundance = rules_2021_shop1_redundant.copy()

# List to hold non-redundant rules
non_redundant_rules_2021_shop1 = []

for _, rule in rules_2021_shop1_checkredundance.iterrows():
    if not is_redundant(rule, rules_2021_shop1_checkredundance):
        non_redundant_rules_2021_shop1.append(rule)

# Convert the list back to a DataFrame
rules_2021_shop1_nonredundant = pd.DataFrame(non_redundant_rules_2021_shop1).reset_index(drop=True)


In [ ]:
print(rules_2021_shop1_nonredundant.head())
print(f"Original number of rules: {len(rules_2021_shop1_redundant)}")
print(f"Number of non-redundant rules: {len(rules_2021_shop1_nonredundant)}")


In [ ]:
rules_2021_shop1_nonredundant.head()

In [ ]:
rules_2021_shop1_redundant.info()

In [ ]:
rules_2021_shop1_nonredundant.head()

In [ ]:
rules_2021_shop1_nonredundant_backup = rules_2021_shop1_nonredundant.copy()

In [ ]:
rules_2021_shop1_nonredundant_unfiltered = rules_2021_shop1_nonredundant.copy()

In [ ]:
#reset
#rules_2021_shop1_nonredundant = rules_2021_shop1_nonredundant_backup.copy()

# Redundant and Unfiltered

In [ ]:
rules_2021_shop1_redundant.head()

In [ ]:
# Assuming 'rules_sorted_2022_shop4' is correctly defined and includes 'antecedents', 'consequents', and 'confidence'
G_2021_shop1 = nx.DiGraph()

# Iterate through rules for 2021 Shop 1 and add edges to the graph
for _, row in rules_2021_shop1_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as weight
    G_2021_shop1.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2021_shop1 = nx.spring_layout(G_2021_shop1)

# Recalculate node sizes based on indegree or another metric
in_degrees_2021_shop1 = dict(G_2021_shop1.in_degree())
node_sizes_2021_shop1 = [in_degrees_2021_shop1[node] * 100 + 1000 for node in G_2021_shop1.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2021_shop1 = (np.array(node_sizes_2021_shop1) - np.min(node_sizes_2021_shop1)) / (np.max(node_sizes_2021_shop1) - np.min(node_sizes_2021_shop1))

# Define a colormap from orange to red for visual consistency
cmap_2021_shop1 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2021_shop1 = [cmap_2021_shop1(size) for size in normalized_sizes_2021_shop1]

# Drawing the network
plt.figure(figsize=(60, 43))
nx.draw_networkx_nodes(G_2021_shop1, pos_2021_shop1, node_size=node_sizes_2021_shop1, node_color=node_colors_2021_shop1, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop1, pos_2021_shop1, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2021_shop1, pos_2021_shop1, font_size=10)

plt.title('Top 5 Association Rules Network for 2021 Shop 1 with Color Gradient')
plt.axis('off')
plt.show()


### Based on LIFT

In [ ]:
G_2021_shop1_lift = nx.DiGraph()

lift_summary_2021_shop1 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2021_shop1_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop1_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2021_shop1:
        lift_summary_2021_shop1[consequents]['total_lift'] += row['lift']
        lift_summary_2021_shop1[consequents]['count'] += 1
    else:
        lift_summary_2021_shop1[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2021_shop1:
    lift_summary_2021_shop1[item]['avg_lift'] = lift_summary_2021_shop1[item]['total_lift'] / lift_summary_2021_shop1[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2021_shop1_lift = [lift_summary_2021_shop1[node]['avg_lift'] * 500 if node in lift_summary_2021_shop1 else 1000 for node in G_2021_shop1_lift.nodes()]

pos_2021_shop1_lift = nx.spring_layout(G_2021_shop1_lift)

# Colormap from orange to red
cmap_2021_shop1_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2021_shop1_lift = np.array(node_sizes_2021_shop1_lift)
normalized_sizes_2021_shop1_lift = (normalized_sizes_2021_shop1_lift - min(normalized_sizes_2021_shop1_lift)) / (max(normalized_sizes_2021_shop1_lift) - min(normalized_sizes_2021_shop1_lift))
node_colors_2021_shop1_lift = [cmap_2021_shop1_lift(size) for size in normalized_sizes_2021_shop1_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2021_shop1_lift, pos_2021_shop1_lift, node_size=node_sizes_2021_shop1_lift, node_color=node_colors_2021_shop1_lift, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop1_lift, pos_2021_shop1_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2021_shop1_lift, pos_2021_shop1_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2021 Shop 1 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


# nonredundant_unfiltered

## Based on Indegree

In [ ]:
G_2021_shop1_nonredundant_unfiltered = nx.DiGraph()

# Iterating through each row to add edges with confidence as an attribute
for _, row in rules_2021_shop1_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop1_nonredundant_unfiltered.add_edge(antecedents, consequents, weight=row['confidence'])

# Defining the layout for network visualization
pos_2021_shop1_nonredundant_unfiltered = nx.spring_layout(G_2021_shop1_nonredundant_unfiltered)

# Calculating node sizes based on the indegree for color mapping
in_degrees_2021_shop1_nonredundant_unfiltered = dict(G_2021_shop1_nonredundant_unfiltered.in_degree())
node_sizes_2021_shop1_nonredundant_unfiltered = [in_degrees_2021_shop1_nonredundant_unfiltered[node] * 100 + 1000 for node in G_2021_shop1_nonredundant_unfiltered.nodes()]

# Normalizing node sizes for color mapping
normalized_sizes_2021_shop1_nonredundant_unfiltered = (np.array(node_sizes_2021_shop1_nonredundant_unfiltered) - min(node_sizes_2021_shop1_nonredundant_unfiltered)) / (max(node_sizes_2021_shop1_nonredundant_unfiltered) - min(node_sizes_2021_shop1_nonredundant_unfiltered))

# Defining a color map from orange to red
cmap_2021_shop1_nonredundant_unfiltered = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])

# Mapping normalized node sizes to colors
node_colors_2021_shop1_nonredundant_unfiltered = [cmap_2021_shop1_nonredundant_unfiltered(size) for size in normalized_sizes_2021_shop1_nonredundant_unfiltered]

# Drawing the network with node sizes and colors
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2021_shop1_nonredundant_unfiltered, pos_2021_shop1_nonredundant_unfiltered, node_size=node_sizes_2021_shop1_nonredundant_unfiltered, node_color=node_colors_2021_shop1_nonredundant_unfiltered, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop1_nonredundant_unfiltered, pos_2021_shop1_nonredundant_unfiltered, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2021_shop1_nonredundant_unfiltered, pos_2021_shop1_nonredundant_unfiltered, font_size=10)

plt.title('Node Size Based on Indegree for 2021 Shop 1')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2021_shop1_lift = nx.DiGraph()

lift_summary_2021_shop1 = {}

for _, row in rules_2021_shop1_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop1_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Update lift summary for consequents
    if consequents in lift_summary_2021_shop1:
        lift_summary_2021_shop1[consequents]['total_lift'] += row['lift']
        lift_summary_2021_shop1[consequents]['count'] += 1
    else:
        lift_summary_2021_shop1[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculate average lift for each consequent
for item in lift_summary_2021_shop1:
    lift_summary_2021_shop1[item]['avg_lift'] = lift_summary_2021_shop1[item]['total_lift'] / lift_summary_2021_shop1[item]['count']

# Node sizes based on average lift, scaled appropriately
node_sizes_2021_shop1_lift = [lift_summary_2021_shop1[node]['avg_lift'] * 500 if node in lift_summary_2021_shop1 else 1000 for node in G_2021_shop1_lift.nodes()]

pos_2021_shop1_lift = nx.spring_layout(G_2021_shop1_lift)

# Use a colormap that transitions from orange to red
cmap_2021_shop1_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2021_shop1_lift = np.array(node_sizes_2021_shop1_lift)
normalized_sizes_2021_shop1_lift = (normalized_sizes_2021_shop1_lift - min(normalized_sizes_2021_shop1_lift)) / (max(normalized_sizes_2021_shop1_lift) - min(normalized_sizes_2021_shop1_lift))
node_colors_2021_shop1_lift = [cmap_2021_shop1_lift(size) for size in normalized_sizes_2021_shop1_lift]

plt.figure(figsize=(50, 34))
nx.draw_networkx_nodes(G_2021_shop1_lift, pos_2021_shop1_lift, node_size=node_sizes_2021_shop1_lift, node_color=node_colors_2021_shop1_lift, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop1_lift, pos_2021_shop1_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2021_shop1_lift, pos_2021_shop1_lift, font_size=10)

plt.title('Node Size Based on Average Lift for 2021 Shop 1')
plt.axis('off')
plt.show()


# Based on conviction

In [ ]:
rules_2021_shop1_nonredundant.head()

In [ ]:
G_2021_shop1_conviction = nx.DiGraph()

conviction_summary_2021_shop1 = {}
count_summary_2021_shop1 = {}

# Assuming 'rules_2021_shop1_nonredundant_unfiltered' contains a 'conviction' column
for _, row in rules_2021_shop1_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop1_conviction.add_edge(antecedents, consequents, weight=row['confidence'], conviction=row['conviction'])
    
    # Aggregate conviction values and count occurrences
    if consequents in conviction_summary_2021_shop1:
        conviction_summary_2021_shop1[consequents] += row['conviction']
        count_summary_2021_shop1[consequents] += 1
    else:
        conviction_summary_2021_shop1[consequents] = row['conviction']
        count_summary_2021_shop1[consequents] = 1

# Calculate average conviction values
average_conviction_summary_2021_shop1 = {k: conviction_summary_2021_shop1[k] / count_summary_2021_shop1[k] for k in conviction_summary_2021_shop1}

# Normalize conviction values for sizing and coloring
max_conviction = max(average_conviction_summary_2021_shop1.values())
min_conviction = min(average_conviction_summary_2021_shop1.values())
conviction_range = max_conviction - min_conviction
node_colors = []
node_sizes = []

for node in G_2021_shop1_conviction.nodes():
    # Get the average conviction value for the node, defaulting to min if not present
    conviction = average_conviction_summary_2021_shop1.get(node, min_conviction)
    # Normalize between 0 and 1
    normalized_conviction = (conviction - min_conviction) / conviction_range if conviction_range else 0
    node_colors.append(normalized_conviction)
    node_sizes.append(normalized_conviction * 1000 + 100)  # Scale and offset size

# Use a colormap for coloring nodes based on conviction
cmap = plt.cm.coolwarm

pos = nx.spring_layout(G_2021_shop1_conviction)

plt.figure(figsize=(40, 22))
nodes = nx.draw_networkx_nodes(G_2021_shop1_conviction, pos, node_color=node_colors, node_size=node_sizes, cmap=cmap, alpha=0.8)
edges = nx.draw_networkx_edges(G_2021_shop1_conviction, pos, arrowstyle='->', arrowsize=10, edge_color='grey', alpha=0.5)
labels = nx.draw_networkx_labels(G_2021_shop1_conviction, pos, font_size=8)

plt.colorbar(nodes, label='Normalized Conviction')
plt.title('Network Graph of 2021 Shop 1 Based on Conviction')
plt.axis('off')
plt.show()

In [ ]:
print(max_conviction)

# Filtered top 30 and non redundant

In [ ]:
rules_2021_shop1_nonredundant.head()

In [ ]:
# Assuming rules_1992_shop2_nonredundant is your sorted rules DataFrame for shop2
rules_2021_shop1_nonredundant_top30 = rules_2021_shop1_nonredundant.copy(deep=True)

In [ ]:
rules_2021_shop1_nonredundant_top30_lift = rules_2021_shop1_nonredundant.copy(deep=True)

In [ ]:
# Select the top 30 rules based on lift or another criterion
rules_2021_shop1_nonredundant_top30_lift = rules_2021_shop1_nonredundant_top30_lift.sort_values(by='lift', ascending=False).head(30)

In [ ]:
rules_2021_shop1_nonredundant_top30_lift.head()

In [ ]:
# Select the top 30 rules based on confidence or another criterion
rules_2021_shop1_nonredundant_top30 = rules_2021_shop1_nonredundant_top30.sort_values(by='confidence', ascending=False).head(30)

In [ ]:
rules_2021_shop1_nonredundant_top30.info()

In [ ]:
rules_2021_shop1_nonredundant_top30['LHS'] = [','.join(list(x)) for x in rules_2021_shop1_nonredundant_top30['antecedents']]
rules_2021_shop1_nonredundant_top30['RHS'] = [','.join(list(x)) for x in rules_2021_shop1_nonredundant_top30['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2021_shop1_nonredundant_top30.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2021_shop1_nonredundant_top30, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2020 Shop 2")
plt.axis('off')
plt.show()


## Based on Indegree

In [ ]:
G_2021_shop1 = nx.DiGraph()

# Assuming 'rules_sorted_2021_shop1' contains your association rules for shop 1 in 2021
for _, row in rules_2021_shop1_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as the weight
    G_2021_shop1.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2021_shop1 = nx.spring_layout(G_2021_shop1)

# Calculate node sizes based on indegree
in_degrees_2021_shop1 = dict(G_2021_shop1.in_degree())
node_sizes_2021_shop1 = [in_degrees_2021_shop1[node] * 100 + 1000 for node in G_2021_shop1.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2021_shop1 = (np.array(node_sizes_2021_shop1) - np.min(node_sizes_2021_shop1)) / (np.max(node_sizes_2021_shop1) - np.min(node_sizes_2021_shop1))

# Define a colormap from orange to red
cmap_2021_shop1 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2021_shop1 = [cmap_2021_shop1(size) for size in normalized_sizes_2021_shop1]

# Drawing the network
plt.figure(figsize=(40, 34))
nx.draw_networkx_nodes(G_2021_shop1, pos_2021_shop1, node_size=node_sizes_2021_shop1, node_color=node_colors_2021_shop1, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop1, pos_2021_shop1, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2021_shop1, pos_2021_shop1, font_size=8)

plt.title('Top 5 Association Rules Network for 2021 Shop 1 with Color Gradient')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2021_shop1_lift = nx.DiGraph()

lift_summary_2021_shop1 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2021_shop1_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop1_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2021_shop1:
        lift_summary_2021_shop1[consequents]['total_lift'] += row['lift']
        lift_summary_2021_shop1[consequents]['count'] += 1
    else:
        lift_summary_2021_shop1[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2021_shop1:
    lift_summary_2021_shop1[item]['avg_lift'] = lift_summary_2021_shop1[item]['total_lift'] / lift_summary_2021_shop1[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2021_shop1_lift = [lift_summary_2021_shop1[node]['avg_lift'] * 500 if node in lift_summary_2021_shop1 else 1000 for node in G_2021_shop1_lift.nodes()]

pos_2021_shop1_lift = nx.spring_layout(G_2021_shop1_lift)

# Colormap from orange to red
cmap_2021_shop1_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2021_shop1_lift = np.array(node_sizes_2021_shop1_lift)
normalized_sizes_2021_shop1_lift = (normalized_sizes_2021_shop1_lift - min(normalized_sizes_2021_shop1_lift)) / (max(normalized_sizes_2021_shop1_lift) - min(normalized_sizes_2021_shop1_lift))
node_colors_2021_shop1_lift = [cmap_2021_shop1_lift(size) for size in normalized_sizes_2021_shop1_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(40, 22))
nx.draw_networkx_nodes(G_2021_shop1_lift, pos_2021_shop1_lift, node_size=node_sizes_2021_shop1_lift, node_color=node_colors_2021_shop1_lift, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop1_lift, pos_2021_shop1_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2021_shop1_lift, pos_2021_shop1_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2021 Shop 1 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2021_shop1_lift = nx.DiGraph()

lift_summary_2021_shop1 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2021_shop1_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop1_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2021_shop1:
        lift_summary_2021_shop1[consequents]['total_lift'] += row['lift']
        lift_summary_2021_shop1[consequents]['count'] += 1
    else:
        lift_summary_2021_shop1[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2021_shop1:
    lift_summary_2021_shop1[item]['avg_lift'] = lift_summary_2021_shop1[item]['total_lift'] / lift_summary_2021_shop1[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2021_shop1_lift = [lift_summary_2021_shop1[node]['avg_lift'] * 500 if node in lift_summary_2021_shop1 else 1000 for node in G_2021_shop1_lift.nodes()]

pos_2021_shop1_lift = nx.spring_layout(G_2021_shop1_lift)

# Colormap from orange to red
cmap_2021_shop1_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2021_shop1_lift = np.array(node_sizes_2021_shop1_lift)
normalized_sizes_2021_shop1_lift = (normalized_sizes_2021_shop1_lift - min(normalized_sizes_2021_shop1_lift)) / (max(normalized_sizes_2021_shop1_lift) - min(normalized_sizes_2021_shop1_lift))
node_colors_2021_shop1_lift = [cmap_2021_shop1_lift(size) for size in normalized_sizes_2021_shop1_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2021_shop1_lift, pos_2021_shop1_lift, node_size=node_sizes_2021_shop1_lift, node_color=node_colors_2021_shop1_lift, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop1_lift, pos_2021_shop1_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2021_shop1_lift, pos_2021_shop1_lift, font_size=10)

plt.title('Top 5 Association Rules Network for 2021 Shop 1 Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2021_shop1 = nx.DiGraph()

# Assuming 'rules_2021_shop1_nonredundant' contains association rules for 2021 Shop 1
for _, row in rules_2021_shop1_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop1.add_edge(antecedents, consequents, confidence=row['confidence'])

# Calculate node sizes based on indegree
in_degrees_2021_shop1 = dict(G_2021_shop1.in_degree())
node_sizes_2021_shop1 = [in_degrees_2021_shop1[node] * 100 + 1000 for node in G_2021_shop1.nodes()]

# Edge width based on confidence
edge_widths = [G_2021_shop1[u][v]['confidence'] * 15 for u, v in G_2021_shop1.edges()]

# Assuming normalized sizes and color mapping is desired
normalized_sizes_2021_shop1 = np.array(list(in_degrees_2021_shop1.values()))
normalized_sizes_2021_shop1 = (normalized_sizes_2021_shop1 - min(normalized_sizes_2021_shop1)) / (max(normalized_sizes_2021_shop1) - min(normalized_sizes_2021_shop1))
cmap_2021_shop1 = mcolors.LinearSegmentedColormap.from_list("MyCMapName", ["blue", "red"])

# Node colors based on a property (here, using indegree for example)
node_colors_2021_shop1 = [cmap_2021_shop1(size) for size in normalized_sizes_2021_shop1]

# Layout
pos = nx.spring_layout(G_2021_shop1, seed=42)

# Drawing
plt.figure(figsize=(40, 30))
nx.draw_networkx_edges(G_2021_shop1, pos, width=edge_widths, edge_color='grey', alpha=0.5)
nx.draw_networkx_nodes(G_2021_shop1, pos, node_size=node_sizes_2021_shop1, node_color=node_colors_2021_shop1, alpha=0.8)
nx.draw_networkx_labels(G_2021_shop1, pos, font_size=8, font_color='black')
plt.title('Network Graph of Association Rules Based on Confidence for 2021 Shop 1')
plt.axis('off')
plt.show()


In [ ]:
G_2021_shop1 = nx.DiGraph()

convictions = []  # To store conviction values for normalization

for _, row in rules_2021_shop1_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    conviction = row['conviction']  # Assuming each row has a 'conviction' column
    convictions.append(conviction)
    G_2021_shop1.add_edge(antecedents, consequents, weight=row['confidence'], conviction=conviction)

# Normalize conviction values for edge width
max_conviction = max(convictions)
min_conviction = min(convictions)
conviction_range = max_conviction - min_conviction
edge_widths = [(G_2021_shop1[u][v]['conviction'] - min_conviction) / conviction_range * 30 + 0.5 for u, v in G_2021_shop1.edges()]  # Scale and offset widths

pos_2021_shop1 = nx.spring_layout(G_2021_shop1)

# Node sizes and colors as before
in_degrees_2021_shop1 = dict(G_2021_shop1.in_degree())
node_sizes_2021_shop1 = [in_degrees_2021_shop1[node] * 100 + 1000 for node in G_2021_shop1.nodes()]
normalized_sizes_2021_shop1 = (np.array(node_sizes_2021_shop1) - min(node_sizes_2021_shop1)) / (max(node_sizes_2021_shop1) - min(node_sizes_2021_shop1))
cmap_2021_shop1 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2021_shop1 = [cmap_2021_shop1(size) for size in normalized_sizes_2021_shop1]

# Drawing the network with conviction-based edge widths
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2021_shop1, pos_2021_shop1, node_size=node_sizes_2021_shop1, node_color=node_colors_2021_shop1, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop1, pos_2021_shop1, width=edge_widths, edge_color='grey', alpha=0.5)
nx.draw_networkx_labels(G_2021_shop1, pos_2021_shop1, font_size=10)

plt.title('Top 5 Association Rules Network for 2021 Shop 1 with Conviction-based Edge Thickness')
plt.axis('off')
plt.show()


In [ ]:
#the node sizes are based on the indegree where each node acts as a consequent,

In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2021_shop1_nonredundant_top30.loc[:, 'antecedents_'] = rules_2021_shop1_nonredundant_top30['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2021_shop1_nonredundant_top30.loc[:, 'consequents_'] = rules_2021_shop1_nonredundant_top30['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2021_shop1_nonredundant_top30, x='consequents_', y='antecedents_', s=rules_2021_shop1_nonredundant_top30['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2021_shop1_nonredundant_top30.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2021 Shop 1", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
print(rules_2021_shop1_nonredundant_top30[['antecedents', 'consequents']].head(10))

## Monetary Analysis of the Top Rules

In [ ]:
# Step 1: Extract unique product codes from the rules
unique_product_codes = set()
for idx, row in rules_2021_shop1_nonredundant_top30.iterrows():
    # Unite antecedents and consequents into a single set
    all_products = row['antecedents'].union(row['consequents'])
    unique_product_codes.update(all_products)

# Step 2: Create a DataFrame with unique product codes and their unit prices
# Filter sales_data for these product codes and drop duplicates to ensure each product code appears once
product_prices = sales_data[sales_data['product_code'].isin(unique_product_codes)][['product_code', 'unit_price']].drop_duplicates()

# Create a dictionary for quick lookup of prices
price_dict = product_prices.set_index('product_code')['unit_price'].to_dict()

# Step 3: Calculate the total price for each rule and collect data for the new DataFrame
data_for_new_df = []
for idx, row in rules_2021_shop1_nonredundant_top30.iterrows():
    # Combine sets of antecedents and consequents
    all_products = row['antecedents'].union(row['consequents'])
    total_price = sum(price_dict.get(prod, 0) for prod in all_products)  # Use get to avoid KeyError if any product code is missing
    data_for_new_df.append({
        'antecedents': row['antecedents'],
        'consequents': row['consequents'],
        'total_unit_price': total_price
    })

# Create a new DataFrame with only antecedents, consequents, and total unit price
rules_and_price_shop1_2021 = pd.DataFrame(data_for_new_df)

# Sort the DataFrame by total_unit_price in descending order
rules_and_price_shop1_2021_sorted = rules_and_price_shop1_2021.sort_values(by='total_unit_price', ascending=False)

# Print the sorted DataFrame to see the specified columns along with their total unit prices
print(rules_and_price_shop1_2021_sorted)


In [ ]:
# Convert 'trx_date' to datetime format if not already done so
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Add a 'year' column if it doesn't exist
sales_data['year'] = sales_data['trx_date'].dt.year

# Step 2: Filter sales_data for shop1 and the year 2021
filtered_sales_data_2021 = sales_data[(sales_data['shopID'] == 1) & (sales_data['year'] == 2021)]

# Step 3: Initialize a list to hold the rule data including total sales value
rule_sales_summaries_2021 = []

# Step 4: Iterate through each rule in the DataFrame
for index, rule in rules_2021_shop1_nonredundant_top30.iterrows():
    all_products = rule['antecedents'].union(rule['consequents'])

    # Filter the sales data for transactions that include all products in the rule
    # This step ensures that we are only considering transactions that contain all specified products
    mask = filtered_sales_data_2021['product_code'].isin(all_products)
    transaction_ids = filtered_sales_data_2021[mask].groupby('trx_id').filter(lambda x: all_products.issubset(set(x['product_code']))).trx_id.unique()
    matched_transactions = filtered_sales_data_2021[filtered_sales_data_2021['trx_id'].isin(transaction_ids)]
    
    # Step 5: Sum the total_value of these transactions
    total_sales_value = matched_transactions['total_value'].sum()

    # Append the result to the list
    rule_sales_summaries_2021.append({
        'antecedents': rule['antecedents'],
        'consequents': rule['consequents'],
        'total_sales_value': total_sales_value
    })

# Step 6: Convert the list to a DataFrame
rules_sales_values_df_2021 = pd.DataFrame(rule_sales_summaries_2021)

# Step 7: Sort by total_sales_value in descending order
sorted_rules_sales_values_df_2021 = rules_sales_values_df_2021.sort_values(by='total_sales_value', ascending=False)

# Step 8: Print the sorted DataFrame
print(sorted_rules_sales_values_df_2021)


## Narrowing down based on Lift

In [ ]:
rules_2021_shop1_nonredundant_top30_lift['LHS'] = [','.join(list(x)) for x in rules_2021_shop1_nonredundant_top30_lift['antecedents']]
rules_2021_shop1_nonredundant_top30_lift['RHS'] = [','.join(list(x)) for x in rules_2021_shop1_nonredundant_top30_lift['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2021_shop1_nonredundant_top30_lift.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2021_shop1_nonredundant_top30_lift, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2021 Shop 2")
plt.axis('off')
plt.show()


In [ ]:
G_2021_shop1_lift = nx.DiGraph()

lift_summary_2021_shop1 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2021_shop1_nonredundant_top30_lift.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop1_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2021_shop1:
        lift_summary_2021_shop1[consequents]['total_lift'] += row['lift']
        lift_summary_2021_shop1[consequents]['count'] += 1
    else:
        lift_summary_2021_shop1[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2021_shop1:
    lift_summary_2021_shop1[item]['avg_lift'] = lift_summary_2021_shop1[item]['total_lift'] / lift_summary_2021_shop1[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2021_shop1_lift = [lift_summary_2021_shop1[node]['avg_lift'] * 500 if node in lift_summary_2021_shop1 else 1000 for node in G_2021_shop1_lift.nodes()]

pos_2021_shop1_lift = nx.spring_layout(G_2021_shop1_lift)

# Colormap from orange to red
cmap_2021_shop1_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2021_shop1_lift = np.array(node_sizes_2021_shop1_lift)
normalized_sizes_2021_shop1_lift = (normalized_sizes_2021_shop1_lift - min(normalized_sizes_2021_shop1_lift)) / (max(normalized_sizes_2021_shop1_lift) - min(normalized_sizes_2021_shop1_lift))
node_colors_2021_shop1_lift = [cmap_2021_shop1_lift(size) for size in normalized_sizes_2021_shop1_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 22))
nx.draw_networkx_nodes(G_2021_shop1_lift, pos_2021_shop1_lift, node_size=node_sizes_2021_shop1_lift, node_color=node_colors_2021_shop1_lift, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop1_lift, pos_2021_shop1_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2021_shop1_lift, pos_2021_shop1_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2021 Shop 1 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2021_shop1_nonredundant_top30_lift.loc[:, 'antecedents_'] = rules_2021_shop1_nonredundant_top30_lift['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2021_shop1_nonredundant_top30_lift.loc[:, 'consequents_'] = rules_2021_shop1_nonredundant_top30_lift['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2021_shop1_nonredundant_top30_lift, x='consequents_', y='antecedents_', s=rules_2021_shop1_nonredundant_top30_lift['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2021_shop1_nonredundant_top30_lift.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2021 Shop 1", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
# Print only the 'antecedents' and 'consequents' columns of the top 10 rules
print(rules_2021_shop1_nonredundant_top30_lift[['antecedents', 'consequents']].head(10))


# Shop1 2022 SEARCH 3

In [ ]:
# Filter for records from shop ID 1 and the year 1993
sales_data_2022_shop1 = sales_data_shop1[(sales_data_shop1['trx_date'].dt.year == 2022)]

In [ ]:
sales_data_2022_shop1.head()

In [ ]:
# Grouping product codes by trx_id
transactions_2022_shop1 = sales_data_2022_shop1.groupby('trx_id')['product_code'].apply(list).tolist()

In [ ]:
# Encode transaction data
encoder_2022_shop1 = TransactionEncoder()
trans_array_2022_shop1 = encoder_2022_shop1.fit(transactions_2022_shop1).transform(transactions_2022_shop1)
trans_df_2022_shop1 = pd.DataFrame(trans_array_2022_shop1, columns=encoder_2022_shop1.columns_)

In [ ]:
# Find frequent itemsets
frequent_itemsets_2022_shop1 = apriori(trans_df_2022_shop1, min_support=0.001, use_colnames=True)

In [ ]:
print(frequent_itemsets_2022_shop1)

In [ ]:
## Generate the association rules - by confidence
rulesConfidence_2022_shop1 = association_rules(frequent_itemsets_2022_shop1, metric="confidence", min_threshold=0.10)
rulesConfidence_2022_shop1.sort_values(by='confidence', ascending=False, inplace=True)


In [ ]:
# Note: This is a hypothetical example, as I can't directly manipulate or access dataframes not provided within this code cell.

# Replace frozen sets with strings
rulesConfidence_2022_shop1['antecedents_'] = rulesConfidence_2022_shop1['antecedents'].apply(lambda a: ','.join(list(a)))
rulesConfidence_2022_shop1['consequents_'] = rulesConfidence_2022_shop1['consequents'].apply(lambda a: ','.join(list(a)))

# Draw
fig, ax = plt.subplots(figsize=(14,30))
ax = plt.scatter(data=rulesConfidence_2022_shop1, x='consequents_', y='antecedents_', s=rulesConfidence_2022_shop1['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rulesConfidence_2022_shop1.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2020 Shop 1", fontsize=14)
plt.xlabel("Consequents (RHS)", fontsize=12)
plt.ylabel("Antecedents (LHS)", fontsize=12)
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate(rotation=45)
plt.tight_layout()

plt.show()



## Eliminate Redundant Rules

In [ ]:
rulesConfidence_2022_shop1.head()

In [ ]:
rulesConfidence_2022_shop1.info()

In [ ]:
# If 'antecedents' and 'consequents' are not frozensets, convert them
rules_2022_shop1_redundant = rulesConfidence_2022_shop1.copy()
rules_2022_shop1_redundant['antecedents'] = rules_2022_shop1_redundant['antecedents'].apply(lambda x: frozenset(x))
rules_2022_shop1_redundant['consequents'] = rules_2022_shop1_redundant['consequents'].apply(lambda x: frozenset(x))


In [ ]:
# Create a copy of the DataFrame to avoid modifying the original while iterating
rules_2022_shop1_checkredundance = rules_2022_shop1_redundant.copy()

# List to hold non-redundant rules
non_redundant_rules_2022_shop1 = []

for _, rule in rules_2022_shop1_checkredundance.iterrows():
    if not is_redundant(rule, rules_2022_shop1_checkredundance):
        non_redundant_rules_2022_shop1.append(rule)

# Convert the list back to a DataFrame
rules_2022_shop1_nonredundant = pd.DataFrame(non_redundant_rules_2022_shop1).reset_index(drop=True)


In [ ]:
print(rules_2022_shop1_nonredundant.head())
print(f"Original number of rules: {len(rules_2022_shop1_redundant)}")
print(f"Number of non-redundant rules: {len(rules_2022_shop1_nonredundant)}")


In [ ]:
rules_2022_shop1_nonredundant.head()

In [ ]:
rules_2022_shop1_redundant.info()

In [ ]:
rules_2022_shop1_nonredundant.head()

In [ ]:
rules_2022_shop1_nonredundant_backup = rules_2022_shop1_nonredundant.copy()

In [ ]:
rules_2022_shop1_nonredundant_unfiltered = rules_2022_shop1_nonredundant.copy()

In [ ]:
#reset
#rules_2022_shop1_nonredundant = rules_2022_shop1_nonredundant_backup.copy()

# Redundant and Unfiltered

In [ ]:
rules_2022_shop1_redundant.head()

In [ ]:
G_2022_shop1 = nx.DiGraph()

# Iterate through rules for 2022 Shop 1 and add edges to the graph
for _, row in rules_2022_shop1_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as weight
    G_2022_shop1.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2022_shop1 = nx.spring_layout(G_2022_shop1)

# Recalculate node sizes based on indegree or another metric
in_degrees_2022_shop1 = dict(G_2022_shop1.in_degree())
node_sizes_2022_shop1 = [in_degrees_2022_shop1[node] * 100 + 1000 for node in G_2022_shop1.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2022_shop1 = (np.array(node_sizes_2022_shop1) - np.min(node_sizes_2022_shop1)) / (np.max(node_sizes_2022_shop1) - np.min(node_sizes_2022_shop1))

# Define a colormap from orange to red for visual consistency
cmap_2022_shop1 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2022_shop1 = [cmap_2022_shop1(size) for size in normalized_sizes_2022_shop1]

# Drawing the network
plt.figure(figsize=(60, 43))
nx.draw_networkx_nodes(G_2022_shop1, pos_2022_shop1, node_size=node_sizes_2022_shop1, node_color=node_colors_2022_shop1, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop1, pos_2022_shop1, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop1, pos_2022_shop1, font_size=10)

plt.title('Top 5 Association Rules Network for 2022 Shop 1 with Color Gradient')
plt.axis('off')
plt.show()


### Based on LIFT

In [ ]:
G_2022_shop1_lift = nx.DiGraph()

lift_summary_2022_shop1 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2022_shop1_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop1_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop1:
        lift_summary_2022_shop1[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop1[consequents]['count'] += 1
    else:
        lift_summary_2022_shop1[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop1:
    lift_summary_2022_shop1[item]['avg_lift'] = lift_summary_2022_shop1[item]['total_lift'] / lift_summary_2022_shop1[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop1_lift = [lift_summary_2022_shop1[node]['avg_lift'] * 500 if node in lift_summary_2022_shop1 else 1000 for node in G_2022_shop1_lift.nodes()]

pos_2022_shop1_lift = nx.spring_layout(G_2022_shop1_lift)

# Colormap from orange to red
cmap_2022_shop1_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop1_lift = np.array(node_sizes_2022_shop1_lift)
normalized_sizes_2022_shop1_lift = (normalized_sizes_2022_shop1_lift - min(normalized_sizes_2022_shop1_lift)) / (max(normalized_sizes_2022_shop1_lift) - min(normalized_sizes_2022_shop1_lift))
node_colors_2022_shop1_lift = [cmap_2022_shop1_lift(size) for size in normalized_sizes_2022_shop1_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(60, 52))
nx.draw_networkx_nodes(G_2022_shop1_lift, pos_2022_shop1_lift, node_size=node_sizes_2022_shop1_lift, node_color=node_colors_2022_shop1_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop1_lift, pos_2022_shop1_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop1_lift, pos_2022_shop1_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2022 Shop 1 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


# nonredundant_unfiltered

## Based on Indegree

In [ ]:
G_2022_shop1_nonredundant_unfiltered = nx.DiGraph()

# Iterating through each row to add edges with confidence as an attribute
for _, row in rules_2022_shop1_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop1_nonredundant_unfiltered.add_edge(antecedents, consequents, weight=row['confidence'])

# Defining the layout for network visualization
pos_2022_shop1_nonredundant_unfiltered = nx.spring_layout(G_2022_shop1_nonredundant_unfiltered)

# Calculating node sizes based on the indegree for color mapping
in_degrees_2022_shop1_nonredundant_unfiltered = dict(G_2022_shop1_nonredundant_unfiltered.in_degree())
node_sizes_2022_shop1_nonredundant_unfiltered = [in_degrees_2022_shop1_nonredundant_unfiltered[node] * 100 + 1000 for node in G_2022_shop1_nonredundant_unfiltered.nodes()]

# Normalizing node sizes for color mapping
normalized_sizes_2022_shop1_nonredundant_unfiltered = (np.array(node_sizes_2022_shop1_nonredundant_unfiltered) - min(node_sizes_2022_shop1_nonredundant_unfiltered)) / (max(node_sizes_2022_shop1_nonredundant_unfiltered) - min(node_sizes_2022_shop1_nonredundant_unfiltered))

# Defining a color map from orange to red
cmap_2022_shop1_nonredundant_unfiltered = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])

# Mapping normalized node sizes to colors
node_colors_2022_shop1_nonredundant_unfiltered = [cmap_2022_shop1_nonredundant_unfiltered(size) for size in normalized_sizes_2022_shop1_nonredundant_unfiltered]

# Drawing the network with node sizes and colors
plt.figure(figsize=(50, 34))
nx.draw_networkx_nodes(G_2022_shop1_nonredundant_unfiltered, pos_2022_shop1_nonredundant_unfiltered, node_size=node_sizes_2022_shop1_nonredundant_unfiltered, node_color=node_colors_2022_shop1_nonredundant_unfiltered, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop1_nonredundant_unfiltered, pos_2022_shop1_nonredundant_unfiltered, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2022_shop1_nonredundant_unfiltered, pos_2022_shop1_nonredundant_unfiltered, font_size=10)

plt.title('Node Size Based on Indegree for 2022 Shop 1')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2022_shop1_lift = nx.DiGraph()

lift_summary_2022_shop1 = {}

for _, row in rules_2022_shop1_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop1_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Update lift summary for consequents
    if consequents in lift_summary_2022_shop1:
        lift_summary_2022_shop1[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop1[consequents]['count'] += 1
    else:
        lift_summary_2022_shop1[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculate average lift for each consequent
for item in lift_summary_2022_shop1:
    lift_summary_2022_shop1[item]['avg_lift'] = lift_summary_2022_shop1[item]['total_lift'] / lift_summary_2022_shop1[item]['count']

# Node sizes based on average lift, scaled appropriately
node_sizes_2022_shop1_lift = [lift_summary_2022_shop1[node]['avg_lift'] * 500 if node in lift_summary_2022_shop1 else 1000 for node in G_2022_shop1_lift.nodes()]

pos_2022_shop1_lift = nx.spring_layout(G_2022_shop1_lift)

# Use a colormap that transitions from orange to red
cmap_2022_shop1_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop1_lift = np.array(node_sizes_2022_shop1_lift)
normalized_sizes_2022_shop1_lift = (normalized_sizes_2022_shop1_lift - min(normalized_sizes_2022_shop1_lift)) / (max(normalized_sizes_2022_shop1_lift) - min(normalized_sizes_2022_shop1_lift))
node_colors_2022_shop1_lift = [cmap_2022_shop1_lift(size) for size in normalized_sizes_2022_shop1_lift]

plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2022_shop1_lift, pos_2022_shop1_lift, node_size=node_sizes_2022_shop1_lift, node_color=node_colors_2022_shop1_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop1_lift, pos_2022_shop1_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop1_lift, pos_2022_shop1_lift, font_size=10)

plt.title('Node Size Based on Average Lift for 2022 Shop 1')
plt.axis('off')
plt.show()


# Based on conviction

In [ ]:
rules_2022_shop1_nonredundant.head()

In [ ]:
G_2022_shop1_conviction = nx.DiGraph()

conviction_summary_2022_shop1 = {}
count_summary_2022_shop1 = {}

# Assuming 'rules_2022_shop1_nonredundant_unfiltered' contains a 'conviction' column
for _, row in rules_2022_shop1_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop1_conviction.add_edge(antecedents, consequents, weight=row['confidence'], conviction=row['conviction'])
    
    # Aggregate conviction values and count occurrences
    if consequents in conviction_summary_2022_shop1:
        conviction_summary_2022_shop1[consequents] += row['conviction']
        count_summary_2022_shop1[consequents] += 1
    else:
        conviction_summary_2022_shop1[consequents] = row['conviction']
        count_summary_2022_shop1[consequents] = 1

# Calculate average conviction values
average_conviction_summary_2022_shop1 = {k: conviction_summary_2022_shop1[k] / count_summary_2022_shop1[k] for k in conviction_summary_2022_shop1}

# Normalize conviction values for sizing and coloring
max_conviction = max(average_conviction_summary_2022_shop1.values())
min_conviction = min(average_conviction_summary_2022_shop1.values())
conviction_range = max_conviction - min_conviction
node_colors = []
node_sizes = []

for node in G_2022_shop1_conviction.nodes():
    # Get the average conviction value for the node, defaulting to min if not present
    conviction = average_conviction_summary_2022_shop1.get(node, min_conviction)
    # Normalize between 0 and 1
    normalized_conviction = (conviction - min_conviction) / conviction_range if conviction_range else 0
    node_colors.append(normalized_conviction)
    node_sizes.append(normalized_conviction * 1000 + 100)  # Scale and offset size

# Use a colormap for coloring nodes based on conviction
cmap = plt.cm.coolwarm

pos = nx.spring_layout(G_2022_shop1_conviction)

plt.figure(figsize=(40, 22))
nodes = nx.draw_networkx_nodes(G_2022_shop1_conviction, pos, node_color=node_colors, node_size=node_sizes, cmap=cmap, alpha=0.8)
edges = nx.draw_networkx_edges(G_2022_shop1_conviction, pos, arrowstyle='->', arrowsize=10, edge_color='grey', alpha=0.5)
labels = nx.draw_networkx_labels(G_2022_shop1_conviction, pos, font_size=8)

plt.colorbar(nodes, label='Normalized Conviction')
plt.title('Network Graph of 2022 Shop 1 Based on Conviction')
plt.axis('off')
plt.show()

In [ ]:
print(max_conviction)

# Filtered top 30 and non redundant

In [ ]:
rules_2022_shop1_nonredundant.head()

In [ ]:
# Assuming rules_1992_shop2_nonredundant is your sorted rules DataFrame for shop2
rules_2022_shop1_nonredundant_top30 = rules_2022_shop1_nonredundant.copy(deep=True)

In [ ]:
rules_2022_shop1_nonredundant_top30_lift = rules_2022_shop1_nonredundant.copy(deep=True)

In [ ]:
# Select the top 30 rules based on lift or another criterion
rules_2022_shop1_nonredundant_top30_lift = rules_2022_shop1_nonredundant_top30_lift.sort_values(by='lift', ascending=False).head(30)

In [ ]:
rules_2022_shop1_nonredundant_top30_lift.head()

In [ ]:
# Select the top 30 rules based on confidence or another criterion
rules_2022_shop1_nonredundant_top30 = rules_2022_shop1_nonredundant_top30.sort_values(by='confidence', ascending=False).head(30)

In [ ]:
rules_2022_shop1_nonredundant_top30.info()

In [ ]:
rules_2022_shop1_nonredundant_top30['LHS'] = [','.join(list(x)) for x in rules_2022_shop1_nonredundant_top30['antecedents']]
rules_2022_shop1_nonredundant_top30['RHS'] = [','.join(list(x)) for x in rules_2022_shop1_nonredundant_top30['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2022_shop1_nonredundant_top30.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2022_shop1_nonredundant_top30, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2022 Shop 1")
plt.axis('off')
plt.show()


## Based on Indegree

In [ ]:
G_2022_shop1 = nx.DiGraph()

# Assuming 'rules_sorted_2022_shop1' contains your association rules for shop 1 in 2022
for _, row in rules_2022_shop1_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as the weight
    G_2022_shop1.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2022_shop1 = nx.spring_layout(G_2022_shop1)

# Calculate node sizes based on indegree
in_degrees_2022_shop1 = dict(G_2022_shop1.in_degree())
node_sizes_2022_shop1 = [in_degrees_2022_shop1[node] * 100 + 1000 for node in G_2022_shop1.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2022_shop1 = (np.array(node_sizes_2022_shop1) - np.min(node_sizes_2022_shop1)) / (np.max(node_sizes_2022_shop1) - np.min(node_sizes_2022_shop1))

# Define a colormap from orange to red
cmap_2022_shop1 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2022_shop1 = [cmap_2022_shop1(size) for size in normalized_sizes_2022_shop1]

# Drawing the network
plt.figure(figsize=(30, 14))
nx.draw_networkx_nodes(G_2022_shop1, pos_2022_shop1, node_size=node_sizes_2022_shop1, node_color=node_colors_2022_shop1, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop1, pos_2022_shop1, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2022_shop1, pos_2022_shop1, font_size=8)

plt.title('Top 5 Association Rules Network for 2022 Shop 1 with Color Gradient')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2022_shop1_lift = nx.DiGraph()

lift_summary_2022_shop1 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2022_shop1_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop1_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop1:
        lift_summary_2022_shop1[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop1[consequents]['count'] += 1
    else:
        lift_summary_2022_shop1[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop1:
    lift_summary_2022_shop1[item]['avg_lift'] = lift_summary_2022_shop1[item]['total_lift'] / lift_summary_2022_shop1[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop1_lift = [lift_summary_2022_shop1[node]['avg_lift'] * 500 if node in lift_summary_2022_shop1 else 1000 for node in G_2022_shop1_lift.nodes()]

pos_2022_shop1_lift = nx.spring_layout(G_2022_shop1_lift)

# Colormap from orange to red
cmap_2022_shop1_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop1_lift = np.array(node_sizes_2022_shop1_lift)
normalized_sizes_2022_shop1_lift = (normalized_sizes_2022_shop1_lift - min(normalized_sizes_2022_shop1_lift)) / (max(normalized_sizes_2022_shop1_lift) - min(normalized_sizes_2022_shop1_lift))
node_colors_2022_shop1_lift = [cmap_2022_shop1_lift(size) for size in normalized_sizes_2022_shop1_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 32))
nx.draw_networkx_nodes(G_2022_shop1_lift, pos_2022_shop1_lift, node_size=node_sizes_2022_shop1_lift, node_color=node_colors_2022_shop1_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop1_lift, pos_2022_shop1_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop1_lift, pos_2022_shop1_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2022 Shop 1 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop1_lift = nx.DiGraph()

lift_summary_2022_shop1 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2022_shop1_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop1_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop1:
        lift_summary_2022_shop1[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop1[consequents]['count'] += 1
    else:
        lift_summary_2022_shop1[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop1:
    lift_summary_2022_shop1[item]['avg_lift'] = lift_summary_2022_shop1[item]['total_lift'] / lift_summary_2022_shop1[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop1_lift = [lift_summary_2022_shop1[node]['avg_lift'] * 500 if node in lift_summary_2022_shop1 else 1000 for node in G_2022_shop1_lift.nodes()]

pos_2022_shop1_lift = nx.spring_layout(G_2022_shop1_lift)

# Colormap from orange to red
cmap_2022_shop1_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop1_lift = np.array(node_sizes_2022_shop1_lift)
normalized_sizes_2022_shop1_lift = (normalized_sizes_2022_shop1_lift - min(normalized_sizes_2022_shop1_lift)) / (max(normalized_sizes_2022_shop1_lift) - min(normalized_sizes_2022_shop1_lift))
node_colors_2022_shop1_lift = [cmap_2022_shop1_lift(size) for size in normalized_sizes_2022_shop1_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2022_shop1_lift, pos_2022_shop1_lift, node_size=node_sizes_2022_shop1_lift, node_color=node_colors_2022_shop1_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop1_lift, pos_2022_shop1_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop1_lift, pos_2022_shop1_lift, font_size=10)

plt.title('Top 5 Association Rules Network for 2022 Shop 1 Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop1 = nx.DiGraph()

# Assuming 'rules_2022_shop1_nonredundant' contains association rules for 2022 Shop 1
for _, row in rules_2022_shop1_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop1.add_edge(antecedents, consequents, confidence=row['confidence'])

# Calculate node sizes based on indegree
in_degrees_2022_shop1 = dict(G_2022_shop1.in_degree())
node_sizes_2022_shop1 = [in_degrees_2022_shop1[node] * 100 + 1000 for node in G_2022_shop1.nodes()]

# Edge width based on confidence
edge_widths_2022_shop1 = [G_2022_shop1[u][v]['confidence'] * 15 for u, v in G_2022_shop1.edges()]

# Assuming normalized sizes and color mapping is desired
normalized_sizes_2022_shop1 = np.array(list(in_degrees_2022_shop1.values()))
normalized_sizes_2022_shop1 = (normalized_sizes_2022_shop1 - min(normalized_sizes_2022_shop1)) / (max(normalized_sizes_2022_shop1) - min(normalized_sizes_2022_shop1))
cmap_2022_shop1 = mcolors.LinearSegmentedColormap.from_list("MyCMapName", ["blue", "red"])

# Node colors based on a property (here, using indegree for example)
node_colors_2022_shop1 = [cmap_2022_shop1(size) for size in normalized_sizes_2022_shop1]

# Layout
pos_2022_shop1 = nx.spring_layout(G_2022_shop1, seed=42)

# Drawing
plt.figure(figsize=(40, 30))
nx.draw_networkx_edges(G_2022_shop1, pos_2022_shop1, width=edge_widths_2022_shop1, edge_color='grey', alpha=0.5)
nx.draw_networkx_nodes(G_2022_shop1, pos_2022_shop1, node_size=node_sizes_2022_shop1, node_color=node_colors_2022_shop1, alpha=0.8)
nx.draw_networkx_labels(G_2022_shop1, pos_2022_shop1, font_size=8, font_color='black')
plt.title('Network Graph of Association Rules Based on Confidence for 2022 Shop 1')
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop1 = nx.DiGraph()

convictions_2022 = []  # To store conviction values for normalization

for _, row in rules_2022_shop1_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    conviction_2022 = row['conviction']  # Assuming each row has a 'conviction' column
    convictions_2022.append(conviction_2022)
    G_2022_shop1.add_edge(antecedents, consequents, weight=row['confidence'], conviction=conviction_2022)

# Normalize conviction values for edge width
max_conviction_2022 = max(convictions_2022)
min_conviction_2022 = min(convictions_2022)
conviction_range_2022 = max_conviction_2022 - min_conviction_2022
edge_widths_2022 = [(G_2022_shop1[u][v]['conviction'] - min_conviction_2022) / conviction_range_2022 * 30 + 0.5 for u, v in G_2022_shop1.edges()]  # Scale and offset widths

pos_2022_shop1 = nx.spring_layout(G_2022_shop1)

# Node sizes and colors as before
in_degrees_2022_shop1 = dict(G_2022_shop1.in_degree())
node_sizes_2022_shop1 = [in_degrees_2022_shop1[node] * 100 + 1000 for node in G_2022_shop1.nodes()]
normalized_sizes_2022_shop1 = (np.array(node_sizes_2022_shop1) - min(node_sizes_2022_shop1)) / (max(node_sizes_2022_shop1) - min(node_sizes_2022_shop1))
cmap_2022_shop1 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2022_shop1 = [cmap_2022_shop1(size) for size in normalized_sizes_2022_shop1]

# Drawing the network with conviction-based edge widths
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2022_shop1, pos_2022_shop1, node_size=node_sizes_2022_shop1, node_color=node_colors_2022_shop1, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop1, pos_2022_shop1, width=edge_widths_2022, edge_color='grey', alpha=0.5)
nx.draw_networkx_labels(G_2022_shop1, pos_2022_shop1, font_size=10)

plt.title('Top 5 Association Rules Network for 2022 Shop 1 with Conviction-based Edge Thickness')
plt.axis('off')
plt.show()


In [ ]:
#the node sizes are based on the indegree where each node acts as a consequent,

In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2022_shop1_nonredundant_top30.loc[:, 'antecedents_'] = rules_2022_shop1_nonredundant_top30['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2022_shop1_nonredundant_top30.loc[:, 'consequents_'] = rules_2022_shop1_nonredundant_top30['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2022_shop1_nonredundant_top30, x='consequents_', y='antecedents_', s=rules_2022_shop1_nonredundant_top30['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2022_shop1_nonredundant_top30.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2022 Shop 1", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
print(rules_2022_shop1_nonredundant_top30[['antecedents', 'consequents']].head(10))

## Monetary Analysis of the Top Rules

In [ ]:
# Step 1: Extract unique product codes from the rules
unique_product_codes = set()
for idx, row in rules_2022_shop1_nonredundant_top30.iterrows():
    # Unite antecedents and consequents into a single set
    all_products = row['antecedents'].union(row['consequents'])
    unique_product_codes.update(all_products)

# Step 2: Create a DataFrame with unique product codes and their unit prices
# Filter sales_data for these product codes and drop duplicates to ensure each product code appears once
product_prices = sales_data[sales_data['product_code'].isin(unique_product_codes)][['product_code', 'unit_price']].drop_duplicates()

# Create a dictionary for quick lookup of prices
price_dict = product_prices.set_index('product_code')['unit_price'].to_dict()

# Step 3: Calculate the total price for each rule and collect data for the new DataFrame
data_for_new_df = []
for idx, row in rules_2022_shop1_nonredundant_top30.iterrows():
    # Combine sets of antecedents and consequents
    all_products = row['antecedents'].union(row['consequents'])
    total_price = sum(price_dict.get(prod, 0) for prod in all_products)  # Use get to avoid KeyError if any product code is missing
    data_for_new_df.append({
        'antecedents': row['antecedents'],
        'consequents': row['consequents'],
        'total_unit_price': total_price
    })

# Create a new DataFrame with only antecedents, consequents, and total unit price
rules_and_price_shop1_2022 = pd.DataFrame(data_for_new_df)

# Sort the DataFrame by total_unit_price in descending order
rules_and_price_shop1_2022_sorted = rules_and_price_shop1_2022.sort_values(by='total_unit_price', ascending=False)

# Print the sorted DataFrame to see the specified columns along with their total unit prices
print(rules_and_price_shop1_2022_sorted)


In [ ]:
# Convert 'trx_date' to datetime format if not already done so
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Add a 'year' column if it doesn't exist
sales_data['year'] = sales_data['trx_date'].dt.year

# Step 2: Filter sales_data for shop1 and the year 2022
filtered_sales_data_2022 = sales_data[(sales_data['shopID'] == 1) & (sales_data['year'] == 2022)]

# Step 3: Initialize a list to hold the rule data including total sales value
rule_sales_summaries_2022 = []

# Step 4: Iterate through each rule in the DataFrame
for index, rule in rules_2022_shop1_nonredundant_top30.iterrows():
    all_products = rule['antecedents'].union(rule['consequents'])

    # Filter the sales data for transactions that include all products in the rule
    # This step ensures that we are only considering transactions that contain all specified products
    mask = filtered_sales_data_2022['product_code'].isin(all_products)
    transaction_ids = filtered_sales_data_2022[mask].groupby('trx_id').filter(lambda x: all_products.issubset(set(x['product_code']))).trx_id.unique()
    matched_transactions = filtered_sales_data_2022[filtered_sales_data_2022['trx_id'].isin(transaction_ids)]
    
    # Step 5: Sum the total_value of these transactions
    total_sales_value = matched_transactions['total_value'].sum()

    # Append the result to the list
    rule_sales_summaries_2022.append({
        'antecedents': rule['antecedents'],
        'consequents': rule['consequents'],
        'total_sales_value': total_sales_value
    })

# Step 6: Convert the list to a DataFrame
rules_sales_values_df_2022 = pd.DataFrame(rule_sales_summaries_2022)

# Step 7: Sort by total_sales_value in descending order
sorted_rules_sales_values_df_2022 = rules_sales_values_df_2022.sort_values(by='total_sales_value', ascending=False)

# Step 8: Print the sorted DataFrame
print(sorted_rules_sales_values_df_2022)


## Narrowing down based on Lift

In [ ]:
rules_2022_shop1_nonredundant_top30_lift['LHS'] = [','.join(list(x)) for x in rules_2022_shop1_nonredundant_top30_lift['antecedents']]
rules_2022_shop1_nonredundant_top30_lift['RHS'] = [','.join(list(x)) for x in rules_2022_shop1_nonredundant_top30_lift['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2022_shop1_nonredundant_top30_lift.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2022_shop1_nonredundant_top30_lift, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2022 Shop 2")
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop1_lift = nx.DiGraph()

lift_summary_2022_shop1 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2022_shop1_nonredundant_top30_lift.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop1_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop1:
        lift_summary_2022_shop1[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop1[consequents]['count'] += 1
    else:
        lift_summary_2022_shop1[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop1:
    lift_summary_2022_shop1[item]['avg_lift'] = lift_summary_2022_shop1[item]['total_lift'] / lift_summary_2022_shop1[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop1_lift = [lift_summary_2022_shop1[node]['avg_lift'] * 500 if node in lift_summary_2022_shop1 else 1000 for node in G_2022_shop1_lift.nodes()]

pos_2022_shop1_lift = nx.spring_layout(G_2022_shop1_lift)

# Colormap from orange to red
cmap_2022_shop1_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop1_lift = np.array(node_sizes_2022_shop1_lift)
normalized_sizes_2022_shop1_lift = (normalized_sizes_2022_shop1_lift - min(normalized_sizes_2022_shop1_lift)) / (max(normalized_sizes_2022_shop1_lift) - min(normalized_sizes_2022_shop1_lift))
node_colors_2022_shop1_lift = [cmap_2022_shop1_lift(size) for size in normalized_sizes_2022_shop1_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2022_shop1_lift, pos_2022_shop1_lift, node_size=node_sizes_2022_shop1_lift, node_color=node_colors_2022_shop1_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop1_lift, pos_2022_shop1_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop1_lift, pos_2022_shop1_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2022 Shop 1 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2022_shop1_nonredundant_top30_lift.loc[:, 'antecedents_'] = rules_2022_shop1_nonredundant_top30_lift['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2022_shop1_nonredundant_top30_lift.loc[:, 'consequents_'] = rules_2022_shop1_nonredundant_top30_lift['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2022_shop1_nonredundant_top30_lift, x='consequents_', y='antecedents_', s=rules_2022_shop1_nonredundant_top30_lift['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2022_shop1_nonredundant_top30_lift.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2022 Shop 1", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
# Print only the 'antecedents' and 'consequents' columns of the top 10 rules
print(rules_2022_shop1_nonredundant_top30_lift[['antecedents', 'consequents']].head(20))


# Shop1 2023 SEARCH 4

In [ ]:
# Filter for records from shop ID 1 and the year 1993
sales_data_2023_shop1 = sales_data_shop1[(sales_data_shop1['trx_date'].dt.year == 2023)]

In [ ]:
sales_data_2023_shop1.head()

In [ ]:
# Grouping product codes by trx_id
transactions_2023_shop1 = sales_data_2023_shop1.groupby('trx_id')['product_code'].apply(list).tolist()

In [ ]:
# Encode transaction data
encoder_2023_shop1 = TransactionEncoder()
trans_array_2023_shop1 = encoder_2023_shop1.fit(transactions_2023_shop1).transform(transactions_2023_shop1)
trans_df_2023_shop1 = pd.DataFrame(trans_array_2023_shop1, columns=encoder_2023_shop1.columns_)

In [ ]:
# Find frequent itemsets
frequent_itemsets_2023_shop1 = apriori(trans_df_2023_shop1, min_support=0.001, use_colnames=True)

In [ ]:
print(frequent_itemsets_2023_shop1)

In [ ]:
## Generate the association rules - by confidence
rulesConfidence_2023_shop1 = association_rules(frequent_itemsets_2023_shop1, metric="confidence", min_threshold=0.10)
rulesConfidence_2023_shop1.sort_values(by='confidence', ascending=False, inplace=True)


In [ ]:
# Note: This is a hypothetical example, as I can't directly manipulate or access dataframes not provided within this code cell.

# Replace frozen sets with strings
rulesConfidence_2023_shop1['antecedents_'] = rulesConfidence_2023_shop1['antecedents'].apply(lambda a: ','.join(list(a)))
rulesConfidence_2023_shop1['consequents_'] = rulesConfidence_2023_shop1['consequents'].apply(lambda a: ','.join(list(a)))

# Draw
fig, ax = plt.subplots(figsize=(14,30))
ax = plt.scatter(data=rulesConfidence_2023_shop1, x='consequents_', y='antecedents_', s=rulesConfidence_2023_shop1['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rulesConfidence_2023_shop1.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2020 Shop 1", fontsize=14)
plt.xlabel("Consequents (RHS)", fontsize=12)
plt.ylabel("Antecedents (LHS)", fontsize=12)
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate(rotation=45)
plt.tight_layout()

plt.show()



## Eliminate Redundant Rules

In [ ]:
rulesConfidence_2023_shop1.head()

In [ ]:
rulesConfidence_2023_shop1.info()

In [ ]:
# If 'antecedents' and 'consequents' are not frozensets, convert them
rules_2023_shop1_redundant = rulesConfidence_2023_shop1.copy()
rules_2023_shop1_redundant['antecedents'] = rules_2023_shop1_redundant['antecedents'].apply(lambda x: frozenset(x))
rules_2023_shop1_redundant['consequents'] = rules_2023_shop1_redundant['consequents'].apply(lambda x: frozenset(x))


In [ ]:
# Create a copy of the DataFrame to avoid modifying the original while iterating
rules_2023_shop1_checkredundance = rules_2023_shop1_redundant.copy()

# List to hold non-redundant rules
non_redundant_rules_2023_shop1 = []

for _, rule in rules_2023_shop1_checkredundance.iterrows():
    if not is_redundant(rule, rules_2023_shop1_checkredundance):
        non_redundant_rules_2023_shop1.append(rule)

# Convert the list back to a DataFrame
rules_2023_shop1_nonredundant = pd.DataFrame(non_redundant_rules_2023_shop1).reset_index(drop=True)


In [ ]:
print(rules_2023_shop1_nonredundant.head())
print(f"Original number of rules: {len(rules_2023_shop1_redundant)}")
print(f"Number of non-redundant rules: {len(rules_2023_shop1_nonredundant)}")


In [ ]:
rules_2023_shop1_nonredundant.head()

rules_2023_shop1_redundant.info()

In [ ]:
rules_2023_shop1_nonredundant.head()

In [ ]:
rules_2023_shop1_nonredundant_backup = rules_2023_shop1_nonredundant.copy()

In [ ]:
rules_2023_shop1_nonredundant_unfiltered = rules_2023_shop1_nonredundant.copy()

In [ ]:
#reset
#rules_2023_shop1_nonredundant = rules_2023_shop1_nonredundant_backup.copy()

# Redundant and Unfiltered

In [ ]:
rules_2023_shop1_redundant.head()

In [ ]:
G_2023_shop1 = nx.DiGraph()

# Iterate through rules for 2023 Shop 1 and add edges to the graph
for _, row in rules_2023_shop1_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as weight
    G_2023_shop1.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2023_shop1 = nx.spring_layout(G_2023_shop1)

# Recalculate node sizes based on indegree or another metric
in_degrees_2023_shop1 = dict(G_2023_shop1.in_degree())
node_sizes_2023_shop1 = [in_degrees_2023_shop1[node] * 100 + 1000 for node in G_2023_shop1.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2023_shop1 = (np.array(node_sizes_2023_shop1) - np.min(node_sizes_2023_shop1)) / (np.max(node_sizes_2023_shop1) - np.min(node_sizes_2023_shop1))

# Define a colormap from orange to red for visual consistency
cmap_2023_shop1 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop1 = [cmap_2023_shop1(size) for size in normalized_sizes_2023_shop1]

# Drawing the network
plt.figure(figsize=(40, 33))
nx.draw_networkx_nodes(G_2023_shop1, pos_2023_shop1, node_size=node_sizes_2023_shop1, node_color=node_colors_2023_shop1, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop1, pos_2023_shop1, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop1, pos_2023_shop1, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 1 with Color Gradient')
plt.axis('off')
plt.show()


### Based on LIFT

In [ ]:
G_2023_shop1_lift = nx.DiGraph()

lift_summary_2023_shop1 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop1_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop1_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop1:
        lift_summary_2023_shop1[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop1[consequents]['count'] += 1
    else:
        lift_summary_2023_shop1[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop1:
    lift_summary_2023_shop1[item]['avg_lift'] = lift_summary_2023_shop1[item]['total_lift'] / lift_summary_2023_shop1[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop1_lift = [lift_summary_2023_shop1[node]['avg_lift'] * 500 if node in lift_summary_2023_shop1 else 1000 for node in G_2023_shop1_lift.nodes()]

pos_2023_shop1_lift = nx.spring_layout(G_2023_shop1_lift)

# Colormap from orange to red
cmap_2023_shop1_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop1_lift = np.array(node_sizes_2023_shop1_lift)
normalized_sizes_2023_shop1_lift = (normalized_sizes_2023_shop1_lift - min(normalized_sizes_2023_shop1_lift)) / (max(normalized_sizes_2023_shop1_lift) - min(normalized_sizes_2023_shop1_lift))
node_colors_2023_shop1_lift = [cmap_2023_shop1_lift(size) for size in normalized_sizes_2023_shop1_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(70, 52))
nx.draw_networkx_nodes(G_2023_shop1_lift, pos_2023_shop1_lift, node_size=node_sizes_2023_shop1_lift, node_color=node_colors_2023_shop1_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop1_lift, pos_2023_shop1_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop1_lift, pos_2023_shop1_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 1 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


# nonredundant_unfiltered

## Based on Indegree

In [ ]:
G_2023_shop1_nonredundant_unfiltered = nx.DiGraph()

# Iterating through each row to add edges with confidence as an attribute
for _, row in rules_2023_shop1_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop1_nonredundant_unfiltered.add_edge(antecedents, consequents, weight=row['confidence'])

# Defining the layout for network visualization
pos_2023_shop1_nonredundant_unfiltered = nx.spring_layout(G_2023_shop1_nonredundant_unfiltered)

# Calculating node sizes based on the indegree for color mapping
in_degrees_2023_shop1_nonredundant_unfiltered = dict(G_2023_shop1_nonredundant_unfiltered.in_degree())
node_sizes_2023_shop1_nonredundant_unfiltered = [in_degrees_2023_shop1_nonredundant_unfiltered[node] * 100 + 1000 for node in G_2023_shop1_nonredundant_unfiltered.nodes()]

# Normalizing node sizes for color mapping
normalized_sizes_2023_shop1_nonredundant_unfiltered = (np.array(node_sizes_2023_shop1_nonredundant_unfiltered) - min(node_sizes_2023_shop1_nonredundant_unfiltered)) / (max(node_sizes_2023_shop1_nonredundant_unfiltered) - min(node_sizes_2023_shop1_nonredundant_unfiltered))

# Defining a color map from orange to red
cmap_2023_shop1_nonredundant_unfiltered = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])

# Mapping normalized node sizes to colors
node_colors_2023_shop1_nonredundant_unfiltered = [cmap_2023_shop1_nonredundant_unfiltered(size) for size in normalized_sizes_2023_shop1_nonredundant_unfiltered]

# Drawing the network with node sizes and colors
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2023_shop1_nonredundant_unfiltered, pos_2023_shop1_nonredundant_unfiltered, node_size=node_sizes_2023_shop1_nonredundant_unfiltered, node_color=node_colors_2023_shop1_nonredundant_unfiltered, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop1_nonredundant_unfiltered, pos_2023_shop1_nonredundant_unfiltered, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2023_shop1_nonredundant_unfiltered, pos_2023_shop1_nonredundant_unfiltered, font_size=10)

plt.title('Node Size Based on Indegree for 2023 Shop 1')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2023_shop1_lift = nx.DiGraph()

lift_summary_2023_shop1 = {}

for _, row in rules_2023_shop1_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop1_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Update lift summary for consequents
    if consequents in lift_summary_2023_shop1:
        lift_summary_2023_shop1[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop1[consequents]['count'] += 1
    else:
        lift_summary_2023_shop1[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculate average lift for each consequent
for item in lift_summary_2023_shop1:
    lift_summary_2023_shop1[item]['avg_lift'] = lift_summary_2023_shop1[item]['total_lift'] / lift_summary_2023_shop1[item]['count']

# Node sizes based on average lift, scaled appropriately
node_sizes_2023_shop1_lift = [lift_summary_2023_shop1[node]['avg_lift'] * 500 if node in lift_summary_2023_shop1 else 1000 for node in G_2023_shop1_lift.nodes()]

pos_2023_shop1_lift = nx.spring_layout(G_2023_shop1_lift)

# Use a colormap that transitions from orange to red
cmap_2023_shop1_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop1_lift = np.array(node_sizes_2023_shop1_lift)
normalized_sizes_2023_shop1_lift = (normalized_sizes_2023_shop1_lift - min(normalized_sizes_2023_shop1_lift)) / (max(normalized_sizes_2023_shop1_lift) - min(normalized_sizes_2023_shop1_lift))
node_colors_2023_shop1_lift = [cmap_2023_shop1_lift(size) for size in normalized_sizes_2023_shop1_lift]

plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2023_shop1_lift, pos_2023_shop1_lift, node_size=node_sizes_2023_shop1_lift, node_color=node_colors_2023_shop1_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop1_lift, pos_2023_shop1_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop1_lift, pos_2023_shop1_lift, font_size=10)

plt.title('Node Size Based on Average Lift for 2023 Shop 1')
plt.axis('off')
plt.show()


# Based on conviction

In [ ]:
rules_2023_shop1_nonredundant.head()

In [ ]:
G_2023_shop1_conviction = nx.DiGraph()

conviction_summary_2023_shop1 = {}
count_summary_2023_shop1 = {}

# Assuming 'rules_2023_shop1_nonredundant_unfiltered' contains a 'conviction' column
for _, row in rules_2023_shop1_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop1_conviction.add_edge(antecedents, consequents, weight=row['confidence'], conviction=row['conviction'])
    
    # Aggregate conviction values and count occurrences
    if consequents in conviction_summary_2023_shop1:
        conviction_summary_2023_shop1[consequents] += row['conviction']
        count_summary_2023_shop1[consequents] += 1
    else:
        conviction_summary_2023_shop1[consequents] = row['conviction']
        count_summary_2023_shop1[consequents] = 1

# Calculate average conviction values
average_conviction_summary_2023_shop1 = {k: conviction_summary_2023_shop1[k] / count_summary_2023_shop1[k] for k in conviction_summary_2023_shop1}

# Normalize conviction values for sizing and coloring
max_conviction = max(average_conviction_summary_2023_shop1.values())
min_conviction = min(average_conviction_summary_2023_shop1.values())
conviction_range = max_conviction - min_conviction
node_colors = []
node_sizes = []

for node in G_2023_shop1_conviction.nodes():
    # Get the average conviction value for the node, defaulting to min if not present
    conviction = average_conviction_summary_2023_shop1.get(node, min_conviction)
    # Normalize between 0 and 1
    normalized_conviction = (conviction - min_conviction) / conviction_range if conviction_range else 0
    node_colors.append(normalized_conviction)
    node_sizes.append(normalized_conviction * 1000 + 100)  # Scale and offset size

# Use a colormap for coloring nodes based on conviction
cmap = plt.cm.coolwarm

pos = nx.spring_layout(G_2023_shop1_conviction)

plt.figure(figsize=(40, 22))
nodes = nx.draw_networkx_nodes(G_2023_shop1_conviction, pos, node_color=node_colors, node_size=node_sizes, cmap=cmap, alpha=0.8)
edges = nx.draw_networkx_edges(G_2023_shop1_conviction, pos, arrowstyle='->', arrowsize=10, edge_color='grey', alpha=0.5)
labels = nx.draw_networkx_labels(G_2023_shop1_conviction, pos, font_size=8)

plt.colorbar(nodes, label='Normalized Conviction')
plt.title('Network Graph of 2023 Shop 1 Based on Conviction')
plt.axis('off')
plt.show()

In [ ]:
print(max_conviction)

# Filtered top 30 and non redundant

In [ ]:
rules_2023_shop1_nonredundant.head()

In [ ]:
# Assuming rules_1992_shop2_nonredundant is your sorted rules DataFrame for shop2
rules_2023_shop1_nonredundant_top30 = rules_2023_shop1_nonredundant.copy(deep=True)

In [ ]:
rules_2023_shop1_nonredundant_top30_lift = rules_2023_shop1_nonredundant.copy(deep=True)

In [ ]:
# Select the top 30 rules based on lift or another criterion
rules_2023_shop1_nonredundant_top30_lift = rules_2023_shop1_nonredundant_top30_lift.sort_values(by='lift', ascending=False).head(30)

In [ ]:
rules_2023_shop1_nonredundant_top30_lift.head()

In [ ]:
# Select the top 30 rules based on confidence or another criterion
rules_2023_shop1_nonredundant_top30 = rules_2023_shop1_nonredundant_top30.sort_values(by='confidence', ascending=False).head(30)

In [ ]:
rules_2023_shop1_nonredundant_top30.info()

In [ ]:
rules_2023_shop1_nonredundant_top30['LHS'] = [','.join(list(x)) for x in rules_2023_shop1_nonredundant_top30['antecedents']]
rules_2023_shop1_nonredundant_top30['RHS'] = [','.join(list(x)) for x in rules_2023_shop1_nonredundant_top30['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2023_shop1_nonredundant_top30.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2023_shop1_nonredundant_top30, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2020 Shop 2")
plt.axis('off')
plt.show()


## Based on Indegree

In [ ]:
G_2023_shop1 = nx.DiGraph()

# Assuming 'rules_sorted_2023_shop1' contains your association rules for shop 1 in 2023
for _, row in rules_2023_shop1_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as the weight
    G_2023_shop1.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2023_shop1 = nx.spring_layout(G_2023_shop1)

# Calculate node sizes based on indegree
in_degrees_2023_shop1 = dict(G_2023_shop1.in_degree())
node_sizes_2023_shop1 = [in_degrees_2023_shop1[node] * 100 + 1000 for node in G_2023_shop1.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2023_shop1 = (np.array(node_sizes_2023_shop1) - np.min(node_sizes_2023_shop1)) / (np.max(node_sizes_2023_shop1) - np.min(node_sizes_2023_shop1))

# Define a colormap from orange to red
cmap_2023_shop1 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop1 = [cmap_2023_shop1(size) for size in normalized_sizes_2023_shop1]

# Drawing the network
plt.figure(figsize=(30, 24))
nx.draw_networkx_nodes(G_2023_shop1, pos_2023_shop1, node_size=node_sizes_2023_shop1, node_color=node_colors_2023_shop1, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop1, pos_2023_shop1, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2023_shop1, pos_2023_shop1, font_size=8)

plt.title('Top 5 Association Rules Network for 2023 Shop 1 with Color Gradient')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2023_shop1_lift = nx.DiGraph()

lift_summary_2023_shop1 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop1_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop1_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop1:
        lift_summary_2023_shop1[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop1[consequents]['count'] += 1
    else:
        lift_summary_2023_shop1[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop1:
    lift_summary_2023_shop1[item]['avg_lift'] = lift_summary_2023_shop1[item]['total_lift'] / lift_summary_2023_shop1[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop1_lift = [lift_summary_2023_shop1[node]['avg_lift'] * 500 if node in lift_summary_2023_shop1 else 1000 for node in G_2023_shop1_lift.nodes()]

pos_2023_shop1_lift = nx.spring_layout(G_2023_shop1_lift)

# Colormap from orange to red
cmap_2023_shop1_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop1_lift = np.array(node_sizes_2023_shop1_lift)
normalized_sizes_2023_shop1_lift = (normalized_sizes_2023_shop1_lift - min(normalized_sizes_2023_shop1_lift)) / (max(normalized_sizes_2023_shop1_lift) - min(normalized_sizes_2023_shop1_lift))
node_colors_2023_shop1_lift = [cmap_2023_shop1_lift(size) for size in normalized_sizes_2023_shop1_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(60, 32))
nx.draw_networkx_nodes(G_2023_shop1_lift, pos_2023_shop1_lift, node_size=node_sizes_2023_shop1_lift, node_color=node_colors_2023_shop1_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop1_lift, pos_2023_shop1_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop1_lift, pos_2023_shop1_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 1 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop1_lift = nx.DiGraph()

lift_summary_2023_shop1 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop1_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop1_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop1:
        lift_summary_2023_shop1[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop1[consequents]['count'] += 1
    else:
        lift_summary_2023_shop1[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop1:
    lift_summary_2023_shop1[item]['avg_lift'] = lift_summary_2023_shop1[item]['total_lift'] / lift_summary_2023_shop1[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop1_lift = [lift_summary_2023_shop1[node]['avg_lift'] * 500 if node in lift_summary_2023_shop1 else 1000 for node in G_2023_shop1_lift.nodes()]

pos_2023_shop1_lift = nx.spring_layout(G_2023_shop1_lift)

# Colormap from orange to red
cmap_2023_shop1_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop1_lift = np.array(node_sizes_2023_shop1_lift)
normalized_sizes_2023_shop1_lift = (normalized_sizes_2023_shop1_lift - min(normalized_sizes_2023_shop1_lift)) / (max(normalized_sizes_2023_shop1_lift) - min(normalized_sizes_2023_shop1_lift))
node_colors_2023_shop1_lift = [cmap_2023_shop1_lift(size) for size in normalized_sizes_2023_shop1_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2023_shop1_lift, pos_2023_shop1_lift, node_size=node_sizes_2023_shop1_lift, node_color=node_colors_2023_shop1_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop1_lift, pos_2023_shop1_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop1_lift, pos_2023_shop1_lift, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 1 Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop1 = nx.DiGraph()

# Assuming 'rules_2023_shop1_nonredundant' contains association rules for 2023 Shop 1
for _, row in rules_2023_shop1_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop1.add_edge(antecedents, consequents, confidence=row['confidence'])

# Calculate node sizes based on indegree
in_degrees_2023_shop1 = dict(G_2023_shop1.in_degree())
node_sizes_2023_shop1 = [in_degrees_2023_shop1[node] * 100 + 1000 for node in G_2023_shop1.nodes()]

# Edge width based on confidence
edge_widths_2023_shop1 = [G_2023_shop1[u][v]['confidence'] * 15 for u, v in G_2023_shop1.edges()]

# Assuming normalized sizes and color mapping is desired
normalized_sizes_2023_shop1 = np.array(list(in_degrees_2023_shop1.values()))
normalized_sizes_2023_shop1 = (normalized_sizes_2023_shop1 - min(normalized_sizes_2023_shop1)) / (max(normalized_sizes_2023_shop1) - min(normalized_sizes_2023_shop1))
cmap_2023_shop1 = mcolors.LinearSegmentedColormap.from_list("MyCMapName", ["blue", "red"])

# Node colors based on a property (here, using indegree for example)
node_colors_2023_shop1 = [cmap_2023_shop1(size) for size in normalized_sizes_2023_shop1]

# Layout
pos_2023_shop1 = nx.spring_layout(G_2023_shop1, seed=42)

# Drawing
plt.figure(figsize=(40, 30))
nx.draw_networkx_edges(G_2023_shop1, pos_2023_shop1, width=edge_widths_2023_shop1, edge_color='grey', alpha=0.5)
nx.draw_networkx_nodes(G_2023_shop1, pos_2023_shop1, node_size=node_sizes_2023_shop1, node_color=node_colors_2023_shop1, alpha=0.8)
nx.draw_networkx_labels(G_2023_shop1, pos_2023_shop1, font_size=8, font_color='black')
plt.title('Network Graph of Association Rules Based on Confidence for 2023 Shop 1')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop1 = nx.DiGraph()

convictions_2023 = []  # To store conviction values for normalization

for _, row in rules_2023_shop1_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    conviction = row['conviction']  # Assuming each row has a 'conviction' column
    convictions_2023.append(conviction)
    G_2023_shop1.add_edge(antecedents, consequents, weight=row['confidence'], conviction=conviction)

# Normalize conviction values for edge width
max_conviction_2023 = max(convictions_2023)
min_conviction_2023 = min(convictions_2023)
conviction_range_2023 = max_conviction_2023 - min_conviction_2023
edge_widths_2023 = [(G_2023_shop1[u][v]['conviction'] - min_conviction_2023) / conviction_range_2023 * 30 + 0.5 for u, v in G_2023_shop1.edges()]  # Scale and offset widths

pos_2023_shop1 = nx.spring_layout(G_2023_shop1)

# Node sizes and colors as before
in_degrees_2023_shop1 = dict(G_2023_shop1.in_degree())
node_sizes_2023_shop1 = [in_degrees_2023_shop1[node] * 100 + 1000 for node in G_2023_shop1.nodes()]
normalized_sizes_2023_shop1 = (np.array(node_sizes_2023_shop1) - min(node_sizes_2023_shop1)) / (max(node_sizes_2023_shop1) - min(node_sizes_2023_shop1))
cmap_2023_shop1 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop1 = [cmap_2023_shop1(size) for size in normalized_sizes_2023_shop1]

# Drawing the network with conviction-based edge widths
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2023_shop1, pos_2023_shop1, node_size=node_sizes_2023_shop1, node_color=node_colors_2023_shop1, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop1, pos_2023_shop1, width=edge_widths_2023, edge_color='grey', alpha=0.5)
nx.draw_networkx_labels(G_2023_shop1, pos_2023_shop1, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 1 with Conviction-based Edge Thickness')
plt.axis('off')
plt.show()


In [ ]:
#the node sizes are based on the indegree where each node acts as a consequent,

In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2023_shop1_nonredundant_top30.loc[:, 'antecedents_'] = rules_2023_shop1_nonredundant_top30['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2023_shop1_nonredundant_top30.loc[:, 'consequents_'] = rules_2023_shop1_nonredundant_top30['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2023_shop1_nonredundant_top30, x='consequents_', y='antecedents_', s=rules_2023_shop1_nonredundant_top30['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2023_shop1_nonredundant_top30.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2023 Shop 1", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
print(rules_2023_shop1_nonredundant_top30[['antecedents', 'consequents']].head(10))

## Monetary Analysis of the Top Rules

In [ ]:
# Step 1: Extract unique product codes from the rules
unique_product_codes = set()
for idx, row in rules_2023_shop1_nonredundant_top30.iterrows():
    # Unite antecedents and consequents into a single set
    all_products = row['antecedents'].union(row['consequents'])
    unique_product_codes.update(all_products)

# Step 2: Create a DataFrame with unique product codes and their unit prices
# Filter sales_data for these product codes and drop duplicates to ensure each product code appears once
product_prices = sales_data[sales_data['product_code'].isin(unique_product_codes)][['product_code', 'unit_price']].drop_duplicates()

# Create a dictionary for quick lookup of prices
price_dict = product_prices.set_index('product_code')['unit_price'].to_dict()

# Step 3: Calculate the total price for each rule and collect data for the new DataFrame
data_for_new_df = []
for idx, row in rules_2023_shop1_nonredundant_top30.iterrows():
    # Combine sets of antecedents and consequents
    all_products = row['antecedents'].union(row['consequents'])
    total_price = sum(price_dict.get(prod, 0) for prod in all_products)  # Use get to avoid KeyError if any product code is missing
    data_for_new_df.append({
        'antecedents': row['antecedents'],
        'consequents': row['consequents'],
        'total_unit_price': total_price
    })

# Create a new DataFrame with only antecedents, consequents, and total unit price
rules_and_price_shop1_2023 = pd.DataFrame(data_for_new_df)

# Sort the DataFrame by total_unit_price in descending order
rules_and_price_shop1_2023_sorted = rules_and_price_shop1_2023.sort_values(by='total_unit_price', ascending=False)

# Print the sorted DataFrame to see the specified columns along with their total unit prices
print(rules_and_price_shop1_2023_sorted)


In [ ]:
# Convert 'trx_date' to datetime format if not already done so
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Add a 'year' column if it doesn't exist
sales_data['year'] = sales_data['trx_date'].dt.year

# Step 2: Filter sales_data for shop1 and the year 2023
filtered_sales_data_2023 = sales_data[(sales_data['shopID'] == 1) & (sales_data['year'] == 2023)]

# Step 3: Initialize a list to hold the rule data including total sales value
rule_sales_summaries_2023 = []

# Step 4: Iterate through each rule in the DataFrame
for index, rule in rules_2023_shop1_nonredundant_top30.iterrows():
    all_products = rule['antecedents'].union(rule['consequents'])

    # Filter the sales data for transactions that include all products in the rule
    # This step ensures that we are only considering transactions that contain all specified products
    mask = filtered_sales_data_2023['product_code'].isin(all_products)
    transaction_ids = filtered_sales_data_2023[mask].groupby('trx_id').filter(lambda x: all_products.issubset(set(x['product_code']))).trx_id.unique()
    matched_transactions = filtered_sales_data_2023[filtered_sales_data_2023['trx_id'].isin(transaction_ids)]
    
    # Step 5: Sum the total_value of these transactions
    total_sales_value = matched_transactions['total_value'].sum()

    # Append the result to the list
    rule_sales_summaries_2023.append({
        'antecedents': rule['antecedents'],
        'consequents': rule['consequents'],
        'total_sales_value': total_sales_value
    })

# Step 6: Convert the list to a DataFrame
rules_sales_values_df_2023 = pd.DataFrame(rule_sales_summaries_2023)

# Step 7: Sort by total_sales_value in descending order
sorted_rules_sales_values_df_2023 = rules_sales_values_df_2023.sort_values(by='total_sales_value', ascending=False)

# Step 8: Print the sorted DataFrame
print(sorted_rules_sales_values_df_2023)


## Narrowing down based on Lift

In [ ]:
rules_2023_shop1_nonredundant_top30_lift['LHS'] = [','.join(list(x)) for x in rules_2023_shop1_nonredundant_top30_lift['antecedents']]
rules_2023_shop1_nonredundant_top30_lift['RHS'] = [','.join(list(x)) for x in rules_2023_shop1_nonredundant_top30_lift['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2023_shop1_nonredundant_top30_lift.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2023_shop1_nonredundant_top30_lift, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2023 Shop 2")
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop1_lift = nx.DiGraph()

lift_summary_2023_shop1 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop1_nonredundant_top30_lift.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop1_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop1:
        lift_summary_2023_shop1[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop1[consequents]['count'] += 1
    else:
        lift_summary_2023_shop1[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop1:
    lift_summary_2023_shop1[item]['avg_lift'] = lift_summary_2023_shop1[item]['total_lift'] / lift_summary_2023_shop1[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop1_lift = [lift_summary_2023_shop1[node]['avg_lift'] * 500 if node in lift_summary_2023_shop1 else 1000 for node in G_2023_shop1_lift.nodes()]

pos_2023_shop1_lift = nx.spring_layout(G_2023_shop1_lift)

# Colormap from orange to red
cmap_2023_shop1_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop1_lift = np.array(node_sizes_2023_shop1_lift)
normalized_sizes_2023_shop1_lift = (normalized_sizes_2023_shop1_lift - min(normalized_sizes_2023_shop1_lift)) / (max(normalized_sizes_2023_shop1_lift) - min(normalized_sizes_2023_shop1_lift))
node_colors_2023_shop1_lift = [cmap_2023_shop1_lift(size) for size in normalized_sizes_2023_shop1_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(60, 32))
nx.draw_networkx_nodes(G_2023_shop1_lift, pos_2023_shop1_lift, node_size=node_sizes_2023_shop1_lift, node_color=node_colors_2023_shop1_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop1_lift, pos_2023_shop1_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop1_lift, pos_2023_shop1_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 1 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2023_shop1_nonredundant_top30_lift.loc[:, 'antecedents_'] = rules_2023_shop1_nonredundant_top30_lift['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2023_shop1_nonredundant_top30_lift.loc[:, 'consequents_'] = rules_2023_shop1_nonredundant_top30_lift['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2023_shop1_nonredundant_top30_lift, x='consequents_', y='antecedents_', s=rules_2023_shop1_nonredundant_top30_lift['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2023_shop1_nonredundant_top30_lift.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2023 Shop 1", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
# Print only the 'antecedents' and 'consequents' columns of the top 10 rules
print(rules_2023_shop1_nonredundant_top30_lift[['antecedents', 'consequents']].head(10))


# Shop2 2020 SEARCH 5

In [ ]:
# Filter for records from shop ID 1 and the year 1993
sales_data_2020_shop2 = sales_data_shop2[(sales_data_shop2['trx_date'].dt.year == 2020)]

In [ ]:
sales_data_2020_shop2.head()

In [ ]:
# Grouping product codes by trx_id
transactions_2020_shop2 = sales_data_2020_shop2.groupby('trx_id')['product_code'].apply(list).tolist()

In [ ]:
# Encode transaction data
encoder_2020_shop2 = TransactionEncoder()
trans_array_2020_shop2 = encoder_2020_shop2.fit(transactions_2020_shop2).transform(transactions_2020_shop2)
trans_df_2020_shop2 = pd.DataFrame(trans_array_2020_shop2, columns=encoder_2020_shop2.columns_)

In [ ]:
# Find frequent itemsets
frequent_itemsets_2020_shop2 = apriori(trans_df_2020_shop2, min_support=0.005, use_colnames=True)

In [ ]:
print(frequent_itemsets_2020_shop2)

In [ ]:
# Generate association rules
rules_2020_shop2 = association_rules(frequent_itemsets_2020_shop2, metric="confidence", min_threshold=0.5)

In [ ]:
rules_2020_shop2.info()

In [ ]:
## Generate the association rules - by confidence
rulesConfidence_2020_shop2 = association_rules(frequent_itemsets_2020_shop2, metric="confidence", min_threshold=0.50)
rulesConfidence_2020_shop2.sort_values(by='confidence', ascending=False, inplace=True)


In [ ]:
# Assuming 'rulesConfidence_2020_shop2' is your DataFrame for 2020 Shop 2

# Replace frozen sets with strings
rulesConfidence_2020_shop2['antecedents_'] = rulesConfidence_2020_shop2['antecedents'].apply(lambda a: ','.join(list(a)))
rulesConfidence_2020_shop2['consequents_'] = rulesConfidence_2020_shop2['consequents'].apply(lambda a: ','.join(list(a)))

# Draw
fig, ax = plt.subplots(figsize=(14,30))
ax = plt.scatter(data=rulesConfidence_2020_shop2, x='consequents_', y='antecedents_', s=rulesConfidence_2020_shop2['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rulesConfidence_2020_shop2.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2020 Shop 2", fontsize=14)
plt.xlabel("Consequents (RHS)", fontsize=12)
plt.ylabel("Antecedents (LHS)", fontsize=12)
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate(rotation=45)
plt.tight_layout()

plt.show()


In [ ]:
# Assuming rules_sorted_2020_shop2 is your sorted rules DataFrame for Shop 2
# Copy the rules and convert frozensets to strings
rulesToPlot_2020_shop2 = rulesConfidence_2020_shop2.copy(deep=True)
rulesToPlot_2020_shop2['LHS'] = [','.join(list(x)) for x in rulesToPlot_2020_shop2['antecedents']]
rulesToPlot_2020_shop2['RHS'] = [','.join(list(x)) for x in rulesToPlot_2020_shop2['consequents']]

# Remove duplicates or reversed rules by sorting the antecedent and consequent strings, 
# converting them to a single sorted string, and dropping duplicates based on this string
rulesToPlot_2020_shop2['sortedRow'] = [sorted([a, b]) for a, b in zip(rulesToPlot_2020_shop2['LHS'], rulesToPlot_2020_shop2['RHS'])]
rulesToPlot_2020_shop2['sortedRow'] = rulesToPlot_2020_shop2['sortedRow'].astype(str)
rulesToPlot_2020_shop2.drop_duplicates(subset=['sortedRow'], inplace=True)

# Select the top 30 rules based on confidence or another criterion as needed
rulesToPlot_2020_shop2 = rulesToPlot_2020_shop2.sort_values(by='confidence', ascending=False).head(30)

# Plot the network graph
fig = plt.figure(figsize=(15, 15)) 
G = nx.from_pandas_edgelist(rulesToPlot_2020_shop2, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5) # k regulates the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for Shop 2 in 2020")
plt.axis('off') # Turn off the axis
plt.show()


In [ ]:
G_2020_shop2 = nx.DiGraph()

# Assuming 'rules_sorted_2020_shop2' contains your association rules for shop 2 in 2020
for _, row in rulesConfidence_2020_shop2.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as the weight
    G_2020_shop2.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2020_shop2 = nx.spring_layout(G_2020_shop2)

# Calculate node sizes based on indegree
in_degrees_2020_shop2 = dict(G_2020_shop2.in_degree())
node_sizes_2020_shop2 = [in_degrees_2020_shop2[node] * 100 + 1000 for node in G_2020_shop2.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2020_shop2 = (np.array(node_sizes_2020_shop2) - np.min(node_sizes_2020_shop2)) / (np.max(node_sizes_2020_shop2) - np.min(node_sizes_2020_shop2))

# Define a colormap from orange to red
cmap_2020_shop2 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2020_shop2 = [cmap_2020_shop2(size) for size in normalized_sizes_2020_shop2]

# Drawing the network
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2020_shop2, pos_2020_shop2, node_size=node_sizes_2020_shop2, node_color=node_colors_2020_shop2, alpha=0.8)
nx.draw_networkx_edges(G_2020_shop2, pos_2020_shop2, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2020_shop2, pos_2020_shop2, font_size=8)

plt.title('Top 5 Association Rules Network for 2020 Shop 2 with Color Gradient')
plt.axis('off')
plt.show()


## Eliminate Redundant Rules

In [ ]:
rulesConfidence_2020_shop2.head(170)

In [ ]:
rulesConfidence_2020_shop2.info()

In [ ]:
# If 'antecedents' and 'consequents' are not frozensets, convert them
rules_2020_shop2_redundant = rulesConfidence_2020_shop2.copy()
rules_2020_shop2_redundant['antecedents'] = rules_2020_shop2_redundant['antecedents'].apply(lambda x: frozenset(x))
rules_2020_shop2_redundant['consequents'] = rules_2020_shop2_redundant['consequents'].apply(lambda x: frozenset(x))


In [ ]:
# Create a copy of the DataFrame to avoid modifying the original while iterating
rules_2020_shop2_checkredundance = rules_2020_shop2_redundant.copy()

# List to hold non-redundant rules
non_redundant_rules_2020_shop2 = []

for _, rule in rules_2020_shop2_checkredundance.iterrows():
    if not is_redundant(rule, rules_2020_shop2_checkredundance):
        non_redundant_rules_2020_shop2.append(rule)

# Convert the list back to a DataFrame
rules_2020_shop2_nonredundant = pd.DataFrame(non_redundant_rules_2020_shop2).reset_index(drop=True)


In [ ]:
print(rules_2020_shop2_nonredundant.head())
print(f"Original number of rules: {len(rules_2020_shop2_redundant)}")
print(f"Number of non-redundant rules: {len(rules_2020_shop2_nonredundant)}")


In [ ]:
rules_2020_shop2_nonredundancy_test = rules_2020_shop2_nonredundant.copy()

In [ ]:
rules_sorted_by_consequents = rules_2020_shop2_nonredundancy_test.sort_values(by=['consequents_', 'confidence'], ascending=[True, False])

# Use a dictionary to track which consequents have been seen
seen_consequents = {}

# Iterate over the sorted DataFrame and only keep a rule if its consequent hasn't been seen yet
radically_filtered_rules = []
for _, row in rules_sorted_by_consequents.iterrows():
    if row['consequents_'] not in seen_consequents:
        radically_filtered_rules.append(row)
        seen_consequents[row['consequents_']] = True

# Convert the list back to a DataFrame
radically_filtered_df = pd.DataFrame(radically_filtered_rules).reset_index(drop=True)

In [ ]:
# Sorting the DataFrame by 'conviction'
sorted_by_conviction_df = radically_filtered_df.sort_values(by='conviction', ascending=False).reset_index(drop=True)

In [ ]:
radically_filtered_df.info()

In [ ]:
sorted_by_conviction_df.head()

In [ ]:
rules_2020_shop2_nonredundant.head()

In [ ]:
rules_2020_shop2_nonredundant_backup = rules_2020_shop2_nonredundant.copy()

In [ ]:
rules_2020_shop2_nonredundant_unfiltered = rules_2020_shop2_nonredundant.copy()

In [ ]:
#reset
#rules_2020_shop2_nonredundant = rules_2020_shop2_nonredundant_backup.copy()

# Redundant and Unfiltered

In [ ]:
rules_2020_shop2_redundant.head()

In [ ]:
# Assuming 'rules_sorted_2020_shop2' is correctly defined and includes 'antecedents', 'consequents', and 'confidence'
G_2020_shop2 = nx.DiGraph()

# Iterate through rules for 2020 shop 2 and add edges to the graph
for _, row in rules_2020_shop2_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as weight
    G_2020_shop2.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2020_shop2 = nx.spring_layout(G_2020_shop2)

# Recalculate node sizes based on indegree or another metric
in_degrees_2020_shop2 = dict(G_2020_shop2.in_degree())
node_sizes_2020_shop2 = [in_degrees_2020_shop2[node] * 100 + 1000 for node in G_2020_shop2.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2020_shop2 = (np.array(node_sizes_2020_shop2) - np.min(node_sizes_2020_shop2)) / (np.max(node_sizes_2020_shop2) - np.min(node_sizes_2020_shop2))

# Define a colormap from orange to red for visual consistency
cmap_2020_shop2 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2020_shop2 = [cmap_2020_shop2(size) for size in normalized_sizes_2020_shop2]

# Drawing the network
plt.figure(figsize=(200, 100))
nx.draw_networkx_nodes(G_2020_shop2, pos_2020_shop2, node_size=node_sizes_2020_shop2, node_color=node_colors_2020_shop2, alpha=0.8)
nx.draw_networkx_edges(G_2020_shop2, pos_2020_shop2, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2020_shop2, pos_2020_shop2, font_size=10)

plt.title('Top 5 Association Rules Network for 2020 Shop 2 with Color Gradient')
plt.axis('off')
plt.show()


### Based on LIFT

In [ ]:
G_2020_shop2_lift = nx.DiGraph()

lift_summary_2020_shop2 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2020_shop2_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2020_shop2_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2020_shop2:
        lift_summary_2020_shop2[consequents]['total_lift'] += row['lift']
        lift_summary_2020_shop2[consequents]['count'] += 1
    else:
        lift_summary_2020_shop2[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2020_shop2:
    lift_summary_2020_shop2[item]['avg_lift'] = lift_summary_2020_shop2[item]['total_lift'] / lift_summary_2020_shop2[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2020_shop2_lift = [lift_summary_2020_shop2[node]['avg_lift'] * 500 if node in lift_summary_2020_shop2 else 1000 for node in G_2020_shop2_lift.nodes()]

pos_2020_shop2_lift = nx.spring_layout(G_2020_shop2_lift)

# Colormap from orange to red
cmap_2020_shop2_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2020_shop2_lift = np.array(node_sizes_2020_shop2_lift)
normalized_sizes_2020_shop2_lift = (normalized_sizes_2020_shop2_lift - min(normalized_sizes_2020_shop2_lift)) / (max(normalized_sizes_2020_shop2_lift) - min(normalized_sizes_2020_shop2_lift))
node_colors_2020_shop2_lift = [cmap_2020_shop2_lift(size) for size in normalized_sizes_2020_shop2_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2020_shop2_lift, pos_2020_shop2_lift, node_size=node_sizes_2020_shop2_lift, node_color=node_colors_2020_shop2_lift, alpha=0.8)
nx.draw_networkx_edges(G_2020_shop2_lift, pos_2020_shop2_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2020_shop2_lift, pos_2020_shop2_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2020 Shop 2 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


# nonredundant_unfiltered

## Based on Indegree

In [ ]:
G_2020_shop2_nonredundant_unfiltered = nx.DiGraph()

# Iterating through each row to add edges with confidence as an attribute
for _, row in rules_2020_shop2_nonredundant_unfiltered.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2020_shop2_nonredundant_unfiltered.add_edge(antecedents, consequents, weight=row['confidence'])

# Defining the layout for network visualization
pos_2020_shop2_nonredundant_unfiltered = nx.spring_layout(G_2020_shop2_nonredundant_unfiltered)

# Calculating node sizes based on the indegree for color mapping
in_degrees_2020_shop2_nonredundant_unfiltered = dict(G_2020_shop2_nonredundant_unfiltered.in_degree())
node_sizes_2020_shop2_nonredundant_unfiltered = [in_degrees_2020_shop2_nonredundant_unfiltered[node] * 100 + 1000 for node in G_2020_shop2_nonredundant_unfiltered.nodes()]

# Normalizing node sizes for color mapping
normalized_sizes_2020_shop2_nonredundant_unfiltered = (np.array(node_sizes_2020_shop2_nonredundant_unfiltered) - min(node_sizes_2020_shop2_nonredundant_unfiltered)) / (max(node_sizes_2020_shop2_nonredundant_unfiltered) - min(node_sizes_2020_shop2_nonredundant_unfiltered))

# Defining a color map from orange to red
cmap_2020_shop2_nonredundant_unfiltered = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])

# Mapping normalized node sizes to colors
node_colors_2020_shop2_nonredundant_unfiltered = [cmap_2020_shop2_nonredundant_unfiltered(size) for size in normalized_sizes_2020_shop2_nonredundant_unfiltered]

# Drawing the network with node sizes and colors
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2020_shop2_nonredundant_unfiltered, pos_2020_shop2_nonredundant_unfiltered, node_size=node_sizes_2020_shop2_nonredundant_unfiltered, node_color=node_colors_2020_shop2_nonredundant_unfiltered, alpha=0.8)
nx.draw_networkx_edges(G_2020_shop2_nonredundant_unfiltered, pos_2020_shop2_nonredundant_unfiltered, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2020_shop2_nonredundant_unfiltered, pos_2020_shop2_nonredundant_unfiltered, font_size=10)

plt.title('Node Size Based on Indegree for 2020 Shop 2')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2020_shop2_lift = nx.DiGraph()

lift_summary_2020_shop2 = {}

for _, row in rules_2020_shop2_nonredundant_unfiltered.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2020_shop2_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Update lift summary for consequents
    if consequents in lift_summary_2020_shop2:
        lift_summary_2020_shop2[consequents]['total_lift'] += row['lift']
        lift_summary_2020_shop2[consequents]['count'] += 1
    else:
        lift_summary_2020_shop2[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculate average lift for each consequent
for item in lift_summary_2020_shop2:
    lift_summary_2020_shop2[item]['avg_lift'] = lift_summary_2020_shop2[item]['total_lift'] / lift_summary_2020_shop2[item]['count']

# Node sizes based on average lift, scaled appropriately
node_sizes_2020_shop2_lift = [lift_summary_2020_shop2[node]['avg_lift'] * 500 if node in lift_summary_2020_shop2 else 1000 for node in G_2020_shop2_lift.nodes()]

pos_2020_shop2_lift = nx.spring_layout(G_2020_shop2_lift)

# Use a colormap that transitions from orange to red
cmap_2020_shop2_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2020_shop2_lift = np.array(node_sizes_2020_shop2_lift)
normalized_sizes_2020_shop2_lift = (normalized_sizes_2020_shop2_lift - min(normalized_sizes_2020_shop2_lift)) / (max(normalized_sizes_2020_shop2_lift) - min(normalized_sizes_2020_shop2_lift))
node_colors_2020_shop2_lift = [cmap_2020_shop2_lift(size) for size in normalized_sizes_2020_shop2_lift]

plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2020_shop2_lift, pos_2020_shop2_lift, node_size=node_sizes_2020_shop2_lift, node_color=node_colors_2020_shop2_lift, alpha=0.8)
nx.draw_networkx_edges(G_2020_shop2_lift, pos_2020_shop2_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2020_shop2_lift, pos_2020_shop2_lift, font_size=10)

plt.title('Node Size Based on Average Lift for 2020 Shop 2')
plt.axis('off')
plt.show()


# Based on conviction

In [ ]:
rules_2020_shop2_nonredundant.head()

In [ ]:
G_2020_shop2_conviction = nx.DiGraph()

conviction_summary_2020_shop2 = {}
inf_rules = set()  # To store nodes from rules with conviction of inf

# Assuming 'rules_2020_shop2_nonredundant_unfiltered' contains a 'conviction' column
for index, row in rules_2020_shop2_nonredundant_unfiltered.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2020_shop2_conviction.add_edge(antecedents, consequents, weight=row['confidence'], conviction=row['conviction'])
    
    if row['conviction'] == float('inf') and index < 126:  # Mark nodes with inf conviction among the first 9
        inf_rules.add(consequents)
    else:
        # Aggregate conviction values excluding inf convictions
        if consequents in conviction_summary_2020_shop2:
            conviction_summary_2020_shop2[consequents] += row['conviction']
        else:
            conviction_summary_2020_shop2[consequents] = row['conviction']

# Creating a custom colormap from white (low conviction) to red (high conviction) through orange
cmap = mcolors.LinearSegmentedColormap.from_list("custom_cmap", ["white", "orange", "red"])

# Normalize conviction values for sizing and coloring, excluding inf conviction values
max_conviction = max(conviction_summary_2020_shop2.values(), default=1)
min_conviction = min(conviction_summary_2020_shop2.values(), default=0)
conviction_range = max_conviction - min_conviction

node_colors = []
node_sizes = []

for node in G_2020_shop2_conviction.nodes():
    if node in inf_rules:
        # Special handling for nodes with inf conviction
        node_colors.append('purple')  # Mark these nodes with purple
        node_sizes.append(2000)  # Assign a larger size
    else:
        conviction = conviction_summary_2020_shop2.get(node, min_conviction)
        normalized_conviction = (conviction - min_conviction) / conviction_range if conviction_range else 0
        # Map normalized conviction to the custom color scale
        node_colors.append(cmap(normalized_conviction))
        node_sizes.append(normalized_conviction * 5000 + 100)

pos = nx.spring_layout(G_2020_shop2_conviction)

# Create a plot figure with specified size
plt.figure(figsize=(30, 32))

# Set up the spring layout positions
pos = nx.spring_layout(G_2020_shop2_conviction, seed=42)

# Draw network components
nodes = nx.draw_networkx_nodes(G_2020_shop2_conviction, pos, node_color=node_colors, node_size=node_sizes, cmap=cmap, alpha=0.8)
edges = nx.draw_networkx_edges(G_2020_shop2_conviction, pos, arrowstyle='->', arrowsize=10, edge_color='grey', alpha=0.5)
labels = nx.draw_networkx_labels(G_2020_shop2_conviction, pos, font_size=8)

# Set up the ScalarMappable with the normalized range and colormap
norm = plt.Normalize(vmin=min_conviction, vmax=max_conviction)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])  # You might not need this line as we're going to use the ScalarMappable

# Create an axes on the right side of ax. The width of cax will be 5%
# of ax and the padding between cax and ax will be fixed at 0.05 inch.
from mpl_toolkits.axes_grid1 import make_axes_locatable
ax = plt.gca()  # Get the current axes
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

# Create the color bar
plt.colorbar(sm, cax=cax, orientation='vertical')
cax.set_ylabel('Normalized Conviction', size=12)
cax.tick_params(labelsize=10)

# Set plot title and turn off axis visibility
plt.title('Network Graph of 2020 Shop 2 Based on Conviction')
plt.axis('off')
plt.show()

# Just a Test to erase all the INF convictions

Very intense cut down for redundancy, because 126 rules had a conviction of INF and confidence of 100%.
Maybe we can see something on this graph

In [ ]:
G_2020_shop2_conviction = nx.DiGraph()

conviction_summary_2020_shop2 = {}
inf_rules = set()  # To store nodes from rules with conviction of inf

# Assuming 'rules_2020_shop2_nonredundant_unfiltered' contains a 'conviction' column
for index, row in sorted_by_conviction_df.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2020_shop2_conviction.add_edge(antecedents, consequents, weight=row['confidence'], conviction=row['conviction'])
    
    if row['conviction'] == float('inf') and index < 30:  # Mark nodes with inf conviction among the first 9
        inf_rules.add(consequents)
    else:
        # Aggregate conviction values excluding inf convictions
        if consequents in conviction_summary_2020_shop2:
            conviction_summary_2020_shop2[consequents] += row['conviction']
        else:
            conviction_summary_2020_shop2[consequents] = row['conviction']

# Creating a custom colormap from white (low conviction) to red (high conviction) through orange
cmap = mcolors.LinearSegmentedColormap.from_list("custom_cmap", ["white", "orange", "red"])

# Normalize conviction values for sizing and coloring, excluding inf conviction values
max_conviction = max(conviction_summary_2020_shop2.values(), default=1)
min_conviction = min(conviction_summary_2020_shop2.values(), default=0)
conviction_range = max_conviction - min_conviction

node_colors = []
node_sizes = []

for node in G_2020_shop2_conviction.nodes():
    if node in inf_rules:
        # Special handling for nodes with inf conviction
        node_colors.append('purple')  # Mark these nodes with purple
        node_sizes.append(2000)  # Assign a larger size
    else:
        conviction = conviction_summary_2020_shop2.get(node, min_conviction)
        normalized_conviction = (conviction - min_conviction) / conviction_range if conviction_range else 0
        # Map normalized conviction to the custom color scale
        node_colors.append(cmap(normalized_conviction))
        node_sizes.append(normalized_conviction * 5000 + 100)

pos = nx.spring_layout(G_2020_shop2_conviction)

# Create a plot figure with specified size
plt.figure(figsize=(30, 32))

# Set up the spring layout positions
pos = nx.spring_layout(G_2020_shop2_conviction, seed=42)

# Draw network components
nodes = nx.draw_networkx_nodes(G_2020_shop2_conviction, pos, node_color=node_colors, node_size=node_sizes, cmap=cmap, alpha=0.8)
edges = nx.draw_networkx_edges(G_2020_shop2_conviction, pos, arrowstyle='->', arrowsize=10, edge_color='grey', alpha=0.5)
labels = nx.draw_networkx_labels(G_2020_shop2_conviction, pos, font_size=8)

# Set up the ScalarMappable with the normalized range and colormap
norm = plt.Normalize(vmin=min_conviction, vmax=max_conviction)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])  # You might not need this line as we're going to use the ScalarMappable

# Create an axes on the right side of ax. The width of cax will be 5%
# of ax and the padding between cax and ax will be fixed at 0.05 inch.
from mpl_toolkits.axes_grid1 import make_axes_locatable
ax = plt.gca()  # Get the current axes
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

# Create the color bar
plt.colorbar(sm, cax=cax, orientation='vertical')
cax.set_ylabel('Normalized Conviction', size=12)
cax.tick_params(labelsize=10)

# Set plot title and turn off axis visibility
plt.title('Network Graph of 2020 Shop 2 Based on Conviction')
plt.axis('off')
plt.show()

In [ ]:
print(max_conviction)

# Filtered top 30 and non redundant

In [ ]:
# Assuming rules_1992_shop2_nonredundant is your sorted rules DataFrame for shop2
rules_2020_shop2_nonredundant_top30 = rules_2020_shop2_nonredundant.copy(deep=True)

In [ ]:
# Select the top 30 rules based on confidence or another criterion
rules_2020_shop2_nonredundant_top30 = rules_2020_shop2_nonredundant_top30.sort_values(by='confidence', ascending=False).head(30)

In [ ]:
rules_2020_shop2_nonredundant_top30.info()

In [ ]:
rules_2020_shop2_nonredundant_top30['LHS'] = [','.join(list(x)) for x in rules_2020_shop2_nonredundant_top30['antecedents']]
rules_2020_shop2_nonredundant_top30['RHS'] = [','.join(list(x)) for x in rules_2020_shop2_nonredundant_top30['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2020_shop2_nonredundant_top30.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2020_shop2_nonredundant_top30, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2020 Shop 2")
plt.axis('off')
plt.show()


## Based on Indegree

In [ ]:
G_2020_shop2 = nx.DiGraph()

# Assuming 'rules_sorted_2020_shop2' contains your association rules for shop 2 in 2020
for _, row in rules_2020_shop2_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as the weight
    G_2020_shop2.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2020_shop2 = nx.spring_layout(G_2020_shop2)

# Calculate node sizes based on indegree
in_degrees_2020_shop2 = dict(G_2020_shop2.in_degree())
node_sizes_2020_shop2 = [in_degrees_2020_shop2[node] * 100 + 1000 for node in G_2020_shop2.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2020_shop2 = (np.array(node_sizes_2020_shop2) - np.min(node_sizes_2020_shop2)) / (np.max(node_sizes_2020_shop2) - np.min(node_sizes_2020_shop2))

# Define a colormap from orange to red
cmap_2020_shop2 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2020_shop2 = [cmap_2020_shop2(size) for size in normalized_sizes_2020_shop2]

# Drawing the network
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2020_shop2, pos_2020_shop2, node_size=node_sizes_2020_shop2, node_color=node_colors_2020_shop2, alpha=0.8)
nx.draw_networkx_edges(G_2020_shop2, pos_2020_shop2, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2020_shop2, pos_2020_shop2, font_size=8)

plt.title('Top 5 Association Rules Network for 2020 Shop 2 with Color Gradient')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2020_shop2_lift = nx.DiGraph()

lift_summary_2020_shop2 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2020_shop2_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2020_shop2_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2020_shop2:
        lift_summary_2020_shop2[consequents]['total_lift'] += row['lift']
        lift_summary_2020_shop2[consequents]['count'] += 1
    else:
        lift_summary_2020_shop2[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2020_shop2:
    lift_summary_2020_shop2[item]['avg_lift'] = lift_summary_2020_shop2[item]['total_lift'] / lift_summary_2020_shop2[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2020_shop2_lift = [lift_summary_2020_shop2[node]['avg_lift'] * 500 if node in lift_summary_2020_shop2 else 1000 for node in G_2020_shop2_lift.nodes()]

pos_2020_shop2_lift = nx.spring_layout(G_2020_shop2_lift)

# Colormap from orange to red
cmap_2020_shop2_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2020_shop2_lift = np.array(node_sizes_2020_shop2_lift)
normalized_sizes_2020_shop2_lift = (normalized_sizes_2020_shop2_lift - min(normalized_sizes_2020_shop2_lift)) / (max(normalized_sizes_2020_shop2_lift) - min(normalized_sizes_2020_shop2_lift))
node_colors_2020_shop2_lift = [cmap_2020_shop2_lift(size) for size in normalized_sizes_2020_shop2_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2020_shop2_lift, pos_2020_shop2_lift, node_size=node_sizes_2020_shop2_lift, node_color=node_colors_2020_shop2_lift, alpha=0.8)
nx.draw_networkx_edges(G_2020_shop2_lift, pos_2020_shop2_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2020_shop2_lift, pos_2020_shop2_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2020 Shop 2 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2020_shop2_lift = nx.DiGraph()

lift_summary_2020_shop2 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2020_shop2_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2020_shop2_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2020_shop2:
        lift_summary_2020_shop2[consequents]['total_lift'] += row['lift']
        lift_summary_2020_shop2[consequents]['count'] += 1
    else:
        lift_summary_2020_shop2[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2020_shop2:
    lift_summary_2020_shop2[item]['avg_lift'] = lift_summary_2020_shop2[item]['total_lift'] / lift_summary_2020_shop2[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2020_shop2_lift = [lift_summary_2020_shop2[node]['avg_lift'] * 500 if node in lift_summary_2020_shop2 else 1000 for node in G_2020_shop2_lift.nodes()]

pos_2020_shop2_lift = nx.spring_layout(G_2020_shop2_lift)

# Colormap from orange to red
cmap_2020_shop2_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2020_shop2_lift = np.array(node_sizes_2020_shop2_lift)
normalized_sizes_2020_shop2_lift = (normalized_sizes_2020_shop2_lift - min(normalized_sizes_2020_shop2_lift)) / (max(normalized_sizes_2020_shop2_lift) - min(normalized_sizes_2020_shop2_lift))
node_colors_2020_shop2_lift = [cmap_2020_shop2_lift(size) for size in normalized_sizes_2020_shop2_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2020_shop2_lift, pos_2020_shop2_lift, node_size=node_sizes_2020_shop2_lift, node_color=node_colors_2020_shop2_lift, alpha=0.8)
nx.draw_networkx_edges(G_2020_shop2_lift, pos_2020_shop2_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2020_shop2_lift, pos_2020_shop2_lift, font_size=10)

plt.title('Top 5 Association Rules Network for 2020 Shop 2 Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2020_shop2 = nx.DiGraph()

# Assuming 'rules_2020_shop2_nonredundant' contains association rules for 2020 Shop 2
for _, row in rules_2020_shop2_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2020_shop2.add_edge(antecedents, consequents, confidence=row['confidence'])

# Calculate node sizes based on indegree
in_degrees_2020_shop2 = dict(G_2020_shop2.in_degree())
node_sizes_2020_shop2 = [in_degrees_2020_shop2[node] * 100 + 1000 for node in G_2020_shop2.nodes()]

# Edge width based on confidence
edge_widths = [G_2020_shop2[u][v]['confidence'] * 15 for u, v in G_2020_shop2.edges()]

# Assuming normalized sizes and color mapping is desired
# This is just a placeholder; you need to define the logic for normalization
normalized_sizes_2020_shop2 = np.array(list(in_degrees_2020_shop2.values()))
normalized_sizes_2020_shop2 = (normalized_sizes_2020_shop2 - min(normalized_sizes_2020_shop2)) / (max(normalized_sizes_2020_shop2) - min(normalized_sizes_2020_shop2))
cmap_2020_shop2 = mcolors.LinearSegmentedColormap.from_list("MyCMapName", ["blue", "red"])

# Node colors based on a property (here, using indegree for example)
node_colors_2020_shop2 = [cmap_2020_shop2(size) for size in normalized_sizes_2020_shop2]

# Layout
pos = nx.spring_layout(G_2020_shop2, seed=42)

# Drawing
plt.figure(figsize=(20, 15))
nx.draw_networkx_edges(G_2020_shop2, pos, width=edge_widths, edge_color='grey', alpha=0.5)
nx.draw_networkx_nodes(G_2020_shop2, pos, node_size=node_sizes_2020_shop2, node_color=node_colors_2020_shop2, alpha=0.8)
nx.draw_networkx_labels(G_2020_shop2, pos, font_size=8, font_color='black')
plt.title('Network Graph of Association Rules Based on Confidence for 2020 Shop 2')
plt.axis('off')
plt.show()


In [ ]:
#the node sizes are based on the indegree where each node acts as a consequent,

In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2020_shop2_nonredundant_top30.loc[:, 'antecedents_'] = rules_2020_shop2_nonredundant_top30['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2020_shop2_nonredundant_top30.loc[:, 'consequents_'] = rules_2020_shop2_nonredundant_top30['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2020_shop2_nonredundant_top30, x='consequents_', y='antecedents_', s=rules_2020_shop2_nonredundant_top30['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2020_shop2_nonredundant_top30.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


# Shop3 2021 SEARCH shop3_1

In [ ]:
# Filter for records from shop ID 1 and the year 1993
sales_data_2021_shop3 = sales_data_shop3[(sales_data_shop3['trx_date'].dt.year == 2021)]

In [ ]:
sales_data_2021_shop3.head()

In [ ]:
# Grouping product codes by trx_id
transactions_2021_shop3 = sales_data_2021_shop3.groupby('trx_id')['product_code'].apply(list).tolist()

In [ ]:
# Encode transaction data for shop3
encoder_2021_shop3 = TransactionEncoder()
trans_array_2021_shop3 = encoder_2021_shop3.fit(transactions_2021_shop3).transform(transactions_2021_shop3)
trans_df_2021_shop3 = pd.DataFrame(trans_array_2021_shop3, columns=encoder_2021_shop3.columns_)

In [ ]:
# Find frequent itemsets
frequent_itemsets_2021_shop3 = apriori(trans_df_2021_shop3, min_support=0.001, use_colnames=True)

In [ ]:
print(frequent_itemsets_2021_shop3)

In [ ]:
# Generate association rules by confidence for shop3
rulesConfidence_2021_shop3 = association_rules(frequent_itemsets_2021_shop3, metric="confidence", min_threshold=0.10)
rulesConfidence_2021_shop3.sort_values(by='confidence', ascending=False, inplace=True)


In [ ]:
# Replace frozen sets with strings
rulesConfidence_2021_shop3['antecedents_'] = rulesConfidence_2021_shop3['antecedents'].apply(lambda a: ','.join(list(a)))
rulesConfidence_2021_shop3['consequents_'] = rulesConfidence_2021_shop3['consequents'].apply(lambda a: ','.join(list(a)))

# Draw
fig, ax = plt.subplots(figsize=(14,30))
ax = plt.scatter(data=rulesConfidence_2021_shop3, x='consequents_', y='antecedents_', s=rulesConfidence_2021_shop3['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rulesConfidence_2021_shop3.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2020 Shop 3", fontsize=14)
plt.xlabel("Consequents (RHS)", fontsize=12)
plt.ylabel("Antecedents (LHS)", fontsize=12)
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate(rotation=45)
plt.tight_layout()

plt.show()


## Eliminate Redundant Rules

In [ ]:
rulesConfidence_2021_shop3.head()

In [ ]:
rulesConfidence_2021_shop3.info()

In [ ]:
# If 'antecedents' and 'consequents' are not frozensets, convert them
rules_2021_shop3_redundant = rulesConfidence_2021_shop3.copy()
rules_2021_shop3_redundant['antecedents'] = rules_2021_shop3_redundant['antecedents'].apply(lambda x: frozenset(x))
rules_2021_shop3_redundant['consequents'] = rules_2021_shop3_redundant['consequents'].apply(lambda x: frozenset(x))


In [ ]:
# Create a copy of the DataFrame to avoid modifying the original while iterating
rules_2021_shop3_checkredundance = rules_2021_shop3_redundant.copy()

# List to hold non-redundant rules
non_redundant_rules_2021_shop3 = []

for _, rule in rules_2021_shop3_checkredundance.iterrows():
    if not is_redundant(rule, rules_2021_shop3_checkredundance):
        non_redundant_rules_2021_shop3.append(rule)

# Convert the list back to a DataFrame
rules_2021_shop3_nonredundant = pd.DataFrame(non_redundant_rules_2021_shop3).reset_index(drop=True)


In [ ]:
print(rules_2021_shop3_nonredundant.head())
print(f"Original number of rules: {len(rules_2021_shop3_redundant)}")
print(f"Number of non-redundant rules: {len(rules_2021_shop3_nonredundant)}")


In [ ]:
rules_2021_shop3_nonredundant.head()

In [ ]:
rules_2021_shop3_redundant.info()

In [ ]:
rules_2021_shop3_nonredundant.head()

In [ ]:
rules_2021_shop3_nonredundant_backup = rules_2021_shop3_nonredundant.copy()

In [ ]:
rules_2021_shop3_nonredundant_unfiltered = rules_2021_shop3_nonredundant.copy()

In [ ]:
#reset
#rules_2021_shop3_nonredundant = rules_2021_shop3_nonredundant_backup.copy()

# Redundant and Unfiltered

In [ ]:
rules_2021_shop3_redundant.head()

In [ ]:
# Assuming 'rules_sorted_2022_shop4' is correctly defined and includes 'antecedents', 'consequents', and 'confidence'
G_2021_shop3 = nx.DiGraph()

# Iterate through rules for 2021 Shop 3 and add edges to the graph
for _, row in rules_2021_shop3_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as weight
    G_2021_shop3.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2021_shop3 = nx.spring_layout(G_2021_shop3)

# Recalculate node sizes based on indegree or another metric
in_degrees_2021_shop3 = dict(G_2021_shop3.in_degree())
node_sizes_2021_shop3 = [in_degrees_2021_shop3[node] * 100 + 1000 for node in G_2021_shop3.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2021_shop3 = (np.array(node_sizes_2021_shop3) - np.min(node_sizes_2021_shop3)) / (np.max(node_sizes_2021_shop3) - np.min(node_sizes_2021_shop3))

# Define a colormap from orange to red for visual consistency
cmap_2021_shop3 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2021_shop3 = [cmap_2021_shop3(size) for size in normalized_sizes_2021_shop3]

# Drawing the network
plt.figure(figsize=(60, 43))
nx.draw_networkx_nodes(G_2021_shop3, pos_2021_shop3, node_size=node_sizes_2021_shop3, node_color=node_colors_2021_shop3, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop3, pos_2021_shop3, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2021_shop3, pos_2021_shop3, font_size=10)

plt.title('Top 5 Association Rules Network for 2021 Shop 3 with Color Gradient')
plt.axis('off')
plt.show()


### Based on LIFT

In [ ]:
G_2021_shop3_lift = nx.DiGraph()

lift_summary_2021_shop3 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2021_shop3_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop3_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2021_shop3:
        lift_summary_2021_shop3[consequents]['total_lift'] += row['lift']
        lift_summary_2021_shop3[consequents]['count'] += 1
    else:
        lift_summary_2021_shop3[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2021_shop3:
    lift_summary_2021_shop3[item]['avg_lift'] = lift_summary_2021_shop3[item]['total_lift'] / lift_summary_2021_shop3[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2021_shop3_lift = [lift_summary_2021_shop3[node]['avg_lift'] * 500 if node in lift_summary_2021_shop3 else 1000 for node in G_2021_shop3_lift.nodes()]

pos_2021_shop3_lift = nx.spring_layout(G_2021_shop3_lift)

# Colormap from orange to red
cmap_2021_shop3_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2021_shop3_lift = np.array(node_sizes_2021_shop3_lift)
normalized_sizes_2021_shop3_lift = (normalized_sizes_2021_shop3_lift - min(normalized_sizes_2021_shop3_lift)) / (max(normalized_sizes_2021_shop3_lift) - min(normalized_sizes_2021_shop3_lift))
node_colors_2021_shop3_lift = [cmap_2021_shop3_lift(size) for size in normalized_sizes_2021_shop3_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2021_shop3_lift, pos_2021_shop3_lift, node_size=node_sizes_2021_shop3_lift, node_color=node_colors_2021_shop3_lift, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop3_lift, pos_2021_shop3_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2021_shop3_lift, pos_2021_shop3_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2021 Shop 3 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


# nonredundant_unfiltered

## Based on Indegree

In [ ]:
G_2021_shop3_nonredundant_unfiltered = nx.DiGraph()

# Iterating through each row to add edges with confidence as an attribute
for _, row in rules_2021_shop3_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop3_nonredundant_unfiltered.add_edge(antecedents, consequents, weight=row['confidence'])

# Defining the layout for network visualization
pos_2021_shop3_nonredundant_unfiltered = nx.spring_layout(G_2021_shop3_nonredundant_unfiltered)

# Calculating node sizes based on the indegree for color mapping
in_degrees_2021_shop3_nonredundant_unfiltered = dict(G_2021_shop3_nonredundant_unfiltered.in_degree())
node_sizes_2021_shop3_nonredundant_unfiltered = [in_degrees_2021_shop3_nonredundant_unfiltered[node] * 100 + 1000 for node in G_2021_shop3_nonredundant_unfiltered.nodes()]

# Normalizing node sizes for color mapping
normalized_sizes_2021_shop3_nonredundant_unfiltered = (np.array(node_sizes_2021_shop3_nonredundant_unfiltered) - min(node_sizes_2021_shop3_nonredundant_unfiltered)) / (max(node_sizes_2021_shop3_nonredundant_unfiltered) - min(node_sizes_2021_shop3_nonredundant_unfiltered))

# Defining a color map from orange to red
cmap_2021_shop3_nonredundant_unfiltered = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])

# Mapping normalized node sizes to colors
node_colors_2021_shop3_nonredundant_unfiltered = [cmap_2021_shop3_nonredundant_unfiltered(size) for size in normalized_sizes_2021_shop3_nonredundant_unfiltered]

# Drawing the network with node sizes and colors
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2021_shop3_nonredundant_unfiltered, pos_2021_shop3_nonredundant_unfiltered, node_size=node_sizes_2021_shop3_nonredundant_unfiltered, node_color=node_colors_2021_shop3_nonredundant_unfiltered, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop3_nonredundant_unfiltered, pos_2021_shop3_nonredundant_unfiltered, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2021_shop3_nonredundant_unfiltered, pos_2021_shop3_nonredundant_unfiltered, font_size=10)

plt.title('Node Size Based on Indegree for 2021 Shop 3')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2021_shop3_lift = nx.DiGraph()

lift_summary_2021_shop3 = {}

# Iterate through each row to add edges with lift as an attribute
for _, row in rules_2021_shop3_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop3_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Update lift summary for consequents
    if consequents in lift_summary_2021_shop3:
        lift_summary_2021_shop3[consequents]['total_lift'] += row['lift']
        lift_summary_2021_shop3[consequents]['count'] += 1
    else:
        lift_summary_2021_shop3[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculate average lift for each consequent
for item in lift_summary_2021_shop3:
    lift_summary_2021_shop3[item]['avg_lift'] = lift_summary_2021_shop3[item]['total_lift'] / lift_summary_2021_shop3[item]['count']

# Node sizes based on average lift, scaled appropriately
node_sizes_2021_shop3_lift = [lift_summary_2021_shop3[node]['avg_lift'] * 500 if node in lift_summary_2021_shop3 else 1000 for node in G_2021_shop3_lift.nodes()]

pos_2021_shop3_lift = nx.spring_layout(G_2021_shop3_lift)

# Use a colormap that transitions from orange to red
cmap_2021_shop3_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2021_shop3_lift = np.array(node_sizes_2021_shop3_lift)
normalized_sizes_2021_shop3_lift = (normalized_sizes_2021_shop3_lift - min(normalized_sizes_2021_shop3_lift)) / (max(normalized_sizes_2021_shop3_lift) - min(normalized_sizes_2021_shop3_lift))
node_colors_2021_shop3_lift = [cmap_2021_shop3_lift(size) for size in normalized_sizes_2021_shop3_lift]

plt.figure(figsize=(50, 34))
nx.draw_networkx_nodes(G_2021_shop3_lift, pos_2021_shop3_lift, node_size=node_sizes_2021_shop3_lift, node_color=node_colors_2021_shop3_lift, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop3_lift, pos_2021_shop3_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2021_shop3_lift, pos_2021_shop3_lift, font_size=10)

plt.title('Node Size Based on Average Lift for 2021 Shop 3')
plt.axis('off')
plt.show()


# Based on conviction

In [ ]:
rules_2021_shop3_nonredundant.head()

In [ ]:
G_2021_shop3_conviction = nx.DiGraph()

conviction_summary_2021_shop3 = {}
count_summary_2021_shop3 = {}

# Assuming 'rules_2021_shop3_nonredundant_unfiltered' contains a 'conviction' column
for _, row in rules_2021_shop3_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop3_conviction.add_edge(antecedents, consequents, weight=row['confidence'], conviction=row['conviction'])
    
    # Aggregate conviction values and count occurrences
    if consequents in conviction_summary_2021_shop3:
        conviction_summary_2021_shop3[consequents] += row['conviction']
        count_summary_2021_shop3[consequents] += 1
    else:
        conviction_summary_2021_shop3[consequents] = row['conviction']
        count_summary_2021_shop3[consequents] = 1

# Calculate average conviction values
average_conviction_summary_2021_shop3 = {k: conviction_summary_2021_shop3[k] / count_summary_2021_shop3[k] for k in conviction_summary_2021_shop3}

# Normalize conviction values for sizing and coloring
max_conviction = max(average_conviction_summary_2021_shop3.values())
min_conviction = min(average_conviction_summary_2021_shop3.values())
conviction_range = max_conviction - min_conviction
node_colors = []
node_sizes = []

for node in G_2021_shop3_conviction.nodes():
    # Get the average conviction value for the node, defaulting to min if not present
    conviction = average_conviction_summary_2021_shop3.get(node, min_conviction)
    # Normalize between 0 and 1
    normalized_conviction = (conviction - min_conviction) / conviction_range if conviction_range else 0
    node_colors.append(normalized_conviction)
    node_sizes.append(normalized_conviction * 1000 + 100)  # Scale and offset size

# Use a colormap for coloring nodes based on conviction
cmap = plt.cm.coolwarm

pos = nx.spring_layout(G_2021_shop3_conviction)

plt.figure(figsize=(40, 22))
nodes = nx.draw_networkx_nodes(G_2021_shop3_conviction, pos, node_color=node_colors, node_size=node_sizes, cmap=cmap, alpha=0.8)
edges = nx.draw_networkx_edges(G_2021_shop3_conviction, pos, arrowstyle='->', arrowsize=10, edge_color='grey', alpha=0.5)
labels = nx.draw_networkx_labels(G_2021_shop3_conviction, pos, font_size=8)

plt.colorbar(nodes, label='Normalized Conviction')
plt.title('Network Graph of 2021 Shop 3 Based on Conviction')
plt.axis('off')
plt.show()

In [ ]:
print(max_conviction)

# Filtered top 30 and non redundant

In [ ]:
rules_2021_shop3_nonredundant.head()

In [ ]:
# Assuming rules_1992_shop2_nonredundant is your sorted rules DataFrame for shop2
rules_2021_shop3_nonredundant_top30 = rules_2021_shop3_nonredundant.copy(deep=True)

In [ ]:
rules_2021_shop3_nonredundant_top30_lift = rules_2021_shop3_nonredundant.copy(deep=True)

In [ ]:
# Select the top 30 rules based on lift or another criterion
rules_2021_shop3_nonredundant_top30_lift = rules_2021_shop3_nonredundant_top30_lift.sort_values(by='lift', ascending=False).head(30)

In [ ]:
rules_2021_shop3_nonredundant_top30_lift.head()

In [ ]:
# Select the top 30 rules based on confidence or another criterion
rules_2021_shop3_nonredundant_top30 = rules_2021_shop3_nonredundant_top30.sort_values(by='confidence', ascending=False).head(30)

In [ ]:
rules_2021_shop3_nonredundant_top30.info()

In [ ]:
rules_2021_shop3_nonredundant_top30['LHS'] = [','.join(list(x)) for x in rules_2021_shop3_nonredundant_top30['antecedents']]
rules_2021_shop3_nonredundant_top30['RHS'] = [','.join(list(x)) for x in rules_2021_shop3_nonredundant_top30['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2021_shop3_nonredundant_top30.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2021_shop3_nonredundant_top30, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2021 Shop 3")
plt.axis('off')
plt.show()


## Based on Indegree

In [ ]:
G_2021_shop3 = nx.DiGraph()

# Assuming 'rules_sorted_2021_shop3' contains association rules for shop 3 in 2021
for _, row in rules_2021_shop3_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as the weight
    G_2021_shop3.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2021_shop3 = nx.spring_layout(G_2021_shop3)

# Calculate node sizes based on indegree
in_degrees_2021_shop3 = dict(G_2021_shop3.in_degree())
node_sizes_2021_shop3 = [in_degrees_2021_shop3[node] * 100 + 1000 for node in G_2021_shop3.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2021_shop3 = (np.array(node_sizes_2021_shop3) - np.min(node_sizes_2021_shop3)) / (np.max(node_sizes_2021_shop3) - np.min(node_sizes_2021_shop3))

# Define a colormap from orange to red
cmap_2021_shop3 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2021_shop3 = [cmap_2021_shop3(size) for size in normalized_sizes_2021_shop3]

# Drawing the network
plt.figure(figsize=(40, 34))
nx.draw_networkx_nodes(G_2021_shop3, pos_2021_shop3, node_size=node_sizes_2021_shop3, node_color=node_colors_2021_shop3, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop3, pos_2021_shop3, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2021_shop3, pos_2021_shop3, font_size=8)

plt.title('Top 5 Association Rules Network for 2021 Shop 3 with Color Gradient')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2021_shop3_lift = nx.DiGraph()

lift_summary_2021_shop3 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2021_shop3_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop3_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2021_shop3:
        lift_summary_2021_shop3[consequents]['total_lift'] += row['lift']
        lift_summary_2021_shop3[consequents]['count'] += 1
    else:
        lift_summary_2021_shop3[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2021_shop3:
    lift_summary_2021_shop3[item]['avg_lift'] = lift_summary_2021_shop3[item]['total_lift'] / lift_summary_2021_shop3[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2021_shop3_lift = [lift_summary_2021_shop3[node]['avg_lift'] * 500 if node in lift_summary_2021_shop3 else 1000 for node in G_2021_shop3_lift.nodes()]

pos_2021_shop3_lift = nx.spring_layout(G_2021_shop3_lift)

# Colormap from orange to red
cmap_2021_shop3_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2021_shop3_lift = np.array(node_sizes_2021_shop3_lift)
normalized_sizes_2021_shop3_lift = (normalized_sizes_2021_shop3_lift - min(normalized_sizes_2021_shop3_lift)) / (max(normalized_sizes_2021_shop3_lift) - min(normalized_sizes_2021_shop3_lift))
node_colors_2021_shop3_lift = [cmap_2021_shop3_lift(size) for size in normalized_sizes_2021_shop3_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(40, 22))
nx.draw_networkx_nodes(G_2021_shop3_lift, pos_2021_shop3_lift, node_size=node_sizes_2021_shop3_lift, node_color=node_colors_2021_shop3_lift, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop3_lift, pos_2021_shop3_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2021_shop3_lift, pos_2021_shop3_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2021 Shop 3 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2021_shop3_lift = nx.DiGraph()

lift_summary_2021_shop3 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2021_shop3_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop3_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2021_shop3:
        lift_summary_2021_shop3[consequents]['total_lift'] += row['lift']
        lift_summary_2021_shop3[consequents]['count'] += 1
    else:
        lift_summary_2021_shop3[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2021_shop3:
    lift_summary_2021_shop3[item]['avg_lift'] = lift_summary_2021_shop3[item]['total_lift'] / lift_summary_2021_shop3[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2021_shop3_lift = [lift_summary_2021_shop3[node]['avg_lift'] * 500 if node in lift_summary_2021_shop3 else 1000 for node in G_2021_shop3_lift.nodes()]

pos_2021_shop3_lift = nx.spring_layout(G_2021_shop3_lift)

# Colormap from orange to red
cmap_2021_shop3_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2021_shop3_lift = np.array(node_sizes_2021_shop3_lift)
normalized_sizes_2021_shop3_lift = (normalized_sizes_2021_shop3_lift - min(normalized_sizes_2021_shop3_lift)) / (max(normalized_sizes_2021_shop3_lift) - min(normalized_sizes_2021_shop3_lift))
node_colors_2021_shop3_lift = [cmap_2021_shop3_lift(size) for size in normalized_sizes_2021_shop3_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2021_shop3_lift, pos_2021_shop3_lift, node_size=node_sizes_2021_shop3_lift, node_color=node_colors_2021_shop3_lift, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop3_lift, pos_2021_shop3_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2021_shop3_lift, pos_2021_shop3_lift, font_size=10)

plt.title('Top 5 Association Rules Network for 2021 Shop 3 Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2021_shop3 = nx.DiGraph()

# Assuming 'rules_2021_shop3_nonredundant' contains association rules for 2021 Shop 3
for _, row in rules_2021_shop3_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop3.add_edge(antecedents, consequents, confidence=row['confidence'])

# Calculate node sizes based on indegree
in_degrees_2021_shop3 = dict(G_2021_shop3.in_degree())
node_sizes_2021_shop3 = [in_degrees_2021_shop3[node] * 100 + 1000 for node in G_2021_shop3.nodes()]

# Edge width based on confidence
edge_widths = [G_2021_shop3[u][v]['confidence'] * 15 for u, v in G_2021_shop3.edges()]

# Assuming normalized sizes and color mapping is desired
normalized_sizes_2021_shop3 = np.array(list(in_degrees_2021_shop3.values()))
normalized_sizes_2021_shop3 = (normalized_sizes_2021_shop3 - min(normalized_sizes_2021_shop3)) / (max(normalized_sizes_2021_shop3) - min(normalized_sizes_2021_shop3))
cmap_2021_shop3 = mcolors.LinearSegmentedColormap.from_list("MyCMapName", ["blue", "red"])

# Node colors based on a property (here, using indegree for example)
node_colors_2021_shop3 = [cmap_2021_shop3(size) for size in normalized_sizes_2021_shop3]

# Layout
pos = nx.spring_layout(G_2021_shop3, seed=42)

# Drawing
plt.figure(figsize=(40, 30))
nx.draw_networkx_edges(G_2021_shop3, pos, width=edge_widths, edge_color='grey', alpha=0.5)
nx.draw_networkx_nodes(G_2021_shop3, pos, node_size=node_sizes_2021_shop3, node_color=node_colors_2021_shop3, alpha=0.8)
nx.draw_networkx_labels(G_2021_shop3, pos, font_size=8, font_color='black')
plt.title('Network Graph of Association Rules Based on Confidence for 2021 Shop 3')
plt.axis('off')
plt.show()

In [ ]:
G_2021_shop3 = nx.DiGraph()

convictions = []  # To store conviction values for normalization

for _, row in rules_2021_shop3_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    conviction = row['conviction']  # Assuming each row has a 'conviction' column
    convictions.append(conviction)
    G_2021_shop3.add_edge(antecedents, consequents, weight=row['confidence'], conviction=conviction)

# Normalize conviction values for edge width
max_conviction = max(convictions)
min_conviction = min(convictions)
conviction_range = max_conviction - min_conviction
edge_widths = [(G_2021_shop3[u][v]['conviction'] - min_conviction) / conviction_range * 30 + 0.5 for u, v in G_2021_shop3.edges()]  # Scale and offset widths

pos_2021_shop3 = nx.spring_layout(G_2021_shop3)

# Node sizes and colors as before
in_degrees_2021_shop3 = dict(G_2021_shop3.in_degree())
node_sizes_2021_shop3 = [in_degrees_2021_shop3[node] * 100 + 1000 for node in G_2021_shop3.nodes()]
normalized_sizes_2021_shop3 = (np.array(node_sizes_2021_shop3) - min(node_sizes_2021_shop3)) / (max(node_sizes_2021_shop3) - min(node_sizes_2021_shop3))
cmap_2021_shop3 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2021_shop3 = [cmap_2021_shop3(size) for size in normalized_sizes_2021_shop3]

# Drawing the network with conviction-based edge widths
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2021_shop3, pos_2021_shop3, node_size=node_sizes_2021_shop3, node_color=node_colors_2021_shop3, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop3, pos_2021_shop3, width=edge_widths, edge_color='grey', alpha=0.5)
nx.draw_networkx_labels(G_2021_shop3, pos_2021_shop3, font_size=10)

plt.title('Top 5 Association Rules Network for 2021 Shop 3 with Conviction-based Edge Thickness')
plt.axis('off')
plt.show()


In [ ]:
#the node sizes are based on the indegree where each node acts as a consequent,

In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2021_shop3_nonredundant_top30.loc[:, 'antecedents_'] = rules_2021_shop3_nonredundant_top30['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2021_shop3_nonredundant_top30.loc[:, 'consequents_'] = rules_2021_shop3_nonredundant_top30['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2021_shop3_nonredundant_top30, x='consequents_', y='antecedents_', s=rules_2021_shop3_nonredundant_top30['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2021_shop3_nonredundant_top30.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2021 Shop 3", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
print(rules_2021_shop3_nonredundant_top30[['antecedents', 'consequents']].head(10))

## Monetary Analysis of the Top Rules

In [ ]:
# Step 1: Extract unique product codes from the rules
unique_product_codes_shop3 = set()
for idx, row in rules_2021_shop3_nonredundant_top30.iterrows():
    # Unite antecedents and consequents into a single set
    all_products = row['antecedents'].union(row['consequents'])
    unique_product_codes_shop3.update(all_products)

# Step 2: Create a DataFrame with unique product codes and their unit prices
# Filter sales_data for these product codes and drop duplicates to ensure each product code appears once
product_prices_shop3 = sales_data_shop3[sales_data_shop3['product_code'].isin(unique_product_codes_shop3)][['product_code', 'unit_price']].drop_duplicates()

# Create a dictionary for quick lookup of prices
price_dict_shop3 = product_prices_shop3.set_index('product_code')['unit_price'].to_dict()

# Step 3: Calculate the total price for each rule and collect data for the new DataFrame
data_for_new_df_shop3 = []
for idx, row in rules_2021_shop3_nonredundant_top30.iterrows():
    # Combine sets of antecedents and consequents
    all_products = row['antecedents'].union(row['consequents'])
    total_price = sum(price_dict_shop3.get(prod, 0) for prod in all_products)  # Use get to avoid KeyError if any product code is missing
    data_for_new_df_shop3.append({
        'antecedents': row['antecedents'],
        'consequents': row['consequents'],
        'total_unit_price': total_price
    })

# Create a new DataFrame with only antecedents, consequents, and total unit price
rules_and_price_shop3_2021 = pd.DataFrame(data_for_new_df_shop3)

# Sort the DataFrame by total_unit_price in descending order
rules_and_price_shop3_2021_sorted = rules_and_price_shop3_2021.sort_values(by='total_unit_price', ascending=False)

# Print the sorted DataFrame to see the specified columns along with their total unit prices
print(rules_and_price_shop3_2021_sorted)


In [ ]:
# Ensure 'trx_date' is in datetime format and add 'year' column if it doesn't exist
sales_data_shop3['trx_date'] = pd.to_datetime(sales_data_shop3['trx_date'])
sales_data_shop3['year'] = sales_data_shop3['trx_date'].dt.year

# Filter sales data for Shop 3 in the year 2021
filtered_sales_data_shop3_2021 = sales_data_shop3[(sales_data_shop3['shopID'] == 3) & (sales_data_shop3['year'] == 2021)]

# Initialize a list to hold the rule data including total sales value
rule_sales_summaries_shop3_2021 = []

# Iterate through each rule in the DataFrame
for index, rule in rules_2021_shop3_nonredundant_top30.iterrows():
    all_products = rule['antecedents'].union(rule['consequents'])
    # Create a mask to filter transactions that include all products in the rule
    matched_transactions = filtered_sales_data_shop3_2021[filtered_sales_data_shop3_2021['product_code'].isin(all_products)]
    matched_transactions = matched_transactions.groupby('trx_id').filter(lambda x: all_products.issubset(set(x['product_code'])))

    # Sum the total_value of these transactions
    total_sales_value = matched_transactions['total_value'].sum()

    # Append the result to the list
    rule_sales_summaries_shop3_2021.append({
        'antecedents': rule['antecedents'],
        'consequents': rule['consequents'],
        'total_sales_value': total_sales_value
    })

# Convert the list to a DataFrame and sort by total_sales_value in descending order
rules_sales_values_shop3_df_2021 = pd.DataFrame(rule_sales_summaries_shop3_2021)
sorted_rules_sales_values_shop3_df_2021 = rules_sales_values_shop3_df_2021.sort_values(by='total_sales_value', ascending=False)

# Print the sorted DataFrame
print(sorted_rules_sales_values_shop3_df_2021)


## Narrowing down based on Lift

In [ ]:
rules_2021_shop3_nonredundant_top30_lift['LHS'] = [','.join(list(x)) for x in rules_2021_shop3_nonredundant_top30_lift['antecedents']]
rules_2021_shop3_nonredundant_top30_lift['RHS'] = [','.join(list(x)) for x in rules_2021_shop3_nonredundant_top30_lift['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2021_shop3_nonredundant_top30_lift.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2021_shop3_nonredundant_top30_lift, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2021 Shop 3")
plt.axis('off')
plt.show()


In [ ]:
G_2021_shop3_lift = nx.DiGraph()

lift_summary_2021_shop3 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2021_shop3_nonredundant_top30_lift.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop3_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2021_shop3:
        lift_summary_2021_shop3[consequents]['total_lift'] += row['lift']
        lift_summary_2021_shop3[consequents]['count'] += 1
    else:
        lift_summary_2021_shop3[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2021_shop3:
    lift_summary_2021_shop3[item]['avg_lift'] = lift_summary_2021_shop3[item]['total_lift'] / lift_summary_2021_shop3[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2021_shop3_lift = [lift_summary_2021_shop3[node]['avg_lift'] * 500 if node in lift_summary_2021_shop3 else 1000 for node in G_2021_shop3_lift.nodes()]

pos_2021_shop3_lift = nx.spring_layout(G_2021_shop3_lift)

# Colormap from orange to red
cmap_2021_shop3_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2021_shop3_lift = np.array(node_sizes_2021_shop3_lift)
normalized_sizes_2021_shop3_lift = (normalized_sizes_2021_shop3_lift - min(normalized_sizes_2021_shop3_lift)) / (max(normalized_sizes_2021_shop3_lift) - min(normalized_sizes_2021_shop3_lift))
node_colors_2021_shop3_lift = [cmap_2021_shop3_lift(size) for size in normalized_sizes_2021_shop3_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 22))
nx.draw_networkx_nodes(G_2021_shop3_lift, pos_2021_shop3_lift, node_size=node_sizes_2021_shop3_lift, node_color=node_colors_2021_shop3_lift, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop3_lift, pos_2021_shop3_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2021_shop3_lift, pos_2021_shop3_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2021 Shop 3 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2021_shop3_nonredundant_top30_lift.loc[:, 'antecedents_'] = rules_2021_shop3_nonredundant_top30_lift['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2021_shop3_nonredundant_top30_lift.loc[:, 'consequents_'] = rules_2021_shop3_nonredundant_top30_lift['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot for shop 3
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2021_shop3_nonredundant_top30_lift, x='consequents_', y='antecedents_', s=rules_2021_shop3_nonredundant_top30_lift['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2021_shop3_nonredundant_top30_lift.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2021 Shop 3", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
# Print only the 'antecedents' and 'consequents' columns of the top 10 rules
print(rules_2021_shop3_nonredundant_top30_lift[['antecedents', 'consequents']].head(10))


# Shop3 2022 SEARCH shop3_2

In [ ]:
# Filter for records from shop ID 3 and the year 2022
sales_data_2022_shop3 = sales_data_shop3[(sales_data_shop3['trx_date'].dt.year == 2022)]

In [ ]:
sales_data_2022_shop3.head()

In [ ]:
# Grouping product codes by trx_id
transactions_2022_shop3 = sales_data_2022_shop3.groupby('trx_id')['product_code'].apply(list).tolist()

In [ ]:
# Encode transaction data
encoder_2022_shop3 = TransactionEncoder()
trans_array_2022_shop3 = encoder_2022_shop3.fit(transactions_2022_shop3).transform(transactions_2022_shop3)
trans_df_2022_shop3 = pd.DataFrame(trans_array_2022_shop3, columns=encoder_2022_shop3.columns_)

In [ ]:
# Find frequent itemsets
frequent_itemsets_2022_shop3 = apriori(trans_df_2022_shop3, min_support=0.001, use_colnames=True)

In [ ]:
print(frequent_itemsets_2022_shop3)

In [ ]:
## Generate the association rules - by confidence for shop 3
rulesConfidence_2022_shop3 = association_rules(frequent_itemsets_2022_shop3, metric="confidence", min_threshold=0.10)
rulesConfidence_2022_shop3.sort_values(by='confidence', ascending=False, inplace=True)


In [ ]:
# Replace frozen sets with strings
rulesConfidence_2022_shop3['antecedents_'] = rulesConfidence_2022_shop3['antecedents'].apply(lambda a: ','.join(list(a)))
rulesConfidence_2022_shop3['consequents_'] = rulesConfidence_2022_shop3['consequents'].apply(lambda a: ','.join(list(a)))

# Draw
fig, ax = plt.subplots(figsize=(14,30))
ax = plt.scatter(data=rulesConfidence_2022_shop3, x='consequents_', y='antecedents_', s=rulesConfidence_2022_shop3['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rulesConfidence_2022_shop3.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2020 Shop 3", fontsize=14)
plt.xlabel("Consequents (RHS)", fontsize=12)
plt.ylabel("Antecedents (LHS)", fontsize=12)
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate(rotation=45)
plt.tight_layout()

plt.show()


## Eliminate Redundant Rules

In [ ]:
rulesConfidence_2022_shop3.head()

In [ ]:
rulesConfidence_2022_shop3.info()

In [ ]:
# If 'antecedents' and 'consequents' are not frozensets, convert them
rules_2022_shop3_redundant = rulesConfidence_2022_shop3.copy()
rules_2022_shop3_redundant['antecedents'] = rules_2022_shop3_redundant['antecedents'].apply(lambda x: frozenset(x))
rules_2022_shop3_redundant['consequents'] = rules_2022_shop3_redundant['consequents'].apply(lambda x: frozenset(x))


In [ ]:
# Create a copy of the DataFrame to avoid modifying the original while iterating
rules_2022_shop3_checkredundance = rules_2022_shop3_redundant.copy()

# List to hold non-redundant rules
non_redundant_rules_2022_shop3 = []

for _, rule in rules_2022_shop3_checkredundance.iterrows():
    if not is_redundant(rule, rules_2022_shop3_checkredundance):
        non_redundant_rules_2022_shop3.append(rule)

# Convert the list back to a DataFrame
rules_2022_shop3_nonredundant = pd.DataFrame(non_redundant_rules_2022_shop3).reset_index(drop=True)


In [ ]:
print(rules_2022_shop3_nonredundant.head())
print(f"Original number of rules: {len(rules_2022_shop3_redundant)}")
print(f"Number of non-redundant rules: {len(rules_2022_shop3_nonredundant)}")


In [ ]:
rules_2022_shop3_nonredundant.head()

In [ ]:
rules_2022_shop3_redundant.info()

In [ ]:
rules_2022_shop3_nonredundant.head()

In [ ]:
rules_2022_shop3_nonredundant_backup = rules_2022_shop3_nonredundant.copy()

In [ ]:
rules_2022_shop3_nonredundant_unfiltered = rules_2022_shop3_nonredundant.copy()

In [ ]:
#reset
#rules_2022_shop3_nonredundant = rules_2022_shop3_nonredundant_backup.copy()

# Redundant and Unfiltered

In [ ]:
rules_2022_shop3_redundant.head()

In [ ]:
G_2022_shop3 = nx.DiGraph()

# Iterate through rules for 2022 Shop 3 and add edges to the graph
for _, row in rules_2022_shop3_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as weight
    G_2022_shop3.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2022_shop3 = nx.spring_layout(G_2022_shop3)

# Recalculate node sizes based on indegree or another metric
in_degrees_2022_shop3 = dict(G_2022_shop3.in_degree())
node_sizes_2022_shop3 = [in_degrees_2022_shop3[node] * 100 + 1000 for node in G_2022_shop3.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2022_shop3 = (np.array(node_sizes_2022_shop3) - np.min(node_sizes_2022_shop3)) / (np.max(node_sizes_2022_shop3) - np.min(node_sizes_2022_shop3))

# Define a colormap from orange to red for visual consistency
cmap_2022_shop3 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2022_shop3 = [cmap_2022_shop3(size) for size in normalized_sizes_2022_shop3]

# Drawing the network
plt.figure(figsize=(60, 43))
nx.draw_networkx_nodes(G_2022_shop3, pos_2022_shop3, node_size=node_sizes_2022_shop3, node_color=node_colors_2022_shop3, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop3, pos_2022_shop3, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop3, pos_2022_shop3, font_size=10)

plt.title('Top 5 Association Rules Network for 2022 Shop 3 with Color Gradient')
plt.axis('off')
plt.show()


### Based on LIFT

In [ ]:
G_2022_shop3_lift = nx.DiGraph()

lift_summary_2022_shop3 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2022_shop3_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop3_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop3:
        lift_summary_2022_shop3[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop3[consequents]['count'] += 1
    else:
        lift_summary_2022_shop3[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop3:
    lift_summary_2022_shop3[item]['avg_lift'] = lift_summary_2022_shop3[item]['total_lift'] / lift_summary_2022_shop3[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop3_lift = [lift_summary_2022_shop3[node]['avg_lift'] * 500 if node in lift_summary_2022_shop3 else 1000 for node in G_2022_shop3_lift.nodes()]

pos_2022_shop3_lift = nx.spring_layout(G_2022_shop3_lift)

# Colormap from orange to red
cmap_2022_shop3_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop3_lift = np.array(node_sizes_2022_shop3_lift)
normalized_sizes_2022_shop3_lift = (normalized_sizes_2022_shop3_lift - min(normalized_sizes_2022_shop3_lift)) / (max(normalized_sizes_2022_shop3_lift) - min(normalized_sizes_2022_shop3_lift))
node_colors_2022_shop3_lift = [cmap_2022_shop3_lift(size) for size in normalized_sizes_2022_shop3_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(60, 52))
nx.draw_networkx_nodes(G_2022_shop3_lift, pos_2022_shop3_lift, node_size=node_sizes_2022_shop3_lift, node_color=node_colors_2022_shop3_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop3_lift, pos_2022_shop3_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop3_lift, pos_2022_shop3_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2022 Shop 3 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


# nonredundant_unfiltered

## Based on Indegree

In [ ]:
G_2022_shop3_nonredundant_unfiltered = nx.DiGraph()

# Iterating through each row to add edges with confidence as an attribute
for _, row in rules_2022_shop3_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop3_nonredundant_unfiltered.add_edge(antecedents, consequents, weight=row['confidence'])

# Defining the layout for network visualization
pos_2022_shop3_nonredundant_unfiltered = nx.spring_layout(G_2022_shop3_nonredundant_unfiltered)

# Calculating node sizes based on the indegree for color mapping
in_degrees_2022_shop3_nonredundant_unfiltered = dict(G_2022_shop3_nonredundant_unfiltered.in_degree())
node_sizes_2022_shop3_nonredundant_unfiltered = [in_degrees_2022_shop3_nonredundant_unfiltered[node] * 100 + 1000 for node in G_2022_shop3_nonredundant_unfiltered.nodes()]

# Normalizing node sizes for color mapping
normalized_sizes_2022_shop3_nonredundant_unfiltered = (np.array(node_sizes_2022_shop3_nonredundant_unfiltered) - min(node_sizes_2022_shop3_nonredundant_unfiltered)) / (max(node_sizes_2022_shop3_nonredundant_unfiltered) - min(node_sizes_2022_shop3_nonredundant_unfiltered))

# Defining a color map from orange to red
cmap_2022_shop3_nonredundant_unfiltered = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])

# Mapping normalized node sizes to colors
node_colors_2022_shop3_nonredundant_unfiltered = [cmap_2022_shop3_nonredundant_unfiltered(size) for size in normalized_sizes_2022_shop3_nonredundant_unfiltered]

# Drawing the network with node sizes and colors
plt.figure(figsize=(50, 34))
nx.draw_networkx_nodes(G_2022_shop3_nonredundant_unfiltered, pos_2022_shop3_nonredundant_unfiltered, node_size=node_sizes_2022_shop3_nonredundant_unfiltered, node_color=node_colors_2022_shop3_nonredundant_unfiltered, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop3_nonredundant_unfiltered, pos_2022_shop3_nonredundant_unfiltered, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2022_shop3_nonredundant_unfiltered, pos_2022_shop3_nonredundant_unfiltered, font_size=10)

plt.title('Node Size Based on Indegree for 2022 Shop 3')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2022_shop3_lift = nx.DiGraph()

lift_summary_2022_shop3 = {}

for _, row in rules_2022_shop3_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop3_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Update lift summary for consequents
    if consequents in lift_summary_2022_shop3:
        lift_summary_2022_shop3[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop3[consequents]['count'] += 1
    else:
        lift_summary_2022_shop3[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculate average lift for each consequent
for item in lift_summary_2022_shop3:
    lift_summary_2022_shop3[item]['avg_lift'] = lift_summary_2022_shop3[item]['total_lift'] / lift_summary_2022_shop3[item]['count']

# Node sizes based on average lift, scaled appropriately
node_sizes_2022_shop3_lift = [lift_summary_2022_shop3[node]['avg_lift'] * 500 if node in lift_summary_2022_shop3 else 1000 for node in G_2022_shop3_lift.nodes()]

pos_2022_shop3_lift = nx.spring_layout(G_2022_shop3_lift)

# Use a colormap that transitions from orange to red
cmap_2022_shop3_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop3_lift = np.array(node_sizes_2022_shop3_lift)
normalized_sizes_2022_shop3_lift = (normalized_sizes_2022_shop3_lift - min(normalized_sizes_2022_shop3_lift)) / (max(normalized_sizes_2022_shop3_lift) - min(normalized_sizes_2022_shop3_lift))
node_colors_2022_shop3_lift = [cmap_2022_shop3_lift(size) for size in normalized_sizes_2022_shop3_lift]

plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2022_shop3_lift, pos_2022_shop3_lift, node_size=node_sizes_2022_shop3_lift, node_color=node_colors_2022_shop3_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop3_lift, pos_2022_shop3_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop3_lift, pos_2022_shop3_lift, font_size=10)

plt.title('Node Size Based on Average Lift for 2022 Shop 3')
plt.axis('off')
plt.show()


# Based on conviction

In [ ]:
rules_2022_shop3_nonredundant.head()

In [ ]:
G_2022_shop3_conviction = nx.DiGraph()

conviction_summary_2022_shop3 = {}
count_summary_2022_shop3 = {}

# Assuming 'rules_2022_shop3_nonredundant_unfiltered' contains a 'conviction' column
for _, row in rules_2022_shop3_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop3_conviction.add_edge(antecedents, consequents, weight=row['confidence'], conviction=row['conviction'])
    
    # Aggregate conviction values and count occurrences
    if consequents in conviction_summary_2022_shop3:
        conviction_summary_2022_shop3[consequents] += row['conviction']
        count_summary_2022_shop3[consequents] += 1
    else:
        conviction_summary_2022_shop3[consequents] = row['conviction']
        count_summary_2022_shop3[consequents] = 1

# Calculate average conviction values
average_conviction_summary_2022_shop3 = {k: conviction_summary_2022_shop3[k] / count_summary_2022_shop3[k] for k in conviction_summary_2022_shop3}

# Normalize conviction values for sizing and coloring
max_conviction = max(average_conviction_summary_2022_shop3.values())
min_conviction = min(average_conviction_summary_2022_shop3.values())
conviction_range = max_conviction - min_conviction
node_colors = []
node_sizes = []

for node in G_2022_shop3_conviction.nodes():
    # Get the average conviction value for the node, defaulting to min if not present
    conviction = average_conviction_summary_2022_shop3.get(node, min_conviction)
    # Normalize between 0 and 1
    normalized_conviction = (conviction - min_conviction) / conviction_range if conviction_range else 0
    node_colors.append(normalized_conviction)
    node_sizes.append(normalized_conviction * 1000 + 100)  # Scale and offset size

# Use a colormap for coloring nodes based on conviction
cmap = plt.cm.coolwarm

pos = nx.spring_layout(G_2022_shop3_conviction)

plt.figure(figsize=(40, 22))
nodes = nx.draw_networkx_nodes(G_2022_shop3_conviction, pos, node_color=node_colors, node_size=node_sizes, cmap=cmap, alpha=0.8)
edges = nx.draw_networkx_edges(G_2022_shop3_conviction, pos, arrowstyle='->', arrowsize=10, edge_color='grey', alpha=0.5)
labels = nx.draw_networkx_labels(G_2022_shop3_conviction, pos, font_size=8)

plt.colorbar(nodes, label='Normalized Conviction')
plt.title('Network Graph of 2022 Shop 3 Based on Conviction')
plt.axis('off')
plt.show()

In [ ]:
print(max_conviction)

# Filtered top 30 and non redundant

In [ ]:
rules_2022_shop3_nonredundant.head()

In [ ]:
# Assuming rules_1992_shop2_nonredundant is your sorted rules DataFrame for shop2
rules_2022_shop3_nonredundant_top30 = rules_2022_shop3_nonredundant.copy(deep=True)

In [ ]:
rules_2022_shop3_nonredundant_top30_lift = rules_2022_shop3_nonredundant.copy(deep=True)

In [ ]:
# Select the top 30 rules based on lift or another criterion
rules_2022_shop3_nonredundant_top30_lift = rules_2022_shop3_nonredundant_top30_lift.sort_values(by='lift', ascending=False).head(30)

In [ ]:
rules_2022_shop3_nonredundant_top30_lift.head()

In [ ]:
# Select the top 30 rules based on confidence or another criterion
rules_2022_shop3_nonredundant_top30 = rules_2022_shop3_nonredundant_top30.sort_values(by='confidence', ascending=False).head(30)

In [ ]:
rules_2022_shop3_nonredundant_top30.info()

In [ ]:
rules_2022_shop3_nonredundant_top30['LHS'] = [','.join(list(x)) for x in rules_2022_shop3_nonredundant_top30['antecedents']]
rules_2022_shop3_nonredundant_top30['RHS'] = [','.join(list(x)) for x in rules_2022_shop3_nonredundant_top30['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2022_shop3_nonredundant_top30.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2022_shop3_nonredundant_top30, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2022 Shop 3")
plt.axis('off')
plt.show()


## Based on Indegree

In [ ]:
G_2022_shop3 = nx.DiGraph()

# Assuming 'rules_sorted_2022_shop3' contains your association rules for shop 3 in 2022
for _, row in rules_2022_shop3_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as the weight
    G_2022_shop3.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2022_shop3 = nx.spring_layout(G_2022_shop3)

# Calculate node sizes based on indegree
in_degrees_2022_shop3 = dict(G_2022_shop3.in_degree())
node_sizes_2022_shop3 = [in_degrees_2022_shop3[node] * 100 + 1000 for node in G_2022_shop3.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2022_shop3 = (np.array(node_sizes_2022_shop3) - np.min(node_sizes_2022_shop3)) / (np.max(node_sizes_2022_shop3) - np.min(node_sizes_2022_shop3))

# Define a colormap from orange to red
cmap_2022_shop3 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2022_shop3 = [cmap_2022_shop3(size) for size in normalized_sizes_2022_shop3]

# Drawing the network
plt.figure(figsize=(30, 14))
nx.draw_networkx_nodes(G_2022_shop3, pos_2022_shop3, node_size=node_sizes_2022_shop3, node_color=node_colors_2022_shop3, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop3, pos_2022_shop3, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2022_shop3, pos_2022_shop3, font_size=8)

plt.title('Top 5 Association Rules Network for 2022 Shop 3 with Color Gradient')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2022_shop3_lift = nx.DiGraph()

lift_summary_2022_shop3 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2022_shop3_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop3_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop3:
        lift_summary_2022_shop3[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop3[consequents]['count'] += 1
    else:
        lift_summary_2022_shop3[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop3:
    lift_summary_2022_shop3[item]['avg_lift'] = lift_summary_2022_shop3[item]['total_lift'] / lift_summary_2022_shop3[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop3_lift = [lift_summary_2022_shop3[node]['avg_lift'] * 500 if node in lift_summary_2022_shop3 else 1000 for node in G_2022_shop3_lift.nodes()]

pos_2022_shop3_lift = nx.spring_layout(G_2022_shop3_lift)

# Colormap from orange to red
cmap_2022_shop3_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop3_lift = np.array(node_sizes_2022_shop3_lift)
normalized_sizes_2022_shop3_lift = (normalized_sizes_2022_shop3_lift - min(normalized_sizes_2022_shop3_lift)) / (max(normalized_sizes_2022_shop3_lift) - min(normalized_sizes_2022_shop3_lift))
node_colors_2022_shop3_lift = [cmap_2022_shop3_lift(size) for size in normalized_sizes_2022_shop3_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 32))
nx.draw_networkx_nodes(G_2022_shop3_lift, pos_2022_shop3_lift, node_size=node_sizes_2022_shop3_lift, node_color=node_colors_2022_shop3_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop3_lift, pos_2022_shop3_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop3_lift, pos_2022_shop3_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2022 Shop 3 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop3_lift = nx.DiGraph()

lift_summary_2022_shop3 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2022_shop3_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop3_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop3:
        lift_summary_2022_shop3[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop3[consequents]['count'] += 1
    else:
        lift_summary_2022_shop3[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop3:
    lift_summary_2022_shop3[item]['avg_lift'] = lift_summary_2022_shop3[item]['total_lift'] / lift_summary_2022_shop3[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop3_lift = [lift_summary_2022_shop3[node]['avg_lift'] * 500 if node in lift_summary_2022_shop3 else 1000 for node in G_2022_shop3_lift.nodes()]

pos_2022_shop3_lift = nx.spring_layout(G_2022_shop3_lift)

# Colormap from orange to red
cmap_2022_shop3_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop3_lift = np.array(node_sizes_2022_shop3_lift)
normalized_sizes_2022_shop3_lift = (normalized_sizes_2022_shop3_lift - min(normalized_sizes_2022_shop3_lift)) / (max(normalized_sizes_2022_shop3_lift) - min(normalized_sizes_2022_shop3_lift))
node_colors_2022_shop3_lift = [cmap_2022_shop3_lift(size) for size in normalized_sizes_2022_shop3_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2022_shop3_lift, pos_2022_shop3_lift, node_size=node_sizes_2022_shop3_lift, node_color=node_colors_2022_shop3_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop3_lift, pos_2022_shop3_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop3_lift, pos_2022_shop3_lift, font_size=10)

plt.title('Top 5 Association Rules Network for 2022 Shop 3 Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop3 = nx.DiGraph()

# Assuming 'rules_2022_shop3_nonredundant' contains association rules for 2022 Shop 3
for _, row in rules_2022_shop3_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop3.add_edge(antecedents, consequents, confidence=row['confidence'])

# Calculate node sizes based on indegree
in_degrees_2022_shop3 = dict(G_2022_shop3.in_degree())
node_sizes_2022_shop3 = [in_degrees_2022_shop3[node] * 100 + 1000 for node in G_2022_shop3.nodes()]

# Edge width based on confidence
edge_widths_2022_shop3 = [G_2022_shop3[u][v]['confidence'] * 15 for u, v in G_2022_shop3.edges()]

# Assuming normalized sizes and color mapping is desired
normalized_sizes_2022_shop3 = np.array(list(in_degrees_2022_shop3.values()))
normalized_sizes_2022_shop3 = (normalized_sizes_2022_shop3 - min(normalized_sizes_2022_shop3)) / (max(normalized_sizes_2022_shop3) - min(normalized_sizes_2022_shop3))
cmap_2022_shop3 = mcolors.LinearSegmentedColormap.from_list("MyCMapName", ["blue", "red"])

# Node colors based on a property (here, using indegree for example)
node_colors_2022_shop3 = [cmap_2022_shop3(size) for size in normalized_sizes_2022_shop3]

# Layout
pos_2022_shop3 = nx.spring_layout(G_2022_shop3, seed=42)

# Drawing
plt.figure(figsize=(40, 30))
nx.draw_networkx_edges(G_2022_shop3, pos_2022_shop3, width=edge_widths_2022_shop3, edge_color='grey', alpha=0.5)
nx.draw_networkx_nodes(G_2022_shop3, pos_2022_shop3, node_size=node_sizes_2022_shop3, node_color=node_colors_2022_shop3, alpha=0.8)
nx.draw_networkx_labels(G_2022_shop3, pos_2022_shop3, font_size=8, font_color='black')
plt.title('Network Graph of Association Rules Based on Confidence for 2022 Shop 3')
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop3 = nx.DiGraph()

convictions_2022_shop3 = []  # To store conviction values for normalization

for _, row in rules_2022_shop3_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    conviction_2022_shop3 = row['conviction']  # Assuming each row has a 'conviction' column
    convictions_2022_shop3.append(conviction_2022_shop3)
    G_2022_shop3.add_edge(antecedents, consequents, weight=row['confidence'], conviction=conviction_2022_shop3)

# Normalize conviction values for edge width
max_conviction_2022_shop3 = max(convictions_2022_shop3)
min_conviction_2022_shop3 = min(convictions_2022_shop3)
conviction_range_2022_shop3 = max_conviction_2022_shop3 - min_conviction_2022_shop3
edge_widths_2022_shop3 = [(G_2022_shop3[u][v]['conviction'] - min_conviction_2022_shop3) / conviction_range_2022_shop3 * 30 + 0.5 for u, v in G_2022_shop3.edges()]  # Scale and offset widths

pos_2022_shop3 = nx.spring_layout(G_2022_shop3)

# Node sizes and colors as before
in_degrees_2022_shop3 = dict(G_2022_shop3.in_degree())
node_sizes_2022_shop3 = [in_degrees_2022_shop3[node] * 100 + 1000 for node in G_2022_shop3.nodes()]
normalized_sizes_2022_shop3 = (np.array(node_sizes_2022_shop3) - min(node_sizes_2022_shop3)) / (max(node_sizes_2022_shop3) - min(node_sizes_2022_shop3))
cmap_2022_shop3 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2022_shop3 = [cmap_2022_shop3(size) for size in normalized_sizes_2022_shop3]

# Drawing the network with conviction-based edge widths
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2022_shop3, pos_2022_shop3, node_size=node_sizes_2022_shop3, node_color=node_colors_2022_shop3, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop3, pos_2022_shop3, width=edge_widths_2022_shop3, edge_color='grey', alpha=0.5)
nx.draw_networkx_labels(G_2022_shop3, pos_2022_shop3, font_size=10)

plt.title('Top 5 Association Rules Network for 2022 Shop 3 with Conviction-based Edge Thickness')
plt.axis('off')
plt.show()


In [ ]:
#the node sizes are based on the indegree where each node acts as a consequent,

In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2022_shop3_nonredundant_top30.loc[:, 'antecedents_'] = rules_2022_shop3_nonredundant_top30['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2022_shop3_nonredundant_top30.loc[:, 'consequents_'] = rules_2022_shop3_nonredundant_top30['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot for Shop 3
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2022_shop3_nonredundant_top30, x='consequents_', y='antecedents_', s=rules_2022_shop3_nonredundant_top30['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules_shop3 = rules_2022_shop3_nonredundant_top30.shape[0]
plt.title(f"Grouped matrix of the {nRules_shop3} rules for 2022 Shop 3", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
print(rules_2022_shop3_nonredundant_top30[['antecedents', 'consequents']].head(10))

## Monetary Analysis of the Top Rules

In [ ]:
# Step 1: Extract unique product codes from the rules for Shop 3
unique_product_codes_shop3 = set()
for idx, row in rules_2022_shop3_nonredundant_top30.iterrows():
    # Unite antecedents and consequents into a single set
    all_products_shop3 = row['antecedents'].union(row['consequents'])
    unique_product_codes_shop3.update(all_products_shop3)

# Step 2: Create a DataFrame with unique product codes and their unit prices for Shop 3
# Filter sales_data for these product codes and drop duplicates to ensure each product code appears once
product_prices_shop3 = sales_data_shop3[sales_data_shop3['product_code'].isin(unique_product_codes_shop3)][['product_code', 'unit_price']].drop_duplicates()

# Create a dictionary for quick lookup of prices for Shop 3
price_dict_shop3 = product_prices_shop3.set_index('product_code')['unit_price'].to_dict()

# Step 3: Calculate the total price for each rule and collect data for the new DataFrame for Shop 3
data_for_new_df_shop3 = []
for idx, row in rules_2022_shop3_nonredundant_top30.iterrows():
    # Combine sets of antecedents and consequents for Shop 3
    all_products_shop3 = row['antecedents'].union(row['consequents'])
    total_price_shop3 = sum(price_dict_shop3.get(prod, 0) for prod in all_products_shop3)  # Use get to avoid KeyError if any product code is missing
    data_for_new_df_shop3.append({
        'antecedents': row['antecedents'],
        'consequents': row['consequents'],
        'total_unit_price': total_price_shop3
    })

# Create a new DataFrame with only antecedents, consequents, and total unit price for Shop 3
rules_and_price_shop3_2022 = pd.DataFrame(data_for_new_df_shop3)

# Sort the DataFrame by total_unit_price in descending order for Shop 3
rules_and_price_shop3_2022_sorted = rules_and_price_shop3_2022.sort_values(by='total_unit_price', ascending=False)

# Print the sorted DataFrame to see the specified columns along with their total unit prices for Shop 3
print(rules_and_price_shop3_2022_sorted)


In [ ]:
# Convert 'trx_date' to datetime format if not already done so for Shop 3
sales_data_shop3['trx_date'] = pd.to_datetime(sales_data_shop3['trx_date'])

# Add a 'year' column if it doesn't exist for Shop 3
sales_data_shop3['year'] = sales_data_shop3['trx_date'].dt.year

# Step 2: Filter sales_data for Shop 3 and the year 2022
filtered_sales_data_shop3_2022 = sales_data_shop3[(sales_data_shop3['shopID'] == 3) & (sales_data_shop3['year'] == 2022)]

# Step 3: Initialize a list to hold the rule data including total sales value for Shop 3
rule_sales_summaries_shop3_2022 = []

# Step 4: Iterate through each rule in the DataFrame for Shop 3
for index, rule in rules_2022_shop3_nonredundant_top30.iterrows():
    all_products_shop3 = rule['antecedents'].union(rule['consequents'])

    # Filter the sales data for transactions that include all products in the rule for Shop 3
    # This step ensures that we are only considering transactions that contain all specified products for Shop 3
    mask_shop3 = filtered_sales_data_shop3_2022['product_code'].isin(all_products_shop3)
    transaction_ids_shop3 = filtered_sales_data_shop3_2022[mask_shop3].groupby('trx_id').filter(lambda x: all_products_shop3.issubset(set(x['product_code']))).trx_id.unique()
    matched_transactions_shop3 = filtered_sales_data_shop3_2022[filtered_sales_data_shop3_2022['trx_id'].isin(transaction_ids_shop3)]
    
    # Step 5: Sum the total_value of these transactions for Shop 3
    total_sales_value_shop3 = matched_transactions_shop3['total_value'].sum()

    # Append the result to the list for Shop 3
    rule_sales_summaries_shop3_2022.append({
        'antecedents': rule['antecedents'],
        'consequents': rule['consequents'],
        'total_sales_value': total_sales_value_shop3
    })

# Step 6: Convert the list to a DataFrame for Shop 3
rules_sales_values_df_shop3_2022 = pd.DataFrame(rule_sales_summaries_shop3_2022)

# Step 7: Sort by total_sales_value in descending order for Shop 3
sorted_rules_sales_values_df_shop3_2022 = rules_sales_values_df_shop3_2022.sort_values(by='total_sales_value', ascending=False)

# Step 8: Print the sorted DataFrame for Shop 3
print(sorted_rules_sales_values_df_shop3_2022)


## Narrowing down based on Lift

In [ ]:
rules_2022_shop3_nonredundant_top30_lift['LHS'] = [','.join(list(x)) for x in rules_2022_shop3_nonredundant_top30_lift['antecedents']]
rules_2022_shop3_nonredundant_top30_lift['RHS'] = [','.join(list(x)) for x in rules_2022_shop3_nonredundant_top30_lift['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS for Shop 3
rules_2022_shop3_nonredundant_top30_lift.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph for Shop 3
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2022_shop3_nonredundant_top30_lift, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2022 Shop 3")
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop3_lift = nx.DiGraph()

lift_summary_2022_shop3 = {}

# Iterating through each row to add edges with lift as an attribute for Shop 3
for _, row in rules_2022_shop3_nonredundant_top30_lift.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop3_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop3:
        lift_summary_2022_shop3[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop3[consequents]['count'] += 1
    else:
        lift_summary_2022_shop3[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop3:
    lift_summary_2022_shop3[item]['avg_lift'] = lift_summary_2022_shop3[item]['total_lift'] / lift_summary_2022_shop3[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop3_lift = [lift_summary_2022_shop3[node]['avg_lift'] * 500 if node in lift_summary_2022_shop3 else 1000 for node in G_2022_shop3_lift.nodes()]

pos_2022_shop3_lift = nx.spring_layout(G_2022_shop3_lift)

# Colormap from orange to red
cmap_2022_shop3_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop3_lift = np.array(node_sizes_2022_shop3_lift)
normalized_sizes_2022_shop3_lift = (normalized_sizes_2022_shop3_lift - min(normalized_sizes_2022_shop3_lift)) / (max(normalized_sizes_2022_shop3_lift) - min(normalized_sizes_2022_shop3_lift))
node_colors_2022_shop3_lift = [cmap_2022_shop3_lift(size) for size in normalized_sizes_2022_shop3_lift]

# Drawing the network with node sizes reflecting average lift for Shop 3
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2022_shop3_lift, pos_2022_shop3_lift, node_size=node_sizes_2022_shop3_lift, node_color=node_colors_2022_shop3_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop3_lift, pos_2022_shop3_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop3_lift, pos_2022_shop3_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2022 Shop 3 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification for Shop 3
rules_2022_shop3_nonredundant_top30_lift.loc[:, 'antecedents_'] = rules_2022_shop3_nonredundant_top30_lift['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2022_shop3_nonredundant_top30_lift.loc[:, 'consequents_'] = rules_2022_shop3_nonredundant_top30_lift['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot for Shop 3
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2022_shop3_nonredundant_top30_lift, x='consequents_', y='antecedents_', s=rules_2022_shop3_nonredundant_top30_lift['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules_shop3 = rules_2022_shop3_nonredundant_top30_lift.shape[0]
plt.title(f"Grouped matrix of the {nRules_shop3} rules for 2022 Shop 3", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
# Print only the 'antecedents' and 'consequents' columns of the top 10 rules
print(rules_2022_shop3_nonredundant_top30_lift[['antecedents', 'consequents']].head(20))


# Shop3 2023 SEARCH shop3_3

In [ ]:
# Filter for records from shop ID 1 and the year 1993
sales_data_2023_shop3 = sales_data_shop3[(sales_data_shop3['trx_date'].dt.year == 2023)]

In [ ]:
sales_data_2023_shop3.head()

In [ ]:
# Grouping product codes by trx_id
transactions_2023_shop3 = sales_data_2023_shop3.groupby('trx_id')['product_code'].apply(list).tolist()

In [ ]:
# Encode transaction data for Shop 3
encoder_2023_shop3 = TransactionEncoder()
trans_array_2023_shop3 = encoder_2023_shop3.fit(transactions_2023_shop3).transform(transactions_2023_shop3)
trans_df_2023_shop3 = pd.DataFrame(trans_array_2023_shop3, columns=encoder_2023_shop3.columns_)

In [ ]:
# Find frequent itemsets
frequent_itemsets_2023_shop3 = apriori(trans_df_2023_shop3, min_support=0.001, use_colnames=True)

In [ ]:
print(frequent_itemsets_2023_shop3)

In [ ]:
# Generate association rules by confidence for Shop 3
rulesConfidence_2023_shop3 = association_rules(frequent_itemsets_2023_shop3, metric="confidence", min_threshold=0.10)
rulesConfidence_2023_shop3.sort_values(by='confidence', ascending=False, inplace=True)


In [ ]:
# Replace frozen sets with strings for Shop 3
rulesConfidence_2023_shop3['antecedents_'] = rulesConfidence_2023_shop3['antecedents'].apply(lambda a: ','.join(list(a)))
rulesConfidence_2023_shop3['consequents_'] = rulesConfidence_2023_shop3['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot for Shop 3
fig, ax = plt.subplots(figsize=(14,30))
ax = plt.scatter(data=rulesConfidence_2023_shop3, x='consequents_', y='antecedents_', s=rulesConfidence_2023_shop3['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules_shop3 = rulesConfidence_2023_shop3.shape[0]
plt.title(f"Grouped matrix of the {nRules_shop3} rules for 2023 Shop 3", fontsize=14)
plt.xlabel("Consequents (RHS)", fontsize=12)
plt.ylabel("Antecedents (LHS)", fontsize=12)
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate(rotation=45)
plt.tight_layout()

plt.show()


## Eliminate Redundant Rules

In [ ]:
rulesConfidence_2023_shop3.head()

In [ ]:
rulesConfidence_2023_shop3.info()

In [ ]:
# If 'antecedents' and 'consequents' are not frozensets, convert them for Shop 3
rules_2023_shop3_redundant = rulesConfidence_2023_shop3.copy()
rules_2023_shop3_redundant['antecedents'] = rules_2023_shop3_redundant['antecedents'].apply(lambda x: frozenset(x))
rules_2023_shop3_redundant['consequents'] = rules_2023_shop3_redundant['consequents'].apply(lambda x: frozenset(x))


In [ ]:
# Create a copy of the DataFrame to avoid modifying the original while iterating
rules_2023_shop3_checkredundance = rules_2023_shop3_redundant.copy()

# List to hold non-redundant rules
non_redundant_rules_2023_shop3 = []

for _, rule in rules_2023_shop3_checkredundance.iterrows():
    if not is_redundant(rule, rules_2023_shop3_checkredundance):
        non_redundant_rules_2023_shop3.append(rule)

# Convert the list back to a DataFrame
rules_2023_shop3_nonredundant = pd.DataFrame(non_redundant_rules_2023_shop3).reset_index(drop=True)


In [ ]:
print(rules_2023_shop3_nonredundant.head())
print(f"Original number of rules: {len(rules_2023_shop3_redundant)}")
print(f"Number of non-redundant rules: {len(rules_2023_shop3_nonredundant)}")


In [ ]:
rules_2023_shop3_nonredundant.head()

In [ ]:
rules_2023_shop3_nonredundant.head()

In [ ]:
rules_2023_shop3_nonredundant_backup = rules_2023_shop3_nonredundant.copy()

In [ ]:
rules_2023_shop3_nonredundant_unfiltered = rules_2023_shop3_nonredundant.copy()

In [ ]:
#reset
#rules_2023_shop3_nonredundant = rules_2023_shop3_nonredundant_backup.copy()

# Redundant and Unfiltered

In [ ]:
rules_2023_shop3_redundant.head()

In [ ]:
G_2023_shop3 = nx.DiGraph()

# Iterate through rules for 2023 Shop 3 and add edges to the graph
for _, row in rules_2023_shop3_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as weight
    G_2023_shop3.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2023_shop3 = nx.spring_layout(G_2023_shop3)

# Recalculate node sizes based on indegree or another metric
in_degrees_2023_shop3 = dict(G_2023_shop3.in_degree())
node_sizes_2023_shop3 = [in_degrees_2023_shop3[node] * 100 + 1000 for node in G_2023_shop3.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2023_shop3 = (np.array(node_sizes_2023_shop3) - np.min(node_sizes_2023_shop3)) / (np.max(node_sizes_2023_shop3) - np.min(node_sizes_2023_shop3))

# Define a colormap from orange to red for visual consistency
cmap_2023_shop3 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop3 = [cmap_2023_shop3(size) for size in normalized_sizes_2023_shop3]

# Drawing the network
plt.figure(figsize=(40, 33))
nx.draw_networkx_nodes(G_2023_shop3, pos_2023_shop3, node_size=node_sizes_2023_shop3, node_color=node_colors_2023_shop3, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop3, pos_2023_shop3, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop3, pos_2023_shop3, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 3 with Color Gradient')
plt.axis('off')
plt.show()


### Based on LIFT

In [ ]:
G_2023_shop3_lift = nx.DiGraph()

lift_summary_2023_shop3 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop3_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop3_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop3:
        lift_summary_2023_shop3[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop3[consequents]['count'] += 1
    else:
        lift_summary_2023_shop3[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop3:
    lift_summary_2023_shop3[item]['avg_lift'] = lift_summary_2023_shop3[item]['total_lift'] / lift_summary_2023_shop3[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop3_lift = [lift_summary_2023_shop3[node]['avg_lift'] * 500 if node in lift_summary_2023_shop3 else 1000 for node in G_2023_shop3_lift.nodes()]

pos_2023_shop3_lift = nx.spring_layout(G_2023_shop3_lift)

# Colormap from orange to red
cmap_2023_shop3_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop3_lift = np.array(node_sizes_2023_shop3_lift)
normalized_sizes_2023_shop3_lift = (normalized_sizes_2023_shop3_lift - min(normalized_sizes_2023_shop3_lift)) / (max(normalized_sizes_2023_shop3_lift) - min(normalized_sizes_2023_shop3_lift))
node_colors_2023_shop3_lift = [cmap_2023_shop3_lift(size) for size in normalized_sizes_2023_shop3_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(70, 52))
nx.draw_networkx_nodes(G_2023_shop3_lift, pos_2023_shop3_lift, node_size=node_sizes_2023_shop3_lift, node_color=node_colors_2023_shop3_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop3_lift, pos_2023_shop3_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop3_lift, pos_2023_shop3_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 3 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


# nonredundant_unfiltered

## Based on Indegree

In [ ]:
G_2023_shop3_nonredundant_unfiltered = nx.DiGraph()

# Iterating through each row to add edges with confidence as an attribute
for _, row in rules_2023_shop3_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop3_nonredundant_unfiltered.add_edge(antecedents, consequents, weight=row['confidence'])

# Defining the layout for network visualization
pos_2023_shop3_nonredundant_unfiltered = nx.spring_layout(G_2023_shop3_nonredundant_unfiltered)

# Calculating node sizes based on the indegree for color mapping
in_degrees_2023_shop3_nonredundant_unfiltered = dict(G_2023_shop3_nonredundant_unfiltered.in_degree())
node_sizes_2023_shop3_nonredundant_unfiltered = [in_degrees_2023_shop3_nonredundant_unfiltered[node] * 100 + 1000 for node in G_2023_shop3_nonredundant_unfiltered.nodes()]

# Normalizing node sizes for color mapping
normalized_sizes_2023_shop3_nonredundant_unfiltered = (np.array(node_sizes_2023_shop3_nonredundant_unfiltered) - min(node_sizes_2023_shop3_nonredundant_unfiltered)) / (max(node_sizes_2023_shop3_nonredundant_unfiltered) - min(node_sizes_2023_shop3_nonredundant_unfiltered))

# Defining a color map from orange to red
cmap_2023_shop3_nonredundant_unfiltered = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])

# Mapping normalized node sizes to colors
node_colors_2023_shop3_nonredundant_unfiltered = [cmap_2023_shop3_nonredundant_unfiltered(size) for size in normalized_sizes_2023_shop3_nonredundant_unfiltered]

# Drawing the network with node sizes and colors
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2023_shop3_nonredundant_unfiltered, pos_2023_shop3_nonredundant_unfiltered, node_size=node_sizes_2023_shop3_nonredundant_unfiltered, node_color=node_colors_2023_shop3_nonredundant_unfiltered, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop3_nonredundant_unfiltered, pos_2023_shop3_nonredundant_unfiltered, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2023_shop3_nonredundant_unfiltered, pos_2023_shop3_nonredundant_unfiltered, font_size=10)

plt.title('Node Size Based on Indegree for 2023 Shop 3')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2023_shop3_lift = nx.DiGraph()

lift_summary_2023_shop3 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop3_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop3_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop3:
        lift_summary_2023_shop3[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop3[consequents]['count'] += 1
    else:
        lift_summary_2023_shop3[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop3:
    lift_summary_2023_shop3[item]['avg_lift'] = lift_summary_2023_shop3[item]['total_lift'] / lift_summary_2023_shop3[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop3_lift = [lift_summary_2023_shop3[node]['avg_lift'] * 500 if node in lift_summary_2023_shop3 else 1000 for node in G_2023_shop3_lift.nodes()]

# Positioning nodes using Spring layout
pos_2023_shop3_lift = nx.spring_layout(G_2023_shop3_lift)

# Colormap from orange to red
cmap_2023_shop3_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop3_lift = np.array(node_sizes_2023_shop3_lift)
normalized_sizes_2023_shop3_lift = (normalized_sizes_2023_shop3_lift - min(normalized_sizes_2023_shop3_lift)) / (max(normalized_sizes_2023_shop3_lift) - min(normalized_sizes_2023_shop3_lift))
node_colors_2023_shop3_lift = [cmap_2023_shop3_lift(size) for size in normalized_sizes_2023_shop3_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2023_shop3_lift, pos_2023_shop3_lift, node_size=node_sizes_2023_shop3_lift, node_color=node_colors_2023_shop3_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop3_lift, pos_2023_shop3_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop3_lift, pos_2023_shop3_lift, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 3 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


# Based on conviction

In [ ]:
rules_2023_shop3_nonredundant.head()

In [ ]:
G_2023_shop3_conviction = nx.DiGraph()

conviction_summary_2023_shop3 = {}
count_summary_2023_shop3 = {}

# Assuming 'rules_2023_shop3_nonredundant_unfiltered' contains a 'conviction' column
for _, row in rules_2023_shop3_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop3_conviction.add_edge(antecedents, consequents, weight=row['confidence'], conviction=row['conviction'])
    
    # Aggregate conviction values and count occurrences
    if consequents in conviction_summary_2023_shop3:
        conviction_summary_2023_shop3[consequents] += row['conviction']
        count_summary_2023_shop3[consequents] += 1
    else:
        conviction_summary_2023_shop3[consequents] = row['conviction']
        count_summary_2023_shop3[consequents] = 1

# Calculate average conviction values
average_conviction_summary_2023_shop3 = {k: conviction_summary_2023_shop3[k] / count_summary_2023_shop3[k] for k in conviction_summary_2023_shop3}

# Normalize conviction values for sizing and coloring
max_conviction = max(average_conviction_summary_2023_shop3.values())
min_conviction = min(average_conviction_summary_2023_shop3.values())
conviction_range = max_conviction - min_conviction
node_colors = []
node_sizes = []

for node in G_2023_shop3_conviction.nodes():
    # Get the average conviction value for the node, defaulting to min if not present
    conviction = average_conviction_summary_2023_shop3.get(node, min_conviction)
    # Normalize between 0 and 1
    normalized_conviction = (conviction - min_conviction) / conviction_range if conviction_range else 0
    node_colors.append(normalized_conviction)
    node_sizes.append(normalized_conviction * 1000 + 100)  # Scale and offset size

# Use a colormap for coloring nodes based on conviction
cmap = plt.cm.coolwarm

pos = nx.spring_layout(G_2023_shop3_conviction)

plt.figure(figsize=(40, 22))
nodes = nx.draw_networkx_nodes(G_2023_shop3_conviction, pos, node_color=node_colors, node_size=node_sizes, cmap=cmap, alpha=0.8)
edges = nx.draw_networkx_edges(G_2023_shop3_conviction, pos, arrowstyle='->', arrowsize=10, edge_color='grey', alpha=0.5)
labels = nx.draw_networkx_labels(G_2023_shop3_conviction, pos, font_size=8)

plt.colorbar(nodes, label='Normalized Conviction')
plt.title('Network Graph of 2023 Shop 3 Based on Conviction')
plt.axis('off')
plt.show()

In [ ]:
print(max_conviction)

# Filtered top 30 and non redundant

In [ ]:
rules_2023_shop3_nonredundant.head()

In [ ]:
rules_2023_shop3_nonredundant_top30 = rules_2023_shop3_nonredundant.copy(deep=True)

In [ ]:
rules_2023_shop3_nonredundant_top30_lift = rules_2023_shop3_nonredundant.copy(deep=True)

In [ ]:
# Select the top 30 rules based on lift or another criterion
rules_2023_shop3_nonredundant_top30_lift = rules_2023_shop3_nonredundant_top30_lift.sort_values(by='lift', ascending=False).head(30)

In [ ]:
rules_2023_shop3_nonredundant_top30_lift.head()

In [ ]:
# Select the top 30 rules based on confidence or another criterion
rules_2023_shop3_nonredundant_top30 = rules_2023_shop3_nonredundant_top30.sort_values(by='confidence', ascending=False).head(30)

In [ ]:
rules_2023_shop3_nonredundant_top30.info()

In [ ]:
rules_2023_shop3_nonredundant_top30['LHS'] = [','.join(list(x)) for x in rules_2023_shop3_nonredundant_top30['antecedents']]
rules_2023_shop3_nonredundant_top30['RHS'] = [','.join(list(x)) for x in rules_2023_shop3_nonredundant_top30['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2023_shop3_nonredundant_top30.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2023_shop3_nonredundant_top30, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2023 Shop 3")
plt.axis('off')
plt.show()


## Based on Indegree

In [ ]:
G_2023_shop3 = nx.DiGraph()

# Assuming 'rules_sorted_2023_shop3' contains your association rules for shop 3 in 2023
for _, row in rules_2023_shop3_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as the weight
    G_2023_shop3.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2023_shop3 = nx.spring_layout(G_2023_shop3)

# Calculate node sizes based on indegree
in_degrees_2023_shop3 = dict(G_2023_shop3.in_degree())
node_sizes_2023_shop3 = [in_degrees_2023_shop3[node] * 100 + 1000 for node in G_2023_shop3.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2023_shop3 = (np.array(node_sizes_2023_shop3) - np.min(node_sizes_2023_shop3)) / (np.max(node_sizes_2023_shop3) - np.min(node_sizes_2023_shop3))

# Define a colormap from orange to red
cmap_2023_shop3 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop3 = [cmap_2023_shop3(size) for size in normalized_sizes_2023_shop3]

# Drawing the network
plt.figure(figsize=(30, 24))
nx.draw_networkx_nodes(G_2023_shop3, pos_2023_shop3, node_size=node_sizes_2023_shop3, node_color=node_colors_2023_shop3, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop3, pos_2023_shop3, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2023_shop3, pos_2023_shop3, font_size=8)

plt.title('Top 5 Association Rules Network for 2023 Shop 3 with Color Gradient')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2023_shop3_lift = nx.DiGraph()

lift_summary_2023_shop3 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop3_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop3_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop3:
        lift_summary_2023_shop3[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop3[consequents]['count'] += 1
    else:
        lift_summary_2023_shop3[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop3:
    lift_summary_2023_shop3[item]['avg_lift'] = lift_summary_2023_shop3[item]['total_lift'] / lift_summary_2023_shop3[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop3_lift = [lift_summary_2023_shop3[node]['avg_lift'] * 500 if node in lift_summary_2023_shop3 else 1000 for node in G_2023_shop3_lift.nodes()]

pos_2023_shop3_lift = nx.spring_layout(G_2023_shop3_lift)

# Colormap from orange to red
cmap_2023_shop3_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop3_lift = np.array(node_sizes_2023_shop3_lift)
normalized_sizes_2023_shop3_lift = (normalized_sizes_2023_shop3_lift - min(normalized_sizes_2023_shop3_lift)) / (max(normalized_sizes_2023_shop3_lift) - min(normalized_sizes_2023_shop3_lift))
node_colors_2023_shop3_lift = [cmap_2023_shop3_lift(size) for size in normalized_sizes_2023_shop3_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(60, 32))
nx.draw_networkx_nodes(G_2023_shop3_lift, pos_2023_shop3_lift, node_size=node_sizes_2023_shop3_lift, node_color=node_colors_2023_shop3_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop3_lift, pos_2023_shop3_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop3_lift, pos_2023_shop3_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 3 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop3_lift = nx.DiGraph()

lift_summary_2023_shop3 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop3_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop3_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop3:
        lift_summary_2023_shop3[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop3[consequents]['count'] += 1
    else:
        lift_summary_2023_shop3[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop3:
    lift_summary_2023_shop3[item]['avg_lift'] = lift_summary_2023_shop3[item]['total_lift'] / lift_summary_2023_shop3[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop3_lift = [lift_summary_2023_shop3[node]['avg_lift'] * 500 if node in lift_summary_2023_shop3 else 1000 for node in G_2023_shop3_lift.nodes()]

pos_2023_shop3_lift = nx.spring_layout(G_2023_shop3_lift)

# Colormap from orange to red
cmap_2023_shop3_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop3_lift = np.array(node_sizes_2023_shop3_lift)
normalized_sizes_2023_shop3_lift = (normalized_sizes_2023_shop3_lift - min(normalized_sizes_2023_shop3_lift)) / (max(normalized_sizes_2023_shop3_lift) - min(normalized_sizes_2023_shop3_lift))
node_colors_2023_shop3_lift = [cmap_2023_shop3_lift(size) for size in normalized_sizes_2023_shop3_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2023_shop3_lift, pos_2023_shop3_lift, node_size=node_sizes_2023_shop3_lift, node_color=node_colors_2023_shop3_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop3_lift, pos_2023_shop3_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop3_lift, pos_2023_shop3_lift, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 3 Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop3 = nx.DiGraph()

# Assuming 'rules_2023_shop3_nonredundant' contains association rules for 2023 Shop 3
for _, row in rules_2023_shop3_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop3.add_edge(antecedents, consequents, confidence=row['confidence'])

# Calculate node sizes based on indegree
in_degrees_2023_shop3 = dict(G_2023_shop3.in_degree())
node_sizes_2023_shop3 = [in_degrees_2023_shop3[node] * 100 + 1000 for node in G_2023_shop3.nodes()]

# Edge width based on confidence
edge_widths_2023_shop3 = [G_2023_shop3[u][v]['confidence'] * 15 for u, v in G_2023_shop3.edges()]

# Assuming normalized sizes and color mapping is desired
normalized_sizes_2023_shop3 = np.array(list(in_degrees_2023_shop3.values()))
normalized_sizes_2023_shop3 = (normalized_sizes_2023_shop3 - min(normalized_sizes_2023_shop3)) / (max(normalized_sizes_2023_shop3) - min(normalized_sizes_2023_shop3))
cmap_2023_shop3 = mcolors.LinearSegmentedColormap.from_list("MyCMapName", ["blue", "red"])

# Node colors based on a property (here, using indegree for example)
node_colors_2023_shop3 = [cmap_2023_shop3(size) for size in normalized_sizes_2023_shop3]

# Layout
pos_2023_shop3 = nx.spring_layout(G_2023_shop3, seed=42)

# Drawing
plt.figure(figsize=(40, 30))
nx.draw_networkx_edges(G_2023_shop3, pos_2023_shop3, width=edge_widths_2023_shop3, edge_color='grey', alpha=0.5)
nx.draw_networkx_nodes(G_2023_shop3, pos_2023_shop3, node_size=node_sizes_2023_shop3, node_color=node_colors_2023_shop3, alpha=0.8)
nx.draw_networkx_labels(G_2023_shop3, pos_2023_shop3, font_size=8, font_color='black')
plt.title('Network Graph of Association Rules Based on Confidence for 2023 Shop 3')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop3 = nx.DiGraph()

convictions_2023 = []  # To store conviction values for normalization

for _, row in rules_2023_shop3_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    conviction = row['conviction']  # Assuming each row has a 'conviction' column
    convictions_2023.append(conviction)
    G_2023_shop3.add_edge(antecedents, consequents, weight=row['confidence'], conviction=conviction)

# Normalize conviction values for edge width
max_conviction_2023 = max(convictions_2023)
min_conviction_2023 = min(convictions_2023)
conviction_range_2023 = max_conviction_2023 - min_conviction_2023
edge_widths_2023 = [(G_2023_shop3[u][v]['conviction'] - min_conviction_2023) / conviction_range_2023 * 30 + 0.5 for u, v in G_2023_shop3.edges()]  # Scale and offset widths

pos_2023_shop3 = nx.spring_layout(G_2023_shop3)

# Node sizes and colors as before
in_degrees_2023_shop3 = dict(G_2023_shop3.in_degree())
node_sizes_2023_shop3 = [in_degrees_2023_shop3[node] * 100 + 1000 for node in G_2023_shop3.nodes()]
normalized_sizes_2023_shop3 = (np.array(node_sizes_2023_shop3) - min(node_sizes_2023_shop3)) / (max(node_sizes_2023_shop3) - min(node_sizes_2023_shop3))
cmap_2023_shop3 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop3 = [cmap_2023_shop3(size) for size in normalized_sizes_2023_shop3]

# Drawing the network with conviction-based edge widths
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2023_shop3, pos_2023_shop3, node_size=node_sizes_2023_shop3, node_color=node_colors_2023_shop3, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop3, pos_2023_shop3, width=edge_widths_2023, edge_color='grey', alpha=0.5)
nx.draw_networkx_labels(G_2023_shop3, pos_2023_shop3, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 3 with Conviction-based Edge Thickness')
plt.axis('off')
plt.show()


In [ ]:
#the node sizes are based on the indegree where each node acts as a consequent,

In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2023_shop3_nonredundant_top30.loc[:, 'antecedents_'] = rules_2023_shop3_nonredundant_top30['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2023_shop3_nonredundant_top30.loc[:, 'consequents_'] = rules_2023_shop3_nonredundant_top30['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2023_shop3_nonredundant_top30, x='consequents_', y='antecedents_', s=rules_2023_shop3_nonredundant_top30['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2023_shop3_nonredundant_top30.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2023 Shop 3", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
print(rules_2023_shop3_nonredundant_top30[['antecedents', 'consequents']].head(10))

## Monetary Analysis of the Top Rules

In [ ]:
# Step 1: Extract unique product codes from the rules
unique_product_codes = set()
for idx, row in rules_2023_shop3_nonredundant_top30.iterrows():
    # Unite antecedents and consequents into a single set
    all_products = row['antecedents'].union(row['consequents'])
    unique_product_codes.update(all_products)

# Step 2: Create a DataFrame with unique product codes and their unit prices
# Filter sales_data for these product codes and drop duplicates to ensure each product code appears once
product_prices = sales_data[sales_data['product_code'].isin(unique_product_codes)][['product_code', 'unit_price']].drop_duplicates()

# Create a dictionary for quick lookup of prices
price_dict = product_prices.set_index('product_code')['unit_price'].to_dict()

# Step 3: Calculate the total price for each rule and collect data for the new DataFrame
data_for_new_df = []
for idx, row in rules_2023_shop3_nonredundant_top30.iterrows():
    # Combine sets of antecedents and consequents
    all_products = row['antecedents'].union(row['consequents'])
    total_price = sum(price_dict.get(prod, 0) for prod in all_products)  # Use get to avoid KeyError if any product code is missing
    data_for_new_df.append({
        'antecedents': row['antecedents'],
        'consequents': row['consequents'],
        'total_unit_price': total_price
    })

# Create a new DataFrame with only antecedents, consequents, and total unit price
rules_and_price_shop3_2023 = pd.DataFrame(data_for_new_df)

# Sort the DataFrame by total_unit_price in descending order
rules_and_price_shop3_2023_sorted = rules_and_price_shop3_2023.sort_values(by='total_unit_price', ascending=False)

# Print the sorted DataFrame to see the specified columns along with their total unit prices
print(rules_and_price_shop3_2023_sorted)


In [ ]:
# Convert 'trx_date' to datetime format if not already done so
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Add a 'year' column if it doesn't exist
sales_data['year'] = sales_data['trx_date'].dt.year

# Step 2: Filter sales_data for shop3 and the year 2023
filtered_sales_data_2023_shop3 = sales_data[(sales_data['shopID'] == 3) & (sales_data['year'] == 2023)]

# Step 3: Initialize a list to hold the rule data including total sales value
rule_sales_summaries_2023_shop3 = []

# Step 4: Iterate through each rule in the DataFrame
for index, rule in rules_2023_shop3_nonredundant_top30.iterrows():
    all_products = rule['antecedents'].union(rule['consequents'])

    # Filter the sales data for transactions that include all products in the rule
    # This step ensures that we are only considering transactions that contain all specified products
    mask = filtered_sales_data_2023_shop3['product_code'].isin(all_products)
    transaction_ids = filtered_sales_data_2023_shop3[mask].groupby('trx_id').filter(lambda x: all_products.issubset(set(x['product_code']))).trx_id.unique()
    matched_transactions = filtered_sales_data_2023_shop3[filtered_sales_data_2023_shop3['trx_id'].isin(transaction_ids)]
    
    # Step 5: Sum the total_value of these transactions
    total_sales_value = matched_transactions['total_value'].sum()

    # Append the result to the list
    rule_sales_summaries_2023_shop3.append({
        'antecedents': rule['antecedents'],
        'consequents': rule['consequents'],
        'total_sales_value': total_sales_value
    })

# Step 6: Convert the list to a DataFrame
rules_sales_values_df_2023_shop3 = pd.DataFrame(rule_sales_summaries_2023_shop3)

# Step 7: Sort by total_sales_value in descending order
sorted_rules_sales_values_df_2023_shop3 = rules_sales_values_df_2023_shop3.sort_values(by='total_sales_value', ascending=False)

# Step 8: Print the sorted DataFrame
print(sorted_rules_sales_values_df_2023_shop3)


## Narrowing down based on Lift

In [ ]:
rules_2023_shop3_nonredundant_top30_lift['LHS'] = [','.join(list(x)) for x in rules_2023_shop3_nonredundant_top30_lift['antecedents']]
rules_2023_shop3_nonredundant_top30_lift['RHS'] = [','.join(list(x)) for x in rules_2023_shop3_nonredundant_top30_lift['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2023_shop3_nonredundant_top30_lift.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2023_shop3_nonredundant_top30_lift, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2023 Shop 3")
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop3_lift = nx.DiGraph()

lift_summary_2023_shop3 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop3_nonredundant_top30_lift.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop3_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop3:
        lift_summary_2023_shop3[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop3[consequents]['count'] += 1
    else:
        lift_summary_2023_shop3[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop3:
    lift_summary_2023_shop3[item]['avg_lift'] = lift_summary_2023_shop3[item]['total_lift'] / lift_summary_2023_shop3[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop3_lift = [lift_summary_2023_shop3[node]['avg_lift'] * 500 if node in lift_summary_2023_shop3 else 1000 for node in G_2023_shop3_lift.nodes()]

pos_2023_shop3_lift = nx.spring_layout(G_2023_shop3_lift)

# Colormap from orange to red
cmap_2023_shop3_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop3_lift = np.array(node_sizes_2023_shop3_lift)
normalized_sizes_2023_shop3_lift = (normalized_sizes_2023_shop3_lift - min(normalized_sizes_2023_shop3_lift)) / (max(normalized_sizes_2023_shop3_lift) - min(normalized_sizes_2023_shop3_lift))
node_colors_2023_shop3_lift = [cmap_2023_shop3_lift(size) for size in normalized_sizes_2023_shop3_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(60, 32))
nx.draw_networkx_nodes(G_2023_shop3_lift, pos_2023_shop3_lift, node_size=node_sizes_2023_shop3_lift, node_color=node_colors_2023_shop3_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop3_lift, pos_2023_shop3_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop3_lift, pos_2023_shop3_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 3 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2023_shop3_nonredundant_top30_lift.loc[:, 'antecedents_'] = rules_2023_shop3_nonredundant_top30_lift['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2023_shop3_nonredundant_top30_lift.loc[:, 'consequents_'] = rules_2023_shop3_nonredundant_top30_lift['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2023_shop3_nonredundant_top30_lift, x='consequents_', y='antecedents_', s=rules_2023_shop3_nonredundant_top30_lift['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2023_shop3_nonredundant_top30_lift.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2023 Shop 3", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
# Print only the 'antecedents' and 'consequents' columns of the top 10 rules
print(rules_2023_shop3_nonredundant_top30_lift[['antecedents', 'consequents']].head(10))


# Shop4 2021 SEARCH 6

In [ ]:
# Filter for records from shop ID 1 and the year 1993
sales_data_2021_shop4 = sales_data_shop4[(sales_data_shop4['trx_date'].dt.year == 2021)]

In [ ]:
sales_data_2021_shop4.head()

In [ ]:
# Grouping product codes by trx_id
transactions_2021_shop4 = sales_data_2021_shop4.groupby('trx_id')['product_code'].apply(list).tolist()

In [ ]:
# Encode transaction data
encoder_2021_shop4 = TransactionEncoder()
trans_array_2021_shop4 = encoder_2021_shop4.fit(transactions_2021_shop4).transform(transactions_2021_shop4)
trans_df_2021_shop4 = pd.DataFrame(trans_array_2021_shop4, columns=encoder_2021_shop4.columns_)

In [ ]:
# Find frequent itemsets
frequent_itemsets_2021_shop4 = apriori(trans_df_2021_shop4, min_support=0.001, use_colnames=True)

In [ ]:
print(frequent_itemsets_2021_shop4)

In [ ]:
## Generate the association rules - by confidence
rulesConfidence_2021_shop4 = association_rules(frequent_itemsets_2021_shop4, metric="confidence", min_threshold=0.10)
rulesConfidence_2021_shop4.sort_values(by='confidence', ascending=False, inplace=True)


In [ ]:
# Note: This is a hypothetical example, as I can't directly manipulate or access dataframes not provided within this code cell.

# Replace frozen sets with strings
rulesConfidence_2021_shop4['antecedents_'] = rulesConfidence_2021_shop4['antecedents'].apply(lambda a: ','.join(list(a)))
rulesConfidence_2021_shop4['consequents_'] = rulesConfidence_2021_shop4['consequents'].apply(lambda a: ','.join(list(a)))

# Draw
fig, ax = plt.subplots(figsize=(14,30))
ax = plt.scatter(data=rulesConfidence_2021_shop4, x='consequents_', y='antecedents_', s=rulesConfidence_2021_shop4['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rulesConfidence_2021_shop4.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2021 Shop 4", fontsize=14)
plt.xlabel("Consequents (RHS)", fontsize=12)
plt.ylabel("Antecedents (LHS)", fontsize=12)
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate(rotation=45)
plt.tight_layout()

plt.show()



## Eliminate Redundant Rules

In [ ]:
rulesConfidence_2021_shop4.head()

In [ ]:
rulesConfidence_2021_shop4.info()

In [ ]:
# If 'antecedents' and 'consequents' are not frozensets, convert them
rules_2021_shop4_redundant = rulesConfidence_2021_shop4.copy()
rules_2021_shop4_redundant['antecedents'] = rules_2021_shop4_redundant['antecedents'].apply(lambda x: frozenset(x))
rules_2021_shop4_redundant['consequents'] = rules_2021_shop4_redundant['consequents'].apply(lambda x: frozenset(x))


In [ ]:
# Create a copy of the DataFrame to avoid modifying the original while iterating
rules_2021_shop4_checkredundance = rules_2021_shop4_redundant.copy()

# List to hold non-redundant rules
non_redundant_rules_2021_shop4 = []

for _, rule in rules_2021_shop4_checkredundance.iterrows():
    if not is_redundant(rule, rules_2021_shop4_checkredundance):
        non_redundant_rules_2021_shop4.append(rule)

# Convert the list back to a DataFrame
rules_2021_shop4_nonredundant = pd.DataFrame(non_redundant_rules_2021_shop4).reset_index(drop=True)


In [ ]:
print(rules_2021_shop4_nonredundant.head())
print(f"Original number of rules: {len(rules_2021_shop4_redundant)}")
print(f"Number of non-redundant rules: {len(rules_2021_shop4_nonredundant)}")


In [ ]:
rules_2021_shop4_nonredundant.head()

In [ ]:
rules_2021_shop4_redundant.info()

In [ ]:
rules_2021_shop4_nonredundant.head()

In [ ]:
rules_2021_shop4_nonredundant_backup = rules_2021_shop4_nonredundant.copy()

In [ ]:
rules_2021_shop4_nonredundant_unfiltered = rules_2021_shop4_nonredundant.copy()

In [ ]:
#reset
#rules_2021_shop4_nonredundant = rules_2021_shop4_nonredundant_backup.copy()

# Redundant and Unfiltered

In [ ]:
rules_2021_shop4_redundant.head()

In [ ]:
G_2021_shop4 = nx.DiGraph()

# Iterate through rules for 2021 Shop 4 and add edges to the graph
for _, row in rules_2021_shop1_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as weight
    G_2021_shop4.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2021_shop4 = nx.spring_layout(G_2021_shop4)

# Recalculate node sizes based on indegree or another metric
in_degrees_2021_shop4 = dict(G_2021_shop4.in_degree())
node_sizes_2021_shop4 = [in_degrees_2021_shop4[node] * 100 + 1000 for node in G_2021_shop4.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2021_shop4 = (np.array(node_sizes_2021_shop4) - np.min(node_sizes_2021_shop4)) / (np.max(node_sizes_2021_shop4) - np.min(node_sizes_2021_shop4))

# Define a colormap from orange to red for visual consistency
cmap_2021_shop4 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2021_shop4 = [cmap_2021_shop4(size) for size in normalized_sizes_2021_shop4]

# Drawing the network
plt.figure(figsize=(60, 43))
nx.draw_networkx_nodes(G_2021_shop4, pos_2021_shop4, node_size=node_sizes_2021_shop4, node_color=node_colors_2021_shop4, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop4, pos_2021_shop4, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2021_shop4, pos_2021_shop4, font_size=10)

plt.title('Top 5 Association Rules Network for 2021 Shop 4 with Color Gradient')
plt.axis('off')
plt.show()


### Based on LIFT

In [ ]:
G_2021_shop4_lift = nx.DiGraph()

lift_summary_2021_shop4 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2021_shop1_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop4_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2021_shop4:
        lift_summary_2021_shop4[consequents]['total_lift'] += row['lift']
        lift_summary_2021_shop4[consequents]['count'] += 1
    else:
        lift_summary_2021_shop4[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2021_shop4:
    lift_summary_2021_shop4[item]['avg_lift'] = lift_summary_2021_shop4[item]['total_lift'] / lift_summary_2021_shop4[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2021_shop4_lift = [lift_summary_2021_shop4[node]['avg_lift'] * 500 if node in lift_summary_2021_shop4 else 1000 for node in G_2021_shop4_lift.nodes()]

pos_2021_shop4_lift = nx.spring_layout(G_2021_shop4_lift)

# Colormap from orange to red
cmap_2021_shop4_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2021_shop4_lift = np.array(node_sizes_2021_shop4_lift)
normalized_sizes_2021_shop4_lift = (normalized_sizes_2021_shop4_lift - min(normalized_sizes_2021_shop4_lift)) / (max(normalized_sizes_2021_shop4_lift) - min(normalized_sizes_2021_shop4_lift))
node_colors_2021_shop4_lift = [cmap_2021_shop4_lift(size) for size in normalized_sizes_2021_shop4_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(70, 52))
nx.draw_networkx_nodes(G_2021_shop4_lift, pos_2021_shop4_lift, node_size=node_sizes_2021_shop4_lift, node_color=node_colors_2021_shop4_lift, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop4_lift, pos_2021_shop4_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2021_shop4_lift, pos_2021_shop4_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2021 Shop 4 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


# nonredundant_unfiltered

## Based on Indegree

In [ ]:
G_2021_shop4_nonredundant_unfiltered = nx.DiGraph()

# Iterating through each row to add edges with confidence as an attribute
for _, row in rules_2021_shop1_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop4_nonredundant_unfiltered.add_edge(antecedents, consequents, weight=row['confidence'])

# Defining the layout for network visualization
pos_2021_shop4_nonredundant_unfiltered = nx.spring_layout(G_2021_shop4_nonredundant_unfiltered)

# Calculating node sizes based on the indegree for color mapping
in_degrees_2021_shop4_nonredundant_unfiltered = dict(G_2021_shop4_nonredundant_unfiltered.in_degree())
node_sizes_2021_shop4_nonredundant_unfiltered = [in_degrees_2021_shop4_nonredundant_unfiltered[node] * 100 + 1000 for node in G_2021_shop4_nonredundant_unfiltered.nodes()]

# Normalizing node sizes for color mapping
normalized_sizes_2021_shop4_nonredundant_unfiltered = (np.array(node_sizes_2021_shop4_nonredundant_unfiltered) - min(node_sizes_2021_shop4_nonredundant_unfiltered)) / (max(node_sizes_2021_shop4_nonredundant_unfiltered) - min(node_sizes_2021_shop4_nonredundant_unfiltered))

# Defining a color map from orange to red
cmap_2021_shop4_nonredundant_unfiltered = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])

# Mapping normalized node sizes to colors
node_colors_2021_shop4_nonredundant_unfiltered = [cmap_2021_shop4_nonredundant_unfiltered(size) for size in normalized_sizes_2021_shop4_nonredundant_unfiltered]

# Drawing the network with node sizes and colors
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2021_shop4_nonredundant_unfiltered, pos_2021_shop4_nonredundant_unfiltered, node_size=node_sizes_2021_shop4_nonredundant_unfiltered, node_color=node_colors_2021_shop4_nonredundant_unfiltered, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop4_nonredundant_unfiltered, pos_2021_shop4_nonredundant_unfiltered, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2021_shop4_nonredundant_unfiltered, pos_2021_shop4_nonredundant_unfiltered, font_size=10)

plt.title('Node Size Based on Indegree for 2021 Shop 4')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2021_shop4_lift = nx.DiGraph()

lift_summary_2021_shop4 = {}

for _, row in rules_2021_shop1_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop4_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Update lift summary for consequents
    if consequents in lift_summary_2021_shop4:
        lift_summary_2021_shop4[consequents]['total_lift'] += row['lift']
        lift_summary_2021_shop4[consequents]['count'] += 1
    else:
        lift_summary_2021_shop4[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculate average lift for each consequent
for item in lift_summary_2021_shop4:
    lift_summary_2021_shop4[item]['avg_lift'] = lift_summary_2021_shop4[item]['total_lift'] / lift_summary_2021_shop4[item]['count']

# Node sizes based on average lift, scaled appropriately
node_sizes_2021_shop4_lift = [lift_summary_2021_shop4[node]['avg_lift'] * 500 if node in lift_summary_2021_shop4 else 1000 for node in G_2021_shop4_lift.nodes()]

pos_2021_shop4_lift = nx.spring_layout(G_2021_shop4_lift)

# Use a colormap that transitions from orange to red
cmap_2021_shop4_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2021_shop4_lift = np.array(node_sizes_2021_shop4_lift)
normalized_sizes_2021_shop4_lift = (normalized_sizes_2021_shop4_lift - min(normalized_sizes_2021_shop4_lift)) / (max(normalized_sizes_2021_shop4_lift) - min(normalized_sizes_2021_shop4_lift))
node_colors_2021_shop4_lift = [cmap_2021_shop4_lift(size) for size in normalized_sizes_2021_shop4_lift]

plt.figure(figsize=(60, 44))
nx.draw_networkx_nodes(G_2021_shop4_lift, pos_2021_shop4_lift, node_size=node_sizes_2021_shop4_lift, node_color=node_colors_2021_shop4_lift, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop4_lift, pos_2021_shop4_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2021_shop4_lift, pos_2021_shop4_lift, font_size=10)

plt.title('Node Size Based on Average Lift for 2021 Shop 4')
plt.axis('off')
plt.show()


# Based on conviction

In [ ]:
rules_2021_shop4_nonredundant.head()

In [ ]:
G_2021_shop4_conviction = nx.DiGraph()

conviction_summary_2021_shop4 = {}
count_summary_2021_shop4 = {}

# Assuming 'rules_2021_shop4_nonredundant_unfiltered' contains a 'conviction' column
for _, row in rules_2021_shop4_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop4_conviction.add_edge(antecedents, consequents, weight=row['confidence'], conviction=row['conviction'])
    
    # Aggregate conviction values and count occurrences
    if consequents in conviction_summary_2021_shop4:
        conviction_summary_2021_shop4[consequents] += row['conviction']
        count_summary_2021_shop4[consequents] += 1
    else:
        conviction_summary_2021_shop4[consequents] = row['conviction']
        count_summary_2021_shop4[consequents] = 1

# Calculate average conviction values
average_conviction_summary_2021_shop4 = {k: conviction_summary_2021_shop4[k] / count_summary_2021_shop4[k] for k in conviction_summary_2021_shop4}

# Normalize conviction values for sizing and coloring
max_conviction = max(average_conviction_summary_2021_shop4.values())
min_conviction = min(average_conviction_summary_2021_shop4.values())
conviction_range = max_conviction - min_conviction
node_colors = []
node_sizes = []

for node in G_2021_shop4_conviction.nodes():
    # Get the average conviction value for the node, defaulting to min if not present
    conviction = average_conviction_summary_2021_shop4.get(node, min_conviction)
    # Normalize between 0 and 1
    normalized_conviction = (conviction - min_conviction) / conviction_range if conviction_range else 0
    node_colors.append(normalized_conviction)
    node_sizes.append(normalized_conviction * 1000 + 100)  # Scale and offset size

# Use a colormap for coloring nodes based on conviction
cmap = plt.cm.coolwarm

pos = nx.spring_layout(G_2021_shop4_conviction)

plt.figure(figsize=(40, 22))
nodes = nx.draw_networkx_nodes(G_2021_shop4_conviction, pos, node_color=node_colors, node_size=node_sizes, cmap=cmap, alpha=0.8)
edges = nx.draw_networkx_edges(G_2021_shop4_conviction, pos, arrowstyle='->', arrowsize=10, edge_color='grey', alpha=0.5)
labels = nx.draw_networkx_labels(G_2021_shop4_conviction, pos, font_size=8)

plt.colorbar(nodes, label='Normalized Conviction')
plt.title('Network Graph of 2021 Shop 4 Based on Conviction')
plt.axis('off')
plt.show()

In [ ]:
print(max_conviction)

# Filtered top 30 and non redundant

In [ ]:
rules_2021_shop4_nonredundant.head()

In [ ]:
# Assuming rules_1992_shop2_nonredundant is your sorted rules DataFrame for shop2
rules_2021_shop4_nonredundant_top30 = rules_2021_shop4_nonredundant.copy(deep=True)

In [ ]:
rules_2021_shop4_nonredundant_top30_lift = rules_2021_shop4_nonredundant.copy(deep=True)

In [ ]:
# Select the top 30 rules based on lift or another criterion
rules_2021_shop4_nonredundant_top30_lift = rules_2021_shop4_nonredundant_top30_lift.sort_values(by='lift', ascending=False).head(30)

In [ ]:
rules_2021_shop4_nonredundant_top30_lift.head()

In [ ]:
# Select the top 30 rules based on confidence or another criterion
rules_2021_shop4_nonredundant_top30 = rules_2021_shop4_nonredundant_top30.sort_values(by='confidence', ascending=False).head(30)

In [ ]:
rules_2021_shop4_nonredundant_top30.info()

In [ ]:
rules_2021_shop4_nonredundant_top30['LHS'] = [','.join(list(x)) for x in rules_2021_shop4_nonredundant_top30['antecedents']]
rules_2021_shop4_nonredundant_top30['RHS'] = [','.join(list(x)) for x in rules_2021_shop4_nonredundant_top30['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2021_shop4_nonredundant_top30.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2021_shop4_nonredundant_top30, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2021 Shop 4")
plt.axis('off')
plt.show()


## Based on Indegree

In [ ]:
G_2021_shop4 = nx.DiGraph()

# Assuming 'rules_sorted_2021_shop4' contains association rules for shop 4 in 2021
for _, row in rules_2021_shop4_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as the weight
    G_2021_shop4.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2021_shop4 = nx.spring_layout(G_2021_shop4)

# Calculate node sizes based on indegree
in_degrees_2021_shop4 = dict(G_2021_shop4.in_degree())
node_sizes_2021_shop4 = [in_degrees_2021_shop4[node] * 100 + 1000 for node in G_2021_shop4.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2021_shop4 = (np.array(node_sizes_2021_shop4) - np.min(node_sizes_2021_shop4)) / (np.max(node_sizes_2021_shop4) - np.min(node_sizes_2021_shop4))

# Define a colormap from orange to red
cmap_2021_shop4 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2021_shop4 = [cmap_2021_shop4(size) for size in normalized_sizes_2021_shop4]

# Drawing the network
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2021_shop4, pos_2021_shop4, node_size=node_sizes_2021_shop4, node_color=node_colors_2021_shop4, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop4, pos_2021_shop4, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2021_shop4, pos_2021_shop4, font_size=8)

plt.title('Top 5 Association Rules Network for 2021 Shop 4 with Color Gradient')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2021_shop4_lift = nx.DiGraph()

lift_summary_2021_shop4 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2021_shop4_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop4_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2021_shop4:
        lift_summary_2021_shop4[consequents]['total_lift'] += row['lift']
        lift_summary_2021_shop4[consequents]['count'] += 1
    else:
        lift_summary_2021_shop4[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2021_shop4:
    lift_summary_2021_shop4[item]['avg_lift'] = lift_summary_2021_shop4[item]['total_lift'] / lift_summary_2021_shop4[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2021_shop4_lift = [lift_summary_2021_shop4[node]['avg_lift'] * 500 if node in lift_summary_2021_shop4 else 1000 for node in G_2021_shop4_lift.nodes()]

pos_2021_shop4_lift = nx.spring_layout(G_2021_shop4_lift)

# Colormap from orange to red
cmap_2021_shop4_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2021_shop4_lift = np.array(node_sizes_2021_shop4_lift)
normalized_sizes_2021_shop4_lift = (normalized_sizes_2021_shop4_lift - min(normalized_sizes_2021_shop4_lift)) / (max(normalized_sizes_2021_shop4_lift) - min(normalized_sizes_2021_shop4_lift))
node_colors_2021_shop4_lift = [cmap_2021_shop4_lift(size) for size in normalized_sizes_2021_shop4_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2021_shop4_lift, pos_2021_shop4_lift, node_size=node_sizes_2021_shop4_lift, node_color=node_colors_2021_shop4_lift, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop4_lift, pos_2021_shop4_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2021_shop4_lift, pos_2021_shop4_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2021 Shop 4 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2021_shop4_lift = nx.DiGraph()

lift_summary_2021_shop4 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2021_shop4_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop4_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2021_shop4:
        lift_summary_2021_shop4[consequents]['total_lift'] += row['lift']
        lift_summary_2021_shop4[consequents]['count'] += 1
    else:
        lift_summary_2021_shop4[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2021_shop4:
    lift_summary_2021_shop4[item]['avg_lift'] = lift_summary_2021_shop4[item]['total_lift'] / lift_summary_2021_shop4[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2021_shop4_lift = [lift_summary_2021_shop4[node]['avg_lift'] * 500 if node in lift_summary_2021_shop4 else 1000 for node in G_2021_shop4_lift.nodes()]

pos_2021_shop4_lift = nx.spring_layout(G_2021_shop4_lift)

# Colormap from orange to red
cmap_2021_shop4_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2021_shop4_lift = np.array(node_sizes_2021_shop4_lift)
normalized_sizes_2021_shop4_lift = (normalized_sizes_2021_shop4_lift - min(normalized_sizes_2021_shop4_lift)) / (max(normalized_sizes_2021_shop4_lift) - min(normalized_sizes_2021_shop4_lift))
node_colors_2021_shop4_lift = [cmap_2021_shop4_lift(size) for size in normalized_sizes_2021_shop4_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2021_shop4_lift, pos_2021_shop4_lift, node_size=node_sizes_2021_shop4_lift, node_color=node_colors_2021_shop4_lift, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop4_lift, pos_2021_shop4_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2021_shop4_lift, pos_2021_shop4_lift, font_size=10)

plt.title('Top 5 Association Rules Network for 2021 Shop 4 Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2021_shop4 = nx.DiGraph()

# Assuming 'rules_2021_shop4_nonredundant' contains association rules for 2021 Shop 4
for _, row in rules_2021_shop4_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop4.add_edge(antecedents, consequents, confidence=row['confidence'])

# Calculate node sizes based on indegree
in_degrees_2021_shop4 = dict(G_2021_shop4.in_degree())
node_sizes_2021_shop4 = [in_degrees_2021_shop4[node] * 100 + 1000 for node in G_2021_shop4.nodes()]

# Edge width based on confidence
edge_widths_2021_shop4 = [G_2021_shop4[u][v]['confidence'] * 15 for u, v in G_2021_shop4.edges()]

# Assuming normalized sizes and color mapping is desired
normalized_sizes_2021_shop4 = np.array(list(in_degrees_2021_shop4.values()))
normalized_sizes_2021_shop4 = (normalized_sizes_2021_shop4 - min(normalized_sizes_2021_shop4)) / (max(normalized_sizes_2021_shop4) - min(normalized_sizes_2021_shop4))
cmap_2021_shop4 = mcolors.LinearSegmentedColormap.from_list("MyCMapName", ["blue", "red"])

# Node colors based on a property (here, using indegree for example)
node_colors_2021_shop4 = [cmap_2021_shop4(size) for size in normalized_sizes_2021_shop4]

# Layout
pos_2021_shop4 = nx.spring_layout(G_2021_shop4, seed=42)

# Drawing
plt.figure(figsize=(40, 30))
nx.draw_networkx_edges(G_2021_shop4, pos_2021_shop4, width=edge_widths_2021_shop4, edge_color='grey', alpha=0.5)
nx.draw_networkx_nodes(G_2021_shop4, pos_2021_shop4, node_size=node_sizes_2021_shop4, node_color=node_colors_2021_shop4, alpha=0.8)
nx.draw_networkx_labels(G_2021_shop4, pos_2021_shop4, font_size=8, font_color='black')
plt.title('Network Graph of Association Rules Based on Confidence for 2021 Shop 4')
plt.axis('off')
plt.show()


In [ ]:
G_2021_shop4 = nx.DiGraph()

convictions_2021_shop4 = []  # To store conviction values for normalization

for _, row in rules_2021_shop4_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    conviction_2021_shop4 = row['conviction']  # Assuming each row has a 'conviction' column
    convictions_2021_shop4.append(conviction_2021_shop4)
    G_2021_shop4.add_edge(antecedents, consequents, weight=row['confidence'], conviction=conviction_2021_shop4)

# Normalize conviction values for edge width
max_conviction_2021_shop4 = max(convictions_2021_shop4)
min_conviction_2021_shop4 = min(convictions_2021_shop4)
conviction_range_2021_shop4 = max_conviction_2021_shop4 - min_conviction_2021_shop4
edge_widths_2021_shop4 = [(G_2021_shop4[u][v]['conviction'] - min_conviction_2021_shop4) / conviction_range_2021_shop4 * 30 + 0.5 for u, v in G_2021_shop4.edges()]  # Scale and offset widths

pos_2021_shop4 = nx.spring_layout(G_2021_shop4)

# Node sizes and colors as before
in_degrees_2021_shop4 = dict(G_2021_shop4.in_degree())
node_sizes_2021_shop4 = [in_degrees_2021_shop4[node] * 100 + 1000 for node in G_2021_shop4.nodes()]
normalized_sizes_2021_shop4 = (np.array(node_sizes_2021_shop4) - min(node_sizes_2021_shop4)) / (max(node_sizes_2021_shop4) - min(node_sizes_2021_shop4))
cmap_2021_shop4 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2021_shop4 = [cmap_2021_shop4(size) for size in normalized_sizes_2021_shop4]

# Drawing the network with conviction-based edge widths
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2021_shop4, pos_2021_shop4, node_size=node_sizes_2021_shop4, node_color=node_colors_2021_shop4, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop4, pos_2021_shop4, width=edge_widths_2021_shop4, edge_color='grey', alpha=0.5)
nx.draw_networkx_labels(G_2021_shop4, pos_2021_shop4, font_size=10)

plt.title('Top 5 Association Rules Network for 2021 Shop 4 with Conviction-based Edge Thickness')
plt.axis('off')
plt.show()


In [ ]:
#the node sizes are based on the indegree where each node acts as a consequent,

In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2021_shop4_nonredundant_top30.loc[:, 'antecedents_'] = rules_2021_shop4_nonredundant_top30['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2021_shop4_nonredundant_top30.loc[:, 'consequents_'] = rules_2021_shop4_nonredundant_top30['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2021_shop4_nonredundant_top30, x='consequents_', y='antecedents_', s=rules_2021_shop4_nonredundant_top30['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2021_shop4_nonredundant_top30.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2021 Shop 4", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
print(rules_2021_shop4_nonredundant_top30[['antecedents', 'consequents']].head(10))

## Monetary Analysis of the Top Rules

In [ ]:
# Step 1: Extract unique product codes from the rules
unique_product_codes = set()
for idx, row in rules_2021_shop4_nonredundant_top30.iterrows():
    # Unite antecedents and consequents into a single set
    all_products = row['antecedents'].union(row['consequents'])
    unique_product_codes.update(all_products)

# Step 2: Create a DataFrame with unique product codes and their unit prices
# Filter sales_data for these product codes and drop duplicates to ensure each product code appears once
product_prices = sales_data[sales_data['product_code'].isin(unique_product_codes)][['product_code', 'unit_price']].drop_duplicates()

# Create a dictionary for quick lookup of prices
price_dict = product_prices.set_index('product_code')['unit_price'].to_dict()

# Step 3: Calculate the total price for each rule and collect data for the new DataFrame
data_for_new_df = []
for idx, row in rules_2021_shop4_nonredundant_top30.iterrows():
    # Combine sets of antecedents and consequents
    all_products = row['antecedents'].union(row['consequents'])
    total_price = sum(price_dict.get(prod, 0) for prod in all_products)  # Use get to avoid KeyError if any product code is missing
    data_for_new_df.append({
        'antecedents': row['antecedents'],
        'consequents': row['consequents'],
        'total_unit_price': total_price
    })

# Create a new DataFrame with only antecedents, consequents, and total unit price
rules_and_price_shop4_2021 = pd.DataFrame(data_for_new_df)

# Sort the DataFrame by total_unit_price in descending order
rules_and_price_shop4_2021_sorted = rules_and_price_shop4_2021.sort_values(by='total_unit_price', ascending=False)

# Print the sorted DataFrame to see the specified columns along with their total unit prices
print(rules_and_price_shop4_2021_sorted)


In [ ]:
# Convert 'trx_date' to datetime format if not already done so
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Add a 'year' column if it doesn't exist
sales_data['year'] = sales_data['trx_date'].dt.year

# Step 2: Filter sales_data for shop4 and the year 2021
filtered_sales_data_2021 = sales_data[(sales_data['shopID'] == 4) & (sales_data['year'] == 2021)]

# Step 3: Initialize a list to hold the rule data including total sales value
rule_sales_summaries_2021 = []

# Step 4: Iterate through each rule in the DataFrame
for index, rule in rules_2021_shop4_nonredundant_top30.iterrows():
    all_products = rule['antecedents'].union(rule['consequents'])

    # Filter the sales data for transactions that include all products in the rule
    # This step ensures that we are only considering transactions that contain all specified products
    mask = filtered_sales_data_2021['product_code'].isin(all_products)
    transaction_ids = filtered_sales_data_2021[mask].groupby('trx_id').filter(lambda x: all_products.issubset(set(x['product_code']))).trx_id.unique()
    matched_transactions = filtered_sales_data_2021[filtered_sales_data_2021['trx_id'].isin(transaction_ids)]
    
    # Step 5: Sum the total_value of these transactions
    total_sales_value = matched_transactions['total_value'].sum()

    # Append the result to the list
    rule_sales_summaries_2021.append({
        'antecedents': rule['antecedents'],
        'consequents': rule['consequents'],
        'total_sales_value': total_sales_value
    })

# Step 6: Convert the list to a DataFrame
rules_sales_values_df_2021 = pd.DataFrame(rule_sales_summaries_2021)

# Step 7: Sort by total_sales_value in descending order
sorted_rules_sales_values_df_2021 = rules_sales_values_df_2021.sort_values(by='total_sales_value', ascending=False)

# Step 8: Print the sorted DataFrame
print(sorted_rules_sales_values_df_2021)


## Narrowing down based on Lift

In [ ]:
rules_2021_shop4_nonredundant_top30_lift['LHS'] = [','.join(list(x)) for x in rules_2021_shop4_nonredundant_top30_lift['antecedents']]
rules_2021_shop4_nonredundant_top30_lift['RHS'] = [','.join(list(x)) for x in rules_2021_shop4_nonredundant_top30_lift['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2021_shop4_nonredundant_top30_lift.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2021_shop4_nonredundant_top30_lift, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2021 Shop 4")
plt.axis('off')
plt.show()


In [ ]:
G_2021_shop4_lift = nx.DiGraph()

lift_summary_2021_shop4 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2021_shop4_nonredundant_top30_lift.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop4_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2021_shop4:
        lift_summary_2021_shop4[consequents]['total_lift'] += row['lift']
        lift_summary_2021_shop4[consequents]['count'] += 1
    else:
        lift_summary_2021_shop4[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2021_shop4:
    lift_summary_2021_shop4[item]['avg_lift'] = lift_summary_2021_shop4[item]['total_lift'] / lift_summary_2021_shop4[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2021_shop4_lift = [lift_summary_2021_shop4[node]['avg_lift'] * 500 if node in lift_summary_2021_shop4 else 1000 for node in G_2021_shop4_lift.nodes()]

pos_2021_shop4_lift = nx.spring_layout(G_2021_shop4_lift)

# Colormap from orange to red
cmap_2021_shop4_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2021_shop4_lift = np.array(node_sizes_2021_shop4_lift)
normalized_sizes_2021_shop4_lift = (normalized_sizes_2021_shop4_lift - min(normalized_sizes_2021_shop4_lift)) / (max(normalized_sizes_2021_shop4_lift) - min(normalized_sizes_2021_shop4_lift))
node_colors_2021_shop4_lift = [cmap_2021_shop4_lift(size) for size in normalized_sizes_2021_shop4_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 32))
nx.draw_networkx_nodes(G_2021_shop4_lift, pos_2021_shop4_lift, node_size=node_sizes_2021_shop4_lift, node_color=node_colors_2021_shop4_lift, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop4_lift, pos_2021_shop4_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2021_shop4_lift, pos_2021_shop4_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2021 Shop 4 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2021_shop4_nonredundant_top30_lift.loc[:, 'antecedents_'] = rules_2021_shop4_nonredundant_top30_lift['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2021_shop4_nonredundant_top30_lift.loc[:, 'consequents_'] = rules_2021_shop4_nonredundant_top30_lift['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2021_shop4_nonredundant_top30_lift, x='consequents_', y='antecedents_', s=rules_2021_shop4_nonredundant_top30_lift['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2021_shop4_nonredundant_top30_lift.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2021 Shop 4", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
# Print only the 'antecedents' and 'consequents' columns of the top 10 rules
print(rules_2021_shop4_nonredundant_top30_lift[['antecedents', 'consequents']].head(10))


# Shop4 2022 SEARCH 7

In [ ]:
# Filter for records from shop ID 1 and the year 1993
sales_data_2022_shop4 = sales_data_shop4[(sales_data_shop4['trx_date'].dt.year == 2022)]

In [ ]:
sales_data_2022_shop4.head()

In [ ]:
# Grouping product codes by trx_id
transactions_2022_shop4 = sales_data_2022_shop4.groupby('trx_id')['product_code'].apply(list).tolist()

In [ ]:
# Encode transaction data
encoder_2022_shop4 = TransactionEncoder()
trans_array_2022_shop4 = encoder_2022_shop4.fit(transactions_2022_shop4).transform(transactions_2022_shop4)
trans_df_2022_shop4 = pd.DataFrame(trans_array_2022_shop4, columns=encoder_2022_shop4.columns_)

In [ ]:
# Find frequent itemsets
frequent_itemsets_2022_shop4 = apriori(trans_df_2022_shop4, min_support=0.001, use_colnames=True)

In [ ]:
print(frequent_itemsets_2022_shop4)

In [ ]:
## Generate the association rules - by confidence
rulesConfidence_2022_shop4 = association_rules(frequent_itemsets_2022_shop4, metric="confidence", min_threshold=0.10)
rulesConfidence_2022_shop4.sort_values(by='confidence', ascending=False, inplace=True)


In [ ]:
# Note: This is a hypothetical example, as I can't directly manipulate or access dataframes not provided within this code cell.

# Replace frozen sets with strings
rulesConfidence_2022_shop4['antecedents_'] = rulesConfidence_2022_shop4['antecedents'].apply(lambda a: ','.join(list(a)))
rulesConfidence_2022_shop4['consequents_'] = rulesConfidence_2022_shop4['consequents'].apply(lambda a: ','.join(list(a)))

# Draw
fig, ax = plt.subplots(figsize=(14,30))
ax = plt.scatter(data=rulesConfidence_2022_shop4, x='consequents_', y='antecedents_', s=rulesConfidence_2022_shop4['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rulesConfidence_2022_shop4.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2020 Shop 1", fontsize=14)
plt.xlabel("Consequents (RHS)", fontsize=12)
plt.ylabel("Antecedents (LHS)", fontsize=12)
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate(rotation=45)
plt.tight_layout()

plt.show()



## Eliminate Redundant Rules

In [ ]:
rulesConfidence_2022_shop4.head()

In [ ]:
rulesConfidence_2022_shop4.info()

In [ ]:
# If 'antecedents' and 'consequents' are not frozensets, convert them
rules_2022_shop4_redundant = rulesConfidence_2022_shop4.copy()
rules_2022_shop4_redundant['antecedents'] = rules_2022_shop4_redundant['antecedents'].apply(lambda x: frozenset(x))
rules_2022_shop4_redundant['consequents'] = rules_2022_shop4_redundant['consequents'].apply(lambda x: frozenset(x))


In [ ]:
# Create a copy of the DataFrame to avoid modifying the original while iterating
rules_2022_shop4_checkredundance = rules_2022_shop4_redundant.copy()

# List to hold non-redundant rules
non_redundant_rules_2022_shop4 = []

for _, rule in rules_2022_shop4_checkredundance.iterrows():
    if not is_redundant(rule, rules_2022_shop4_checkredundance):
        non_redundant_rules_2022_shop4.append(rule)

# Convert the list back to a DataFrame
rules_2022_shop4_nonredundant = pd.DataFrame(non_redundant_rules_2022_shop4).reset_index(drop=True)


In [ ]:
print(rules_2022_shop4_nonredundant.head())
print(f"Original number of rules: {len(rules_2022_shop4_redundant)}")
print(f"Number of non-redundant rules: {len(rules_2022_shop4_nonredundant)}")


In [ ]:
rules_2022_shop4_nonredundant.head()

In [ ]:
rules_2022_shop4_redundant.info()

In [ ]:
rules_2022_shop4_nonredundant.head()

In [ ]:
rules_2022_shop4_nonredundant_backup = rules_2022_shop4_nonredundant.copy()

In [ ]:
rules_2022_shop4_nonredundant_unfiltered = rules_2022_shop4_nonredundant.copy()

In [ ]:
#reset
#rules_2022_shop4_nonredundant = rules_2022_shop4_nonredundant_backup.copy()

# Redundant and Unfiltered

In [ ]:
rules_2022_shop4_redundant.head()

In [ ]:
G_2022_shop4 = nx.DiGraph()

# Iterate through rules for 2022 Shop 4 and add edges to the graph
for _, row in rules_2022_shop1_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as weight
    G_2022_shop4.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2022_shop4 = nx.spring_layout(G_2022_shop4)

# Recalculate node sizes based on indegree or another metric
in_degrees_2022_shop4 = dict(G_2022_shop4.in_degree())
node_sizes_2022_shop4 = [in_degrees_2022_shop4[node] * 100 + 1000 for node in G_2022_shop4.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2022_shop4 = (np.array(node_sizes_2022_shop4) - np.min(node_sizes_2022_shop4)) / (np.max(node_sizes_2022_shop4) - np.min(node_sizes_2022_shop4))

# Define a colormap from orange to red for visual consistency
cmap_2022_shop4 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2022_shop4 = [cmap_2022_shop4(size) for size in normalized_sizes_2022_shop4]

# Drawing the network
plt.figure(figsize=(50, 33))
nx.draw_networkx_nodes(G_2022_shop4, pos_2022_shop4, node_size=node_sizes_2022_shop4, node_color=node_colors_2022_shop4, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop4, pos_2022_shop4, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop4, pos_2022_shop4, font_size=10)

plt.title('Top 5 Association Rules Network for 2022 Shop 4 with Color Gradient')
plt.axis('off')
plt.show()


### Based on LIFT

In [ ]:
G_2022_shop4_lift = nx.DiGraph()

lift_summary_2022_shop4 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2022_shop1_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop4_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop4:
        lift_summary_2022_shop4[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop4[consequents]['count'] += 1
    else:
        lift_summary_2022_shop4[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop4:
    lift_summary_2022_shop4[item]['avg_lift'] = lift_summary_2022_shop4[item]['total_lift'] / lift_summary_2022_shop4[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop4_lift = [lift_summary_2022_shop4[node]['avg_lift'] * 500 if node in lift_summary_2022_shop4 else 1000 for node in G_2022_shop4_lift.nodes()]

pos_2022_shop4_lift = nx.spring_layout(G_2022_shop4_lift)

# Colormap from orange to red
cmap_2022_shop4_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop4_lift = np.array(node_sizes_2022_shop4_lift)
normalized_sizes_2022_shop4_lift = (normalized_sizes_2022_shop4_lift - min(normalized_sizes_2022_shop4_lift)) / (max(normalized_sizes_2022_shop4_lift) - min(normalized_sizes_2022_shop4_lift))
node_colors_2022_shop4_lift = [cmap_2022_shop4_lift(size) for size in normalized_sizes_2022_shop4_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(70, 52))
nx.draw_networkx_nodes(G_2022_shop4_lift, pos_2022_shop4_lift, node_size=node_sizes_2022_shop4_lift, node_color=node_colors_2022_shop4_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop4_lift, pos_2022_shop4_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop4_lift, pos_2022_shop4_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2022 Shop 4 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


# nonredundant_unfiltered

## Based on Indegree

In [ ]:
G_2022_shop4_nonredundant_unfiltered = nx.DiGraph()

# Iterating through each row to add edges with confidence as an attribute
for _, row in rules_2022_shop1_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop4_nonredundant_unfiltered.add_edge(antecedents, consequents, weight=row['confidence'])

# Defining the layout for network visualization
pos_2022_shop4_nonredundant_unfiltered = nx.spring_layout(G_2022_shop4_nonredundant_unfiltered)

# Calculating node sizes based on the indegree for color mapping
in_degrees_2022_shop4_nonredundant_unfiltered = dict(G_2022_shop4_nonredundant_unfiltered.in_degree())
node_sizes_2022_shop4_nonredundant_unfiltered = [in_degrees_2022_shop4_nonredundant_unfiltered[node] * 100 + 1000 for node in G_2022_shop4_nonredundant_unfiltered.nodes()]

# Normalizing node sizes for color mapping
normalized_sizes_2022_shop4_nonredundant_unfiltered = (np.array(node_sizes_2022_shop4_nonredundant_unfiltered) - min(node_sizes_2022_shop4_nonredundant_unfiltered)) / (max(node_sizes_2022_shop4_nonredundant_unfiltered) - min(node_sizes_2022_shop4_nonredundant_unfiltered))

# Defining a color map from orange to red
cmap_2022_shop4_nonredundant_unfiltered = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])

# Mapping normalized node sizes to colors
node_colors_2022_shop4_nonredundant_unfiltered = [cmap_2022_shop4_nonredundant_unfiltered(size) for size in normalized_sizes_2022_shop4_nonredundant_unfiltered]

# Drawing the network with node sizes and colors
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2022_shop4_nonredundant_unfiltered, pos_2022_shop4_nonredundant_unfiltered, node_size=node_sizes_2022_shop4_nonredundant_unfiltered, node_color=node_colors_2022_shop4_nonredundant_unfiltered, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop4_nonredundant_unfiltered, pos_2022_shop4_nonredundant_unfiltered, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2022_shop4_nonredundant_unfiltered, pos_2022_shop4_nonredundant_unfiltered, font_size=10)

plt.title('Node Size Based on Indegree for 2022 Shop 4')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2022_shop4_lift = nx.DiGraph()

lift_summary_2022_shop4 = {}

for _, row in rules_2022_shop1_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop4_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Update lift summary for consequents
    if consequents in lift_summary_2022_shop4:
        lift_summary_2022_shop4[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop4[consequents]['count'] += 1
    else:
        lift_summary_2022_shop4[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculate average lift for each consequent
for item in lift_summary_2022_shop4:
    lift_summary_2022_shop4[item]['avg_lift'] = lift_summary_2022_shop4[item]['total_lift'] / lift_summary_2022_shop4[item]['count']

# Node sizes based on average lift, scaled appropriately
node_sizes_2022_shop4_lift = [lift_summary_2022_shop4[node]['avg_lift'] * 500 if node in lift_summary_2022_shop4 else 1000 for node in G_2022_shop4_lift.nodes()]

pos_2022_shop4_lift = nx.spring_layout(G_2022_shop4_lift)

# Use a colormap that transitions from orange to red
cmap_2022_shop4_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop4_lift = np.array(node_sizes_2022_shop4_lift)
normalized_sizes_2022_shop4_lift = (normalized_sizes_2022_shop4_lift - min(normalized_sizes_2022_shop4_lift)) / (max(normalized_sizes_2022_shop4_lift) - min(normalized_sizes_2022_shop4_lift))
node_colors_2022_shop4_lift = [cmap_2022_shop4_lift(size) for size in normalized_sizes_2022_shop4_lift]

plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2022_shop4_lift, pos_2022_shop4_lift, node_size=node_sizes_2022_shop4_lift, node_color=node_colors_2022_shop4_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop4_lift, pos_2022_shop4_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop4_lift, pos_2022_shop4_lift, font_size=10)

plt.title('Node Size Based on Average Lift for 2022 Shop 4')
plt.axis('off')
plt.show()


# Based on conviction

In [ ]:
rules_2022_shop4_nonredundant.head()

In [ ]:
G_2022_shop4_conviction = nx.DiGraph()

conviction_summary_2022_shop4 = {}
count_summary_2022_shop4 = {}

# Assuming 'rules_2022_shop4_nonredundant_unfiltered' contains a 'conviction' column
for _, row in rules_2022_shop4_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop4_conviction.add_edge(antecedents, consequents, weight=row['confidence'], conviction=row['conviction'])
    
    # Aggregate conviction values and count occurrences
    if consequents in conviction_summary_2022_shop4:
        conviction_summary_2022_shop4[consequents] += row['conviction']
        count_summary_2022_shop4[consequents] += 1
    else:
        conviction_summary_2022_shop4[consequents] = row['conviction']
        count_summary_2022_shop4[consequents] = 1

# Calculate average conviction values
average_conviction_summary_2022_shop4 = {k: conviction_summary_2022_shop4[k] / count_summary_2022_shop4[k] for k in conviction_summary_2022_shop4}

# Normalize conviction values for sizing and coloring
max_conviction = max(average_conviction_summary_2022_shop4.values())
min_conviction = min(average_conviction_summary_2022_shop4.values())
conviction_range = max_conviction - min_conviction
node_colors = []
node_sizes = []

for node in G_2022_shop4_conviction.nodes():
    # Get the average conviction value for the node, defaulting to min if not present
    conviction = average_conviction_summary_2022_shop4.get(node, min_conviction)
    # Normalize between 0 and 1
    normalized_conviction = (conviction - min_conviction) / conviction_range if conviction_range else 0
    node_colors.append(normalized_conviction)
    node_sizes.append(normalized_conviction * 1000 + 100)  # Scale and offset size

# Use a colormap for coloring nodes based on conviction
cmap = plt.cm.coolwarm

pos = nx.spring_layout(G_2022_shop4_conviction)

plt.figure(figsize=(40, 22))
nodes = nx.draw_networkx_nodes(G_2022_shop4_conviction, pos, node_color=node_colors, node_size=node_sizes, cmap=cmap, alpha=0.8)
edges = nx.draw_networkx_edges(G_2022_shop4_conviction, pos, arrowstyle='->', arrowsize=10, edge_color='grey', alpha=0.5)
labels = nx.draw_networkx_labels(G_2022_shop4_conviction, pos, font_size=8)

plt.colorbar(nodes, label='Normalized Conviction')
plt.title('Network Graph of 2022 Shop 4 Based on Conviction')
plt.axis('off')
plt.show()

In [ ]:
print(max_conviction)

# Filtered top 30 and non redundant

In [ ]:
rules_2022_shop4_nonredundant.head()

In [ ]:
# Assuming rules_1992_shop2_nonredundant is your sorted rules DataFrame for shop2
rules_2022_shop4_nonredundant_top30 = rules_2022_shop4_nonredundant.copy(deep=True)

In [ ]:
rules_2022_shop4_nonredundant_top30_lift = rules_2022_shop4_nonredundant.copy(deep=True)

In [ ]:
# Select the top 30 rules based on lift or another criterion
rules_2022_shop4_nonredundant_top30_lift = rules_2022_shop4_nonredundant_top30_lift.sort_values(by='lift', ascending=False).head(30)

In [ ]:
rules_2022_shop4_nonredundant_top30_lift.head()

In [ ]:
# Select the top 30 rules based on confidence or another criterion
rules_2022_shop4_nonredundant_top30 = rules_2022_shop4_nonredundant_top30.sort_values(by='confidence', ascending=False).head(30)

In [ ]:
rules_2022_shop4_nonredundant_top30.info()

In [ ]:
rules_2022_shop4_nonredundant_top30['LHS'] = [','.join(list(x)) for x in rules_2022_shop4_nonredundant_top30['antecedents']]
rules_2022_shop4_nonredundant_top30['RHS'] = [','.join(list(x)) for x in rules_2022_shop4_nonredundant_top30['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2022_shop4_nonredundant_top30.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2022_shop4_nonredundant_top30, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2022 Shop 4")
plt.axis('off')
plt.show()


## Based on Indegree

In [ ]:
G_2022_shop4 = nx.DiGraph()

# Assuming 'rules_sorted_2022_shop4' contains association rules for shop 4 in 2022
for _, row in rules_2022_shop4_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as the weight
    G_2022_shop4.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2022_shop4 = nx.spring_layout(G_2022_shop4)

# Calculate node sizes based on indegree
in_degrees_2022_shop4 = dict(G_2022_shop4.in_degree())
node_sizes_2022_shop4 = [in_degrees_2022_shop4[node] * 100 + 1000 for node in G_2022_shop4.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2022_shop4 = (np.array(node_sizes_2022_shop4) - np.min(node_sizes_2022_shop4)) / (np.max(node_sizes_2022_shop4) - np.min(node_sizes_2022_shop4))

# Define a colormap from orange to red
cmap_2022_shop4 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2022_shop4 = [cmap_2022_shop4(size) for size in normalized_sizes_2022_shop4]

# Drawing the network
plt.figure(figsize=(30, 34))
nx.draw_networkx_nodes(G_2022_shop4, pos_2022_shop4, node_size=node_sizes_2022_shop4, node_color=node_colors_2022_shop4, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop4, pos_2022_shop4, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2022_shop4, pos_2022_shop4, font_size=8)

plt.title('Top 5 Association Rules Network for 2022 Shop 4 with Color Gradient')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2022_shop4_lift = nx.DiGraph()

lift_summary_2022_shop4 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2022_shop4_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop4_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop4:
        lift_summary_2022_shop4[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop4[consequents]['count'] += 1
    else:
        lift_summary_2022_shop4[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop4:
    lift_summary_2022_shop4[item]['avg_lift'] = lift_summary_2022_shop4[item]['total_lift'] / lift_summary_2022_shop4[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop4_lift = [lift_summary_2022_shop4[node]['avg_lift'] * 500 if node in lift_summary_2022_shop4 else 1000 for node in G_2022_shop4_lift.nodes()]

pos_2022_shop4_lift = nx.spring_layout(G_2022_shop4_lift)

# Colormap from orange to red
cmap_2022_shop4_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop4_lift = np.array(node_sizes_2022_shop4_lift)
normalized_sizes_2022_shop4_lift = (normalized_sizes_2022_shop4_lift - min(normalized_sizes_2022_shop4_lift)) / (max(normalized_sizes_2022_shop4_lift) - min(normalized_sizes_2022_shop4_lift))
node_colors_2022_shop4_lift = [cmap_2022_shop4_lift(size) for size in normalized_sizes_2022_shop4_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 32))
nx.draw_networkx_nodes(G_2022_shop4_lift, pos_2022_shop4_lift, node_size=node_sizes_2022_shop4_lift, node_color=node_colors_2022_shop4_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop4_lift, pos_2022_shop4_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop4_lift, pos_2022_shop4_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2022 Shop 4 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop4_lift = nx.DiGraph()

lift_summary_2022_shop4 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2022_shop4_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop4_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop4:
        lift_summary_2022_shop4[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop4[consequents]['count'] += 1
    else:
        lift_summary_2022_shop4[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop4:
    lift_summary_2022_shop4[item]['avg_lift'] = lift_summary_2022_shop4[item]['total_lift'] / lift_summary_2022_shop4[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop4_lift = [lift_summary_2022_shop4[node]['avg_lift'] * 500 if node in lift_summary_2022_shop4 else 1000 for node in G_2022_shop4_lift.nodes()]

pos_2022_shop4_lift = nx.spring_layout(G_2022_shop4_lift)

# Colormap from orange to red
cmap_2022_shop4_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop4_lift = np.array(node_sizes_2022_shop4_lift)
normalized_sizes_2022_shop4_lift = (normalized_sizes_2022_shop4_lift - min(normalized_sizes_2022_shop4_lift)) / (max(normalized_sizes_2022_shop4_lift) - min(normalized_sizes_2022_shop4_lift))
node_colors_2022_shop4_lift = [cmap_2022_shop4_lift(size) for size in normalized_sizes_2022_shop4_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2022_shop4_lift, pos_2022_shop4_lift, node_size=node_sizes_2022_shop4_lift, node_color=node_colors_2022_shop4_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop4_lift, pos_2022_shop4_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop4_lift, pos_2022_shop4_lift, font_size=10)

plt.title('Top 5 Association Rules Network for 2022 Shop 4 Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop4 = nx.DiGraph()

# Assuming 'rules_2022_shop4_nonredundant' contains association rules for 2022 Shop 4
for _, row in rules_2022_shop4_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop4.add_edge(antecedents, consequents, confidence=row['confidence'])

# Calculate node sizes based on indegree
in_degrees_2022_shop4 = dict(G_2022_shop4.in_degree())
node_sizes_2022_shop4 = [in_degrees_2022_shop4[node] * 100 + 1000 for node in G_2022_shop4.nodes()]

# Edge width based on confidence
edge_widths_2022_shop4 = [G_2022_shop4[u][v]['confidence'] * 45 for u, v in G_2022_shop4.edges()]

# Assuming normalized sizes and color mapping is desired
normalized_sizes_2022_shop4 = np.array(list(in_degrees_2022_shop4.values()))
normalized_sizes_2022_shop4 = (normalized_sizes_2022_shop4 - min(normalized_sizes_2022_shop4)) / (max(normalized_sizes_2022_shop4) - min(normalized_sizes_2022_shop4))
cmap_2022_shop4 = mcolors.LinearSegmentedColormap.from_list("MyCMapName", ["blue", "red"])

# Node colors based on a property (here, using indegree for example)
node_colors_2022_shop4 = [cmap_2022_shop4(size) for size in normalized_sizes_2022_shop4]

# Layout
pos_2022_shop4 = nx.spring_layout(G_2022_shop4, seed=42)

# Drawing
plt.figure(figsize=(40, 30))
nx.draw_networkx_edges(G_2022_shop4, pos_2022_shop4, width=edge_widths_2022_shop4, edge_color='grey', alpha=0.5)
nx.draw_networkx_nodes(G_2022_shop4, pos_2022_shop4, node_size=node_sizes_2022_shop4, node_color=node_colors_2022_shop4, alpha=0.8)
nx.draw_networkx_labels(G_2022_shop4, pos_2022_shop4, font_size=8, font_color='black')
plt.title('Network Graph of Association Rules Based on Confidence for 2022 Shop 4')
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop4 = nx.DiGraph()

convictions_2022_shop4 = []  # To store conviction values for normalization

for _, row in rules_2022_shop4_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    conviction_2022_shop4 = row['conviction']  # Assuming each row has a 'conviction' column
    convictions_2022_shop4.append(conviction_2022_shop4)
    G_2022_shop4.add_edge(antecedents, consequents, weight=row['confidence'], conviction=conviction_2022_shop4)

# Normalize conviction values for edge width
max_conviction_2022_shop4 = max(convictions_2022_shop4)
min_conviction_2022_shop4 = min(convictions_2022_shop4)
conviction_range_2022_shop4 = max_conviction_2022_shop4 - min_conviction_2022_shop4
edge_widths_2022_shop4 = [(G_2022_shop4[u][v]['conviction'] - min_conviction_2022_shop4) / conviction_range_2022_shop4 * 30 + 0.5 for u, v in G_2022_shop4.edges()]  # Scale and offset widths

pos_2022_shop4 = nx.spring_layout(G_2022_shop4)

# Node sizes and colors as before
in_degrees_2022_shop4 = dict(G_2022_shop4.in_degree())
node_sizes_2022_shop4 = [in_degrees_2022_shop4[node] * 100 + 1000 for node in G_2022_shop4.nodes()]
normalized_sizes_2022_shop4 = (np.array(node_sizes_2022_shop4) - min(node_sizes_2022_shop4)) / (max(node_sizes_2022_shop4) - min(node_sizes_2022_shop4))
cmap_2022_shop4 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2022_shop4 = [cmap_2022_shop4(size) for size in normalized_sizes_2022_shop4]

# Drawing the network with conviction-based edge widths
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2022_shop4, pos_2022_shop4, node_size=node_sizes_2022_shop4, node_color=node_colors_2022_shop4, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop4, pos_2022_shop4, width=edge_widths_2022_shop4, edge_color='grey', alpha=0.5)
nx.draw_networkx_labels(G_2022_shop4, pos_2022_shop4, font_size=10)

plt.title('Top 5 Association Rules Network for 2022 Shop 4 with Conviction-based Edge Thickness')
plt.axis('off')
plt.show()


In [ ]:
#the node sizes are based on the indegree where each node acts as a consequent,

In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2022_shop4_nonredundant_top30.loc[:, 'antecedents_'] = rules_2022_shop4_nonredundant_top30['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2022_shop4_nonredundant_top30.loc[:, 'consequents_'] = rules_2022_shop4_nonredundant_top30['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2022_shop4_nonredundant_top30, x='consequents_', y='antecedents_', s=rules_2022_shop4_nonredundant_top30['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2022_shop4_nonredundant_top30.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2022 Shop 4", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
print(rules_2022_shop4_nonredundant_top30[['antecedents', 'consequents']].head(10))

## Monetary Analysis of the Top Rules

In [ ]:
# Step 1: Extract unique product codes from the rules
unique_product_codes = set()
for idx, row in rules_2022_shop4_nonredundant_top30.iterrows():
    # Unite antecedents and consequents into a single set
    all_products = row['antecedents'].union(row['consequents'])
    unique_product_codes.update(all_products)

# Step 2: Create a DataFrame with unique product codes and their unit prices
# Filter sales_data for these product codes and drop duplicates to ensure each product code appears once
product_prices = sales_data[sales_data['product_code'].isin(unique_product_codes)][['product_code', 'unit_price']].drop_duplicates()

# Create a dictionary for quick lookup of prices
price_dict = product_prices.set_index('product_code')['unit_price'].to_dict()

# Step 3: Calculate the total price for each rule and collect data for the new DataFrame
data_for_new_df = []
for idx, row in rules_2022_shop4_nonredundant_top30.iterrows():
    # Combine sets of antecedents and consequents
    all_products = row['antecedents'].union(row['consequents'])
    total_price = sum(price_dict.get(prod, 0) for prod in all_products)  # Use get to avoid KeyError if any product code is missing
    data_for_new_df.append({
        'antecedents': row['antecedents'],
        'consequents': row['consequents'],
        'total_unit_price': total_price
    })

# Create a new DataFrame with only antecedents, consequents, and total unit price
rules_and_price_shop4_2022 = pd.DataFrame(data_for_new_df)

# Sort the DataFrame by total_unit_price in descending order
rules_and_price_shop4_2022_sorted = rules_and_price_shop4_2022.sort_values(by='total_unit_price', ascending=False)

# Print the sorted DataFrame to see the specified columns along with their total unit prices
print(rules_and_price_shop4_2022_sorted)


In [ ]:
# Convert 'trx_date' to datetime format if not already done so
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Add a 'year' column if it doesn't exist
sales_data['year'] = sales_data['trx_date'].dt.year

# Step 2: Filter sales_data for shop4 and the year 2022
filtered_sales_data_2022 = sales_data[(sales_data['shopID'] == 4) & (sales_data['year'] == 2022)]

# Step 3: Initialize a list to hold the rule data including total sales value
rule_sales_summaries_2022 = []

# Step 4: Iterate through each rule in the DataFrame
for index, rule in rules_2022_shop4_nonredundant_top30.iterrows():
    all_products = rule['antecedents'].union(rule['consequents'])

    # Filter the sales data for transactions that include all products in the rule
    # This step ensures that we are only considering transactions that contain all specified products
    mask = filtered_sales_data_2022['product_code'].isin(all_products)
    transaction_ids = filtered_sales_data_2022[mask].groupby('trx_id').filter(lambda x: all_products.issubset(set(x['product_code']))).trx_id.unique()
    matched_transactions = filtered_sales_data_2022[filtered_sales_data_2022['trx_id'].isin(transaction_ids)]
    
    # Step 5: Sum the total_value of these transactions
    total_sales_value = matched_transactions['total_value'].sum()

    # Append the result to the list
    rule_sales_summaries_2022.append({
        'antecedents': rule['antecedents'],
        'consequents': rule['consequents'],
        'total_sales_value': total_sales_value
    })

# Step 6: Convert the list to a DataFrame
rules_sales_values_df_2022 = pd.DataFrame(rule_sales_summaries_2022)

# Step 7: Sort by total_sales_value in descending order
sorted_rules_sales_values_df_2022 = rules_sales_values_df_2022.sort_values(by='total_sales_value', ascending=False)

# Step 8: Print the sorted DataFrame
print(sorted_rules_sales_values_df_2022)


## Narrowing down based on Lift

In [ ]:
rules_2022_shop4_nonredundant_top30_lift['LHS'] = [','.join(list(x)) for x in rules_2022_shop4_nonredundant_top30_lift['antecedents']]
rules_2022_shop4_nonredundant_top30_lift['RHS'] = [','.join(list(x)) for x in rules_2022_shop4_nonredundant_top30_lift['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2022_shop4_nonredundant_top30_lift.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2022_shop4_nonredundant_top30_lift, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2022 Shop 4")
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop4_lift = nx.DiGraph()

lift_summary_2022_shop4 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2022_shop4_nonredundant_top30_lift.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop4_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop4:
        lift_summary_2022_shop4[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop4[consequents]['count'] += 1
    else:
        lift_summary_2022_shop4[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop4:
    lift_summary_2022_shop4[item]['avg_lift'] = lift_summary_2022_shop4[item]['total_lift'] / lift_summary_2022_shop4[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop4_lift = [lift_summary_2022_shop4[node]['avg_lift'] * 500 if node in lift_summary_2022_shop4 else 1000 for node in G_2022_shop4_lift.nodes()]

pos_2022_shop4_lift = nx.spring_layout(G_2022_shop4_lift)

# Colormap from orange to red
cmap_2022_shop4_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop4_lift = np.array(node_sizes_2022_shop4_lift)
normalized_sizes_2022_shop4_lift = (normalized_sizes_2022_shop4_lift - min(normalized_sizes_2022_shop4_lift)) / (max(normalized_sizes_2022_shop4_lift) - min(normalized_sizes_2022_shop4_lift))
node_colors_2022_shop4_lift = [cmap_2022_shop4_lift(size) for size in normalized_sizes_2022_shop4_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(60, 52))
nx.draw_networkx_nodes(G_2022_shop4_lift, pos_2022_shop4_lift, node_size=node_sizes_2022_shop4_lift, node_color=node_colors_2022_shop4_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop4_lift, pos_2022_shop4_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop4_lift, pos_2022_shop4_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2022 Shop 4 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2022_shop4_nonredundant_top30_lift.loc[:, 'antecedents_'] = rules_2022_shop4_nonredundant_top30_lift['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2022_shop4_nonredundant_top30_lift.loc[:, 'consequents_'] = rules_2022_shop4_nonredundant_top30_lift['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2022_shop4_nonredundant_top30_lift, x='consequents_', y='antecedents_', s=rules_2022_shop4_nonredundant_top30_lift['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2022_shop4_nonredundant_top30_lift.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2022 Shop 4", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
# Print only the 'antecedents' and 'consequents' columns of the top 10 rules
print(rules_2022_shop4_nonredundant_top30_lift[['antecedents', 'consequents']].head(10))


# Shop 4 2023 SEARCH 8

In [ ]:
# Filter for records from shop ID 1 and the year 1993
sales_data_2023_shop4 = sales_data_shop4[(sales_data_shop4['trx_date'].dt.year == 2023)]

In [ ]:
sales_data_2023_shop4.head()

In [ ]:
# Grouping product codes by trx_id
transactions_2023_shop4 = sales_data_2023_shop4.groupby('trx_id')['product_code'].apply(list).tolist()

In [ ]:
# Encode transaction data
encoder_2023_shop4 = TransactionEncoder()
trans_array_2023_shop4 = encoder_2023_shop4.fit(transactions_2023_shop4).transform(transactions_2023_shop4)
trans_df_2023_shop4 = pd.DataFrame(trans_array_2023_shop4, columns=encoder_2023_shop4.columns_)

In [ ]:
# Find frequent itemsets
frequent_itemsets_2023_shop4 = apriori(trans_df_2023_shop4, min_support=0.001, use_colnames=True)

In [ ]:
print(frequent_itemsets_2023_shop4)

In [ ]:
## Generate the association rules - by confidence
rulesConfidence_2023_shop4 = association_rules(frequent_itemsets_2023_shop4, metric="confidence", min_threshold=0.10)
rulesConfidence_2023_shop4.sort_values(by='confidence', ascending=False, inplace=True)


In [ ]:
# Note: This is a hypothetical example, as I can't directly manipulate or access dataframes not provided within this code cell.

# Replace frozen sets with strings
rulesConfidence_2023_shop4['antecedents_'] = rulesConfidence_2023_shop4['antecedents'].apply(lambda a: ','.join(list(a)))
rulesConfidence_2023_shop4['consequents_'] = rulesConfidence_2023_shop4['consequents'].apply(lambda a: ','.join(list(a)))

# Draw
fig, ax = plt.subplots(figsize=(14,30))
ax = plt.scatter(data=rulesConfidence_2023_shop4, x='consequents_', y='antecedents_', s=rulesConfidence_2023_shop4['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rulesConfidence_2023_shop4.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2023 Shop 4", fontsize=14)
plt.xlabel("Consequents (RHS)", fontsize=12)
plt.ylabel("Antecedents (LHS)", fontsize=12)
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate(rotation=45)
plt.tight_layout()

plt.show()



## Eliminate Redundant Rules

In [ ]:
rulesConfidence_2023_shop4.head()

In [ ]:
rulesConfidence_2023_shop4.info()

In [ ]:
# If 'antecedents' and 'consequents' are not frozensets, convert them
rules_2023_shop4_redundant = rulesConfidence_2023_shop4.copy()
rules_2023_shop4_redundant['antecedents'] = rules_2023_shop4_redundant['antecedents'].apply(lambda x: frozenset(x))
rules_2023_shop4_redundant['consequents'] = rules_2023_shop4_redundant['consequents'].apply(lambda x: frozenset(x))


In [ ]:
# Create a copy of the DataFrame to avoid modifying the original while iterating
rules_2023_shop4_checkredundance = rules_2023_shop4_redundant.copy()

# List to hold non-redundant rules
non_redundant_rules_2023_shop4 = []

for _, rule in rules_2023_shop4_checkredundance.iterrows():
    if not is_redundant(rule, rules_2023_shop4_checkredundance):
        non_redundant_rules_2023_shop4.append(rule)

# Convert the list back to a DataFrame
rules_2023_shop4_nonredundant = pd.DataFrame(non_redundant_rules_2023_shop4).reset_index(drop=True)


In [ ]:
print(rules_2023_shop4_nonredundant.head())
print(f"Original number of rules: {len(rules_2023_shop4_redundant)}")
print(f"Number of non-redundant rules: {len(rules_2023_shop4_nonredundant)}")


In [ ]:
rules_2023_shop4_nonredundant.head()

rules_2023_shop1_redundant.info()

In [ ]:
rules_2023_shop4_nonredundant.head()

In [ ]:
rules_2023_shop4_nonredundant_backup = rules_2023_shop4_nonredundant.copy()

In [ ]:
rules_2023_shop4_nonredundant_unfiltered = rules_2023_shop4_nonredundant.copy()

In [ ]:
#reset
#rules_2023_shop4_nonredundant = rules_2023_shop4_nonredundant_backup.copy()

# Redundant and Unfiltered

In [ ]:
rules_2023_shop4_redundant.head()

In [ ]:
G_2023_shop4 = nx.DiGraph()

# Iterate through rules for 2023 Shop 4 and add edges to the graph
for _, row in rules_2023_shop4_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as weight
    G_2023_shop4.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2023_shop4 = nx.spring_layout(G_2023_shop4)

# Recalculate node sizes based on indegree or another metric
in_degrees_2023_shop4 = dict(G_2023_shop4.in_degree())
node_sizes_2023_shop4 = [in_degrees_2023_shop4[node] * 100 + 1000 for node in G_2023_shop4.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2023_shop4 = (np.array(node_sizes_2023_shop4) - np.min(node_sizes_2023_shop4)) / (np.max(node_sizes_2023_shop4) - np.min(node_sizes_2023_shop4))

# Define a colormap from orange to red for visual consistency
cmap_2023_shop4 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop4 = [cmap_2023_shop4(size) for size in normalized_sizes_2023_shop4]

# Drawing the network
plt.figure(figsize=(60, 43))
nx.draw_networkx_nodes(G_2023_shop4, pos_2023_shop4, node_size=node_sizes_2023_shop4, node_color=node_colors_2023_shop4, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop4, pos_2023_shop4, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop4, pos_2023_shop4, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 4 with Color Gradient')
plt.axis('off')
plt.show()


### Based on LIFT

In [ ]:
G_2023_shop4_lift = nx.DiGraph()

lift_summary_2023_shop4 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop4_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop4_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop4:
        lift_summary_2023_shop4[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop4[consequents]['count'] += 1
    else:
        lift_summary_2023_shop4[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop4:
    lift_summary_2023_shop4[item]['avg_lift'] = lift_summary_2023_shop4[item]['total_lift'] / lift_summary_2023_shop4[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop4_lift = [lift_summary_2023_shop4[node]['avg_lift'] * 500 if node in lift_summary_2023_shop4 else 1000 for node in G_2023_shop4_lift.nodes()]

pos_2023_shop4_lift = nx.spring_layout(G_2023_shop4_lift)

# Colormap from orange to red
cmap_2023_shop4_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop4_lift = np.array(node_sizes_2023_shop4_lift)
normalized_sizes_2023_shop4_lift = (normalized_sizes_2023_shop4_lift - min(normalized_sizes_2023_shop4_lift)) / (max(normalized_sizes_2023_shop4_lift) - min(normalized_sizes_2023_shop4_lift))
node_colors_2023_shop4_lift = [cmap_2023_shop4_lift(size) for size in normalized_sizes_2023_shop4_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(60, 43))
nx.draw_networkx_nodes(G_2023_shop4_lift, pos_2023_shop4_lift, node_size=node_sizes_2023_shop4_lift, node_color=node_colors_2023_shop4_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop4_lift, pos_2023_shop4_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop4_lift, pos_2023_shop4_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 4 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


# nonredundant_unfiltered

## Based on Indegree

In [ ]:
G_2023_shop4_nonredundant_unfiltered = nx.DiGraph()

# Iterating through each row to add edges with confidence as an attribute
for _, row in rules_2023_shop4_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop4_nonredundant_unfiltered.add_edge(antecedents, consequents, weight=row['confidence'])

# Defining the layout for network visualization
pos_2023_shop4_nonredundant_unfiltered = nx.spring_layout(G_2023_shop4_nonredundant_unfiltered)

# Calculating node sizes based on the indegree for color mapping
in_degrees_2023_shop4_nonredundant_unfiltered = dict(G_2023_shop4_nonredundant_unfiltered.in_degree())
node_sizes_2023_shop4_nonredundant_unfiltered = [in_degrees_2023_shop4_nonredundant_unfiltered[node] * 100 + 1000 for node in G_2023_shop4_nonredundant_unfiltered.nodes()]

# Normalizing node sizes for color mapping
normalized_sizes_2023_shop4_nonredundant_unfiltered = (np.array(node_sizes_2023_shop4_nonredundant_unfiltered) - min(node_sizes_2023_shop4_nonredundant_unfiltered)) / (max(node_sizes_2023_shop4_nonredundant_unfiltered) - min(node_sizes_2023_shop4_nonredundant_unfiltered))

# Defining a color map from orange to red
cmap_2023_shop4_nonredundant_unfiltered = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])

# Mapping normalized node sizes to colors
node_colors_2023_shop4_nonredundant_unfiltered = [cmap_2023_shop4_nonredundant_unfiltered(size) for size in normalized_sizes_2023_shop4_nonredundant_unfiltered]

# Drawing the network with node sizes and colors
plt.figure(figsize=(60, 43))
nx.draw_networkx_nodes(G_2023_shop4_nonredundant_unfiltered, pos_2023_shop4_nonredundant_unfiltered, node_size=node_sizes_2023_shop4_nonredundant_unfiltered, node_color=node_colors_2023_shop4_nonredundant_unfiltered, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop4_nonredundant_unfiltered, pos_2023_shop4_nonredundant_unfiltered, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2023_shop4_nonredundant_unfiltered, pos_2023_shop4_nonredundant_unfiltered, font_size=10)

plt.title('Node Size Based on Indegree for 2023 Shop 4')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2023_shop4_lift = nx.DiGraph()

lift_summary_2023_shop4 = {}

for _, row in rules_2023_shop4_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop4_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Update lift summary for consequents
    if consequents in lift_summary_2023_shop4:
        lift_summary_2023_shop4[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop4[consequents]['count'] += 1
    else:
        lift_summary_2023_shop4[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculate average lift for each consequent
for item in lift_summary_2023_shop4:
    lift_summary_2023_shop4[item]['avg_lift'] = lift_summary_2023_shop4[item]['total_lift'] / lift_summary_2023_shop4[item]['count']

# Node sizes based on average lift, scaled appropriately
node_sizes_2023_shop4_lift = [lift_summary_2023_shop4[node]['avg_lift'] * 500 if node in lift_summary_2023_shop4 else 1000 for node in G_2023_shop4_lift.nodes()]

pos_2023_shop4_lift = nx.spring_layout(G_2023_shop4_lift)

# Use a colormap that transitions from orange to red
cmap_2023_shop4_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop4_lift = np.array(node_sizes_2023_shop4_lift)
normalized_sizes_2023_shop4_lift = (normalized_sizes_2023_shop4_lift - min(normalized_sizes_2023_shop4_lift)) / (max(normalized_sizes_2023_shop4_lift) - min(normalized_sizes_2023_shop4_lift))
node_colors_2023_shop4_lift = [cmap_2023_shop4_lift(size) for size in normalized_sizes_2023_shop4_lift]

plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2023_shop4_lift, pos_2023_shop4_lift, node_size=node_sizes_2023_shop4_lift, node_color=node_colors_2023_shop4_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop4_lift, pos_2023_shop4_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop4_lift, pos_2023_shop4_lift, font_size=10)

plt.title('Node Size Based on Average Lift for 2023 Shop 4')
plt.axis('off')
plt.show()


# Based on conviction

In [ ]:
rules_2023_shop4_nonredundant.head()

In [ ]:
G_2023_shop4_conviction = nx.DiGraph()

conviction_summary_2023_shop4 = {}
count_summary_2023_shop4 = {}

# Assuming 'rules_2023_shop4_nonredundant_unfiltered' contains a 'conviction' column
for _, row in rules_2023_shop4_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop4_conviction.add_edge(antecedents, consequents, weight=row['confidence'], conviction=row['conviction'])
    
    # Aggregate conviction values and count occurrences
    if consequents in conviction_summary_2023_shop4:
        conviction_summary_2023_shop4[consequents] += row['conviction']
        count_summary_2023_shop4[consequents] += 1
    else:
        conviction_summary_2023_shop4[consequents] = row['conviction']
        count_summary_2023_shop4[consequents] = 1

# Calculate average conviction values
average_conviction_summary_2023_shop4 = {k: conviction_summary_2023_shop4[k] / count_summary_2023_shop4[k] for k in conviction_summary_2023_shop4}

# Normalize conviction values for sizing and coloring
max_conviction = max(average_conviction_summary_2023_shop4.values())
min_conviction = min(average_conviction_summary_2023_shop4.values())
conviction_range = max_conviction - min_conviction
node_colors = []
node_sizes = []

for node in G_2023_shop4_conviction.nodes():
    # Get the average conviction value for the node, defaulting to min if not present
    conviction = average_conviction_summary_2023_shop4.get(node, min_conviction)
    # Normalize between 0 and 1
    normalized_conviction = (conviction - min_conviction) / conviction_range if conviction_range else 0
    node_colors.append(normalized_conviction)
    node_sizes.append(normalized_conviction * 1000 + 100)  # Scale and offset size

# Use a colormap for coloring nodes based on conviction
cmap = plt.cm.coolwarm

pos = nx.spring_layout(G_2023_shop4_conviction)

plt.figure(figsize=(40, 22))
nodes = nx.draw_networkx_nodes(G_2023_shop4_conviction, pos, node_color=node_colors, node_size=node_sizes, cmap=cmap, alpha=0.8)
edges = nx.draw_networkx_edges(G_2023_shop4_conviction, pos, arrowstyle='->', arrowsize=10, edge_color='grey', alpha=0.5)
labels = nx.draw_networkx_labels(G_2023_shop4_conviction, pos, font_size=8)

plt.colorbar(nodes, label='Normalized Conviction')
plt.title('Network Graph of 2023 Shop 4 Based on Conviction')
plt.axis('off')
plt.show()

In [ ]:
print(max_conviction)

# Filtered top 30 and non redundant

In [ ]:
rules_2023_shop4_nonredundant.head()

In [ ]:
# Assuming rules_1992_shop2_nonredundant is your sorted rules DataFrame for shop2
rules_2023_shop4_nonredundant_top30 = rules_2023_shop4_nonredundant.copy(deep=True)

In [ ]:
rules_2023_shop4_nonredundant_top30_lift = rules_2023_shop4_nonredundant.copy(deep=True)

In [ ]:
# Select the top 30 rules based on lift or another criterion
rules_2023_shop4_nonredundant_top30_lift = rules_2023_shop4_nonredundant_top30_lift.sort_values(by='lift', ascending=False).head(30)

In [ ]:
rules_2023_shop4_nonredundant_top30_lift.head()

In [ ]:
# Select the top 30 rules based on confidence or another criterion
rules_2023_shop4_nonredundant_top30 = rules_2023_shop4_nonredundant_top30.sort_values(by='confidence', ascending=False).head(30)

In [ ]:
rules_2023_shop4_nonredundant_top30.info()

In [ ]:
rules_2023_shop4_nonredundant_top30['LHS'] = [','.join(list(x)) for x in rules_2023_shop4_nonredundant_top30['antecedents']]
rules_2023_shop4_nonredundant_top30['RHS'] = [','.join(list(x)) for x in rules_2023_shop4_nonredundant_top30['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2023_shop4_nonredundant_top30.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2023_shop4_nonredundant_top30, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2023 Shop 4")
plt.axis('off')
plt.show()


## Based on Indegree

In [ ]:
G_2023_shop4 = nx.DiGraph()

# Assuming 'rules_sorted_2023_shop4' contains association rules for shop 4 in 2023
for _, row in rules_2023_shop4_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as the weight
    G_2023_shop4.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2023_shop4 = nx.spring_layout(G_2023_shop4)

# Calculate node sizes based on indegree
in_degrees_2023_shop4 = dict(G_2023_shop4.in_degree())
node_sizes_2023_shop4 = [in_degrees_2023_shop4[node] * 100 + 1000 for node in G_2023_shop4.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2023_shop4 = (np.array(node_sizes_2023_shop4) - np.min(node_sizes_2023_shop4)) / (np.max(node_sizes_2023_shop4) - np.min(node_sizes_2023_shop4))

# Define a colormap from orange to red
cmap_2023_shop4 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop4 = [cmap_2023_shop4(size) for size in normalized_sizes_2023_shop4]

# Drawing the network
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2023_shop4, pos_2023_shop4, node_size=node_sizes_2023_shop4, node_color=node_colors_2023_shop4, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop4, pos_2023_shop4, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2023_shop4, pos_2023_shop4, font_size=8)

plt.title('Top 5 Association Rules Network for 2023 Shop 4 with Color Gradient')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2023_shop4_lift = nx.DiGraph()

lift_summary_2023_shop4 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop4_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop4_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop4:
        lift_summary_2023_shop4[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop4[consequents]['count'] += 1
    else:
        lift_summary_2023_shop4[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop4:
    lift_summary_2023_shop4[item]['avg_lift'] = lift_summary_2023_shop4[item]['total_lift'] / lift_summary_2023_shop4[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop4_lift = [lift_summary_2023_shop4[node]['avg_lift'] * 500 if node in lift_summary_2023_shop4 else 1000 for node in G_2023_shop4_lift.nodes()]

pos_2023_shop4_lift = nx.spring_layout(G_2023_shop4_lift)

# Colormap from orange to red
cmap_2023_shop4_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop4_lift = np.array(node_sizes_2023_shop4_lift)
normalized_sizes_2023_shop4_lift = (normalized_sizes_2023_shop4_lift - min(normalized_sizes_2023_shop4_lift)) / (max(normalized_sizes_2023_shop4_lift) - min(normalized_sizes_2023_shop4_lift))
node_colors_2023_shop4_lift = [cmap_2023_shop4_lift(size) for size in normalized_sizes_2023_shop4_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(40, 22))
nx.draw_networkx_nodes(G_2023_shop4_lift, pos_2023_shop4_lift, node_size=node_sizes_2023_shop4_lift, node_color=node_colors_2023_shop4_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop4_lift, pos_2023_shop4_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop4_lift, pos_2023_shop4_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 4 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop4_lift = nx.DiGraph()

lift_summary_2023_shop4 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop4_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop4_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop4:
        lift_summary_2023_shop4[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop4[consequents]['count'] += 1
    else:
        lift_summary_2023_shop4[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop4:
    lift_summary_2023_shop4[item]['avg_lift'] = lift_summary_2023_shop4[item]['total_lift'] / lift_summary_2023_shop4[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop4_lift = [lift_summary_2023_shop4[node]['avg_lift'] * 500 if node in lift_summary_2023_shop4 else 1000 for node in G_2023_shop4_lift.nodes()]

pos_2023_shop4_lift = nx.spring_layout(G_2023_shop4_lift)

# Colormap from orange to red
cmap_2023_shop4_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop4_lift = np.array(node_sizes_2023_shop4_lift)
normalized_sizes_2023_shop4_lift = (normalized_sizes_2023_shop4_lift - min(normalized_sizes_2023_shop4_lift)) / (max(normalized_sizes_2023_shop4_lift) - min(normalized_sizes_2023_shop4_lift))
node_colors_2023_shop4_lift = [cmap_2023_shop4_lift(size) for size in normalized_sizes_2023_shop4_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2023_shop4_lift, pos_2023_shop4_lift, node_size=node_sizes_2023_shop4_lift, node_color=node_colors_2023_shop4_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop4_lift, pos_2023_shop4_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop4_lift, pos_2023_shop4_lift, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 4 Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop4 = nx.DiGraph()

# Assuming 'rules_2023_shop4_nonredundant' contains association rules for 2023 Shop 4
for _, row in rules_2023_shop4_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop4.add_edge(antecedents, consequents, confidence=row['confidence'])

# Calculate node sizes based on indegree
in_degrees_2023_shop4 = dict(G_2023_shop4.in_degree())
node_sizes_2023_shop4 = [in_degrees_2023_shop4[node] * 100 + 1000 for node in G_2023_shop4.nodes()]

# Edge width based on confidence
edge_widths_2023_shop4 = [G_2023_shop4[u][v]['confidence'] * 15 for u, v in G_2023_shop4.edges()]

# Assuming normalized sizes and color mapping is desired
normalized_sizes_2023_shop4 = np.array(list(in_degrees_2023_shop4.values()))
normalized_sizes_2023_shop4 = (normalized_sizes_2023_shop4 - min(normalized_sizes_2023_shop4)) / (max(normalized_sizes_2023_shop4) - min(normalized_sizes_2023_shop4))
cmap_2023_shop4 = mcolors.LinearSegmentedColormap.from_list("MyCMapName", ["blue", "red"])

# Node colors based on a property (here, using indegree for example)
node_colors_2023_shop4 = [cmap_2023_shop4(size) for size in normalized_sizes_2023_shop4]

# Layout
pos_2023_shop4 = nx.spring_layout(G_2023_shop4, seed=42)

# Drawing
plt.figure(figsize=(40, 30))
nx.draw_networkx_edges(G_2023_shop4, pos_2023_shop4, width=edge_widths_2023_shop4, edge_color='grey', alpha=0.5)
nx.draw_networkx_nodes(G_2023_shop4, pos_2023_shop4, node_size=node_sizes_2023_shop4, node_color=node_colors_2023_shop4, alpha=0.8)
nx.draw_networkx_labels(G_2023_shop4, pos_2023_shop4, font_size=8, font_color='black')
plt.title('Network Graph of Association Rules Based on Confidence for 2023 Shop 4')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop4 = nx.DiGraph()

convictions_2023_shop4 = []  # To store conviction values for normalization

for _, row in rules_2023_shop4_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    conviction = row['conviction']  # Assuming each row has a 'conviction' column
    convictions_2023_shop4.append(conviction)
    G_2023_shop4.add_edge(antecedents, consequents, weight=row['confidence'], conviction=conviction)

# Normalize conviction values for edge width
max_conviction_2023_shop4 = max(convictions_2023_shop4)
min_conviction_2023_shop4 = min(convictions_2023_shop4)
conviction_range_2023_shop4 = max_conviction_2023_shop4 - min_conviction_2023_shop4
edge_widths_2023_shop4 = [(G_2023_shop4[u][v]['conviction'] - min_conviction_2023_shop4) / conviction_range_2023_shop4 * 30 + 0.5 for u, v in G_2023_shop4.edges()]  # Scale and offset widths

pos_2023_shop4 = nx.spring_layout(G_2023_shop4)

# Node sizes and colors as before
in_degrees_2023_shop4 = dict(G_2023_shop4.in_degree())
node_sizes_2023_shop4 = [in_degrees_2023_shop4[node] * 100 + 1000 for node in G_2023_shop4.nodes()]
normalized_sizes_2023_shop4 = (np.array(node_sizes_2023_shop4) - min(node_sizes_2023_shop4)) / (max(node_sizes_2023_shop4) - min(node_sizes_2023_shop4))
cmap_2023_shop4 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop4 = [cmap_2023_shop4(size) for size in normalized_sizes_2023_shop4]

# Drawing the network with conviction-based edge widths
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2023_shop4, pos_2023_shop4, node_size=node_sizes_2023_shop4, node_color=node_colors_2023_shop4, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop4, pos_2023_shop4, width=edge_widths_2023_shop4, edge_color='grey', alpha=0.5)
nx.draw_networkx_labels(G_2023_shop4, pos_2023_shop4, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 4 with Conviction-based Edge Thickness')
plt.axis('off')
plt.show()


In [ ]:
#the node sizes are based on the indegree where each node acts as a consequent,

In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2023_shop4_nonredundant_top30.loc[:, 'antecedents_'] = rules_2023_shop4_nonredundant_top30['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2023_shop4_nonredundant_top30.loc[:, 'consequents_'] = rules_2023_shop4_nonredundant_top30['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2023_shop4_nonredundant_top30, x='consequents_', y='antecedents_', s=rules_2023_shop4_nonredundant_top30['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2023_shop4_nonredundant_top30.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2023 Shop 4", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
print(rules_2023_shop4_nonredundant_top30[['antecedents', 'consequents']].head(10))

## Monetary Analysis of the Top Rules

In [ ]:
# Step 1: Extract unique product codes from the rules
unique_product_codes = set()
for idx, row in rules_2023_shop4_nonredundant_top30.iterrows():
    # Unite antecedents and consequents into a single set
    all_products = row['antecedents'].union(row['consequents'])
    unique_product_codes.update(all_products)

# Step 2: Create a DataFrame with unique product codes and their unit prices
# Filter sales_data for these product codes and drop duplicates to ensure each product code appears once
product_prices = sales_data[sales_data['product_code'].isin(unique_product_codes)][['product_code', 'unit_price']].drop_duplicates()

# Create a dictionary for quick lookup of prices
price_dict = product_prices.set_index('product_code')['unit_price'].to_dict()

# Step 3: Calculate the total price for each rule and collect data for the new DataFrame
data_for_new_df = []
for idx, row in rules_2023_shop4_nonredundant_top30.iterrows():
    # Combine sets of antecedents and consequents
    all_products = row['antecedents'].union(row['consequents'])
    total_price = sum(price_dict.get(prod, 0) for prod in all_products)  # Use get to avoid KeyError if any product code is missing
    data_for_new_df.append({
        'antecedents': row['antecedents'],
        'consequents': row['consequents'],
        'total_unit_price': total_price
    })

# Create a new DataFrame with only antecedents, consequents, and total unit price
rules_and_price_shop4_2023 = pd.DataFrame(data_for_new_df)

# Sort the DataFrame by total_unit_price in descending order
rules_and_price_shop4_2023_sorted = rules_and_price_shop4_2023.sort_values(by='total_unit_price', ascending=False)

# Print the sorted DataFrame to see the specified columns along with their total unit prices
print(rules_and_price_shop4_2023_sorted)


In [ ]:
# Convert 'trx_date' to datetime format if not already done so
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Add a 'year' column if it doesn't exist
sales_data['year'] = sales_data['trx_date'].dt.year

# Step 2: Filter sales_data for shop4 and the year 2023
filtered_sales_data_2023 = sales_data[(sales_data['shopID'] == 4) & (sales_data['year'] == 2023)]

# Step 3: Initialize a list to hold the rule data including total sales value
rule_sales_summaries_2023 = []

# Step 4: Iterate through each rule in the DataFrame
for index, rule in rules_2023_shop4_nonredundant_top30.iterrows():
    all_products = rule['antecedents'].union(rule['consequents'])

    # Filter the sales data for transactions that include all products in the rule
    # This step ensures that we are only considering transactions that contain all specified products
    mask = filtered_sales_data_2023['product_code'].isin(all_products)
    transaction_ids = filtered_sales_data_2023[mask].groupby('trx_id').filter(lambda x: all_products.issubset(set(x['product_code']))).trx_id.unique()
    matched_transactions = filtered_sales_data_2023[filtered_sales_data_2023['trx_id'].isin(transaction_ids)]
    
    # Step 5: Sum the total_value of these transactions
    total_sales_value = matched_transactions['total_value'].sum()

    # Append the result to the list
    rule_sales_summaries_2023.append({
        'antecedents': rule['antecedents'],
        'consequents': rule['consequents'],
        'total_sales_value': total_sales_value
    })

# Step 6: Convert the list to a DataFrame
rules_sales_values_df_2023 = pd.DataFrame(rule_sales_summaries_2023)

# Step 7: Sort by total_sales_value in descending order
sorted_rules_sales_values_df_2023 = rules_sales_values_df_2023.sort_values(by='total_sales_value', ascending=False)

# Step 8: Print the sorted DataFrame
print(sorted_rules_sales_values_df_2023)


## Narrowing down based on Lift

In [ ]:
rules_2023_shop4_nonredundant_top30_lift['LHS'] = [','.join(list(x)) for x in rules_2023_shop4_nonredundant_top30_lift['antecedents']]
rules_2023_shop4_nonredundant_top30_lift['RHS'] = [','.join(list(x)) for x in rules_2023_shop4_nonredundant_top30_lift['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2023_shop4_nonredundant_top30_lift.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2023_shop4_nonredundant_top30_lift, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2023 Shop 4")
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop4_lift = nx.DiGraph()

lift_summary_2023_shop4 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop4_nonredundant_top30_lift.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop4_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop4:
        lift_summary_2023_shop4[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop4[consequents]['count'] += 1
    else:
        lift_summary_2023_shop4[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop4:
    lift_summary_2023_shop4[item]['avg_lift'] = lift_summary_2023_shop4[item]['total_lift'] / lift_summary_2023_shop4[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop4_lift = [lift_summary_2023_shop4[node]['avg_lift'] * 500 if node in lift_summary_2023_shop4 else 1000 for node in G_2023_shop4_lift.nodes()]

pos_2023_shop4_lift = nx.spring_layout(G_2023_shop4_lift)

# Colormap from orange to red
cmap_2023_shop4_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop4_lift = np.array(node_sizes_2023_shop4_lift)
normalized_sizes_2023_shop4_lift = (normalized_sizes_2023_shop4_lift - min(normalized_sizes_2023_shop4_lift)) / (max(normalized_sizes_2023_shop4_lift) - min(normalized_sizes_2023_shop4_lift))
node_colors_2023_shop4_lift = [cmap_2023_shop4_lift(size) for size in normalized_sizes_2023_shop4_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(60, 42))
nx.draw_networkx_nodes(G_2023_shop4_lift, pos_2023_shop4_lift, node_size=node_sizes_2023_shop4_lift, node_color=node_colors_2023_shop4_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop4_lift, pos_2023_shop4_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop4_lift, pos_2023_shop4_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 4 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2023_shop4_nonredundant_top30_lift.loc[:, 'antecedents_'] = rules_2023_shop4_nonredundant_top30_lift['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2023_shop4_nonredundant_top30_lift.loc[:, 'consequents_'] = rules_2023_shop4_nonredundant_top30_lift['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2023_shop4_nonredundant_top30_lift, x='consequents_', y='antecedents_', s=rules_2023_shop4_nonredundant_top30_lift['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2023_shop4_nonredundant_top30_lift.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2023 Shop 4", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
# Print only the 'antecedents' and 'consequents' columns of the top 10 rules
print(rules_2023_shop4_nonredundant_top30_lift[['antecedents', 'consequents']].head(10))


# Shop5 2021 SEARCH 9

In [ ]:
# Filter for records from shop ID 1 and the year 1993
sales_data_2021_shop5 = sales_data_shop5[(sales_data_shop5['trx_date'].dt.year == 2021)]

In [ ]:
sales_data_2021_shop5.head()

In [ ]:
# Grouping product codes by trx_id
transactions_2021_shop5 = sales_data_2021_shop5.groupby('trx_id')['product_code'].apply(list).tolist()

In [ ]:
# Encode transaction data
encoder_2021_shop5 = TransactionEncoder()
trans_array_2021_shop5 = encoder_2021_shop5.fit(transactions_2021_shop5).transform(transactions_2021_shop5)
trans_df_2021_shop5 = pd.DataFrame(trans_array_2021_shop5, columns=encoder_2021_shop5.columns_)

In [ ]:
# Find frequent itemsets
frequent_itemsets_2021_shop5 = apriori(trans_df_2021_shop5, min_support=0.001, use_colnames=True)

In [ ]:
print(frequent_itemsets_2021_shop5)

In [ ]:
## Generate the association rules - by confidence
rulesConfidence_2021_shop5 = association_rules(frequent_itemsets_2021_shop5, metric="confidence", min_threshold=0.10)
rulesConfidence_2021_shop5.sort_values(by='confidence', ascending=False, inplace=True)


In [ ]:
# Note: This is a hypothetical example, as I can't directly manipulate or access dataframes not provided within this code cell.

# Replace frozen sets with strings
rulesConfidence_2021_shop5['antecedents_'] = rulesConfidence_2021_shop5['antecedents'].apply(lambda a: ','.join(list(a)))
rulesConfidence_2021_shop5['consequents_'] = rulesConfidence_2021_shop5['consequents'].apply(lambda a: ','.join(list(a)))

# Draw
fig, ax = plt.subplots(figsize=(14,30))
ax = plt.scatter(data=rulesConfidence_2021_shop5, x='consequents_', y='antecedents_', s=rulesConfidence_2021_shop5['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rulesConfidence_2021_shop5.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2021 Shop 5", fontsize=14)
plt.xlabel("Consequents (RHS)", fontsize=12)
plt.ylabel("Antecedents (LHS)", fontsize=12)
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate(rotation=45)
plt.tight_layout()

plt.show()



## Eliminate Redundant Rules

In [ ]:
rulesConfidence_2021_shop5.head()

In [ ]:
rulesConfidence_2021_shop5.info()

In [ ]:
# If 'antecedents' and 'consequents' are not frozensets, convert them
rules_2021_shop5_redundant = rulesConfidence_2021_shop5.copy()
rules_2021_shop5_redundant['antecedents'] = rules_2021_shop5_redundant['antecedents'].apply(lambda x: frozenset(x))
rules_2021_shop5_redundant['consequents'] = rules_2021_shop5_redundant['consequents'].apply(lambda x: frozenset(x))


In [ ]:
# Create a copy of the DataFrame to avoid modifying the original while iterating
rules_2021_shop5_checkredundance = rules_2021_shop5_redundant.copy()

# List to hold non-redundant rules
non_redundant_rules_2021_shop5 = []

for _, rule in rules_2021_shop5_checkredundance.iterrows():
    if not is_redundant(rule, rules_2021_shop5_checkredundance):
        non_redundant_rules_2021_shop5.append(rule)

# Convert the list back to a DataFrame
rules_2021_shop5_nonredundant = pd.DataFrame(non_redundant_rules_2021_shop5).reset_index(drop=True)


In [ ]:
print(rules_2021_shop5_nonredundant.head())
print(f"Original number of rules: {len(rules_2021_shop5_redundant)}")
print(f"Number of non-redundant rules: {len(rules_2021_shop5_nonredundant)}")


In [ ]:
rules_2021_shop5_nonredundant.head()

In [ ]:
rules_2021_shop5_redundant.info()

In [ ]:
rules_2021_shop5_nonredundant.head()

In [ ]:
rules_2021_shop5_nonredundant_backup = rules_2021_shop5_nonredundant.copy()

In [ ]:
rules_2021_shop5_nonredundant_unfiltered = rules_2021_shop5_nonredundant.copy()

In [ ]:
#reset
#rules_2021_shop5_nonredundant = rules_2021_shop5_nonredundant_backup.copy()

# Redundant and Unfiltered

In [ ]:
rules_2021_shop5_redundant.head()

In [ ]:
G_2021_shop5 = nx.DiGraph()

# Iterate through rules for 2021 Shop 5 and add edges to the graph
for _, row in rules_2021_shop5_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as weight
    G_2021_shop5.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2021_shop5 = nx.spring_layout(G_2021_shop5)

# Recalculate node sizes based on indegree or another metric
in_degrees_2021_shop5 = dict(G_2021_shop5.in_degree())
node_sizes_2021_shop5 = [in_degrees_2021_shop5[node] * 100 + 1000 for node in G_2021_shop5.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2021_shop5 = (np.array(node_sizes_2021_shop5) - np.min(node_sizes_2021_shop5)) / (np.max(node_sizes_2021_shop5) - np.min(node_sizes_2021_shop5))

# Define a colormap from orange to red for visual consistency
cmap_2021_shop5 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2021_shop5 = [cmap_2021_shop5(size) for size in normalized_sizes_2021_shop5]

# Drawing the network
plt.figure(figsize=(60, 43))
nx.draw_networkx_nodes(G_2021_shop5, pos_2021_shop5, node_size=node_sizes_2021_shop5, node_color=node_colors_2021_shop5, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop5, pos_2021_shop5, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2021_shop5, pos_2021_shop5, font_size=10)

plt.title('Top 5 Association Rules Network for 2021 Shop 5 with Color Gradient')
plt.axis('off')
plt.show()


### Based on LIFT

In [ ]:
G_2021_shop5_lift = nx.DiGraph()

lift_summary_2021_shop5 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2021_shop5_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop5_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2021_shop5:
        lift_summary_2021_shop5[consequents]['total_lift'] += row['lift']
        lift_summary_2021_shop5[consequents]['count'] += 1
    else:
        lift_summary_2021_shop5[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2021_shop5:
    lift_summary_2021_shop5[item]['avg_lift'] = lift_summary_2021_shop5[item]['total_lift'] / lift_summary_2021_shop5[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2021_shop5_lift = [lift_summary_2021_shop5[node]['avg_lift'] * 500 if node in lift_summary_2021_shop5 else 1000 for node in G_2021_shop5_lift.nodes()]

pos_2021_shop5_lift = nx.spring_layout(G_2021_shop5_lift)

# Colormap from orange to red
cmap_2021_shop5_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2021_shop5_lift = np.array(node_sizes_2021_shop5_lift)
normalized_sizes_2021_shop5_lift = (normalized_sizes_2021_shop5_lift - min(normalized_sizes_2021_shop5_lift)) / (max(normalized_sizes_2021_shop5_lift) - min(normalized_sizes_2021_shop5_lift))
node_colors_2021_shop5_lift = [cmap_2021_shop5_lift(size) for size in normalized_sizes_2021_shop5_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(60, 42))
nx.draw_networkx_nodes(G_2021_shop5_lift, pos_2021_shop5_lift, node_size=node_sizes_2021_shop5_lift, node_color=node_colors_2021_shop5_lift, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop5_lift, pos_2021_shop5_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2021_shop5_lift, pos_2021_shop5_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2021 Shop 5 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


# nonredundant_unfiltered

## Based on Indegree

In [ ]:
G_2021_shop5_nonredundant_unfiltered = nx.DiGraph()

# Iterating through each row to add edges with confidence as an attribute
for _, row in rules_2021_shop5_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop5_nonredundant_unfiltered.add_edge(antecedents, consequents, weight=row['confidence'])

# Defining the layout for network visualization
pos_2021_shop5_nonredundant_unfiltered = nx.spring_layout(G_2021_shop5_nonredundant_unfiltered)

# Calculating node sizes based on the indegree for color mapping
in_degrees_2021_shop5_nonredundant_unfiltered = dict(G_2021_shop5_nonredundant_unfiltered.in_degree())
node_sizes_2021_shop5_nonredundant_unfiltered = [in_degrees_2021_shop5_nonredundant_unfiltered[node] * 100 + 1000 for node in G_2021_shop5_nonredundant_unfiltered.nodes()]

# Normalizing node sizes for color mapping
normalized_sizes_2021_shop5_nonredundant_unfiltered = (np.array(node_sizes_2021_shop5_nonredundant_unfiltered) - min(node_sizes_2021_shop5_nonredundant_unfiltered)) / (max(node_sizes_2021_shop5_nonredundant_unfiltered) - min(node_sizes_2021_shop5_nonredundant_unfiltered))

# Defining a color map from orange to red
cmap_2021_shop5_nonredundant_unfiltered = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])

# Mapping normalized node sizes to colors
node_colors_2021_shop5_nonredundant_unfiltered = [cmap_2021_shop5_nonredundant_unfiltered(size) for size in normalized_sizes_2021_shop5_nonredundant_unfiltered]

# Drawing the network with node sizes and colors
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2021_shop5_nonredundant_unfiltered, pos_2021_shop5_nonredundant_unfiltered, node_size=node_sizes_2021_shop5_nonredundant_unfiltered, node_color=node_colors_2021_shop5_nonredundant_unfiltered, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop5_nonredundant_unfiltered, pos_2021_shop5_nonredundant_unfiltered, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2021_shop5_nonredundant_unfiltered, pos_2021_shop5_nonredundant_unfiltered, font_size=10)

plt.title('Node Size Based on Indegree for 2021 Shop 5')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2021_shop5_lift = nx.DiGraph()

lift_summary_2021_shop5 = {}

for _, row in rules_2021_shop5_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop5_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Update lift summary for consequents
    if consequents in lift_summary_2021_shop5:
        lift_summary_2021_shop5[consequents]['total_lift'] += row['lift']
        lift_summary_2021_shop5[consequents]['count'] += 1
    else:
        lift_summary_2021_shop5[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculate average lift for each consequent
for item in lift_summary_2021_shop5:
    lift_summary_2021_shop5[item]['avg_lift'] = lift_summary_2021_shop5[item]['total_lift'] / lift_summary_2021_shop5[item]['count']

# Node sizes based on average lift, scaled appropriately
node_sizes_2021_shop5_lift = [lift_summary_2021_shop5[node]['avg_lift'] * 500 if node in lift_summary_2021_shop5 else 1000 for node in G_2021_shop5_lift.nodes()]

pos_2021_shop5_lift = nx.spring_layout(G_2021_shop5_lift)

# Use a colormap that transitions from orange to red
cmap_2021_shop5_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2021_shop5_lift = np.array(node_sizes_2021_shop5_lift)
normalized_sizes_2021_shop5_lift = (normalized_sizes_2021_shop5_lift - min(normalized_sizes_2021_shop5_lift)) / (max(normalized_sizes_2021_shop5_lift) - min(normalized_sizes_2021_shop5_lift))
node_colors_2021_shop5_lift = [cmap_2021_shop5_lift(size) for size in normalized_sizes_2021_shop5_lift]

plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2021_shop5_lift, pos_2021_shop5_lift, node_size=node_sizes_2021_shop5_lift, node_color=node_colors_2021_shop5_lift, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop5_lift, pos_2021_shop5_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2021_shop5_lift, pos_2021_shop5_lift, font_size=10)

plt.title('Node Size Based on Average Lift for 2021 Shop 5')
plt.axis('off')
plt.show()


# Based on conviction

In [ ]:
rules_2021_shop5_nonredundant.head()

In [ ]:
G_2021_shop5_conviction = nx.DiGraph()

conviction_summary_2021_shop5 = {}
count_summary_2021_shop5 = {}

# Assuming 'rules_2021_shop5_nonredundant_unfiltered' contains a 'conviction' column
for _, row in rules_2021_shop5_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop5_conviction.add_edge(antecedents, consequents, weight=row['confidence'], conviction=row['conviction'])
    
    # Aggregate conviction values and count occurrences
    if consequents in conviction_summary_2021_shop5:
        conviction_summary_2021_shop5[consequents] += row['conviction']
        count_summary_2021_shop5[consequents] += 1
    else:
        conviction_summary_2021_shop5[consequents] = row['conviction']
        count_summary_2021_shop5[consequents] = 1

# Calculate average conviction values
average_conviction_summary_2021_shop5 = {k: conviction_summary_2021_shop5[k] / count_summary_2021_shop5[k] for k in conviction_summary_2021_shop5}

# Normalize conviction values for sizing and coloring
max_conviction = max(average_conviction_summary_2021_shop5.values())
min_conviction = min(average_conviction_summary_2021_shop5.values())
conviction_range = max_conviction - min_conviction
node_colors = []
node_sizes = []

for node in G_2021_shop5_conviction.nodes():
    # Get the average conviction value for the node, defaulting to min if not present
    conviction = average_conviction_summary_2021_shop5.get(node, min_conviction)
    # Normalize between 0 and 1
    normalized_conviction = (conviction - min_conviction) / conviction_range if conviction_range else 0
    node_colors.append(normalized_conviction)
    node_sizes.append(normalized_conviction * 1000 + 100)  # Scale and offset size

# Use a colormap for coloring nodes based on conviction
cmap = plt.cm.coolwarm

pos = nx.spring_layout(G_2021_shop5_conviction)

plt.figure(figsize=(40, 22))
nodes = nx.draw_networkx_nodes(G_2021_shop5_conviction, pos, node_color=node_colors, node_size=node_sizes, cmap=cmap, alpha=0.8)
edges = nx.draw_networkx_edges(G_2021_shop5_conviction, pos, arrowstyle='->', arrowsize=10, edge_color='grey', alpha=0.5)
labels = nx.draw_networkx_labels(G_2021_shop5_conviction, pos, font_size=8)

plt.colorbar(nodes, label='Normalized Conviction')
plt.title('Network Graph of 2021 Shop 5 Based on Conviction')
plt.axis('off')
plt.show()

In [ ]:
print(max_conviction)

# Filtered top 30 and non redundant

In [ ]:
rules_2021_shop5_nonredundant.head()

In [ ]:
# Assuming rules_1992_shop2_nonredundant is your sorted rules DataFrame for shop2
rules_2021_shop5_nonredundant_top30 = rules_2021_shop5_nonredundant.copy(deep=True)

In [ ]:
rules_2021_shop5_nonredundant_top30_lift = rules_2021_shop5_nonredundant.copy(deep=True)

In [ ]:
# Select the top 30 rules based on lift or another criterion
rules_2021_shop5_nonredundant_top30_lift = rules_2021_shop5_nonredundant_top30_lift.sort_values(by='lift', ascending=False).head(30)

In [ ]:
rules_2021_shop5_nonredundant_top30_lift.head()

In [ ]:
# Select the top 30 rules based on confidence or another criterion
rules_2021_shop5_nonredundant_top30 = rules_2021_shop5_nonredundant_top30.sort_values(by='confidence', ascending=False).head(30)

In [ ]:
rules_2021_shop5_nonredundant_top30.info()

In [ ]:
rules_2021_shop5_nonredundant_top30['LHS'] = [','.join(list(x)) for x in rules_2021_shop5_nonredundant_top30['antecedents']]
rules_2021_shop5_nonredundant_top30['RHS'] = [','.join(list(x)) for x in rules_2021_shop5_nonredundant_top30['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2021_shop5_nonredundant_top30.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2021_shop5_nonredundant_top30, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2021 Shop 5")
plt.axis('off')
plt.show()


## Based on Indegree

In [ ]:
G_2021_shop5 = nx.DiGraph()

# Assuming 'rules_sorted_2021_shop5' contains association rules for shop 5 in 2021
for _, row in rules_2021_shop5_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as the weight
    G_2021_shop5.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2021_shop5 = nx.spring_layout(G_2021_shop5)

# Calculate node sizes based on indegree
in_degrees_2021_shop5 = dict(G_2021_shop5.in_degree())
node_sizes_2021_shop5 = [in_degrees_2021_shop5[node] * 100 + 1000 for node in G_2021_shop5.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2021_shop5 = (np.array(node_sizes_2021_shop5) - np.min(node_sizes_2021_shop5)) / (np.max(node_sizes_2021_shop5) - np.min(node_sizes_2021_shop5))

# Define a colormap from orange to red
cmap_2021_shop5 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2021_shop5 = [cmap_2021_shop5(size) for size in normalized_sizes_2021_shop5]

# Drawing the network
plt.figure(figsize=(30, 24))
nx.draw_networkx_nodes(G_2021_shop5, pos_2021_shop5, node_size=node_sizes_2021_shop5, node_color=node_colors_2021_shop5, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop5, pos_2021_shop5, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2021_shop5, pos_2021_shop5, font_size=8)

plt.title('Top 5 Association Rules Network for 2021 Shop 5 with Color Gradient')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2021_shop5_lift = nx.DiGraph()

lift_summary_2021_shop5 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2021_shop5_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop5_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2021_shop5:
        lift_summary_2021_shop5[consequents]['total_lift'] += row['lift']
        lift_summary_2021_shop5[consequents]['count'] += 1
    else:
        lift_summary_2021_shop5[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2021_shop5:
    lift_summary_2021_shop5[item]['avg_lift'] = lift_summary_2021_shop5[item]['total_lift'] / lift_summary_2021_shop5[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2021_shop5_lift = [lift_summary_2021_shop5[node]['avg_lift'] * 500 if node in lift_summary_2021_shop5 else 1000 for node in G_2021_shop5_lift.nodes()]

pos_2021_shop5_lift = nx.spring_layout(G_2021_shop5_lift)

# Colormap from orange to red
cmap_2021_shop5_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2021_shop5_lift = np.array(node_sizes_2021_shop5_lift)
normalized_sizes_2021_shop5_lift = (normalized_sizes_2021_shop5_lift - min(normalized_sizes_2021_shop5_lift)) / (max(normalized_sizes_2021_shop5_lift) - min(normalized_sizes_2021_shop5_lift))
node_colors_2021_shop5_lift = [cmap_2021_shop5_lift(size) for size in normalized_sizes_2021_shop5_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(30, 22))
nx.draw_networkx_nodes(G_2021_shop5_lift, pos_2021_shop5_lift, node_size=node_sizes_2021_shop5_lift, node_color=node_colors_2021_shop5_lift, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop5_lift, pos_2021_shop5_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2021_shop5_lift, pos_2021_shop5_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2021 Shop 5 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2021_shop5_lift = nx.DiGraph()

lift_summary_2021_shop5 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2021_shop5_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop5_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2021_shop5:
        lift_summary_2021_shop5[consequents]['total_lift'] += row['lift']
        lift_summary_2021_shop5[consequents]['count'] += 1
    else:
        lift_summary_2021_shop5[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2021_shop5:
    lift_summary_2021_shop5[item]['avg_lift'] = lift_summary_2021_shop5[item]['total_lift'] / lift_summary_2021_shop5[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2021_shop5_lift = [lift_summary_2021_shop5[node]['avg_lift'] * 500 if node in lift_summary_2021_shop5 else 1000 for node in G_2021_shop5_lift.nodes()]

pos_2021_shop5_lift = nx.spring_layout(G_2021_shop5_lift)

# Colormap from orange to red
cmap_2021_shop5_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2021_shop5_lift = np.array(node_sizes_2021_shop5_lift)
normalized_sizes_2021_shop5_lift = (normalized_sizes_2021_shop5_lift - min(normalized_sizes_2021_shop5_lift)) / (max(normalized_sizes_2021_shop5_lift) - min(normalized_sizes_2021_shop5_lift))
node_colors_2021_shop5_lift = [cmap_2021_shop5_lift(size) for size in normalized_sizes_2021_shop5_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2021_shop5_lift, pos_2021_shop5_lift, node_size=node_sizes_2021_shop5_lift, node_color=node_colors_2021_shop5_lift, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop5_lift, pos_2021_shop5_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2021_shop5_lift, pos_2021_shop5_lift, font_size=10)

plt.title('Top 5 Association Rules Network for 2021 Shop 5 Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2021_shop5 = nx.DiGraph()

# Assuming 'rules_2021_shop5_nonredundant' contains association rules for 2021 Shop 5
for _, row in rules_2021_shop5_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop5.add_edge(antecedents, consequents, confidence=row['confidence'])

# Calculate node sizes based on indegree
in_degrees_2021_shop5 = dict(G_2021_shop5.in_degree())
node_sizes_2021_shop5 = [in_degrees_2021_shop5[node] * 100 + 1000 for node in G_2021_shop5.nodes()]

# Edge width based on confidence
edge_widths_2021_shop5 = [G_2021_shop5[u][v]['confidence'] * 15 for u, v in G_2021_shop5.edges()]

# Assuming normalized sizes and color mapping is desired
normalized_sizes_2021_shop5 = np.array(list(in_degrees_2021_shop5.values()))
normalized_sizes_2021_shop5 = (normalized_sizes_2021_shop5 - min(normalized_sizes_2021_shop5)) / (max(normalized_sizes_2021_shop5) - min(normalized_sizes_2021_shop5))
cmap_2021_shop5 = mcolors.LinearSegmentedColormap.from_list("MyCMapName", ["blue", "red"])

# Node colors based on a property (here, using indegree for example)
node_colors_2021_shop5 = [cmap_2021_shop5(size) for size in normalized_sizes_2021_shop5]

# Layout
pos_2021_shop5 = nx.spring_layout(G_2021_shop5, seed=42)

# Drawing
plt.figure(figsize=(40, 30))
nx.draw_networkx_edges(G_2021_shop5, pos_2021_shop5, width=edge_widths_2021_shop5, edge_color='grey', alpha=0.5)
nx.draw_networkx_nodes(G_2021_shop5, pos_2021_shop5, node_size=node_sizes_2021_shop5, node_color=node_colors_2021_shop5, alpha=0.8)
nx.draw_networkx_labels(G_2021_shop5, pos_2021_shop5, font_size=8, font_color='black')
plt.title('Network Graph of Association Rules Based on Confidence for 2021 Shop 5')
plt.axis('off')
plt.show()


In [ ]:
G_2021_shop5 = nx.DiGraph()

convictions_2021_shop5 = []  # To store conviction values for normalization

for _, row in rules_2021_shop5_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    conviction_2021_shop5 = row['conviction']  # Assuming each row has a 'conviction' column
    convictions_2021_shop5.append(conviction_2021_shop5)
    G_2021_shop5.add_edge(antecedents, consequents, weight=row['confidence'], conviction=conviction_2021_shop5)

# Normalize conviction values for edge width
max_conviction_2021_shop5 = max(convictions_2021_shop5)
min_conviction_2021_shop5 = min(convictions_2021_shop5)
conviction_range_2021_shop5 = max_conviction_2021_shop5 - min_conviction_2021_shop5
edge_widths_2021_shop5 = [(G_2021_shop5[u][v]['conviction'] - min_conviction_2021_shop5) / conviction_range_2021_shop5 * 30 + 0.5 for u, v in G_2021_shop5.edges()]  # Scale and offset widths

pos_2021_shop5 = nx.spring_layout(G_2021_shop5)

# Node sizes and colors as before
in_degrees_2021_shop5 = dict(G_2021_shop5.in_degree())
node_sizes_2021_shop5 = [in_degrees_2021_shop5[node] * 100 + 1000 for node in G_2021_shop5.nodes()]
normalized_sizes_2021_shop5 = (np.array(node_sizes_2021_shop5) - min(node_sizes_2021_shop5)) / (max(node_sizes_2021_shop5) - min(node_sizes_2021_shop5))
cmap_2021_shop5 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2021_shop5 = [cmap_2021_shop5(size) for size in normalized_sizes_2021_shop5]

# Drawing the network with conviction-based edge widths
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2021_shop5, pos_2021_shop5, node_size=node_sizes_2021_shop5, node_color=node_colors_2021_shop5, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop5, pos_2021_shop5, width=edge_widths_2021_shop5, edge_color='grey', alpha=0.5)
nx.draw_networkx_labels(G_2021_shop5, pos_2021_shop5, font_size=10)

plt.title('Top 5 Association Rules Network for 2021 Shop 5 with Conviction-based Edge Thickness')
plt.axis('off')
plt.show()


In [ ]:
#the node sizes are based on the indegree where each node acts as a consequent,

In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2021_shop5_nonredundant_top30.loc[:, 'antecedents_'] = rules_2021_shop5_nonredundant_top30['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2021_shop5_nonredundant_top30.loc[:, 'consequents_'] = rules_2021_shop5_nonredundant_top30['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2021_shop5_nonredundant_top30, x='consequents_', y='antecedents_', s=rules_2021_shop5_nonredundant_top30['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2021_shop5_nonredundant_top30.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2021 Shop 5", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
print(rules_2021_shop5_nonredundant_top30[['antecedents', 'consequents']].head(10))

## Monetary Analysis of the Top Rules

In [ ]:
# Step 1: Extract unique product codes from the rules
unique_product_codes = set()
for idx, row in rules_2021_shop5_nonredundant_top30.iterrows():
    # Unite antecedents and consequents into a single set
    all_products = row['antecedents'].union(row['consequents'])
    unique_product_codes.update(all_products)

# Step 2: Create a DataFrame with unique product codes and their unit prices
# Filter sales_data for these product codes and drop duplicates to ensure each product code appears once
product_prices = sales_data[sales_data['product_code'].isin(unique_product_codes)][['product_code', 'unit_price']].drop_duplicates()

# Create a dictionary for quick lookup of prices
price_dict = product_prices.set_index('product_code')['unit_price'].to_dict()

# Step 3: Calculate the total price for each rule and collect data for the new DataFrame
data_for_new_df = []
for idx, row in rules_2021_shop5_nonredundant_top30.iterrows():
    # Combine sets of antecedents and consequents
    all_products = row['antecedents'].union(row['consequents'])
    total_price = sum(price_dict.get(prod, 0) for prod in all_products)  # Use get to avoid KeyError if any product code is missing
    data_for_new_df.append({
        'antecedents': row['antecedents'],
        'consequents': row['consequents'],
        'total_unit_price': total_price
    })

# Create a new DataFrame with only antecedents, consequents, and total unit price
rules_and_price_shop5_2021 = pd.DataFrame(data_for_new_df)

# Sort the DataFrame by total_unit_price in descending order
rules_and_price_shop5_2021_sorted = rules_and_price_shop5_2021.sort_values(by='total_unit_price', ascending=False)

# Print the sorted DataFrame to see the specified columns along with their total unit prices
print(rules_and_price_shop5_2021_sorted)


In [ ]:
# Convert 'trx_date' to datetime format if not already done so
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Add a 'year' column if it doesn't exist
sales_data['year'] = sales_data['trx_date'].dt.year

# Step 2: Filter sales_data for shop5 and the year 2021
filtered_sales_data_2021 = sales_data[(sales_data['shopID'] == 5) & (sales_data['year'] == 2021)]

# Step 3: Initialize a list to hold the rule data including total sales value
rule_sales_summaries_2021 = []

# Step 4: Iterate through each rule in the DataFrame
for index, rule in rules_2021_shop5_nonredundant_top30.iterrows():
    all_products = rule['antecedents'].union(rule['consequents'])

    # Filter the sales data for transactions that include all products in the rule
    # This step ensures that we are only considering transactions that contain all specified products
    mask = filtered_sales_data_2021['product_code'].isin(all_products)
    transaction_ids = filtered_sales_data_2021[mask].groupby('trx_id').filter(lambda x: all_products.issubset(set(x['product_code']))).trx_id.unique()
    matched_transactions = filtered_sales_data_2021[filtered_sales_data_2021['trx_id'].isin(transaction_ids)]
    
    # Step 5: Sum the total_value of these transactions
    total_sales_value = matched_transactions['total_value'].sum()

    # Append the result to the list
    rule_sales_summaries_2021.append({
        'antecedents': rule['antecedents'],
        'consequents': rule['consequents'],
        'total_sales_value': total_sales_value
    })

# Step 6: Convert the list to a DataFrame
rules_sales_values_df_2021 = pd.DataFrame(rule_sales_summaries_2021)

# Step 7: Sort by total_sales_value in descending order
sorted_rules_sales_values_df_2021 = rules_sales_values_df_2021.sort_values(by='total_sales_value', ascending=False)

# Step 8: Print the sorted DataFrame
print(sorted_rules_sales_values_df_2021)


## Narrowing down based on Lift

In [ ]:
rules_2021_shop5_nonredundant_top30_lift['LHS'] = [','.join(list(x)) for x in rules_2021_shop5_nonredundant_top30_lift['antecedents']]
rules_2021_shop5_nonredundant_top30_lift['RHS'] = [','.join(list(x)) for x in rules_2021_shop5_nonredundant_top30_lift['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2021_shop5_nonredundant_top30_lift.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2021_shop5_nonredundant_top30_lift, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2021 Shop 5")
plt.axis('off')
plt.show()


In [ ]:
G_2021_shop5_lift = nx.DiGraph()

lift_summary_2021_shop5 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2021_shop5_nonredundant_top30_lift.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop5_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2021_shop5:
        lift_summary_2021_shop5[consequents]['total_lift'] += row['lift']
        lift_summary_2021_shop5[consequents]['count'] += 1
    else:
        lift_summary_2021_shop5[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2021_shop5:
    lift_summary_2021_shop5[item]['avg_lift'] = lift_summary_2021_shop5[item]['total_lift'] / lift_summary_2021_shop5[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2021_shop5_lift = [lift_summary_2021_shop5[node]['avg_lift'] * 500 if node in lift_summary_2021_shop5 else 1000 for node in G_2021_shop5_lift.nodes()]

pos_2021_shop5_lift = nx.spring_layout(G_2021_shop5_lift)

# Colormap from orange to red
cmap_2021_shop5_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2021_shop5_lift = np.array(node_sizes_2021_shop5_lift)
normalized_sizes_2021_shop5_lift = (normalized_sizes_2021_shop5_lift - min(normalized_sizes_2021_shop5_lift)) / (max(normalized_sizes_2021_shop5_lift) - min(normalized_sizes_2021_shop5_lift))
node_colors_2021_shop5_lift = [cmap_2021_shop5_lift(size) for size in normalized_sizes_2021_shop5_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(60, 32))
nx.draw_networkx_nodes(G_2021_shop5_lift, pos_2021_shop5_lift, node_size=node_sizes_2021_shop5_lift, node_color=node_colors_2021_shop5_lift, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop5_lift, pos_2021_shop5_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2021_shop5_lift, pos_2021_shop5_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2021 Shop 5 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2021_shop5_nonredundant_top30_lift.loc[:, 'antecedents_'] = rules_2021_shop5_nonredundant_top30_lift['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2021_shop5_nonredundant_top30_lift.loc[:, 'consequents_'] = rules_2021_shop5_nonredundant_top30_lift['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2021_shop5_nonredundant_top30_lift, x='consequents_', y='antecedents_', s=rules_2021_shop5_nonredundant_top30_lift['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2021_shop5_nonredundant_top30_lift.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2021 Shop 5", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
# Print only the 'antecedents' and 'consequents' columns of the top 10 rules
print(rules_2021_shop5_nonredundant_top30_lift[['antecedents', 'consequents']].head(10))


# Shop5 2022 SEARCH 10

In [ ]:
# Filter for records from shop ID 1 and the year 1993
sales_data_2022_shop5 = sales_data_shop5[(sales_data_shop5['trx_date'].dt.year == 2022)]

In [ ]:
sales_data_2022_shop5.head()

In [ ]:
# Grouping product codes by trx_id
transactions_2022_shop5 = sales_data_2022_shop5.groupby('trx_id')['product_code'].apply(list).tolist()

In [ ]:
# Encode transaction data
encoder_2022_shop5 = TransactionEncoder()
trans_array_2022_shop5 = encoder_2022_shop5.fit(transactions_2022_shop5).transform(transactions_2022_shop5)
trans_df_2022_shop5 = pd.DataFrame(trans_array_2022_shop5, columns=encoder_2022_shop5.columns_)

In [ ]:
# Find frequent itemsets
frequent_itemsets_2022_shop5 = apriori(trans_df_2022_shop5, min_support=0.001, use_colnames=True)

In [ ]:
print(frequent_itemsets_2022_shop5)

In [ ]:
## Generate the association rules - by confidence
rulesConfidence_2022_shop5 = association_rules(frequent_itemsets_2022_shop5, metric="confidence", min_threshold=0.10)
rulesConfidence_2022_shop5.sort_values(by='confidence', ascending=False, inplace=True)


In [ ]:
# Note: This is a hypothetical example, as I can't directly manipulate or access dataframes not provided within this code cell.

# Replace frozen sets with strings
rulesConfidence_2022_shop5['antecedents_'] = rulesConfidence_2022_shop5['antecedents'].apply(lambda a: ','.join(list(a)))
rulesConfidence_2022_shop5['consequents_'] = rulesConfidence_2022_shop5['consequents'].apply(lambda a: ','.join(list(a)))

# Draw
fig, ax = plt.subplots(figsize=(14,30))
ax = plt.scatter(data=rulesConfidence_2022_shop5, x='consequents_', y='antecedents_', s=rulesConfidence_2022_shop5['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rulesConfidence_2022_shop5.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2020 Shop 1", fontsize=14)
plt.xlabel("Consequents (RHS)", fontsize=12)
plt.ylabel("Antecedents (LHS)", fontsize=12)
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate(rotation=45)
plt.tight_layout()

plt.show()



## Eliminate Redundant Rules

In [ ]:
rulesConfidence_2022_shop5.head()

In [ ]:
rulesConfidence_2022_shop5.info()

In [ ]:
# If 'antecedents' and 'consequents' are not frozensets, convert them
rules_2022_shop5_redundant = rulesConfidence_2022_shop5.copy()
rules_2022_shop5_redundant['antecedents'] = rules_2022_shop5_redundant['antecedents'].apply(lambda x: frozenset(x))
rules_2022_shop5_redundant['consequents'] = rules_2022_shop5_redundant['consequents'].apply(lambda x: frozenset(x))


In [ ]:
# Create a copy of the DataFrame to avoid modifying the original while iterating
rules_2022_shop5_checkredundance = rules_2022_shop5_redundant.copy()

# List to hold non-redundant rules
non_redundant_rules_2022_shop5 = []

for _, rule in rules_2022_shop5_checkredundance.iterrows():
    if not is_redundant(rule, rules_2022_shop5_checkredundance):
        non_redundant_rules_2022_shop5.append(rule)

# Convert the list back to a DataFrame
rules_2022_shop5_nonredundant = pd.DataFrame(non_redundant_rules_2022_shop5).reset_index(drop=True)


In [ ]:
print(rules_2022_shop5_nonredundant.head())
print(f"Original number of rules: {len(rules_2022_shop5_redundant)}")
print(f"Number of non-redundant rules: {len(rules_2022_shop5_nonredundant)}")


In [ ]:
rules_2022_shop5_nonredundant.head()

In [ ]:
rules_2022_shop5_redundant.info()

In [ ]:
rules_2022_shop5_nonredundant.head()

In [ ]:
rules_2022_shop5_nonredundant_backup = rules_2022_shop5_nonredundant.copy()

In [ ]:
rules_2022_shop5_nonredundant_unfiltered = rules_2022_shop5_nonredundant.copy()

In [ ]:
#reset
#rules_2022_shop5_nonredundant = rules_2022_shop5_nonredundant_backup.copy()

# Redundant and Unfiltered

In [ ]:
rules_2022_shop5_redundant.head()

In [ ]:
G_2022_shop5 = nx.DiGraph()

# Iterate through rules for 2022 Shop 5 and add edges to the graph
for _, row in rules_2022_shop5_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as weight
    G_2022_shop5.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2022_shop5 = nx.spring_layout(G_2022_shop5)

# Recalculate node sizes based on indegree or another metric
in_degrees_2022_shop5 = dict(G_2022_shop5.in_degree())
node_sizes_2022_shop5 = [in_degrees_2022_shop5[node] * 100 + 1000 for node in G_2022_shop5.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2022_shop5 = (np.array(node_sizes_2022_shop5) - np.min(node_sizes_2022_shop5)) / (np.max(node_sizes_2022_shop5) - np.min(node_sizes_2022_shop5))

# Define a colormap from orange to red for visual consistency
cmap_2022_shop5 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2022_shop5 = [cmap_2022_shop5(size) for size in normalized_sizes_2022_shop5]

# Drawing the network
plt.figure(figsize=(60, 43))
nx.draw_networkx_nodes(G_2022_shop5, pos_2022_shop5, node_size=node_sizes_2022_shop5, node_color=node_colors_2022_shop5, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop5, pos_2022_shop5, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop5, pos_2022_shop5, font_size=10)

plt.title('Top 5 Association Rules Network for 2022 Shop 5 with Color Gradient')
plt.axis('off')
plt.show()


### Based on LIFT

In [ ]:
G_2022_shop5_lift = nx.DiGraph()

lift_summary_2022_shop5 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2022_shop5_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop5_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop5:
        lift_summary_2022_shop5[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop5[consequents]['count'] += 1
    else:
        lift_summary_2022_shop5[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop5:
    lift_summary_2022_shop5[item]['avg_lift'] = lift_summary_2022_shop5[item]['total_lift'] / lift_summary_2022_shop5[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop5_lift = [lift_summary_2022_shop5[node]['avg_lift'] * 500 if node in lift_summary_2022_shop5 else 1000 for node in G_2022_shop5_lift.nodes()]

pos_2022_shop5_lift = nx.spring_layout(G_2022_shop5_lift)

# Colormap from orange to red
cmap_2022_shop5_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop5_lift = np.array(node_sizes_2022_shop5_lift)
normalized_sizes_2022_shop5_lift = (normalized_sizes_2022_shop5_lift - min(normalized_sizes_2022_shop5_lift)) / (max(normalized_sizes_2022_shop5_lift) - min(normalized_sizes_2022_shop5_lift))
node_colors_2022_shop5_lift = [cmap_2022_shop5_lift(size) for size in normalized_sizes_2022_shop5_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2022_shop5_lift, pos_2022_shop5_lift, node_size=node_sizes_2022_shop5_lift, node_color=node_colors_2022_shop5_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop5_lift, pos_2022_shop5_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop5_lift, pos_2022_shop5_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2022 Shop 5 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


# nonredundant_unfiltered

## Based on Indegree

In [ ]:
G_2022_shop5_nonredundant_unfiltered = nx.DiGraph()

# Iterating through each row to add edges with confidence as an attribute
for _, row in rules_2022_shop5_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop5_nonredundant_unfiltered.add_edge(antecedents, consequents, weight=row['confidence'])

# Defining the layout for network visualization
pos_2022_shop5_nonredundant_unfiltered = nx.spring_layout(G_2022_shop5_nonredundant_unfiltered)

# Calculating node sizes based on the indegree for color mapping
in_degrees_2022_shop5_nonredundant_unfiltered = dict(G_2022_shop5_nonredundant_unfiltered.in_degree())
node_sizes_2022_shop5_nonredundant_unfiltered = [in_degrees_2022_shop5_nonredundant_unfiltered[node] * 100 + 1000 for node in G_2022_shop5_nonredundant_unfiltered.nodes()]

# Normalizing node sizes for color mapping
normalized_sizes_2022_shop5_nonredundant_unfiltered = (np.array(node_sizes_2022_shop5_nonredundant_unfiltered) - min(node_sizes_2022_shop5_nonredundant_unfiltered)) / (max(node_sizes_2022_shop5_nonredundant_unfiltered) - min(node_sizes_2022_shop5_nonredundant_unfiltered))

# Defining a color map from orange to red
cmap_2022_shop5_nonredundant_unfiltered = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])

# Mapping normalized node sizes to colors
node_colors_2022_shop5_nonredundant_unfiltered = [cmap_2022_shop5_nonredundant_unfiltered(size) for size in normalized_sizes_2022_shop5_nonredundant_unfiltered]

# Drawing the network with node sizes and colors
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2022_shop5_nonredundant_unfiltered, pos_2022_shop5_nonredundant_unfiltered, node_size=node_sizes_2022_shop5_nonredundant_unfiltered, node_color=node_colors_2022_shop5_nonredundant_unfiltered, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop5_nonredundant_unfiltered, pos_2022_shop5_nonredundant_unfiltered, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2022_shop5_nonredundant_unfiltered, pos_2022_shop5_nonredundant_unfiltered, font_size=10)

plt.title('Node Size Based on Indegree for 2022 Shop 5')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2022_shop5_lift = nx.DiGraph()

lift_summary_2022_shop5 = {}

for _, row in rules_2022_shop5_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop5_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Update lift summary for consequents
    if consequents in lift_summary_2022_shop5:
        lift_summary_2022_shop5[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop5[consequents]['count'] += 1
    else:
        lift_summary_2022_shop5[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculate average lift for each consequent
for item in lift_summary_2022_shop5:
    lift_summary_2022_shop5[item]['avg_lift'] = lift_summary_2022_shop5[item]['total_lift'] / lift_summary_2022_shop5[item]['count']

# Node sizes based on average lift, scaled appropriately
node_sizes_2022_shop5_lift = [lift_summary_2022_shop5[node]['avg_lift'] * 500 if node in lift_summary_2022_shop5 else 1000 for node in G_2022_shop5_lift.nodes()]

pos_2022_shop5_lift = nx.spring_layout(G_2022_shop5_lift)

# Use a colormap that transitions from orange to red
cmap_2022_shop5_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop5_lift = np.array(node_sizes_2022_shop5_lift)
normalized_sizes_2022_shop5_lift = (normalized_sizes_2022_shop5_lift - min(normalized_sizes_2022_shop5_lift)) / (max(normalized_sizes_2022_shop5_lift) - min(normalized_sizes_2022_shop5_lift))
node_colors_2022_shop5_lift = [cmap_2022_shop5_lift(size) for size in normalized_sizes_2022_shop5_lift]

plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2022_shop5_lift, pos_2022_shop5_lift, node_size=node_sizes_2022_shop5_lift, node_color=node_colors_2022_shop5_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop5_lift, pos_2022_shop5_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop5_lift, pos_2022_shop5_lift, font_size=10)

plt.title('Node Size Based on Average Lift for 2022 Shop 5')
plt.axis('off')
plt.show()


# Based on conviction

In [ ]:
rules_2022_shop5_nonredundant.head()

In [ ]:
G_2022_shop5_conviction = nx.DiGraph()

conviction_summary_2022_shop5 = {}
count_summary_2022_shop5 = {}

# Assuming 'rules_2022_shop5_nonredundant_unfiltered' contains a 'conviction' column
for _, row in rules_2022_shop5_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop5_conviction.add_edge(antecedents, consequents, weight=row['confidence'], conviction=row['conviction'])
    
    # Aggregate conviction values and count occurrences
    if consequents in conviction_summary_2022_shop5:
        conviction_summary_2022_shop5[consequents] += row['conviction']
        count_summary_2022_shop5[consequents] += 1
    else:
        conviction_summary_2022_shop5[consequents] = row['conviction']
        count_summary_2022_shop5[consequents] = 1

# Calculate average conviction values
average_conviction_summary_2022_shop5 = {k: conviction_summary_2022_shop5[k] / count_summary_2022_shop5[k] for k in conviction_summary_2022_shop5}

# Normalize conviction values for sizing and coloring
max_conviction = max(average_conviction_summary_2022_shop5.values())
min_conviction = min(average_conviction_summary_2022_shop5.values())
conviction_range = max_conviction - min_conviction
node_colors = []
node_sizes = []

for node in G_2022_shop5_conviction.nodes():
    # Get the average conviction value for the node, defaulting to min if not present
    conviction = average_conviction_summary_2022_shop5.get(node, min_conviction)
    # Normalize between 0 and 1
    normalized_conviction = (conviction - min_conviction) / conviction_range if conviction_range else 0
    node_colors.append(normalized_conviction)
    node_sizes.append(normalized_conviction * 1000 + 100)  # Scale and offset size

# Use a colormap for coloring nodes based on conviction
cmap = plt.cm.coolwarm

pos = nx.spring_layout(G_2022_shop5_conviction)

plt.figure(figsize=(40, 22))
nodes = nx.draw_networkx_nodes(G_2022_shop5_conviction, pos, node_color=node_colors, node_size=node_sizes, cmap=cmap, alpha=0.8)
edges = nx.draw_networkx_edges(G_2022_shop5_conviction, pos, arrowstyle='->', arrowsize=10, edge_color='grey', alpha=0.5)
labels = nx.draw_networkx_labels(G_2022_shop5_conviction, pos, font_size=8)

plt.colorbar(nodes, label='Normalized Conviction')
plt.title('Network Graph of 2022 Shop 5 Based on Conviction')
plt.axis('off')
plt.show()

In [ ]:
print(max_conviction)

# Filtered top 30 and non redundant

In [ ]:
rules_2022_shop5_nonredundant.head()

In [ ]:
# Assuming rules_1992_shop2_nonredundant is your sorted rules DataFrame for shop2
rules_2022_shop5_nonredundant_top30 = rules_2022_shop5_nonredundant.copy(deep=True)

In [ ]:
rules_2022_shop5_nonredundant_top30_lift = rules_2022_shop5_nonredundant.copy(deep=True)

In [ ]:
# Select the top 30 rules based on lift or another criterion
rules_2022_shop5_nonredundant_top30_lift = rules_2022_shop5_nonredundant_top30_lift.sort_values(by='lift', ascending=False).head(30)

In [ ]:
rules_2022_shop5_nonredundant_top30_lift.head()

In [ ]:
# Select the top 30 rules based on confidence or another criterion
rules_2022_shop5_nonredundant_top30 = rules_2022_shop5_nonredundant_top30.sort_values(by='confidence', ascending=False).head(30)

In [ ]:
rules_2022_shop5_nonredundant_top30.info()

In [ ]:
rules_2022_shop5_nonredundant_top30['LHS'] = [','.join(list(x)) for x in rules_2022_shop5_nonredundant_top30['antecedents']]
rules_2022_shop5_nonredundant_top30['RHS'] = [','.join(list(x)) for x in rules_2022_shop5_nonredundant_top30['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2022_shop5_nonredundant_top30.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2022_shop5_nonredundant_top30, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2022 Shop 5")
plt.axis('off')
plt.show()


## Based on Indegree

In [ ]:
G_2022_shop5 = nx.DiGraph()

# Assuming 'rules_sorted_2022_shop5' contains association rules for shop 5 in 2022
for _, row in rules_2022_shop5_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as the weight
    G_2022_shop5.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2022_shop5 = nx.spring_layout(G_2022_shop5)

# Calculate node sizes based on indegree
in_degrees_2022_shop5 = dict(G_2022_shop5.in_degree())
node_sizes_2022_shop5 = [in_degrees_2022_shop5[node] * 100 + 1000 for node in G_2022_shop5.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2022_shop5 = (np.array(node_sizes_2022_shop5) - np.min(node_sizes_2022_shop5)) / (np.max(node_sizes_2022_shop5) - np.min(node_sizes_2022_shop5))

# Define a colormap from orange to red
cmap_2022_shop5 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2022_shop5 = [cmap_2022_shop5(size) for size in normalized_sizes_2022_shop5]

# Drawing the network
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2022_shop5, pos_2022_shop5, node_size=node_sizes_2022_shop5, node_color=node_colors_2022_shop5, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop5, pos_2022_shop5, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2022_shop5, pos_2022_shop5, font_size=8)

plt.title('Top 5 Association Rules Network for 2022 Shop 5 with Color Gradient')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2022_shop5_lift = nx.DiGraph()

lift_summary_2022_shop5 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2022_shop5_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop5_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop5:
        lift_summary_2022_shop5[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop5[consequents]['count'] += 1
    else:
        lift_summary_2022_shop5[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop5:
    lift_summary_2022_shop5[item]['avg_lift'] = lift_summary_2022_shop5[item]['total_lift'] / lift_summary_2022_shop5[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop5_lift = [lift_summary_2022_shop5[node]['avg_lift'] * 500 if node in lift_summary_2022_shop5 else 1000 for node in G_2022_shop5_lift.nodes()]

pos_2022_shop5_lift = nx.spring_layout(G_2022_shop5_lift)

# Colormap from orange to red
cmap_2022_shop5_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop5_lift = np.array(node_sizes_2022_shop5_lift)
normalized_sizes_2022_shop5_lift = (normalized_sizes_2022_shop5_lift - min(normalized_sizes_2022_shop5_lift)) / (max(normalized_sizes_2022_shop5_lift) - min(normalized_sizes_2022_shop5_lift))
node_colors_2022_shop5_lift = [cmap_2022_shop5_lift(size) for size in normalized_sizes_2022_shop5_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2022_shop5_lift, pos_2022_shop5_lift, node_size=node_sizes_2022_shop5_lift, node_color=node_colors_2022_shop5_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop5_lift, pos_2022_shop5_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop5_lift, pos_2022_shop5_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2022 Shop 5 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop5_lift = nx.DiGraph()

lift_summary_2022_shop5 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2022_shop5_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop5_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop5:
        lift_summary_2022_shop5[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop5[consequents]['count'] += 1
    else:
        lift_summary_2022_shop5[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop5:
    lift_summary_2022_shop5[item]['avg_lift'] = lift_summary_2022_shop5[item]['total_lift'] / lift_summary_2022_shop5[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop5_lift = [lift_summary_2022_shop5[node]['avg_lift'] * 500 if node in lift_summary_2022_shop5 else 1000 for node in G_2022_shop5_lift.nodes()]

pos_2022_shop5_lift = nx.spring_layout(G_2022_shop5_lift)

# Colormap from orange to red
cmap_2022_shop5_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop5_lift = np.array(node_sizes_2022_shop5_lift)
normalized_sizes_2022_shop5_lift = (normalized_sizes_2022_shop5_lift - min(normalized_sizes_2022_shop5_lift)) / (max(normalized_sizes_2022_shop5_lift) - min(normalized_sizes_2022_shop5_lift))
node_colors_2022_shop5_lift = [cmap_2022_shop5_lift(size) for size in normalized_sizes_2022_shop5_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2022_shop5_lift, pos_2022_shop5_lift, node_size=node_sizes_2022_shop5_lift, node_color=node_colors_2022_shop5_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop5_lift, pos_2022_shop5_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop5_lift, pos_2022_shop5_lift, font_size=10)

plt.title('Top 5 Association Rules Network for 2022 Shop 5 Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop5 = nx.DiGraph()

# Assuming 'rules_2022_shop5_nonredundant' contains association rules for 2022 Shop 5
for _, row in rules_2022_shop5_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop5.add_edge(antecedents, consequents, confidence=row['confidence'])

# Calculate node sizes based on indegree
in_degrees_2022_shop5 = dict(G_2022_shop5.in_degree())
node_sizes_2022_shop5 = [in_degrees_2022_shop5[node] * 100 + 1000 for node in G_2022_shop5.nodes()]

# Edge width based on confidence
edge_widths_2022_shop5 = [G_2022_shop5[u][v]['confidence'] * 15 for u, v in G_2022_shop5.edges()]

# Assuming normalized sizes and color mapping is desired
normalized_sizes_2022_shop5 = np.array(list(in_degrees_2022_shop5.values()))
normalized_sizes_2022_shop5 = (normalized_sizes_2022_shop5 - min(normalized_sizes_2022_shop5)) / (max(normalized_sizes_2022_shop5) - min(normalized_sizes_2022_shop5))
cmap_2022_shop5 = mcolors.LinearSegmentedColormap.from_list("MyCMapName", ["blue", "red"])

# Node colors based on a property (here, using indegree for example)
node_colors_2022_shop5 = [cmap_2022_shop5(size) for size in normalized_sizes_2022_shop5]

# Layout
pos_2022_shop5 = nx.spring_layout(G_2022_shop5, seed=42)

# Drawing
plt.figure(figsize=(40, 30))
nx.draw_networkx_edges(G_2022_shop5, pos_2022_shop5, width=edge_widths_2022_shop5, edge_color='grey', alpha=0.5)
nx.draw_networkx_nodes(G_2022_shop5, pos_2022_shop5, node_size=node_sizes_2022_shop5, node_color=node_colors_2022_shop5, alpha=0.8)
nx.draw_networkx_labels(G_2022_shop5, pos_2022_shop5, font_size=8, font_color='black')
plt.title('Network Graph of Association Rules Based on Confidence for 2022 Shop 5')
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop5 = nx.DiGraph()

convictions_2022_shop5 = []  # To store conviction values for normalization

for _, row in rules_2022_shop5_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    conviction_2022_shop5 = row['conviction']  # Assuming each row has a 'conviction' column
    convictions_2022_shop5.append(conviction_2022_shop5)
    G_2022_shop5.add_edge(antecedents, consequents, weight=row['confidence'], conviction=conviction_2022_shop5)

# Normalize conviction values for edge width
max_conviction_2022_shop5 = max(convictions_2022_shop5)
min_conviction_2022_shop5 = min(convictions_2022_shop5)
conviction_range_2022_shop5 = max_conviction_2022_shop5 - min_conviction_2022_shop5
edge_widths_2022_shop5 = [(G_2022_shop5[u][v]['conviction'] - min_conviction_2022_shop5) / conviction_range_2022_shop5 * 30 + 0.5 for u, v in G_2022_shop5.edges()]  # Scale and offset widths

pos_2022_shop5 = nx.spring_layout(G_2022_shop5)

# Node sizes and colors as before
in_degrees_2022_shop5 = dict(G_2022_shop5.in_degree())
node_sizes_2022_shop5 = [in_degrees_2022_shop5[node] * 100 + 1000 for node in G_2022_shop5.nodes()]
normalized_sizes_2022_shop5 = (np.array(node_sizes_2022_shop5) - min(node_sizes_2022_shop5)) / (max(node_sizes_2022_shop5) - min(node_sizes_2022_shop5))
cmap_2022_shop5 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2022_shop5 = [cmap_2022_shop5(size) for size in normalized_sizes_2022_shop5]

# Drawing the network with conviction-based edge widths
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2022_shop5, pos_2022_shop5, node_size=node_sizes_2022_shop5, node_color=node_colors_2022_shop5, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop5, pos_2022_shop5, width=edge_widths_2022_shop5, edge_color='grey', alpha=0.5)
nx.draw_networkx_labels(G_2022_shop5, pos_2022_shop5, font_size=10)

plt.title('Top 5 Association Rules Network for 2022 Shop 5 with Conviction-based Edge Thickness')
plt.axis('off')
plt.show()


In [ ]:
#the node sizes are based on the indegree where each node acts as a consequent,

In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2022_shop5_nonredundant_top30.loc[:, 'antecedents_'] = rules_2022_shop5_nonredundant_top30['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2022_shop5_nonredundant_top30.loc[:, 'consequents_'] = rules_2022_shop5_nonredundant_top30['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2022_shop5_nonredundant_top30, x='consequents_', y='antecedents_', s=rules_2022_shop5_nonredundant_top30['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2022_shop5_nonredundant_top30.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2022 Shop 5", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
print(rules_2022_shop5_nonredundant_top30[['antecedents', 'consequents']].head(10))

## Monetary Analysis of the Top Rules

In [ ]:
# Step 1: Extract unique product codes from the rules
unique_product_codes = set()
for idx, row in rules_2022_shop5_nonredundant_top30.iterrows():
    # Unite antecedents and consequents into a single set
    all_products = row['antecedents'].union(row['consequents'])
    unique_product_codes.update(all_products)

# Step 2: Create a DataFrame with unique product codes and their unit prices
# Filter sales_data for these product codes and drop duplicates to ensure each product code appears once
product_prices = sales_data[sales_data['product_code'].isin(unique_product_codes)][['product_code', 'unit_price']].drop_duplicates()

# Create a dictionary for quick lookup of prices
price_dict = product_prices.set_index('product_code')['unit_price'].to_dict()

# Step 3: Calculate the total price for each rule and collect data for the new DataFrame
data_for_new_df = []
for idx, row in rules_2022_shop5_nonredundant_top30.iterrows():
    # Combine sets of antecedents and consequents
    all_products = row['antecedents'].union(row['consequents'])
    total_price = sum(price_dict.get(prod, 0) for prod in all_products)  # Use get to avoid KeyError if any product code is missing
    data_for_new_df.append({
        'antecedents': row['antecedents'],
        'consequents': row['consequents'],
        'total_unit_price': total_price
    })

# Create a new DataFrame with only antecedents, consequents, and total unit price
rules_and_price_shop5_2022 = pd.DataFrame(data_for_new_df)

# Sort the DataFrame by total_unit_price in descending order
rules_and_price_shop5_2022_sorted = rules_and_price_shop5_2022.sort_values(by='total_unit_price', ascending=False)

# Print the sorted DataFrame to see the specified columns along with their total unit prices
print(rules_and_price_shop5_2022_sorted)


In [ ]:
# Convert 'trx_date' to datetime format if not already done so
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Add a 'year' column if it doesn't exist
sales_data['year'] = sales_data['trx_date'].dt.year

# Step 2: Filter sales_data for shop5 and the year 2022
filtered_sales_data_2022 = sales_data[(sales_data['shopID'] == 5) & (sales_data['year'] == 2022)]

# Step 3: Initialize a list to hold the rule data including total sales value
rule_sales_summaries_2022 = []

# Step 4: Iterate through each rule in the DataFrame
for index, rule in rules_2022_shop5_nonredundant_top30.iterrows():
    all_products = rule['antecedents'].union(rule['consequents'])

    # Filter the sales data for transactions that include all products in the rule
    # This step ensures that we are only considering transactions that contain all specified products
    mask = filtered_sales_data_2022['product_code'].isin(all_products)
    transaction_ids = filtered_sales_data_2022[mask].groupby('trx_id').filter(lambda x: all_products.issubset(set(x['product_code']))).trx_id.unique()
    matched_transactions = filtered_sales_data_2022[filtered_sales_data_2022['trx_id'].isin(transaction_ids)]
    
    # Step 5: Sum the total_value of these transactions
    total_sales_value = matched_transactions['total_value'].sum()

    # Append the result to the list
    rule_sales_summaries_2022.append({
        'antecedents': rule['antecedents'],
        'consequents': rule['consequents'],
        'total_sales_value': total_sales_value
    })

# Step 6: Convert the list to a DataFrame
rules_sales_values_df_2022 = pd.DataFrame(rule_sales_summaries_2022)

# Step 7: Sort by total_sales_value in descending order
sorted_rules_sales_values_df_2022 = rules_sales_values_df_2022.sort_values(by='total_sales_value', ascending=False)

# Step 8: Print the sorted DataFrame
print(sorted_rules_sales_values_df_2022)


## Narrowing down based on Lift

In [ ]:
rules_2022_shop5_nonredundant_top30_lift['LHS'] = [','.join(list(x)) for x in rules_2022_shop5_nonredundant_top30_lift['antecedents']]
rules_2022_shop5_nonredundant_top30_lift['RHS'] = [','.join(list(x)) for x in rules_2022_shop5_nonredundant_top30_lift['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2022_shop5_nonredundant_top30_lift.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2022_shop5_nonredundant_top30_lift, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2022 Shop 5")
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop5_lift = nx.DiGraph()

lift_summary_2022_shop5 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2022_shop5_nonredundant_top30_lift.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop5_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop5:
        lift_summary_2022_shop5[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop5[consequents]['count'] += 1
    else:
        lift_summary_2022_shop5[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop5:
    lift_summary_2022_shop5[item]['avg_lift'] = lift_summary_2022_shop5[item]['total_lift'] / lift_summary_2022_shop5[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop5_lift = [lift_summary_2022_shop5[node]['avg_lift'] * 500 if node in lift_summary_2022_shop5 else 1000 for node in G_2022_shop5_lift.nodes()]

pos_2022_shop5_lift = nx.spring_layout(G_2022_shop5_lift)

# Colormap from orange to red
cmap_2022_shop5_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop5_lift = np.array(node_sizes_2022_shop5_lift)
normalized_sizes_2022_shop5_lift = (normalized_sizes_2022_shop5_lift - min(normalized_sizes_2022_shop5_lift)) / (max(normalized_sizes_2022_shop5_lift) - min(normalized_sizes_2022_shop5_lift))
node_colors_2022_shop5_lift = [cmap_2022_shop5_lift(size) for size in normalized_sizes_2022_shop5_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2022_shop5_lift, pos_2022_shop5_lift, node_size=node_sizes_2022_shop5_lift, node_color=node_colors_2022_shop5_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop5_lift, pos_2022_shop5_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop5_lift, pos_2022_shop5_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2022 Shop 5 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2022_shop5_nonredundant_top30_lift.loc[:, 'antecedents_'] = rules_2022_shop5_nonredundant_top30_lift['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2022_shop5_nonredundant_top30_lift.loc[:, 'consequents_'] = rules_2022_shop5_nonredundant_top30_lift['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2022_shop5_nonredundant_top30_lift, x='consequents_', y='antecedents_', s=rules_2022_shop5_nonredundant_top30_lift['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2022_shop5_nonredundant_top30_lift.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2022 Shop 5", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
# Print only the 'antecedents' and 'consequents' columns of the top 10 rules
print(rules_2022_shop5_nonredundant_top30_lift[['antecedents', 'consequents']].head(10))


# Shop 5 2023 SEARCH 11

In [ ]:
# Filter for records from shop ID 1 and the year 1993
sales_data_2023_shop5 = sales_data_shop5[(sales_data_shop5['trx_date'].dt.year == 2023)]

In [ ]:
sales_data_2023_shop5.head()

In [ ]:
# Grouping product codes by trx_id
transactions_2023_shop5 = sales_data_2023_shop5.groupby('trx_id')['product_code'].apply(list).tolist()

In [ ]:
# Encode transaction data
encoder_2023_shop5 = TransactionEncoder()
trans_array_2023_shop5 = encoder_2023_shop5.fit(transactions_2023_shop5).transform(transactions_2023_shop5)
trans_df_2023_shop5 = pd.DataFrame(trans_array_2023_shop5, columns=encoder_2023_shop5.columns_)

In [ ]:
# Find frequent itemsets
frequent_itemsets_2023_shop5 = apriori(trans_df_2023_shop5, min_support=0.001, use_colnames=True)

In [ ]:
print(frequent_itemsets_2023_shop5)

In [ ]:
## Generate the association rules - by confidence
rulesConfidence_2023_shop5 = association_rules(frequent_itemsets_2023_shop5, metric="confidence", min_threshold=0.10)
rulesConfidence_2023_shop5.sort_values(by='confidence', ascending=False, inplace=True)


In [ ]:
# Note: This is a hypothetical example, as I can't directly manipulate or access dataframes not provided within this code cell.

# Replace frozen sets with strings
rulesConfidence_2023_shop5['antecedents_'] = rulesConfidence_2023_shop5['antecedents'].apply(lambda a: ','.join(list(a)))
rulesConfidence_2023_shop5['consequents_'] = rulesConfidence_2023_shop5['consequents'].apply(lambda a: ','.join(list(a)))

# Draw
fig, ax = plt.subplots(figsize=(14,30))
ax = plt.scatter(data=rulesConfidence_2023_shop5, x='consequents_', y='antecedents_', s=rulesConfidence_2023_shop5['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rulesConfidence_2023_shop5.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2023 Shop 5", fontsize=14)
plt.xlabel("Consequents (RHS)", fontsize=12)
plt.ylabel("Antecedents (LHS)", fontsize=12)
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate(rotation=45)
plt.tight_layout()

plt.show()



## Eliminate Redundant Rules

In [ ]:
rulesConfidence_2023_shop5.head()

In [ ]:
rulesConfidence_2023_shop5.info()

In [ ]:
# If 'antecedents' and 'consequents' are not frozensets, convert them
rules_2023_shop5_redundant = rulesConfidence_2023_shop5.copy()
rules_2023_shop5_redundant['antecedents'] = rules_2023_shop5_redundant['antecedents'].apply(lambda x: frozenset(x))
rules_2023_shop5_redundant['consequents'] = rules_2023_shop5_redundant['consequents'].apply(lambda x: frozenset(x))


In [ ]:
# Create a copy of the DataFrame to avoid modifying the original while iterating
rules_2023_shop5_checkredundance = rules_2023_shop5_redundant.copy()

# List to hold non-redundant rules
non_redundant_rules_2023_shop5 = []

for _, rule in rules_2023_shop5_checkredundance.iterrows():
    if not is_redundant(rule, rules_2023_shop5_checkredundance):
        non_redundant_rules_2023_shop5.append(rule)

# Convert the list back to a DataFrame
rules_2023_shop5_nonredundant = pd.DataFrame(non_redundant_rules_2023_shop5).reset_index(drop=True)


In [ ]:
print(rules_2023_shop5_nonredundant.head())
print(f"Original number of rules: {len(rules_2023_shop5_redundant)}")
print(f"Number of non-redundant rules: {len(rules_2023_shop5_nonredundant)}")


In [ ]:
rules_2023_shop5_nonredundant.head()

rules_2023_shop1_redundant.info()

In [ ]:
rules_2023_shop5_nonredundant.head()

In [ ]:
rules_2023_shop5_nonredundant_backup = rules_2023_shop5_nonredundant.copy()

In [ ]:
rules_2023_shop5_nonredundant_unfiltered = rules_2023_shop5_nonredundant.copy()

In [ ]:
#reset
#rules_2023_shop5_nonredundant = rules_2023_shop5_nonredundant_backup.copy()

# Redundant and Unfiltered

In [ ]:
rules_2023_shop5_redundant.head()

In [ ]:
G_2023_shop5 = nx.DiGraph()

# Iterate through rules for 2023 Shop 5 and add edges to the graph
for _, row in rules_2023_shop5_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as weight
    G_2023_shop5.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2023_shop5 = nx.spring_layout(G_2023_shop5)

# Recalculate node sizes based on indegree or another metric
in_degrees_2023_shop5 = dict(G_2023_shop5.in_degree())
node_sizes_2023_shop5 = [in_degrees_2023_shop5[node] * 100 + 1000 for node in G_2023_shop5.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2023_shop5 = (np.array(node_sizes_2023_shop5) - np.min(node_sizes_2023_shop5)) / (np.max(node_sizes_2023_shop5) - np.min(node_sizes_2023_shop5))

# Define a colormap from orange to red for visual consistency
cmap_2023_shop5 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop5 = [cmap_2023_shop5(size) for size in normalized_sizes_2023_shop5]

# Drawing the network
plt.figure(figsize=(60, 43))
nx.draw_networkx_nodes(G_2023_shop5, pos_2023_shop5, node_size=node_sizes_2023_shop5, node_color=node_colors_2023_shop5, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop5, pos_2023_shop5, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop5, pos_2023_shop5, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 5 with Color Gradient')
plt.axis('off')
plt.show()


### Based on LIFT

In [ ]:
G_2023_shop5_lift = nx.DiGraph()

lift_summary_2023_shop5 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop5_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop5_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop5:
        lift_summary_2023_shop5[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop5[consequents]['count'] += 1
    else:
        lift_summary_2023_shop5[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop5:
    lift_summary_2023_shop5[item]['avg_lift'] = lift_summary_2023_shop5[item]['total_lift'] / lift_summary_2023_shop5[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop5_lift = [lift_summary_2023_shop5[node]['avg_lift'] * 500 if node in lift_summary_2023_shop5 else 1000 for node in G_2023_shop5_lift.nodes()]

pos_2023_shop5_lift = nx.spring_layout(G_2023_shop5_lift)

# Colormap from orange to red
cmap_2023_shop5_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop5_lift = np.array(node_sizes_2023_shop5_lift)
normalized_sizes_2023_shop5_lift = (normalized_sizes_2023_shop5_lift - min(normalized_sizes_2023_shop5_lift)) / (max(normalized_sizes_2023_shop5_lift) - min(normalized_sizes_2023_shop5_lift))
node_colors_2023_shop5_lift = [cmap_2023_shop5_lift(size) for size in normalized_sizes_2023_shop5_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2023_shop5_lift, pos_2023_shop5_lift, node_size=node_sizes_2023_shop5_lift, node_color=node_colors_2023_shop5_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop5_lift, pos_2023_shop5_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop5_lift, pos_2023_shop5_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 5 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


# nonredundant_unfiltered

## Based on Indegree

In [ ]:
G_2023_shop5_nonredundant_unfiltered = nx.DiGraph()

# Iterating through each row to add edges with confidence as an attribute
for _, row in rules_2023_shop5_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop5_nonredundant_unfiltered.add_edge(antecedents, consequents, weight=row['confidence'])

# Defining the layout for network visualization
pos_2023_shop5_nonredundant_unfiltered = nx.spring_layout(G_2023_shop5_nonredundant_unfiltered)

# Calculating node sizes based on the indegree for color mapping
in_degrees_2023_shop5_nonredundant_unfiltered = dict(G_2023_shop5_nonredundant_unfiltered.in_degree())
node_sizes_2023_shop5_nonredundant_unfiltered = [in_degrees_2023_shop5_nonredundant_unfiltered[node] * 100 + 1000 for node in G_2023_shop5_nonredundant_unfiltered.nodes()]

# Normalizing node sizes for color mapping
normalized_sizes_2023_shop5_nonredundant_unfiltered = (np.array(node_sizes_2023_shop5_nonredundant_unfiltered) - min(node_sizes_2023_shop5_nonredundant_unfiltered)) / (max(node_sizes_2023_shop5_nonredundant_unfiltered) - min(node_sizes_2023_shop5_nonredundant_unfiltered))

# Defining a color map from orange to red
cmap_2023_shop5_nonredundant_unfiltered = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])

# Mapping normalized node sizes to colors
node_colors_2023_shop5_nonredundant_unfiltered = [cmap_2023_shop5_nonredundant_unfiltered(size) for size in normalized_sizes_2023_shop5_nonredundant_unfiltered]

# Drawing the network with node sizes and colors
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2023_shop5_nonredundant_unfiltered, pos_2023_shop5_nonredundant_unfiltered, node_size=node_sizes_2023_shop5_nonredundant_unfiltered, node_color=node_colors_2023_shop5_nonredundant_unfiltered, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop5_nonredundant_unfiltered, pos_2023_shop5_nonredundant_unfiltered, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2023_shop5_nonredundant_unfiltered, pos_2023_shop5_nonredundant_unfiltered, font_size=10)

plt.title('Node Size Based on Indegree for 2023 Shop 5')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2023_shop5_lift = nx.DiGraph()

lift_summary_2023_shop5 = {}

for _, row in rules_2023_shop5_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop5_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Update lift summary for consequents
    if consequents in lift_summary_2023_shop5:
        lift_summary_2023_shop5[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop5[consequents]['count'] += 1
    else:
        lift_summary_2023_shop5[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculate average lift for each consequent
for item in lift_summary_2023_shop5:
    lift_summary_2023_shop5[item]['avg_lift'] = lift_summary_2023_shop5[item]['total_lift'] / lift_summary_2023_shop5[item]['count']

# Node sizes based on average lift, scaled appropriately
node_sizes_2023_shop5_lift = [lift_summary_2023_shop5[node]['avg_lift'] * 500 if node in lift_summary_2023_shop5 else 1000 for node in G_2023_shop5_lift.nodes()]

pos_2023_shop5_lift = nx.spring_layout(G_2023_shop5_lift)

# Use a colormap that transitions from orange to red
cmap_2023_shop5_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop5_lift = np.array(node_sizes_2023_shop5_lift)
normalized_sizes_2023_shop5_lift = (normalized_sizes_2023_shop5_lift - min(normalized_sizes_2023_shop5_lift)) / (max(normalized_sizes_2023_shop5_lift) - min(normalized_sizes_2023_shop5_lift))
node_colors_2023_shop5_lift = [cmap_2023_shop5_lift(size) for size in normalized_sizes_2023_shop5_lift]

plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2023_shop5_lift, pos_2023_shop5_lift, node_size=node_sizes_2023_shop5_lift, node_color=node_colors_2023_shop5_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop5_lift, pos_2023_shop5_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop5_lift, pos_2023_shop5_lift, font_size=10)

plt.title('Node Size Based on Average Lift for 2023 Shop 5')
plt.axis('off')
plt.show()


# Based on conviction

In [ ]:
rules_2023_shop5_nonredundant.head()

In [ ]:
G_2023_shop5_conviction = nx.DiGraph()

conviction_summary_2023_shop5 = {}
count_summary_2023_shop5 = {}

# Assuming 'rules_2023_shop5_nonredundant_unfiltered' contains a 'conviction' column
for _, row in rules_2023_shop5_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop5_conviction.add_edge(antecedents, consequents, weight=row['confidence'], conviction=row['conviction'])
    
    # Aggregate conviction values and count occurrences
    if consequents in conviction_summary_2023_shop5:
        conviction_summary_2023_shop5[consequents] += row['conviction']
        count_summary_2023_shop5[consequents] += 1
    else:
        conviction_summary_2023_shop5[consequents] = row['conviction']
        count_summary_2023_shop5[consequents] = 1

# Calculate average conviction values
average_conviction_summary_2023_shop5 = {k: conviction_summary_2023_shop5[k] / count_summary_2023_shop5[k] for k in conviction_summary_2023_shop5}

# Normalize conviction values for sizing and coloring
max_conviction = max(average_conviction_summary_2023_shop5.values())
min_conviction = min(average_conviction_summary_2023_shop5.values())
conviction_range = max_conviction - min_conviction
node_colors = []
node_sizes = []

for node in G_2023_shop5_conviction.nodes():
    # Get the average conviction value for the node, defaulting to min if not present
    conviction = average_conviction_summary_2023_shop5.get(node, min_conviction)
    # Normalize between 0 and 1
    normalized_conviction = (conviction - min_conviction) / conviction_range if conviction_range else 0
    node_colors.append(normalized_conviction)
    node_sizes.append(normalized_conviction * 1000 + 100)  # Scale and offset size

# Use a colormap for coloring nodes based on conviction
cmap = plt.cm.coolwarm

pos = nx.spring_layout(G_2023_shop5_conviction)

plt.figure(figsize=(40, 22))
nodes = nx.draw_networkx_nodes(G_2023_shop5_conviction, pos, node_color=node_colors, node_size=node_sizes, cmap=cmap, alpha=0.8)
edges = nx.draw_networkx_edges(G_2023_shop5_conviction, pos, arrowstyle='->', arrowsize=10, edge_color='grey', alpha=0.5)
labels = nx.draw_networkx_labels(G_2023_shop5_conviction, pos, font_size=8)

plt.colorbar(nodes, label='Normalized Conviction')
plt.title('Network Graph of 2023 Shop 5 Based on Conviction')
plt.axis('off')
plt.show()

In [ ]:
print(max_conviction)

# Filtered top 30 and non redundant

In [ ]:
rules_2023_shop5_nonredundant.head()

In [ ]:
# Assuming rules_1992_shop2_nonredundant is your sorted rules DataFrame for shop2
rules_2023_shop5_nonredundant_top30 = rules_2023_shop5_nonredundant.copy(deep=True)

In [ ]:
rules_2023_shop5_nonredundant_top30_lift = rules_2023_shop5_nonredundant.copy(deep=True)

In [ ]:
# Select the top 30 rules based on lift or another criterion
rules_2023_shop5_nonredundant_top30_lift = rules_2023_shop5_nonredundant_top30_lift.sort_values(by='lift', ascending=False).head(30)

In [ ]:
rules_2023_shop5_nonredundant_top30_lift.head()

In [ ]:
# Select the top 30 rules based on confidence or another criterion
rules_2023_shop5_nonredundant_top30 = rules_2023_shop5_nonredundant_top30.sort_values(by='confidence', ascending=False).head(30)

In [ ]:
rules_2023_shop5_nonredundant_top30.info()

In [ ]:
rules_2023_shop5_nonredundant_top30['LHS'] = [','.join(list(x)) for x in rules_2023_shop5_nonredundant_top30['antecedents']]
rules_2023_shop5_nonredundant_top30['RHS'] = [','.join(list(x)) for x in rules_2023_shop5_nonredundant_top30['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2023_shop5_nonredundant_top30.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2023_shop5_nonredundant_top30, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2023 Shop 5")
plt.axis('off')
plt.show()


## Based on Indegree

In [ ]:
G_2023_shop5 = nx.DiGraph()

# Assuming 'rules_sorted_2023_shop5' contains association rules for shop 5 in 2023
for _, row in rules_2023_shop5_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as the weight
    G_2023_shop5.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2023_shop5 = nx.spring_layout(G_2023_shop5)

# Calculate node sizes based on indegree
in_degrees_2023_shop5 = dict(G_2023_shop5.in_degree())
node_sizes_2023_shop5 = [in_degrees_2023_shop5[node] * 100 + 1000 for node in G_2023_shop5.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2023_shop5 = (np.array(node_sizes_2023_shop5) - np.min(node_sizes_2023_shop5)) / (np.max(node_sizes_2023_shop5) - np.min(node_sizes_2023_shop5))

# Define a colormap from orange to red
cmap_2023_shop5 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop5 = [cmap_2023_shop5(size) for size in normalized_sizes_2023_shop5]

# Drawing the network
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2023_shop5, pos_2023_shop5, node_size=node_sizes_2023_shop5, node_color=node_colors_2023_shop5, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop5, pos_2023_shop5, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2023_shop5, pos_2023_shop5, font_size=8)

plt.title('Top 5 Association Rules Network for 2023 Shop 5 with Color Gradient')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2023_shop5_lift = nx.DiGraph()

lift_summary_2023_shop5 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop5_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop5_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop5:
        lift_summary_2023_shop5[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop5[consequents]['count'] += 1
    else:
        lift_summary_2023_shop5[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop5:
    lift_summary_2023_shop5[item]['avg_lift'] = lift_summary_2023_shop5[item]['total_lift'] / lift_summary_2023_shop5[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop5_lift = [lift_summary_2023_shop5[node]['avg_lift'] * 500 if node in lift_summary_2023_shop5 else 1000 for node in G_2023_shop5_lift.nodes()]

pos_2023_shop5_lift = nx.spring_layout(G_2023_shop5_lift)

# Colormap from orange to red
cmap_2023_shop5_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop5_lift = np.array(node_sizes_2023_shop5_lift)
normalized_sizes_2023_shop5_lift = (normalized_sizes_2023_shop5_lift - min(normalized_sizes_2023_shop5_lift)) / (max(normalized_sizes_2023_shop5_lift) - min(normalized_sizes_2023_shop5_lift))
node_colors_2023_shop5_lift = [cmap_2023_shop5_lift(size) for size in normalized_sizes_2023_shop5_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2023_shop5_lift, pos_2023_shop5_lift, node_size=node_sizes_2023_shop5_lift, node_color=node_colors_2023_shop5_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop5_lift, pos_2023_shop5_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop5_lift, pos_2023_shop5_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 5 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop5_lift = nx.DiGraph()

lift_summary_2023_shop5 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop5_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop5_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop5:
        lift_summary_2023_shop5[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop5[consequents]['count'] += 1
    else:
        lift_summary_2023_shop5[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop5:
    lift_summary_2023_shop5[item]['avg_lift'] = lift_summary_2023_shop5[item]['total_lift'] / lift_summary_2023_shop5[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop5_lift = [lift_summary_2023_shop5[node]['avg_lift'] * 500 if node in lift_summary_2023_shop5 else 1000 for node in G_2023_shop5_lift.nodes()]

pos_2023_shop5_lift = nx.spring_layout(G_2023_shop5_lift)

# Colormap from orange to red
cmap_2023_shop5_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop5_lift = np.array(node_sizes_2023_shop5_lift)
normalized_sizes_2023_shop5_lift = (normalized_sizes_2023_shop5_lift - min(normalized_sizes_2023_shop5_lift)) / (max(normalized_sizes_2023_shop5_lift) - min(normalized_sizes_2023_shop5_lift))
node_colors_2023_shop5_lift = [cmap_2023_shop5_lift(size) for size in normalized_sizes_2023_shop5_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2023_shop5_lift, pos_2023_shop5_lift, node_size=node_sizes_2023_shop5_lift, node_color=node_colors_2023_shop5_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop5_lift, pos_2023_shop5_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop5_lift, pos_2023_shop5_lift, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 5 Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop5 = nx.DiGraph()

# Assuming 'rules_2023_shop5_nonredundant' contains association rules for 2023 Shop 5
for _, row in rules_2023_shop5_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop5.add_edge(antecedents, consequents, confidence=row['confidence'])

# Calculate node sizes based on indegree
in_degrees_2023_shop5 = dict(G_2023_shop5.in_degree())
node_sizes_2023_shop5 = [in_degrees_2023_shop5[node] * 100 + 1000 for node in G_2023_shop5.nodes()]

# Edge width based on confidence
edge_widths_2023_shop5 = [G_2023_shop5[u][v]['confidence'] * 15 for u, v in G_2023_shop5.edges()]

# Assuming normalized sizes and color mapping is desired
normalized_sizes_2023_shop5 = np.array(list(in_degrees_2023_shop5.values()))
normalized_sizes_2023_shop5 = (normalized_sizes_2023_shop5 - min(normalized_sizes_2023_shop5)) / (max(normalized_sizes_2023_shop5) - min(normalized_sizes_2023_shop5))
cmap_2023_shop5 = mcolors.LinearSegmentedColormap.from_list("MyCMapName", ["blue", "red"])

# Node colors based on a property (here, using indegree for example)
node_colors_2023_shop5 = [cmap_2023_shop5(size) for size in normalized_sizes_2023_shop5]

# Layout
pos_2023_shop5 = nx.spring_layout(G_2023_shop5, seed=42)

# Drawing
plt.figure(figsize=(40, 30))
nx.draw_networkx_edges(G_2023_shop5, pos_2023_shop5, width=edge_widths_2023_shop5, edge_color='grey', alpha=0.5)
nx.draw_networkx_nodes(G_2023_shop5, pos_2023_shop5, node_size=node_sizes_2023_shop5, node_color=node_colors_2023_shop5, alpha=0.8)
nx.draw_networkx_labels(G_2023_shop5, pos_2023_shop5, font_size=8, font_color='black')
plt.title('Network Graph of Association Rules Based on Confidence for 2023 Shop 5')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop5 = nx.DiGraph()

convictions_2023_shop5 = []  # To store conviction values for normalization

for _, row in rules_2023_shop5_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    conviction = row['conviction']  # Assuming each row has a 'conviction' column
    convictions_2023_shop5.append(conviction)
    G_2023_shop5.add_edge(antecedents, consequents, weight=row['confidence'], conviction=conviction)

# Normalize conviction values for edge width
max_conviction_2023_shop5 = max(convictions_2023_shop5)
min_conviction_2023_shop5 = min(convictions_2023_shop5)
conviction_range_2023_shop5 = max_conviction_2023_shop5 - min_conviction_2023_shop5
edge_widths_2023_shop5 = [(G_2023_shop5[u][v]['conviction'] - min_conviction_2023_shop5) / conviction_range_2023_shop5 * 30 + 0.5 for u, v in G_2023_shop5.edges()]  # Scale and offset widths

pos_2023_shop5 = nx.spring_layout(G_2023_shop5)

# Node sizes and colors as before
in_degrees_2023_shop5 = dict(G_2023_shop5.in_degree())
node_sizes_2023_shop5 = [in_degrees_2023_shop5[node] * 100 + 1000 for node in G_2023_shop5.nodes()]
normalized_sizes_2023_shop5 = (np.array(node_sizes_2023_shop5) - min(node_sizes_2023_shop5)) / (max(node_sizes_2023_shop5) - min(node_sizes_2023_shop5))
cmap_2023_shop5 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop5 = [cmap_2023_shop5(size) for size in normalized_sizes_2023_shop5]

# Drawing the network with conviction-based edge widths
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2023_shop5, pos_2023_shop5, node_size=node_sizes_2023_shop5, node_color=node_colors_2023_shop5, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop5, pos_2023_shop5, width=edge_widths_2023_shop5, edge_color='grey', alpha=0.5)
nx.draw_networkx_labels(G_2023_shop5, pos_2023_shop5, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 5 with Conviction-based Edge Thickness')
plt.axis('off')
plt.show()


In [ ]:
#the node sizes are based on the indegree where each node acts as a consequent,

In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2023_shop5_nonredundant_top30.loc[:, 'antecedents_'] = rules_2023_shop5_nonredundant_top30['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2023_shop5_nonredundant_top30.loc[:, 'consequents_'] = rules_2023_shop5_nonredundant_top30['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2023_shop5_nonredundant_top30, x='consequents_', y='antecedents_', s=rules_2023_shop5_nonredundant_top30['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2023_shop5_nonredundant_top30.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2023 Shop 5", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
print(rules_2023_shop5_nonredundant_top30[['antecedents', 'consequents']].head(10))

## Monetary Analysis of the Top Rules

In [ ]:
# Step 1: Extract unique product codes from the rules
unique_product_codes = set()
for idx, row in rules_2023_shop5_nonredundant_top30.iterrows():
    # Unite antecedents and consequents into a single set
    all_products = row['antecedents'].union(row['consequents'])
    unique_product_codes.update(all_products)

# Step 2: Create a DataFrame with unique product codes and their unit prices
# Filter sales_data for these product codes and drop duplicates to ensure each product code appears once
product_prices = sales_data[sales_data['product_code'].isin(unique_product_codes)][['product_code', 'unit_price']].drop_duplicates()

# Create a dictionary for quick lookup of prices
price_dict = product_prices.set_index('product_code')['unit_price'].to_dict()

# Step 3: Calculate the total price for each rule and collect data for the new DataFrame
data_for_new_df = []
for idx, row in rules_2023_shop5_nonredundant_top30.iterrows():
    # Combine sets of antecedents and consequents
    all_products = row['antecedents'].union(row['consequents'])
    total_price = sum(price_dict.get(prod, 0) for prod in all_products)  # Use get to avoid KeyError if any product code is missing
    data_for_new_df.append({
        'antecedents': row['antecedents'],
        'consequents': row['consequents'],
        'total_unit_price': total_price
    })

# Create a new DataFrame with only antecedents, consequents, and total unit price
rules_and_price_shop5_2023 = pd.DataFrame(data_for_new_df)

# Sort the DataFrame by total_unit_price in descending order
rules_and_price_shop5_2023_sorted = rules_and_price_shop5_2023.sort_values(by='total_unit_price', ascending=False)

# Print the sorted DataFrame to see the specified columns along with their total unit prices
print(rules_and_price_shop5_2023_sorted)


In [ ]:
# Convert 'trx_date' to datetime format if not already done so
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Add a 'year' column if it doesn't exist
sales_data['year'] = sales_data['trx_date'].dt.year

# Step 2: Filter sales_data for shop5 and the year 2023
filtered_sales_data_2023 = sales_data[(sales_data['shopID'] == 5) & (sales_data['year'] == 2023)]

# Step 3: Initialize a list to hold the rule data including total sales value
rule_sales_summaries_2023 = []

# Step 4: Iterate through each rule in the DataFrame
for index, rule in rules_2023_shop5_nonredundant_top30.iterrows():
    all_products = rule['antecedents'].union(rule['consequents'])

    # Filter the sales data for transactions that include all products in the rule
    # This step ensures that we are only considering transactions that contain all specified products
    mask = filtered_sales_data_2023['product_code'].isin(all_products)
    transaction_ids = filtered_sales_data_2023[mask].groupby('trx_id').filter(lambda x: all_products.issubset(set(x['product_code']))).trx_id.unique()
    matched_transactions = filtered_sales_data_2023[filtered_sales_data_2023['trx_id'].isin(transaction_ids)]
    
    # Step 5: Sum the total_value of these transactions
    total_sales_value = matched_transactions['total_value'].sum()

    # Append the result to the list
    rule_sales_summaries_2023.append({
        'antecedents': rule['antecedents'],
        'consequents': rule['consequents'],
        'total_sales_value': total_sales_value
    })

# Step 6: Convert the list to a DataFrame
rules_sales_values_df_2023 = pd.DataFrame(rule_sales_summaries_2023)

# Step 7: Sort by total_sales_value in descending order
sorted_rules_sales_values_df_2023 = rules_sales_values_df_2023.sort_values(by='total_sales_value', ascending=False)

# Step 8: Print the sorted DataFrame
print(sorted_rules_sales_values_df_2023)


## Narrowing down based on Lift

In [ ]:
rules_2023_shop5_nonredundant_top30_lift['LHS'] = [','.join(list(x)) for x in rules_2023_shop5_nonredundant_top30_lift['antecedents']]
rules_2023_shop5_nonredundant_top30_lift['RHS'] = [','.join(list(x)) for x in rules_2023_shop5_nonredundant_top30_lift['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2023_shop5_nonredundant_top30_lift.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2023_shop5_nonredundant_top30_lift, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2023 Shop 5")
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop5_lift = nx.DiGraph()

lift_summary_2023_shop5 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop5_nonredundant_top30_lift.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop5_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop5:
        lift_summary_2023_shop5[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop5[consequents]['count'] += 1
    else:
        lift_summary_2023_shop5[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop5:
    lift_summary_2023_shop5[item]['avg_lift'] = lift_summary_2023_shop5[item]['total_lift'] / lift_summary_2023_shop5[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop5_lift = [lift_summary_2023_shop5[node]['avg_lift'] * 500 if node in lift_summary_2023_shop5 else 1000 for node in G_2023_shop5_lift.nodes()]

pos_2023_shop5_lift = nx.spring_layout(G_2023_shop5_lift)

# Colormap from orange to red
cmap_2023_shop5_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop5_lift = np.array(node_sizes_2023_shop5_lift)
normalized_sizes_2023_shop5_lift = (normalized_sizes_2023_shop5_lift - min(normalized_sizes_2023_shop5_lift)) / (max(normalized_sizes_2023_shop5_lift) - min(normalized_sizes_2023_shop5_lift))
node_colors_2023_shop5_lift = [cmap_2023_shop5_lift(size) for size in normalized_sizes_2023_shop5_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2023_shop5_lift, pos_2023_shop5_lift, node_size=node_sizes_2023_shop5_lift, node_color=node_colors_2023_shop5_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop5_lift, pos_2023_shop5_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop5_lift, pos_2023_shop5_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 5 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2023_shop5_nonredundant_top30_lift.loc[:, 'antecedents_'] = rules_2023_shop5_nonredundant_top30_lift['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2023_shop5_nonredundant_top30_lift.loc[:, 'consequents_'] = rules_2023_shop5_nonredundant_top30_lift['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2023_shop5_nonredundant_top30_lift, x='consequents_', y='antecedents_', s=rules_2023_shop5_nonredundant_top30_lift['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2023_shop5_nonredundant_top30_lift.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2023 Shop 5", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
# Print only the 'antecedents' and 'consequents' columns of the top 10 rules
print(rules_2023_shop5_nonredundant_top30_lift[['antecedents', 'consequents']].head(10))


# Shop6 2022 SEARCH shop6_1

In [ ]:
sales_data_2022_shop6 = sales_data_shop6[(sales_data_shop6['trx_date'].dt.year == 2022)]

In [ ]:
sales_data_2022_shop6.head()

In [ ]:
# Grouping product codes by trx_id
transactions_2022_shop6 = sales_data_2022_shop6.groupby('trx_id')['product_code'].apply(list).tolist()

In [ ]:
# Encode transaction data
encoder_2022_shop6 = TransactionEncoder()
trans_array_2022_shop6 = encoder_2022_shop6.fit(transactions_2022_shop6).transform(transactions_2022_shop6)
trans_df_2022_shop6 = pd.DataFrame(trans_array_2022_shop6, columns=encoder_2022_shop6.columns_)

In [ ]:
# Find frequent itemsets
frequent_itemsets_2022_shop6 = apriori(trans_df_2022_shop6, min_support=0.001, use_colnames=True)

In [ ]:
print(frequent_itemsets_2022_shop6)

In [ ]:
## Generate the association rules - by confidence
rulesConfidence_2022_shop6 = association_rules(frequent_itemsets_2022_shop6, metric="confidence", min_threshold=0.10)
rulesConfidence_2022_shop6.sort_values(by='confidence', ascending=False, inplace=True)


In [ ]:
# Replace frozen sets with strings
rulesConfidence_2022_shop6['antecedents_'] = rulesConfidence_2022_shop6['antecedents'].apply(lambda a: ','.join(list(a)))
rulesConfidence_2022_shop6['consequents_'] = rulesConfidence_2022_shop6['consequents'].apply(lambda a: ','.join(list(a)))

# Draw
fig, ax = plt.subplots(figsize=(14,30))
ax = plt.scatter(data=rulesConfidence_2022_shop6, x='consequents_', y='antecedents_', s=rulesConfidence_2022_shop6['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rulesConfidence_2022_shop6.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2020 Shop 6", fontsize=14)
plt.xlabel("Consequents (RHS)", fontsize=12)
plt.ylabel("Antecedents (LHS)", fontsize=12)
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate(rotation=45)
plt.tight_layout()

plt.show()


## Eliminate Redundant Rules

In [ ]:
rulesConfidence_2022_shop6.head()

In [ ]:
rulesConfidence_2022_shop6.info()

In [ ]:
# If 'antecedents' and 'consequents' are not frozensets, convert them
rules_2022_shop6_redundant = rulesConfidence_2022_shop6.copy()
rules_2022_shop6_redundant['antecedents'] = rules_2022_shop6_redundant['antecedents'].apply(lambda x: frozenset(x))
rules_2022_shop6_redundant['consequents'] = rules_2022_shop6_redundant['consequents'].apply(lambda x: frozenset(x))


In [ ]:
# Create a copy of the DataFrame to avoid modifying the original while iterating
rules_2022_shop6_checkredundance = rules_2022_shop6_redundant.copy()

# List to hold non-redundant rules
non_redundant_rules_2022_shop6 = []

for _, rule in rules_2022_shop6_checkredundance.iterrows():
    if not is_redundant(rule, rules_2022_shop6_checkredundance):
        non_redundant_rules_2022_shop6.append(rule)

# Convert the list back to a DataFrame
rules_2022_shop6_nonredundant = pd.DataFrame(non_redundant_rules_2022_shop6).reset_index(drop=True)


In [ ]:
print(rules_2022_shop6_nonredundant.head())
print(f"Original number of rules: {len(rules_2022_shop6_redundant)}")
print(f"Number of non-redundant rules: {len(rules_2022_shop6_nonredundant)}")


In [ ]:
rules_2022_shop6_nonredundant.head()

In [ ]:
rules_2022_shop6_redundant.info()

In [ ]:
rules_2022_shop6_nonredundant.head()

In [ ]:
rules_2022_shop6_nonredundant_backup = rules_2022_shop6_nonredundant.copy()

In [ ]:
rules_2022_shop6_nonredundant_unfiltered = rules_2022_shop6_nonredundant.copy()

# Redundant and Unfiltered

In [ ]:
rules_2022_shop6_redundant.head()

In [ ]:
G_2022_shop6 = nx.DiGraph()

# Iterate through rules for 2022 Shop 6 and add edges to the graph
for _, row in rules_2022_shop6_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as weight
    G_2022_shop6.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2022_shop6 = nx.spring_layout(G_2022_shop6)

# Recalculate node sizes based on indegree or another metric
in_degrees_2022_shop6 = dict(G_2022_shop6.in_degree())
node_sizes_2022_shop6 = [in_degrees_2022_shop6[node] * 100 + 1000 for node in G_2022_shop6.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2022_shop6 = (np.array(node_sizes_2022_shop6) - np.min(node_sizes_2022_shop6)) / (np.max(node_sizes_2022_shop6) - np.min(node_sizes_2022_shop6))

# Define a colormap from orange to red for visual consistency
cmap_2022_shop6 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2022_shop6 = [cmap_2022_shop6(size) for size in normalized_sizes_2022_shop6]

# Drawing the network
plt.figure(figsize=(60, 43))
nx.draw_networkx_nodes(G_2022_shop6, pos_2022_shop6, node_size=node_sizes_2022_shop6, node_color=node_colors_2022_shop6, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop6, pos_2022_shop6, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop6, pos_2022_shop6, font_size=10)

plt.title('Top 5 Association Rules Network for 2022 Shop 6 with Color Gradient')
plt.axis('off')
plt.show()


### Based on LIFT

In [ ]:
G_2022_shop6_lift = nx.DiGraph()

lift_summary_2022_shop6 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2022_shop6_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop6_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop6:
        lift_summary_2022_shop6[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop6[consequents]['count'] += 1
    else:
        lift_summary_2022_shop6[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop6:
    lift_summary_2022_shop6[item]['avg_lift'] = lift_summary_2022_shop6[item]['total_lift'] / lift_summary_2022_shop6[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop6_lift = [lift_summary_2022_shop6[node]['avg_lift'] * 500 if node in lift_summary_2022_shop6 else 1000 for node in G_2022_shop6_lift.nodes()]

pos_2022_shop6_lift = nx.spring_layout(G_2022_shop6_lift)

# Colormap from orange to red
cmap_2022_shop6_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop6_lift = np.array(node_sizes_2022_shop6_lift)
normalized_sizes_2022_shop6_lift = (normalized_sizes_2022_shop6_lift - min(normalized_sizes_2022_shop6_lift)) / (max(normalized_sizes_2022_shop6_lift) - min(normalized_sizes_2022_shop6_lift))
node_colors_2022_shop6_lift = [cmap_2022_shop6_lift(size) for size in normalized_sizes_2022_shop6_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2022_shop6_lift, pos_2022_shop6_lift, node_size=node_sizes_2022_shop6_lift, node_color=node_colors_2022_shop6_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop6_lift, pos_2022_shop6_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop6_lift, pos_2022_shop6_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2022 Shop 6 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


# nonredundant_unfiltered

## Based on Indegree

In [ ]:
G_2022_shop6_nonredundant_unfiltered = nx.DiGraph()

# Iterating through each row to add edges with confidence as an attribute
for _, row in rules_2022_shop6_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop6_nonredundant_unfiltered.add_edge(antecedents, consequents, weight=row['confidence'])

# Defining the layout for network visualization
pos_2022_shop6_nonredundant_unfiltered = nx.spring_layout(G_2022_shop6_nonredundant_unfiltered)

# Calculating node sizes based on the indegree for color mapping
in_degrees_2022_shop6_nonredundant_unfiltered = dict(G_2022_shop6_nonredundant_unfiltered.in_degree())
node_sizes_2022_shop6_nonredundant_unfiltered = [in_degrees_2022_shop6_nonredundant_unfiltered[node] * 100 + 1000 for node in G_2022_shop6_nonredundant_unfiltered.nodes()]

# Normalizing node sizes for color mapping
normalized_sizes_2022_shop6_nonredundant_unfiltered = (np.array(node_sizes_2022_shop6_nonredundant_unfiltered) - min(node_sizes_2022_shop6_nonredundant_unfiltered)) / (max(node_sizes_2022_shop6_nonredundant_unfiltered) - min(node_sizes_2022_shop6_nonredundant_unfiltered))

# Defining a color map from orange to red
cmap_2022_shop6_nonredundant_unfiltered = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])

# Mapping normalized node sizes to colors
node_colors_2022_shop6_nonredundant_unfiltered = [cmap_2022_shop6_nonredundant_unfiltered(size) for size in normalized_sizes_2022_shop6_nonredundant_unfiltered]

# Drawing the network with node sizes and colors
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2022_shop6_nonredundant_unfiltered, pos_2022_shop6_nonredundant_unfiltered, node_size=node_sizes_2022_shop6_nonredundant_unfiltered, node_color=node_colors_2022_shop6_nonredundant_unfiltered, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop6_nonredundant_unfiltered, pos_2022_shop6_nonredundant_unfiltered, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2022_shop6_nonredundant_unfiltered, pos_2022_shop6_nonredundant_unfiltered, font_size=10)

plt.title('Node Size Based on Indegree for 2022 Shop 6')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2022_shop6_lift = nx.DiGraph()

lift_summary_2022_shop6 = {}

for _, row in rules_2022_shop6_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop6_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Update lift summary for consequents
    if consequents in lift_summary_2022_shop6:
        lift_summary_2022_shop6[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop6[consequents]['count'] += 1
    else:
        lift_summary_2022_shop6[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculate average lift for each consequent
for item in lift_summary_2022_shop6:
    lift_summary_2022_shop6[item]['avg_lift'] = lift_summary_2022_shop6[item]['total_lift'] / lift_summary_2022_shop6[item]['count']

# Node sizes based on average lift, scaled appropriately
node_sizes_2022_shop6_lift = [lift_summary_2022_shop6[node]['avg_lift'] * 500 if node in lift_summary_2022_shop6 else 1000 for node in G_2022_shop6_lift.nodes()]

pos_2022_shop6_lift = nx.spring_layout(G_2022_shop6_lift)

# Use a colormap that transitions from orange to red
cmap_2022_shop6_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop6_lift = np.array(node_sizes_2022_shop6_lift)
normalized_sizes_2022_shop6_lift = (normalized_sizes_2022_shop6_lift - min(normalized_sizes_2022_shop6_lift)) / (max(normalized_sizes_2022_shop6_lift) - min(normalized_sizes_2022_shop6_lift))
node_colors_2022_shop6_lift = [cmap_2022_shop6_lift(size) for size in normalized_sizes_2022_shop6_lift]

plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2022_shop6_lift, pos_2022_shop6_lift, node_size=node_sizes_2022_shop6_lift, node_color=node_colors_2022_shop6_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop6_lift, pos_2022_shop6_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop6_lift, pos_2022_shop6_lift, font_size=10)

plt.title('Node Size Based on Average Lift for 2022 Shop 6')
plt.axis('off')
plt.show()


# Based on conviction

In [ ]:
rules_2022_shop6_nonredundant.head()

In [ ]:
G_2022_shop6_conviction = nx.DiGraph()

conviction_summary_2022_shop6 = {}
count_summary_2022_shop6 = {}

# Assuming 'rules_2022_shop6_nonredundant_unfiltered' contains a 'conviction' column
for _, row in rules_2022_shop6_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop6_conviction.add_edge(antecedents, consequents, weight=row['confidence'], conviction=row['conviction'])
    
    # Aggregate conviction values and count occurrences
    if consequents in conviction_summary_2022_shop6:
        conviction_summary_2022_shop6[consequents] += row['conviction']
        count_summary_2022_shop6[consequents] += 1
    else:
        conviction_summary_2022_shop6[consequents] = row['conviction']
        count_summary_2022_shop6[consequents] = 1

# Calculate average conviction values
average_conviction_summary_2022_shop6 = {k: conviction_summary_2022_shop6[k] / count_summary_2022_shop6[k] for k in conviction_summary_2022_shop6}

# Normalize conviction values for sizing and coloring
max_conviction = max(average_conviction_summary_2022_shop6.values())
min_conviction = min(average_conviction_summary_2022_shop6.values())
conviction_range = max_conviction - min_conviction
node_colors = []
node_sizes = []

for node in G_2022_shop6_conviction.nodes():
    # Get the average conviction value for the node, defaulting to min if not present
    conviction = average_conviction_summary_2022_shop6.get(node, min_conviction)
    # Normalize between 0 and 1
    normalized_conviction = (conviction - min_conviction) / conviction_range if conviction_range else 0
    node_colors.append(normalized_conviction)
    node_sizes.append(normalized_conviction * 1000 + 100)  # Scale and offset size

# Use a colormap for coloring nodes based on conviction
cmap = plt.cm.coolwarm

pos = nx.spring_layout(G_2022_shop6_conviction)

plt.figure(figsize=(40, 22))
nodes = nx.draw_networkx_nodes(G_2022_shop6_conviction, pos, node_color=node_colors, node_size=node_sizes, cmap=cmap, alpha=0.8)
edges = nx.draw_networkx_edges(G_2022_shop6_conviction, pos, arrowstyle='->', arrowsize=10, edge_color='grey', alpha=0.5)
labels = nx.draw_networkx_labels(G_2022_shop6_conviction, pos, font_size=8)

plt.colorbar(nodes, label='Normalized Conviction')
plt.title('Network Graph of 2022 Shop 6 Based on Conviction')
plt.axis('off')
plt.show()

In [ ]:
print(max_conviction)

# Filtered top 30 and non redundant

In [ ]:
rules_2022_shop6_nonredundant.head()

In [ ]:
# Assuming rules_1992_shop2_nonredundant is your sorted rules DataFrame for shop2
rules_2022_shop6_nonredundant_top30 = rules_2022_shop6_nonredundant.copy(deep=True)

In [ ]:
rules_2022_shop6_nonredundant_top30_lift = rules_2022_shop6_nonredundant.copy(deep=True)

In [ ]:
# Select the top 30 rules based on lift or another criterion
rules_2022_shop6_nonredundant_top30_lift = rules_2022_shop6_nonredundant_top30_lift.sort_values(by='lift', ascending=False).head(30)

In [ ]:
rules_2022_shop6_nonredundant_top30_lift.head()

In [ ]:
# Select the top 30 rules based on confidence or another criterion
rules_2022_shop6_nonredundant_top30 = rules_2022_shop6_nonredundant_top30.sort_values(by='confidence', ascending=False).head(30)

In [ ]:
rules_2022_shop6_nonredundant_top30.info()

In [ ]:
rules_2022_shop6_nonredundant_top30['LHS'] = [','.join(list(x)) for x in rules_2022_shop6_nonredundant_top30['antecedents']]
rules_2022_shop6_nonredundant_top30['RHS'] = [','.join(list(x)) for x in rules_2022_shop6_nonredundant_top30['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2022_shop6_nonredundant_top30.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2022_shop6_nonredundant_top30, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2022 Shop 6")
plt.axis('off')
plt.show()


## Based on Indegree

In [ ]:
G_2022_shop6 = nx.DiGraph()

# Assuming 'rules_sorted_2022_shop6' contains association rules for shop 6 in 2022
for _, row in rules_2022_shop6_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as the weight
    G_2022_shop6.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2022_shop6 = nx.spring_layout(G_2022_shop6)

# Calculate node sizes based on indegree
in_degrees_2022_shop6 = dict(G_2022_shop6.in_degree())
node_sizes_2022_shop6 = [in_degrees_2022_shop6[node] * 100 + 1000 for node in G_2022_shop6.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2022_shop6 = (np.array(node_sizes_2022_shop6) - np.min(node_sizes_2022_shop6)) / (np.max(node_sizes_2022_shop6) - np.min(node_sizes_2022_shop6))

# Define a colormap from orange to red
cmap_2022_shop6 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2022_shop6 = [cmap_2022_shop6(size) for size in normalized_sizes_2022_shop6]

# Drawing the network
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2022_shop6, pos_2022_shop6, node_size=node_sizes_2022_shop6, node_color=node_colors_2022_shop6, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop6, pos_2022_shop6, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2022_shop6, pos_2022_shop6, font_size=8)

plt.title('Top 5 Association Rules Network for 2022 Shop 6 with Color Gradient')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2022_shop6_lift = nx.DiGraph()

lift_summary_2022_shop6 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2022_shop6_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop6_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop6:
        lift_summary_2022_shop6[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop6[consequents]['count'] += 1
    else:
        lift_summary_2022_shop6[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop6:
    lift_summary_2022_shop6[item]['avg_lift'] = lift_summary_2022_shop6[item]['total_lift'] / lift_summary_2022_shop6[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop6_lift = [lift_summary_2022_shop6[node]['avg_lift'] * 500 if node in lift_summary_2022_shop6 else 1000 for node in G_2022_shop6_lift.nodes()]

pos_2022_shop6_lift = nx.spring_layout(G_2022_shop6_lift)

# Colormap from orange to red
cmap_2022_shop6_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop6_lift = np.array(node_sizes_2022_shop6_lift)
normalized_sizes_2022_shop6_lift = (normalized_sizes_2022_shop6_lift - min(normalized_sizes_2022_shop6_lift)) / (max(normalized_sizes_2022_shop6_lift) - min(normalized_sizes_2022_shop6_lift))
node_colors_2022_shop6_lift = [cmap_2022_shop6_lift(size) for size in normalized_sizes_2022_shop6_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2022_shop6_lift, pos_2022_shop6_lift, node_size=node_sizes_2022_shop6_lift, node_color=node_colors_2022_shop6_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop6_lift, pos_2022_shop6_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop6_lift, pos_2022_shop6_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2022 Shop 6 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop6_lift = nx.DiGraph()

lift_summary_2022_shop6 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2022_shop6_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop6_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop6:
        lift_summary_2022_shop6[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop6[consequents]['count'] += 1
    else:
        lift_summary_2022_shop6[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop6:
    lift_summary_2022_shop6[item]['avg_lift'] = lift_summary_2022_shop6[item]['total_lift'] / lift_summary_2022_shop6[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop6_lift = [lift_summary_2022_shop6[node]['avg_lift'] * 500 if node in lift_summary_2022_shop6 else 1000 for node in G_2022_shop6_lift.nodes()]

pos_2022_shop6_lift = nx.spring_layout(G_2022_shop6_lift)

# Colormap from orange to red
cmap_2022_shop6_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop6_lift = np.array(node_sizes_2022_shop6_lift)
normalized_sizes_2022_shop6_lift = (normalized_sizes_2022_shop6_lift - min(normalized_sizes_2022_shop6_lift)) / (max(normalized_sizes_2022_shop6_lift) - min(normalized_sizes_2022_shop6_lift))
node_colors_2022_shop6_lift = [cmap_2022_shop6_lift(size) for size in normalized_sizes_2022_shop6_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2022_shop6_lift, pos_2022_shop6_lift, node_size=node_sizes_2022_shop6_lift, node_color=node_colors_2022_shop6_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop6_lift, pos_2022_shop6_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop6_lift, pos_2022_shop6_lift, font_size=10)

plt.title('Top 5 Association Rules Network for 2022 Shop 6 Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop6 = nx.DiGraph()

# Assuming 'rules_2022_shop7_nonredundant_top30' contains association rules for 2022 Shop 6
for _, row in rules_2022_shop6_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop6.add_edge(antecedents, consequents, confidence=row['confidence'])

# Calculate node sizes based on indegree
in_degrees_2022_shop6 = dict(G_2022_shop6.in_degree())
node_sizes_2022_shop6 = [in_degrees_2022_shop6[node] * 100 + 1000 for node in G_2022_shop6.nodes()]

# Edge width based on confidence
edge_widths_2022_shop6 = [G_2022_shop6[u][v]['confidence'] * 15 for u, v in G_2022_shop6.edges()]

# Assuming normalized sizes and color mapping is desired
normalized_sizes_2022_shop6 = np.array(list(in_degrees_2022_shop6.values()))
normalized_sizes_2022_shop6 = (normalized_sizes_2022_shop6 - min(normalized_sizes_2022_shop6)) / (max(normalized_sizes_2022_shop6) - min(normalized_sizes_2022_shop6))
cmap_2022_shop6 = mcolors.LinearSegmentedColormap.from_list("MyCMapName", ["blue", "red"])

# Node colors based on a property (here, using indegree for example)
node_colors_2022_shop6 = [cmap_2022_shop6(size) for size in normalized_sizes_2022_shop6]

# Layout
pos_2022_shop6 = nx.spring_layout(G_2022_shop6, seed=42)

# Drawing
plt.figure(figsize=(40, 30))
nx.draw_networkx_edges(G_2022_shop6, pos_2022_shop6, width=edge_widths_2022_shop6, edge_color='grey', alpha=0.5)
nx.draw_networkx_nodes(G_2022_shop6, pos_2022_shop6, node_size=node_sizes_2022_shop6, node_color=node_colors_2022_shop6, alpha=0.8)
nx.draw_networkx_labels(G_2022_shop6, pos_2022_shop6, font_size=8, font_color='black')
plt.title('Network Graph of Association Rules Based on Confidence for 2022 Shop 6')
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop6 = nx.DiGraph()

convictions_2022_shop6 = []  # To store conviction values for normalization

for _, row in rules_2022_shop6_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    conviction_2022_shop6 = row['conviction']  # Assuming each row has a 'conviction' column
    convictions_2022_shop6.append(conviction_2022_shop6)
    G_2022_shop6.add_edge(antecedents, consequents, weight=row['confidence'], conviction=conviction_2022_shop6)

# Normalize conviction values for edge width
max_conviction_2022_shop6 = max(convictions_2022_shop6)
min_conviction_2022_shop6 = min(convictions_2022_shop6)
conviction_range_2022_shop6 = max_conviction_2022_shop6 - min_conviction_2022_shop6
edge_widths_2022_shop6 = [(G_2022_shop6[u][v]['conviction'] - min_conviction_2022_shop6) / conviction_range_2022_shop6 * 30 + 0.5 for u, v in G_2022_shop6.edges()]  # Scale and offset widths

pos_2022_shop6 = nx.spring_layout(G_2022_shop6)

# Node sizes and colors as before
in_degrees_2022_shop6 = dict(G_2022_shop6.in_degree())
node_sizes_2022_shop6 = [in_degrees_2022_shop6[node] * 100 + 1000 for node in G_2022_shop6.nodes()]
normalized_sizes_2022_shop6 = (np.array(node_sizes_2022_shop6) - min(node_sizes_2022_shop6)) / (max(node_sizes_2022_shop6) - min(node_sizes_2022_shop6))
cmap_2022_shop6 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2022_shop6 = [cmap_2022_shop6(size) for size in normalized_sizes_2022_shop6]

# Drawing the network with conviction-based edge widths
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2022_shop6, pos_2022_shop6, node_size=node_sizes_2022_shop6, node_color=node_colors_2022_shop6, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop6, pos_2022_shop6, width=edge_widths_2022_shop6, edge_color='grey', alpha=0.5)
nx.draw_networkx_labels(G_2022_shop6, pos_2022_shop6, font_size=10)

plt.title('Top 5 Association Rules Network for 2022 Shop 6 with Conviction-based Edge Thickness')
plt.axis('off')
plt.show()


In [ ]:
#the node sizes are based on the indegree where each node acts as a consequent,

In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2022_shop6_nonredundant_top30.loc[:, 'antecedents_'] = rules_2022_shop6_nonredundant_top30['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2022_shop6_nonredundant_top30.loc[:, 'consequents_'] = rules_2022_shop6_nonredundant_top30['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2022_shop6_nonredundant_top30, x='consequents_', y='antecedents_', s=rules_2022_shop6_nonredundant_top30['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2022_shop6_nonredundant_top30.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2022 Shop 6", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
print(rules_2022_shop6_nonredundant_top30[['antecedents', 'consequents']].head(10))

## Monetary Analysis of the Top Rules

In [ ]:
# Step 1: Extract unique product codes from the rules
unique_product_codes = set()
for idx, row in rules_2022_shop6_nonredundant_top30.iterrows():
    # Unite antecedents and consequents into a single set
    all_products = row['antecedents'].union(row['consequents'])
    unique_product_codes.update(all_products)

# Step 2: Create a DataFrame with unique product codes and their unit prices
# Filter sales_data for these product codes and drop duplicates to ensure each product code appears once
product_prices = sales_data[sales_data['product_code'].isin(unique_product_codes)][['product_code', 'unit_price']].drop_duplicates()

# Create a dictionary for quick lookup of prices
price_dict = product_prices.set_index('product_code')['unit_price'].to_dict()

# Step 3: Calculate the total price for each rule and collect data for the new DataFrame
data_for_new_df = []
for idx, row in rules_2022_shop6_nonredundant_top30.iterrows():
    # Combine sets of antecedents and consequents
    all_products = row['antecedents'].union(row['consequents'])
    total_price = sum(price_dict.get(prod, 0) for prod in all_products)  # Use get to avoid KeyError if any product code is missing
    data_for_new_df.append({
        'antecedents': row['antecedents'],
        'consequents': row['consequents'],
        'total_unit_price': total_price
    })

# Create a new DataFrame with only antecedents, consequents, and total unit price
rules_and_price_shop6_2022 = pd.DataFrame(data_for_new_df)

# Sort the DataFrame by total_unit_price in descending order
rules_and_price_shop6_2022_sorted = rules_and_price_shop6_2022.sort_values(by='total_unit_price', ascending=False)

# Print the sorted DataFrame to see the specified columns along with their total unit prices
print(rules_and_price_shop6_2022_sorted)


In [ ]:
# Convert 'trx_date' to datetime format if not already done so
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Add a 'year' column if it doesn't exist
sales_data['year'] = sales_data['trx_date'].dt.year

# Step 2: Filter sales_data for shop6 and the year 2022
filtered_sales_data_2022 = sales_data[(sales_data['shopID'] == 6) & (sales_data['year'] == 2022)]

# Step 3: Initialize a list to hold the rule data including total sales value
rule_sales_summaries_2022 = []

# Step 4: Iterate through each rule in the DataFrame
for index, rule in rules_2022_shop6_nonredundant_top30.iterrows():
    all_products = rule['antecedents'].union(rule['consequents'])

    # Filter the sales data for transactions that include all products in the rule
    # This step ensures that we are only considering transactions that contain all specified products
    mask = filtered_sales_data_2022['product_code'].isin(all_products)
    transaction_ids = filtered_sales_data_2022[mask].groupby('trx_id').filter(lambda x: all_products.issubset(set(x['product_code']))).trx_id.unique()
    matched_transactions = filtered_sales_data_2022[filtered_sales_data_2022['trx_id'].isin(transaction_ids)]
    
    # Step 5: Sum the total_value of these transactions
    total_sales_value = matched_transactions['total_value'].sum()

    # Append the result to the list
    rule_sales_summaries_2022.append({
        'antecedents': rule['antecedents'],
        'consequents': rule['consequents'],
        'total_sales_value': total_sales_value
    })

# Step 6: Convert the list to a DataFrame
rules_sales_values_df_2022 = pd.DataFrame(rule_sales_summaries_2022)

# Step 7: Sort by total_sales_value in descending order
sorted_rules_sales_values_df_2022 = rules_sales_values_df_2022.sort_values(by='total_sales_value', ascending=False)

# Step 8: Print the sorted DataFrame
print(sorted_rules_sales_values_df_2022)


## Narrowing down based on Lift

In [ ]:
rules_2022_shop6_nonredundant_top30_lift['LHS'] = [','.join(list(x)) for x in rules_2022_shop6_nonredundant_top30_lift['antecedents']]
rules_2022_shop6_nonredundant_top30_lift['RHS'] = [','.join(list(x)) for x in rules_2022_shop6_nonredundant_top30_lift['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2022_shop6_nonredundant_top30_lift.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2022_shop6_nonredundant_top30_lift, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2022 Shop 6 with Lift")
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop6_lift = nx.DiGraph()

lift_summary_2022_shop6 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2022_shop6_nonredundant_top30_lift.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop6_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop6:
        lift_summary_2022_shop6[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop6[consequents]['count'] += 1
    else:
        lift_summary_2022_shop6[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop6:
    lift_summary_2022_shop6[item]['avg_lift'] = lift_summary_2022_shop6[item]['total_lift'] / lift_summary_2022_shop6[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop6_lift = [lift_summary_2022_shop6[node]['avg_lift'] * 500 if node in lift_summary_2022_shop6 else 1000 for node in G_2022_shop6_lift.nodes()]

pos_2022_shop6_lift = nx.spring_layout(G_2022_shop6_lift)

# Colormap from orange to red
cmap_2022_shop6_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop6_lift = np.array(node_sizes_2022_shop6_lift)
normalized_sizes_2022_shop6_lift = (normalized_sizes_2022_shop6_lift - min(normalized_sizes_2022_shop6_lift)) / (max(normalized_sizes_2022_shop6_lift) - min(normalized_sizes_2022_shop6_lift))
node_colors_2022_shop6_lift = [cmap_2022_shop6_lift(size) for size in normalized_sizes_2022_shop6_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2022_shop6_lift, pos_2022_shop6_lift, node_size=node_sizes_2022_shop6_lift, node_color=node_colors_2022_shop6_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop6_lift, pos_2022_shop6_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop6_lift, pos_2022_shop6_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2022 Shop 6 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
rules_2022_shop6_nonredundant_top30_lift.loc[:, 'antecedents_'] = rules_2022_shop6_nonredundant_top30_lift['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2022_shop6_nonredundant_top30_lift.loc[:, 'consequents_'] = rules_2022_shop6_nonredundant_top30_lift['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2022_shop6_nonredundant_top30_lift, x='consequents_', y='antecedents_', s=rules_2022_shop6_nonredundant_top30_lift['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2022_shop6_nonredundant_top30_lift.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2022 Shop 6", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
# Print only the 'antecedents' and 'consequents' columns of the top 10 rules
print(rules_2022_shop6_nonredundant_top30_lift[['antecedents', 'consequents']].head(10))


# Shop6 2023 SEARCH shop6_2

In [ ]:
# Filter for records from shop ID 1 and the year 1993
sales_data_2023_shop6 = sales_data_shop6[(sales_data_shop6['trx_date'].dt.year == 2023)]

In [ ]:
sales_data_2023_shop6.head()

In [ ]:
# Grouping product codes by trx_id
transactions_2023_shop6 = sales_data_2023_shop6.groupby('trx_id')['product_code'].apply(list).tolist()

In [ ]:
# Encode transaction data
encoder_2023_shop6 = TransactionEncoder()
trans_array_2023_shop6 = encoder_2023_shop6.fit(transactions_2023_shop6).transform(transactions_2023_shop6)
trans_df_2023_shop6 = pd.DataFrame(trans_array_2023_shop6, columns=encoder_2023_shop6.columns_)

In [ ]:
# Find frequent itemsets
frequent_itemsets_2023_shop6 = apriori(trans_df_2023_shop6, min_support=0.001, use_colnames=True)

In [ ]:
print(frequent_itemsets_2023_shop6)

In [ ]:
rulesConfidence_2023_shop6 = association_rules(frequent_itemsets_2023_shop6, metric="confidence", min_threshold=0.10)
rulesConfidence_2023_shop6.sort_values(by='confidence', ascending=False, inplace=True)


In [ ]:
# Replace frozen sets with strings
rulesConfidence_2023_shop6['antecedents_'] = rulesConfidence_2023_shop6['antecedents'].apply(lambda a: ','.join(list(a)))
rulesConfidence_2023_shop6['consequents_'] = rulesConfidence_2023_shop6['consequents'].apply(lambda a: ','.join(list(a)))

# Draw
fig, ax = plt.subplots(figsize=(14,30))
ax = plt.scatter(data=rulesConfidence_2023_shop6, x='consequents_', y='antecedents_', s=rulesConfidence_2023_shop6['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rulesConfidence_2023_shop6.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2023 Shop 6", fontsize=14)
plt.xlabel("Consequents (RHS)", fontsize=12)
plt.ylabel("Antecedents (LHS)", fontsize=12)
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate(rotation=45)
plt.tight_layout()

plt.show()


## Eliminate Redundant Rules

In [ ]:
rulesConfidence_2023_shop6.head()

In [ ]:
rulesConfidence_2023_shop6.info()

In [ ]:
# If 'antecedents' and 'consequents' are not frozensets, convert them
rules_2023_shop6_redundant = rulesConfidence_2023_shop6.copy()
rules_2023_shop6_redundant['antecedents'] = rules_2023_shop6_redundant['antecedents'].apply(lambda x: frozenset(x))
rules_2023_shop6_redundant['consequents'] = rules_2023_shop6_redundant['consequents'].apply(lambda x: frozenset(x))


In [ ]:
# Create a copy of the DataFrame to avoid modifying the original while iterating
rules_2023_shop6_checkredundance = rules_2023_shop6_redundant.copy()

# List to hold non-redundant rules
non_redundant_rules_2023_shop6 = []

for _, rule in rules_2023_shop6_checkredundance.iterrows():
    if not is_redundant(rule, rules_2023_shop6_checkredundance):
        non_redundant_rules_2023_shop6.append(rule)

# Convert the list back to a DataFrame
rules_2023_shop6_nonredundant = pd.DataFrame(non_redundant_rules_2023_shop6).reset_index(drop=True)


In [ ]:
print(rules_2023_shop6_nonredundant.head())
print(f"Original number of rules: {len(rules_2023_shop6_redundant)}")
print(f"Number of non-redundant rules: {len(rules_2023_shop6_nonredundant)}")


In [ ]:
rules_2023_shop6_nonredundant.head()

In [ ]:
rules_2023_shop6_redundant.info()

In [ ]:
rules_2023_shop6_nonredundant.head()

In [ ]:
rules_2023_shop6_nonredundant_backup = rules_2023_shop6_nonredundant.copy()

In [ ]:
rules_2023_shop6_nonredundant_unfiltered = rules_2023_shop6_nonredundant.copy()

In [ ]:
#reset
#rules_2023_shop6_nonredundant = rules_2023_shop6_nonredundant_backup.copy()

# Redundant and Unfiltered

In [ ]:
rules_2023_shop6_redundant.head()

In [ ]:
G_2023_shop6 = nx.DiGraph()

# Iterate through rules for 2023 Shop 6 and add edges to the graph
for _, row in rules_2023_shop6_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as weight
    G_2023_shop6.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2023_shop6 = nx.spring_layout(G_2023_shop6)

# Recalculate node sizes based on indegree or another metric
in_degrees_2023_shop6 = dict(G_2023_shop6.in_degree())
node_sizes_2023_shop6 = [in_degrees_2023_shop6[node] * 100 + 1000 for node in G_2023_shop6.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2023_shop6 = (np.array(node_sizes_2023_shop6) - np.min(node_sizes_2023_shop6)) / (np.max(node_sizes_2023_shop6) - np.min(node_sizes_2023_shop6))

# Define a colormap from orange to red for visual consistency
cmap_2023_shop6 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop6 = [cmap_2023_shop6(size) for size in normalized_sizes_2023_shop6]

# Drawing the network
plt.figure(figsize=(60, 43))
nx.draw_networkx_nodes(G_2023_shop6, pos_2023_shop6, node_size=node_sizes_2023_shop6, node_color=node_colors_2023_shop6, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop6, pos_2023_shop6, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop6, pos_2023_shop6, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 6 with Color Gradient')
plt.axis('off')
plt.show()


### Based on LIFT

In [ ]:
G_2023_shop6_lift = nx.DiGraph()

lift_summary_2023_shop6 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop6_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop6_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop6:
        lift_summary_2023_shop6[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop6[consequents]['count'] += 1
    else:
        lift_summary_2023_shop6[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop6:
    lift_summary_2023_shop6[item]['avg_lift'] = lift_summary_2023_shop6[item]['total_lift'] / lift_summary_2023_shop6[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop6_lift = [lift_summary_2023_shop6[node]['avg_lift'] * 500 if node in lift_summary_2023_shop6 else 1000 for node in G_2023_shop6_lift.nodes()]

pos_2023_shop6_lift = nx.spring_layout(G_2023_shop6_lift)

# Colormap from orange to red
cmap_2023_shop6_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop6_lift = np.array(node_sizes_2023_shop6_lift)
normalized_sizes_2023_shop6_lift = (normalized_sizes_2023_shop6_lift - min(normalized_sizes_2023_shop6_lift)) / (max(normalized_sizes_2023_shop6_lift) - min(normalized_sizes_2023_shop6_lift))
node_colors_2023_shop6_lift = [cmap_2023_shop6_lift(size) for size in normalized_sizes_2023_shop6_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2023_shop6_lift, pos_2023_shop6_lift, node_size=node_sizes_2023_shop6_lift, node_color=node_colors_2023_shop6_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop6_lift, pos_2023_shop6_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop6_lift, pos_2023_shop6_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 6 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


# nonredundant_unfiltered

## Based on Indegree

In [ ]:
G_2023_shop6_nonredundant_unfiltered = nx.DiGraph()

# Iterating through each row to add edges with confidence as an attribute
for _, row in rules_2023_shop6_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop6_nonredundant_unfiltered.add_edge(antecedents, consequents, weight=row['confidence'])

# Defining the layout for network visualization
pos_2023_shop6_nonredundant_unfiltered = nx.spring_layout(G_2023_shop6_nonredundant_unfiltered)

# Calculating node sizes based on the indegree for color mapping
in_degrees_2023_shop6_nonredundant_unfiltered = dict(G_2023_shop6_nonredundant_unfiltered.in_degree())
node_sizes_2023_shop6_nonredundant_unfiltered = [in_degrees_2023_shop6_nonredundant_unfiltered[node] * 100 + 1000 for node in G_2023_shop6_nonredundant_unfiltered.nodes()]

# Normalizing node sizes for color mapping
normalized_sizes_2023_shop6_nonredundant_unfiltered = (np.array(node_sizes_2023_shop6_nonredundant_unfiltered) - min(node_sizes_2023_shop6_nonredundant_unfiltered)) / (max(node_sizes_2023_shop6_nonredundant_unfiltered) - min(node_sizes_2023_shop6_nonredundant_unfiltered))

# Defining a color map from orange to red
cmap_2023_shop6_nonredundant_unfiltered = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])

# Mapping normalized node sizes to colors
node_colors_2023_shop6_nonredundant_unfiltered = [cmap_2023_shop6_nonredundant_unfiltered(size) for size in normalized_sizes_2023_shop6_nonredundant_unfiltered]

# Drawing the network with node sizes and colors
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2023_shop6_nonredundant_unfiltered, pos_2023_shop6_nonredundant_unfiltered, node_size=node_sizes_2023_shop6_nonredundant_unfiltered, node_color=node_colors_2023_shop6_nonredundant_unfiltered, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop6_nonredundant_unfiltered, pos_2023_shop6_nonredundant_unfiltered, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2023_shop6_nonredundant_unfiltered, pos_2023_shop6_nonredundant_unfiltered, font_size=10)

plt.title('Node Size Based on Indegree for 2023 Shop 6')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2023_shop6_lift = nx.DiGraph()

lift_summary_2023_shop6 = {}

for _, row in rules_2023_shop6_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop6_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Update lift summary for consequents
    if consequents in lift_summary_2023_shop6:
        lift_summary_2023_shop6[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop6[consequents]['count'] += 1
    else:
        lift_summary_2023_shop6[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculate average lift for each consequent
for item in lift_summary_2023_shop6:
    lift_summary_2023_shop6[item]['avg_lift'] = lift_summary_2023_shop6[item]['total_lift'] / lift_summary_2023_shop6[item]['count']

# Node sizes based on average lift, scaled appropriately
node_sizes_2023_shop6_lift = [lift_summary_2023_shop6[node]['avg_lift'] * 500 if node in lift_summary_2023_shop6 else 1000 for node in G_2023_shop6_lift.nodes()]

pos_2023_shop6_lift = nx.spring_layout(G_2023_shop6_lift)

# Use a colormap that transitions from orange to red
cmap_2023_shop6_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop6_lift = np.array(node_sizes_2023_shop6_lift)
normalized_sizes_2023_shop6_lift = (normalized_sizes_2023_shop6_lift - min(normalized_sizes_2023_shop6_lift)) / (max(normalized_sizes_2023_shop6_lift) - min(normalized_sizes_2023_shop6_lift))
node_colors_2023_shop6_lift = [cmap_2023_shop6_lift(size) for size in normalized_sizes_2023_shop6_lift]

plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2023_shop6_lift, pos_2023_shop6_lift, node_size=node_sizes_2023_shop6_lift, node_color=node_colors_2023_shop6_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop6_lift, pos_2023_shop6_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop6_lift, pos_2023_shop6_lift, font_size=10)

plt.title('Node Size Based on Average Lift for 2023 Shop 6')
plt.axis('off')
plt.show()


# Based on conviction

In [ ]:
rules_2023_shop6_nonredundant.head()

In [ ]:
G_2023_shop6_conviction = nx.DiGraph()

conviction_summary_2023_shop6 = {}
count_summary_2023_shop6 = {}

# Assuming 'rules_2023_shop6_nonredundant_unfiltered' contains a 'conviction' column
for _, row in rules_2023_shop6_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop6_conviction.add_edge(antecedents, consequents, weight=row['confidence'], conviction=row['conviction'])
    
    # Aggregate conviction values and count occurrences
    if consequents in conviction_summary_2023_shop6:
        conviction_summary_2023_shop6[consequents] += row['conviction']
        count_summary_2023_shop6[consequents] += 1
    else:
        conviction_summary_2023_shop6[consequents] = row['conviction']
        count_summary_2023_shop6[consequents] = 1

# Calculate average conviction values
average_conviction_summary_2023_shop6 = {k: conviction_summary_2023_shop6[k] / count_summary_2023_shop6[k] for k in conviction_summary_2023_shop6}

# Normalize conviction values for sizing and coloring
max_conviction = max(average_conviction_summary_2023_shop6.values())
min_conviction = min(average_conviction_summary_2023_shop6.values())
conviction_range = max_conviction - min_conviction
node_colors = []
node_sizes = []

for node in G_2023_shop6_conviction.nodes():
    # Get the average conviction value for the node, defaulting to min if not present
    conviction = average_conviction_summary_2023_shop6.get(node, min_conviction)
    # Normalize between 0 and 1
    normalized_conviction = (conviction - min_conviction) / conviction_range if conviction_range else 0
    node_colors.append(normalized_conviction)
    node_sizes.append(normalized_conviction * 1000 + 100)  # Scale and offset size

# Use a colormap for coloring nodes based on conviction
cmap = plt.cm.coolwarm

pos = nx.spring_layout(G_2023_shop6_conviction)

plt.figure(figsize=(40, 22))
nodes = nx.draw_networkx_nodes(G_2023_shop6_conviction, pos, node_color=node_colors, node_size=node_sizes, cmap=cmap, alpha=0.8)
edges = nx.draw_networkx_edges(G_2023_shop6_conviction, pos, arrowstyle='->', arrowsize=10, edge_color='grey', alpha=0.5)
labels = nx.draw_networkx_labels(G_2023_shop6_conviction, pos, font_size=8)

plt.colorbar(nodes, label='Normalized Conviction')
plt.title('Network Graph of 2023 Shop 6 Based on Conviction')
plt.axis('off')
plt.show()

In [ ]:
print(max_conviction)

# Filtered top 30 and non redundant

In [ ]:
rules_2023_shop6_nonredundant.head()

In [ ]:
# Assuming rules_1992_shop2_nonredundant is your sorted rules DataFrame for shop2
rules_2023_shop6_nonredundant_top30 = rules_2023_shop6_nonredundant.copy(deep=True)

In [ ]:
rules_2023_shop6_nonredundant_top30_lift = rules_2023_shop6_nonredundant.copy(deep=True)

In [ ]:
# Select the top 30 rules based on lift or another criterion
rules_2023_shop6_nonredundant_top30_lift = rules_2023_shop6_nonredundant_top30_lift.sort_values(by='lift', ascending=False).head(30)

In [ ]:
rules_2023_shop6_nonredundant_top30_lift.head()

In [ ]:
# Select the top 30 rules based on confidence or another criterion
rules_2023_shop6_nonredundant_top30 = rules_2023_shop6_nonredundant_top30.sort_values(by='confidence', ascending=False).head(30)

In [ ]:
rules_2023_shop6_nonredundant_top30.info()

In [ ]:
rules_2023_shop6_nonredundant_top30['LHS'] = [','.join(list(x)) for x in rules_2023_shop6_nonredundant_top30['antecedents']]
rules_2023_shop6_nonredundant_top30['RHS'] = [','.join(list(x)) for x in rules_2023_shop6_nonredundant_top30['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2023_shop6_nonredundant_top30.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2023_shop6_nonredundant_top30, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2023 Shop 6")
plt.axis('off')
plt.show()


## Based on Indegree

In [ ]:
G_2023_shop6 = nx.DiGraph()

# Assuming 'rules_sorted_2023_shop6' contains association rules for shop 6 in 2023
for _, row in rules_2023_shop6_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as the weight
    G_2023_shop6.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2023_shop6 = nx.spring_layout(G_2023_shop6)

# Calculate node sizes based on indegree
in_degrees_2023_shop6 = dict(G_2023_shop6.in_degree())
node_sizes_2023_shop6 = [in_degrees_2023_shop6[node] * 100 + 1000 for node in G_2023_shop6.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2023_shop6 = (np.array(node_sizes_2023_shop6) - np.min(node_sizes_2023_shop6)) / (np.max(node_sizes_2023_shop6) - np.min(node_sizes_2023_shop6))

# Define a colormap from orange to red
cmap_2023_shop6 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop6 = [cmap_2023_shop6(size) for size in normalized_sizes_2023_shop6]

# Drawing the network
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2023_shop6, pos_2023_shop6, node_size=node_sizes_2023_shop6, node_color=node_colors_2023_shop6, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop6, pos_2023_shop6, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2023_shop6, pos_2023_shop6, font_size=8)

plt.title('Top 5 Association Rules Network for 2023 Shop 6 with Color Gradient')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2023_shop6_lift = nx.DiGraph()

lift_summary_2023_shop6 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop6_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop6_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop6:
        lift_summary_2023_shop6[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop6[consequents]['count'] += 1
    else:
        lift_summary_2023_shop6[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop6:
    lift_summary_2023_shop6[item]['avg_lift'] = lift_summary_2023_shop6[item]['total_lift'] / lift_summary_2023_shop6[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop6_lift = [lift_summary_2023_shop6[node]['avg_lift'] * 500 if node in lift_summary_2023_shop6 else 1000 for node in G_2023_shop6_lift.nodes()]

pos_2023_shop6_lift = nx.spring_layout(G_2023_shop6_lift)

# Colormap from orange to red
cmap_2023_shop6_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop6_lift = np.array(node_sizes_2023_shop6_lift)
normalized_sizes_2023_shop6_lift = (normalized_sizes_2023_shop6_lift - min(normalized_sizes_2023_shop6_lift)) / (max(normalized_sizes_2023_shop6_lift) - min(normalized_sizes_2023_shop6_lift))
node_colors_2023_shop6_lift = [cmap_2023_shop6_lift(size) for size in normalized_sizes_2023_shop6_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2023_shop6_lift, pos_2023_shop6_lift, node_size=node_sizes_2023_shop6_lift, node_color=node_colors_2023_shop6_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop6_lift, pos_2023_shop6_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop6_lift, pos_2023_shop6_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 6 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop6_lift = nx.DiGraph()

lift_summary_2023_shop6 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop6_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop6_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop6:
        lift_summary_2023_shop6[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop6[consequents]['count'] += 1
    else:
        lift_summary_2023_shop6[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop6:
    lift_summary_2023_shop6[item]['avg_lift'] = lift_summary_2023_shop6[item]['total_lift'] / lift_summary_2023_shop6[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop6_lift = [lift_summary_2023_shop6[node]['avg_lift'] * 500 if node in lift_summary_2023_shop6 else 1000 for node in G_2023_shop6_lift.nodes()]

pos_2023_shop6_lift = nx.spring_layout(G_2023_shop6_lift)

# Colormap from orange to red
cmap_2023_shop6_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop6_lift = np.array(node_sizes_2023_shop6_lift)
normalized_sizes_2023_shop6_lift = (normalized_sizes_2023_shop6_lift - min(normalized_sizes_2023_shop6_lift)) / (max(normalized_sizes_2023_shop6_lift) - min(normalized_sizes_2023_shop6_lift))
node_colors_2023_shop6_lift = [cmap_2023_shop6_lift(size) for size in normalized_sizes_2023_shop6_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2023_shop6_lift, pos_2023_shop6_lift, node_size=node_sizes_2023_shop6_lift, node_color=node_colors_2023_shop6_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop6_lift, pos_2023_shop6_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop6_lift, pos_2023_shop6_lift, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 6 Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop6 = nx.DiGraph()

# Assuming 'rules_2023_shop6_nonredundant' contains association rules for 2023 Shop 6
for _, row in rules_2023_shop6_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop6.add_edge(antecedents, consequents, confidence=row['confidence'])

# Calculate node sizes based on indegree
in_degrees_2023_shop6 = dict(G_2023_shop6.in_degree())
node_sizes_2023_shop6 = [in_degrees_2023_shop6[node] * 100 + 1000 for node in G_2023_shop6.nodes()]

# Edge width based on confidence
edge_widths_2023_shop6 = [G_2023_shop6[u][v]['confidence'] * 15 for u, v in G_2023_shop6.edges()]

# Assuming normalized sizes and color mapping is desired
normalized_sizes_2023_shop6 = np.array(list(in_degrees_2023_shop6.values()))
normalized_sizes_2023_shop6 = (normalized_sizes_2023_shop6 - min(normalized_sizes_2023_shop6)) / (max(normalized_sizes_2023_shop6) - min(normalized_sizes_2023_shop6))
cmap_2023_shop6 = mcolors.LinearSegmentedColormap.from_list("MyCMapName", ["blue", "red"])

# Node colors based on a property (here, using indegree for example)
node_colors_2023_shop6 = [cmap_2023_shop6(size) for size in normalized_sizes_2023_shop6]

# Layout
pos_2023_shop6 = nx.spring_layout(G_2023_shop6, seed=42)

# Drawing
plt.figure(figsize=(40, 30))
nx.draw_networkx_edges(G_2023_shop6, pos_2023_shop6, width=edge_widths_2023_shop6, edge_color='grey', alpha=0.5)
nx.draw_networkx_nodes(G_2023_shop6, pos_2023_shop6, node_size=node_sizes_2023_shop6, node_color=node_colors_2023_shop6, alpha=0.8)
nx.draw_networkx_labels(G_2023_shop6, pos_2023_shop6, font_size=8, font_color='black')
plt.title('Network Graph of Association Rules Based on Confidence for 2023 Shop 6')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop6 = nx.DiGraph()

convictions_2023_shop6 = []  # To store conviction values for normalization

for _, row in rules_2023_shop6_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    conviction = row['conviction']  # Assuming each row has a 'conviction' column
    convictions_2023_shop6.append(conviction)
    G_2023_shop6.add_edge(antecedents, consequents, weight=row['confidence'], conviction=conviction)

# Normalize conviction values for edge width
max_conviction_2023_shop6 = max(convictions_2023_shop6)
min_conviction_2023_shop6 = min(convictions_2023_shop6)
conviction_range_2023_shop6 = max_conviction_2023_shop6 - min_conviction_2023_shop6
edge_widths_2023_shop6 = [(G_2023_shop6[u][v]['conviction'] - min_conviction_2023_shop6) / conviction_range_2023_shop6 * 30 + 0.5 for u, v in G_2023_shop6.edges()]  # Scale and offset widths

pos_2023_shop6 = nx.spring_layout(G_2023_shop6)

# Node sizes and colors as before
in_degrees_2023_shop6 = dict(G_2023_shop6.in_degree())
node_sizes_2023_shop6 = [in_degrees_2023_shop6[node] * 100 + 1000 for node in G_2023_shop6.nodes()]
normalized_sizes_2023_shop6 = (np.array(node_sizes_2023_shop6) - min(node_sizes_2023_shop6)) / (max(node_sizes_2023_shop6) - min(node_sizes_2023_shop6))
cmap_2023_shop6 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop6 = [cmap_2023_shop6(size) for size in normalized_sizes_2023_shop6]

# Drawing the network with conviction-based edge widths
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2023_shop6, pos_2023_shop6, node_size=node_sizes_2023_shop6, node_color=node_colors_2023_shop6, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop6, pos_2023_shop6, width=edge_widths_2023_shop6, edge_color='grey', alpha=0.5)
nx.draw_networkx_labels(G_2023_shop6, pos_2023_shop6, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 6 with Conviction-based Edge Thickness')
plt.axis('off')
plt.show()


In [ ]:
#the node sizes are based on the indegree where each node acts as a consequent,

In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2023_shop6_nonredundant_top30.loc[:, 'antecedents_'] = rules_2023_shop6_nonredundant_top30['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2023_shop6_nonredundant_top30.loc[:, 'consequents_'] = rules_2023_shop6_nonredundant_top30['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2023_shop6_nonredundant_top30, x='consequents_', y='antecedents_', s=rules_2023_shop6_nonredundant_top30['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2023_shop6_nonredundant_top30.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2023 Shop 6", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
print(rules_2023_shop6_nonredundant_top30[['antecedents', 'consequents']].head(31))

## Monetary Analysis of the Top Rules

In [ ]:
# Step 1: Extract unique product codes from the rules
unique_product_codes = set()
for idx, row in rules_2023_shop6_nonredundant_top30.iterrows():
    # Unite antecedents and consequents into a single set
    all_products = row['antecedents'].union(row['consequents'])
    unique_product_codes.update(all_products)

# Step 2: Create a DataFrame with unique product codes and their unit prices
# Filter sales_data for these product codes and drop duplicates to ensure each product code appears once
product_prices = sales_data[sales_data['product_code'].isin(unique_product_codes)][['product_code', 'unit_price']].drop_duplicates()

# Create a dictionary for quick lookup of prices
price_dict = product_prices.set_index('product_code')['unit_price'].to_dict()

# Step 3: Calculate the total price for each rule and collect data for the new DataFrame
data_for_new_df = []
for idx, row in rules_2023_shop6_nonredundant_top30.iterrows():
    # Combine sets of antecedents and consequents
    all_products = row['antecedents'].union(row['consequents'])
    total_price = sum(price_dict.get(prod, 0) for prod in all_products)  # Use get to avoid KeyError if any product code is missing
    data_for_new_df.append({
        'antecedents': row['antecedents'],
        'consequents': row['consequents'],
        'total_unit_price': total_price
    })

# Create a new DataFrame with only antecedents, consequents, and total unit price
rules_and_price_shop6_2023 = pd.DataFrame(data_for_new_df)

# Sort the DataFrame by total_unit_price in descending order
rules_and_price_shop6_2023_sorted = rules_and_price_shop6_2023.sort_values(by='total_unit_price', ascending=False)

# Print the sorted DataFrame to see the specified columns along with their total unit prices
print(rules_and_price_shop6_2023_sorted)


In [ ]:
# Convert 'trx_date' to datetime format if not already done so
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Add a 'year' column if it doesn't exist
sales_data['year'] = sales_data['trx_date'].dt.year

# Step 2: Filter sales_data for shop6 and the year 2023
filtered_sales_data_shop6_2023 = sales_data[(sales_data['shopID'] == 6) & (sales_data['year'] == 2023)]

# Step 3: Initialize a list to hold the rule data including total sales value
rule_sales_summaries_shop6_2023 = []

# Step 4: Iterate through each rule in the DataFrame
for index, rule in rules_2023_shop6_nonredundant_top30.iterrows():
    all_products = rule['antecedents'].union(rule['consequents'])

    # Filter the sales data for transactions that include all products in the rule
    # This step ensures that we are only considering transactions that contain all specified products
    mask = filtered_sales_data_shop6_2023['product_code'].isin(all_products)
    transaction_ids = filtered_sales_data_shop6_2023[mask].groupby('trx_id').filter(lambda x: all_products.issubset(set(x['product_code']))).trx_id.unique()
    matched_transactions = filtered_sales_data_shop6_2023[filtered_sales_data_shop6_2023['trx_id'].isin(transaction_ids)]
    
    # Step 5: Sum the total_value of these transactions
    total_sales_value = matched_transactions['total_value'].sum()

    # Append the result to the list
    rule_sales_summaries_shop6_2023.append({
        'antecedents': rule['antecedents'],
        'consequents': rule['consequents'],
        'total_sales_value': total_sales_value
    })

# Step 6: Convert the list to a DataFrame
rules_sales_values_df_shop6_2023 = pd.DataFrame(rule_sales_summaries_shop6_2023)

# Step 7: Sort by total_sales_value in descending order
sorted_rules_sales_values_df_shop6_2023 = rules_sales_values_df_shop6_2023.sort_values(by='total_sales_value', ascending=False)

# Step 8: Print the sorted DataFrame
print(sorted_rules_sales_values_df_shop6_2023)


## Narrowing down based on Lift

In [ ]:
rules_2023_shop6_nonredundant_top30_lift['LHS'] = [','.join(list(x)) for x in rules_2023_shop6_nonredundant_top30_lift['antecedents']]
rules_2023_shop6_nonredundant_top30_lift['RHS'] = [','.join(list(x)) for x in rules_2023_shop6_nonredundant_top30_lift['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2023_shop6_nonredundant_top30_lift.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2023_shop6_nonredundant_top30_lift, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2023 Shop 6")
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop6_lift = nx.DiGraph()

lift_summary_2023_shop6 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop6_nonredundant_top30_lift.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop6_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop6:
        lift_summary_2023_shop6[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop6[consequents]['count'] += 1
    else:
        lift_summary_2023_shop6[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop6:
    lift_summary_2023_shop6[item]['avg_lift'] = lift_summary_2023_shop6[item]['total_lift'] / lift_summary_2023_shop6[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop6_lift = [lift_summary_2023_shop6[node]['avg_lift'] * 500 if node in lift_summary_2023_shop6 else 1000 for node in G_2023_shop6_lift.nodes()]

pos_2023_shop6_lift = nx.spring_layout(G_2023_shop6_lift)

# Colormap from orange to red
cmap_2023_shop6_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop6_lift = np.array(node_sizes_2023_shop6_lift)
normalized_sizes_2023_shop6_lift = (normalized_sizes_2023_shop6_lift - min(normalized_sizes_2023_shop6_lift)) / (max(normalized_sizes_2023_shop6_lift) - min(normalized_sizes_2023_shop6_lift))
node_colors_2023_shop6_lift = [cmap_2023_shop6_lift(size) for size in normalized_sizes_2023_shop6_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2023_shop6_lift, pos_2023_shop6_lift, node_size=node_sizes_2023_shop6_lift, node_color=node_colors_2023_shop6_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop6_lift, pos_2023_shop6_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop6_lift, pos_2023_shop6_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 6 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2023_shop6_nonredundant_top30_lift.loc[:, 'antecedents_'] = rules_2023_shop6_nonredundant_top30_lift['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2023_shop6_nonredundant_top30_lift.loc[:, 'consequents_'] = rules_2023_shop6_nonredundant_top30_lift['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2023_shop6_nonredundant_top30_lift, x='consequents_', y='antecedents_', s=rules_2023_shop6_nonredundant_top30_lift['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2023_shop6_nonredundant_top30_lift.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2023 Shop 6", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
# Print only the 'antecedents' and 'consequents' columns of the top 10 rules
print(rules_2023_shop6_nonredundant_top30_lift[['antecedents', 'consequents']].head(10))


# Shop7 2021 SEARCH 12

In [ ]:
# Filter for records from shop ID 1 and the year 1993
sales_data_2021_shop7 = sales_data_shop7[(sales_data_shop7['trx_date'].dt.year == 2021)]

In [ ]:
sales_data_2021_shop7.head()

In [ ]:
# Grouping product codes by trx_id
transactions_2021_shop7 = sales_data_2021_shop7.groupby('trx_id')['product_code'].apply(list).tolist()

In [ ]:
# Encode transaction data
encoder_2021_shop7 = TransactionEncoder()
trans_array_2021_shop7 = encoder_2021_shop7.fit(transactions_2021_shop7).transform(transactions_2021_shop7)
trans_df_2021_shop7 = pd.DataFrame(trans_array_2021_shop7, columns=encoder_2021_shop7.columns_)

In [ ]:
# Find frequent itemsets
frequent_itemsets_2021_shop7= apriori(trans_df_2021_shop7, min_support=0.001, use_colnames=True)

In [ ]:
print(frequent_itemsets_2021_shop7)

In [ ]:
## Generate the association rules - by confidence
rulesConfidence_2021_shop7 = association_rules(frequent_itemsets_2021_shop7, metric="confidence", min_threshold=0.10)
rulesConfidence_2021_shop7.sort_values(by='confidence', ascending=False, inplace=True)


In [ ]:
# Note: This code assumes you have already loaded and processed the dataframe 'rulesConfidence_2021_shop7'.

# Replace frozen sets with strings
rulesConfidence_2021_shop7['antecedents_'] = rulesConfidence_2021_shop7['antecedents'].apply(lambda a: ','.join(list(a)))
rulesConfidence_2021_shop7['consequents_'] = rulesConfidence_2021_shop7['consequents'].apply(lambda a: ','.join(list(a)))

# Draw
fig, ax = plt.subplots(figsize=(14, 30))
ax = plt.scatter(data=rulesConfidence_2021_shop7, x='consequents_', y='antecedents_', s=rulesConfidence_2021_shop7['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rulesConfidence_2021_shop7.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2021 Shop 7", fontsize=14)
plt.xlabel("Consequents (RHS)", fontsize=12)
plt.ylabel("Antecedents (LHS)", fontsize=12)
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate(rotation=45)
plt.tight_layout()

plt.show()


## Eliminate Redundant Rules

In [ ]:
rulesConfidence_2021_shop7.head()

In [ ]:
rulesConfidence_2021_shop7.info()

In [ ]:
# If 'antecedents' and 'consequents' are not frozensets, convert them
rules_2021_shop7_redundant = rulesConfidence_2021_shop7.copy()
rules_2021_shop7_redundant['antecedents'] = rules_2021_shop7_redundant['antecedents'].apply(lambda x: frozenset(x))
rules_2021_shop7_redundant['consequents'] = rules_2021_shop7_redundant['consequents'].apply(lambda x: frozenset(x))


In [ ]:
# Create a copy of the DataFrame to avoid modifying the original while iterating
rules_2021_shop7_checkredundance = rules_2021_shop7_redundant.copy()

# List to hold non-redundant rules
non_redundant_rules_2021_shop7 = []

for _, rule in rules_2021_shop7_checkredundance.iterrows():
    if not is_redundant(rule, rules_2021_shop7_checkredundance):
        non_redundant_rules_2021_shop7.append(rule)

# Convert the list back to a DataFrame
rules_2021_shop7_nonredundant = pd.DataFrame(non_redundant_rules_2021_shop7).reset_index(drop=True)


In [ ]:
print(rules_2021_shop7_nonredundant.head())
print(f"Original number of rules: {len(rules_2021_shop7_redundant)}")
print(f"Number of non-redundant rules: {len(rules_2021_shop7_nonredundant)}")


In [ ]:
rules_2021_shop7_nonredundant.head()

In [ ]:
rules_2021_shop7_redundant.info()

In [ ]:
rules_2021_shop7_nonredundant.head()

In [ ]:
rules_2021_shop7_nonredundant_backup = rules_2021_shop7_nonredundant.copy()

In [ ]:
rules_2021_shop7_nonredundant_unfiltered = rules_2021_shop7_nonredundant.copy()

In [ ]:
#reset
#rules_2021_shop7_nonredundant = rules_2021_shop7_nonredundant_backup.copy()

# Redundant and Unfiltered

In [ ]:
rules_2021_shop7_redundant.head()

In [ ]:
G_2021_shop7 = nx.DiGraph()

# Iterate through rules for 2021 Shop 7 and add edges to the graph
for _, row in rules_2021_shop7_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as weight
    G_2021_shop7.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2021_shop7 = nx.spring_layout(G_2021_shop7)

# Recalculate node sizes based on indegree or another metric
in_degrees_2021_shop7 = dict(G_2021_shop7.in_degree())
node_sizes_2021_shop7 = [in_degrees_2021_shop7[node] * 100 + 1000 for node in G_2021_shop7.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2021_shop7 = (np.array(node_sizes_2021_shop7) - np.min(node_sizes_2021_shop7)) / (np.max(node_sizes_2021_shop7) - np.min(node_sizes_2021_shop7))

# Define a colormap from orange to red for visual consistency
cmap_2021_shop7 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2021_shop7 = [cmap_2021_shop7(size) for size in normalized_sizes_2021_shop7]

# Drawing the network
plt.figure(figsize=(60, 43))
nx.draw_networkx_nodes(G_2021_shop7, pos_2021_shop7, node_size=node_sizes_2021_shop7, node_color=node_colors_2021_shop7, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop7, pos_2021_shop7, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2021_shop7, pos_2021_shop7, font_size=10)

plt.title('Top 5 Association Rules Network for 2021 Shop 7 with Color Gradient')
plt.axis('off')
plt.show()


### Based on LIFT

In [ ]:
G_2021_shop7_lift = nx.DiGraph()

lift_summary_2021_shop7 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2021_shop7_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop7_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2021_shop7:
        lift_summary_2021_shop7[consequents]['total_lift'] += row['lift']
        lift_summary_2021_shop7[consequents]['count'] += 1
    else:
        lift_summary_2021_shop7[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2021_shop7:
    lift_summary_2021_shop7[item]['avg_lift'] = lift_summary_2021_shop7[item]['total_lift'] / lift_summary_2021_shop7[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2021_shop7_lift = [lift_summary_2021_shop7[node]['avg_lift'] * 500 if node in lift_summary_2021_shop7 else 1000 for node in G_2021_shop7_lift.nodes()]

pos_2021_shop7_lift = nx.spring_layout(G_2021_shop7_lift)

# Colormap from orange to red
cmap_2021_shop7_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2021_shop7_lift = np.array(node_sizes_2021_shop7_lift)
normalized_sizes_2021_shop7_lift = (normalized_sizes_2021_shop7_lift - min(normalized_sizes_2021_shop7_lift)) / (max(normalized_sizes_2021_shop7_lift) - min(normalized_sizes_2021_shop7_lift))
node_colors_2021_shop7_lift = [cmap_2021_shop7_lift(size) for size in normalized_sizes_2021_shop7_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2021_shop7_lift, pos_2021_shop7_lift, node_size=node_sizes_2021_shop7_lift, node_color=node_colors_2021_shop7_lift, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop7_lift, pos_2021_shop7_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2021_shop7_lift, pos_2021_shop7_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2021 Shop 7 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


# nonredundant_unfiltered

## Based on Indegree

In [ ]:
G_2021_shop7_nonredundant_unfiltered = nx.DiGraph()

# Iterating through each row to add edges with confidence as an attribute
for _, row in rules_2021_shop7_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop7_nonredundant_unfiltered.add_edge(antecedents, consequents, weight=row['confidence'])

# Defining the layout for network visualization
pos_2021_shop7_nonredundant_unfiltered = nx.spring_layout(G_2021_shop7_nonredundant_unfiltered)

# Calculating node sizes based on the indegree for color mapping
in_degrees_2021_shop7_nonredundant_unfiltered = dict(G_2021_shop7_nonredundant_unfiltered.in_degree())
node_sizes_2021_shop7_nonredundant_unfiltered = [in_degrees_2021_shop7_nonredundant_unfiltered[node] * 100 + 1000 for node in G_2021_shop7_nonredundant_unfiltered.nodes()]

# Normalizing node sizes for color mapping
normalized_sizes_2021_shop7_nonredundant_unfiltered = (np.array(node_sizes_2021_shop7_nonredundant_unfiltered) - min(node_sizes_2021_shop7_nonredundant_unfiltered)) / (max(node_sizes_2021_shop7_nonredundant_unfiltered) - min(node_sizes_2021_shop7_nonredundant_unfiltered))

# Defining a color map from orange to red
cmap_2021_shop7_nonredundant_unfiltered = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])

# Mapping normalized node sizes to colors
node_colors_2021_shop7_nonredundant_unfiltered = [cmap_2021_shop7_nonredundant_unfiltered(size) for size in normalized_sizes_2021_shop7_nonredundant_unfiltered]

# Drawing the network with node sizes and colors
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2021_shop7_nonredundant_unfiltered, pos_2021_shop7_nonredundant_unfiltered, node_size=node_sizes_2021_shop7_nonredundant_unfiltered, node_color=node_colors_2021_shop7_nonredundant_unfiltered, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop7_nonredundant_unfiltered, pos_2021_shop7_nonredundant_unfiltered, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2021_shop7_nonredundant_unfiltered, pos_2021_shop7_nonredundant_unfiltered, font_size=10)

plt.title('Node Size Based on Indegree for 2021 Shop 7')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2021_shop7_lift = nx.DiGraph()

lift_summary_2021_shop7 = {}

for _, row in rules_2021_shop7_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop7_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Update lift summary for consequents
    if consequents in lift_summary_2021_shop7:
        lift_summary_2021_shop7[consequents]['total_lift'] += row['lift']
        lift_summary_2021_shop7[consequents]['count'] += 1
    else:
        lift_summary_2021_shop7[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculate average lift for each consequent
for item in lift_summary_2021_shop7:
    lift_summary_2021_shop7[item]['avg_lift'] = lift_summary_2021_shop7[item]['total_lift'] / lift_summary_2021_shop7[item]['count']

# Node sizes based on average lift, scaled appropriately
node_sizes_2021_shop7_lift = [lift_summary_2021_shop7[node]['avg_lift'] * 500 if node in lift_summary_2021_shop7 else 1000 for node in G_2021_shop7_lift.nodes()]

pos_2021_shop7_lift = nx.spring_layout(G_2021_shop7_lift)

# Use a colormap that transitions from orange to red
cmap_2021_shop7_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2021_shop7_lift = np.array(node_sizes_2021_shop7_lift)
normalized_sizes_2021_shop7_lift = (normalized_sizes_2021_shop7_lift - min(normalized_sizes_2021_shop7_lift)) / (max(normalized_sizes_2021_shop7_lift) - min(normalized_sizes_2021_shop7_lift))
node_colors_2021_shop7_lift = [cmap_2021_shop7_lift(size) for size in normalized_sizes_2021_shop7_lift]

plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2021_shop7_lift, pos_2021_shop7_lift, node_size=node_sizes_2021_shop7_lift, node_color=node_colors_2021_shop7_lift, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop7_lift, pos_2021_shop7_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2021_shop7_lift, pos_2021_shop7_lift, font_size=10)

plt.title('Node Size Based on Average Lift for 2021 Shop 7')
plt.axis('off')
plt.show()


# Based on conviction

In [ ]:
rules_2021_shop7_nonredundant.head()

In [ ]:
G_2021_shop7_conviction = nx.DiGraph()

conviction_summary_2021_shop7 = {}
count_summary_2021_shop7 = {}

# Assuming 'rules_2021_shop7_nonredundant_unfiltered' contains a 'conviction' column
for _, row in rules_2021_shop7_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop7_conviction.add_edge(antecedents, consequents, weight=row['confidence'], conviction=row['conviction'])
    
    # Aggregate conviction values and count occurrences
    if consequents in conviction_summary_2021_shop7:
        conviction_summary_2021_shop7[consequents] += row['conviction']
        count_summary_2021_shop7[consequents] += 1
    else:
        conviction_summary_2021_shop7[consequents] = row['conviction']
        count_summary_2021_shop7[consequents] = 1

# Calculate average conviction values
average_conviction_summary_2021_shop7 = {k: conviction_summary_2021_shop7[k] / count_summary_2021_shop7[k] for k in conviction_summary_2021_shop7}

# Normalize conviction values for sizing and coloring
max_conviction = max(average_conviction_summary_2021_shop7.values())
min_conviction = min(average_conviction_summary_2021_shop7.values())
conviction_range = max_conviction - min_conviction
node_colors = []
node_sizes = []

for node in G_2021_shop7_conviction.nodes():
    # Get the average conviction value for the node, defaulting to min if not present
    conviction = average_conviction_summary_2021_shop7.get(node, min_conviction)
    # Normalize between 0 and 1
    normalized_conviction = (conviction - min_conviction) / conviction_range if conviction_range else 0
    node_colors.append(normalized_conviction)
    node_sizes.append(normalized_conviction * 1000 + 100)  # Scale and offset size

# Use a colormap for coloring nodes based on conviction
cmap = plt.cm.coolwarm

pos = nx.spring_layout(G_2021_shop7_conviction)

plt.figure(figsize=(40, 22))
nodes = nx.draw_networkx_nodes(G_2021_shop7_conviction, pos, node_color=node_colors, node_size=node_sizes, cmap=cmap, alpha=0.8)
edges = nx.draw_networkx_edges(G_2021_shop7_conviction, pos, arrowstyle='->', arrowsize=10, edge_color='grey', alpha=0.5)
labels = nx.draw_networkx_labels(G_2021_shop7_conviction, pos, font_size=8)

plt.colorbar(nodes, label='Normalized Conviction')
plt.title('Network Graph of 2021 Shop 7 Based on Conviction')
plt.axis('off')
plt.show()

In [ ]:
print(max_conviction)

# Filtered top 30 and non redundant

In [ ]:
rules_2021_shop7_nonredundant.head()

In [ ]:
# Assuming rules_1992_shop2_nonredundant is your sorted rules DataFrame for shop2
rules_2021_shop7_nonredundant_top30 = rules_2021_shop7_nonredundant.copy(deep=True)

In [ ]:
rules_2021_shop7_nonredundant_top30_lift = rules_2021_shop7_nonredundant.copy(deep=True)

In [ ]:
# Select the top 30 rules based on lift or another criterion
rules_2021_shop7_nonredundant_top30_lift = rules_2021_shop7_nonredundant_top30_lift.sort_values(by='lift', ascending=False).head(30)

In [ ]:
rules_2021_shop7_nonredundant_top30_lift.head()

In [ ]:
# Select the top 30 rules based on confidence or another criterion
rules_2021_shop7_nonredundant_top30 = rules_2021_shop7_nonredundant_top30.sort_values(by='confidence', ascending=False).head(30)

In [ ]:
rules_2021_shop7_nonredundant_top30.info()

In [ ]:
rules_2021_shop7_nonredundant_top30['LHS'] = [','.join(list(x)) for x in rules_2021_shop7_nonredundant_top30['antecedents']]
rules_2021_shop7_nonredundant_top30['RHS'] = [','.join(list(x)) for x in rules_2021_shop7_nonredundant_top30['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2021_shop7_nonredundant_top30.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2021_shop7_nonredundant_top30, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2021 Shop 7")
plt.axis('off')
plt.show()


## Based on Indegree

In [ ]:
G_2021_shop7 = nx.DiGraph()

# Assuming 'rules_sorted_2021_shop7' contains association rules for shop 7 in 2021
for _, row in rules_2021_shop7_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as the weight
    G_2021_shop7.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2021_shop7 = nx.spring_layout(G_2021_shop7)

# Calculate node sizes based on indegree
in_degrees_2021_shop7 = dict(G_2021_shop7.in_degree())
node_sizes_2021_shop7 = [in_degrees_2021_shop7[node] * 100 + 1000 for node in G_2021_shop7.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2021_shop7 = (np.array(node_sizes_2021_shop7) - np.min(node_sizes_2021_shop7)) / (np.max(node_sizes_2021_shop7) - np.min(node_sizes_2021_shop7))

# Define a colormap from orange to red
cmap_2021_shop7 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2021_shop7 = [cmap_2021_shop7(size) for size in normalized_sizes_2021_shop7]

# Drawing the network
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2021_shop7, pos_2021_shop7, node_size=node_sizes_2021_shop7, node_color=node_colors_2021_shop7, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop7, pos_2021_shop7, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2021_shop7, pos_2021_shop7, font_size=8)

plt.title('Top 5 Association Rules Network for 2021 Shop 7 with Color Gradient')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2021_shop7_lift = nx.DiGraph()

lift_summary_2021_shop7 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2021_shop7_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop7_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2021_shop7:
        lift_summary_2021_shop7[consequents]['total_lift'] += row['lift']
        lift_summary_2021_shop7[consequents]['count'] += 1
    else:
        lift_summary_2021_shop7[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2021_shop7:
    lift_summary_2021_shop7[item]['avg_lift'] = lift_summary_2021_shop7[item]['total_lift'] / lift_summary_2021_shop7[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2021_shop7_lift = [lift_summary_2021_shop7[node]['avg_lift'] * 500 if node in lift_summary_2021_shop7 else 1000 for node in G_2021_shop7_lift.nodes()]

pos_2021_shop7_lift = nx.spring_layout(G_2021_shop7_lift)

# Colormap from orange to red
cmap_2021_shop7_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2021_shop7_lift = np.array(node_sizes_2021_shop7_lift)
normalized_sizes_2021_shop7_lift = (normalized_sizes_2021_shop7_lift - min(normalized_sizes_2021_shop7_lift)) / (max(normalized_sizes_2021_shop7_lift) - min(normalized_sizes_2021_shop7_lift))
node_colors_2021_shop7_lift = [cmap_2021_shop7_lift(size) for size in normalized_sizes_2021_shop7_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2021_shop7_lift, pos_2021_shop7_lift, node_size=node_sizes_2021_shop7_lift, node_color=node_colors_2021_shop7_lift, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop7_lift, pos_2021_shop7_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2021_shop7_lift, pos_2021_shop7_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2021 Shop 7 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2021_shop7_lift = nx.DiGraph()

lift_summary_2021_shop7 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2021_shop7_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop7_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2021_shop7:
        lift_summary_2021_shop7[consequents]['total_lift'] += row['lift']
        lift_summary_2021_shop7[consequents]['count'] += 1
    else:
        lift_summary_2021_shop7[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2021_shop7:
    lift_summary_2021_shop7[item]['avg_lift'] = lift_summary_2021_shop7[item]['total_lift'] / lift_summary_2021_shop7[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2021_shop7_lift = [lift_summary_2021_shop7[node]['avg_lift'] * 500 if node in lift_summary_2021_shop7 else 1000 for node in G_2021_shop7_lift.nodes()]

pos_2021_shop7_lift = nx.spring_layout(G_2021_shop7_lift)

# Colormap from orange to red
cmap_2021_shop7_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2021_shop7_lift = np.array(node_sizes_2021_shop7_lift)
normalized_sizes_2021_shop7_lift = (normalized_sizes_2021_shop7_lift - min(normalized_sizes_2021_shop7_lift)) / (max(normalized_sizes_2021_shop7_lift) - min(normalized_sizes_2021_shop7_lift))
node_colors_2021_shop7_lift = [cmap_2021_shop7_lift(size) for size in normalized_sizes_2021_shop7_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2021_shop7_lift, pos_2021_shop7_lift, node_size=node_sizes_2021_shop7_lift, node_color=node_colors_2021_shop7_lift, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop7_lift, pos_2021_shop7_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2021_shop7_lift, pos_2021_shop7_lift, font_size=10)

plt.title('Top 5 Association Rules Network for 2021 Shop 7 Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2021_shop7 = nx.DiGraph()

# Assuming 'rules_2021_shop7_nonredundant_top30' contains association rules for 2021 Shop 7
for _, row in rules_2021_shop7_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop7.add_edge(antecedents, consequents, confidence=row['confidence'])

# Calculate node sizes based on indegree
in_degrees_2021_shop7 = dict(G_2021_shop7.in_degree())
node_sizes_2021_shop7 = [in_degrees_2021_shop7[node] * 100 + 1000 for node in G_2021_shop7.nodes()]

# Edge width based on confidence
edge_widths_2021_shop7 = [G_2021_shop7[u][v]['confidence'] * 15 for u, v in G_2021_shop7.edges()]

# Assuming normalized sizes and color mapping is desired
normalized_sizes_2021_shop7 = np.array(list(in_degrees_2021_shop7.values()))
normalized_sizes_2021_shop7 = (normalized_sizes_2021_shop7 - min(normalized_sizes_2021_shop7)) / (max(normalized_sizes_2021_shop7) - min(normalized_sizes_2021_shop7))
cmap_2021_shop7 = mcolors.LinearSegmentedColormap.from_list("MyCMapName", ["blue", "red"])

# Node colors based on a property (here, using indegree for example)
node_colors_2021_shop7 = [cmap_2021_shop7(size) for size in normalized_sizes_2021_shop7]

# Layout
pos_2021_shop7 = nx.spring_layout(G_2021_shop7, seed=42)

# Drawing
plt.figure(figsize=(40, 30))
nx.draw_networkx_edges(G_2021_shop7, pos_2021_shop7, width=edge_widths_2021_shop7, edge_color='grey', alpha=0.5)
nx.draw_networkx_nodes(G_2021_shop7, pos_2021_shop7, node_size=node_sizes_2021_shop7, node_color=node_colors_2021_shop7, alpha=0.8)
nx.draw_networkx_labels(G_2021_shop7, pos_2021_shop7, font_size=8, font_color='black')
plt.title('Network Graph of Association Rules Based on Confidence for 2021 Shop 7')
plt.axis('off')
plt.show()


In [ ]:
G_2021_shop7 = nx.DiGraph()

convictions_2021_shop7 = []  # To store conviction values for normalization

for _, row in rules_2021_shop7_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    conviction_2021_shop7 = row['conviction']  # Assuming each row has a 'conviction' column
    convictions_2021_shop7.append(conviction_2021_shop7)
    G_2021_shop7.add_edge(antecedents, consequents, weight=row['confidence'], conviction=conviction_2021_shop7)

# Normalize conviction values for edge width
max_conviction_2021_shop7 = max(convictions_2021_shop7)
min_conviction_2021_shop7 = min(convictions_2021_shop7)
conviction_range_2021_shop7 = max_conviction_2021_shop7 - min_conviction_2021_shop7
edge_widths_2021_shop7 = [(G_2021_shop7[u][v]['conviction'] - min_conviction_2021_shop7) / conviction_range_2021_shop7 * 30 + 0.5 for u, v in G_2021_shop7.edges()]  # Scale and offset widths

pos_2021_shop7 = nx.spring_layout(G_2021_shop7)

# Node sizes and colors as before
in_degrees_2021_shop7 = dict(G_2021_shop7.in_degree())
node_sizes_2021_shop7 = [in_degrees_2021_shop7[node] * 100 + 1000 for node in G_2021_shop7.nodes()]
normalized_sizes_2021_shop7 = (np.array(node_sizes_2021_shop7) - min(node_sizes_2021_shop7)) / (max(node_sizes_2021_shop7) - min(node_sizes_2021_shop7))
cmap_2021_shop7 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2021_shop7 = [cmap_2021_shop7(size) for size in normalized_sizes_2021_shop7]

# Drawing the network with conviction-based edge widths
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2021_shop7, pos_2021_shop7, node_size=node_sizes_2021_shop7, node_color=node_colors_2021_shop7, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop7, pos_2021_shop7, width=edge_widths_2021_shop7, edge_color='grey', alpha=0.5)
nx.draw_networkx_labels(G_2021_shop7, pos_2021_shop7, font_size=10)

plt.title('Top 5 Association Rules Network for 2021 Shop 7 with Conviction-based Edge Thickness')
plt.axis('off')
plt.show()


In [ ]:
#the node sizes are based on the indegree where each node acts as a consequent,

In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2021_shop7_nonredundant_top30.loc[:, 'antecedents_'] = rules_2021_shop7_nonredundant_top30['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2021_shop7_nonredundant_top30.loc[:, 'consequents_'] = rules_2021_shop7_nonredundant_top30['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2021_shop7_nonredundant_top30, x='consequents_', y='antecedents_', s=rules_2021_shop7_nonredundant_top30['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2021_shop7_nonredundant_top30.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2021 Shop 7", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
print(rules_2021_shop7_nonredundant_top30[['antecedents', 'consequents']].head(10))

## Monetary Analysis of the Top Rules

In [ ]:
# Step 1: Extract unique product codes from the rules
unique_product_codes = set()
for idx, row in rules_2021_shop7_nonredundant_top30.iterrows():
    # Unite antecedents and consequents into a single set
    all_products = row['antecedents'].union(row['consequents'])
    unique_product_codes.update(all_products)

# Step 2: Create a DataFrame with unique product codes and their unit prices
# Filter sales_data for these product codes and drop duplicates to ensure each product code appears once
product_prices = sales_data[sales_data['product_code'].isin(unique_product_codes)][['product_code', 'unit_price']].drop_duplicates()

# Create a dictionary for quick lookup of prices
price_dict = product_prices.set_index('product_code')['unit_price'].to_dict()

# Step 3: Calculate the total price for each rule and collect data for the new DataFrame
data_for_new_df = []
for idx, row in rules_2021_shop7_nonredundant_top30.iterrows():
    # Combine sets of antecedents and consequents
    all_products = row['antecedents'].union(row['consequents'])
    total_price = sum(price_dict.get(prod, 0) for prod in all_products)  # Use get to avoid KeyError if any product code is missing
    data_for_new_df.append({
        'antecedents': row['antecedents'],
        'consequents': row['consequents'],
        'total_unit_price': total_price
    })

# Create a new DataFrame with only antecedents, consequents, and total unit price
rules_and_price_shop7_2021 = pd.DataFrame(data_for_new_df)

# Sort the DataFrame by total_unit_price in descending order
rules_and_price_shop7_2021_sorted = rules_and_price_shop7_2021.sort_values(by='total_unit_price', ascending=False)

# Print the sorted DataFrame to see the specified columns along with their total unit prices
print(rules_and_price_shop7_2021_sorted)


In [ ]:
# Convert 'trx_date' to datetime format if not already done so
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Add a 'year' column if it doesn't exist
sales_data['year'] = sales_data['trx_date'].dt.year

# Step 2: Filter sales_data for shop7 and the year 2021
filtered_sales_data_2021 = sales_data[(sales_data['shopID'] == 7) & (sales_data['year'] == 2021)]

# Step 3: Initialize a list to hold the rule data including total sales value
rule_sales_summaries_2021 = []

# Step 4: Iterate through each rule in the DataFrame
for index, rule in rules_2021_shop7_nonredundant_top30.iterrows():
    all_products = rule['antecedents'].union(rule['consequents'])

    # Filter the sales data for transactions that include all products in the rule
    # This step ensures that we are only considering transactions that contain all specified products
    mask = filtered_sales_data_2021['product_code'].isin(all_products)
    transaction_ids = filtered_sales_data_2021[mask].groupby('trx_id').filter(lambda x: all_products.issubset(set(x['product_code']))).trx_id.unique()
    matched_transactions = filtered_sales_data_2021[filtered_sales_data_2021['trx_id'].isin(transaction_ids)]
    
    # Step 5: Sum the total_value of these transactions
    total_sales_value = matched_transactions['total_value'].sum()

    # Append the result to the list
    rule_sales_summaries_2021.append({
        'antecedents': rule['antecedents'],
        'consequents': rule['consequents'],
        'total_sales_value': total_sales_value
    })

# Step 6: Convert the list to a DataFrame
rules_sales_values_df_2021 = pd.DataFrame(rule_sales_summaries_2021)

# Step 7: Sort by total_sales_value in descending order
sorted_rules_sales_values_df_2021 = rules_sales_values_df_2021.sort_values(by='total_sales_value', ascending=False)

# Step 8: Print the sorted DataFrame
print(sorted_rules_sales_values_df_2021)


## Narrowing down based on Lift

In [ ]:
rules_2021_shop7_nonredundant_top30_lift['LHS'] = [','.join(list(x)) for x in rules_2021_shop7_nonredundant_top30_lift['antecedents']]
rules_2021_shop7_nonredundant_top30_lift['RHS'] = [','.join(list(x)) for x in rules_2021_shop7_nonredundant_top30_lift['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2021_shop7_nonredundant_top30_lift.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2021_shop7_nonredundant_top30_lift, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2021 Shop 7")
plt.axis('off')
plt.show()


In [ ]:
G_2021_shop7_lift = nx.DiGraph()

lift_summary_2021_shop7 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2021_shop7_nonredundant_top30_lift.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2021_shop7_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2021_shop7:
        lift_summary_2021_shop7[consequents]['total_lift'] += row['lift']
        lift_summary_2021_shop7[consequents]['count'] += 1
    else:
        lift_summary_2021_shop7[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2021_shop7:
    lift_summary_2021_shop7[item]['avg_lift'] = lift_summary_2021_shop7[item]['total_lift'] / lift_summary_2021_shop7[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2021_shop7_lift = [lift_summary_2021_shop7[node]['avg_lift'] * 500 if node in lift_summary_2021_shop7 else 1000 for node in G_2021_shop7_lift.nodes()]

pos_2021_shop7_lift = nx.spring_layout(G_2021_shop7_lift)

# Colormap from orange to red
cmap_2021_shop7_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2021_shop7_lift = np.array(node_sizes_2021_shop7_lift)
normalized_sizes_2021_shop7_lift = (normalized_sizes_2021_shop7_lift - min(normalized_sizes_2021_shop7_lift)) / (max(normalized_sizes_2021_shop7_lift) - min(normalized_sizes_2021_shop7_lift))
node_colors_2021_shop7_lift = [cmap_2021_shop7_lift(size) for size in normalized_sizes_2021_shop7_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2021_shop7_lift, pos_2021_shop7_lift, node_size=node_sizes_2021_shop7_lift, node_color=node_colors_2021_shop7_lift, alpha=0.8)
nx.draw_networkx_edges(G_2021_shop7_lift, pos_2021_shop7_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2021_shop7_lift, pos_2021_shop7_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2021 Shop 7 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2021_shop7_nonredundant_top30_lift.loc[:, 'antecedents_'] = rules_2021_shop7_nonredundant_top30_lift['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2021_shop7_nonredundant_top30_lift.loc[:, 'consequents_'] = rules_2021_shop7_nonredundant_top30_lift['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2021_shop7_nonredundant_top30_lift, x='consequents_', y='antecedents_', s=rules_2021_shop7_nonredundant_top30_lift['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2021_shop7_nonredundant_top30_lift.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2021 Shop 7", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
# Print only the 'antecedents' and 'consequents' columns of the top 10 rules
print(rules_2021_shop7_nonredundant_top30_lift[['antecedents', 'consequents']].head(10))


# Shop7 2022 SEARCH 13

In [ ]:
# Filter for records from shop ID 1 and the year 1993
sales_data_2022_shop7 = sales_data_shop7[(sales_data_shop7['trx_date'].dt.year == 2022)]

In [ ]:
sales_data_2022_shop7.head()

In [ ]:
# Grouping product codes by trx_id
transactions_2022_shop7 = sales_data_2022_shop7.groupby('trx_id')['product_code'].apply(list).tolist()

In [ ]:
# Encode transaction data
encoder_2022_shop7 = TransactionEncoder()
trans_array_2022_shop7 = encoder_2022_shop7.fit(transactions_2022_shop7).transform(transactions_2022_shop7)
trans_df_2022_shop7 = pd.DataFrame(trans_array_2022_shop7, columns=encoder_2022_shop7.columns_)

In [ ]:
# Find frequent itemsets
frequent_itemsets_2022_shop7 = apriori(trans_df_2022_shop7, min_support=0.001, use_colnames=True)

In [ ]:
print(frequent_itemsets_2022_shop7)

In [ ]:
## Generate the association rules - by confidence
rulesConfidence_2022_shop7 = association_rules(frequent_itemsets_2022_shop7, metric="confidence", min_threshold=0.10)
rulesConfidence_2022_shop7.sort_values(by='confidence', ascending=False, inplace=True)


In [ ]:
# Note: This is a hypothetical example, as I can't directly manipulate or access dataframes not provided within this code cell.

# Replace frozen sets with strings
rulesConfidence_2022_shop7['antecedents_'] = rulesConfidence_2022_shop7['antecedents'].apply(lambda a: ','.join(list(a)))
rulesConfidence_2022_shop7['consequents_'] = rulesConfidence_2022_shop7['consequents'].apply(lambda a: ','.join(list(a)))

# Draw
fig, ax = plt.subplots(figsize=(14,30))
ax = plt.scatter(data=rulesConfidence_2022_shop7, x='consequents_', y='antecedents_', s=rulesConfidence_2022_shop7['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rulesConfidence_2022_shop7.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2020 Shop 7", fontsize=14)
plt.xlabel("Consequents (RHS)", fontsize=12)
plt.ylabel("Antecedents (LHS)", fontsize=12)
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate(rotation=45)
plt.tight_layout()

plt.show()




## Eliminate Redundant Rules

In [ ]:
rulesConfidence_2022_shop7.head()

In [ ]:
rulesConfidence_2022_shop7.info()

In [ ]:
# If 'antecedents' and 'consequents' are not frozensets, convert them
rules_2022_shop7_redundant = rulesConfidence_2022_shop7.copy()
rules_2022_shop7_redundant['antecedents'] = rules_2022_shop7_redundant['antecedents'].apply(lambda x: frozenset(x))
rules_2022_shop7_redundant['consequents'] = rules_2022_shop7_redundant['consequents'].apply(lambda x: frozenset(x))


In [ ]:
# Create a copy of the DataFrame to avoid modifying the original while iterating
rules_2022_shop7_checkredundance = rules_2022_shop7_redundant.copy()

# List to hold non-redundant rules
non_redundant_rules_2022_shop7 = []

for _, rule in rules_2022_shop7_checkredundance.iterrows():
    if not is_redundant(rule, rules_2022_shop7_checkredundance):
        non_redundant_rules_2022_shop7.append(rule)

# Convert the list back to a DataFrame
rules_2022_shop7_nonredundant = pd.DataFrame(non_redundant_rules_2022_shop7).reset_index(drop=True)


In [ ]:
print(rules_2022_shop7_nonredundant.head())
print(f"Original number of rules: {len(rules_2022_shop7_redundant)}")
print(f"Number of non-redundant rules: {len(rules_2022_shop7_nonredundant)}")


In [ ]:
rules_2022_shop7_nonredundant.head()

In [ ]:
rules_2022_shop7_redundant.info()

In [ ]:
rules_2022_shop7_nonredundant.head()

In [ ]:
rules_2022_shop7_nonredundant_backup = rules_2022_shop7_nonredundant.copy()

In [ ]:
rules_2022_shop7_nonredundant_unfiltered = rules_2022_shop7_nonredundant.copy()

In [ ]:
#reset
#rules_2022_shop7_nonredundant = rules_2022_shop7_nonredundant_backup.copy()

# Redundant and Unfiltered

In [ ]:
rules_2022_shop7_redundant.head()

In [ ]:
G_2022_shop7 = nx.DiGraph()

# Iterate through rules for 2022 Shop 7 and add edges to the graph
for _, row in rules_2022_shop7_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as weight
    G_2022_shop7.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2022_shop7 = nx.spring_layout(G_2022_shop7)

# Recalculate node sizes based on indegree or another metric
in_degrees_2022_shop7 = dict(G_2022_shop7.in_degree())
node_sizes_2022_shop7 = [in_degrees_2022_shop7[node] * 100 + 1000 for node in G_2022_shop7.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2022_shop7 = (np.array(node_sizes_2022_shop7) - np.min(node_sizes_2022_shop7)) / (np.max(node_sizes_2022_shop7) - np.min(node_sizes_2022_shop7))

# Define a colormap from orange to red for visual consistency
cmap_2022_shop7 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2022_shop7 = [cmap_2022_shop7(size) for size in normalized_sizes_2022_shop7]

# Drawing the network
plt.figure(figsize=(60, 43))
nx.draw_networkx_nodes(G_2022_shop7, pos_2022_shop7, node_size=node_sizes_2022_shop7, node_color=node_colors_2022_shop7, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop7, pos_2022_shop7, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop7, pos_2022_shop7, font_size=10)

plt.title('Top 5 Association Rules Network for 2022 Shop 7 with Color Gradient')
plt.axis('off')
plt.show()


### Based on LIFT

In [ ]:
G_2022_shop7_lift = nx.DiGraph()

lift_summary_2022_shop7 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2022_shop7_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop7_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop7:
        lift_summary_2022_shop7[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop7[consequents]['count'] += 1
    else:
        lift_summary_2022_shop7[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop7:
    lift_summary_2022_shop7[item]['avg_lift'] = lift_summary_2022_shop7[item]['total_lift'] / lift_summary_2022_shop7[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop7_lift = [lift_summary_2022_shop7[node]['avg_lift'] * 500 if node in lift_summary_2022_shop7 else 1000 for node in G_2022_shop7_lift.nodes()]

pos_2022_shop7_lift = nx.spring_layout(G_2022_shop7_lift)

# Colormap from orange to red
cmap_2022_shop7_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop7_lift = np.array(node_sizes_2022_shop7_lift)
normalized_sizes_2022_shop7_lift = (normalized_sizes_2022_shop7_lift - min(normalized_sizes_2022_shop7_lift)) / (max(normalized_sizes_2022_shop7_lift) - min(normalized_sizes_2022_shop7_lift))
node_colors_2022_shop7_lift = [cmap_2022_shop7_lift(size) for size in normalized_sizes_2022_shop7_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2022_shop7_lift, pos_2022_shop7_lift, node_size=node_sizes_2022_shop7_lift, node_color=node_colors_2022_shop7_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop7_lift, pos_2022_shop7_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop7_lift, pos_2022_shop7_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2022 Shop 7 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


# nonredundant_unfiltered

## Based on Indegree

In [ ]:
G_2022_shop7_nonredundant_unfiltered = nx.DiGraph()

# Iterating through each row to add edges with confidence as an attribute
for _, row in rules_2022_shop7_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop7_nonredundant_unfiltered.add_edge(antecedents, consequents, weight=row['confidence'])

# Defining the layout for network visualization
pos_2022_shop7_nonredundant_unfiltered = nx.spring_layout(G_2022_shop7_nonredundant_unfiltered)

# Calculating node sizes based on the indegree for color mapping
in_degrees_2022_shop7_nonredundant_unfiltered = dict(G_2022_shop7_nonredundant_unfiltered.in_degree())
node_sizes_2022_shop7_nonredundant_unfiltered = [in_degrees_2022_shop7_nonredundant_unfiltered[node] * 100 + 1000 for node in G_2022_shop7_nonredundant_unfiltered.nodes()]

# Normalizing node sizes for color mapping
normalized_sizes_2022_shop7_nonredundant_unfiltered = (np.array(node_sizes_2022_shop7_nonredundant_unfiltered) - min(node_sizes_2022_shop7_nonredundant_unfiltered)) / (max(node_sizes_2022_shop7_nonredundant_unfiltered) - min(node_sizes_2022_shop7_nonredundant_unfiltered))

# Defining a color map from orange to red
cmap_2022_shop7_nonredundant_unfiltered = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])

# Mapping normalized node sizes to colors
node_colors_2022_shop7_nonredundant_unfiltered = [cmap_2022_shop7_nonredundant_unfiltered(size) for size in normalized_sizes_2022_shop7_nonredundant_unfiltered]

# Drawing the network with node sizes and colors
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2022_shop7_nonredundant_unfiltered, pos_2022_shop7_nonredundant_unfiltered, node_size=node_sizes_2022_shop7_nonredundant_unfiltered, node_color=node_colors_2022_shop7_nonredundant_unfiltered, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop7_nonredundant_unfiltered, pos_2022_shop7_nonredundant_unfiltered, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2022_shop7_nonredundant_unfiltered, pos_2022_shop7_nonredundant_unfiltered, font_size=10)

plt.title('Node Size Based on Indegree for 2022 Shop 7')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2022_shop7_lift = nx.DiGraph()

lift_summary_2022_shop7 = {}

for _, row in rules_2022_shop7_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop7_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Update lift summary for consequents
    if consequents in lift_summary_2022_shop7:
        lift_summary_2022_shop7[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop7[consequents]['count'] += 1
    else:
        lift_summary_2022_shop7[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculate average lift for each consequent
for item in lift_summary_2022_shop7:
    lift_summary_2022_shop7[item]['avg_lift'] = lift_summary_2022_shop7[item]['total_lift'] / lift_summary_2022_shop7[item]['count']

# Node sizes based on average lift, scaled appropriately
node_sizes_2022_shop7_lift = [lift_summary_2022_shop7[node]['avg_lift'] * 500 if node in lift_summary_2022_shop7 else 1000 for node in G_2022_shop7_lift.nodes()]

pos_2022_shop7_lift = nx.spring_layout(G_2022_shop7_lift)

# Use a colormap that transitions from orange to red
cmap_2022_shop7_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop7_lift = np.array(node_sizes_2022_shop7_lift)
normalized_sizes_2022_shop7_lift = (normalized_sizes_2022_shop7_lift - min(normalized_sizes_2022_shop7_lift)) / (max(normalized_sizes_2022_shop7_lift) - min(normalized_sizes_2022_shop7_lift))
node_colors_2022_shop7_lift = [cmap_2022_shop7_lift(size) for size in normalized_sizes_2022_shop7_lift]

plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2022_shop7_lift, pos_2022_shop7_lift, node_size=node_sizes_2022_shop7_lift, node_color=node_colors_2022_shop7_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop7_lift, pos_2022_shop7_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop7_lift, pos_2022_shop7_lift, font_size=10)

plt.title('Node Size Based on Average Lift for 2022 Shop 7')
plt.axis('off')
plt.show()


# Based on conviction

In [ ]:
rules_2022_shop7_nonredundant.head()

In [ ]:
G_2022_shop7_conviction = nx.DiGraph()

conviction_summary_2022_shop7 = {}
count_summary_2022_shop7 = {}

# Assuming 'rules_2022_shop7_nonredundant_unfiltered' contains a 'conviction' column
for _, row in rules_2022_shop7_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop7_conviction.add_edge(antecedents, consequents, weight=row['confidence'], conviction=row['conviction'])
    
    # Aggregate conviction values and count occurrences
    if consequents in conviction_summary_2022_shop7:
        conviction_summary_2022_shop7[consequents] += row['conviction']
        count_summary_2022_shop7[consequents] += 1
    else:
        conviction_summary_2022_shop7[consequents] = row['conviction']
        count_summary_2022_shop7[consequents] = 1

# Calculate average conviction values
average_conviction_summary_2022_shop7 = {k: conviction_summary_2022_shop7[k] / count_summary_2022_shop7[k] for k in conviction_summary_2022_shop7}

# Normalize conviction values for sizing and coloring
max_conviction = max(average_conviction_summary_2022_shop7.values())
min_conviction = min(average_conviction_summary_2022_shop7.values())
conviction_range = max_conviction - min_conviction
node_colors = []
node_sizes = []

for node in G_2022_shop7_conviction.nodes():
    # Get the average conviction value for the node, defaulting to min if not present
    conviction = average_conviction_summary_2022_shop7.get(node, min_conviction)
    # Normalize between 0 and 1
    normalized_conviction = (conviction - min_conviction) / conviction_range if conviction_range else 0
    node_colors.append(normalized_conviction)
    node_sizes.append(normalized_conviction * 1000 + 100)  # Scale and offset size

# Use a colormap for coloring nodes based on conviction
cmap = plt.cm.coolwarm

pos = nx.spring_layout(G_2022_shop7_conviction)

plt.figure(figsize=(40, 22))
nodes = nx.draw_networkx_nodes(G_2022_shop7_conviction, pos, node_color=node_colors, node_size=node_sizes, cmap=cmap, alpha=0.8)
edges = nx.draw_networkx_edges(G_2022_shop7_conviction, pos, arrowstyle='->', arrowsize=10, edge_color='grey', alpha=0.5)
labels = nx.draw_networkx_labels(G_2022_shop7_conviction, pos, font_size=8)

plt.colorbar(nodes, label='Normalized Conviction')
plt.title('Network Graph of 2022 Shop 7 Based on Conviction')
plt.axis('off')
plt.show()

In [ ]:
print(max_conviction)

# Filtered top 30 and non redundant

In [ ]:
rules_2022_shop7_nonredundant.head()

In [ ]:
# Assuming rules_1992_shop2_nonredundant is your sorted rules DataFrame for shop2
rules_2022_shop7_nonredundant_top30 = rules_2022_shop7_nonredundant.copy(deep=True)

In [ ]:
rules_2022_shop7_nonredundant_top30_lift = rules_2022_shop7_nonredundant.copy(deep=True)

In [ ]:
# Select the top 30 rules based on lift or another criterion
rules_2022_shop7_nonredundant_top30_lift = rules_2022_shop7_nonredundant_top30_lift.sort_values(by='lift', ascending=False).head(30)

In [ ]:
rules_2022_shop7_nonredundant_top30_lift.head()

In [ ]:
# Select the top 30 rules based on confidence or another criterion
rules_2022_shop7_nonredundant_top30 = rules_2022_shop7_nonredundant_top30.sort_values(by='confidence', ascending=False).head(30)

In [ ]:
rules_2022_shop7_nonredundant_top30.info()

In [ ]:
rules_2022_shop7_nonredundant_top30['LHS'] = [','.join(list(x)) for x in rules_2022_shop7_nonredundant_top30['antecedents']]
rules_2022_shop7_nonredundant_top30['RHS'] = [','.join(list(x)) for x in rules_2022_shop7_nonredundant_top30['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2022_shop7_nonredundant_top30.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2022_shop7_nonredundant_top30, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2022 Shop 7")
plt.axis('off')
plt.show()


## Based on Indegree

In [ ]:
G_2022_shop7 = nx.DiGraph()

# Assuming 'rules_sorted_2022_shop7' contains association rules for shop 7 in 2022
for _, row in rules_2022_shop7_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as the weight
    G_2022_shop7.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2022_shop7 = nx.spring_layout(G_2022_shop7)

# Calculate node sizes based on indegree
in_degrees_2022_shop7 = dict(G_2022_shop7.in_degree())
node_sizes_2022_shop7 = [in_degrees_2022_shop7[node] * 100 + 1000 for node in G_2022_shop7.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2022_shop7 = (np.array(node_sizes_2022_shop7) - np.min(node_sizes_2022_shop7)) / (np.max(node_sizes_2022_shop7) - np.min(node_sizes_2022_shop7))

# Define a colormap from orange to red
cmap_2022_shop7 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2022_shop7 = [cmap_2022_shop7(size) for size in normalized_sizes_2022_shop7]

# Drawing the network
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2022_shop7, pos_2022_shop7, node_size=node_sizes_2022_shop7, node_color=node_colors_2022_shop7, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop7, pos_2022_shop7, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2022_shop7, pos_2022_shop7, font_size=8)

plt.title('Top 5 Association Rules Network for 2022 Shop 7 with Color Gradient')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2022_shop7_lift = nx.DiGraph()

lift_summary_2022_shop7 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2022_shop7_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop7_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop7:
        lift_summary_2022_shop7[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop7[consequents]['count'] += 1
    else:
        lift_summary_2022_shop7[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop7:
    lift_summary_2022_shop7[item]['avg_lift'] = lift_summary_2022_shop7[item]['total_lift'] / lift_summary_2022_shop7[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop7_lift = [lift_summary_2022_shop7[node]['avg_lift'] * 500 if node in lift_summary_2022_shop7 else 1000 for node in G_2022_shop7_lift.nodes()]

pos_2022_shop7_lift = nx.spring_layout(G_2022_shop7_lift)

# Colormap from orange to red
cmap_2022_shop7_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop7_lift = np.array(node_sizes_2022_shop7_lift)
normalized_sizes_2022_shop7_lift = (normalized_sizes_2022_shop7_lift - min(normalized_sizes_2022_shop7_lift)) / (max(normalized_sizes_2022_shop7_lift) - min(normalized_sizes_2022_shop7_lift))
node_colors_2022_shop7_lift = [cmap_2022_shop7_lift(size) for size in normalized_sizes_2022_shop7_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2022_shop7_lift, pos_2022_shop7_lift, node_size=node_sizes_2022_shop7_lift, node_color=node_colors_2022_shop7_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop7_lift, pos_2022_shop7_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop7_lift, pos_2022_shop7_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2022 Shop 7 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop7_lift = nx.DiGraph()

lift_summary_2022_shop7 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2022_shop7_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop7_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop7:
        lift_summary_2022_shop7[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop7[consequents]['count'] += 1
    else:
        lift_summary_2022_shop7[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop7:
    lift_summary_2022_shop7[item]['avg_lift'] = lift_summary_2022_shop7[item]['total_lift'] / lift_summary_2022_shop7[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop7_lift = [lift_summary_2022_shop7[node]['avg_lift'] * 500 if node in lift_summary_2022_shop7 else 1000 for node in G_2022_shop7_lift.nodes()]

pos_2022_shop7_lift = nx.spring_layout(G_2022_shop7_lift)

# Colormap from orange to red
cmap_2022_shop7_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop7_lift = np.array(node_sizes_2022_shop7_lift)
normalized_sizes_2022_shop7_lift = (normalized_sizes_2022_shop7_lift - min(normalized_sizes_2022_shop7_lift)) / (max(normalized_sizes_2022_shop7_lift) - min(normalized_sizes_2022_shop7_lift))
node_colors_2022_shop7_lift = [cmap_2022_shop7_lift(size) for size in normalized_sizes_2022_shop7_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2022_shop7_lift, pos_2022_shop7_lift, node_size=node_sizes_2022_shop7_lift, node_color=node_colors_2022_shop7_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop7_lift, pos_2022_shop7_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop7_lift, pos_2022_shop7_lift, font_size=10)

plt.title('Top 5 Association Rules Network for 2022 Shop 7 Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop7 = nx.DiGraph()

# Assuming 'rules_2022_shop7_nonredundant_top30' contains association rules for 2022 Shop 7
for _, row in rules_2022_shop7_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop7.add_edge(antecedents, consequents, confidence=row['confidence'])

# Calculate node sizes based on indegree
in_degrees_2022_shop7 = dict(G_2022_shop7.in_degree())
node_sizes_2022_shop7 = [in_degrees_2022_shop7[node] * 100 + 1000 for node in G_2022_shop7.nodes()]

# Edge width based on confidence
edge_widths_2022_shop7 = [G_2022_shop7[u][v]['confidence'] * 15 for u, v in G_2022_shop7.edges()]

# Assuming normalized sizes and color mapping is desired
normalized_sizes_2022_shop7 = np.array(list(in_degrees_2022_shop7.values()))
normalized_sizes_2022_shop7 = (normalized_sizes_2022_shop7 - min(normalized_sizes_2022_shop7)) / (max(normalized_sizes_2022_shop7) - min(normalized_sizes_2022_shop7))
cmap_2022_shop7 = mcolors.LinearSegmentedColormap.from_list("MyCMapName", ["blue", "red"])

# Node colors based on a property (here, using indegree for example)
node_colors_2022_shop7 = [cmap_2022_shop7(size) for size in normalized_sizes_2022_shop7]

# Layout
pos_2022_shop7 = nx.spring_layout(G_2022_shop7, seed=42)

# Drawing
plt.figure(figsize=(40, 30))
nx.draw_networkx_edges(G_2022_shop7, pos_2022_shop7, width=edge_widths_2022_shop7, edge_color='grey', alpha=0.5)
nx.draw_networkx_nodes(G_2022_shop7, pos_2022_shop7, node_size=node_sizes_2022_shop7, node_color=node_colors_2022_shop7, alpha=0.8)
nx.draw_networkx_labels(G_2022_shop7, pos_2022_shop7, font_size=8, font_color='black')
plt.title('Network Graph of Association Rules Based on Confidence for 2022 Shop 7')
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop7 = nx.DiGraph()

convictions_2022_shop7 = []  # To store conviction values for normalization

for _, row in rules_2022_shop7_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    conviction_2022_shop7 = row['conviction']  # Assuming each row has a 'conviction' column
    convictions_2022_shop7.append(conviction_2022_shop7)
    G_2022_shop7.add_edge(antecedents, consequents, weight=row['confidence'], conviction=conviction_2022_shop7)

# Normalize conviction values for edge width
max_conviction_2022_shop7 = max(convictions_2022_shop7)
min_conviction_2022_shop7 = min(convictions_2022_shop7)
conviction_range_2022_shop7 = max_conviction_2022_shop7 - min_conviction_2022_shop7
edge_widths_2022_shop7 = [(G_2022_shop7[u][v]['conviction'] - min_conviction_2022_shop7) / conviction_range_2022_shop7 * 30 + 0.5 for u, v in G_2022_shop7.edges()]  # Scale and offset widths

pos_2022_shop7 = nx.spring_layout(G_2022_shop7)

# Node sizes and colors as before
in_degrees_2022_shop7 = dict(G_2022_shop7.in_degree())
node_sizes_2022_shop7 = [in_degrees_2022_shop7[node] * 100 + 1000 for node in G_2022_shop7.nodes()]
normalized_sizes_2022_shop7 = (np.array(node_sizes_2022_shop7) - min(node_sizes_2022_shop7)) / (max(node_sizes_2022_shop7) - min(node_sizes_2022_shop7))
cmap_2022_shop7 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2022_shop7 = [cmap_2022_shop7(size) for size in normalized_sizes_2022_shop7]

# Drawing the network with conviction-based edge widths
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2022_shop7, pos_2022_shop7, node_size=node_sizes_2022_shop7, node_color=node_colors_2022_shop7, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop7, pos_2022_shop7, width=edge_widths_2022_shop7, edge_color='grey', alpha=0.5)
nx.draw_networkx_labels(G_2022_shop7, pos_2022_shop7, font_size=10)

plt.title('Top 5 Association Rules Network for 2022 Shop 7 with Conviction-based Edge Thickness')
plt.axis('off')
plt.show()


In [ ]:
#the node sizes are based on the indegree where each node acts as a consequent,

In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2022_shop7_nonredundant_top30.loc[:, 'antecedents_'] = rules_2022_shop7_nonredundant_top30['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2022_shop7_nonredundant_top30.loc[:, 'consequents_'] = rules_2022_shop7_nonredundant_top30['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2022_shop7_nonredundant_top30, x='consequents_', y='antecedents_', s=rules_2022_shop7_nonredundant_top30['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2022_shop7_nonredundant_top30.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2022 Shop 7", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
print(rules_2022_shop7_nonredundant_top30[['antecedents', 'consequents']].head(10))

## Monetary Analysis of the Top Rules

In [ ]:
# Step 1: Extract unique product codes from the rules
unique_product_codes = set()
for idx, row in rules_2022_shop7_nonredundant_top30.iterrows():
    # Unite antecedents and consequents into a single set
    all_products = row['antecedents'].union(row['consequents'])
    unique_product_codes.update(all_products)

# Step 2: Create a DataFrame with unique product codes and their unit prices
# Filter sales_data for these product codes and drop duplicates to ensure each product code appears once
product_prices = sales_data[sales_data['product_code'].isin(unique_product_codes)][['product_code', 'unit_price']].drop_duplicates()

# Create a dictionary for quick lookup of prices
price_dict = product_prices.set_index('product_code')['unit_price'].to_dict()

# Step 3: Calculate the total price for each rule and collect data for the new DataFrame
data_for_new_df = []
for idx, row in rules_2022_shop7_nonredundant_top30.iterrows():
    # Combine sets of antecedents and consequents
    all_products = row['antecedents'].union(row['consequents'])
    total_price = sum(price_dict.get(prod, 0) for prod in all_products)  # Use get to avoid KeyError if any product code is missing
    data_for_new_df.append({
        'antecedents': row['antecedents'],
        'consequents': row['consequents'],
        'total_unit_price': total_price
    })

# Create a new DataFrame with only antecedents, consequents, and total unit price
rules_and_price_shop7_2022 = pd.DataFrame(data_for_new_df)

# Sort the DataFrame by total_unit_price in descending order
rules_and_price_shop7_2022_sorted = rules_and_price_shop7_2022.sort_values(by='total_unit_price', ascending=False)

# Print the sorted DataFrame to see the specified columns along with their total unit prices
print(rules_and_price_shop7_2022_sorted)


In [ ]:
# Convert 'trx_date' to datetime format if not already done so
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Add a 'year' column if it doesn't exist
sales_data['year'] = sales_data['trx_date'].dt.year

# Step 2: Filter sales_data for shop7 and the year 2022
filtered_sales_data_2022 = sales_data[(sales_data['shopID'] == 7) & (sales_data['year'] == 2022)]

# Step 3: Initialize a list to hold the rule data including total sales value
rule_sales_summaries_2022 = []

# Step 4: Iterate through each rule in the DataFrame
for index, rule in rules_2022_shop7_nonredundant_top30.iterrows():
    all_products = rule['antecedents'].union(rule['consequents'])

    # Filter the sales data for transactions that include all products in the rule
    # This step ensures that we are only considering transactions that contain all specified products
    mask = filtered_sales_data_2022['product_code'].isin(all_products)
    transaction_ids = filtered_sales_data_2022[mask].groupby('trx_id').filter(lambda x: all_products.issubset(set(x['product_code']))).trx_id.unique()
    matched_transactions = filtered_sales_data_2022[filtered_sales_data_2022['trx_id'].isin(transaction_ids)]
    
    # Step 5: Sum the total_value of these transactions
    total_sales_value = matched_transactions['total_value'].sum()

    # Append the result to the list
    rule_sales_summaries_2022.append({
        'antecedents': rule['antecedents'],
        'consequents': rule['consequents'],
        'total_sales_value': total_sales_value
    })

# Step 6: Convert the list to a DataFrame
rules_sales_values_df_2022 = pd.DataFrame(rule_sales_summaries_2022)

# Step 7: Sort by total_sales_value in descending order
sorted_rules_sales_values_df_2022 = rules_sales_values_df_2022.sort_values(by='total_sales_value', ascending=False)

# Step 8: Print the sorted DataFrame
print(sorted_rules_sales_values_df_2022)


## Narrowing down based on Lift

In [ ]:
rules_2022_shop7_nonredundant_top30_lift['LHS'] = [','.join(list(x)) for x in rules_2022_shop7_nonredundant_top30_lift['antecedents']]
rules_2022_shop7_nonredundant_top30_lift['RHS'] = [','.join(list(x)) for x in rules_2022_shop7_nonredundant_top30_lift['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2022_shop7_nonredundant_top30_lift.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2022_shop7_nonredundant_top30_lift, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2022 Shop 7")
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop7_lift = nx.DiGraph()

lift_summary_2022_shop7 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2022_shop7_nonredundant_top30_lift.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop7_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop7:
        lift_summary_2022_shop7[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop7[consequents]['count'] += 1
    else:
        lift_summary_2022_shop7[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop7:
    lift_summary_2022_shop7[item]['avg_lift'] = lift_summary_2022_shop7[item]['total_lift'] / lift_summary_2022_shop7[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop7_lift = [lift_summary_2022_shop7[node]['avg_lift'] * 500 if node in lift_summary_2022_shop7 else 1000 for node in G_2022_shop7_lift.nodes()]

pos_2022_shop7_lift = nx.spring_layout(G_2022_shop7_lift)

# Colormap from orange to red
cmap_2022_shop7_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop7_lift = np.array(node_sizes_2022_shop7_lift)
normalized_sizes_2022_shop7_lift = (normalized_sizes_2022_shop7_lift - min(normalized_sizes_2022_shop7_lift)) / (max(normalized_sizes_2022_shop7_lift) - min(normalized_sizes_2022_shop7_lift))
node_colors_2022_shop7_lift = [cmap_2022_shop7_lift(size) for size in normalized_sizes_2022_shop7_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2022_shop7_lift, pos_2022_shop7_lift, node_size=node_sizes_2022_shop7_lift, node_color=node_colors_2022_shop7_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop7_lift, pos_2022_shop7_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop7_lift, pos_2022_shop7_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2022 Shop 7 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2022_shop7_nonredundant_top30_lift.loc[:, 'antecedents_'] = rules_2022_shop7_nonredundant_top30_lift['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2022_shop7_nonredundant_top30_lift.loc[:, 'consequents_'] = rules_2022_shop7_nonredundant_top30_lift['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2022_shop7_nonredundant_top30_lift, x='consequents_', y='antecedents_', s=rules_2022_shop7_nonredundant_top30_lift['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2022_shop7_nonredundant_top30_lift.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2022 Shop 7", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
# Print only the 'antecedents' and 'consequents' columns of the top 10 rules
print(rules_2022_shop7_nonredundant_top30_lift[['antecedents', 'consequents']].head(10))


# Shop7 2023 SEARCH 14

In [ ]:
# Filter for records from shop ID 1 and the year 1993
sales_data_2023_shop7 = sales_data_shop7[(sales_data_shop7['trx_date'].dt.year == 2023)]

In [ ]:
sales_data_2023_shop7.head()

In [ ]:
# Grouping product codes by trx_id
transactions_2023_shop7 = sales_data_2023_shop7.groupby('trx_id')['product_code'].apply(list).tolist()

In [ ]:
# Encode transaction data
encoder_2023_shop7 = TransactionEncoder()
trans_array_2023_shop7 = encoder_2023_shop7.fit(transactions_2023_shop7).transform(transactions_2023_shop7)
trans_df_2023_shop7 = pd.DataFrame(trans_array_2023_shop7, columns=encoder_2023_shop7.columns_)

In [ ]:
# Find frequent itemsets
frequent_itemsets_2023_shop7 = apriori(trans_df_2023_shop7, min_support=0.001, use_colnames=True)

In [ ]:
print(frequent_itemsets_2023_shop7)

In [ ]:
## Generate the association rules - by confidence
rulesConfidence_2023_shop7 = association_rules(frequent_itemsets_2023_shop7, metric="confidence", min_threshold=0.10)
rulesConfidence_2023_shop7.sort_values(by='confidence', ascending=False, inplace=True)


In [ ]:
# Replace frozen sets with strings
rulesConfidence_2023_shop7['antecedents_'] = rulesConfidence_2023_shop7['antecedents'].apply(lambda a: ','.join(list(a)))
rulesConfidence_2023_shop7['consequents_'] = rulesConfidence_2023_shop7['consequents'].apply(lambda a: ','.join(list(a)))

# Draw
fig, ax = plt.subplots(figsize=(14,30))
ax = plt.scatter(data=rulesConfidence_2023_shop7, x='consequents_', y='antecedents_', s=rulesConfidence_2023_shop7['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rulesConfidence_2023_shop7.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2023 Shop 7", fontsize=14)
plt.xlabel("Consequents (RHS)", fontsize=12)
plt.ylabel("Antecedents (LHS)", fontsize=12)
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate(rotation=45)
plt.tight_layout()

plt.show()


## Eliminate Redundant Rules

In [ ]:
rulesConfidence_2023_shop7.head()

In [ ]:
rulesConfidence_2023_shop7.info()

In [ ]:
# If 'antecedents' and 'consequents' are not frozensets, convert them
rules_2023_shop7_redundant = rulesConfidence_2023_shop7.copy()
rules_2023_shop7_redundant['antecedents'] = rules_2023_shop7_redundant['antecedents'].apply(lambda x: frozenset(x))
rules_2023_shop7_redundant['consequents'] = rules_2023_shop7_redundant['consequents'].apply(lambda x: frozenset(x))


In [ ]:
# Create a copy of the DataFrame to avoid modifying the original while iterating
rules_2023_shop7_checkredundance = rules_2023_shop7_redundant.copy()

# List to hold non-redundant rules
non_redundant_rules_2023_shop7 = []

for _, rule in rules_2023_shop7_checkredundance.iterrows():
    if not is_redundant(rule, rules_2023_shop7_checkredundance):
        non_redundant_rules_2023_shop7.append(rule)

# Convert the list back to a DataFrame
rules_2023_shop7_nonredundant = pd.DataFrame(non_redundant_rules_2023_shop7).reset_index(drop=True)


In [ ]:
print(rules_2023_shop7_nonredundant.head())
print(f"Original number of rules: {len(rules_2023_shop7_redundant)}")
print(f"Number of non-redundant rules: {len(rules_2023_shop7_nonredundant)}")


In [ ]:
rules_2023_shop7_nonredundant.head()

In [ ]:
rules_2023_shop7_redundant.info()

In [ ]:
rules_2023_shop7_nonredundant.head()

In [ ]:
rules_2023_shop7_nonredundant_backup = rules_2023_shop7_nonredundant.copy()

In [ ]:
rules_2023_shop7_nonredundant_unfiltered = rules_2023_shop7_nonredundant.copy()

In [ ]:
#reset
#rules_2023_shop7_nonredundant = rules_2023_shop7_nonredundant_backup.copy()

# Redundant and Unfiltered

In [ ]:
rules_2023_shop7_redundant.head()

In [ ]:
G_2023_shop7 = nx.DiGraph()

# Iterate through rules for 2023 Shop 7 and add edges to the graph
for _, row in rules_2023_shop7_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as weight
    G_2023_shop7.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2023_shop7 = nx.spring_layout(G_2023_shop7)

# Recalculate node sizes based on indegree or another metric
in_degrees_2023_shop7 = dict(G_2023_shop7.in_degree())
node_sizes_2023_shop7 = [in_degrees_2023_shop7[node] * 100 + 1000 for node in G_2023_shop7.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2023_shop7 = (np.array(node_sizes_2023_shop7) - np.min(node_sizes_2023_shop7)) / (np.max(node_sizes_2023_shop7) - np.min(node_sizes_2023_shop7))

# Define a colormap from orange to red for visual consistency
cmap_2023_shop7 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop7 = [cmap_2023_shop7(size) for size in normalized_sizes_2023_shop7]

# Drawing the network
plt.figure(figsize=(60, 43))
nx.draw_networkx_nodes(G_2023_shop7, pos_2023_shop7, node_size=node_sizes_2023_shop7, node_color=node_colors_2023_shop7, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop7, pos_2023_shop7, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop7, pos_2023_shop7, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 7 with Color Gradient')
plt.axis('off')
plt.show()


### Based on LIFT

In [ ]:
G_2023_shop7_lift = nx.DiGraph()

lift_summary_2023_shop7 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop7_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop7_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop7:
        lift_summary_2023_shop7[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop7[consequents]['count'] += 1
    else:
        lift_summary_2023_shop7[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop7:
    lift_summary_2023_shop7[item]['avg_lift'] = lift_summary_2023_shop7[item]['total_lift'] / lift_summary_2023_shop7[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop7_lift = [lift_summary_2023_shop7[node]['avg_lift'] * 500 if node in lift_summary_2023_shop7 else 1000 for node in G_2023_shop7_lift.nodes()]

pos_2023_shop7_lift = nx.spring_layout(G_2023_shop7_lift)

# Colormap from orange to red
cmap_2023_shop7_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop7_lift = np.array(node_sizes_2023_shop7_lift)
normalized_sizes_2023_shop7_lift = (normalized_sizes_2023_shop7_lift - min(normalized_sizes_2023_shop7_lift)) / (max(normalized_sizes_2023_shop7_lift) - min(normalized_sizes_2023_shop7_lift))
node_colors_2023_shop7_lift = [cmap_2023_shop7_lift(size) for size in normalized_sizes_2023_shop7_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2023_shop7_lift, pos_2023_shop7_lift, node_size=node_sizes_2023_shop7_lift, node_color=node_colors_2023_shop7_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop7_lift, pos_2023_shop7_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop7_lift, pos_2023_shop7_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 7 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


# nonredundant_unfiltered

## Based on Indegree

In [ ]:
G_2023_shop7_nonredundant_unfiltered = nx.DiGraph()

# Iterating through each row to add edges with confidence as an attribute
for _, row in rules_2023_shop7_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop7_nonredundant_unfiltered.add_edge(antecedents, consequents, weight=row['confidence'])

# Defining the layout for network visualization
pos_2023_shop7_nonredundant_unfiltered = nx.spring_layout(G_2023_shop7_nonredundant_unfiltered)

# Calculating node sizes based on the indegree for color mapping
in_degrees_2023_shop7_nonredundant_unfiltered = dict(G_2023_shop7_nonredundant_unfiltered.in_degree())
node_sizes_2023_shop7_nonredundant_unfiltered = [in_degrees_2023_shop7_nonredundant_unfiltered[node] * 100 + 1000 for node in G_2023_shop7_nonredundant_unfiltered.nodes()]

# Normalizing node sizes for color mapping
normalized_sizes_2023_shop7_nonredundant_unfiltered = (np.array(node_sizes_2023_shop7_nonredundant_unfiltered) - min(node_sizes_2023_shop7_nonredundant_unfiltered)) / (max(node_sizes_2023_shop7_nonredundant_unfiltered) - min(node_sizes_2023_shop7_nonredundant_unfiltered))

# Defining a color map from orange to red
cmap_2023_shop7_nonredundant_unfiltered = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])

# Mapping normalized node sizes to colors
node_colors_2023_shop7_nonredundant_unfiltered = [cmap_2023_shop7_nonredundant_unfiltered(size) for size in normalized_sizes_2023_shop7_nonredundant_unfiltered]

# Drawing the network with node sizes and colors
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2023_shop7_nonredundant_unfiltered, pos_2023_shop7_nonredundant_unfiltered, node_size=node_sizes_2023_shop7_nonredundant_unfiltered, node_color=node_colors_2023_shop7_nonredundant_unfiltered, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop7_nonredundant_unfiltered, pos_2023_shop7_nonredundant_unfiltered, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2023_shop7_nonredundant_unfiltered, pos_2023_shop7_nonredundant_unfiltered, font_size=10)

plt.title('Node Size Based on Indegree for 2023 Shop 7')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2023_shop7_lift = nx.DiGraph()

lift_summary_2023_shop7 = {}

for _, row in rules_2023_shop7_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop7_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Update lift summary for consequents
    if consequents in lift_summary_2023_shop7:
        lift_summary_2023_shop7[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop7[consequents]['count'] += 1
    else:
        lift_summary_2023_shop7[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculate average lift for each consequent
for item in lift_summary_2023_shop7:
    lift_summary_2023_shop7[item]['avg_lift'] = lift_summary_2023_shop7[item]['total_lift'] / lift_summary_2023_shop7[item]['count']

# Node sizes based on average lift, scaled appropriately
node_sizes_2023_shop7_lift = [lift_summary_2023_shop7[node]['avg_lift'] * 500 if node in lift_summary_2023_shop7 else 1000 for node in G_2023_shop7_lift.nodes()]

pos_2023_shop7_lift = nx.spring_layout(G_2023_shop7_lift)

# Use a colormap that transitions from orange to red
cmap_2023_shop7_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop7_lift = np.array(node_sizes_2023_shop7_lift)
normalized_sizes_2023_shop7_lift = (normalized_sizes_2023_shop7_lift - min(normalized_sizes_2023_shop7_lift)) / (max(normalized_sizes_2023_shop7_lift) - min(normalized_sizes_2023_shop7_lift))
node_colors_2023_shop7_lift = [cmap_2023_shop7_lift(size) for size in normalized_sizes_2023_shop7_lift]

plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2023_shop7_lift, pos_2023_shop7_lift, node_size=node_sizes_2023_shop7_lift, node_color=node_colors_2023_shop7_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop7_lift, pos_2023_shop7_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop7_lift, pos_2023_shop7_lift, font_size=10)

plt.title('Node Size Based on Average Lift for 2023 Shop 7')
plt.axis('off')
plt.show()


# Based on conviction

In [ ]:
rules_2023_shop7_nonredundant.head()

In [ ]:
G_2023_shop7_conviction = nx.DiGraph()

conviction_summary_2023_shop7 = {}
count_summary_2023_shop7 = {}

# Assuming 'rules_2023_shop7_nonredundant_unfiltered' contains a 'conviction' column
for _, row in rules_2023_shop7_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop7_conviction.add_edge(antecedents, consequents, weight=row['confidence'], conviction=row['conviction'])
    
    # Aggregate conviction values and count occurrences
    if consequents in conviction_summary_2023_shop7:
        conviction_summary_2023_shop7[consequents] += row['conviction']
        count_summary_2023_shop7[consequents] += 1
    else:
        conviction_summary_2023_shop7[consequents] = row['conviction']
        count_summary_2023_shop7[consequents] = 1

# Calculate average conviction values
average_conviction_summary_2023_shop7 = {k: conviction_summary_2023_shop7[k] / count_summary_2023_shop7[k] for k in conviction_summary_2023_shop7}

# Normalize conviction values for sizing and coloring
max_conviction = max(average_conviction_summary_2023_shop7.values())
min_conviction = min(average_conviction_summary_2023_shop7.values())
conviction_range = max_conviction - min_conviction
node_colors = []
node_sizes = []

for node in G_2023_shop7_conviction.nodes():
    # Get the average conviction value for the node, defaulting to min if not present
    conviction = average_conviction_summary_2023_shop7.get(node, min_conviction)
    # Normalize between 0 and 1
    normalized_conviction = (conviction - min_conviction) / conviction_range if conviction_range else 0
    node_colors.append(normalized_conviction)
    node_sizes.append(normalized_conviction * 1000 + 100)  # Scale and offset size

# Use a colormap for coloring nodes based on conviction
cmap = plt.cm.coolwarm

pos = nx.spring_layout(G_2023_shop7_conviction)

plt.figure(figsize=(40, 22))
nodes = nx.draw_networkx_nodes(G_2023_shop7_conviction, pos, node_color=node_colors, node_size=node_sizes, cmap=cmap, alpha=0.8)
edges = nx.draw_networkx_edges(G_2023_shop7_conviction, pos, arrowstyle='->', arrowsize=10, edge_color='grey', alpha=0.5)
labels = nx.draw_networkx_labels(G_2023_shop7_conviction, pos, font_size=8)

plt.colorbar(nodes, label='Normalized Conviction')
plt.title('Network Graph of 2023 Shop 7 Based on Conviction')
plt.axis('off')
plt.show()

In [ ]:
print(max_conviction)

# Filtered top 30 and non redundant

In [ ]:
rules_2023_shop7_nonredundant.head()

In [ ]:
# Assuming rules_1992_shop2_nonredundant is your sorted rules DataFrame for shop2
rules_2023_shop7_nonredundant_top30 = rules_2023_shop7_nonredundant.copy(deep=True)

In [ ]:
rules_2023_shop7_nonredundant_top30_lift = rules_2023_shop7_nonredundant.copy(deep=True)

In [ ]:
# Select the top 30 rules based on lift or another criterion
rules_2023_shop7_nonredundant_top30_lift = rules_2023_shop7_nonredundant_top30_lift.sort_values(by='lift', ascending=False).head(30)

In [ ]:
rules_2023_shop7_nonredundant_top30_lift.head()

In [ ]:
# Select the top 30 rules based on confidence or another criterion
rules_2023_shop7_nonredundant_top30 = rules_2023_shop7_nonredundant_top30.sort_values(by='confidence', ascending=False).head(30)

In [ ]:
rules_2023_shop7_nonredundant_top30.info()

In [ ]:
rules_2023_shop7_nonredundant_top30['LHS'] = [','.join(list(x)) for x in rules_2023_shop7_nonredundant_top30['antecedents']]
rules_2023_shop7_nonredundant_top30['RHS'] = [','.join(list(x)) for x in rules_2023_shop7_nonredundant_top30['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2023_shop7_nonredundant_top30.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2023_shop7_nonredundant_top30, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2023 Shop 7")
plt.axis('off')
plt.show()


## Based on Indegree

In [ ]:
G_2023_shop7 = nx.DiGraph()

# Assuming 'rules_sorted_2023_shop7' contains association rules for shop 7 in 2023
for _, row in rules_2023_shop7_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as the weight
    G_2023_shop7.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2023_shop7 = nx.spring_layout(G_2023_shop7)

# Calculate node sizes based on indegree
in_degrees_2023_shop7 = dict(G_2023_shop7.in_degree())
node_sizes_2023_shop7 = [in_degrees_2023_shop7[node] * 100 + 1000 for node in G_2023_shop7.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2023_shop7 = (np.array(node_sizes_2023_shop7) - np.min(node_sizes_2023_shop7)) / (np.max(node_sizes_2023_shop7) - np.min(node_sizes_2023_shop7))

# Define a colormap from orange to red
cmap_2023_shop7 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop7 = [cmap_2023_shop7(size) for size in normalized_sizes_2023_shop7]

# Drawing the network
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2023_shop7, pos_2023_shop7, node_size=node_sizes_2023_shop7, node_color=node_colors_2023_shop7, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop7, pos_2023_shop7, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2023_shop7, pos_2023_shop7, font_size=8)

plt.title('Top 5 Association Rules Network for 2023 Shop 7 with Color Gradient')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2023_shop7_lift = nx.DiGraph()

lift_summary_2023_shop7 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop7_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop7_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop7:
        lift_summary_2023_shop7[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop7[consequents]['count'] += 1
    else:
        lift_summary_2023_shop7[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop7:
    lift_summary_2023_shop7[item]['avg_lift'] = lift_summary_2023_shop7[item]['total_lift'] / lift_summary_2023_shop7[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop7_lift = [lift_summary_2023_shop7[node]['avg_lift'] * 500 if node in lift_summary_2023_shop7 else 1000 for node in G_2023_shop7_lift.nodes()]

pos_2023_shop7_lift = nx.spring_layout(G_2023_shop7_lift)

# Colormap from orange to red
cmap_2023_shop7_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop7_lift = np.array(node_sizes_2023_shop7_lift)
normalized_sizes_2023_shop7_lift = (normalized_sizes_2023_shop7_lift - min(normalized_sizes_2023_shop7_lift)) / (max(normalized_sizes_2023_shop7_lift) - min(normalized_sizes_2023_shop7_lift))
node_colors_2023_shop7_lift = [cmap_2023_shop7_lift(size) for size in normalized_sizes_2023_shop7_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2023_shop7_lift, pos_2023_shop7_lift, node_size=node_sizes_2023_shop7_lift, node_color=node_colors_2023_shop7_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop7_lift, pos_2023_shop7_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop7_lift, pos_2023_shop7_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 7 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop7_lift = nx.DiGraph()

lift_summary_2023_shop7 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop7_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop7_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop7:
        lift_summary_2023_shop7[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop7[consequents]['count'] += 1
    else:
        lift_summary_2023_shop7[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop7:
    lift_summary_2023_shop7[item]['avg_lift'] = lift_summary_2023_shop7[item]['total_lift'] / lift_summary_2023_shop7[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop7_lift = [lift_summary_2023_shop7[node]['avg_lift'] * 500 if node in lift_summary_2023_shop7 else 1000 for node in G_2023_shop7_lift.nodes()]

pos_2023_shop7_lift = nx.spring_layout(G_2023_shop7_lift)

# Colormap from orange to red
cmap_2023_shop7_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop7_lift = np.array(node_sizes_2023_shop7_lift)
normalized_sizes_2023_shop7_lift = (normalized_sizes_2023_shop7_lift - min(normalized_sizes_2023_shop7_lift)) / (max(normalized_sizes_2023_shop7_lift) - min(normalized_sizes_2023_shop7_lift))
node_colors_2023_shop7_lift = [cmap_2023_shop7_lift(size) for size in normalized_sizes_2023_shop7_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2023_shop7_lift, pos_2023_shop7_lift, node_size=node_sizes_2023_shop7_lift, node_color=node_colors_2023_shop7_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop7_lift, pos_2023_shop7_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop7_lift, pos_2023_shop7_lift, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 7 Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop7 = nx.DiGraph()

# Assuming 'rules_2023_shop7_nonredundant' contains association rules for 2023 Shop 7
for _, row in rules_2023_shop7_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop7.add_edge(antecedents, consequents, confidence=row['confidence'])

# Calculate node sizes based on indegree
in_degrees_2023_shop7 = dict(G_2023_shop7.in_degree())
node_sizes_2023_shop7 = [in_degrees_2023_shop7[node] * 100 + 1000 for node in G_2023_shop7.nodes()]

# Edge width based on confidence
edge_widths_2023_shop7 = [G_2023_shop7[u][v]['confidence'] * 15 for u, v in G_2023_shop7.edges()]

# Assuming normalized sizes and color mapping is desired
normalized_sizes_2023_shop7 = np.array(list(in_degrees_2023_shop7.values()))
normalized_sizes_2023_shop7 = (normalized_sizes_2023_shop7 - min(normalized_sizes_2023_shop7)) / (max(normalized_sizes_2023_shop7) - min(normalized_sizes_2023_shop7))
cmap_2023_shop7 = mcolors.LinearSegmentedColormap.from_list("MyCMapName", ["blue", "red"])

# Node colors based on a property (here, using indegree for example)
node_colors_2023_shop7 = [cmap_2023_shop7(size) for size in normalized_sizes_2023_shop7]

# Layout
pos_2023_shop7 = nx.spring_layout(G_2023_shop7, seed=42)

# Drawing
plt.figure(figsize=(40, 30))
nx.draw_networkx_edges(G_2023_shop7, pos_2023_shop7, width=edge_widths_2023_shop7, edge_color='grey', alpha=0.5)
nx.draw_networkx_nodes(G_2023_shop7, pos_2023_shop7, node_size=node_sizes_2023_shop7, node_color=node_colors_2023_shop7, alpha=0.8)
nx.draw_networkx_labels(G_2023_shop7, pos_2023_shop7, font_size=8, font_color='black')
plt.title('Network Graph of Association Rules Based on Confidence for 2023 Shop 7')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop7 = nx.DiGraph()

convictions_2023_shop7 = []  # To store conviction values for normalization

for _, row in rules_2023_shop7_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    conviction = row['conviction']  # Assuming each row has a 'conviction' column
    convictions_2023_shop7.append(conviction)
    G_2023_shop7.add_edge(antecedents, consequents, weight=row['confidence'], conviction=conviction)

# Normalize conviction values for edge width
max_conviction_2023_shop7 = max(convictions_2023_shop7)
min_conviction_2023_shop7 = min(convictions_2023_shop7)
conviction_range_2023_shop7 = max_conviction_2023_shop7 - min_conviction_2023_shop7
edge_widths_2023_shop7 = [(G_2023_shop7[u][v]['conviction'] - min_conviction_2023_shop7) / conviction_range_2023_shop7 * 30 + 0.5 for u, v in G_2023_shop7.edges()]  # Scale and offset widths

pos_2023_shop7 = nx.spring_layout(G_2023_shop7)

# Node sizes and colors as before
in_degrees_2023_shop7 = dict(G_2023_shop7.in_degree())
node_sizes_2023_shop7 = [in_degrees_2023_shop7[node] * 100 + 1000 for node in G_2023_shop7.nodes()]
normalized_sizes_2023_shop7 = (np.array(node_sizes_2023_shop7) - min(node_sizes_2023_shop7)) / (max(node_sizes_2023_shop7) - min(node_sizes_2023_shop7))
cmap_2023_shop7 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop7 = [cmap_2023_shop7(size) for size in normalized_sizes_2023_shop7]

# Drawing the network with conviction-based edge widths
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2023_shop7, pos_2023_shop7, node_size=node_sizes_2023_shop7, node_color=node_colors_2023_shop7, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop7, pos_2023_shop7, width=edge_widths_2023_shop7, edge_color='grey', alpha=0.5)
nx.draw_networkx_labels(G_2023_shop7, pos_2023_shop7, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 7 with Conviction-based Edge Thickness')
plt.axis('off')
plt.show()


In [ ]:
#the node sizes are based on the indegree where each node acts as a consequent,

In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2023_shop7_nonredundant_top30.loc[:, 'antecedents_'] = rules_2023_shop7_nonredundant_top30['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2023_shop7_nonredundant_top30.loc[:, 'consequents_'] = rules_2023_shop7_nonredundant_top30['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2023_shop7_nonredundant_top30, x='consequents_', y='antecedents_', s=rules_2023_shop7_nonredundant_top30['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2023_shop7_nonredundant_top30.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2023 Shop 7", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
print(rules_2023_shop7_nonredundant_top30[['antecedents', 'consequents']].head(31))

## Monetary Analysis of the Top Rules

In [ ]:
# Step 1: Extract unique product codes from the rules
unique_product_codes = set()
for idx, row in rules_2023_shop7_nonredundant_top30.iterrows():
    # Unite antecedents and consequents into a single set
    all_products = row['antecedents'].union(row['consequents'])
    unique_product_codes.update(all_products)

# Step 2: Create a DataFrame with unique product codes and their unit prices
# Filter sales_data for these product codes and drop duplicates to ensure each product code appears once
product_prices = sales_data[sales_data['product_code'].isin(unique_product_codes)][['product_code', 'unit_price']].drop_duplicates()

# Create a dictionary for quick lookup of prices
price_dict = product_prices.set_index('product_code')['unit_price'].to_dict()

# Step 3: Calculate the total price for each rule and collect data for the new DataFrame
data_for_new_df = []
for idx, row in rules_2023_shop7_nonredundant_top30.iterrows():
    # Combine sets of antecedents and consequents
    all_products = row['antecedents'].union(row['consequents'])
    total_price = sum(price_dict.get(prod, 0) for prod in all_products)  # Use get to avoid KeyError if any product code is missing
    data_for_new_df.append({
        'antecedents': row['antecedents'],
        'consequents': row['consequents'],
        'total_unit_price': total_price
    })

# Create a new DataFrame with only antecedents, consequents, and total unit price
rules_and_price_shop7_2023 = pd.DataFrame(data_for_new_df)

# Sort the DataFrame by total_unit_price in descending order
rules_and_price_shop7_2023_sorted = rules_and_price_shop7_2023.sort_values(by='total_unit_price', ascending=False)

# Print the sorted DataFrame to see the specified columns along with their total unit prices
print(rules_and_price_shop7_2023_sorted)


In [ ]:
# Convert 'trx_date' to datetime format if not already done so
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Add a 'year' column if it doesn't exist
sales_data['year'] = sales_data['trx_date'].dt.year

# Step 2: Filter sales_data for shop7 and the year 2023
filtered_sales_data_2023 = sales_data[(sales_data['shopID'] == 7) & (sales_data['year'] == 2023)]

# Step 3: Initialize a list to hold the rule data including total sales value
rule_sales_summaries_2023 = []

# Step 4: Iterate through each rule in the DataFrame
for index, rule in rules_2023_shop7_nonredundant_top30.iterrows():
    all_products = rule['antecedents'].union(rule['consequents'])

    # Filter the sales data for transactions that include all products in the rule
    # This step ensures that we are only considering transactions that contain all specified products
    mask = filtered_sales_data_2023['product_code'].isin(all_products)
    transaction_ids = filtered_sales_data_2023[mask].groupby('trx_id').filter(lambda x: all_products.issubset(set(x['product_code']))).trx_id.unique()
    matched_transactions = filtered_sales_data_2023[filtered_sales_data_2023['trx_id'].isin(transaction_ids)]
    
    # Step 5: Sum the total_value of these transactions
    total_sales_value = matched_transactions['total_value'].sum()

    # Append the result to the list
    rule_sales_summaries_2023.append({
        'antecedents': rule['antecedents'],
        'consequents': rule['consequents'],
        'total_sales_value': total_sales_value
    })

# Step 6: Convert the list to a DataFrame
rules_sales_values_df_2023 = pd.DataFrame(rule_sales_summaries_2023)

# Step 7: Sort by total_sales_value in descending order
sorted_rules_sales_values_df_2023 = rules_sales_values_df_2023.sort_values(by='total_sales_value', ascending=False)

# Step 8: Print the sorted DataFrame
print(sorted_rules_sales_values_df_2023)


## Narrowing down based on Lift

In [ ]:
rules_2023_shop7_nonredundant_top30_lift['LHS'] = [','.join(list(x)) for x in rules_2023_shop7_nonredundant_top30_lift['antecedents']]
rules_2023_shop7_nonredundant_top30_lift['RHS'] = [','.join(list(x)) for x in rules_2023_shop7_nonredundant_top30_lift['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2023_shop7_nonredundant_top30_lift.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2023_shop7_nonredundant_top30_lift, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2023 Shop 7")
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop7_lift = nx.DiGraph()

lift_summary_2023_shop7 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop7_nonredundant_top30_lift.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop7_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop7:
        lift_summary_2023_shop7[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop7[consequents]['count'] += 1
    else:
        lift_summary_2023_shop7[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop7:
    lift_summary_2023_shop7[item]['avg_lift'] = lift_summary_2023_shop7[item]['total_lift'] / lift_summary_2023_shop7[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop7_lift = [lift_summary_2023_shop7[node]['avg_lift'] * 500 if node in lift_summary_2023_shop7 else 1000 for node in G_2023_shop7_lift.nodes()]

pos_2023_shop7_lift = nx.spring_layout(G_2023_shop7_lift)

# Colormap from orange to red
cmap_2023_shop7_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop7_lift = np.array(node_sizes_2023_shop7_lift)
normalized_sizes_2023_shop7_lift = (normalized_sizes_2023_shop7_lift - min(normalized_sizes_2023_shop7_lift)) / (max(normalized_sizes_2023_shop7_lift) - min(normalized_sizes_2023_shop7_lift))
node_colors_2023_shop7_lift = [cmap_2023_shop7_lift(size) for size in normalized_sizes_2023_shop7_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2023_shop7_lift, pos_2023_shop7_lift, node_size=node_sizes_2023_shop7_lift, node_color=node_colors_2023_shop7_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop7_lift, pos_2023_shop7_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop7_lift, pos_2023_shop7_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 7 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2023_shop7_nonredundant_top30_lift.loc[:, 'antecedents_'] = rules_2023_shop7_nonredundant_top30_lift['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2023_shop7_nonredundant_top30_lift.loc[:, 'consequents_'] = rules_2023_shop7_nonredundant_top30_lift['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2023_shop7_nonredundant_top30_lift, x='consequents_', y='antecedents_', s=rules_2023_shop7_nonredundant_top30_lift['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2023_shop7_nonredundant_top30_lift.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2023 Shop 7", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
# Print only the 'antecedents' and 'consequents' columns of the top 10 rules
print(rules_2023_shop7_nonredundant_top30_lift[['antecedents', 'consequents']].head(10))


# Shop8 2022 SEARCH 15

In [ ]:
# Filter for records from shop ID 1 and the year 1993
sales_data_2022_shop8 = sales_data_shop8[(sales_data_shop8['trx_date'].dt.year == 2022)]

In [ ]:
sales_data_2022_shop8.head()

In [ ]:
# Grouping product codes by trx_id
transactions_2022_shop8 = sales_data_2022_shop8.groupby('trx_id')['product_code'].apply(list).tolist()

In [ ]:
# Encode transaction data
encoder_2022_shop8 = TransactionEncoder()
trans_array_2022_shop8 = encoder_2022_shop8.fit(transactions_2022_shop8).transform(transactions_2022_shop8)
trans_df_2022_shop8 = pd.DataFrame(trans_array_2022_shop8, columns=encoder_2022_shop8.columns_)

In [ ]:
# Find frequent itemsets
frequent_itemsets_2022_shop8 = apriori(trans_df_2022_shop8, min_support=0.001, use_colnames=True)

In [ ]:
print(frequent_itemsets_2022_shop8)

In [ ]:
## Generate the association rules - by confidence
rulesConfidence_2022_shop8 = association_rules(frequent_itemsets_2022_shop8, metric="confidence", min_threshold=0.10)
rulesConfidence_2022_shop8.sort_values(by='confidence', ascending=False, inplace=True)


In [ ]:
# Note: This is a hypothetical example, as I can't directly manipulate or access dataframes not provided within this code cell.

# Replace frozen sets with strings
rulesConfidence_2022_shop8['antecedents_'] = rulesConfidence_2022_shop8['antecedents'].apply(lambda a: ','.join(list(a)))
rulesConfidence_2022_shop8['consequents_'] = rulesConfidence_2022_shop8['consequents'].apply(lambda a: ','.join(list(a)))

# Draw
fig, ax = plt.subplots(figsize=(14,30))
ax = plt.scatter(data=rulesConfidence_2022_shop8, x='consequents_', y='antecedents_', s=rulesConfidence_2022_shop8['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rulesConfidence_2022_shop8.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2020 Shop 7", fontsize=14)
plt.xlabel("Consequents (RHS)", fontsize=12)
plt.ylabel("Antecedents (LHS)", fontsize=12)
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate(rotation=45)
plt.tight_layout()

plt.show()




## Eliminate Redundant Rules

In [ ]:
rulesConfidence_2022_shop8.head()

In [ ]:
rulesConfidence_2022_shop8.info()

In [ ]:
# If 'antecedents' and 'consequents' are not frozensets, convert them
rules_2022_shop8_redundant = rulesConfidence_2022_shop8.copy()
rules_2022_shop8_redundant['antecedents'] = rules_2022_shop8_redundant['antecedents'].apply(lambda x: frozenset(x))
rules_2022_shop8_redundant['consequents'] = rules_2022_shop8_redundant['consequents'].apply(lambda x: frozenset(x))


In [ ]:
# Create a copy of the DataFrame to avoid modifying the original while iterating
rules_2022_shop8_checkredundance = rules_2022_shop8_redundant.copy()

# List to hold non-redundant rules
non_redundant_rules_2022_shop8 = []

for _, rule in rules_2022_shop8_checkredundance.iterrows():
    if not is_redundant(rule, rules_2022_shop8_checkredundance):
        non_redundant_rules_2022_shop8.append(rule)

# Convert the list back to a DataFrame
rules_2022_shop8_nonredundant = pd.DataFrame(non_redundant_rules_2022_shop8).reset_index(drop=True)


In [ ]:
print(rules_2022_shop8_nonredundant.head())
print(f"Original number of rules: {len(rules_2022_shop8_redundant)}")
print(f"Number of non-redundant rules: {len(rules_2022_shop8_nonredundant)}")


In [ ]:
rules_2022_shop8_nonredundant.head()

In [ ]:
rules_2022_shop8_redundant.info()

In [ ]:
rules_2022_shop8_nonredundant.head()

In [ ]:
rules_2022_shop8_nonredundant_backup = rules_2022_shop8_nonredundant.copy()

In [ ]:
rules_2022_shop8_nonredundant_unfiltered = rules_2022_shop8_nonredundant.copy()

In [ ]:
#reset
#rules_2022_shop8_nonredundant = rules_2022_shop8_nonredundant_backup.copy()

# Redundant and Unfiltered

In [ ]:
rules_2022_shop8_redundant.head()

In [ ]:
G_2022_shop8 = nx.DiGraph()

# Iterate through rules for 2022 Shop 8 and add edges to the graph
for _, row in rules_2022_shop8_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as weight
    G_2022_shop8.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2022_shop8 = nx.spring_layout(G_2022_shop8)

# Recalculate node sizes based on indegree or another metric
in_degrees_2022_shop8 = dict(G_2022_shop8.in_degree())
node_sizes_2022_shop8 = [in_degrees_2022_shop8[node] * 100 + 1000 for node in G_2022_shop8.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2022_shop8 = (np.array(node_sizes_2022_shop8) - np.min(node_sizes_2022_shop8)) / (np.max(node_sizes_2022_shop8) - np.min(node_sizes_2022_shop8))

# Define a colormap from orange to red for visual consistency
cmap_2022_shop8 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2022_shop8 = [cmap_2022_shop8(size) for size in normalized_sizes_2022_shop8]

# Drawing the network
plt.figure(figsize=(60, 43))
nx.draw_networkx_nodes(G_2022_shop8, pos_2022_shop8, node_size=node_sizes_2022_shop8, node_color=node_colors_2022_shop8, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop8, pos_2022_shop8, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop8, pos_2022_shop8, font_size=10)

plt.title('Top 5 Association Rules Network for 2022 Shop 8 with Color Gradient')
plt.axis('off')
plt.show()


### Based on LIFT

In [ ]:
G_2022_shop8_lift = nx.DiGraph()

lift_summary_2022_shop8 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2022_shop8_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop8_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop8:
        lift_summary_2022_shop8[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop8[consequents]['count'] += 1
    else:
        lift_summary_2022_shop8[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop8:
    lift_summary_2022_shop8[item]['avg_lift'] = lift_summary_2022_shop8[item]['total_lift'] / lift_summary_2022_shop8[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop8_lift = [lift_summary_2022_shop8[node]['avg_lift'] * 500 if node in lift_summary_2022_shop8 else 1000 for node in G_2022_shop8_lift.nodes()]

pos_2022_shop8_lift = nx.spring_layout(G_2022_shop8_lift)

# Colormap from orange to red
cmap_2022_shop8_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop8_lift = np.array(node_sizes_2022_shop8_lift)
normalized_sizes_2022_shop8_lift = (normalized_sizes_2022_shop8_lift - min(normalized_sizes_2022_shop8_lift)) / (max(normalized_sizes_2022_shop8_lift) - min(normalized_sizes_2022_shop8_lift))
node_colors_2022_shop8_lift = [cmap_2022_shop8_lift(size) for size in normalized_sizes_2022_shop8_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2022_shop8_lift, pos_2022_shop8_lift, node_size=node_sizes_2022_shop8_lift, node_color=node_colors_2022_shop8_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop8_lift, pos_2022_shop8_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop8_lift, pos_2022_shop8_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2022 Shop 8 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


# nonredundant_unfiltered

## Based on Indegree

In [ ]:
G_2022_shop8_nonredundant_unfiltered = nx.DiGraph()

# Iterating through each row to add edges with confidence as an attribute
for _, row in rules_2022_shop8_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop8_nonredundant_unfiltered.add_edge(antecedents, consequents, weight=row['confidence'])

# Defining the layout for network visualization
pos_2022_shop8_nonredundant_unfiltered = nx.spring_layout(G_2022_shop8_nonredundant_unfiltered)

# Calculating node sizes based on the indegree for color mapping
in_degrees_2022_shop8_nonredundant_unfiltered = dict(G_2022_shop8_nonredundant_unfiltered.in_degree())
node_sizes_2022_shop8_nonredundant_unfiltered = [in_degrees_2022_shop8_nonredundant_unfiltered[node] * 100 + 1000 for node in G_2022_shop8_nonredundant_unfiltered.nodes()]

# Normalizing node sizes for color mapping
normalized_sizes_2022_shop8_nonredundant_unfiltered = (np.array(node_sizes_2022_shop8_nonredundant_unfiltered) - min(node_sizes_2022_shop8_nonredundant_unfiltered)) / (max(node_sizes_2022_shop8_nonredundant_unfiltered) - min(node_sizes_2022_shop8_nonredundant_unfiltered))

# Defining a color map from orange to red
cmap_2022_shop8_nonredundant_unfiltered = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])

# Mapping normalized node sizes to colors
node_colors_2022_shop8_nonredundant_unfiltered = [cmap_2022_shop8_nonredundant_unfiltered(size) for size in normalized_sizes_2022_shop8_nonredundant_unfiltered]

# Drawing the network with node sizes and colors
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2022_shop8_nonredundant_unfiltered, pos_2022_shop8_nonredundant_unfiltered, node_size=node_sizes_2022_shop8_nonredundant_unfiltered, node_color=node_colors_2022_shop8_nonredundant_unfiltered, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop8_nonredundant_unfiltered, pos_2022_shop8_nonredundant_unfiltered, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2022_shop8_nonredundant_unfiltered, pos_2022_shop8_nonredundant_unfiltered, font_size=10)

plt.title('Node Size Based on Indegree for 2022 Shop 8')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2022_shop8_lift = nx.DiGraph()

lift_summary_2022_shop8 = {}

for _, row in rules_2022_shop8_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop8_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Update lift summary for consequents
    if consequents in lift_summary_2022_shop8:
        lift_summary_2022_shop8[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop8[consequents]['count'] += 1
    else:
        lift_summary_2022_shop8[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculate average lift for each consequent
for item in lift_summary_2022_shop8:
    lift_summary_2022_shop8[item]['avg_lift'] = lift_summary_2022_shop8[item]['total_lift'] / lift_summary_2022_shop8[item]['count']

# Node sizes based on average lift, scaled appropriately
node_sizes_2022_shop8_lift = [lift_summary_2022_shop8[node]['avg_lift'] * 500 if node in lift_summary_2022_shop8 else 1000 for node in G_2022_shop8_lift.nodes()]

pos_2022_shop8_lift = nx.spring_layout(G_2022_shop8_lift)

# Use a colormap that transitions from orange to red
cmap_2022_shop8_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop8_lift = np.array(node_sizes_2022_shop8_lift)
normalized_sizes_2022_shop8_lift = (normalized_sizes_2022_shop8_lift - min(normalized_sizes_2022_shop8_lift)) / (max(normalized_sizes_2022_shop8_lift) - min(normalized_sizes_2022_shop8_lift))
node_colors_2022_shop8_lift = [cmap_2022_shop8_lift(size) for size in normalized_sizes_2022_shop8_lift]

plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2022_shop8_lift, pos_2022_shop8_lift, node_size=node_sizes_2022_shop8_lift, node_color=node_colors_2022_shop8_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop8_lift, pos_2022_shop8_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop8_lift, pos_2022_shop8_lift, font_size=10)

plt.title('Node Size Based on Average Lift for 2022 Shop 8')
plt.axis('off')
plt.show()


# Based on conviction

In [ ]:
rules_2022_shop8_nonredundant.head()

In [ ]:
G_2022_shop8_conviction = nx.DiGraph()

conviction_summary_2022_shop8 = {}
count_summary_2022_shop8 = {}

# Assuming 'rules_2022_shop8_nonredundant_unfiltered' contains a 'conviction' column
for _, row in rules_2022_shop8_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop8_conviction.add_edge(antecedents, consequents, weight=row['confidence'], conviction=row['conviction'])
    
    # Aggregate conviction values and count occurrences
    if consequents in conviction_summary_2022_shop8:
        conviction_summary_2022_shop8[consequents] += row['conviction']
        count_summary_2022_shop8[consequents] += 1
    else:
        conviction_summary_2022_shop8[consequents] = row['conviction']
        count_summary_2022_shop8[consequents] = 1

# Calculate average conviction values
average_conviction_summary_2022_shop8 = {k: conviction_summary_2022_shop8[k] / count_summary_2022_shop8[k] for k in conviction_summary_2022_shop8}

# Normalize conviction values for sizing and coloring
max_conviction = max(average_conviction_summary_2022_shop8.values())
min_conviction = min(average_conviction_summary_2022_shop8.values())
conviction_range = max_conviction - min_conviction
node_colors = []
node_sizes = []

for node in G_2022_shop8_conviction.nodes():
    # Get the average conviction value for the node, defaulting to min if not present
    conviction = average_conviction_summary_2022_shop8.get(node, min_conviction)
    # Normalize between 0 and 1
    normalized_conviction = (conviction - min_conviction) / conviction_range if conviction_range else 0
    node_colors.append(normalized_conviction)
    node_sizes.append(normalized_conviction * 1000 + 100)  # Scale and offset size

# Use a colormap for coloring nodes based on conviction
cmap = plt.cm.coolwarm

pos = nx.spring_layout(G_2022_shop8_conviction)

plt.figure(figsize=(40, 22))
nodes = nx.draw_networkx_nodes(G_2022_shop8_conviction, pos, node_color=node_colors, node_size=node_sizes, cmap=cmap, alpha=0.8)
edges = nx.draw_networkx_edges(G_2022_shop8_conviction, pos, arrowstyle='->', arrowsize=10, edge_color='grey', alpha=0.5)
labels = nx.draw_networkx_labels(G_2022_shop8_conviction, pos, font_size=8)

plt.colorbar(nodes, label='Normalized Conviction')
plt.title('Network Graph of 2022 Shop 8 Based on Conviction')
plt.axis('off')
plt.show()

In [ ]:
print(max_conviction)

# Filtered top 30 and non redundant

In [ ]:
rules_2022_shop8_nonredundant.head()

In [ ]:
# Assuming rules_1992_shop2_nonredundant is your sorted rules DataFrame for shop2
rules_2022_shop8_nonredundant_top30 = rules_2022_shop8_nonredundant.copy(deep=True)

In [ ]:
rules_2022_shop8_nonredundant_top30_lift = rules_2022_shop8_nonredundant.copy(deep=True)

In [ ]:
# Select the top 30 rules based on lift or another criterion
rules_2022_shop8_nonredundant_top30_lift = rules_2022_shop8_nonredundant_top30_lift.sort_values(by='lift', ascending=False).head(30)

In [ ]:
rules_2022_shop8_nonredundant_top30_lift.head()

In [ ]:
# Select the top 30 rules based on confidence or another criterion
rules_2022_shop8_nonredundant_top30 = rules_2022_shop8_nonredundant_top30.sort_values(by='confidence', ascending=False).head(30)

In [ ]:
rules_2022_shop8_nonredundant_top30.info()

In [ ]:
rules_2022_shop8_nonredundant_top30['LHS'] = [','.join(list(x)) for x in rules_2022_shop8_nonredundant_top30['antecedents']]
rules_2022_shop8_nonredundant_top30['RHS'] = [','.join(list(x)) for x in rules_2022_shop8_nonredundant_top30['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2022_shop8_nonredundant_top30.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2022_shop8_nonredundant_top30, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2022 Shop 8")
plt.axis('off')
plt.show()


## Based on Indegree

In [ ]:
G_2022_shop8 = nx.DiGraph()

# Assuming 'rules_sorted_2022_shop8' contains association rules for shop 8 in 2022
for _, row in rules_2022_shop8_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as the weight
    G_2022_shop8.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2022_shop8 = nx.spring_layout(G_2022_shop8)

# Calculate node sizes based on indegree
in_degrees_2022_shop8 = dict(G_2022_shop8.in_degree())
node_sizes_2022_shop8 = [in_degrees_2022_shop8[node] * 100 + 1000 for node in G_2022_shop8.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2022_shop8 = (np.array(node_sizes_2022_shop8) - np.min(node_sizes_2022_shop8)) / (np.max(node_sizes_2022_shop8) - np.min(node_sizes_2022_shop8))

# Define a colormap from orange to red
cmap_2022_shop8 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2022_shop8 = [cmap_2022_shop8(size) for size in normalized_sizes_2022_shop8]

# Drawing the network
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2022_shop8, pos_2022_shop8, node_size=node_sizes_2022_shop8, node_color=node_colors_2022_shop8, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop8, pos_2022_shop8, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2022_shop8, pos_2022_shop8, font_size=8)

plt.title('Top 5 Association Rules Network for 2022 Shop 8 with Color Gradient')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2022_shop8_lift = nx.DiGraph()

lift_summary_2022_shop8 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2022_shop8_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop8_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop8:
        lift_summary_2022_shop8[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop8[consequents]['count'] += 1
    else:
        lift_summary_2022_shop8[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop8:
    lift_summary_2022_shop8[item]['avg_lift'] = lift_summary_2022_shop8[item]['total_lift'] / lift_summary_2022_shop8[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop8_lift = [lift_summary_2022_shop8[node]['avg_lift'] * 500 if node in lift_summary_2022_shop8 else 1000 for node in G_2022_shop8_lift.nodes()]

pos_2022_shop8_lift = nx.spring_layout(G_2022_shop8_lift)

# Colormap from orange to red
cmap_2022_shop8_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop8_lift = np.array(node_sizes_2022_shop8_lift)
normalized_sizes_2022_shop8_lift = (normalized_sizes_2022_shop8_lift - min(normalized_sizes_2022_shop8_lift)) / (max(normalized_sizes_2022_shop8_lift) - min(normalized_sizes_2022_shop8_lift))
node_colors_2022_shop8_lift = [cmap_2022_shop8_lift(size) for size in normalized_sizes_2022_shop8_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2022_shop8_lift, pos_2022_shop8_lift, node_size=node_sizes_2022_shop8_lift, node_color=node_colors_2022_shop8_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop8_lift, pos_2022_shop8_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop8_lift, pos_2022_shop8_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2022 Shop 8 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop8_lift = nx.DiGraph()

lift_summary_2022_shop8 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2022_shop8_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop8_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop8:
        lift_summary_2022_shop8[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop8[consequents]['count'] += 1
    else:
        lift_summary_2022_shop8[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop8:
    lift_summary_2022_shop8[item]['avg_lift'] = lift_summary_2022_shop8[item]['total_lift'] / lift_summary_2022_shop8[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop8_lift = [lift_summary_2022_shop8[node]['avg_lift'] * 500 if node in lift_summary_2022_shop8 else 1000 for node in G_2022_shop8_lift.nodes()]

pos_2022_shop8_lift = nx.spring_layout(G_2022_shop8_lift)

# Colormap from orange to red
cmap_2022_shop8_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop8_lift = np.array(node_sizes_2022_shop8_lift)
normalized_sizes_2022_shop8_lift = (normalized_sizes_2022_shop8_lift - min(normalized_sizes_2022_shop8_lift)) / (max(normalized_sizes_2022_shop8_lift) - min(normalized_sizes_2022_shop8_lift))
node_colors_2022_shop8_lift = [cmap_2022_shop8_lift(size) for size in normalized_sizes_2022_shop8_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2022_shop8_lift, pos_2022_shop8_lift, node_size=node_sizes_2022_shop8_lift, node_color=node_colors_2022_shop8_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop8_lift, pos_2022_shop8_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop8_lift, pos_2022_shop8_lift, font_size=10)

plt.title('Top 5 Association Rules Network for 2022 Shop 8 Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop8 = nx.DiGraph()

# Assuming 'rules_2022_shop7_nonredundant_top30' contains association rules for 2022 Shop 8
for _, row in rules_2022_shop8_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop8.add_edge(antecedents, consequents, confidence=row['confidence'])

# Calculate node sizes based on indegree
in_degrees_2022_shop8 = dict(G_2022_shop8.in_degree())
node_sizes_2022_shop8 = [in_degrees_2022_shop8[node] * 100 + 1000 for node in G_2022_shop8.nodes()]

# Edge width based on confidence
edge_widths_2022_shop8 = [G_2022_shop8[u][v]['confidence'] * 15 for u, v in G_2022_shop8.edges()]

# Assuming normalized sizes and color mapping is desired
normalized_sizes_2022_shop8 = np.array(list(in_degrees_2022_shop8.values()))
normalized_sizes_2022_shop8 = (normalized_sizes_2022_shop8 - min(normalized_sizes_2022_shop8)) / (max(normalized_sizes_2022_shop8) - min(normalized_sizes_2022_shop8))
cmap_2022_shop8 = mcolors.LinearSegmentedColormap.from_list("MyCMapName", ["blue", "red"])

# Node colors based on a property (here, using indegree for example)
node_colors_2022_shop8 = [cmap_2022_shop8(size) for size in normalized_sizes_2022_shop8]

# Layout
pos_2022_shop8 = nx.spring_layout(G_2022_shop8, seed=42)

# Drawing
plt.figure(figsize=(40, 30))
nx.draw_networkx_edges(G_2022_shop8, pos_2022_shop8, width=edge_widths_2022_shop8, edge_color='grey', alpha=0.5)
nx.draw_networkx_nodes(G_2022_shop8, pos_2022_shop8, node_size=node_sizes_2022_shop8, node_color=node_colors_2022_shop8, alpha=0.8)
nx.draw_networkx_labels(G_2022_shop8, pos_2022_shop8, font_size=8, font_color='black')
plt.title('Network Graph of Association Rules Based on Confidence for 2022 Shop 8')
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop8 = nx.DiGraph()

convictions_2022_shop8 = []  # To store conviction values for normalization

for _, row in rules_2022_shop8_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    conviction_2022_shop8 = row['conviction']  # Assuming each row has a 'conviction' column
    convictions_2022_shop8.append(conviction_2022_shop8)
    G_2022_shop8.add_edge(antecedents, consequents, weight=row['confidence'], conviction=conviction_2022_shop8)

# Normalize conviction values for edge width
max_conviction_2022_shop8 = max(convictions_2022_shop8)
min_conviction_2022_shop8 = min(convictions_2022_shop8)
conviction_range_2022_shop8 = max_conviction_2022_shop8 - min_conviction_2022_shop8
edge_widths_2022_shop8 = [(G_2022_shop8[u][v]['conviction'] - min_conviction_2022_shop8) / conviction_range_2022_shop8 * 30 + 0.5 for u, v in G_2022_shop8.edges()]  # Scale and offset widths

pos_2022_shop8 = nx.spring_layout(G_2022_shop8)

# Node sizes and colors as before
in_degrees_2022_shop8 = dict(G_2022_shop8.in_degree())
node_sizes_2022_shop8 = [in_degrees_2022_shop8[node] * 100 + 1000 for node in G_2022_shop8.nodes()]
normalized_sizes_2022_shop8 = (np.array(node_sizes_2022_shop8) - min(node_sizes_2022_shop8)) / (max(node_sizes_2022_shop8) - min(node_sizes_2022_shop8))
cmap_2022_shop8 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2022_shop8 = [cmap_2022_shop8(size) for size in normalized_sizes_2022_shop8]

# Drawing the network with conviction-based edge widths
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2022_shop8, pos_2022_shop8, node_size=node_sizes_2022_shop8, node_color=node_colors_2022_shop8, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop8, pos_2022_shop8, width=edge_widths_2022_shop8, edge_color='grey', alpha=0.5)
nx.draw_networkx_labels(G_2022_shop8, pos_2022_shop8, font_size=10)

plt.title('Top 5 Association Rules Network for 2022 Shop 8 with Conviction-based Edge Thickness')
plt.axis('off')
plt.show()


In [ ]:
#the node sizes are based on the indegree where each node acts as a consequent,

In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2022_shop8_nonredundant_top30.loc[:, 'antecedents_'] = rules_2022_shop8_nonredundant_top30['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2022_shop8_nonredundant_top30.loc[:, 'consequents_'] = rules_2022_shop8_nonredundant_top30['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2022_shop8_nonredundant_top30, x='consequents_', y='antecedents_', s=rules_2022_shop8_nonredundant_top30['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2022_shop8_nonredundant_top30.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2022 Shop 8", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
print(rules_2022_shop8_nonredundant_top30[['antecedents', 'consequents']].head(10))

## Monetary Analysis of the Top Rules

In [ ]:
# Step 1: Extract unique product codes from the rules
unique_product_codes = set()
for idx, row in rules_2022_shop8_nonredundant_top30.iterrows():
    # Unite antecedents and consequents into a single set
    all_products = row['antecedents'].union(row['consequents'])
    unique_product_codes.update(all_products)

# Step 2: Create a DataFrame with unique product codes and their unit prices
# Filter sales_data for these product codes and drop duplicates to ensure each product code appears once
product_prices = sales_data[sales_data['product_code'].isin(unique_product_codes)][['product_code', 'unit_price']].drop_duplicates()

# Create a dictionary for quick lookup of prices
price_dict = product_prices.set_index('product_code')['unit_price'].to_dict()

# Step 3: Calculate the total price for each rule and collect data for the new DataFrame
data_for_new_df = []
for idx, row in rules_2022_shop8_nonredundant_top30.iterrows():
    # Combine sets of antecedents and consequents
    all_products = row['antecedents'].union(row['consequents'])
    total_price = sum(price_dict.get(prod, 0) for prod in all_products)  # Use get to avoid KeyError if any product code is missing
    data_for_new_df.append({
        'antecedents': row['antecedents'],
        'consequents': row['consequents'],
        'total_unit_price': total_price
    })

# Create a new DataFrame with only antecedents, consequents, and total unit price
rules_and_price_shop8_2022 = pd.DataFrame(data_for_new_df)

# Sort the DataFrame by total_unit_price in descending order
rules_and_price_shop8_2022_sorted = rules_and_price_shop8_2022.sort_values(by='total_unit_price', ascending=False)

# Print the sorted DataFrame to see the specified columns along with their total unit prices
print(rules_and_price_shop8_2022_sorted)


In [ ]:
# Convert 'trx_date' to datetime format if not already done so
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Add a 'year' column if it doesn't exist
sales_data['year'] = sales_data['trx_date'].dt.year

# Step 2: Filter sales_data for shop8 and the year 2022
filtered_sales_data_2022 = sales_data[(sales_data['shopID'] == 8) & (sales_data['year'] == 2022)]

# Step 3: Initialize a list to hold the rule data including total sales value
rule_sales_summaries_2022 = []

# Step 4: Iterate through each rule in the DataFrame
for index, rule in rules_2022_shop8_nonredundant_top30.iterrows():
    all_products = rule['antecedents'].union(rule['consequents'])

    # Filter the sales data for transactions that include all products in the rule
    mask = filtered_sales_data_2022['product_code'].isin(all_products)
    transaction_ids = filtered_sales_data_2022[mask].groupby('trx_id').filter(lambda x: all_products.issubset(set(x['product_code']))).trx_id.unique()
    matched_transactions = filtered_sales_data_2022[filtered_sales_data_2022['trx_id'].isin(transaction_ids)]
    
    # Step 5: Sum the total_value of these transactions
    total_sales_value = matched_transactions['total_value'].sum()

    # Append the result to the list
    rule_sales_summaries_2022.append({
        'antecedents': rule['antecedents'],
        'consequents': rule['consequents'],
        'total_sales_value': total_sales_value
    })

# Step 6: Convert the list to a DataFrame
rules_sales_values_df_2022 = pd.DataFrame(rule_sales_summaries_2022)

# Step 7: Sort by total_sales_value in descending order
sorted_rules_sales_values_df_2022 = rules_sales_values_df_2022.sort_values(by='total_sales_value', ascending=False)

# Step 8: Print the sorted DataFrame
print(sorted_rules_sales_values_df_2022)


## Narrowing down based on Lift

In [ ]:
rules_2022_shop8_nonredundant_top30_lift['LHS'] = [','.join(list(x)) for x in rules_2022_shop8_nonredundant_top30_lift['antecedents']]
rules_2022_shop8_nonredundant_top30_lift['RHS'] = [','.join(list(x)) for x in rules_2022_shop8_nonredundant_top30_lift['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2022_shop8_nonredundant_top30_lift.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2022_shop8_nonredundant_top30_lift, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2022 Shop 8")
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop8_lift = nx.DiGraph()

lift_summary_2022_shop8 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2022_shop8_nonredundant_top30_lift.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop8_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop8:
        lift_summary_2022_shop8[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop8[consequents]['count'] += 1
    else:
        lift_summary_2022_shop8[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop8:
    lift_summary_2022_shop8[item]['avg_lift'] = lift_summary_2022_shop8[item]['total_lift'] / lift_summary_2022_shop8[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop8_lift = [lift_summary_2022_shop8[node]['avg_lift'] * 500 if node in lift_summary_2022_shop8 else 1000 for node in G_2022_shop8_lift.nodes()]

pos_2022_shop8_lift = nx.spring_layout(G_2022_shop8_lift)

# Colormap from orange to red
cmap_2022_shop8_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop8_lift = np.array(node_sizes_2022_shop8_lift)
normalized_sizes_2022_shop8_lift = (normalized_sizes_2022_shop8_lift - min(normalized_sizes_2022_shop8_lift)) / (max(normalized_sizes_2022_shop8_lift) - min(normalized_sizes_2022_shop8_lift))
node_colors_2022_shop8_lift = [cmap_2022_shop8_lift(size) for size in normalized_sizes_2022_shop8_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2022_shop8_lift, pos_2022_shop8_lift, node_size=node_sizes_2022_shop8_lift, node_color=node_colors_2022_shop8_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop8_lift, pos_2022_shop8_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop8_lift, pos_2022_shop8_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2022 Shop 8 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2022_shop8_nonredundant_top30_lift.loc[:, 'antecedents_'] = rules_2022_shop8_nonredundant_top30_lift['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2022_shop8_nonredundant_top30_lift.loc[:, 'consequents_'] = rules_2022_shop8_nonredundant_top30_lift['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2022_shop8_nonredundant_top30_lift, x='consequents_', y='antecedents_', s=rules_2022_shop8_nonredundant_top30_lift['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2022_shop8_nonredundant_top30_lift.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2022 Shop 8", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
# Print only the 'antecedents' and 'consequents' columns of the top 10 rules
print(rules_2022_shop8_nonredundant_top30_lift[['antecedents', 'consequents']].head(10))


# Shop 8 2023 SEARCH 16

In [ ]:
# Filter for records from shop ID 1 and the year 1993
sales_data_2023_shop8 = sales_data_shop8[(sales_data_shop8['trx_date'].dt.year == 2023)]

In [ ]:
sales_data_2023_shop8.head()

In [ ]:
# Grouping product codes by trx_id
transactions_2023_shop8 = sales_data_2023_shop8.groupby('trx_id')['product_code'].apply(list).tolist()

In [ ]:
# Encode transaction data
encoder_2023_shop8 = TransactionEncoder()
trans_array_2023_shop8 = encoder_2023_shop8.fit(transactions_2023_shop8).transform(transactions_2023_shop8)
trans_df_2023_shop8 = pd.DataFrame(trans_array_2023_shop8, columns=encoder_2023_shop8.columns_)

In [ ]:
# Find frequent itemsets
frequent_itemsets_2023_shop8 = apriori(trans_df_2023_shop8, min_support=0.001, use_colnames=True)

In [ ]:
print(frequent_itemsets_2023_shop8)

In [ ]:
## Generate the association rules - by confidence
rulesConfidence_2023_shop8 = association_rules(frequent_itemsets_2023_shop8, metric="confidence", min_threshold=0.10)
rulesConfidence_2023_shop8.sort_values(by='confidence', ascending=False, inplace=True)


In [ ]:
# Replace frozen sets with strings
rulesConfidence_2023_shop8['antecedents_'] = rulesConfidence_2023_shop8['antecedents'].apply(lambda a: ','.join(list(a)))
rulesConfidence_2023_shop8['consequents_'] = rulesConfidence_2023_shop8['consequents'].apply(lambda a: ','.join(list(a)))

# Draw
fig, ax = plt.subplots(figsize=(14,30))
ax = plt.scatter(data=rulesConfidence_2023_shop8, x='consequents_', y='antecedents_', s=rulesConfidence_2023_shop8['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rulesConfidence_2023_shop8.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2023 Shop 8", fontsize=14)
plt.xlabel("Consequents (RHS)", fontsize=12)
plt.ylabel("Antecedents (LHS)", fontsize=12)
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate(rotation=45)
plt.tight_layout()

plt.show()


## Eliminate Redundant Rules

In [ ]:
rulesConfidence_2023_shop8.head()

In [ ]:
rulesConfidence_2023_shop8.info()

In [ ]:
# If 'antecedents' and 'consequents' are not frozensets, convert them
rules_2023_shop8_redundant = rulesConfidence_2023_shop8.copy()
rules_2023_shop8_redundant['antecedents'] = rules_2023_shop8_redundant['antecedents'].apply(lambda x: frozenset(x))
rules_2023_shop8_redundant['consequents'] = rules_2023_shop8_redundant['consequents'].apply(lambda x: frozenset(x))


In [ ]:
# Create a copy of the DataFrame to avoid modifying the original while iterating
rules_2023_shop8_checkredundance = rules_2023_shop8_redundant.copy()

# List to hold non-redundant rules
non_redundant_rules_2023_shop8 = []

for _, rule in rules_2023_shop8_checkredundance.iterrows():
    if not is_redundant(rule, rules_2023_shop8_checkredundance):
        non_redundant_rules_2023_shop8.append(rule)

# Convert the list back to a DataFrame
rules_2023_shop8_nonredundant = pd.DataFrame(non_redundant_rules_2023_shop8).reset_index(drop=True)


In [ ]:
print(rules_2023_shop8_nonredundant.head())
print(f"Original number of rules: {len(rules_2023_shop8_redundant)}")
print(f"Number of non-redundant rules: {len(rules_2023_shop8_nonredundant)}")


In [ ]:
rules_2023_shop8_nonredundant.head()

In [ ]:
rules_2023_shop8_redundant.info()

In [ ]:
rules_2023_shop8_nonredundant.head()

In [ ]:
rules_2023_shop8_nonredundant_backup = rules_2023_shop8_nonredundant.copy()

In [ ]:
rules_2023_shop8_nonredundant_unfiltered = rules_2023_shop8_nonredundant.copy()

In [ ]:
#reset
#rules_2023_shop8_nonredundant = rules_2023_shop8_nonredundant_backup.copy()

# Redundant and Unfiltered

In [ ]:
rules_2023_shop8_redundant.head()

In [ ]:
G_2023_shop8 = nx.DiGraph()

# Iterate through rules for 2023 Shop 8 and add edges to the graph
for _, row in rules_2023_shop8_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as weight
    G_2023_shop8.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2023_shop8 = nx.spring_layout(G_2023_shop8)

# Recalculate node sizes based on indegree or another metric
in_degrees_2023_shop8 = dict(G_2023_shop8.in_degree())
node_sizes_2023_shop8 = [in_degrees_2023_shop8[node] * 100 + 1000 for node in G_2023_shop8.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2023_shop8 = (np.array(node_sizes_2023_shop8) - np.min(node_sizes_2023_shop8)) / (np.max(node_sizes_2023_shop8) - np.min(node_sizes_2023_shop8))

# Define a colormap from orange to red for visual consistency
cmap_2023_shop8 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop8 = [cmap_2023_shop8(size) for size in normalized_sizes_2023_shop8]

# Drawing the network
plt.figure(figsize=(60, 43))
nx.draw_networkx_nodes(G_2023_shop8, pos_2023_shop8, node_size=node_sizes_2023_shop8, node_color=node_colors_2023_shop8, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop8, pos_2023_shop8, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop8, pos_2023_shop8, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 8 with Color Gradient')
plt.axis('off')
plt.show()


### Based on LIFT

In [ ]:
G_2023_shop8_lift = nx.DiGraph()

lift_summary_2023_shop8 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop8_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop8_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop8:
        lift_summary_2023_shop8[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop8[consequents]['count'] += 1
    else:
        lift_summary_2023_shop8[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop8:
    lift_summary_2023_shop8[item]['avg_lift'] = lift_summary_2023_shop8[item]['total_lift'] / lift_summary_2023_shop8[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop8_lift = [lift_summary_2023_shop8[node]['avg_lift'] * 500 if node in lift_summary_2023_shop8 else 1000 for node in G_2023_shop8_lift.nodes()]

pos_2023_shop8_lift = nx.spring_layout(G_2023_shop8_lift)

# Colormap from orange to red
cmap_2023_shop8_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop8_lift = np.array(node_sizes_2023_shop8_lift)
normalized_sizes_2023_shop8_lift = (normalized_sizes_2023_shop8_lift - min(normalized_sizes_2023_shop8_lift)) / (max(normalized_sizes_2023_shop8_lift) - min(normalized_sizes_2023_shop8_lift))
node_colors_2023_shop8_lift = [cmap_2023_shop8_lift(size) for size in normalized_sizes_2023_shop8_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2023_shop8_lift, pos_2023_shop8_lift, node_size=node_sizes_2023_shop8_lift, node_color=node_colors_2023_shop8_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop8_lift, pos_2023_shop8_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop8_lift, pos_2023_shop8_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 8 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


# nonredundant_unfiltered

## Based on Indegree

In [ ]:
G_2023_shop8_nonredundant_unfiltered = nx.DiGraph()

# Iterating through each row to add edges with confidence as an attribute
for _, row in rules_2023_shop8_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop8_nonredundant_unfiltered.add_edge(antecedents, consequents, weight=row['confidence'])

# Defining the layout for network visualization
pos_2023_shop8_nonredundant_unfiltered = nx.spring_layout(G_2023_shop8_nonredundant_unfiltered)

# Calculating node sizes based on the indegree for color mapping
in_degrees_2023_shop8_nonredundant_unfiltered = dict(G_2023_shop8_nonredundant_unfiltered.in_degree())
node_sizes_2023_shop8_nonredundant_unfiltered = [in_degrees_2023_shop8_nonredundant_unfiltered[node] * 100 + 1000 for node in G_2023_shop8_nonredundant_unfiltered.nodes()]

# Normalizing node sizes for color mapping
normalized_sizes_2023_shop8_nonredundant_unfiltered = (np.array(node_sizes_2023_shop8_nonredundant_unfiltered) - min(node_sizes_2023_shop8_nonredundant_unfiltered)) / (max(node_sizes_2023_shop8_nonredundant_unfiltered) - min(node_sizes_2023_shop8_nonredundant_unfiltered))

# Defining a color map from orange to red
cmap_2023_shop8_nonredundant_unfiltered = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])

# Mapping normalized node sizes to colors
node_colors_2023_shop8_nonredundant_unfiltered = [cmap_2023_shop8_nonredundant_unfiltered(size) for size in normalized_sizes_2023_shop8_nonredundant_unfiltered]

# Drawing the network with node sizes and colors
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2023_shop8_nonredundant_unfiltered, pos_2023_shop8_nonredundant_unfiltered, node_size=node_sizes_2023_shop8_nonredundant_unfiltered, node_color=node_colors_2023_shop8_nonredundant_unfiltered, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop8_nonredundant_unfiltered, pos_2023_shop8_nonredundant_unfiltered, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2023_shop8_nonredundant_unfiltered, pos_2023_shop8_nonredundant_unfiltered, font_size=10)

plt.title('Node Size Based on Indegree for 2023 Shop 8')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2023_shop8_lift = nx.DiGraph()

lift_summary_2023_shop8 = {}

for _, row in rules_2023_shop8_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop8_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Update lift summary for consequents
    if consequents in lift_summary_2023_shop8:
        lift_summary_2023_shop8[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop8[consequents]['count'] += 1
    else:
        lift_summary_2023_shop8[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculate average lift for each consequent
for item in lift_summary_2023_shop8:
    lift_summary_2023_shop8[item]['avg_lift'] = lift_summary_2023_shop8[item]['total_lift'] / lift_summary_2023_shop8[item]['count']

# Node sizes based on average lift, scaled appropriately
node_sizes_2023_shop8_lift = [lift_summary_2023_shop8[node]['avg_lift'] * 500 if node in lift_summary_2023_shop8 else 1000 for node in G_2023_shop8_lift.nodes()]

pos_2023_shop8_lift = nx.spring_layout(G_2023_shop8_lift)

# Use a colormap that transitions from orange to red
cmap_2023_shop8_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop8_lift = np.array(node_sizes_2023_shop8_lift)
normalized_sizes_2023_shop8_lift = (normalized_sizes_2023_shop8_lift - min(normalized_sizes_2023_shop8_lift)) / (max(normalized_sizes_2023_shop8_lift) - min(normalized_sizes_2023_shop8_lift))
node_colors_2023_shop8_lift = [cmap_2023_shop8_lift(size) for size in normalized_sizes_2023_shop8_lift]

plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2023_shop8_lift, pos_2023_shop8_lift, node_size=node_sizes_2023_shop8_lift, node_color=node_colors_2023_shop8_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop8_lift, pos_2023_shop8_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop8_lift, pos_2023_shop8_lift, font_size=10)

plt.title('Node Size Based on Average Lift for 2023 Shop 8')
plt.axis('off')
plt.show()


# Based on conviction

In [ ]:
rules_2023_shop8_nonredundant.head()

In [ ]:
G_2023_shop8_conviction = nx.DiGraph()

conviction_summary_2023_shop8 = {}
count_summary_2023_shop8 = {}

# Assuming 'rules_2023_shop8_nonredundant_unfiltered' contains a 'conviction' column
for _, row in rules_2023_shop8_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop8_conviction.add_edge(antecedents, consequents, weight=row['confidence'], conviction=row['conviction'])
    
    # Aggregate conviction values and count occurrences
    if consequents in conviction_summary_2023_shop8:
        conviction_summary_2023_shop8[consequents] += row['conviction']
        count_summary_2023_shop8[consequents] += 1
    else:
        conviction_summary_2023_shop8[consequents] = row['conviction']
        count_summary_2023_shop8[consequents] = 1

# Calculate average conviction values
average_conviction_summary_2023_shop8 = {k: conviction_summary_2023_shop8[k] / count_summary_2023_shop8[k] for k in conviction_summary_2023_shop8}

# Normalize conviction values for sizing and coloring
max_conviction = max(average_conviction_summary_2023_shop8.values())
min_conviction = min(average_conviction_summary_2023_shop8.values())
conviction_range = max_conviction - min_conviction
node_colors = []
node_sizes = []

for node in G_2023_shop8_conviction.nodes():
    # Get the average conviction value for the node, defaulting to min if not present
    conviction = average_conviction_summary_2023_shop8.get(node, min_conviction)
    # Normalize between 0 and 1
    normalized_conviction = (conviction - min_conviction) / conviction_range if conviction_range else 0
    node_colors.append(normalized_conviction)
    node_sizes.append(normalized_conviction * 1000 + 100)  # Scale and offset size

# Use a colormap for coloring nodes based on conviction
cmap = plt.cm.coolwarm

pos = nx.spring_layout(G_2023_shop8_conviction)

plt.figure(figsize=(40, 22))
nodes = nx.draw_networkx_nodes(G_2023_shop8_conviction, pos, node_color=node_colors, node_size=node_sizes, cmap=cmap, alpha=0.8)
edges = nx.draw_networkx_edges(G_2023_shop8_conviction, pos, arrowstyle='->', arrowsize=10, edge_color='grey', alpha=0.5)
labels = nx.draw_networkx_labels(G_2023_shop8_conviction, pos, font_size=8)

plt.colorbar(nodes, label='Normalized Conviction')
plt.title('Network Graph of 2023 Shop 8 Based on Conviction')
plt.axis('off')
plt.show()

In [ ]:
print(max_conviction)

# Filtered top 30 and non redundant

In [ ]:
rules_2023_shop8_nonredundant.head()

In [ ]:
# Assuming rules_1992_shop2_nonredundant is your sorted rules DataFrame for shop2
rules_2023_shop8_nonredundant_top30 = rules_2023_shop8_nonredundant.copy(deep=True)

In [ ]:
rules_2023_shop8_nonredundant_top30_lift = rules_2023_shop8_nonredundant.copy(deep=True)

In [ ]:
# Select the top 30 rules based on lift or another criterion
rules_2023_shop8_nonredundant_top30_lift = rules_2023_shop8_nonredundant_top30_lift.sort_values(by='lift', ascending=False).head(30)

In [ ]:
rules_2023_shop8_nonredundant_top30_lift.head()

In [ ]:
# Select the top 30 rules based on confidence or another criterion
rules_2023_shop8_nonredundant_top30 = rules_2023_shop8_nonredundant_top30.sort_values(by='confidence', ascending=False).head(30)

In [ ]:
rules_2023_shop8_nonredundant_top30.info()

In [ ]:
rules_2023_shop8_nonredundant_top30['LHS'] = [','.join(list(x)) for x in rules_2023_shop8_nonredundant_top30['antecedents']]
rules_2023_shop8_nonredundant_top30['RHS'] = [','.join(list(x)) for x in rules_2023_shop8_nonredundant_top30['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2023_shop8_nonredundant_top30.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2023_shop8_nonredundant_top30, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2023 Shop 8")
plt.axis('off')
plt.show()


## Based on Indegree

In [ ]:
G_2023_shop8 = nx.DiGraph()

# Assuming 'rules_sorted_2023_shop8' contains association rules for shop 8 in 2023
for _, row in rules_2023_shop8_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as the weight
    G_2023_shop8.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2023_shop8 = nx.spring_layout(G_2023_shop8)

# Calculate node sizes based on indegree
in_degrees_2023_shop8 = dict(G_2023_shop8.in_degree())
node_sizes_2023_shop8 = [in_degrees_2023_shop8[node] * 100 + 1000 for node in G_2023_shop8.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2023_shop8 = (np.array(node_sizes_2023_shop8) - np.min(node_sizes_2023_shop8)) / (np.max(node_sizes_2023_shop8) - np.min(node_sizes_2023_shop8))

# Define a colormap from orange to red
cmap_2023_shop8 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop8 = [cmap_2023_shop8(size) for size in normalized_sizes_2023_shop8]

# Drawing the network
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2023_shop8, pos_2023_shop8, node_size=node_sizes_2023_shop8, node_color=node_colors_2023_shop8, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop8, pos_2023_shop8, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2023_shop8, pos_2023_shop8, font_size=8)

plt.title('Top 5 Association Rules Network for 2023 Shop 8 with Color Gradient')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2023_shop8_lift = nx.DiGraph()

lift_summary_2023_shop8 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop8_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop8_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop8:
        lift_summary_2023_shop8[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop8[consequents]['count'] += 1
    else:
        lift_summary_2023_shop8[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop8:
    lift_summary_2023_shop8[item]['avg_lift'] = lift_summary_2023_shop8[item]['total_lift'] / lift_summary_2023_shop8[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop8_lift = [lift_summary_2023_shop8[node]['avg_lift'] * 500 if node in lift_summary_2023_shop8 else 1000 for node in G_2023_shop8_lift.nodes()]

pos_2023_shop8_lift = nx.spring_layout(G_2023_shop8_lift)

# Colormap from orange to red
cmap_2023_shop8_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop8_lift = np.array(node_sizes_2023_shop8_lift)
normalized_sizes_2023_shop8_lift = (normalized_sizes_2023_shop8_lift - min(normalized_sizes_2023_shop8_lift)) / (max(normalized_sizes_2023_shop8_lift) - min(normalized_sizes_2023_shop8_lift))
node_colors_2023_shop8_lift = [cmap_2023_shop8_lift(size) for size in normalized_sizes_2023_shop8_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2023_shop8_lift, pos_2023_shop8_lift, node_size=node_sizes_2023_shop8_lift, node_color=node_colors_2023_shop8_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop8_lift, pos_2023_shop8_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop8_lift, pos_2023_shop8_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 8 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop8_lift = nx.DiGraph()

lift_summary_2023_shop8 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop8_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop8_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop8:
        lift_summary_2023_shop8[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop8[consequents]['count'] += 1
    else:
        lift_summary_2023_shop8[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop8:
    lift_summary_2023_shop8[item]['avg_lift'] = lift_summary_2023_shop8[item]['total_lift'] / lift_summary_2023_shop8[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop8_lift = [lift_summary_2023_shop8[node]['avg_lift'] * 500 if node in lift_summary_2023_shop8 else 1000 for node in G_2023_shop8_lift.nodes()]

pos_2023_shop8_lift = nx.spring_layout(G_2023_shop8_lift)

# Colormap from orange to red
cmap_2023_shop8_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop8_lift = np.array(node_sizes_2023_shop8_lift)
normalized_sizes_2023_shop8_lift = (normalized_sizes_2023_shop8_lift - min(normalized_sizes_2023_shop8_lift)) / (max(normalized_sizes_2023_shop8_lift) - min(normalized_sizes_2023_shop8_lift))
node_colors_2023_shop8_lift = [cmap_2023_shop8_lift(size) for size in normalized_sizes_2023_shop8_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2023_shop8_lift, pos_2023_shop8_lift, node_size=node_sizes_2023_shop8_lift, node_color=node_colors_2023_shop8_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop8_lift, pos_2023_shop8_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop8_lift, pos_2023_shop8_lift, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 8 Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop8 = nx.DiGraph()

# Assuming 'rules_2023_shop8_nonredundant' contains association rules for 2023 Shop 8
for _, row in rules_2023_shop8_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop8.add_edge(antecedents, consequents, confidence=row['confidence'])

# Calculate node sizes based on indegree
in_degrees_2023_shop8 = dict(G_2023_shop8.in_degree())
node_sizes_2023_shop8 = [in_degrees_2023_shop8[node] * 100 + 1000 for node in G_2023_shop8.nodes()]

# Edge width based on confidence
edge_widths_2023_shop8 = [G_2023_shop8[u][v]['confidence'] * 15 for u, v in G_2023_shop8.edges()]

# Assuming normalized sizes and color mapping is desired
normalized_sizes_2023_shop8 = np.array(list(in_degrees_2023_shop8.values()))
normalized_sizes_2023_shop8 = (normalized_sizes_2023_shop8 - min(normalized_sizes_2023_shop8)) / (max(normalized_sizes_2023_shop8) - min(normalized_sizes_2023_shop8))
cmap_2023_shop8 = mcolors.LinearSegmentedColormap.from_list("MyCMapName", ["blue", "red"])

# Node colors based on a property (here, using indegree for example)
node_colors_2023_shop8 = [cmap_2023_shop8(size) for size in normalized_sizes_2023_shop8]

# Layout
pos_2023_shop8 = nx.spring_layout(G_2023_shop8, seed=42)

# Drawing
plt.figure(figsize=(40, 30))
nx.draw_networkx_edges(G_2023_shop8, pos_2023_shop8, width=edge_widths_2023_shop8, edge_color='grey', alpha=0.5)
nx.draw_networkx_nodes(G_2023_shop8, pos_2023_shop8, node_size=node_sizes_2023_shop8, node_color=node_colors_2023_shop8, alpha=0.8)
nx.draw_networkx_labels(G_2023_shop8, pos_2023_shop8, font_size=8, font_color='black')
plt.title('Network Graph of Association Rules Based on Confidence for 2023 Shop 8')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop8 = nx.DiGraph()

convictions_2023_shop8 = []  # To store conviction values for normalization

for _, row in rules_2023_shop8_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    conviction = row['conviction']  # Assuming each row has a 'conviction' column
    convictions_2023_shop8.append(conviction)
    G_2023_shop8.add_edge(antecedents, consequents, weight=row['confidence'], conviction=conviction)

# Normalize conviction values for edge width
max_conviction_2023_shop8 = max(convictions_2023_shop8)
min_conviction_2023_shop8 = min(convictions_2023_shop8)
conviction_range_2023_shop8 = max_conviction_2023_shop8 - min_conviction_2023_shop8
edge_widths_2023_shop8 = [(G_2023_shop8[u][v]['conviction'] - min_conviction_2023_shop8) / conviction_range_2023_shop8 * 30 + 0.5 for u, v in G_2023_shop8.edges()]  # Scale and offset widths

pos_2023_shop8 = nx.spring_layout(G_2023_shop8)

# Node sizes and colors as before
in_degrees_2023_shop8 = dict(G_2023_shop8.in_degree())
node_sizes_2023_shop8 = [in_degrees_2023_shop8[node] * 100 + 1000 for node in G_2023_shop8.nodes()]
normalized_sizes_2023_shop8 = (np.array(node_sizes_2023_shop8) - min(node_sizes_2023_shop8)) / (max(node_sizes_2023_shop8) - min(node_sizes_2023_shop8))
cmap_2023_shop8 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop8 = [cmap_2023_shop8(size) for size in normalized_sizes_2023_shop8]

# Drawing the network with conviction-based edge widths
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2023_shop8, pos_2023_shop8, node_size=node_sizes_2023_shop8, node_color=node_colors_2023_shop8, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop8, pos_2023_shop8, width=edge_widths_2023_shop8, edge_color='grey', alpha=0.5)
nx.draw_networkx_labels(G_2023_shop8, pos_2023_shop8, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 8 with Conviction-based Edge Thickness')
plt.axis('off')
plt.show()


In [ ]:
#the node sizes are based on the indegree where each node acts as a consequent,

In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2023_shop8_nonredundant_top30.loc[:, 'antecedents_'] = rules_2023_shop8_nonredundant_top30['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2023_shop8_nonredundant_top30.loc[:, 'consequents_'] = rules_2023_shop8_nonredundant_top30['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2023_shop8_nonredundant_top30, x='consequents_', y='antecedents_', s=rules_2023_shop8_nonredundant_top30['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2023_shop8_nonredundant_top30.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2023 Shop 8", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
print(rules_2023_shop8_nonredundant_top30[['antecedents', 'consequents']].head(31))

## Monetary Analysis of the Top Rules

In [ ]:
# Step 1: Extract unique product codes from the rules
unique_product_codes = set()
for idx, row in rules_2023_shop8_nonredundant_top30.iterrows():
    # Unite antecedents and consequents into a single set
    all_products = row['antecedents'].union(row['consequents'])
    unique_product_codes.update(all_products)

# Step 2: Create a DataFrame with unique product codes and their unit prices
# Filter sales_data for these product codes and drop duplicates to ensure each product code appears once
product_prices = sales_data[sales_data['product_code'].isin(unique_product_codes)][['product_code', 'unit_price']].drop_duplicates()

# Create a dictionary for quick lookup of prices
price_dict = product_prices.set_index('product_code')['unit_price'].to_dict()

# Step 3: Calculate the total price for each rule and collect data for the new DataFrame
data_for_new_df = []
for idx, row in rules_2023_shop8_nonredundant_top30.iterrows():
    # Combine sets of antecedents and consequents
    all_products = row['antecedents'].union(row['consequents'])
    total_price = sum(price_dict.get(prod, 0) for prod in all_products)  # Use get to avoid KeyError if any product code is missing
    data_for_new_df.append({
        'antecedents': row['antecedents'],
        'consequents': row['consequents'],
        'total_unit_price': total_price
    })

# Create a new DataFrame with only antecedents, consequents, and total unit price
rules_and_price_shop8_2023 = pd.DataFrame(data_for_new_df)

# Sort the DataFrame by total_unit_price in descending order
rules_and_price_shop8_2023_sorted = rules_and_price_shop8_2023.sort_values(by='total_unit_price', ascending=False)

# Print the sorted DataFrame to see the specified columns along with their total unit prices
print(rules_and_price_shop8_2023_sorted)


In [ ]:
# Convert 'trx_date' to datetime format if not already done so
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Add a 'year' column if it doesn't exist
sales_data['year'] = sales_data['trx_date'].dt.year

# Step 2: Filter sales_data for shop8 and the year 2023
filtered_sales_data_2023 = sales_data[(sales_data['shopID'] == 8) & (sales_data['year'] == 2023)]

# Step 3: Initialize a list to hold the rule data including total sales value
rule_sales_summaries_2023 = []

# Step 4: Iterate through each rule in the DataFrame
for index, rule in rules_2023_shop8_nonredundant_top30.iterrows():
    all_products = rule['antecedents'].union(rule['consequents'])

    # Filter the sales data for transactions that include all products in the rule
    # This step ensures that we are only considering transactions that contain all specified products
    mask = filtered_sales_data_2023['product_code'].isin(all_products)
    transaction_ids = filtered_sales_data_2023[mask].groupby('trx_id').filter(lambda x: all_products.issubset(set(x['product_code']))).trx_id.unique()
    matched_transactions = filtered_sales_data_2023[filtered_sales_data_2023['trx_id'].isin(transaction_ids)]
    
    # Step 5: Sum the total_value of these transactions
    total_sales_value = matched_transactions['total_value'].sum()

    # Append the result to the list
    rule_sales_summaries_2023.append({
        'antecedents': rule['antecedents'],
        'consequents': rule['consequents'],
        'total_sales_value': total_sales_value
    })

# Step 6: Convert the list to a DataFrame
rules_sales_values_df_2023 = pd.DataFrame(rule_sales_summaries_2023)

# Step 7: Sort by total_sales_value in descending order
sorted_rules_sales_values_df_2023 = rules_sales_values_df_2023.sort_values(by='total_sales_value', ascending=False)

# Step 8: Print the sorted DataFrame
print(sorted_rules_sales_values_df_2023)


## Narrowing down based on Lift

In [ ]:
rules_2023_shop8_nonredundant_top30_lift['LHS'] = [','.join(list(x)) for x in rules_2023_shop8_nonredundant_top30_lift['antecedents']]
rules_2023_shop8_nonredundant_top30_lift['RHS'] = [','.join(list(x)) for x in rules_2023_shop8_nonredundant_top30_lift['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2023_shop8_nonredundant_top30_lift.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2023_shop8_nonredundant_top30_lift, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2023 Shop 8")
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop8_lift = nx.DiGraph()

lift_summary_2023_shop8 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop8_nonredundant_top30_lift.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop8_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop8:
        lift_summary_2023_shop8[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop8[consequents]['count'] += 1
    else:
        lift_summary_2023_shop8[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop8:
    lift_summary_2023_shop8[item]['avg_lift'] = lift_summary_2023_shop8[item]['total_lift'] / lift_summary_2023_shop8[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop8_lift = [lift_summary_2023_shop8[node]['avg_lift'] * 500 if node in lift_summary_2023_shop8 else 1000 for node in G_2023_shop8_lift.nodes()]

pos_2023_shop8_lift = nx.spring_layout(G_2023_shop8_lift)

# Colormap from orange to red
cmap_2023_shop8_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop8_lift = np.array(node_sizes_2023_shop8_lift)
normalized_sizes_2023_shop8_lift = (normalized_sizes_2023_shop8_lift - min(normalized_sizes_2023_shop8_lift)) / (max(normalized_sizes_2023_shop8_lift) - min(normalized_sizes_2023_shop8_lift))
node_colors_2023_shop8_lift = [cmap_2023_shop8_lift(size) for size in normalized_sizes_2023_shop8_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2023_shop8_lift, pos_2023_shop8_lift, node_size=node_sizes_2023_shop8_lift, node_color=node_colors_2023_shop8_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop8_lift, pos_2023_shop8_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop8_lift, pos_2023_shop8_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 8 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2023_shop8_nonredundant_top30_lift.loc[:, 'antecedents_'] = rules_2023_shop8_nonredundant_top30_lift['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2023_shop8_nonredundant_top30_lift.loc[:, 'consequents_'] = rules_2023_shop8_nonredundant_top30_lift['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2023_shop8_nonredundant_top30_lift, x='consequents_', y='antecedents_', s=rules_2023_shop8_nonredundant_top30_lift['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2023_shop8_nonredundant_top30_lift.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2023 Shop 8", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
# Print only the 'antecedents' and 'consequents' columns of the top 10 rules
print(rules_2023_shop8_nonredundant_top30_lift[['antecedents', 'consequents']].head(10))


# Shop9 2022 SEARCH 17

In [ ]:
# Filter for records from shop ID 1 and the year 1993
sales_data_2022_shop9 = sales_data_shop9[(sales_data_shop9['trx_date'].dt.year == 2022)]

In [ ]:
sales_data_2022_shop9.head()

In [ ]:
# Grouping product codes by trx_id
transactions_2022_shop9 = sales_data_2022_shop9.groupby('trx_id')['product_code'].apply(list).tolist()

In [ ]:
# Encode transaction data
encoder_2022_shop9 = TransactionEncoder()
trans_array_2022_shop9 = encoder_2022_shop9.fit(transactions_2022_shop9).transform(transactions_2022_shop9)
trans_df_2022_shop9 = pd.DataFrame(trans_array_2022_shop9, columns=encoder_2022_shop9.columns_)

In [ ]:
# Find frequent itemsets
frequent_itemsets_2022_shop9 = apriori(trans_df_2022_shop9, min_support=0.001, use_colnames=True)

In [ ]:
print(frequent_itemsets_2022_shop9)

In [ ]:
## Generate the association rules - by confidence
rulesConfidence_2022_shop9 = association_rules(frequent_itemsets_2022_shop9, metric="confidence", min_threshold=0.10)
rulesConfidence_2022_shop9.sort_values(by='confidence', ascending=False, inplace=True)


In [ ]:
# Note: This is a hypothetical example, as I can't directly manipulate or access dataframes not provided within this code cell.

# Replace frozen sets with strings
rulesConfidence_2022_shop9['antecedents_'] = rulesConfidence_2022_shop9['antecedents'].apply(lambda a: ','.join(list(a)))
rulesConfidence_2022_shop9['consequents_'] = rulesConfidence_2022_shop9['consequents'].apply(lambda a: ','.join(list(a)))

# Draw
fig, ax = plt.subplots(figsize=(14,30))
ax = plt.scatter(data=rulesConfidence_2022_shop9, x='consequents_', y='antecedents_', s=rulesConfidence_2022_shop9['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rulesConfidence_2022_shop9.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2020 Shop 9", fontsize=14)
plt.xlabel("Consequents (RHS)", fontsize=12)
plt.ylabel("Antecedents (LHS)", fontsize=12)
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate(rotation=45)
plt.tight_layout()

plt.show()


## Eliminate Redundant Rules

In [ ]:
rulesConfidence_2022_shop9.head()

In [ ]:
rulesConfidence_2022_shop9.info()

In [ ]:
# If 'antecedents' and 'consequents' are not frozensets, convert them
rules_2022_shop9_redundant = rulesConfidence_2022_shop9.copy()
rules_2022_shop9_redundant['antecedents'] = rules_2022_shop9_redundant['antecedents'].apply(lambda x: frozenset(x))
rules_2022_shop9_redundant['consequents'] = rules_2022_shop9_redundant['consequents'].apply(lambda x: frozenset(x))


In [ ]:
# Create a copy of the DataFrame to avoid modifying the original while iterating
rules_2022_shop9_checkredundance = rules_2022_shop9_redundant.copy()

# List to hold non-redundant rules
non_redundant_rules_2022_shop9 = []

for _, rule in rules_2022_shop9_checkredundance.iterrows():
    if not is_redundant(rule, rules_2022_shop9_checkredundance):
        non_redundant_rules_2022_shop9.append(rule)

# Convert the list back to a DataFrame
rules_2022_shop9_nonredundant = pd.DataFrame(non_redundant_rules_2022_shop9).reset_index(drop=True)


In [ ]:
print(rules_2022_shop9_nonredundant.head())
print(f"Original number of rules: {len(rules_2022_shop9_redundant)}")
print(f"Number of non-redundant rules: {len(rules_2022_shop9_nonredundant)}")


In [ ]:
rules_2022_shop9_nonredundant.head()

In [ ]:
rules_2022_shop9_redundant.info()

In [ ]:
rules_2022_shop9_nonredundant.head()

In [ ]:
rules_2022_shop9_nonredundant_backup = rules_2022_shop9_nonredundant.copy()

In [ ]:
rules_2022_shop9_nonredundant_unfiltered = rules_2022_shop9_nonredundant.copy()

In [ ]:
#reset
#rules_2022_shop9_nonredundant = rules_2022_shop9_nonredundant_backup.copy()

# Redundant and Unfiltered

In [ ]:
rules_2022_shop9_redundant.head()

In [ ]:
G_2022_shop9 = nx.DiGraph()

# Iterate through rules for 2022 Shop 9 and add edges to the graph
for _, row in rules_2022_shop9_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as weight
    G_2022_shop9.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2022_shop9 = nx.spring_layout(G_2022_shop9)

# Recalculate node sizes based on indegree or another metric
in_degrees_2022_shop9 = dict(G_2022_shop9.in_degree())
node_sizes_2022_shop9 = [in_degrees_2022_shop9[node] * 100 + 1000 for node in G_2022_shop9.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2022_shop9 = (np.array(node_sizes_2022_shop9) - np.min(node_sizes_2022_shop9)) / (np.max(node_sizes_2022_shop9) - np.min(node_sizes_2022_shop9))

# Define a colormap from orange to red for visual consistency
cmap_2022_shop9 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2022_shop9 = [cmap_2022_shop9(size) for size in normalized_sizes_2022_shop9]

# Drawing the network
plt.figure(figsize=(60, 43))
nx.draw_networkx_nodes(G_2022_shop9, pos_2022_shop9, node_size=node_sizes_2022_shop9, node_color=node_colors_2022_shop9, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop9, pos_2022_shop9, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop9, pos_2022_shop9, font_size=10)

plt.title('Top 5 Association Rules Network for 2022 Shop 9 with Color Gradient')
plt.axis('off')
plt.show()


### Based on LIFT

In [ ]:
G_2022_shop9_lift = nx.DiGraph()

lift_summary_2022_shop9 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2022_shop9_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop9_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop9:
        lift_summary_2022_shop9[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop9[consequents]['count'] += 1
    else:
        lift_summary_2022_shop9[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop9:
    lift_summary_2022_shop9[item]['avg_lift'] = lift_summary_2022_shop9[item]['total_lift'] / lift_summary_2022_shop9[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop9_lift = [lift_summary_2022_shop9[node]['avg_lift'] * 500 if node in lift_summary_2022_shop9 else 1000 for node in G_2022_shop9_lift.nodes()]

pos_2022_shop9_lift = nx.spring_layout(G_2022_shop9_lift)

# Colormap from orange to red
cmap_2022_shop9_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop9_lift = np.array(node_sizes_2022_shop9_lift)
normalized_sizes_2022_shop9_lift = (normalized_sizes_2022_shop9_lift - min(normalized_sizes_2022_shop9_lift)) / (max(normalized_sizes_2022_shop9_lift) - min(normalized_sizes_2022_shop9_lift))
node_colors_2022_shop9_lift = [cmap_2022_shop9_lift(size) for size in normalized_sizes_2022_shop9_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2022_shop9_lift, pos_2022_shop9_lift, node_size=node_sizes_2022_shop9_lift, node_color=node_colors_2022_shop9_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop9_lift, pos_2022_shop9_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop9_lift, pos_2022_shop9_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2022 Shop 9 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


# nonredundant_unfiltered

## Based on Indegree

In [ ]:
G_2022_shop9_nonredundant_unfiltered = nx.DiGraph()

# Iterating through each row to add edges with confidence as an attribute
for _, row in rules_2022_shop9_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop9_nonredundant_unfiltered.add_edge(antecedents, consequents, weight=row['confidence'])

# Defining the layout for network visualization
pos_2022_shop9_nonredundant_unfiltered = nx.spring_layout(G_2022_shop9_nonredundant_unfiltered)

# Calculating node sizes based on the indegree for color mapping
in_degrees_2022_shop9_nonredundant_unfiltered = dict(G_2022_shop9_nonredundant_unfiltered.in_degree())
node_sizes_2022_shop9_nonredundant_unfiltered = [in_degrees_2022_shop9_nonredundant_unfiltered[node] * 100 + 1000 for node in G_2022_shop9_nonredundant_unfiltered.nodes()]

# Normalizing node sizes for color mapping
normalized_sizes_2022_shop9_nonredundant_unfiltered = (np.array(node_sizes_2022_shop9_nonredundant_unfiltered) - min(node_sizes_2022_shop9_nonredundant_unfiltered)) / (max(node_sizes_2022_shop9_nonredundant_unfiltered) - min(node_sizes_2022_shop9_nonredundant_unfiltered))

# Defining a color map from orange to red
cmap_2022_shop9_nonredundant_unfiltered = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])

# Mapping normalized node sizes to colors
node_colors_2022_shop9_nonredundant_unfiltered = [cmap_2022_shop9_nonredundant_unfiltered(size) for size in normalized_sizes_2022_shop9_nonredundant_unfiltered]

# Drawing the network with node sizes and colors
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2022_shop9_nonredundant_unfiltered, pos_2022_shop9_nonredundant_unfiltered, node_size=node_sizes_2022_shop9_nonredundant_unfiltered, node_color=node_colors_2022_shop9_nonredundant_unfiltered, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop9_nonredundant_unfiltered, pos_2022_shop9_nonredundant_unfiltered, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2022_shop9_nonredundant_unfiltered, pos_2022_shop9_nonredundant_unfiltered, font_size=10)

plt.title('Node Size Based on Indegree for 2022 Shop 9')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2022_shop9_lift = nx.DiGraph()

lift_summary_2022_shop9 = {}

for _, row in rules_2022_shop9_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop9_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Update lift summary for consequents
    if consequents in lift_summary_2022_shop9:
        lift_summary_2022_shop9[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop9[consequents]['count'] += 1
    else:
        lift_summary_2022_shop9[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculate average lift for each consequent
for item in lift_summary_2022_shop9:
    lift_summary_2022_shop9[item]['avg_lift'] = lift_summary_2022_shop9[item]['total_lift'] / lift_summary_2022_shop9[item]['count']

# Node sizes based on average lift, scaled appropriately
node_sizes_2022_shop9_lift = [lift_summary_2022_shop9[node]['avg_lift'] * 500 if node in lift_summary_2022_shop9 else 1000 for node in G_2022_shop9_lift.nodes()]

pos_2022_shop9_lift = nx.spring_layout(G_2022_shop9_lift)

# Use a colormap that transitions from orange to red
cmap_2022_shop9_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop9_lift = np.array(node_sizes_2022_shop9_lift)
normalized_sizes_2022_shop9_lift = (normalized_sizes_2022_shop9_lift - min(normalized_sizes_2022_shop9_lift)) / (max(normalized_sizes_2022_shop9_lift) - min(normalized_sizes_2022_shop9_lift))
node_colors_2022_shop9_lift = [cmap_2022_shop9_lift(size) for size in normalized_sizes_2022_shop9_lift]

plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2022_shop9_lift, pos_2022_shop9_lift, node_size=node_sizes_2022_shop9_lift, node_color=node_colors_2022_shop9_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop9_lift, pos_2022_shop9_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop9_lift, pos_2022_shop9_lift, font_size=10)

plt.title('Node Size Based on Average Lift for 2022 Shop 9')
plt.axis('off')
plt.show()


# Based on conviction

In [ ]:
rules_2022_shop9_nonredundant.head()

In [ ]:
G_2022_shop9_conviction = nx.DiGraph()

conviction_summary_2022_shop9 = {}
count_summary_2022_shop9 = {}

# Assuming 'rules_2022_shop9_nonredundant_unfiltered' contains a 'conviction' column
for _, row in rules_2022_shop9_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop9_conviction.add_edge(antecedents, consequents, weight=row['confidence'], conviction=row['conviction'])
    
    # Aggregate conviction values and count occurrences
    if consequents in conviction_summary_2022_shop9:
        conviction_summary_2022_shop9[consequents] += row['conviction']
        count_summary_2022_shop9[consequents] += 1
    else:
        conviction_summary_2022_shop9[consequents] = row['conviction']
        count_summary_2022_shop9[consequents] = 1

# Calculate average conviction values
average_conviction_summary_2022_shop9 = {k: conviction_summary_2022_shop9[k] / count_summary_2022_shop9[k] for k in conviction_summary_2022_shop9}

# Normalize conviction values for sizing and coloring
max_conviction = max(average_conviction_summary_2022_shop9.values())
min_conviction = min(average_conviction_summary_2022_shop9.values())
conviction_range = max_conviction - min_conviction
node_colors = []
node_sizes = []

for node in G_2022_shop9_conviction.nodes():
    # Get the average conviction value for the node, defaulting to min if not present
    conviction = average_conviction_summary_2022_shop9.get(node, min_conviction)
    # Normalize between 0 and 1
    normalized_conviction = (conviction - min_conviction) / conviction_range if conviction_range else 0
    node_colors.append(normalized_conviction)
    node_sizes.append(normalized_conviction * 1000 + 100)  # Scale and offset size

# Use a colormap for coloring nodes based on conviction
cmap = plt.cm.coolwarm

pos = nx.spring_layout(G_2022_shop9_conviction)

plt.figure(figsize=(40, 22))
nodes = nx.draw_networkx_nodes(G_2022_shop9_conviction, pos, node_color=node_colors, node_size=node_sizes, cmap=cmap, alpha=0.8)
edges = nx.draw_networkx_edges(G_2022_shop9_conviction, pos, arrowstyle='->', arrowsize=10, edge_color='grey', alpha=0.5)
labels = nx.draw_networkx_labels(G_2022_shop9_conviction, pos, font_size=8)

plt.colorbar(nodes, label='Normalized Conviction')
plt.title('Network Graph of 2022 Shop 9 Based on Conviction')
plt.axis('off')
plt.show()

In [ ]:
print(max_conviction)

# Filtered top 30 and non redundant

In [ ]:
rules_2022_shop9_nonredundant.head()

In [ ]:
# Assuming rules_1992_shop2_nonredundant is your sorted rules DataFrame for shop2
rules_2022_shop9_nonredundant_top30 = rules_2022_shop9_nonredundant.copy(deep=True)

In [ ]:
rules_2022_shop9_nonredundant_top30_lift = rules_2022_shop9_nonredundant.copy(deep=True)

In [ ]:
# Select the top 30 rules based on lift or another criterion
rules_2022_shop9_nonredundant_top30_lift = rules_2022_shop9_nonredundant_top30_lift.sort_values(by='lift', ascending=False).head(30)

In [ ]:
rules_2022_shop9_nonredundant_top30_lift.head()

In [ ]:
# Select the top 30 rules based on confidence or another criterion
rules_2022_shop9_nonredundant_top30 = rules_2022_shop9_nonredundant_top30.sort_values(by='confidence', ascending=False).head(30)

In [ ]:
rules_2022_shop9_nonredundant_top30.info()

In [ ]:
rules_2022_shop9_nonredundant_top30['LHS'] = [','.join(list(x)) for x in rules_2022_shop9_nonredundant_top30['antecedents']]
rules_2022_shop9_nonredundant_top30['RHS'] = [','.join(list(x)) for x in rules_2022_shop9_nonredundant_top30['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2022_shop9_nonredundant_top30.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2022_shop9_nonredundant_top30, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2022 Shop 9")
plt.axis('off')
plt.show()


## Based on Indegree

In [ ]:
G_2022_shop9 = nx.DiGraph()

# Assuming 'rules_sorted_2022_shop9' contains association rules for shop 9 in 2022
for _, row in rules_2022_shop9_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as the weight
    G_2022_shop9.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2022_shop9 = nx.spring_layout(G_2022_shop9)

# Calculate node sizes based on indegree
in_degrees_2022_shop9 = dict(G_2022_shop9.in_degree())
node_sizes_2022_shop9 = [in_degrees_2022_shop9[node] * 100 + 1000 for node in G_2022_shop9.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2022_shop9 = (np.array(node_sizes_2022_shop9) - np.min(node_sizes_2022_shop9)) / (np.max(node_sizes_2022_shop9) - np.min(node_sizes_2022_shop9))

# Define a colormap from orange to red
cmap_2022_shop9 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2022_shop9 = [cmap_2022_shop9(size) for size in normalized_sizes_2022_shop9]

# Drawing the network
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2022_shop9, pos_2022_shop9, node_size=node_sizes_2022_shop9, node_color=node_colors_2022_shop9, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop9, pos_2022_shop9, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2022_shop9, pos_2022_shop9, font_size=8)

plt.title('Top 5 Association Rules Network for 2022 Shop 9 with Color Gradient')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2022_shop9_lift = nx.DiGraph()

lift_summary_2022_shop9 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2022_shop9_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop9_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop9:
        lift_summary_2022_shop9[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop9[consequents]['count'] += 1
    else:
        lift_summary_2022_shop9[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop9:
    lift_summary_2022_shop9[item]['avg_lift'] = lift_summary_2022_shop9[item]['total_lift'] / lift_summary_2022_shop9[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop9_lift = [lift_summary_2022_shop9[node]['avg_lift'] * 500 if node in lift_summary_2022_shop9 else 1000 for node in G_2022_shop9_lift.nodes()]

pos_2022_shop9_lift = nx.spring_layout(G_2022_shop9_lift)

# Colormap from orange to red
cmap_2022_shop9_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop9_lift = np.array(node_sizes_2022_shop9_lift)
normalized_sizes_2022_shop9_lift = (normalized_sizes_2022_shop9_lift - min(normalized_sizes_2022_shop9_lift)) / (max(normalized_sizes_2022_shop9_lift) - min(normalized_sizes_2022_shop9_lift))
node_colors_2022_shop9_lift = [cmap_2022_shop9_lift(size) for size in normalized_sizes_2022_shop9_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2022_shop9_lift, pos_2022_shop9_lift, node_size=node_sizes_2022_shop9_lift, node_color=node_colors_2022_shop9_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop9_lift, pos_2022_shop9_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop9_lift, pos_2022_shop9_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2022 Shop 9 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop9_lift = nx.DiGraph()

lift_summary_2022_shop9 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2022_shop9_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop9_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop9:
        lift_summary_2022_shop9[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop9[consequents]['count'] += 1
    else:
        lift_summary_2022_shop9[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop9:
    lift_summary_2022_shop9[item]['avg_lift'] = lift_summary_2022_shop9[item]['total_lift'] / lift_summary_2022_shop9[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop9_lift = [lift_summary_2022_shop9[node]['avg_lift'] * 500 if node in lift_summary_2022_shop9 else 1000 for node in G_2022_shop9_lift.nodes()]

pos_2022_shop9_lift = nx.spring_layout(G_2022_shop9_lift)

# Colormap from orange to red
cmap_2022_shop9_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop9_lift = np.array(node_sizes_2022_shop9_lift)
normalized_sizes_2022_shop9_lift = (normalized_sizes_2022_shop9_lift - min(normalized_sizes_2022_shop9_lift)) / (max(normalized_sizes_2022_shop9_lift) - min(normalized_sizes_2022_shop9_lift))
node_colors_2022_shop9_lift = [cmap_2022_shop9_lift(size) for size in normalized_sizes_2022_shop9_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2022_shop9_lift, pos_2022_shop9_lift, node_size=node_sizes_2022_shop9_lift, node_color=node_colors_2022_shop9_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop9_lift, pos_2022_shop9_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop9_lift, pos_2022_shop9_lift, font_size=10)

plt.title('Top 5 Association Rules Network for 2022 Shop 9 Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop9 = nx.DiGraph()

# Assuming 'rules_2022_shop9_nonredundant_top30' contains association rules for 2022 Shop 9
for _, row in rules_2022_shop9_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop9.add_edge(antecedents, consequents, confidence=row['confidence'])

# Calculate node sizes based on indegree
in_degrees_2022_shop9 = dict(G_2022_shop9.in_degree())
node_sizes_2022_shop9 = [in_degrees_2022_shop9[node] * 100 + 1000 for node in G_2022_shop9.nodes()]

# Edge width based on confidence
edge_widths_2022_shop9 = [G_2022_shop9[u][v]['confidence'] * 15 for u, v in G_2022_shop9.edges()]

# Assuming normalized sizes and color mapping is desired
normalized_sizes_2022_shop9 = np.array(list(in_degrees_2022_shop9.values()))
normalized_sizes_2022_shop9 = (normalized_sizes_2022_shop9 - min(normalized_sizes_2022_shop9)) / (max(normalized_sizes_2022_shop9) - min(normalized_sizes_2022_shop9))
cmap_2022_shop9 = mcolors.LinearSegmentedColormap.from_list("MyCMapName", ["blue", "red"])

# Node colors based on a property (here, using indegree for example)
node_colors_2022_shop9 = [cmap_2022_shop9(size) for size in normalized_sizes_2022_shop9]

# Layout
pos_2022_shop9 = nx.spring_layout(G_2022_shop9, seed=42)

# Drawing
plt.figure(figsize=(40, 30))
nx.draw_networkx_edges(G_2022_shop9, pos_2022_shop9, width=edge_widths_2022_shop9, edge_color='grey', alpha=0.5)
nx.draw_networkx_nodes(G_2022_shop9, pos_2022_shop9, node_size=node_sizes_2022_shop9, node_color=node_colors_2022_shop9, alpha=0.8)
nx.draw_networkx_labels(G_2022_shop9, pos_2022_shop9, font_size=8, font_color='black')
plt.title('Network Graph of Association Rules Based on Confidence for 2022 Shop 9')
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop9 = nx.DiGraph()

convictions_2022_shop9 = []  # To store conviction values for normalization

for _, row in rules_2022_shop9_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    conviction_2022_shop9 = row['conviction']  # Assuming each row has a 'conviction' column
    convictions_2022_shop9.append(conviction_2022_shop9)
    G_2022_shop9.add_edge(antecedents, consequents, weight=row['confidence'], conviction=conviction_2022_shop9)

# Normalize conviction values for edge width
max_conviction_2022_shop9 = max(convictions_2022_shop9)
min_conviction_2022_shop9 = min(convictions_2022_shop9)
conviction_range_2022_shop9 = max_conviction_2022_shop9 - min_conviction_2022_shop9
edge_widths_2022_shop9 = [(G_2022_shop9[u][v]['conviction'] - min_conviction_2022_shop9) / conviction_range_2022_shop9 * 30 + 0.5 for u, v in G_2022_shop9.edges()]  # Scale and offset widths

pos_2022_shop9 = nx.spring_layout(G_2022_shop9)

# Node sizes and colors as before
in_degrees_2022_shop9 = dict(G_2022_shop9.in_degree())
node_sizes_2022_shop9 = [in_degrees_2022_shop9[node] * 100 + 1000 for node in G_2022_shop9.nodes()]
normalized_sizes_2022_shop9 = (np.array(node_sizes_2022_shop9) - min(node_sizes_2022_shop9)) / (max(node_sizes_2022_shop9) - min(node_sizes_2022_shop9))
cmap_2022_shop9 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2022_shop9 = [cmap_2022_shop9(size) for size in normalized_sizes_2022_shop9]

# Drawing the network with conviction-based edge widths
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2022_shop9, pos_2022_shop9, node_size=node_sizes_2022_shop9, node_color=node_colors_2022_shop9, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop9, pos_2022_shop9, width=edge_widths_2022_shop9, edge_color='grey', alpha=0.5)
nx.draw_networkx_labels(G_2022_shop9, pos_2022_shop9, font_size=10)

plt.title('Top 5 Association Rules Network for 2022 Shop 9 with Conviction-based Edge Thickness')
plt.axis('off')
plt.show()


In [ ]:
#the node sizes are based on the indegree where each node acts as a consequent,

In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2022_shop9_nonredundant_top30.loc[:, 'antecedents_'] = rules_2022_shop9_nonredundant_top30['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2022_shop9_nonredundant_top30.loc[:, 'consequents_'] = rules_2022_shop9_nonredundant_top30['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2022_shop9_nonredundant_top30, x='consequents_', y='antecedents_', s=rules_2022_shop9_nonredundant_top30['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2022_shop9_nonredundant_top30.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2022 Shop 9", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
print(rules_2022_shop9_nonredundant_top30[['antecedents', 'consequents']].head(10))

## Monetary Analysis of the Top Rules

In [ ]:
# Step 1: Extract unique product codes from the rules
unique_product_codes = set()
for idx, row in rules_2022_shop9_nonredundant_top30.iterrows():
    # Unite antecedents and consequents into a single set
    all_products = row['antecedents'].union(row['consequents'])
    unique_product_codes.update(all_products)

# Step 2: Create a DataFrame with unique product codes and their unit prices
# Filter sales_data for these product codes and drop duplicates to ensure each product code appears once
product_prices = sales_data[sales_data['product_code'].isin(unique_product_codes)][['product_code', 'unit_price']].drop_duplicates()

# Create a dictionary for quick lookup of prices
price_dict = product_prices.set_index('product_code')['unit_price'].to_dict()

# Step 3: Calculate the total price for each rule and collect data for the new DataFrame
data_for_new_df = []
for idx, row in rules_2022_shop9_nonredundant_top30.iterrows():
    # Combine sets of antecedents and consequents
    all_products = row['antecedents'].union(row['consequents'])
    total_price = sum(price_dict.get(prod, 0) for prod in all_products)  # Use get to avoid KeyError if any product code is missing
    data_for_new_df.append({
        'antecedents': row['antecedents'],
        'consequents': row['consequents'],
        'total_unit_price': total_price
    })

# Create a new DataFrame with only antecedents, consequents, and total unit price
rules_and_price_shop9_2022 = pd.DataFrame(data_for_new_df)

# Sort the DataFrame by total_unit_price in descending order
rules_and_price_shop9_2022_sorted = rules_and_price_shop9_2022.sort_values(by='total_unit_price', ascending=False)

# Print the sorted DataFrame to see the specified columns along with their total unit prices
print(rules_and_price_shop9_2022_sorted)


In [ ]:
# Convert 'trx_date' to datetime format if not already done so
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Add a 'year' column if it doesn't exist
sales_data['year'] = sales_data['trx_date'].dt.year

# Step 2: Filter sales_data for shop9 and the year 2022
filtered_sales_data_2022 = sales_data[(sales_data['shopID'] == 9) & (sales_data['year'] == 2022)]

# Step 3: Initialize a list to hold the rule data including total sales value
rule_sales_summaries_2022 = []

# Step 4: Iterate through each rule in the DataFrame
for index, rule in rules_2022_shop9_nonredundant_top30.iterrows():
    all_products = rule['antecedents'].union(rule['consequents'])

    # Filter the sales data for transactions that include all products in the rule
    # This step ensures that we are only considering transactions that contain all specified products
    mask = filtered_sales_data_2022['product_code'].isin(all_products)
    transaction_ids = filtered_sales_data_2022[mask].groupby('trx_id').filter(lambda x: all_products.issubset(set(x['product_code']))).trx_id.unique()
    matched_transactions = filtered_sales_data_2022[filtered_sales_data_2022['trx_id'].isin(transaction_ids)]
    
    # Step 5: Sum the total_value of these transactions
    total_sales_value = matched_transactions['total_value'].sum()

    # Append the result to the list
    rule_sales_summaries_2022.append({
        'antecedents': rule['antecedents'],
        'consequents': rule['consequents'],
        'total_sales_value': total_sales_value
    })

# Step 6: Convert the list to a DataFrame
rules_sales_values_df_2022 = pd.DataFrame(rule_sales_summaries_2022)

# Step 7: Sort by total_sales_value in descending order
sorted_rules_sales_values_df_2022 = rules_sales_values_df_2022.sort_values(by='total_sales_value', ascending=False)

# Step 8: Print the sorted DataFrame
print(sorted_rules_sales_values_df_2022)


## Narrowing down based on Lift

In [ ]:
rules_2022_shop9_nonredundant_top30_lift['LHS'] = [','.join(list(x)) for x in rules_2022_shop9_nonredundant_top30_lift['antecedents']]
rules_2022_shop9_nonredundant_top30_lift['RHS'] = [','.join(list(x)) for x in rules_2022_shop9_nonredundant_top30_lift['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2022_shop9_nonredundant_top30_lift.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2022_shop9_nonredundant_top30_lift, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2022 Shop 9")
plt.axis('off')
plt.show()


In [ ]:
G_2022_shop9_lift = nx.DiGraph()

lift_summary_2022_shop9 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2022_shop9_nonredundant_top30_lift.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2022_shop9_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2022_shop9:
        lift_summary_2022_shop9[consequents]['total_lift'] += row['lift']
        lift_summary_2022_shop9[consequents]['count'] += 1
    else:
        lift_summary_2022_shop9[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2022_shop9:
    lift_summary_2022_shop9[item]['avg_lift'] = lift_summary_2022_shop9[item]['total_lift'] / lift_summary_2022_shop9[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2022_shop9_lift = [lift_summary_2022_shop9[node]['avg_lift'] * 500 if node in lift_summary_2022_shop9 else 1000 for node in G_2022_shop9_lift.nodes()]

pos_2022_shop9_lift = nx.spring_layout(G_2022_shop9_lift)

# Colormap from orange to red
cmap_2022_shop9_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2022_shop9_lift = np.array(node_sizes_2022_shop9_lift)
normalized_sizes_2022_shop9_lift = (normalized_sizes_2022_shop9_lift - min(normalized_sizes_2022_shop9_lift)) / (max(normalized_sizes_2022_shop9_lift) - min(normalized_sizes_2022_shop9_lift))
node_colors_2022_shop9_lift = [cmap_2022_shop9_lift(size) for size in normalized_sizes_2022_shop9_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2022_shop9_lift, pos_2022_shop9_lift, node_size=node_sizes_2022_shop9_lift, node_color=node_colors_2022_shop9_lift, alpha=0.8)
nx.draw_networkx_edges(G_2022_shop9_lift, pos_2022_shop9_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2022_shop9_lift, pos_2022_shop9_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2022 Shop 9 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2022_shop9_nonredundant_top30_lift.loc[:, 'antecedents_'] = rules_2022_shop9_nonredundant_top30_lift['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2022_shop9_nonredundant_top30_lift.loc[:, 'consequents_'] = rules_2022_shop9_nonredundant_top30_lift['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2022_shop9_nonredundant_top30_lift, x='consequents_', y='antecedents_', s=rules_2022_shop9_nonredundant_top30_lift['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2022_shop9_nonredundant_top30_lift.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2022 Shop 9", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
# Print only the 'antecedents' and 'consequents' columns of the top 10 rules
print(rules_2022_shop9_nonredundant_top30_lift[['antecedents', 'consequents']].head(10))


# Shop 9 2023 SEARCH 18

In [ ]:
# Filter for records from shop ID 1 and the year 1993
sales_data_2023_shop9 = sales_data_shop9[(sales_data_shop9['trx_date'].dt.year == 2023)]

In [ ]:
sales_data_2023_shop9.head()

In [ ]:
# Grouping product codes by trx_id
transactions_2023_shop9 = sales_data_2023_shop9.groupby('trx_id')['product_code'].apply(list).tolist()

In [ ]:
# Encode transaction data
encoder_2023_shop9 = TransactionEncoder()
trans_array_2023_shop9 = encoder_2023_shop9.fit(transactions_2023_shop9).transform(transactions_2023_shop9)
trans_df_2023_shop9 = pd.DataFrame(trans_array_2023_shop9, columns=encoder_2023_shop9.columns_)

In [ ]:
# Find frequent itemsets
frequent_itemsets_2023_shop9 = apriori(trans_df_2023_shop9, min_support=0.001, use_colnames=True)

In [ ]:
print(frequent_itemsets_2023_shop9)

In [ ]:
## Generate the association rules - by confidence
rulesConfidence_2023_shop9 = association_rules(frequent_itemsets_2023_shop9, metric="confidence", min_threshold=0.10)
rulesConfidence_2023_shop9.sort_values(by='confidence', ascending=False, inplace=True)


In [ ]:
# Replace frozen sets with strings
rulesConfidence_2023_shop9['antecedents_'] = rulesConfidence_2023_shop9['antecedents'].apply(lambda a: ','.join(list(a)))
rulesConfidence_2023_shop9['consequents_'] = rulesConfidence_2023_shop9['consequents'].apply(lambda a: ','.join(list(a)))

# Draw
fig, ax = plt.subplots(figsize=(14,30))
ax = plt.scatter(data=rulesConfidence_2023_shop9, x='consequents_', y='antecedents_', s=rulesConfidence_2023_shop9['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rulesConfidence_2023_shop9.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2023 Shop 9", fontsize=14)
plt.xlabel("Consequents (RHS)", fontsize=12)
plt.ylabel("Antecedents (LHS)", fontsize=12)
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate(rotation=45)
plt.tight_layout()

plt.show()


## Eliminate Redundant Rules

In [ ]:
rulesConfidence_2023_shop9.head()

In [ ]:
rulesConfidence_2023_shop9.info()

In [ ]:
# If 'antecedents' and 'consequents' are not frozensets, convert them
rules_2023_shop9_redundant = rulesConfidence_2023_shop9.copy()
rules_2023_shop9_redundant['antecedents'] = rules_2023_shop9_redundant['antecedents'].apply(lambda x: frozenset(x))
rules_2023_shop9_redundant['consequents'] = rules_2023_shop9_redundant['consequents'].apply(lambda x: frozenset(x))


In [ ]:
# Create a copy of the DataFrame to avoid modifying the original while iterating
rules_2023_shop9_checkredundance = rules_2023_shop9_redundant.copy()

# List to hold non-redundant rules
non_redundant_rules_2023_shop9 = []

for _, rule in rules_2023_shop9_checkredundance.iterrows():
    if not is_redundant(rule, rules_2023_shop9_checkredundance):
        non_redundant_rules_2023_shop9.append(rule)

# Convert the list back to a DataFrame
rules_2023_shop9_nonredundant = pd.DataFrame(non_redundant_rules_2023_shop9).reset_index(drop=True)


In [ ]:
print(rules_2023_shop9_nonredundant.head())
print(f"Original number of rules: {len(rules_2023_shop9_redundant)}")
print(f"Number of non-redundant rules: {len(rules_2023_shop9_nonredundant)}")


In [ ]:
rules_2023_shop9_nonredundant.head()

In [ ]:
rules_2023_shop9_redundant.info()

In [ ]:
rules_2023_shop9_nonredundant.head()

In [ ]:
rules_2023_shop9_nonredundant_backup = rules_2023_shop9_nonredundant.copy()

In [ ]:
rules_2023_shop9_nonredundant_unfiltered = rules_2023_shop9_nonredundant.copy()

In [ ]:
#reset
#rules_2023_shop9_nonredundant = rules_2023_shop9_nonredundant_backup.copy()

# Redundant and Unfiltered

In [ ]:
rules_2023_shop9_redundant.head()

In [ ]:
G_2023_shop9 = nx.DiGraph()

# Iterate through rules for 2023 Shop 9 and add edges to the graph
for _, row in rules_2023_shop9_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as weight
    G_2023_shop9.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2023_shop9 = nx.spring_layout(G_2023_shop9)

# Recalculate node sizes based on indegree or another metric
in_degrees_2023_shop9 = dict(G_2023_shop9.in_degree())
node_sizes_2023_shop9 = [in_degrees_2023_shop9[node] * 100 + 1000 for node in G_2023_shop9.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2023_shop9 = (np.array(node_sizes_2023_shop9) - np.min(node_sizes_2023_shop9)) / (np.max(node_sizes_2023_shop9) - np.min(node_sizes_2023_shop9))

# Define a colormap from orange to red for visual consistency
cmap_2023_shop9 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop9 = [cmap_2023_shop9(size) for size in normalized_sizes_2023_shop9]

# Drawing the network
plt.figure(figsize=(60, 43))
nx.draw_networkx_nodes(G_2023_shop9, pos_2023_shop9, node_size=node_sizes_2023_shop9, node_color=node_colors_2023_shop9, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop9, pos_2023_shop9, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop9, pos_2023_shop9, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 9 with Color Gradient')
plt.axis('off')
plt.show()


### Based on LIFT

In [ ]:
G_2023_shop9_lift = nx.DiGraph()

lift_summary_2023_shop9 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop9_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop9_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop9:
        lift_summary_2023_shop9[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop9[consequents]['count'] += 1
    else:
        lift_summary_2023_shop9[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop9:
    lift_summary_2023_shop9[item]['avg_lift'] = lift_summary_2023_shop9[item]['total_lift'] / lift_summary_2023_shop9[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop9_lift = [lift_summary_2023_shop9[node]['avg_lift'] * 500 if node in lift_summary_2023_shop9 else 1000 for node in G_2023_shop9_lift.nodes()]

pos_2023_shop9_lift = nx.spring_layout(G_2023_shop9_lift)

# Colormap from orange to red
cmap_2023_shop9_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop9_lift = np.array(node_sizes_2023_shop9_lift)
normalized_sizes_2023_shop9_lift = (normalized_sizes_2023_shop9_lift - min(normalized_sizes_2023_shop9_lift)) / (max(normalized_sizes_2023_shop9_lift) - min(normalized_sizes_2023_shop9_lift))
node_colors_2023_shop9_lift = [cmap_2023_shop9_lift(size) for size in normalized_sizes_2023_shop9_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2023_shop9_lift, pos_2023_shop9_lift, node_size=node_sizes_2023_shop9_lift, node_color=node_colors_2023_shop9_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop9_lift, pos_2023_shop9_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop9_lift, pos_2023_shop9_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 9 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


# nonredundant_unfiltered

## Based on Indegree

In [ ]:
G_2023_shop9_nonredundant_unfiltered = nx.DiGraph()

# Iterating through each row to add edges with confidence as an attribute
for _, row in rules_2023_shop9_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop9_nonredundant_unfiltered.add_edge(antecedents, consequents, weight=row['confidence'])

# Defining the layout for network visualization
pos_2023_shop9_nonredundant_unfiltered = nx.spring_layout(G_2023_shop9_nonredundant_unfiltered)

# Calculating node sizes based on the indegree for color mapping
in_degrees_2023_shop9_nonredundant_unfiltered = dict(G_2023_shop9_nonredundant_unfiltered.in_degree())
node_sizes_2023_shop9_nonredundant_unfiltered = [in_degrees_2023_shop9_nonredundant_unfiltered[node] * 100 + 1000 for node in G_2023_shop9_nonredundant_unfiltered.nodes()]

# Normalizing node sizes for color mapping
normalized_sizes_2023_shop9_nonredundant_unfiltered = (np.array(node_sizes_2023_shop9_nonredundant_unfiltered) - min(node_sizes_2023_shop9_nonredundant_unfiltered)) / (max(node_sizes_2023_shop9_nonredundant_unfiltered) - min(node_sizes_2023_shop9_nonredundant_unfiltered))

# Defining a color map from orange to red
cmap_2023_shop9_nonredundant_unfiltered = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])

# Mapping normalized node sizes to colors
node_colors_2023_shop9_nonredundant_unfiltered = [cmap_2023_shop9_nonredundant_unfiltered(size) for size in normalized_sizes_2023_shop9_nonredundant_unfiltered]

# Drawing the network with node sizes and colors
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2023_shop9_nonredundant_unfiltered, pos_2023_shop9_nonredundant_unfiltered, node_size=node_sizes_2023_shop9_nonredundant_unfiltered, node_color=node_colors_2023_shop9_nonredundant_unfiltered, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop9_nonredundant_unfiltered, pos_2023_shop9_nonredundant_unfiltered, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2023_shop9_nonredundant_unfiltered, pos_2023_shop9_nonredundant_unfiltered, font_size=10)

plt.title('Node Size Based on Indegree for 2023 Shop 9')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2023_shop9_lift = nx.DiGraph()

lift_summary_2023_shop9 = {}

for _, row in rules_2023_shop9_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop9_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Update lift summary for consequents
    if consequents in lift_summary_2023_shop9:
        lift_summary_2023_shop9[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop9[consequents]['count'] += 1
    else:
        lift_summary_2023_shop9[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculate average lift for each consequent
for item in lift_summary_2023_shop9:
    lift_summary_2023_shop9[item]['avg_lift'] = lift_summary_2023_shop9[item]['total_lift'] / lift_summary_2023_shop9[item]['count']

# Node sizes based on average lift, scaled appropriately
node_sizes_2023_shop9_lift = [lift_summary_2023_shop9[node]['avg_lift'] * 500 if node in lift_summary_2023_shop9 else 1000 for node in G_2023_shop9_lift.nodes()]

pos_2023_shop9_lift = nx.spring_layout(G_2023_shop9_lift)

# Use a colormap that transitions from orange to red
cmap_2023_shop9_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop9_lift = np.array(node_sizes_2023_shop9_lift)
normalized_sizes_2023_shop9_lift = (normalized_sizes_2023_shop9_lift - min(normalized_sizes_2023_shop9_lift)) / (max(normalized_sizes_2023_shop9_lift) - min(normalized_sizes_2023_shop9_lift))
node_colors_2023_shop9_lift = [cmap_2023_shop9_lift(size) for size in normalized_sizes_2023_shop9_lift]

plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2023_shop9_lift, pos_2023_shop9_lift, node_size=node_sizes_2023_shop9_lift, node_color=node_colors_2023_shop9_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop9_lift, pos_2023_shop9_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop9_lift, pos_2023_shop9_lift, font_size=10)

plt.title('Node Size Based on Average Lift for 2023 Shop 9')
plt.axis('off')
plt.show()


# Based on conviction

In [ ]:
rules_2023_shop9_nonredundant.head()

In [ ]:
G_2023_shop9_conviction = nx.DiGraph()

conviction_summary_2023_shop9 = {}
count_summary_2023_shop9 = {}

# Assuming 'rules_2023_shop9_nonredundant_unfiltered' contains a 'conviction' column
for _, row in rules_2023_shop9_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop9_conviction.add_edge(antecedents, consequents, weight=row['confidence'], conviction=row['conviction'])
    
    # Aggregate conviction values and count occurrences
    if consequents in conviction_summary_2023_shop9:
        conviction_summary_2023_shop9[consequents] += row['conviction']
        count_summary_2023_shop9[consequents] += 1
    else:
        conviction_summary_2023_shop9[consequents] = row['conviction']
        count_summary_2023_shop9[consequents] = 1

# Calculate average conviction values
average_conviction_summary_2023_shop9 = {k: conviction_summary_2023_shop9[k] / count_summary_2023_shop9[k] for k in conviction_summary_2023_shop9}

# Normalize conviction values for sizing and coloring
max_conviction = max(average_conviction_summary_2023_shop9.values())
min_conviction = min(average_conviction_summary_2023_shop9.values())
conviction_range = max_conviction - min_conviction
node_colors = []
node_sizes = []

for node in G_2023_shop9_conviction.nodes():
    # Get the average conviction value for the node, defaulting to min if not present
    conviction = average_conviction_summary_2023_shop9.get(node, min_conviction)
    # Normalize between 0 and 1
    normalized_conviction = (conviction - min_conviction) / conviction_range if conviction_range else 0
    node_colors.append(normalized_conviction)
    node_sizes.append(normalized_conviction * 1000 + 100)  # Scale and offset size

# Use a colormap for coloring nodes based on conviction
cmap = plt.cm.coolwarm

pos = nx.spring_layout(G_2023_shop9_conviction)

plt.figure(figsize=(40, 22))
nodes = nx.draw_networkx_nodes(G_2023_shop9_conviction, pos, node_color=node_colors, node_size=node_sizes, cmap=cmap, alpha=0.8)
edges = nx.draw_networkx_edges(G_2023_shop9_conviction, pos, arrowstyle='->', arrowsize=10, edge_color='grey', alpha=0.5)
labels = nx.draw_networkx_labels(G_2023_shop9_conviction, pos, font_size=8)

plt.colorbar(nodes, label='Normalized Conviction')
plt.title('Network Graph of 2023 Shop 9 Based on Conviction')
plt.axis('off')
plt.show()

In [ ]:
print(max_conviction)

# Filtered top 30 and non redundant

In [ ]:
rules_2023_shop9_nonredundant.head()

In [ ]:
# Assuming rules_1992_shop2_nonredundant is your sorted rules DataFrame for shop2
rules_2023_shop9_nonredundant_top30 = rules_2023_shop9_nonredundant.copy(deep=True)

In [ ]:
rules_2023_shop9_nonredundant_top30_lift = rules_2023_shop9_nonredundant.copy(deep=True)

In [ ]:
# Select the top 30 rules based on lift or another criterion
rules_2023_shop9_nonredundant_top30_lift = rules_2023_shop9_nonredundant_top30_lift.sort_values(by='lift', ascending=False).head(30)

In [ ]:
rules_2023_shop9_nonredundant_top30_lift.head()

In [ ]:
# Select the top 30 rules based on confidence or another criterion
rules_2023_shop9_nonredundant_top30 = rules_2023_shop9_nonredundant_top30.sort_values(by='confidence', ascending=False).head(30)

In [ ]:
rules_2023_shop9_nonredundant_top30.info()

In [ ]:
rules_2023_shop9_nonredundant_top30['LHS'] = [','.join(list(x)) for x in rules_2023_shop9_nonredundant_top30['antecedents']]
rules_2023_shop9_nonredundant_top30['RHS'] = [','.join(list(x)) for x in rules_2023_shop9_nonredundant_top30['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2023_shop9_nonredundant_top30.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2023_shop9_nonredundant_top30, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2023 Shop 9")
plt.axis('off')
plt.show()


## Based on Indegree

In [ ]:
G_2023_shop9 = nx.DiGraph()

# Assuming 'rules_sorted_2023_shop9' contains association rules for shop 9 in 2023
for _, row in rules_2023_shop9_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as the weight
    G_2023_shop9.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2023_shop9 = nx.spring_layout(G_2023_shop9)

# Calculate node sizes based on indegree
in_degrees_2023_shop9 = dict(G_2023_shop9.in_degree())
node_sizes_2023_shop9 = [in_degrees_2023_shop9[node] * 100 + 1000 for node in G_2023_shop9.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2023_shop9 = (np.array(node_sizes_2023_shop9) - np.min(node_sizes_2023_shop9)) / (np.max(node_sizes_2023_shop9) - np.min(node_sizes_2023_shop9))

# Define a colormap from orange to red
cmap_2023_shop9 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop9 = [cmap_2023_shop9(size) for size in normalized_sizes_2023_shop9]

# Drawing the network
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2023_shop9, pos_2023_shop9, node_size=node_sizes_2023_shop9, node_color=node_colors_2023_shop9, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop9, pos_2023_shop9, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2023_shop9, pos_2023_shop9, font_size=8)

plt.title('Top 5 Association Rules Network for 2023 Shop 9 with Color Gradient')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2023_shop9_lift = nx.DiGraph()

lift_summary_2023_shop9 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop9_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop9_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop9:
        lift_summary_2023_shop9[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop9[consequents]['count'] += 1
    else:
        lift_summary_2023_shop9[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop9:
    lift_summary_2023_shop9[item]['avg_lift'] = lift_summary_2023_shop9[item]['total_lift'] / lift_summary_2023_shop9[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop9_lift = [lift_summary_2023_shop9[node]['avg_lift'] * 500 if node in lift_summary_2023_shop9 else 1000 for node in G_2023_shop9_lift.nodes()]

pos_2023_shop9_lift = nx.spring_layout(G_2023_shop9_lift)

# Colormap from orange to red
cmap_2023_shop9_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop9_lift = np.array(node_sizes_2023_shop9_lift)
normalized_sizes_2023_shop9_lift = (normalized_sizes_2023_shop9_lift - min(normalized_sizes_2023_shop9_lift)) / (max(normalized_sizes_2023_shop9_lift) - min(normalized_sizes_2023_shop9_lift))
node_colors_2023_shop9_lift = [cmap_2023_shop9_lift(size) for size in normalized_sizes_2023_shop9_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2023_shop9_lift, pos_2023_shop9_lift, node_size=node_sizes_2023_shop9_lift, node_color=node_colors_2023_shop9_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop9_lift, pos_2023_shop9_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop9_lift, pos_2023_shop9_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 9 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop9_lift = nx.DiGraph()

lift_summary_2023_shop9 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop9_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop9_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop9:
        lift_summary_2023_shop9[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop9[consequents]['count'] += 1
    else:
        lift_summary_2023_shop9[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop9:
    lift_summary_2023_shop9[item]['avg_lift'] = lift_summary_2023_shop9[item]['total_lift'] / lift_summary_2023_shop9[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop9_lift = [lift_summary_2023_shop9[node]['avg_lift'] * 500 if node in lift_summary_2023_shop9 else 1000 for node in G_2023_shop9_lift.nodes()]

pos_2023_shop9_lift = nx.spring_layout(G_2023_shop9_lift)

# Colormap from orange to red
cmap_2023_shop9_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop9_lift = np.array(node_sizes_2023_shop9_lift)
normalized_sizes_2023_shop9_lift = (normalized_sizes_2023_shop9_lift - min(normalized_sizes_2023_shop9_lift)) / (max(normalized_sizes_2023_shop9_lift) - min(normalized_sizes_2023_shop9_lift))
node_colors_2023_shop9_lift = [cmap_2023_shop9_lift(size) for size in normalized_sizes_2023_shop9_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2023_shop9_lift, pos_2023_shop9_lift, node_size=node_sizes_2023_shop9_lift, node_color=node_colors_2023_shop9_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop9_lift, pos_2023_shop9_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop9_lift, pos_2023_shop9_lift, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 9 Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop9 = nx.DiGraph()

# Assuming 'rules_2023_shop9_nonredundant' contains association rules for 2023 Shop 9
for _, row in rules_2023_shop9_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop9.add_edge(antecedents, consequents, confidence=row['confidence'])

# Calculate node sizes based on indegree
in_degrees_2023_shop9 = dict(G_2023_shop9.in_degree())
node_sizes_2023_shop9 = [in_degrees_2023_shop9[node] * 100 + 1000 for node in G_2023_shop9.nodes()]

# Edge width based on confidence
edge_widths_2023_shop9 = [G_2023_shop9[u][v]['confidence'] * 15 for u, v in G_2023_shop9.edges()]

# Assuming normalized sizes and color mapping is desired
normalized_sizes_2023_shop9 = np.array(list(in_degrees_2023_shop9.values()))
normalized_sizes_2023_shop9 = (normalized_sizes_2023_shop9 - min(normalized_sizes_2023_shop9)) / (max(normalized_sizes_2023_shop9) - min(normalized_sizes_2023_shop9))
cmap_2023_shop9 = mcolors.LinearSegmentedColormap.from_list("MyCMapName", ["blue", "red"])

# Node colors based on a property (here, using indegree for example)
node_colors_2023_shop9 = [cmap_2023_shop9(size) for size in normalized_sizes_2023_shop9]

# Layout
pos_2023_shop9 = nx.spring_layout(G_2023_shop9, seed=42)

# Drawing
plt.figure(figsize=(40, 30))
nx.draw_networkx_edges(G_2023_shop9, pos_2023_shop9, width=edge_widths_2023_shop9, edge_color='grey', alpha=0.5)
nx.draw_networkx_nodes(G_2023_shop9, pos_2023_shop9, node_size=node_sizes_2023_shop9, node_color=node_colors_2023_shop9, alpha=0.8)
nx.draw_networkx_labels(G_2023_shop9, pos_2023_shop9, font_size=8, font_color='black')
plt.title('Network Graph of Association Rules Based on Confidence for 2023 Shop 9')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop9 = nx.DiGraph()

convictions_2023_shop9 = []  # To store conviction values for normalization

for _, row in rules_2023_shop9_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    conviction = row['conviction']  # Assuming each row has a 'conviction' column
    convictions_2023_shop9.append(conviction)
    G_2023_shop9.add_edge(antecedents, consequents, weight=row['confidence'], conviction=conviction)

# Normalize conviction values for edge width
max_conviction_2023_shop9 = max(convictions_2023_shop9)
min_conviction_2023_shop9 = min(convictions_2023_shop9)
conviction_range_2023_shop9 = max_conviction_2023_shop9 - min_conviction_2023_shop9
edge_widths_2023_shop9 = [(G_2023_shop9[u][v]['conviction'] - min_conviction_2023_shop9) / conviction_range_2023_shop9 * 30 + 0.5 for u, v in G_2023_shop9.edges()]  # Scale and offset widths

pos_2023_shop9 = nx.spring_layout(G_2023_shop9)

# Node sizes and colors as before
in_degrees_2023_shop9 = dict(G_2023_shop9.in_degree())
node_sizes_2023_shop9 = [in_degrees_2023_shop9[node] * 100 + 1000 for node in G_2023_shop9.nodes()]
normalized_sizes_2023_shop9 = (np.array(node_sizes_2023_shop9) - min(node_sizes_2023_shop9)) / (max(node_sizes_2023_shop9) - min(node_sizes_2023_shop9))
cmap_2023_shop9 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop9 = [cmap_2023_shop9(size) for size in normalized_sizes_2023_shop9]

# Drawing the network with conviction-based edge widths
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2023_shop9, pos_2023_shop9, node_size=node_sizes_2023_shop9, node_color=node_colors_2023_shop9, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop9, pos_2023_shop9, width=edge_widths_2023_shop9, edge_color='grey', alpha=0.5)
nx.draw_networkx_labels(G_2023_shop9, pos_2023_shop9, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 9 with Conviction-based Edge Thickness')
plt.axis('off')
plt.show()


In [ ]:
#the node sizes are based on the indegree where each node acts as a consequent,

In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2023_shop9_nonredundant_top30.loc[:, 'antecedents_'] = rules_2023_shop9_nonredundant_top30['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2023_shop9_nonredundant_top30.loc[:, 'consequents_'] = rules_2023_shop9_nonredundant_top30['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2023_shop9_nonredundant_top30, x='consequents_', y='antecedents_', s=rules_2023_shop9_nonredundant_top30['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2023_shop9_nonredundant_top30.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2023 Shop 9", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
print(rules_2023_shop9_nonredundant_top30[['antecedents', 'consequents']].head(31))

## Monetary Analysis of the Top Rules

In [ ]:
# Step 1: Extract unique product codes from the rules
unique_product_codes = set()
for idx, row in rules_2023_shop9_nonredundant_top30.iterrows():
    # Unite antecedents and consequents into a single set
    all_products = row['antecedents'].union(row['consequents'])
    unique_product_codes.update(all_products)

# Step 2: Create a DataFrame with unique product codes and their unit prices
# Filter sales_data for these product codes and drop duplicates to ensure each product code appears once
product_prices = sales_data[sales_data['product_code'].isin(unique_product_codes)][['product_code', 'unit_price']].drop_duplicates()

# Create a dictionary for quick lookup of prices
price_dict = product_prices.set_index('product_code')['unit_price'].to_dict()

# Step 3: Calculate the total price for each rule and collect data for the new DataFrame
data_for_new_df = []
for idx, row in rules_2023_shop9_nonredundant_top30.iterrows():
    # Combine sets of antecedents and consequents
    all_products = row['antecedents'].union(row['consequents'])
    total_price = sum(price_dict.get(prod, 0) for prod in all_products)  # Use get to avoid KeyError if any product code is missing
    data_for_new_df.append({
        'antecedents': row['antecedents'],
        'consequents': row['consequents'],
        'total_unit_price': total_price
    })

# Create a new DataFrame with only antecedents, consequents, and total unit price
rules_and_price_shop9_2023 = pd.DataFrame(data_for_new_df)

# Sort the DataFrame by total_unit_price in descending order
rules_and_price_shop9_2023_sorted = rules_and_price_shop9_2023.sort_values(by='total_unit_price', ascending=False)

# Print the sorted DataFrame to see the specified columns along with their total unit prices
print(rules_and_price_shop9_2023_sorted)


In [ ]:
# Convert 'trx_date' to datetime format if not already done so
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Add a 'year' column if it doesn't exist
sales_data['year'] = sales_data['trx_date'].dt.year

# Step 2: Filter sales_data for shop9 and the year 2023
filtered_sales_data_2023 = sales_data[(sales_data['shopID'] == 9) & (sales_data['year'] == 2023)]

# Step 3: Initialize a list to hold the rule data including total sales value
rule_sales_summaries_2023 = []

# Step 4: Iterate through each rule in the DataFrame
for index, rule in rules_2023_shop9_nonredundant_top30.iterrows():
    all_products = rule['antecedents'].union(rule['consequents'])

    # Filter the sales data for transactions that include all products in the rule
    # This step ensures that we are only considering transactions that contain all specified products
    mask = filtered_sales_data_2023['product_code'].isin(all_products)
    transaction_ids = filtered_sales_data_2023[mask].groupby('trx_id').filter(lambda x: all_products.issubset(set(x['product_code']))).trx_id.unique()
    matched_transactions = filtered_sales_data_2023[filtered_sales_data_2023['trx_id'].isin(transaction_ids)]
    
    # Step 5: Sum the total_value of these transactions
    total_sales_value = matched_transactions['total_value'].sum()

    # Append the result to the list
    rule_sales_summaries_2023.append({
        'antecedents': rule['antecedents'],
        'consequents': rule['consequents'],
        'total_sales_value': total_sales_value
    })

# Step 6: Convert the list to a DataFrame
rules_sales_values_df_2023 = pd.DataFrame(rule_sales_summaries_2023)

# Step 7: Sort by total_sales_value in descending order
sorted_rules_sales_values_df_2023 = rules_sales_values_df_2023.sort_values(by='total_sales_value', ascending=False)

# Step 8: Print the sorted DataFrame
print(sorted_rules_sales_values_df_2023)


## Narrowing down based on Lift

In [ ]:
rules_2023_shop9_nonredundant_top30_lift['LHS'] = [','.join(list(x)) for x in rules_2023_shop9_nonredundant_top30_lift['antecedents']]
rules_2023_shop9_nonredundant_top30_lift['RHS'] = [','.join(list(x)) for x in rules_2023_shop9_nonredundant_top30_lift['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2023_shop9_nonredundant_top30_lift.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2023_shop9_nonredundant_top30_lift, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2023 Shop 9")
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop9_lift = nx.DiGraph()

lift_summary_2023_shop9 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop9_nonredundant_top30_lift.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop9_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop9:
        lift_summary_2023_shop9[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop9[consequents]['count'] += 1
    else:
        lift_summary_2023_shop9[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop9:
    lift_summary_2023_shop9[item]['avg_lift'] = lift_summary_2023_shop9[item]['total_lift'] / lift_summary_2023_shop9[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop9_lift = [lift_summary_2023_shop9[node]['avg_lift'] * 500 if node in lift_summary_2023_shop9 else 1000 for node in G_2023_shop9_lift.nodes()]

pos_2023_shop9_lift = nx.spring_layout(G_2023_shop9_lift)

# Colormap from orange to red
cmap_2023_shop9_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop9_lift = np.array(node_sizes_2023_shop9_lift)
normalized_sizes_2023_shop9_lift = (normalized_sizes_2023_shop9_lift - min(normalized_sizes_2023_shop9_lift)) / (max(normalized_sizes_2023_shop9_lift) - min(normalized_sizes_2023_shop9_lift))
node_colors_2023_shop9_lift = [cmap_2023_shop9_lift(size) for size in normalized_sizes_2023_shop9_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2023_shop9_lift, pos_2023_shop9_lift, node_size=node_sizes_2023_shop9_lift, node_color=node_colors_2023_shop9_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop9_lift, pos_2023_shop9_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop9_lift, pos_2023_shop9_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 9 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2023_shop9_nonredundant_top30_lift.loc[:, 'antecedents_'] = rules_2023_shop9_nonredundant_top30_lift['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2023_shop9_nonredundant_top30_lift.loc[:, 'consequents_'] = rules_2023_shop9_nonredundant_top30_lift['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot for Shop 9
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2023_shop9_nonredundant_top30_lift, x='consequents_', y='antecedents_', s=rules_2023_shop9_nonredundant_top30_lift['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2023_shop9_nonredundant_top30_lift.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2023 Shop 9", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


# Shop 11 2023 SEARCH 19

In [ ]:
# Filter for records from shop ID 1 and the year 1993
sales_data_2023_shop11 = sales_data_shop11[(sales_data_shop11['trx_date'].dt.year == 2023)]

In [ ]:
sales_data_2023_shop11.head()

In [ ]:
# Grouping product codes by trx_id
transactions_2023_shop11 = sales_data_2023_shop11.groupby('trx_id')['product_code'].apply(list).tolist()

In [ ]:
# Encode transaction data
encoder_2023_shop11 = TransactionEncoder()
trans_array_2023_shop11 = encoder_2023_shop11.fit(transactions_2023_shop11).transform(transactions_2023_shop11)
trans_df_2023_shop11 = pd.DataFrame(trans_array_2023_shop11, columns=encoder_2023_shop11.columns_)

In [ ]:
# Find frequent itemsets
frequent_itemsets_2023_shop11 = apriori(trans_df_2023_shop11, min_support=0.001, use_colnames=True)

In [ ]:
print(frequent_itemsets_2023_shop11)

In [ ]:
## Generate the association rules - by confidence
rulesConfidence_2023_shop11 = association_rules(frequent_itemsets_2023_shop11, metric="confidence", min_threshold=0.10)
rulesConfidence_2023_shop11.sort_values(by='confidence', ascending=False, inplace=True)


In [ ]:
# Replace frozen sets with strings
rulesConfidence_2023_shop11['antecedents_'] = rulesConfidence_2023_shop11['antecedents'].apply(lambda a: ','.join(list(a)))
rulesConfidence_2023_shop11['consequents_'] = rulesConfidence_2023_shop11['consequents'].apply(lambda a: ','.join(list(a)))

# Draw
fig, ax = plt.subplots(figsize=(14,30))
ax = plt.scatter(data=rulesConfidence_2023_shop11, x='consequents_', y='antecedents_', s=rulesConfidence_2023_shop11['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rulesConfidence_2023_shop11.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2023 Shop 11", fontsize=14)
plt.xlabel("Consequents (RHS)", fontsize=12)
plt.ylabel("Antecedents (LHS)", fontsize=12)
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate(rotation=45)
plt.tight_layout()

plt.show()


## Eliminate Redundant Rules

In [ ]:
rulesConfidence_2023_shop11.head()

In [ ]:
rulesConfidence_2023_shop11.info()

In [ ]:
# If 'antecedents' and 'consequents' are not frozensets, convert them
rules_2023_shop11_redundant = rulesConfidence_2023_shop11.copy()
rules_2023_shop11_redundant['antecedents'] = rules_2023_shop11_redundant['antecedents'].apply(lambda x: frozenset(x))
rules_2023_shop11_redundant['consequents'] = rules_2023_shop11_redundant['consequents'].apply(lambda x: frozenset(x))


In [ ]:
# Create a copy of the DataFrame to avoid modifying the original while iterating
rules_2023_shop11_checkredundance = rules_2023_shop11_redundant.copy()

# List to hold non-redundant rules
non_redundant_rules_2023_shop11 = []

for _, rule in rules_2023_shop11_checkredundance.iterrows():
    if not is_redundant(rule, rules_2023_shop11_checkredundance):
        non_redundant_rules_2023_shop11.append(rule)

# Convert the list back to a DataFrame
rules_2023_shop11_nonredundant = pd.DataFrame(non_redundant_rules_2023_shop11).reset_index(drop=True)


In [ ]:
print(rules_2023_shop11_nonredundant.head())
print(f"Original number of rules: {len(rules_2023_shop11_redundant)}")
print(f"Number of non-redundant rules: {len(rules_2023_shop11_nonredundant)}")


In [ ]:
rules_2023_shop11_nonredundant.head()

In [ ]:
rules_2023_shop11_redundant.info()

In [ ]:
rules_2023_shop11_nonredundant.head()

In [ ]:
rules_2023_shop11_nonredundant_backup = rules_2023_shop11_nonredundant.copy()

In [ ]:
rules_2023_shop11_nonredundant_unfiltered = rules_2023_shop11_nonredundant.copy()

In [ ]:
#reset
#rules_2023_shop11_nonredundant = rules_2023_shop11_nonredundant_backup.copy()

# Redundant and Unfiltered

In [ ]:
rules_2023_shop11_redundant.head()

In [ ]:
G_2023_shop11 = nx.DiGraph()

# Iterate through rules for 2023 Shop 11 and add edges to the graph
for _, row in rules_2023_shop11_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as weight
    G_2023_shop11.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2023_shop11 = nx.spring_layout(G_2023_shop11)

# Recalculate node sizes based on indegree or another metric
in_degrees_2023_shop11 = dict(G_2023_shop11.in_degree())
node_sizes_2023_shop11 = [in_degrees_2023_shop11[node] * 100 + 1000 for node in G_2023_shop11.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2023_shop11 = (np.array(node_sizes_2023_shop11) - np.min(node_sizes_2023_shop11)) / (np.max(node_sizes_2023_shop11) - np.min(node_sizes_2023_shop11))

# Define a colormap from orange to red for visual consistency
cmap_2023_shop11 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop11 = [cmap_2023_shop11(size) for size in normalized_sizes_2023_shop11]

# Drawing the network
plt.figure(figsize=(60, 43))
nx.draw_networkx_nodes(G_2023_shop11, pos_2023_shop11, node_size=node_sizes_2023_shop11, node_color=node_colors_2023_shop11, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop11, pos_2023_shop11, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop11, pos_2023_shop11, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 11 with Color Gradient')
plt.axis('off')
plt.show()


### Based on LIFT

In [ ]:
G_2023_shop11_lift = nx.DiGraph()

lift_summary_2023_shop11 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop11_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop11_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop11:
        lift_summary_2023_shop11[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop11[consequents]['count'] += 1
    else:
        lift_summary_2023_shop11[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop11:
    lift_summary_2023_shop11[item]['avg_lift'] = lift_summary_2023_shop11[item]['total_lift'] / lift_summary_2023_shop11[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop11_lift = [lift_summary_2023_shop11[node]['avg_lift'] * 500 if node in lift_summary_2023_shop11 else 1000 for node in G_2023_shop11_lift.nodes()]

pos_2023_shop11_lift = nx.spring_layout(G_2023_shop11_lift)

# Colormap from orange to red
cmap_2023_shop11_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop11_lift = np.array(node_sizes_2023_shop11_lift)
normalized_sizes_2023_shop11_lift = (normalized_sizes_2023_shop11_lift - min(normalized_sizes_2023_shop11_lift)) / (max(normalized_sizes_2023_shop11_lift) - min(normalized_sizes_2023_shop11_lift))
node_colors_2023_shop11_lift = [cmap_2023_shop11_lift(size) for size in normalized_sizes_2023_shop11_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2023_shop11_lift, pos_2023_shop11_lift, node_size=node_sizes_2023_shop11_lift, node_color=node_colors_2023_shop11_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop11_lift, pos_2023_shop11_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop11_lift, pos_2023_shop11_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 11 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


# nonredundant_unfiltered

## Based on Indegree

In [ ]:
G_2023_shop11_nonredundant_unfiltered = nx.DiGraph()

# Iterating through each row to add edges with confidence as an attribute
for _, row in rules_2023_shop11_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop11_nonredundant_unfiltered.add_edge(antecedents, consequents, weight=row['confidence'])

# Defining the layout for network visualization
pos_2023_shop11_nonredundant_unfiltered = nx.spring_layout(G_2023_shop11_nonredundant_unfiltered)

# Calculating node sizes based on the indegree for color mapping
in_degrees_2023_shop11_nonredundant_unfiltered = dict(G_2023_shop11_nonredundant_unfiltered.in_degree())
node_sizes_2023_shop11_nonredundant_unfiltered = [in_degrees_2023_shop11_nonredundant_unfiltered[node] * 100 + 1000 for node in G_2023_shop11_nonredundant_unfiltered.nodes()]

# Normalizing node sizes for color mapping
normalized_sizes_2023_shop11_nonredundant_unfiltered = (np.array(node_sizes_2023_shop11_nonredundant_unfiltered) - min(node_sizes_2023_shop11_nonredundant_unfiltered)) / (max(node_sizes_2023_shop11_nonredundant_unfiltered) - min(node_sizes_2023_shop11_nonredundant_unfiltered))

# Defining a color map from orange to red
cmap_2023_shop11_nonredundant_unfiltered = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])

# Mapping normalized node sizes to colors
node_colors_2023_shop11_nonredundant_unfiltered = [cmap_2023_shop11_nonredundant_unfiltered(size) for size in normalized_sizes_2023_shop11_nonredundant_unfiltered]

# Drawing the network with node sizes and colors
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2023_shop11_nonredundant_unfiltered, pos_2023_shop11_nonredundant_unfiltered, node_size=node_sizes_2023_shop11_nonredundant_unfiltered, node_color=node_colors_2023_shop11_nonredundant_unfiltered, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop11_nonredundant_unfiltered, pos_2023_shop11_nonredundant_unfiltered, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2023_shop11_nonredundant_unfiltered, pos_2023_shop11_nonredundant_unfiltered, font_size=10)

plt.title('Node Size Based on Indegree for 2023 Shop 11')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2023_shop11_lift = nx.DiGraph()

lift_summary_2023_shop11 = {}

# Iterate through each row to add edges with lift as an attribute
for _, row in rules_2023_shop11_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop11_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Update lift summary for consequents
    if consequents in lift_summary_2023_shop11:
        lift_summary_2023_shop11[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop11[consequents]['count'] += 1
    else:
        lift_summary_2023_shop11[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculate average lift for each consequent
for item in lift_summary_2023_shop11:
    lift_summary_2023_shop11[item]['avg_lift'] = lift_summary_2023_shop11[item]['total_lift'] / lift_summary_2023_shop11[item]['count']

# Node sizes based on average lift, scaled appropriately
node_sizes_2023_shop11_lift = [lift_summary_2023_shop11[node]['avg_lift'] * 500 if node in lift_summary_2023_shop11 else 1000 for node in G_2023_shop11_lift.nodes()]

pos_2023_shop11_lift = nx.spring_layout(G_2023_shop11_lift)

# Use a colormap that transitions from orange to red
cmap_2023_shop11_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop11_lift = np.array(node_sizes_2023_shop11_lift)
normalized_sizes_2023_shop11_lift = (normalized_sizes_2023_shop11_lift - min(normalized_sizes_2023_shop11_lift)) / (max(normalized_sizes_2023_shop11_lift) - min(normalized_sizes_2023_shop11_lift))
node_colors_2023_shop11_lift = [cmap_2023_shop11_lift(size) for size in normalized_sizes_2023_shop11_lift]

plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2023_shop11_lift, pos_2023_shop11_lift, node_size=node_sizes_2023_shop11_lift, node_color=node_colors_2023_shop11_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop11_lift, pos_2023_shop11_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop11_lift, pos_2023_shop11_lift, font_size=10)

plt.title('Node Size Based on Average Lift for 2023 Shop 11')
plt.axis('off')
plt.show()


# Based on conviction

In [ ]:
rules_2023_shop11_nonredundant.head()

In [ ]:
G_2023_shop11_conviction = nx.DiGraph()

conviction_summary_2023_shop11 = {}
count_summary_2023_shop11 = {}

# Assuming 'rules_2023_shop11_nonredundant_unfiltered' contains a 'conviction' column
for _, row in rules_2023_shop11_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop11_conviction.add_edge(antecedents, consequents, weight=row['confidence'], conviction=row['conviction'])
    
    # Aggregate conviction values and count occurrences
    if consequents in conviction_summary_2023_shop11:
        conviction_summary_2023_shop11[consequents] += row['conviction']
        count_summary_2023_shop11[consequents] += 1
    else:
        conviction_summary_2023_shop11[consequents] = row['conviction']
        count_summary_2023_shop11[consequents] = 1

# Calculate average conviction values
average_conviction_summary_2023_shop11 = {k: conviction_summary_2023_shop11[k] / count_summary_2023_shop11[k] for k in conviction_summary_2023_shop11}

# Normalize conviction values for sizing and coloring
max_conviction = max(average_conviction_summary_2023_shop11.values())
min_conviction = min(average_conviction_summary_2023_shop11.values())
conviction_range = max_conviction - min_conviction
node_colors = []
node_sizes = []

for node in G_2023_shop11_conviction.nodes():
    # Get the average conviction value for the node, defaulting to min if not present
    conviction = average_conviction_summary_2023_shop11.get(node, min_conviction)
    # Normalize between 0 and 1
    normalized_conviction = (conviction - min_conviction) / conviction_range if conviction_range else 0
    node_colors.append(normalized_conviction)
    node_sizes.append(normalized_conviction * 1000 + 100)  # Scale and offset size

# Use a colormap for coloring nodes based on conviction
cmap = plt.cm.coolwarm

pos = nx.spring_layout(G_2023_shop11_conviction)

plt.figure(figsize=(40, 22))
nodes = nx.draw_networkx_nodes(G_2023_shop11_conviction, pos, node_color=node_colors, node_size=node_sizes, cmap=cmap, alpha=0.8)
edges = nx.draw_networkx_edges(G_2023_shop11_conviction, pos, arrowstyle='->', arrowsize=10, edge_color='grey', alpha=0.5)
labels = nx.draw_networkx_labels(G_2023_shop11_conviction, pos, font_size=8)

plt.colorbar(nodes, label='Normalized Conviction')
plt.title('Network Graph of 2023 Shop 11 Based on Conviction')
plt.axis('off')
plt.show()

In [ ]:
print(max_conviction)

# Filtered top 30 and non redundant

In [ ]:
rules_2023_shop11_nonredundant.head()

In [ ]:
# Assuming rules_1992_shop2_nonredundant is your sorted rules DataFrame for shop2
rules_2023_shop11_nonredundant_top30 = rules_2023_shop11_nonredundant.copy(deep=True)

In [ ]:
rules_2023_shop11_nonredundant_top30_lift = rules_2023_shop11_nonredundant.copy(deep=True)

In [ ]:
# Select the top 30 rules based on lift or another criterion
rules_2023_shop11_nonredundant_top30_lift = rules_2023_shop11_nonredundant_top30_lift.sort_values(by='lift', ascending=False).head(30)

In [ ]:
rules_2023_shop11_nonredundant_top30_lift.head()

In [ ]:
# Select the top 30 rules based on confidence or another criterion
rules_2023_shop11_nonredundant_top30 = rules_2023_shop11_nonredundant_top30.sort_values(by='confidence', ascending=False).head(30)

In [ ]:
rules_2023_shop11_nonredundant_top30.info()

In [ ]:
rules_2023_shop11_nonredundant_top30['LHS'] = [','.join(list(x)) for x in rules_2023_shop11_nonredundant_top30['antecedents']]
rules_2023_shop11_nonredundant_top30['RHS'] = [','.join(list(x)) for x in rules_2023_shop11_nonredundant_top30['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2023_shop11_nonredundant_top30.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2023_shop11_nonredundant_top30, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2023 Shop 11")
plt.axis('off')
plt.show()


## Based on Indegree

In [ ]:
G_2023_shop11 = nx.DiGraph()

# Assuming 'rules_sorted_2023_shop11' contains association rules for shop 11 in 2023
for _, row in rules_2023_shop11_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as the weight
    G_2023_shop11.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2023_shop11 = nx.spring_layout(G_2023_shop11)

# Calculate node sizes based on indegree
in_degrees_2023_shop11 = dict(G_2023_shop11.in_degree())
node_sizes_2023_shop11 = [in_degrees_2023_shop11[node] * 100 + 1000 for node in G_2023_shop11.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2023_shop11 = (np.array(node_sizes_2023_shop11) - np.min(node_sizes_2023_shop11)) / (np.max(node_sizes_2023_shop11) - np.min(node_sizes_2023_shop11))

# Define a colormap from orange to red
cmap_2023_shop11 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop11 = [cmap_2023_shop11(size) for size in normalized_sizes_2023_shop11]

# Drawing the network
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2023_shop11, pos_2023_shop11, node_size=node_sizes_2023_shop11, node_color=node_colors_2023_shop11, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop11, pos_2023_shop11, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2023_shop11, pos_2023_shop11, font_size=8)

plt.title('Top 5 Association Rules Network for 2023 Shop 11 with Color Gradient')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2023_shop11_lift = nx.DiGraph()

lift_summary_2023_shop11 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop11_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop11_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop11:
        lift_summary_2023_shop11[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop11[consequents]['count'] += 1
    else:
        lift_summary_2023_shop11[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop11:
    lift_summary_2023_shop11[item]['avg_lift'] = lift_summary_2023_shop11[item]['total_lift'] / lift_summary_2023_shop11[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop11_lift = [lift_summary_2023_shop11[node]['avg_lift'] * 500 if node in lift_summary_2023_shop11 else 1000 for node in G_2023_shop11_lift.nodes()]

pos_2023_shop11_lift = nx.spring_layout(G_2023_shop11_lift)

# Colormap from orange to red
cmap_2023_shop11_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop11_lift = np.array(node_sizes_2023_shop11_lift)
normalized_sizes_2023_shop11_lift = (normalized_sizes_2023_shop11_lift - min(normalized_sizes_2023_shop11_lift)) / (max(normalized_sizes_2023_shop11_lift) - min(normalized_sizes_2023_shop11_lift))
node_colors_2023_shop11_lift = [cmap_2023_shop11_lift(size) for size in normalized_sizes_2023_shop11_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2023_shop11_lift, pos_2023_shop11_lift, node_size=node_sizes_2023_shop11_lift, node_color=node_colors_2023_shop11_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop11_lift, pos_2023_shop11_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop11_lift, pos_2023_shop11_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 11 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop11_lift = nx.DiGraph()

lift_summary_2023_shop11 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop11_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop11_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop11:
        lift_summary_2023_shop11[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop11[consequents]['count'] += 1
    else:
        lift_summary_2023_shop11[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop11:
    lift_summary_2023_shop11[item]['avg_lift'] = lift_summary_2023_shop11[item]['total_lift'] / lift_summary_2023_shop11[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop11_lift = [lift_summary_2023_shop11[node]['avg_lift'] * 500 if node in lift_summary_2023_shop11 else 1000 for node in G_2023_shop11_lift.nodes()]

pos_2023_shop11_lift = nx.spring_layout(G_2023_shop11_lift)

# Colormap from orange to red
cmap_2023_shop11_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop11_lift = np.array(node_sizes_2023_shop11_lift)
normalized_sizes_2023_shop11_lift = (normalized_sizes_2023_shop11_lift - min(normalized_sizes_2023_shop11_lift)) / (max(normalized_sizes_2023_shop11_lift) - min(normalized_sizes_2023_shop11_lift))
node_colors_2023_shop11_lift = [cmap_2023_shop11_lift(size) for size in normalized_sizes_2023_shop11_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2023_shop11_lift, pos_2023_shop11_lift, node_size=node_sizes_2023_shop11_lift, node_color=node_colors_2023_shop11_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop11_lift, pos_2023_shop11_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop11_lift, pos_2023_shop11_lift, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 11 Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop11 = nx.DiGraph()

# Assuming 'rules_2023_shop11_nonredundant' contains association rules for 2023 Shop 11
for _, row in rules_2023_shop11_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop11.add_edge(antecedents, consequents, confidence=row['confidence'])

# Calculate node sizes based on indegree
in_degrees_2023_shop11 = dict(G_2023_shop11.in_degree())
node_sizes_2023_shop11 = [in_degrees_2023_shop11[node] * 100 + 1000 for node in G_2023_shop11.nodes()]

# Edge width based on confidence
edge_widths_2023_shop11 = [G_2023_shop11[u][v]['confidence'] * 15 for u, v in G_2023_shop11.edges()]

# Assuming normalized sizes and color mapping is desired
normalized_sizes_2023_shop11 = np.array(list(in_degrees_2023_shop11.values()))
normalized_sizes_2023_shop11 = (normalized_sizes_2023_shop11 - min(normalized_sizes_2023_shop11)) / (max(normalized_sizes_2023_shop11) - min(normalized_sizes_2023_shop11))
cmap_2023_shop11 = mcolors.LinearSegmentedColormap.from_list("MyCMapName", ["blue", "red"])

# Node colors based on a property (here, using indegree for example)
node_colors_2023_shop11 = [cmap_2023_shop11(size) for size in normalized_sizes_2023_shop11]

# Layout
pos_2023_shop11 = nx.spring_layout(G_2023_shop11, seed=42)

# Drawing
plt.figure(figsize=(40, 30))
nx.draw_networkx_edges(G_2023_shop11, pos_2023_shop11, width=edge_widths_2023_shop11, edge_color='grey', alpha=0.5)
nx.draw_networkx_nodes(G_2023_shop11, pos_2023_shop11, node_size=node_sizes_2023_shop11, node_color=node_colors_2023_shop11, alpha=0.8)
nx.draw_networkx_labels(G_2023_shop11, pos_2023_shop11, font_size=8, font_color='black')
plt.title('Network Graph of Association Rules Based on Confidence for 2023 Shop 11')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop11 = nx.DiGraph()

convictions_2023_shop11 = []  # To store conviction values for normalization

for _, row in rules_2023_shop11_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    conviction = row['conviction']  # Assuming each row has a 'conviction' column
    convictions_2023_shop11.append(conviction)
    G_2023_shop11.add_edge(antecedents, consequents, weight=row['confidence'], conviction=conviction)

# Normalize conviction values for edge width
max_conviction_2023_shop11 = max(convictions_2023_shop11)
min_conviction_2023_shop11 = min(convictions_2023_shop11)
conviction_range_2023_shop11 = max_conviction_2023_shop11 - min_conviction_2023_shop11
edge_widths_2023_shop11 = [(G_2023_shop11[u][v]['conviction'] - min_conviction_2023_shop11) / conviction_range_2023_shop11 * 30 + 0.5 for u, v in G_2023_shop11.edges()]  # Scale and offset widths

pos_2023_shop11 = nx.spring_layout(G_2023_shop11)

# Node sizes and colors as before
in_degrees_2023_shop11 = dict(G_2023_shop11.in_degree())
node_sizes_2023_shop11 = [in_degrees_2023_shop11[node] * 100 + 1000 for node in G_2023_shop11.nodes()]
normalized_sizes_2023_shop11 = (np.array(node_sizes_2023_shop11) - min(node_sizes_2023_shop11)) / (max(node_sizes_2023_shop11) - min(node_sizes_2023_shop11))
cmap_2023_shop11 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop11 = [cmap_2023_shop11(size) for size in normalized_sizes_2023_shop11]

# Drawing the network with conviction-based edge widths
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2023_shop11, pos_2023_shop11, node_size=node_sizes_2023_shop11, node_color=node_colors_2023_shop11, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop11, pos_2023_shop11, width=edge_widths_2023_shop11, edge_color='grey', alpha=0.5)
nx.draw_networkx_labels(G_2023_shop11, pos_2023_shop11, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 11 with Conviction-based Edge Thickness')
plt.axis('off')
plt.show()


In [ ]:
#the node sizes are based on the indegree where each node acts as a consequent,

In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2023_shop11_nonredundant_top30.loc[:, 'antecedents_'] = rules_2023_shop11_nonredundant_top30['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2023_shop11_nonredundant_top30.loc[:, 'consequents_'] = rules_2023_shop11_nonredundant_top30['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2023_shop11_nonredundant_top30, x='consequents_', y='antecedents_', s=rules_2023_shop11_nonredundant_top30['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2023_shop11_nonredundant_top30.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2023 Shop 11", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
print(rules_2023_shop11_nonredundant_top30[['antecedents', 'consequents']].head(31))

## Monetary Analysis of the Top Rules

In [ ]:
# Step 1: Extract unique product codes from the rules
unique_product_codes = set()
for idx, row in rules_2023_shop11_nonredundant_top30.iterrows():
    # Unite antecedents and consequents into a single set
    all_products = row['antecedents'].union(row['consequents'])
    unique_product_codes.update(all_products)

# Step 2: Create a DataFrame with unique product codes and their unit prices
# Filter sales_data for these product codes and drop duplicates to ensure each product code appears once
product_prices = sales_data[sales_data['product_code'].isin(unique_product_codes)][['product_code', 'unit_price']].drop_duplicates()

# Create a dictionary for quick lookup of prices
price_dict = product_prices.set_index('product_code')['unit_price'].to_dict()

# Step 3: Calculate the total price for each rule and collect data for the new DataFrame
data_for_new_df = []
for idx, row in rules_2023_shop11_nonredundant_top30.iterrows():
    # Combine sets of antecedents and consequents
    all_products = row['antecedents'].union(row['consequents'])
    total_price = sum(price_dict.get(prod, 0) for prod in all_products)  # Use get to avoid KeyError if any product code is missing
    data_for_new_df.append({
        'antecedents': row['antecedents'],
        'consequents': row['consequents'],
        'total_unit_price': total_price
    })

# Create a new DataFrame with only antecedents, consequents, and total unit price
rules_and_price_shop11_2023 = pd.DataFrame(data_for_new_df)

# Sort the DataFrame by total_unit_price in descending order
rules_and_price_shop11_2023_sorted = rules_and_price_shop11_2023.sort_values(by='total_unit_price', ascending=False)

# Print the sorted DataFrame to see the specified columns along with their total unit prices
print(rules_and_price_shop11_2023_sorted)


In [ ]:
# Convert 'trx_date' to datetime format if not already done so
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Add a 'year' column if it doesn't exist
sales_data['year'] = sales_data['trx_date'].dt.year

# Step 2: Filter sales_data for shop11 and the year 2023
filtered_sales_data_2023 = sales_data[(sales_data['shopID'] == 11) & (sales_data['year'] == 2023)]

# Step 3: Initialize a list to hold the rule data including total sales value
rule_sales_summaries_2023 = []

# Step 4: Iterate through each rule in the DataFrame
for index, rule in rules_2023_shop11_nonredundant_top30.iterrows():
    all_products = rule['antecedents'].union(rule['consequents'])

    # Filter the sales data for transactions that include all products in the rule
    # This step ensures that we are only considering transactions that contain all specified products
    mask = filtered_sales_data_2023['product_code'].isin(all_products)
    transaction_ids = filtered_sales_data_2023[mask].groupby('trx_id').filter(lambda x: all_products.issubset(set(x['product_code']))).trx_id.unique()
    matched_transactions = filtered_sales_data_2023[filtered_sales_data_2023['trx_id'].isin(transaction_ids)]
    
    # Step 5: Sum the total_value of these transactions
    total_sales_value = matched_transactions['total_value'].sum()

    # Append the result to the list
    rule_sales_summaries_2023.append({
        'antecedents': rule['antecedents'],
        'consequents': rule['consequents'],
        'total_sales_value': total_sales_value
    })

# Step 6: Convert the list to a DataFrame
rules_sales_values_df_2023 = pd.DataFrame(rule_sales_summaries_2023)

# Step 7: Sort by total_sales_value in descending order
sorted_rules_sales_values_df_2023 = rules_sales_values_df_2023.sort_values(by='total_sales_value', ascending=False)

# Step 8: Print the sorted DataFrame
print(sorted_rules_sales_values_df_2023)


## Narrowing down based on Lift

In [ ]:
rules_2023_shop11_nonredundant_top30_lift['LHS'] = [','.join(list(x)) for x in rules_2023_shop11_nonredundant_top30_lift['antecedents']]
rules_2023_shop11_nonredundant_top30_lift['RHS'] = [','.join(list(x)) for x in rules_2023_shop11_nonredundant_top30_lift['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2023_shop11_nonredundant_top30_lift.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2023_shop11_nonredundant_top30_lift, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2023 Shop 11")
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop11_lift = nx.DiGraph()

lift_summary_2023_shop11 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop11_nonredundant_top30_lift.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop11_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop11:
        lift_summary_2023_shop11[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop11[consequents]['count'] += 1
    else:
        lift_summary_2023_shop11[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop11:
    lift_summary_2023_shop11[item]['avg_lift'] = lift_summary_2023_shop11[item]['total_lift'] / lift_summary_2023_shop11[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop11_lift = [lift_summary_2023_shop11[node]['avg_lift'] * 500 if node in lift_summary_2023_shop11 else 1000 for node in G_2023_shop11_lift.nodes()]

pos_2023_shop11_lift = nx.spring_layout(G_2023_shop11_lift)

# Colormap from orange to red
cmap_2023_shop11_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop11_lift = np.array(node_sizes_2023_shop11_lift)
normalized_sizes_2023_shop11_lift = (normalized_sizes_2023_shop11_lift - min(normalized_sizes_2023_shop11_lift)) / (max(normalized_sizes_2023_shop11_lift) - min(normalized_sizes_2023_shop11_lift))
node_colors_2023_shop11_lift = [cmap_2023_shop11_lift(size) for size in normalized_sizes_2023_shop11_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2023_shop11_lift, pos_2023_shop11_lift, node_size=node_sizes_2023_shop11_lift, node_color=node_colors_2023_shop11_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop11_lift, pos_2023_shop11_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop11_lift, pos_2023_shop11_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 11 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2023_shop11_nonredundant_top30_lift.loc[:, 'antecedents_'] = rules_2023_shop11_nonredundant_top30_lift['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2023_shop11_nonredundant_top30_lift.loc[:, 'consequents_'] = rules_2023_shop11_nonredundant_top30_lift['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2023_shop11_nonredundant_top30_lift, x='consequents_', y='antecedents_', s=rules_2023_shop11_nonredundant_top30_lift['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2023_shop11_nonredundant_top30_lift.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2023 Shop 11", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


# Shop 12 2023 SEARCH 20

In [ ]:
# Filter for records from shop ID 1 and the year 1993
sales_data_2023_shop12 = sales_data_shop12[(sales_data_shop12['trx_date'].dt.year == 2023)]

In [ ]:
sales_data_2023_shop12.head()

In [ ]:
# Grouping product codes by trx_id
transactions_2023_shop12 = sales_data_2023_shop12.groupby('trx_id')['product_code'].apply(list).tolist()

In [ ]:
# Encode transaction data
encoder_2023_shop12 = TransactionEncoder()
trans_array_2023_shop12 = encoder_2023_shop12.fit(transactions_2023_shop12).transform(transactions_2023_shop12)
trans_df_2023_shop12 = pd.DataFrame(trans_array_2023_shop12, columns=encoder_2023_shop12.columns_)

In [ ]:
# Find frequent itemsets
frequent_itemsets_2023_shop12 = apriori(trans_df_2023_shop12, min_support=0.001, use_colnames=True)

In [ ]:
print(frequent_itemsets_2023_shop12)

In [ ]:
## Generate the association rules - by confidence
rulesConfidence_2023_shop12 = association_rules(frequent_itemsets_2023_shop12, metric="confidence", min_threshold=0.10)
rulesConfidence_2023_shop12.sort_values(by='confidence', ascending=False, inplace=True)


In [ ]:
# Replace frozen sets with strings
rulesConfidence_2023_shop12['antecedents_'] = rulesConfidence_2023_shop12['antecedents'].apply(lambda a: ','.join(list(a)))
rulesConfidence_2023_shop12['consequents_'] = rulesConfidence_2023_shop12['consequents'].apply(lambda a: ','.join(list(a)))

# Draw
fig, ax = plt.subplots(figsize=(14, 30))
ax = plt.scatter(data=rulesConfidence_2023_shop12, x='consequents_', y='antecedents_', s=rulesConfidence_2023_shop12['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rulesConfidence_2023_shop12.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2023 Shop 12", fontsize=14)
plt.xlabel("Consequents (RHS)", fontsize=12)
plt.ylabel("Antecedents (LHS)", fontsize=12)
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate(rotation=45)
plt.tight_layout()

plt.show()


## Eliminate Redundant Rules

In [ ]:
rulesConfidence_2023_shop12.head()

In [ ]:
rulesConfidence_2023_shop12.info()

In [ ]:
# If 'antecedents' and 'consequents' are not frozensets, convert them
rules_2023_shop12_redundant = rulesConfidence_2023_shop12.copy()
rules_2023_shop12_redundant['antecedents'] = rules_2023_shop12_redundant['antecedents'].apply(lambda x: frozenset(x))
rules_2023_shop12_redundant['consequents'] = rules_2023_shop12_redundant['consequents'].apply(lambda x: frozenset(x))


In [ ]:
# Create a copy of the DataFrame to avoid modifying the original while iterating
rules_2023_shop12_checkredundance = rules_2023_shop12_redundant.copy()

# List to hold non-redundant rules
non_redundant_rules_2023_shop12 = []

for _, rule in rules_2023_shop12_checkredundance.iterrows():
    if not is_redundant(rule, rules_2023_shop12_checkredundance):
        non_redundant_rules_2023_shop12.append(rule)

# Convert the list back to a DataFrame
rules_2023_shop12_nonredundant = pd.DataFrame(non_redundant_rules_2023_shop12).reset_index(drop=True)


In [ ]:
print(rules_2023_shop12_nonredundant.head())
print(f"Original number of rules: {len(rules_2023_shop12_redundant)}")
print(f"Number of non-redundant rules: {len(rules_2023_shop12_nonredundant)}")


In [ ]:
rules_2023_shop12_nonredundant.head()

In [ ]:
rules_2023_shop12_redundant.info()

In [ ]:
rules_2023_shop12_nonredundant.head()

In [ ]:
rules_2023_shop12_nonredundant_backup = rules_2023_shop12_nonredundant.copy()

In [ ]:
rules_2023_shop12_nonredundant_unfiltered = rules_2023_shop12_nonredundant.copy()

In [ ]:
#reset
#rules_2023_shop12_nonredundant = rules_2023_shop12_nonredundant_backup.copy()

# Redundant and Unfiltered

In [ ]:
rules_2023_shop12_redundant.head()

In [ ]:
G_2023_shop12 = nx.DiGraph()

# Iterate through rules for 2023 Shop 12 and add edges to the graph
for _, row in rules_2023_shop12_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as weight
    G_2023_shop12.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2023_shop12 = nx.spring_layout(G_2023_shop12)

# Recalculate node sizes based on indegree or another metric
in_degrees_2023_shop12 = dict(G_2023_shop12.in_degree())
node_sizes_2023_shop12 = [in_degrees_2023_shop12[node] * 100 + 1000 for node in G_2023_shop12.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2023_shop12 = (np.array(node_sizes_2023_shop12) - np.min(node_sizes_2023_shop12)) / (np.max(node_sizes_2023_shop12) - np.min(node_sizes_2023_shop12))

# Define a colormap from orange to red for visual consistency
cmap_2023_shop12 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop12 = [cmap_2023_shop12(size) for size in normalized_sizes_2023_shop12]

# Drawing the network
plt.figure(figsize=(60, 43))
nx.draw_networkx_nodes(G_2023_shop12, pos_2023_shop12, node_size=node_sizes_2023_shop12, node_color=node_colors_2023_shop12, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop12, pos_2023_shop12, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop12, pos_2023_shop12, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 12 with Color Gradient')
plt.axis('off')
plt.show()


### Based on LIFT

In [ ]:
G_2023_shop12_lift = nx.DiGraph()

lift_summary_2023_shop12 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop12_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop12_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop12:
        lift_summary_2023_shop12[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop12[consequents]['count'] += 1
    else:
        lift_summary_2023_shop12[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop12:
    lift_summary_2023_shop12[item]['avg_lift'] = lift_summary_2023_shop12[item]['total_lift'] / lift_summary_2023_shop12[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop12_lift = [lift_summary_2023_shop12[node]['avg_lift'] * 500 if node in lift_summary_2023_shop12 else 1000 for node in G_2023_shop12_lift.nodes()]

pos_2023_shop12_lift = nx.spring_layout(G_2023_shop12_lift)

# Colormap from orange to red
cmap_2023_shop12_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop12_lift = np.array(node_sizes_2023_shop12_lift)
normalized_sizes_2023_shop12_lift = (normalized_sizes_2023_shop12_lift - min(normalized_sizes_2023_shop12_lift)) / (max(normalized_sizes_2023_shop12_lift) - min(normalized_sizes_2023_shop12_lift))
node_colors_2023_shop12_lift = [cmap_2023_shop12_lift(size) for size in normalized_sizes_2023_shop12_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2023_shop12_lift, pos_2023_shop12_lift, node_size=node_sizes_2023_shop12_lift, node_color=node_colors_2023_shop12_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop12_lift, pos_2023_shop12_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop12_lift, pos_2023_shop12_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 12 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


# nonredundant_unfiltered

## Based on Indegree

In [ ]:
G_2023_shop12_nonredundant_unfiltered = nx.DiGraph()

# Iterating through each row to add edges with confidence as an attribute
for _, row in rules_2023_shop12_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop12_nonredundant_unfiltered.add_edge(antecedents, consequents, weight=row['confidence'])

# Defining the layout for network visualization
pos_2023_shop12_nonredundant_unfiltered = nx.spring_layout(G_2023_shop12_nonredundant_unfiltered)

# Calculating node sizes based on the indegree for color mapping
in_degrees_2023_shop12_nonredundant_unfiltered = dict(G_2023_shop12_nonredundant_unfiltered.in_degree())
node_sizes_2023_shop12_nonredundant_unfiltered = [in_degrees_2023_shop12_nonredundant_unfiltered[node] * 100 + 1000 for node in G_2023_shop12_nonredundant_unfiltered.nodes()]

# Normalizing node sizes for color mapping
normalized_sizes_2023_shop12_nonredundant_unfiltered = (np.array(node_sizes_2023_shop12_nonredundant_unfiltered) - min(node_sizes_2023_shop12_nonredundant_unfiltered)) / (max(node_sizes_2023_shop12_nonredundant_unfiltered) - min(node_sizes_2023_shop12_nonredundant_unfiltered))

# Defining a color map from orange to red
cmap_2023_shop12_nonredundant_unfiltered = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])

# Mapping normalized node sizes to colors
node_colors_2023_shop12_nonredundant_unfiltered = [cmap_2023_shop12_nonredundant_unfiltered(size) for size in normalized_sizes_2023_shop12_nonredundant_unfiltered]

# Drawing the network with node sizes and colors
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2023_shop12_nonredundant_unfiltered, pos_2023_shop12_nonredundant_unfiltered, node_size=node_sizes_2023_shop12_nonredundant_unfiltered, node_color=node_colors_2023_shop12_nonredundant_unfiltered, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop12_nonredundant_unfiltered, pos_2023_shop12_nonredundant_unfiltered, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2023_shop12_nonredundant_unfiltered, pos_2023_shop12_nonredundant_unfiltered, font_size=10)

plt.title('Node Size Based on Indegree for 2023 Shop 12')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2023_shop12_lift = nx.DiGraph()

lift_summary_2023_shop12 = {}

# Iterate through each row to add edges with lift as an attribute
for _, row in rules_2023_shop12_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop12_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Update lift summary for consequents
    if consequents in lift_summary_2023_shop12:
        lift_summary_2023_shop12[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop12[consequents]['count'] += 1
    else:
        lift_summary_2023_shop12[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculate average lift for each consequent
for item in lift_summary_2023_shop12:
    lift_summary_2023_shop12[item]['avg_lift'] = lift_summary_2023_shop12[item]['total_lift'] / lift_summary_2023_shop12[item]['count']

# Node sizes based on average lift, scaled appropriately
node_sizes_2023_shop12_lift = [lift_summary_2023_shop12[node]['avg_lift'] * 500 if node in lift_summary_2023_shop12 else 1000 for node in G_2023_shop12_lift.nodes()]

pos_2023_shop12_lift = nx.spring_layout(G_2023_shop12_lift)

# Use a colormap that transitions from orange to red
cmap_2023_shop12_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop12_lift = np.array(node_sizes_2023_shop12_lift)
normalized_sizes_2023_shop12_lift = (normalized_sizes_2023_shop12_lift - min(normalized_sizes_2023_shop12_lift)) / (max(normalized_sizes_2023_shop12_lift) - min(normalized_sizes_2023_shop12_lift))
node_colors_2023_shop12_lift = [cmap_2023_shop12_lift(size) for size in normalized_sizes_2023_shop12_lift]

plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2023_shop12_lift, pos_2023_shop12_lift, node_size=node_sizes_2023_shop12_lift, node_color=node_colors_2023_shop12_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop12_lift, pos_2023_shop12_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop12_lift, pos_2023_shop12_lift, font_size=10)

plt.title('Node Size Based on Average Lift for 2023 Shop 12')
plt.axis('off')
plt.show()


# Based on conviction

In [ ]:
rules_2023_shop12_nonredundant.head()

In [ ]:
G_2023_shop12_conviction = nx.DiGraph()

conviction_summary_2023_shop12 = {}
count_summary_2023_shop12 = {}

# Assuming 'rules_2023_shop12_nonredundant_unfiltered' contains a 'conviction' column
for _, row in rules_2023_shop12_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop12_conviction.add_edge(antecedents, consequents, weight=row['confidence'], conviction=row['conviction'])
    
    # Aggregate conviction values and count occurrences
    if consequents in conviction_summary_2023_shop12:
        conviction_summary_2023_shop12[consequents] += row['conviction']
        count_summary_2023_shop12[consequents] += 1
    else:
        conviction_summary_2023_shop12[consequents] = row['conviction']
        count_summary_2023_shop12[consequents] = 1

# Calculate average conviction values
average_conviction_summary_2023_shop12 = {k: conviction_summary_2023_shop12[k] / count_summary_2023_shop12[k] for k in conviction_summary_2023_shop12}

# Normalize conviction values for sizing and coloring
max_conviction = max(average_conviction_summary_2023_shop12.values())
min_conviction = min(average_conviction_summary_2023_shop12.values())
conviction_range = max_conviction - min_conviction
node_colors = []
node_sizes = []

for node in G_2023_shop12_conviction.nodes():
    # Get the average conviction value for the node, defaulting to min if not present
    conviction = average_conviction_summary_2023_shop12.get(node, min_conviction)
    # Normalize between 0 and 1
    normalized_conviction = (conviction - min_conviction) / conviction_range if conviction_range else 0
    node_colors.append(normalized_conviction)
    node_sizes.append(normalized_conviction * 1000 + 100)  # Scale and offset size

# Use a colormap for coloring nodes based on conviction
cmap = plt.cm.coolwarm

pos = nx.spring_layout(G_2023_shop12_conviction)

plt.figure(figsize=(40, 22))
nodes = nx.draw_networkx_nodes(G_2023_shop12_conviction, pos, node_color=node_colors, node_size=node_sizes, cmap=cmap, alpha=0.8)
edges = nx.draw_networkx_edges(G_2023_shop12_conviction, pos, arrowstyle='->', arrowsize=10, edge_color='grey', alpha=0.5)
labels = nx.draw_networkx_labels(G_2023_shop12_conviction, pos, font_size=8)

plt.colorbar(nodes, label='Normalized Conviction')
plt.title('Network Graph of 2023 Shop 12 Based on Conviction')
plt.axis('off')
plt.show()

In [ ]:
print(max_conviction)

# Filtered top 30 and non redundant

In [ ]:
rules_2023_shop12_nonredundant.head()

In [ ]:
# Assuming rules_1992_shop2_nonredundant is your sorted rules DataFrame for shop2
rules_2023_shop12_nonredundant_top30 = rules_2023_shop12_nonredundant.copy(deep=True)

In [ ]:
rules_2023_shop12_nonredundant_top30_lift = rules_2023_shop12_nonredundant.copy(deep=True)

In [ ]:
# Select the top 30 rules based on lift or another criterion
rules_2023_shop12_nonredundant_top30_lift = rules_2023_shop12_nonredundant_top30_lift.sort_values(by='lift', ascending=False).head(30)

In [ ]:
rules_2023_shop12_nonredundant_top30_lift.head()

In [ ]:
# Select the top 30 rules based on confidence or another criterion
rules_2023_shop12_nonredundant_top30 = rules_2023_shop12_nonredundant_top30.sort_values(by='confidence', ascending=False).head(30)

In [ ]:
rules_2023_shop12_nonredundant_top30.info()

In [ ]:
rules_2023_shop12_nonredundant_top30['LHS'] = [','.join(list(x)) for x in rules_2023_shop12_nonredundant_top30['antecedents']]
rules_2023_shop12_nonredundant_top30['RHS'] = [','.join(list(x)) for x in rules_2023_shop12_nonredundant_top30['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2023_shop12_nonredundant_top30.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2023_shop12_nonredundant_top30, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2023 Shop 12")
plt.axis('off')
plt.show()


## Based on Indegree

In [ ]:
G_2023_shop12 = nx.DiGraph()

# Assuming 'rules_sorted_2023_shop12' contains association rules for shop 12 in 2023
for _, row in rules_2023_shop12_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as the weight
    G_2023_shop12.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2023_shop12 = nx.spring_layout(G_2023_shop12)

# Calculate node sizes based on indegree
in_degrees_2023_shop12 = dict(G_2023_shop12.in_degree())
node_sizes_2023_shop12 = [in_degrees_2023_shop12[node] * 100 + 1000 for node in G_2023_shop12.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2023_shop12 = (np.array(node_sizes_2023_shop12) - np.min(node_sizes_2023_shop12)) / (np.max(node_sizes_2023_shop12) - np.min(node_sizes_2023_shop12))

# Define a colormap from orange to red
cmap_2023_shop12 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop12 = [cmap_2023_shop12(size) for size in normalized_sizes_2023_shop12]

# Drawing the network
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2023_shop12, pos_2023_shop12, node_size=node_sizes_2023_shop12, node_color=node_colors_2023_shop12, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop12, pos_2023_shop12, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2023_shop12, pos_2023_shop12, font_size=8)

plt.title('Top 5 Association Rules Network for 2023 Shop 12 with Color Gradient')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2023_shop12_lift = nx.DiGraph()

lift_summary_2023_shop12 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop12_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop12_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop12:
        lift_summary_2023_shop12[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop12[consequents]['count'] += 1
    else:
        lift_summary_2023_shop12[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop12:
    lift_summary_2023_shop12[item]['avg_lift'] = lift_summary_2023_shop12[item]['total_lift'] / lift_summary_2023_shop12[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop12_lift = [lift_summary_2023_shop12[node]['avg_lift'] * 500 if node in lift_summary_2023_shop12 else 1000 for node in G_2023_shop12_lift.nodes()]

pos_2023_shop12_lift = nx.spring_layout(G_2023_shop12_lift)

# Colormap from orange to red
cmap_2023_shop12_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop12_lift = np.array(node_sizes_2023_shop12_lift)
normalized_sizes_2023_shop12_lift = (normalized_sizes_2023_shop12_lift - min(normalized_sizes_2023_shop12_lift)) / (max(normalized_sizes_2023_shop12_lift) - min(normalized_sizes_2023_shop12_lift))
node_colors_2023_shop12_lift = [cmap_2023_shop12_lift(size) for size in normalized_sizes_2023_shop12_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2023_shop12_lift, pos_2023_shop12_lift, node_size=node_sizes_2023_shop12_lift, node_color=node_colors_2023_shop12_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop12_lift, pos_2023_shop12_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop12_lift, pos_2023_shop12_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 12 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop12_lift = nx.DiGraph()

lift_summary_2023_shop12 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop12_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop12_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop12:
        lift_summary_2023_shop12[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop12[consequents]['count'] += 1
    else:
        lift_summary_2023_shop12[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop12:
    lift_summary_2023_shop12[item]['avg_lift'] = lift_summary_2023_shop12[item]['total_lift'] / lift_summary_2023_shop12[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop12_lift = [lift_summary_2023_shop12[node]['avg_lift'] * 500 if node in lift_summary_2023_shop12 else 1000 for node in G_2023_shop12_lift.nodes()]

pos_2023_shop12_lift = nx.spring_layout(G_2023_shop12_lift)

# Colormap from orange to red
cmap_2023_shop12_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop12_lift = np.array(node_sizes_2023_shop12_lift)
normalized_sizes_2023_shop12_lift = (normalized_sizes_2023_shop12_lift - min(normalized_sizes_2023_shop12_lift)) / (max(normalized_sizes_2023_shop12_lift) - min(normalized_sizes_2023_shop12_lift))
node_colors_2023_shop12_lift = [cmap_2023_shop12_lift(size) for size in normalized_sizes_2023_shop12_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2023_shop12_lift, pos_2023_shop12_lift, node_size=node_sizes_2023_shop12_lift, node_color=node_colors_2023_shop12_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop12_lift, pos_2023_shop12_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop12_lift, pos_2023_shop12_lift, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 12 Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop12 = nx.DiGraph()

# Assuming 'rules_2023_shop12_nonredundant' contains association rules for 2023 Shop 12
for _, row in rules_2023_shop12_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop12.add_edge(antecedents, consequents, confidence=row['confidence'])

# Calculate node sizes based on indegree
in_degrees_2023_shop12 = dict(G_2023_shop12.in_degree())
node_sizes_2023_shop12 = [in_degrees_2023_shop12[node] * 100 + 1000 for node in G_2023_shop12.nodes()]

# Edge width based on confidence
edge_widths_2023_shop12 = [G_2023_shop12[u][v]['confidence'] * 15 for u, v in G_2023_shop12.edges()]

# Assuming normalized sizes and color mapping is desired
normalized_sizes_2023_shop12 = np.array(list(in_degrees_2023_shop12.values()))
normalized_sizes_2023_shop12 = (normalized_sizes_2023_shop12 - min(normalized_sizes_2023_shop12)) / (max(normalized_sizes_2023_shop12) - min(normalized_sizes_2023_shop12))
cmap_2023_shop12 = mcolors.LinearSegmentedColormap.from_list("MyCMapName", ["blue", "red"])

# Node colors based on a property (here, using indegree for example)
node_colors_2023_shop12 = [cmap_2023_shop12(size) for size in normalized_sizes_2023_shop12]

# Layout
pos_2023_shop12 = nx.spring_layout(G_2023_shop12, seed=42)

# Drawing
plt.figure(figsize=(40, 30))
nx.draw_networkx_edges(G_2023_shop12, pos_2023_shop12, width=edge_widths_2023_shop12, edge_color='grey', alpha=0.5)
nx.draw_networkx_nodes(G_2023_shop12, pos_2023_shop12, node_size=node_sizes_2023_shop12, node_color=node_colors_2023_shop12, alpha=0.8)
nx.draw_networkx_labels(G_2023_shop12, pos_2023_shop12, font_size=8, font_color='black')
plt.title('Network Graph of Association Rules Based on Confidence for 2023 Shop 12')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop12 = nx.DiGraph()

convictions_2023_shop12 = []  # To store conviction values for normalization

for _, row in rules_2023_shop12_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    conviction = row['conviction']  # Assuming each row has a 'conviction' column
    convictions_2023_shop12.append(conviction)
    G_2023_shop12.add_edge(antecedents, consequents, weight=row['confidence'], conviction=conviction)

# Normalize conviction values for edge width
max_conviction_2023_shop12 = max(convictions_2023_shop12)
min_conviction_2023_shop12 = min(convictions_2023_shop12)
conviction_range_2023_shop12 = max_conviction_2023_shop12 - min_conviction_2023_shop12
edge_widths_2023_shop12 = [(G_2023_shop12[u][v]['conviction'] - min_conviction_2023_shop12) / conviction_range_2023_shop12 * 30 + 0.5 for u, v in G_2023_shop12.edges()]  # Scale and offset widths

pos_2023_shop12 = nx.spring_layout(G_2023_shop12)

# Node sizes and colors as before
in_degrees_2023_shop12 = dict(G_2023_shop12.in_degree())
node_sizes_2023_shop12 = [in_degrees_2023_shop12[node] * 100 + 1000 for node in G_2023_shop12.nodes()]
normalized_sizes_2023_shop12 = (np.array(node_sizes_2023_shop12) - min(node_sizes_2023_shop12)) / (max(node_sizes_2023_shop12) - min(node_sizes_2023_shop12))
cmap_2023_shop12 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop12 = [cmap_2023_shop12(size) for size in normalized_sizes_2023_shop12]

# Drawing the network with conviction-based edge widths
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2023_shop12, pos_2023_shop12, node_size=node_sizes_2023_shop12, node_color=node_colors_2023_shop12, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop12, pos_2023_shop12, width=edge_widths_2023_shop12, edge_color='grey', alpha=0.5)
nx.draw_networkx_labels(G_2023_shop12, pos_2023_shop12, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 12 with Conviction-based Edge Thickness')
plt.axis('off')
plt.show()


In [ ]:
#the node sizes are based on the indegree where each node acts as a consequent,

In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2023_shop12_nonredundant_top30.loc[:, 'antecedents_'] = rules_2023_shop12_nonredundant_top30['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2023_shop12_nonredundant_top30.loc[:, 'consequents_'] = rules_2023_shop12_nonredundant_top30['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2023_shop12_nonredundant_top30, x='consequents_', y='antecedents_', s=rules_2023_shop12_nonredundant_top30['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2023_shop12_nonredundant_top30.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2023 Shop 12", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
print(rules_2023_shop12_nonredundant_top30[['antecedents', 'consequents']].head(31))

## Monetary Analysis of the Top Rules

In [ ]:
# Step 1: Extract unique product codes from the rules
unique_product_codes = set()
for idx, row in rules_2023_shop12_nonredundant_top30.iterrows():
    # Unite antecedents and consequents into a single set
    all_products = row['antecedents'].union(row['consequents'])
    unique_product_codes.update(all_products)

# Step 2: Create a DataFrame with unique product codes and their unit prices
# Filter sales_data for these product codes and drop duplicates to ensure each product code appears once
product_prices = sales_data[sales_data['product_code'].isin(unique_product_codes)][['product_code', 'unit_price']].drop_duplicates()

# Create a dictionary for quick lookup of prices
price_dict = product_prices.set_index('product_code')['unit_price'].to_dict()

# Step 3: Calculate the total price for each rule and collect data for the new DataFrame
data_for_new_df = []
for idx, row in rules_2023_shop12_nonredundant_top30.iterrows():
    # Combine sets of antecedents and consequents
    all_products = row['antecedents'].union(row['consequents'])
    total_price = sum(price_dict.get(prod, 0) for prod in all_products)  # Use get to avoid KeyError if any product code is missing
    data_for_new_df.append({
        'antecedents': row['antecedents'],
        'consequents': row['consequents'],
        'total_unit_price': total_price
    })

# Create a new DataFrame with only antecedents, consequents, and total unit price
rules_and_price_shop12_2023 = pd.DataFrame(data_for_new_df)

# Sort the DataFrame by total_unit_price in descending order
rules_and_price_shop12_2023_sorted = rules_and_price_shop12_2023.sort_values(by='total_unit_price', ascending=False)

# Print the sorted DataFrame to see the specified columns along with their total unit prices
print(rules_and_price_shop12_2023_sorted)


In [ ]:
# Convert 'trx_date' to datetime format if not already done so
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Add a 'year' column if it doesn't exist
sales_data['year'] = sales_data['trx_date'].dt.year

# Step 2: Filter sales_data for shop12 and the year 2023
filtered_sales_data_2023 = sales_data[(sales_data['shopID'] == 12) & (sales_data['year'] == 2023)]

# Step 3: Initialize a list to hold the rule data including total sales value
rule_sales_summaries_2023 = []

# Step 4: Iterate through each rule in the DataFrame
for index, rule in rules_2023_shop12_nonredundant_top30.iterrows():
    all_products = rule['antecedents'].union(rule['consequents'])

    # Filter the sales data for transactions that include all products in the rule
    # This step ensures that we are only considering transactions that contain all specified products
    mask = filtered_sales_data_2023['product_code'].isin(all_products)
    transaction_ids = filtered_sales_data_2023[mask].groupby('trx_id').filter(lambda x: all_products.issubset(set(x['product_code']))).trx_id.unique()
    matched_transactions = filtered_sales_data_2023[filtered_sales_data_2023['trx_id'].isin(transaction_ids)]
    
    # Step 5: Sum the total_value of these transactions
    total_sales_value = matched_transactions['total_value'].sum()

    # Append the result to the list
    rule_sales_summaries_2023.append({
        'antecedents': rule['antecedents'],
        'consequents': rule['consequents'],
        'total_sales_value': total_sales_value
    })

# Step 6: Convert the list to a DataFrame
rules_sales_values_df_2023 = pd.DataFrame(rule_sales_summaries_2023)

# Step 7: Sort by total_sales_value in descending order
sorted_rules_sales_values_df_2023 = rules_sales_values_df_2023.sort_values(by='total_sales_value', ascending=False)

# Step 8: Print the sorted DataFrame
print(sorted_rules_sales_values_df_2023)

## Narrowing down based on Lift

In [ ]:
rules_2023_shop12_nonredundant_top30_lift['LHS'] = [','.join(list(x)) for x in rules_2023_shop12_nonredundant_top30_lift['antecedents']]
rules_2023_shop12_nonredundant_top30_lift['RHS'] = [','.join(list(x)) for x in rules_2023_shop12_nonredundant_top30_lift['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2023_shop12_nonredundant_top30_lift.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2023_shop12_nonredundant_top30_lift, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2023 Shop 12")
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop12_lift = nx.DiGraph()

lift_summary_2023_shop12 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop12_nonredundant_top30_lift.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop12_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop12:
        lift_summary_2023_shop12[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop12[consequents]['count'] += 1
    else:
        lift_summary_2023_shop12[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop12:
    lift_summary_2023_shop12[item]['avg_lift'] = lift_summary_2023_shop12[item]['total_lift'] / lift_summary_2023_shop12[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop12_lift = [lift_summary_2023_shop12[node]['avg_lift'] * 500 if node in lift_summary_2023_shop12 else 1000 for node in G_2023_shop12_lift.nodes()]

pos_2023_shop12_lift = nx.spring_layout(G_2023_shop12_lift)

# Colormap from orange to red
cmap_2023_shop12_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop12_lift = np.array(node_sizes_2023_shop12_lift)
normalized_sizes_2023_shop12_lift = (normalized_sizes_2023_shop12_lift - min(normalized_sizes_2023_shop12_lift)) / (max(normalized_sizes_2023_shop12_lift) - min(normalized_sizes_2023_shop12_lift))
node_colors_2023_shop12_lift = [cmap_2023_shop12_lift(size) for size in normalized_sizes_2023_shop12_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2023_shop12_lift, pos_2023_shop12_lift, node_size=node_sizes_2023_shop12_lift, node_color=node_colors_2023_shop12_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop12_lift, pos_2023_shop12_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop12_lift, pos_2023_shop12_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 12 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2023_shop12_nonredundant_top30_lift.loc[:, 'antecedents_'] = rules_2023_shop12_nonredundant_top30_lift['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2023_shop12_nonredundant_top30_lift.loc[:, 'consequents_'] = rules_2023_shop12_nonredundant_top30_lift['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2023_shop12_nonredundant_top30_lift, x='consequents_', y='antecedents_', s=rules_2023_shop12_nonredundant_top30_lift['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2023_shop12_nonredundant_top30_lift.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2023 Shop 12", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


# Shop 13 2023 SEARCH 21

In [ ]:
# Filter for records from shop ID 1 and the year 1993
sales_data_2023_shop13 = sales_data_shop13[(sales_data_shop13['trx_date'].dt.year == 2023)]

In [ ]:
sales_data_2023_shop13.head()

In [ ]:
# Grouping product codes by trx_id
transactions_2023_shop13 = sales_data_2023_shop13.groupby('trx_id')['product_code'].apply(list).tolist()

In [ ]:
# Encode transaction data
encoder_2023_shop13 = TransactionEncoder()
trans_array_2023_shop13 = encoder_2023_shop13.fit(transactions_2023_shop13).transform(transactions_2023_shop13)
trans_df_2023_shop13 = pd.DataFrame(trans_array_2023_shop13, columns=encoder_2023_shop13.columns_)

In [ ]:
# Find frequent itemsets
frequent_itemsets_2023_shop13 = apriori(trans_df_2023_shop13, min_support=0.001, use_colnames=True)

In [ ]:
print(frequent_itemsets_2023_shop13)

In [ ]:
## Generate the association rules - by confidence
rulesConfidence_2023_shop13 = association_rules(frequent_itemsets_2023_shop13, metric="confidence", min_threshold=0.10)
rulesConfidence_2023_shop13.sort_values(by='confidence', ascending=False, inplace=True)


In [ ]:
# Replace frozen sets with strings
rulesConfidence_2023_shop13['antecedents_'] = rulesConfidence_2023_shop13['antecedents'].apply(lambda a: ','.join(list(a)))
rulesConfidence_2023_shop13['consequents_'] = rulesConfidence_2023_shop13['consequents'].apply(lambda a: ','.join(list(a)))

# Draw
fig, ax = plt.subplots(figsize=(14,30))
ax = plt.scatter(data=rulesConfidence_2023_shop13, x='consequents_', y='antecedents_', s=rulesConfidence_2023_shop13['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rulesConfidence_2023_shop13.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2023 Shop 13", fontsize=14)
plt.xlabel("Consequents (RHS)", fontsize=12)
plt.ylabel("Antecedents (LHS)", fontsize=12)
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate(rotation=45)
plt.tight_layout()

plt.show()


## Eliminate Redundant Rules

In [ ]:
rulesConfidence_2023_shop13.head()

In [ ]:
rulesConfidence_2023_shop13.info()

In [ ]:
# If 'antecedents' and 'consequents' are not frozensets, convert them
rules_2023_shop13_redundant = rulesConfidence_2023_shop13.copy()
rules_2023_shop13_redundant['antecedents'] = rules_2023_shop13_redundant['antecedents'].apply(lambda x: frozenset(x))
rules_2023_shop13_redundant['consequents'] = rules_2023_shop13_redundant['consequents'].apply(lambda x: frozenset(x))


In [ ]:
# Create a copy of the DataFrame to avoid modifying the original while iterating
rules_2023_shop13_checkredundance = rules_2023_shop13_redundant.copy()

# List to hold non-redundant rules
non_redundant_rules_2023_shop13 = []

for _, rule in rules_2023_shop13_checkredundance.iterrows():
    if not is_redundant(rule, rules_2023_shop13_checkredundance):
        non_redundant_rules_2023_shop13.append(rule)

# Convert the list back to a DataFrame
rules_2023_shop13_nonredundant = pd.DataFrame(non_redundant_rules_2023_shop13).reset_index(drop=True)


In [ ]:
print(rules_2023_shop13_nonredundant.head())
print(f"Original number of rules: {len(rules_2023_shop13_redundant)}")
print(f"Number of non-redundant rules: {len(rules_2023_shop13_nonredundant)}")


In [ ]:
rules_2023_shop13_nonredundant.head()

In [ ]:
rules_2023_shop13_redundant.info()

In [ ]:
rules_2023_shop13_nonredundant.head()

In [ ]:
rules_2023_shop13_nonredundant_backup = rules_2023_shop13_nonredundant.copy()

In [ ]:
rules_2023_shop13_nonredundant_unfiltered = rules_2023_shop13_nonredundant.copy()

In [ ]:
#reset
#rules_2023_shop13_nonredundant = rules_2023_shop13_nonredundant_backup.copy()

# Redundant and Unfiltered

In [ ]:
rules_2023_shop13_redundant.head()

In [ ]:
G_2023_shop13 = nx.DiGraph()

# Iterate through rules for 2023 Shop 13 and add edges to the graph
for _, row in rules_2023_shop11_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as weight
    G_2023_shop13.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2023_shop13 = nx.spring_layout(G_2023_shop13)

# Recalculate node sizes based on indegree or another metric
in_degrees_2023_shop13 = dict(G_2023_shop13.in_degree())
node_sizes_2023_shop13 = [in_degrees_2023_shop13[node] * 100 + 1000 for node in G_2023_shop13.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2023_shop13 = (np.array(node_sizes_2023_shop13) - np.min(node_sizes_2023_shop13)) / (np.max(node_sizes_2023_shop13) - np.min(node_sizes_2023_shop13))

# Define a colormap from orange to red for visual consistency
cmap_2023_shop13 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop13 = [cmap_2023_shop13(size) for size in normalized_sizes_2023_shop13]

# Drawing the network
plt.figure(figsize=(60, 43))
nx.draw_networkx_nodes(G_2023_shop13, pos_2023_shop13, node_size=node_sizes_2023_shop13, node_color=node_colors_2023_shop13, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop13, pos_2023_shop13, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop13, pos_2023_shop13, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 13 with Color Gradient')
plt.axis('off')
plt.show()


### Based on LIFT

In [ ]:
G_2023_shop13_lift = nx.DiGraph()

lift_summary_2023_shop13 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop11_redundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop13_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop13:
        lift_summary_2023_shop13[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop13[consequents]['count'] += 1
    else:
        lift_summary_2023_shop13[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop13:
    lift_summary_2023_shop13[item]['avg_lift'] = lift_summary_2023_shop13[item]['total_lift'] / lift_summary_2023_shop13[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop13_lift = [lift_summary_2023_shop13[node]['avg_lift'] * 500 if node in lift_summary_2023_shop13 else 1000 for node in G_2023_shop13_lift.nodes()]

pos_2023_shop13_lift = nx.spring_layout(G_2023_shop13_lift)

# Colormap from orange to red
cmap_2023_shop13_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop13_lift = np.array(node_sizes_2023_shop13_lift)
normalized_sizes_2023_shop13_lift = (normalized_sizes_2023_shop13_lift - min(normalized_sizes_2023_shop13_lift)) / (max(normalized_sizes_2023_shop13_lift) - min(normalized_sizes_2023_shop13_lift))
node_colors_2023_shop13_lift = [cmap_2023_shop13_lift(size) for size in normalized_sizes_2023_shop13_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2023_shop13_lift, pos_2023_shop13_lift, node_size=node_sizes_2023_shop13_lift, node_color=node_colors_2023_shop13_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop13_lift, pos_2023_shop13_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop13_lift, pos_2023_shop13_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 13 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


# nonredundant_unfiltered

## Based on Indegree

In [ ]:
G_2023_shop13_nonredundant_unfiltered = nx.DiGraph()

# Iterating through each row to add edges with confidence as an attribute
for _, row in rules_2023_shop11_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop13_nonredundant_unfiltered.add_edge(antecedents, consequents, weight=row['confidence'])

# Defining the layout for network visualization
pos_2023_shop13_nonredundant_unfiltered = nx.spring_layout(G_2023_shop13_nonredundant_unfiltered)

# Calculating node sizes based on the indegree for color mapping
in_degrees_2023_shop13_nonredundant_unfiltered = dict(G_2023_shop13_nonredundant_unfiltered.in_degree())
node_sizes_2023_shop13_nonredundant_unfiltered = [in_degrees_2023_shop13_nonredundant_unfiltered[node] * 100 + 1000 for node in G_2023_shop13_nonredundant_unfiltered.nodes()]

# Normalizing node sizes for color mapping
normalized_sizes_2023_shop13_nonredundant_unfiltered = (np.array(node_sizes_2023_shop13_nonredundant_unfiltered) - min(node_sizes_2023_shop13_nonredundant_unfiltered)) / (max(node_sizes_2023_shop13_nonredundant_unfiltered) - min(node_sizes_2023_shop13_nonredundant_unfiltered))

# Defining a color map from orange to red
cmap_2023_shop13_nonredundant_unfiltered = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])

# Mapping normalized node sizes to colors
node_colors_2023_shop13_nonredundant_unfiltered = [cmap_2023_shop13_nonredundant_unfiltered(size) for size in normalized_sizes_2023_shop13_nonredundant_unfiltered]

# Drawing the network with node sizes and colors
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2023_shop13_nonredundant_unfiltered, pos_2023_shop13_nonredundant_unfiltered, node_size=node_sizes_2023_shop13_nonredundant_unfiltered, node_color=node_colors_2023_shop13_nonredundant_unfiltered, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop13_nonredundant_unfiltered, pos_2023_shop13_nonredundant_unfiltered, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2023_shop13_nonredundant_unfiltered, pos_2023_shop13_nonredundant_unfiltered, font_size=10)

plt.title('Node Size Based on Indegree for 2023 Shop 13')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2023_shop13_lift = nx.DiGraph()

lift_summary_2023_shop13 = {}

# Iterate through each row to add edges with lift as an attribute
for _, row in rules_2023_shop11_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop13_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Update lift summary for consequents
    if consequents in lift_summary_2023_shop13:
        lift_summary_2023_shop13[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop13[consequents]['count'] += 1
    else:
        lift_summary_2023_shop13[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculate average lift for each consequent
for item in lift_summary_2023_shop13:
    lift_summary_2023_shop13[item]['avg_lift'] = lift_summary_2023_shop13[item]['total_lift'] / lift_summary_2023_shop13[item]['count']

# Node sizes based on average lift, scaled appropriately
node_sizes_2023_shop13_lift = [lift_summary_2023_shop13[node]['avg_lift'] * 500 if node in lift_summary_2023_shop13 else 1000 for node in G_2023_shop13_lift.nodes()]

pos_2023_shop13_lift = nx.spring_layout(G_2023_shop13_lift)

# Use a colormap that transitions from orange to red
cmap_2023_shop13_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop13_lift = np.array(node_sizes_2023_shop13_lift)
normalized_sizes_2023_shop13_lift = (normalized_sizes_2023_shop13_lift - min(normalized_sizes_2023_shop13_lift)) / (max(normalized_sizes_2023_shop13_lift) - min(normalized_sizes_2023_shop13_lift))
node_colors_2023_shop13_lift = [cmap_2023_shop13_lift(size) for size in normalized_sizes_2023_shop13_lift]

plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2023_shop13_lift, pos_2023_shop13_lift, node_size=node_sizes_2023_shop13_lift, node_color=node_colors_2023_shop13_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop13_lift, pos_2023_shop13_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop13_lift, pos_2023_shop13_lift, font_size=10)

plt.title('Node Size Based on Average Lift for 2023 Shop 13')
plt.axis('off')
plt.show()


# Based on conviction

In [ ]:
rules_2023_shop13_nonredundant.head()

In [ ]:
G_2023_shop13_conviction = nx.DiGraph()

conviction_summary_2023_shop13 = {}
count_summary_2023_shop13 = {}

# Assuming 'rules_2023_shop13_nonredundant_unfiltered' contains a 'conviction' column
for _, row in rules_2023_shop13_nonredundant.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop13_conviction.add_edge(antecedents, consequents, weight=row['confidence'], conviction=row['conviction'])
    
    # Aggregate conviction values and count occurrences
    if consequents in conviction_summary_2023_shop13:
        conviction_summary_2023_shop13[consequents] += row['conviction']
        count_summary_2023_shop13[consequents] += 1
    else:
        conviction_summary_2023_shop13[consequents] = row['conviction']
        count_summary_2023_shop13[consequents] = 1

# Calculate average conviction values
average_conviction_summary_2023_shop13 = {k: conviction_summary_2023_shop13[k] / count_summary_2023_shop13[k] for k in conviction_summary_2023_shop13}

# Normalize conviction values for sizing and coloring
max_conviction = max(average_conviction_summary_2023_shop13.values())
min_conviction = min(average_conviction_summary_2023_shop13.values())
conviction_range = max_conviction - min_conviction
node_colors = []
node_sizes = []

for node in G_2023_shop13_conviction.nodes():
    # Get the average conviction value for the node, defaulting to min if not present
    conviction = average_conviction_summary_2023_shop13.get(node, min_conviction)
    # Normalize between 0 and 1
    normalized_conviction = (conviction - min_conviction) / conviction_range if conviction_range else 0
    node_colors.append(normalized_conviction)
    node_sizes.append(normalized_conviction * 1000 + 100)  # Scale and offset size

# Use a colormap for coloring nodes based on conviction
cmap = plt.cm.coolwarm

pos = nx.spring_layout(G_2023_shop13_conviction)

plt.figure(figsize=(40, 22))
nodes = nx.draw_networkx_nodes(G_2023_shop13_conviction, pos, node_color=node_colors, node_size=node_sizes, cmap=cmap, alpha=0.8)
edges = nx.draw_networkx_edges(G_2023_shop13_conviction, pos, arrowstyle='->', arrowsize=10, edge_color='grey', alpha=0.5)
labels = nx.draw_networkx_labels(G_2023_shop13_conviction, pos, font_size=8)

plt.colorbar(nodes, label='Normalized Conviction')
plt.title('Network Graph of 2023 Shop 13 Based on Conviction')
plt.axis('off')
plt.show()

In [ ]:
print(max_conviction)

# Filtered top 30 and non redundant

In [ ]:
rules_2023_shop13_nonredundant.head()

In [ ]:
# Assuming rules_1992_shop2_nonredundant is your sorted rules DataFrame for shop2
rules_2023_shop13_nonredundant_top30 = rules_2023_shop13_nonredundant.copy(deep=True)

In [ ]:
rules_2023_shop13_nonredundant_top30_lift = rules_2023_shop13_nonredundant.copy(deep=True)

In [ ]:
# Select the top 30 rules based on lift or another criterion
rules_2023_shop13_nonredundant_top30_lift = rules_2023_shop13_nonredundant_top30_lift.sort_values(by='lift', ascending=False).head(30)

In [ ]:
rules_2023_shop13_nonredundant_top30_lift.head()

In [ ]:
# Select the top 30 rules based on confidence or another criterion
rules_2023_shop13_nonredundant_top30 = rules_2023_shop13_nonredundant_top30.sort_values(by='confidence', ascending=False).head(30)

In [ ]:
rules_2023_shop13_nonredundant_top30.info()

In [ ]:
rules_2023_shop13_nonredundant_top30['LHS'] = [','.join(list(x)) for x in rules_2023_shop13_nonredundant_top30['antecedents']]
rules_2023_shop13_nonredundant_top30['RHS'] = [','.join(list(x)) for x in rules_2023_shop13_nonredundant_top30['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2023_shop13_nonredundant_top30.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2023_shop13_nonredundant_top30, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2023 Shop 13")
plt.axis('off')
plt.show()


## Based on Indegree

In [ ]:
G_2023_shop13 = nx.DiGraph()

# Assuming 'rules_sorted_2023_shop13' contains association rules for shop 13 in 2023
for _, row in rules_2023_shop13_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    # Adding the edge with confidence as the weight
    G_2023_shop13.add_edge(antecedents, consequents, weight=row['confidence'])

# Position nodes using the Spring layout
pos_2023_shop13 = nx.spring_layout(G_2023_shop13)

# Calculate node sizes based on indegree
in_degrees_2023_shop13 = dict(G_2023_shop13.in_degree())
node_sizes_2023_shop13 = [in_degrees_2023_shop13[node] * 100 + 1000 for node in G_2023_shop13.nodes()]

# Normalize these sizes for the color mapping
normalized_sizes_2023_shop13 = (np.array(node_sizes_2023_shop13) - np.min(node_sizes_2023_shop13)) / (np.max(node_sizes_2023_shop13) - np.min(node_sizes_2023_shop13))

# Define a colormap from orange to red
cmap_2023_shop13 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop13 = [cmap_2023_shop13(size) for size in normalized_sizes_2023_shop13]

# Drawing the network
plt.figure(figsize=(60, 34))
nx.draw_networkx_nodes(G_2023_shop13, pos_2023_shop13, node_size=node_sizes_2023_shop13, node_color=node_colors_2023_shop13, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop13, pos_2023_shop13, arrows=True, edge_color='grey', width=1)
nx.draw_networkx_labels(G_2023_shop13, pos_2023_shop13, font_size=8)

plt.title('Top 5 Association Rules Network for 2023 Shop 13 with Color Gradient')
plt.axis('off')
plt.show()


## Based on Lift

In [ ]:
G_2023_shop13_lift = nx.DiGraph()

lift_summary_2023_shop13 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop13_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop13_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop13:
        lift_summary_2023_shop13[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop13[consequents]['count'] += 1
    else:
        lift_summary_2023_shop13[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop13:
    lift_summary_2023_shop13[item]['avg_lift'] = lift_summary_2023_shop13[item]['total_lift'] / lift_summary_2023_shop13[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop13_lift = [lift_summary_2023_shop13[node]['avg_lift'] * 500 if node in lift_summary_2023_shop13 else 1000 for node in G_2023_shop13_lift.nodes()]

pos_2023_shop13_lift = nx.spring_layout(G_2023_shop13_lift)

# Colormap from orange to red
cmap_2023_shop13_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop13_lift = np.array(node_sizes_2023_shop13_lift)
normalized_sizes_2023_shop13_lift = (normalized_sizes_2023_shop13_lift - min(normalized_sizes_2023_shop13_lift)) / (max(normalized_sizes_2023_shop13_lift) - min(normalized_sizes_2023_shop13_lift))
node_colors_2023_shop13_lift = [cmap_2023_shop13_lift(size) for size in normalized_sizes_2023_shop13_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2023_shop13_lift, pos_2023_shop13_lift, node_size=node_sizes_2023_shop13_lift, node_color=node_colors_2023_shop13_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop13_lift, pos_2023_shop13_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop13_lift, pos_2023_shop13_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 13 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop13_lift = nx.DiGraph()

lift_summary_2023_shop13 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop13_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop13_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop13:
        lift_summary_2023_shop13[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop13[consequents]['count'] += 1
    else:
        lift_summary_2023_shop13[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop13:
    lift_summary_2023_shop13[item]['avg_lift'] = lift_summary_2023_shop13[item]['total_lift'] / lift_summary_2023_shop13[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop13_lift = [lift_summary_2023_shop13[node]['avg_lift'] * 500 if node in lift_summary_2023_shop13 else 1000 for node in G_2023_shop13_lift.nodes()]

pos_2023_shop13_lift = nx.spring_layout(G_2023_shop13_lift)

# Colormap from orange to red
cmap_2023_shop13_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop13_lift = np.array(node_sizes_2023_shop13_lift)
normalized_sizes_2023_shop13_lift = (normalized_sizes_2023_shop13_lift - min(normalized_sizes_2023_shop13_lift)) / (max(normalized_sizes_2023_shop13_lift) - min(normalized_sizes_2023_shop13_lift))
node_colors_2023_shop13_lift = [cmap_2023_shop13_lift(size) for size in normalized_sizes_2023_shop13_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2023_shop13_lift, pos_2023_shop13_lift, node_size=node_sizes_2023_shop13_lift, node_color=node_colors_2023_shop13_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop13_lift, pos_2023_shop13_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop13_lift, pos_2023_shop13_lift, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 13 Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop13 = nx.DiGraph()

# Assuming 'rules_2023_shop13_nonredundant' contains association rules for 2023 Shop 13
for _, row in rules_2023_shop13_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop13.add_edge(antecedents, consequents, confidence=row['confidence'])

# Calculate node sizes based on indegree
in_degrees_2023_shop13 = dict(G_2023_shop13.in_degree())
node_sizes_2023_shop13 = [in_degrees_2023_shop13[node] * 100 + 1000 for node in G_2023_shop13.nodes()]

# Edge width based on confidence
edge_widths_2023_shop13 = [G_2023_shop13[u][v]['confidence'] * 15 for u, v in G_2023_shop13.edges()]

# Assuming normalized sizes and color mapping is desired
normalized_sizes_2023_shop13 = np.array(list(in_degrees_2023_shop13.values()))
normalized_sizes_2023_shop13 = (normalized_sizes_2023_shop13 - min(normalized_sizes_2023_shop13)) / (max(normalized_sizes_2023_shop13) - min(normalized_sizes_2023_shop13))
cmap_2023_shop13 = mcolors.LinearSegmentedColormap.from_list("MyCMapName", ["blue", "red"])

# Node colors based on a property (here, using indegree for example)
node_colors_2023_shop13 = [cmap_2023_shop13(size) for size in normalized_sizes_2023_shop13]

# Layout
pos_2023_shop13 = nx.spring_layout(G_2023_shop13, seed=42)

# Drawing
plt.figure(figsize=(40, 30))
nx.draw_networkx_edges(G_2023_shop13, pos_2023_shop13, width=edge_widths_2023_shop13, edge_color='grey', alpha=0.5)
nx.draw_networkx_nodes(G_2023_shop13, pos_2023_shop13, node_size=node_sizes_2023_shop13, node_color=node_colors_2023_shop13, alpha=0.8)
nx.draw_networkx_labels(G_2023_shop13, pos_2023_shop13, font_size=8, font_color='black')
plt.title('Network Graph of Association Rules Based on Confidence for 2023 Shop 13')
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop13 = nx.DiGraph()

convictions_2023_shop13 = []  # To store conviction values for normalization

for _, row in rules_2023_shop13_nonredundant_top30.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    conviction = row['conviction']  # Assuming each row has a 'conviction' column
    convictions_2023_shop13.append(conviction)
    G_2023_shop13.add_edge(antecedents, consequents, weight=row['confidence'], conviction=conviction)

# Normalize conviction values for edge width
max_conviction_2023_shop13 = max(convictions_2023_shop13)
min_conviction_2023_shop13 = min(convictions_2023_shop13)
conviction_range_2023_shop13 = max_conviction_2023_shop13 - min_conviction_2023_shop13
edge_widths_2023_shop13 = [(G_2023_shop13[u][v]['conviction'] - min_conviction_2023_shop13) / conviction_range_2023_shop13 * 30 + 0.5 for u, v in G_2023_shop13.edges()]  # Scale and offset widths

pos_2023_shop13 = nx.spring_layout(G_2023_shop13)

# Node sizes and colors as before
in_degrees_2023_shop13 = dict(G_2023_shop13.in_degree())
node_sizes_2023_shop13 = [in_degrees_2023_shop13[node] * 100 + 1000 for node in G_2023_shop13.nodes()]
normalized_sizes_2023_shop13 = (np.array(node_sizes_2023_shop13) - min(node_sizes_2023_shop13)) / (max(node_sizes_2023_shop13) - min(node_sizes_2023_shop13))
cmap_2023_shop13 = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
node_colors_2023_shop13 = [cmap_2023_shop13(size) for size in normalized_sizes_2023_shop13]

# Drawing the network with conviction-based edge widths
plt.figure(figsize=(50, 25))
nx.draw_networkx_nodes(G_2023_shop13, pos_2023_shop13, node_size=node_sizes_2023_shop13, node_color=node_colors_2023_shop13, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop13, pos_2023_shop13, width=edge_widths_2023_shop13, edge_color='grey', alpha=0.5)
nx.draw_networkx_labels(G_2023_shop13, pos_2023_shop13, font_size=10)

plt.title('Top 5 Association Rules Network for 2023 Shop 13 with Conviction-based Edge Thickness')
plt.axis('off')
plt.show()


In [ ]:
#the node sizes are based on the indegree where each node acts as a consequent,

In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2023_shop13_nonredundant_top30.loc[:, 'antecedents_'] = rules_2023_shop13_nonredundant_top30['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2023_shop13_nonredundant_top30.loc[:, 'consequents_'] = rules_2023_shop13_nonredundant_top30['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2023_shop13_nonredundant_top30, x='consequents_', y='antecedents_', s=rules_2023_shop13_nonredundant_top30['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2023_shop13_nonredundant_top30.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2023 Shop 13", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
print(rules_2023_shop13_nonredundant_top30[['antecedents', 'consequents']].head(31))

## Monetary Analysis of the Top Rules

In [ ]:
# Step 1: Extract unique product codes from the rules
unique_product_codes = set()
for idx, row in rules_2023_shop13_nonredundant_top30.iterrows():
    # Unite antecedents and consequents into a single set
    all_products = row['antecedents'].union(row['consequents'])
    unique_product_codes.update(all_products)

# Step 2: Create a DataFrame with unique product codes and their unit prices
# Filter sales_data for these product codes and drop duplicates to ensure each product code appears once
product_prices = sales_data[sales_data['product_code'].isin(unique_product_codes)][['product_code', 'unit_price']].drop_duplicates()

# Create a dictionary for quick lookup of prices
price_dict = product_prices.set_index('product_code')['unit_price'].to_dict()

# Step 3: Calculate the total price for each rule and collect data for the new DataFrame
data_for_new_df = []
for idx, row in rules_2023_shop13_nonredundant_top30.iterrows():
    # Combine sets of antecedents and consequents
    all_products = row['antecedents'].union(row['consequents'])
    total_price = sum(price_dict.get(prod, 0) for prod in all_products)  # Use get to avoid KeyError if any product code is missing
    data_for_new_df.append({
        'antecedents': row['antecedents'],
        'consequents': row['consequents'],
        'total_unit_price': total_price
    })

# Create a new DataFrame with only antecedents, consequents, and total unit price
rules_and_price_shop13_2023 = pd.DataFrame(data_for_new_df)

# Sort the DataFrame by total_unit_price in descending order
rules_and_price_shop13_2023_sorted = rules_and_price_shop13_2023.sort_values(by='total_unit_price', ascending=False)

# Print the sorted DataFrame to see the specified columns along with their total unit prices
print(rules_and_price_shop13_2023_sorted)


In [ ]:
# Convert 'trx_date' to datetime format if not already done so
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Add a 'year' column if it doesn't exist
sales_data['year'] = sales_data['trx_date'].dt.year

# Step 2: Filter sales_data for shop13 and the year 2023
filtered_sales_data_2023 = sales_data[(sales_data['shopID'] == 13) & (sales_data['year'] == 2023)]

# Step 3: Initialize a list to hold the rule data including total sales value
rule_sales_summaries_2023 = []

# Step 4: Iterate through each rule in the DataFrame
for index, rule in rules_2023_shop13_nonredundant_top30.iterrows():
    all_products = rule['antecedents'].union(rule['consequents'])

    # Filter the sales data for transactions that include all products in the rule
    # This step ensures that we are only considering transactions that contain all specified products
    mask = filtered_sales_data_2023['product_code'].isin(all_products)
    transaction_ids = filtered_sales_data_2023[mask].groupby('trx_id').filter(lambda x: all_products.issubset(set(x['product_code']))).trx_id.unique()
    matched_transactions = filtered_sales_data_2023[filtered_sales_data_2023['trx_id'].isin(transaction_ids)]
    
    # Step 5: Sum the total_value of these transactions
    total_sales_value = matched_transactions['total_value'].sum()

    # Append the result to the list
    rule_sales_summaries_2023.append({
        'antecedents': rule['antecedents'],
        'consequents': rule['consequents'],
        'total_sales_value': total_sales_value
    })

# Step 6: Convert the list to a DataFrame
rules_sales_values_df_2023 = pd.DataFrame(rule_sales_summaries_2023)

# Step 7: Sort by total_sales_value in descending order
sorted_rules_sales_values_df_2023 = rules_sales_values_df_2023.sort_values(by='total_sales_value', ascending=False)

# Step 8: Print the sorted DataFrame
print(sorted_rules_sales_values_df_2023)


## Narrowing down based on Lift

In [ ]:
rules_2023_shop13_nonredundant_top30_lift['LHS'] = [','.join(list(x)) for x in rules_2023_shop13_nonredundant_top30_lift['antecedents']]
rules_2023_shop13_nonredundant_top30_lift['RHS'] = [','.join(list(x)) for x in rules_2023_shop13_nonredundant_top30_lift['consequents']]

# Remove duplicates or reversed rules by directly dropping duplicates based on LHS and RHS
rules_2023_shop13_nonredundant_top30_lift.drop_duplicates(subset=['LHS', 'RHS'], inplace=True)

# Plot the network graph
fig = plt.figure(figsize=(15, 15))
G = nx.from_pandas_edgelist(rules_2023_shop13_nonredundant_top30_lift, 'LHS', 'RHS')
np.random.seed(1234)
pos = nx.spring_layout(G, k=0.5)  # Adjust k if needed to regulate the distance between nodes
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=pos, font_size=10, edge_color="gray")
plt.title("Network Graph of Top 30 Confidence Rules for 2023 Shop 13")
plt.axis('off')
plt.show()


In [ ]:
G_2023_shop13_lift = nx.DiGraph()

lift_summary_2023_shop13 = {}

# Iterating through each row to add edges with lift as an attribute
for _, row in rules_2023_shop13_nonredundant_top30_lift.iterrows():
    antecedents = ','.join(list(row['antecedents']))
    consequents = ','.join(list(row['consequents']))
    G_2023_shop13_lift.add_edge(antecedents, consequents, weight=row['confidence'], lift=row['lift'])
    
    # Updating lift summary for each consequent
    if consequents in lift_summary_2023_shop13:
        lift_summary_2023_shop13[consequents]['total_lift'] += row['lift']
        lift_summary_2023_shop13[consequents]['count'] += 1
    else:
        lift_summary_2023_shop13[consequents] = {'total_lift': row['lift'], 'count': 1}

# Calculating average lift for each consequent to determine node sizes
for item in lift_summary_2023_shop13:
    lift_summary_2023_shop13[item]['avg_lift'] = lift_summary_2023_shop13[item]['total_lift'] / lift_summary_2023_shop13[item]['count']

# Node sizes based on average lift, scaled for visibility
node_sizes_2023_shop13_lift = [lift_summary_2023_shop13[node]['avg_lift'] * 500 if node in lift_summary_2023_shop13 else 1000 for node in G_2023_shop13_lift.nodes()]

pos_2023_shop13_lift = nx.spring_layout(G_2023_shop13_lift)

# Colormap from orange to red
cmap_2023_shop13_lift = mcolors.LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])
normalized_sizes_2023_shop13_lift = np.array(node_sizes_2023_shop13_lift)
normalized_sizes_2023_shop13_lift = (normalized_sizes_2023_shop13_lift - min(normalized_sizes_2023_shop13_lift)) / (max(normalized_sizes_2023_shop13_lift) - min(normalized_sizes_2023_shop13_lift))
node_colors_2023_shop13_lift = [cmap_2023_shop13_lift(size) for size in normalized_sizes_2023_shop13_lift]

# Drawing the network with node sizes reflecting average lift
plt.figure(figsize=(100, 52))
nx.draw_networkx_nodes(G_2023_shop13_lift, pos_2023_shop13_lift, node_size=node_sizes_2023_shop13_lift, node_color=node_colors_2023_shop13_lift, alpha=0.8)
nx.draw_networkx_edges(G_2023_shop13_lift, pos_2023_shop13_lift, arrows=True, edge_color='black', width=1)
nx.draw_networkx_labels(G_2023_shop13_lift, pos_2023_shop13_lift, font_size=11)

plt.title('Top 5 Association Rules Network for 2023 Shop 13 with Color Gradient Based on Lift')
plt.axis('off')
plt.show()


In [ ]:
# Replace frozen sets with strings using .loc to ensure direct modification
rules_2023_shop13_nonredundant_top30_lift.loc[:, 'antecedents_'] = rules_2023_shop13_nonredundant_top30_lift['antecedents'].apply(lambda a: ','.join(list(a)))
rules_2023_shop13_nonredundant_top30_lift.loc[:, 'consequents_'] = rules_2023_shop13_nonredundant_top30_lift['consequents'].apply(lambda a: ','.join(list(a)))

# Draw the bubble plot
fig, ax = plt.subplots(figsize=(14, 14))
scatter = ax.scatter(data=rules_2023_shop13_nonredundant_top30_lift, x='consequents_', y='antecedents_', s=rules_2023_shop13_nonredundant_top30_lift['lift']*20, edgecolors="red", c="white", zorder=2)

# Decoration
nRules = rules_2023_shop13_nonredundant_top30_lift.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules for 2023 Shop 13", fontsize=14)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls="--", zorder=1)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()





# Meta Analysis, Cross Shop Analysis Search 22 Filtered Based on Confidence

In [ ]:
# Print only the 'antecedents' and 'consequents' columns of the top 10 rules
print(rules_2023_shop7_nonredundant_top30_lift[['antecedents', 'consequents']].head(10))


In [ ]:
rules_2023_shop7_nonredundant_top30.info()

In [ ]:
# Example dictionary structure
meta_dfs = {
    'shop1': {
        '2021': rules_2021_shop1_nonredundant_top30,
        '2022': rules_2022_shop1_nonredundant_top30,
        '2023': rules_2023_shop1_nonredundant_top30,
    },
    'shop3': {
        '2021': rules_2021_shop3_nonredundant_top30,
        '2022': rules_2022_shop3_nonredundant_top30,
        '2023': rules_2023_shop3_nonredundant_top30,
    },
    'shop4': {
        '2021': rules_2021_shop4_nonredundant_top30,
        '2022': rules_2022_shop4_nonredundant_top30,
        '2023': rules_2023_shop4_nonredundant_top30,
    },
    'shop5': {
        '2021': rules_2021_shop5_nonredundant_top30,
        '2022': rules_2022_shop5_nonredundant_top30,
        '2023': rules_2023_shop5_nonredundant_top30,
    },
    'shop6': {
        '2022': rules_2022_shop6_nonredundant_top30,
        '2023': rules_2023_shop6_nonredundant_top30,

    },
    'shop7': {
        '2021': rules_2021_shop7_nonredundant_top30,
        '2022': rules_2022_shop7_nonredundant_top30,
        '2023': rules_2023_shop7_nonredundant_top30,
    },
     'shop8': {
        '2022': rules_2022_shop8_nonredundant_top30,
        '2023': rules_2023_shop8_nonredundant_top30,
    },
      'shop9': {
        '2022': rules_2022_shop9_nonredundant_top30,
        '2023': rules_2023_shop9_nonredundant_top30,
    },
    
       'shop11': {
        '2023': rules_2023_shop11_nonredundant_top30,
    },
    
     'shop12': {
        '2023': rules_2023_shop12_nonredundant_top30,
    },
    
      'shop13': {
        '2023': rules_2023_shop13_nonredundant_top30,
    },



    
    # Repeat for all shops
}

In [ ]:
meta_dfs_redundant = {
    'shop1': {
        '2020': rules_2020_shop1_redundant,
        '2021': rules_2021_shop1_redundant,
        '2022': rules_2022_shop1_redundant,
        '2023': rules_2023_shop1_redundant,
    },
    'shop3': {
        '2021': rules_2021_shop3_redundant,
        '2022': rules_2022_shop3_redundant,
        '2023': rules_2023_shop3_redundant,
    },
    'shop4': {
        '2021': rules_2021_shop4_redundant,
        '2022': rules_2022_shop4_redundant,
        '2023': rules_2023_shop4_redundant,
    },
    'shop5': {
        '2021': rules_2021_shop5_redundant,
        '2022': rules_2022_shop5_redundant,
        '2023': rules_2023_shop5_redundant,
    },
    'shop6': {
        '2022': rules_2022_shop6_redundant,
        '2023': rules_2023_shop6_redundant,
    },
    'shop7': {
        '2021': rules_2021_shop7_redundant,
        '2022': rules_2022_shop7_redundant,
        '2023': rules_2023_shop7_redundant,
    },
    'shop8': {
        '2022': rules_2022_shop8_redundant,
        '2023': rules_2023_shop8_redundant,
    },
    'shop9': {
        '2022': rules_2022_shop9_redundant,
        '2023': rules_2023_shop9_redundant,
    },
    'shop11': {
        '2023': rules_2023_shop11_redundant,
    },
    'shop12': {
        '2023': rules_2023_shop12_redundant,
    },
    'shop13': {
        '2023': rules_2023_shop13_redundant,
    },
    # Repeat for all shops
}


In [ ]:
# Example dictionary structure
meta_dfs_2023 = {
    'shop1': {
        '2023': rules_2023_shop1_nonredundant_top30,
    },
    'shop3': {
        '2023': rules_2023_shop3_nonredundant_top30,
    },
    'shop4': {
        '2023': rules_2023_shop4_nonredundant_top30,
    },
    'shop5': {
        '2023': rules_2023_shop5_nonredundant_top30,
    },
    'shop6': {
        '2023': rules_2023_shop6_nonredundant_top30,

    },
    'shop7': {
        '2023': rules_2023_shop7_nonredundant_top30,
    },
     'shop8': {
        '2023': rules_2023_shop8_nonredundant_top30,
    },
      'shop9': {
        '2023': rules_2023_shop9_nonredundant_top30,
    },
    
       'shop11': {
        '2023': rules_2023_shop11_nonredundant_top30,
    },
    
     'shop12': {
        '2023': rules_2023_shop12_nonredundant_top30,
    },
    
      'shop13': {
        '2023': rules_2023_shop13_nonredundant_top30,
    },



    
    # Repeat for all shops
}

In [ ]:
# Example dictionary structure
meta_dfs_lift = {
    'shop1': {
        '2021': rules_2021_shop1_nonredundant_top30_lift,
        '2022': rules_2022_shop1_nonredundant_top30_lift,
        '2023': rules_2023_shop1_nonredundant_top30_lift,
    },
    'shop3': {
        '2021': rules_2021_shop3_nonredundant_top30_lift,
        '2022': rules_2022_shop3_nonredundant_top30_lift,
        '2023': rules_2023_shop3_nonredundant_top30_lift,
    },
    'shop4': {
        '2021': rules_2021_shop4_nonredundant_top30_lift,
        '2022': rules_2022_shop4_nonredundant_top30_lift,
        '2023': rules_2023_shop4_nonredundant_top30_lift,
    },
    'shop5': {
        '2021': rules_2021_shop5_nonredundant_top30_lift,
        '2022': rules_2022_shop5_nonredundant_top30_lift,
        '2023': rules_2023_shop5_nonredundant_top30_lift,
    },
    'shop6': {
        '2022': rules_2022_shop6_nonredundant_top30_lift,
        '2023': rules_2023_shop6_nonredundant_top30_lift,

    },
    'shop7': {
        '2021': rules_2021_shop7_nonredundant_top30_lift,
        '2022': rules_2022_shop7_nonredundant_top30_lift,
        '2023': rules_2023_shop7_nonredundant_top30_lift,
    },
     'shop8': {
        '2022': rules_2022_shop8_nonredundant_top30_lift,
        '2023': rules_2023_shop8_nonredundant_top30_lift,
    },
      'shop9': {
        '2022': rules_2022_shop9_nonredundant_top30_lift,
        '2023': rules_2023_shop9_nonredundant_top30_lift,
    },
    
       'shop11': {
        '2023': rules_2023_shop11_nonredundant_top30_lift,
    },
    
       'shop12': {
        '2023': rules_2023_shop12_nonredundant_top30_lift,
    },
    
      'shop13': {
        '2023': rules_2023_shop13_nonredundant_top30_lift,
    },

}

In [ ]:
# Save the dictionary to a file
with open('meta_dfs_lift.pkl', 'wb') as file:
    pickle.dump(meta_dfs_lift, file)

In [ ]:
# Save the dictionary to a file
with open('meta_dfs_2023.pkl', 'wb') as file:
    pickle.dump(meta_dfs_2023, file)

In [ ]:
# Save the dictionary to a file
with open('meta_dfs.pkl', 'wb') as file:
    pickle.dump(meta_dfs, file)

In [ ]:
# Save the dictionary to a file
with open('meta_dfs_redundant.pkl', 'wb') as file:
    pickle.dump(meta_dfs_redundant, file)

# Meta Analysis continues in the next file